In [6]:
from __future__ import absolute_import, division, unicode_literals

import sys
import os
import io
import pdb
import numpy as np
import logging
import json
sys.path.insert(0, '..')
import senteval

In [2]:
from barlowbert_pl import LitBarlowBert
from transformers import BertTokenizerFast
from tqdm import tqdm
from datasets import Dataset

In [3]:
import pytorch_lightning as pl
import torch

In [4]:
# Set PATHs
PATH_TO_DATA = '/mounts/Users/cisintern/jabbar/git/SentEval/data'

In [5]:
def prepare(params, samples):
#     dataset = Dataset.from_dict({'text':samples})
#     dataset = dataset.map(lambda x: params.tokenizer,remove_columns=['text'])
#     dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
#     params.dataset = dataset
    return

In [31]:
def batcher(params, batch):
    tokens = [tokenizer.encode_plus(b,return_token_type_ids=False,
                                       padding='max_length',max_length=128,
                                       truncation=True,is_split_into_words=True,
                                       return_tensors='pt') 
                 for b in batch]
    
    inp = {'input_ids' : torch.cat(list_to_dict(tokens)['input_ids'],0),
           'attention_mask' : torch.cat(list_to_dict(tokens)['attention_mask'],0)
          }
    
    embedding = params.model(inp)
    return embedding.detach().numpy()

In [32]:
ckpt = '/mounts/Users/cisintern/jabbar/git/barlowtwins/pl_model/lightning_tb_logs/bert_small_bs128_lr0001_ngpu2_20mil_nodiv_lambda0.0005_maxmeancls/checkpoints/epoch=3-step=249999.ckpt'

In [33]:
data = torch.load(ckpt)

FileNotFoundError: [Errno 2] No such file or directory: '/mounts/Users/cisintern/jabbar/git/barlowtwins/pl_model/lightning_tb_logs/bert_small_bs128_lr0001_ngpu2_20mil_nodiv_lambda0.0005_maxmeancls/checkpoints/epoch=3-step=249999.ckpt'

In [34]:
data.keys()

NameError: name 'data' is not defined

In [22]:
data['hyper_parameters']['config'].pool_type='cat'

In [27]:
torch.save(data,'test.pkl')

In [7]:
model = LitBarlowBert.load_from_checkpoint('test.pkl')

In [15]:
params_senteval = {'task_path': PATH_TO_DATA,'usepytorch': True, 'kfold': 5}
params_senteval['tokenizer'] = BertTokenizerFast.from_pretrained('bert-base-uncased', padding=True,truncation=True,)
params_senteval['model'] = LitBarlowBert.load_from_checkpoint('test.pkl')

In [39]:
# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.ERROR)

In [42]:
se = senteval.engine.SE(params_senteval, batcher, prepare)
transfer_tasks = ['STS12']#, 'STS13', 'STS14', 'STS15', 'STS16',
#                   'MR', 'CR', 'MPQA', 'SUBJ', 'SST2', 'SST5', 'TREC', 'MRPC',
#                   'SICKEntailment', 'SICKRelatedness', 'STSBenchmark',
#                   'Length', 'WordContent', 'Depth', 'TopConstituents',
#                   'BigramShift', 'Tense', 'SubjNumber', 'ObjNumber',
#                   'OddManOut', 'CoordinationInversion']

In [43]:
results = se.eval(transfer_tasks)

2021-08-20 12:50:41,184 : ***** Transfer task : STS12 *****


2021-08-20 12:51:09,969 : MSRpar : pearson = 0.0696, spearman = 0.0789
2021-08-20 12:51:36,085 : MSRvid : pearson = -0.0068, spearman = 0.0076
2021-08-20 12:51:51,226 : SMTeuroparl : pearson = 0.2161, spearman = 0.3019
2021-08-20 12:52:15,978 : surprise.OnWN : pearson = 0.2536, spearman = 0.3579
2021-08-20 12:52:29,243 : surprise.SMTnews : pearson = 0.4680, spearman = 0.3928
2021-08-20 12:52:29,244 : ALL (weighted average) : Pearson = 0.1683,             Spearman = 0.2023
2021-08-20 12:52:29,244 : ALL (average) : Pearson = 0.2001,             Spearman = 0.2278



In [44]:
results

{'STS12': {'MSRpar': {'pearson': (0.06957553862394471, 0.05684056209058047),
   'spearman': SpearmanrResult(correlation=0.07887675282034763, pvalue=0.030781045428580466),
   'nsamples': 750},
  'MSRvid': {'pearson': (-0.006826791215307831, 0.8519352854996692),
   'spearman': SpearmanrResult(correlation=0.007636174392143271, pvalue=0.8346203888519323),
   'nsamples': 750},
  'SMTeuroparl': {'pearson': (0.21609112180286907, 2.978602044387983e-06),
   'spearman': SpearmanrResult(correlation=0.30192680673769184, pvalue=3.956149339632064e-11),
   'nsamples': 459},
  'surprise.OnWN': {'pearson': (0.2536300732058745, 1.7897625229929198e-12),
   'spearman': SpearmanrResult(correlation=0.3578909490854054, pvalue=4.409560133793652e-24),
   'nsamples': 750},
  'surprise.SMTnews': {'pearson': (0.46798232439552534,
    4.1557581888661696e-23),
   'spearman': SpearmanrResult(correlation=0.39282929984984816, pvalue=3.574542880488457e-16),
   'nsamples': 399},
  'all': {'pearson': {'mean': 0.200090453

In [9]:
batch = [['moore', 'had', 'no', 'immediate', 'comment', 'tuesday', '.'], ['the', 'notification', 'was', 'first', 'reported', 'friday', 'by', 'msnbc', '.'], ['druce', 'will', 'face', 'murder', 'charges', ',', 'conte', 'said', '.'], ['cisco', 'pared', 'spending', 'to', 'compensate', 'for', 'sluggish', 'sales', '.'], ['three', 'no', 'votes', 'would', 'kill', 'it', 'for', 'now', '.'], ['we', 'think', 'they', 'have', 'the', 'right', 'to', 'choose', '.'], ['the', 'dvd-cca', 'then', 'appealed', 'to', 'the', 'state', 'supreme', 'court', '.'], ['two', 'years', 'later', ',', 'the', 'insurance', 'coverage', 'would', 'begin', '.'], ['its', 'keywords', 'are', 'supported', 'by', 'chinese', 'language', 'web', 'portals', '.'], ['chairman', 'michael', 'powell', 'and', 'fcc', 'colleagues', 'at', 'the', 'wednesday', 'hearing', '.'], ['new', 'legit', 'download', 'service', 'launches', 'with', 'pc', 'users', 'in', 'mind', '.'], ['sens.', 'john', 'kerry', 'and', 'bob', 'graham', 'declined', 'invitations', 'to', 'speak', '.'], ['he', 'also', 'could', 'be', 'barred', 'permanently', 'from', 'the', 'securities', 'industry', '.'], ['officials', 'with', 'north', 'carolina', 'speedway', 'at', 'rockingham', 'could', 'not', 'be', 'reached', '.'], ['in', 'the', 'united', 'states', ',', '20.7', 'percent', 'of', 'all', 'women', 'smoke', '.'], ['"', 'and', 'about', 'eight', 'to', '10', 'seconds', 'down', ',', 'i', 'hit', '.'], ['about', '1,557', 'genes', 'on', 'chromosome', '6', 'are', 'thought', 'to', 'be', 'functional', '.'], ['earlier', 'thursday', ',', 'peoplesoft', 'formally', 'rejected', 'the', 'unsolicited', 'bid', 'from', 'oracle', '.'], ['who', 'will', 'succeed', 'mr.', 'o', "'neill", 'at', 'hyundai', 'is', 'not', 'clear', '.'], ['the', 'woman', 'was', 'hospitalized', 'june', '15', ',', 'kansas', 'health', 'officials', 'said', '.'], ['mr.', 'grassley', 'and', 'mr.', 'baucus', 'rejected', 'any', 'disparity', 'in', 'drug', 'benefits', '.'], ['representatives', 'for', 'puretunes', 'could', 'not', 'immediately', 'be', 'reached', 'for', 'comment', 'wednesday', '.'], ['chiron', 'already', 'has', 'nearly', '20', 'percent', 'acceptances', 'from', 'powderject', "'s", 'shareholders', '.'], ['he', 'projected', 'vanderpool', 'will', 'be', 'available', 'within', 'the', 'next', 'five', 'years', '.'], ['enron', 'spokesman', 'eric', 'thode', 'declined', 'to', 'comment', 'on', 'the', 'mediation', 'order', '.'], ['the', 'embraer', 'jets', 'are', 'scheduled', 'to', 'be', 'delivered', 'by', 'september', '2006', '.'], ['fifty-seven', 'senators', ',', 'including', '24', 'republicans', ',', 'have', 'signed', 'the', 'letter', '.'], ['marc', 'garber', ',', 'whitley', "'s", 'attorney', ',', 'had', 'a', 'different', 'view', '.'], ['the', 'blasts', 'killed', 'two', 'people', 'and', 'injured', 'more', 'than', '150', 'others', '.'], ['similar', 'bills', 'passed', 'the', 'house', 'twice', 'but', 'died', 'in', 'the', 'senate', '.'], ['the', 'california', 'farm', 'bureau', 'did', 'not', 'immediately', 'return', 'calls', 'seeking', 'comment', '.'], ['q', ':', 'can', 'i', 'buy', 'coverage', 'for', 'prescription', 'drugs', 'right', 'away', '?'], ['as', 'of', 'tuesday', ',', 'almost', '250', 'health-care', 'workers', 'were', 'in', 'quarantine', '.'], ['at', 'his', 'request', ',', 'he', 'will', 'be', 'reassigned', 'within', 'the', 'district', '.'], ['a', 'federal', 'magistrate', 'in', 'fort', 'lauderdale', 'ordered', 'him', 'held', 'without', 'bail', '.'], ['another', 'was', 'in', 'serious', 'condition', 'at', 'northwest', 'medical', 'center', 'in', 'springdale', '.'], ['a', 'judge', 'ordered', 'the', 'child', 'placed', 'in', 'state', 'custody', 'aug.', '8', '.'], ['her', 'body', 'was', 'found', 'several', 'weeks', 'later', 'in', 'the', 'green', 'river', '.'], ['democrats', 'currently', 'hold', 'a', '17-15', 'advantage', 'in', 'the', 'state', "'s", 'congressional', 'delegation', '.'], ['tisha', 'kresler', ',', 'a', 'spokeswoman', 'for', 'global', 'crossing', ',', 'declined', 'to', 'comment', '.'], ['gregory', 'parseghian', ',', 'a', 'former', 'investment', 'banker', ',', 'was', 'appointed', 'chief', 'executive', '.'], ['but', 'it', 'is', 'clear', 'that', 'the', 'joint', 'intelligence', 'committee', 'was', 'not', 'involved', '.'], ['he', 'will', 'replace', 'ron', 'dittemore', ',', 'who', 'announced', 'his', 'resignation', 'april', '23', '.'], ['graves', 'reported', 'from', 'albuquerque', ',', 'villafranca', 'from', 'austin', 'and', 'ratcliffe', 'from', 'laredo', '.'], ['but', 'senate', 'criticism', 'of', 'the', 'house', 'plan', 'tuesday', 'came', 'on', 'several', 'fronts', '.'], ['"', 'frank', 'quattrone', 'is', 'innocent', ',', '"', 'keker', 'said', 'in', 'a', 'statement', '.'], ['lawtey', 'is', 'not', 'the', 'first', 'faith-based', 'program', 'in', 'florida', "'s", 'prison', 'system', '.'], ['dotson', 'was', 'arrested', 'monday', 'in', 'his', 'native', 'maryland', 'and', 'charged', 'with', 'murder', '.'], ['the', 'show', "'s", 'closure', 'affected', 'third-quarter', 'earnings', 'per', 'share', 'by', 'a', 'penny', '.'], ['carlson', 'on', 'tuesday', 'said', 'he', 'would', 'not', 'recuse', 'himself', 'from', 'the', 'case', '.'], ['they', 'are', 'being', 'held', 'on', 'immigration', 'violations', 'as', 'the', 'incident', 'is', 'investigated', '.'], ['the', 'settlement', 'includes', '$', '4.1', 'million', 'in', 'attorneys', "'", 'fees', 'and', 'expenses', '.'], ['in', 'the', 'year-ago', 'period', ',', 'pearson', 'posted', 'a', '26', 'million', 'pre-tax', 'profit', '.'], ['through', 'thursday', ',', 'oracle', 'said', '34.75', 'million', 'peoplesoft', 'shares', 'had', 'been', 'tendered', '.'], ['other', 'changes', 'in', 'the', 'plan', 'refine', 'his', 'original', 'vision', ',', 'libeskind', 'said', '.'], ['the', 'american', 'conservatives', 'are', 'clearly', 'in', 'the', 'minority', 'within', 'the', 'episcopal', 'church', '.'], ['toronto', 'yesterday', 'had', '64', 'probable', 'cases', 'of', 'sars', 'and', 'nine', 'suspect', 'cases', '.'], ['between', '1993', 'and', '2002', ',', '142', 'allegations', 'of', 'sexual', 'assault', 'were', 'reported', '.'], ['the', 'chelsea', 'defender', 'marcel', 'desailly', 'has', 'been', 'the', 'latest', 'to', 'speak', 'out', '.'], ['we', 'intend', 'to', 'appeal', 'vigorously', 'and', 'still', 'expect', 'to', 'be', 'vindicated', 'ultimately', '.'], ['putin', 'hailed', 'saudi', 'arabia', 'as', '``one', 'of', 'the', 'most', 'important', 'muslim', 'nations', '.'], ['under', 'mexican', 'law', ',', 'bounty', 'hunting', 'is', 'considered', 'a', 'form', 'of', 'kidnapping', '.'], ['the', 'tech-heavy', 'nasdaq', 'stock', 'markets', 'composite', 'index', 'added', '1.16', 'points', 'to', '1,504.04', '.'], ['amd', "'s", 'chip', 'sales', 'jumped', '7', 'percent', 'year-on-year', 'to', '$', '402', 'million', '.'], ['authorities', 'in', 'ohio', ',', 'indiana', 'and', 'michigan', 'have', 'searched', 'for', 'the', 'bodies', '.'], ['the', 'technology-laced', 'nasdaq', 'composite', 'index', 'dipped', '0.08', 'of', 'a', 'point', 'to', '1,646', '.'], ['the', 'dead', 'woman', 'was', 'also', 'wearing', 'a', 'ring', 'and', 'a', 'cartier', 'watch', '.'], ['about', '120', 'potential', 'jurors', 'were', 'being', 'asked', 'to', 'complete', 'a', 'lengthy', 'questionnaire', '.'], ['initial', 'autopsy', 'reports', 'show', 'they', 'died', 'of', 'dehydration', ',', 'hyperthermia', 'and', 'suffocation', '.'], ['the', 'compensation', 'committee', 'will', 'prohibit', 'securities', 'industry', 'directors', 'from', 'sitting', 'on', 'it', '.'], ['darren', 'dopp', ',', 'a', 'spitzer', 'spokesman', ',', 'declined', 'to', 'comment', 'late', 'thursday', '.'], ['"', 'we', 'need', 'to', 'change', 'old', 'habits', 'and', 'seriously', 'rethink', 'business-as-usual', '.', '"'], ['prosecutor', 'jim', 'hardin', 'called', 'the', 'decision', 'a', 'victory', 'for', 'kathleen', 'peterson', "'s", 'family', '.'], ['that', 'objection', 'was', 'cited', 'by', 'peoplesoft', "'s", 'board', 'in', 'rejecting', 'oracle', "'s", 'offer', '.'], ['prosecutors', 'filed', 'a', 'motion', 'informing', 'lee', 'they', 'intend', 'to', 'seek', 'the', 'death', 'penalty', '.'], ['oracle', 'shares', 'were', 'also', 'higher', ',', 'trading', 'up', '12', 'cents', 'at', '$', '13.60', '.'], ['both', 'papers', 'are', 'being', 'published', 'today', 'in', 'the', 'new', 'england', 'journal', 'of', 'medicine', '.'], ['selenski', 'had', 'previously', 'spent', 'seven', 'years', 'in', 'prison', 'on', 'a', 'bank', 'robbery', 'conviction', '.'], ['sterling', 'was', 'down', '0.8', 'percent', 'against', 'the', 'dollar', 'at', '$', '1.5875', 'gbp', '=', '.'], ['he', 'also', 'reaffirmed', 'his', 'wish', 'to', 'resolve', 'the', 'north', 'korean', 'nuclear', 'crisis', 'peacefully', '.'], ['october', 'heating', 'oil', 'futures', 'settled', '.85', 'cent', 'lower', 'at', '69.89', 'cents', 'a', 'gallon', '.'], ['but', 'i', 'would', 'rather', 'be', 'talking', 'about', 'high', 'standards', 'than', 'low', 'standards', '.', '"'], ['the', 'department', 'also', 'sent', 'water', 'and', 'soil', 'samples', 'to', 'the', 'state', 'health', 'laboratory', '.'], ['"', 'the', 'gloves', 'are', 'off', ',', '"', 'said', 'unifi', 'official', 'rob', 'o', "'neill", '.'], ['four', 'other', 'men', 'who', 'were', 'also', 'charged', 'in', 'june', 'have', 'already', 'pleaded', 'guilty', '.'], ['the', 'computers', 'were', 'reportedly', 'located', 'in', 'the', 'u.s.', ',', 'canada', 'and', 'south', 'korea', '.'], ['sendmail', 'said', 'the', 'system', 'can', 'even', 'be', 'set', 'up', 'to', 'permit', 'business-only', 'usage', '.'], ['that', 'package', 'included', 'increases', 'to', 'both', 'the', 'city', "'s", 'sales', 'and', 'income', 'taxes', '.'], ['the', 'judge', 'also', 'refused', 'to', 'postpone', 'the', 'trial', 'date', 'of', 'sept', '.', '29', '.'], ['revenue', 'rose', 'to', '$', '616.5', 'million', 'from', '$', '610.6', 'million', 'a', 'year', 'earlier', '.'], ['one', ',', 'capt.', 'doug', 'mcdonald', ',', 'remained', 'hospitalized', 'in', 'critical', 'condition', 'on', 'thursday', '.'], ['"', 'tony', "'s", 'not', 'feeling', 'well', ',', '"', 'spurs', 'coach', 'gregg', 'popovich', 'said', '.'], ['the', 'securities', 'and', 'exchange', 'commission', 'has', 'also', 'initiated', 'an', 'informal', 'probe', 'of', 'coke', '.'], ['it', 'has', 'a', 'margin', 'of', 'error', 'of', 'plus', 'or', 'minus', '4', 'percentage', 'points', '.'], ['oracle', 'shares', 'fell', '27', 'cents', ',', 'or', '2', 'percent', ',', 'to', '$', '13.09', '.'], ['the', 'commerce', 'commission', 'would', 'have', 'been', 'a', 'significant', 'hurdle', 'for', 'such', 'a', 'deal', '.'], ['he', 'told', 'the', 'jury', 'stubbs', 'is', 'a', '"', 'cold', ',', 'calculating', 'killer', '.', '"'], ['ernst', '&', 'young', 'has', 'denied', 'any', 'wrongdoing', 'and', 'plans', 'to', 'fight', 'the', 'allegations', '.'], ['on', 'saturday', ',', 'a', '149mph', 'serve', 'against', 'agassi', 'equalled', 'rusedski', "'s", 'world', 'record', '.'], ['they', 'found', 'molecules', 'that', 'can', 'only', 'be', 'produced', 'when', 'ozone', 'breaks', 'down', 'cholesterol', '.'], ['bioreliance', "'s", 'stock', 'closed', 'down', '2', 'cents', 'yesterday', 'at', '$', '47.98', 'per', 'share', '.'], ['germans', 'account', 'for', 'more', 'than', '40', 'percent', 'of', 'foreign', 'visitors', 'coming', 'to', 'italy', '.'], ['shelley', "'s", 'office', 'reported', 'friday', 'that', '575,926', 'signatures', 'have', 'been', 'reported', 'to', 'him', '.'], ['shares', 'of', 'coke', 'closed', 'new', 'york', 'stock', 'exchange', 'trading', 'thursday', 'at', '$', '44.01', '.'], ['they', 'said', 'the', 'child', 'was', 'taken', 'to', 'nassau', 'county', 'medical', 'center', 'for', 'observation', '.'], ['in', 'kentucky', ',', 'democratic', 'attorney', 'general', 'ben', 'chandler', 'faces', 'u.s.', 'rep.', 'ernie', 'fletcher', '.'], ['previously', 'he', 'had', 'seen', 'the', 'grinning', 'mechanic', 'from', 'east', 'java', 'only', 'on', 'television', '.'], ['governor', 'gray', 'davis', 'estimated', 'yesterday', 'that', 'the', 'fires', 'could', 'cost', 'nearly', '$', '2', 'billion', '.'], ['instead', ',', 'tickets', 'for', 'the', 'jersey', 'jam', 'went', 'on', 'sale', 'last', 'night', 'through', 'ticketmaster', '.'], ['july', '18', ':', 'hurlbert', 'files', 'a', 'single', 'count', 'of', 'felony', 'sexual', 'assault', 'against', 'bryant', '.'], ['jordan', 'green', ',', 'the', 'prelate', "'s", 'private', 'lawyer', ',', 'said', 'he', 'had', 'no', 'comment', '.'], ['yes', 'filed', 'suit', 'yesterday', 'in', 'the', 'supreme', 'court', 'of', 'new', 'york', 'county', 'in', 'manhattan', '.'], ['rep.', 'peter', 'deutsch', ',', 'd-fla', '.', ',', 'criticized', 'the', 'fda', "'s", 'efforts', 'in', 'florida', '.'], ['glover', 'spoke', 'at', 'a', 'news', 'conference', 'that', 'included', 'about', '20', 'relatives', 'of', 'the', 'victims', '.'], ['eric', 'gagne', 'pitched', 'a', 'perfect', 'ninth', 'for', 'his', '23rd', 'save', 'in', 'as', 'many', 'opportunities', '.'], ['his', 'dissent', 'was', 'joined', 'by', 'chief', 'justice', 'william', 'h.', 'rehnquist', 'and', 'justice', 'clarence', 'thomas', '.'], ['the', 'company', 'has', 'expanded', 'into', 'providing', 'other', 'services', 'for', 'buyers', ',', 'including', 'payment', 'services', '.'], ['microsoft', 'acquired', 'virtual', 'pc', 'when', 'it', 'bought', 'the', 'assets', 'of', 'connectix', 'earlier', 'this', 'year', '.'], ['congress', 'twice', 'before', 'passed', 'similar', 'restrictions', 'only', 'to', 'see', 'president', 'bill', 'clinton', 'veto', 'them', '.'], ['in', 'exchange', ',', 'north', 'korea', 'would', 'be', 'required', 'to', 'end', 'its', 'nuclear', 'weapons', 'program', '.'], ['no', 'charges', 'were', 'announced', ',', 'but', 'the', 'statement', 'said', 'legal', 'proceedings', 'were', 'expected', 'monday', '.'], ['in', 'september', ',', 'a', '27-year-old', 'singapore', 'researcher', 'contracted', 'sars', 'while', 'working', 'in', 'a', 'laboratory', '.'], ['the', 'company', 'claims', 'it', "'s", 'the', 'largest', 'single', 'apple', 'var', 'xserve', 'sale', 'to', 'date', '.'], ['excluding', 'one-time', 'items', ',', 'the', 'company', 'enjoyed', 'a', 'profit', 'of', '6', 'cents', 'a', 'share', '.'], ['the', 'standard', 'edition', 'is', '$', '15,000', 'per', 'processor', 'or', '$', '300', 'per', 'named', 'user', '.'], ['one', 'had', 'also', 'pointed', 'to', 'the', 'word', 'refugee', 'in', 'an', 'english', '/', 'turkish', 'dictionary', '.'], ['the', 'measures', 'could', 'be', 'taken', 'up', 'by', 'the', 'full', 'senate', 'as', 'early', 'as', 'friday', '.'], ['advancing', 'issues', 'outnumbered', 'decliners', 'about', '8', 'to', '5', 'on', 'the', 'new', 'york', 'stock', 'exchange', '.']]
samples = [['moore', 'had', 'no', 'immediate', 'comment', 'tuesday', '.'], ['the', 'notification', 'was', 'first', 'reported', 'friday', 'by', 'msnbc', '.'], ['druce', 'will', 'face', 'murder', 'charges', ',', 'conte', 'said', '.'], ['cisco', 'pared', 'spending', 'to', 'compensate', 'for', 'sluggish', 'sales', '.'], ['three', 'no', 'votes', 'would', 'kill', 'it', 'for', 'now', '.'], ['we', 'think', 'they', 'have', 'the', 'right', 'to', 'choose', '.'], ['the', 'dvd-cca', 'then', 'appealed', 'to', 'the', 'state', 'supreme', 'court', '.'], ['two', 'years', 'later', ',', 'the', 'insurance', 'coverage', 'would', 'begin', '.'], ['its', 'keywords', 'are', 'supported', 'by', 'chinese', 'language', 'web', 'portals', '.'], ['chairman', 'michael', 'powell', 'and', 'fcc', 'colleagues', 'at', 'the', 'wednesday', 'hearing', '.'], ['new', 'legit', 'download', 'service', 'launches', 'with', 'pc', 'users', 'in', 'mind', '.'], ['sens.', 'john', 'kerry', 'and', 'bob', 'graham', 'declined', 'invitations', 'to', 'speak', '.'], ['he', 'also', 'could', 'be', 'barred', 'permanently', 'from', 'the', 'securities', 'industry', '.'], ['officials', 'with', 'north', 'carolina', 'speedway', 'at', 'rockingham', 'could', 'not', 'be', 'reached', '.'], ['in', 'the', 'united', 'states', ',', '20.7', 'percent', 'of', 'all', 'women', 'smoke', '.'], ['"', 'and', 'about', 'eight', 'to', '10', 'seconds', 'down', ',', 'i', 'hit', '.'], ['about', '1,557', 'genes', 'on', 'chromosome', '6', 'are', 'thought', 'to', 'be', 'functional', '.'], ['earlier', 'thursday', ',', 'peoplesoft', 'formally', 'rejected', 'the', 'unsolicited', 'bid', 'from', 'oracle', '.'], ['who', 'will', 'succeed', 'mr.', 'o', "'neill", 'at', 'hyundai', 'is', 'not', 'clear', '.'], ['the', 'woman', 'was', 'hospitalized', 'june', '15', ',', 'kansas', 'health', 'officials', 'said', '.'], ['mr.', 'grassley', 'and', 'mr.', 'baucus', 'rejected', 'any', 'disparity', 'in', 'drug', 'benefits', '.'], ['representatives', 'for', 'puretunes', 'could', 'not', 'immediately', 'be', 'reached', 'for', 'comment', 'wednesday', '.'], ['chiron', 'already', 'has', 'nearly', '20', 'percent', 'acceptances', 'from', 'powderject', "'s", 'shareholders', '.'], ['he', 'projected', 'vanderpool', 'will', 'be', 'available', 'within', 'the', 'next', 'five', 'years', '.'], ['enron', 'spokesman', 'eric', 'thode', 'declined', 'to', 'comment', 'on', 'the', 'mediation', 'order', '.'], ['the', 'embraer', 'jets', 'are', 'scheduled', 'to', 'be', 'delivered', 'by', 'september', '2006', '.'], ['fifty-seven', 'senators', ',', 'including', '24', 'republicans', ',', 'have', 'signed', 'the', 'letter', '.'], ['marc', 'garber', ',', 'whitley', "'s", 'attorney', ',', 'had', 'a', 'different', 'view', '.'], ['the', 'blasts', 'killed', 'two', 'people', 'and', 'injured', 'more', 'than', '150', 'others', '.'], ['similar', 'bills', 'passed', 'the', 'house', 'twice', 'but', 'died', 'in', 'the', 'senate', '.'], ['the', 'california', 'farm', 'bureau', 'did', 'not', 'immediately', 'return', 'calls', 'seeking', 'comment', '.'], ['q', ':', 'can', 'i', 'buy', 'coverage', 'for', 'prescription', 'drugs', 'right', 'away', '?'], ['as', 'of', 'tuesday', ',', 'almost', '250', 'health-care', 'workers', 'were', 'in', 'quarantine', '.'], ['at', 'his', 'request', ',', 'he', 'will', 'be', 'reassigned', 'within', 'the', 'district', '.'], ['a', 'federal', 'magistrate', 'in', 'fort', 'lauderdale', 'ordered', 'him', 'held', 'without', 'bail', '.'], ['another', 'was', 'in', 'serious', 'condition', 'at', 'northwest', 'medical', 'center', 'in', 'springdale', '.'], ['a', 'judge', 'ordered', 'the', 'child', 'placed', 'in', 'state', 'custody', 'aug.', '8', '.'], ['her', 'body', 'was', 'found', 'several', 'weeks', 'later', 'in', 'the', 'green', 'river', '.'], ['democrats', 'currently', 'hold', 'a', '17-15', 'advantage', 'in', 'the', 'state', "'s", 'congressional', 'delegation', '.'], ['tisha', 'kresler', ',', 'a', 'spokeswoman', 'for', 'global', 'crossing', ',', 'declined', 'to', 'comment', '.'], ['gregory', 'parseghian', ',', 'a', 'former', 'investment', 'banker', ',', 'was', 'appointed', 'chief', 'executive', '.'], ['but', 'it', 'is', 'clear', 'that', 'the', 'joint', 'intelligence', 'committee', 'was', 'not', 'involved', '.'], ['he', 'will', 'replace', 'ron', 'dittemore', ',', 'who', 'announced', 'his', 'resignation', 'april', '23', '.'], ['graves', 'reported', 'from', 'albuquerque', ',', 'villafranca', 'from', 'austin', 'and', 'ratcliffe', 'from', 'laredo', '.'], ['but', 'senate', 'criticism', 'of', 'the', 'house', 'plan', 'tuesday', 'came', 'on', 'several', 'fronts', '.'], ['"', 'frank', 'quattrone', 'is', 'innocent', ',', '"', 'keker', 'said', 'in', 'a', 'statement', '.'], ['lawtey', 'is', 'not', 'the', 'first', 'faith-based', 'program', 'in', 'florida', "'s", 'prison', 'system', '.'], ['dotson', 'was', 'arrested', 'monday', 'in', 'his', 'native', 'maryland', 'and', 'charged', 'with', 'murder', '.'], ['the', 'show', "'s", 'closure', 'affected', 'third-quarter', 'earnings', 'per', 'share', 'by', 'a', 'penny', '.'], ['carlson', 'on', 'tuesday', 'said', 'he', 'would', 'not', 'recuse', 'himself', 'from', 'the', 'case', '.'], ['they', 'are', 'being', 'held', 'on', 'immigration', 'violations', 'as', 'the', 'incident', 'is', 'investigated', '.'], ['the', 'settlement', 'includes', '$', '4.1', 'million', 'in', 'attorneys', "'", 'fees', 'and', 'expenses', '.'], ['in', 'the', 'year-ago', 'period', ',', 'pearson', 'posted', 'a', '26', 'million', 'pre-tax', 'profit', '.'], ['through', 'thursday', ',', 'oracle', 'said', '34.75', 'million', 'peoplesoft', 'shares', 'had', 'been', 'tendered', '.'], ['other', 'changes', 'in', 'the', 'plan', 'refine', 'his', 'original', 'vision', ',', 'libeskind', 'said', '.'], ['the', 'american', 'conservatives', 'are', 'clearly', 'in', 'the', 'minority', 'within', 'the', 'episcopal', 'church', '.'], ['toronto', 'yesterday', 'had', '64', 'probable', 'cases', 'of', 'sars', 'and', 'nine', 'suspect', 'cases', '.'], ['between', '1993', 'and', '2002', ',', '142', 'allegations', 'of', 'sexual', 'assault', 'were', 'reported', '.'], ['the', 'chelsea', 'defender', 'marcel', 'desailly', 'has', 'been', 'the', 'latest', 'to', 'speak', 'out', '.'], ['we', 'intend', 'to', 'appeal', 'vigorously', 'and', 'still', 'expect', 'to', 'be', 'vindicated', 'ultimately', '.'], ['putin', 'hailed', 'saudi', 'arabia', 'as', '``one', 'of', 'the', 'most', 'important', 'muslim', 'nations', '.'], ['under', 'mexican', 'law', ',', 'bounty', 'hunting', 'is', 'considered', 'a', 'form', 'of', 'kidnapping', '.'], ['the', 'tech-heavy', 'nasdaq', 'stock', 'markets', 'composite', 'index', 'added', '1.16', 'points', 'to', '1,504.04', '.'], ['amd', "'s", 'chip', 'sales', 'jumped', '7', 'percent', 'year-on-year', 'to', '$', '402', 'million', '.'], ['authorities', 'in', 'ohio', ',', 'indiana', 'and', 'michigan', 'have', 'searched', 'for', 'the', 'bodies', '.'], ['the', 'technology-laced', 'nasdaq', 'composite', 'index', 'dipped', '0.08', 'of', 'a', 'point', 'to', '1,646', '.'], ['the', 'dead', 'woman', 'was', 'also', 'wearing', 'a', 'ring', 'and', 'a', 'cartier', 'watch', '.'], ['about', '120', 'potential', 'jurors', 'were', 'being', 'asked', 'to', 'complete', 'a', 'lengthy', 'questionnaire', '.'], ['initial', 'autopsy', 'reports', 'show', 'they', 'died', 'of', 'dehydration', ',', 'hyperthermia', 'and', 'suffocation', '.'], ['the', 'compensation', 'committee', 'will', 'prohibit', 'securities', 'industry', 'directors', 'from', 'sitting', 'on', 'it', '.'], ['darren', 'dopp', ',', 'a', 'spitzer', 'spokesman', ',', 'declined', 'to', 'comment', 'late', 'thursday', '.'], ['"', 'we', 'need', 'to', 'change', 'old', 'habits', 'and', 'seriously', 'rethink', 'business-as-usual', '.', '"'], ['prosecutor', 'jim', 'hardin', 'called', 'the', 'decision', 'a', 'victory', 'for', 'kathleen', 'peterson', "'s", 'family', '.'], ['that', 'objection', 'was', 'cited', 'by', 'peoplesoft', "'s", 'board', 'in', 'rejecting', 'oracle', "'s", 'offer', '.'], ['prosecutors', 'filed', 'a', 'motion', 'informing', 'lee', 'they', 'intend', 'to', 'seek', 'the', 'death', 'penalty', '.'], ['oracle', 'shares', 'were', 'also', 'higher', ',', 'trading', 'up', '12', 'cents', 'at', '$', '13.60', '.'], ['both', 'papers', 'are', 'being', 'published', 'today', 'in', 'the', 'new', 'england', 'journal', 'of', 'medicine', '.'], ['selenski', 'had', 'previously', 'spent', 'seven', 'years', 'in', 'prison', 'on', 'a', 'bank', 'robbery', 'conviction', '.'], ['sterling', 'was', 'down', '0.8', 'percent', 'against', 'the', 'dollar', 'at', '$', '1.5875', 'gbp', '=', '.'], ['he', 'also', 'reaffirmed', 'his', 'wish', 'to', 'resolve', 'the', 'north', 'korean', 'nuclear', 'crisis', 'peacefully', '.'], ['october', 'heating', 'oil', 'futures', 'settled', '.85', 'cent', 'lower', 'at', '69.89', 'cents', 'a', 'gallon', '.'], ['but', 'i', 'would', 'rather', 'be', 'talking', 'about', 'high', 'standards', 'than', 'low', 'standards', '.', '"'], ['the', 'department', 'also', 'sent', 'water', 'and', 'soil', 'samples', 'to', 'the', 'state', 'health', 'laboratory', '.'], ['"', 'the', 'gloves', 'are', 'off', ',', '"', 'said', 'unifi', 'official', 'rob', 'o', "'neill", '.'], ['four', 'other', 'men', 'who', 'were', 'also', 'charged', 'in', 'june', 'have', 'already', 'pleaded', 'guilty', '.'], ['the', 'computers', 'were', 'reportedly', 'located', 'in', 'the', 'u.s.', ',', 'canada', 'and', 'south', 'korea', '.'], ['sendmail', 'said', 'the', 'system', 'can', 'even', 'be', 'set', 'up', 'to', 'permit', 'business-only', 'usage', '.'], ['that', 'package', 'included', 'increases', 'to', 'both', 'the', 'city', "'s", 'sales', 'and', 'income', 'taxes', '.'], ['the', 'judge', 'also', 'refused', 'to', 'postpone', 'the', 'trial', 'date', 'of', 'sept', '.', '29', '.'], ['revenue', 'rose', 'to', '$', '616.5', 'million', 'from', '$', '610.6', 'million', 'a', 'year', 'earlier', '.'], ['one', ',', 'capt.', 'doug', 'mcdonald', ',', 'remained', 'hospitalized', 'in', 'critical', 'condition', 'on', 'thursday', '.'], ['"', 'tony', "'s", 'not', 'feeling', 'well', ',', '"', 'spurs', 'coach', 'gregg', 'popovich', 'said', '.'], ['the', 'securities', 'and', 'exchange', 'commission', 'has', 'also', 'initiated', 'an', 'informal', 'probe', 'of', 'coke', '.'], ['it', 'has', 'a', 'margin', 'of', 'error', 'of', 'plus', 'or', 'minus', '4', 'percentage', 'points', '.'], ['oracle', 'shares', 'fell', '27', 'cents', ',', 'or', '2', 'percent', ',', 'to', '$', '13.09', '.'], ['the', 'commerce', 'commission', 'would', 'have', 'been', 'a', 'significant', 'hurdle', 'for', 'such', 'a', 'deal', '.'], ['he', 'told', 'the', 'jury', 'stubbs', 'is', 'a', '"', 'cold', ',', 'calculating', 'killer', '.', '"'], ['ernst', '&', 'young', 'has', 'denied', 'any', 'wrongdoing', 'and', 'plans', 'to', 'fight', 'the', 'allegations', '.'], ['on', 'saturday', ',', 'a', '149mph', 'serve', 'against', 'agassi', 'equalled', 'rusedski', "'s", 'world', 'record', '.'], ['they', 'found', 'molecules', 'that', 'can', 'only', 'be', 'produced', 'when', 'ozone', 'breaks', 'down', 'cholesterol', '.'], ['bioreliance', "'s", 'stock', 'closed', 'down', '2', 'cents', 'yesterday', 'at', '$', '47.98', 'per', 'share', '.'], ['germans', 'account', 'for', 'more', 'than', '40', 'percent', 'of', 'foreign', 'visitors', 'coming', 'to', 'italy', '.'], ['shelley', "'s", 'office', 'reported', 'friday', 'that', '575,926', 'signatures', 'have', 'been', 'reported', 'to', 'him', '.'], ['shares', 'of', 'coke', 'closed', 'new', 'york', 'stock', 'exchange', 'trading', 'thursday', 'at', '$', '44.01', '.'], ['they', 'said', 'the', 'child', 'was', 'taken', 'to', 'nassau', 'county', 'medical', 'center', 'for', 'observation', '.'], ['in', 'kentucky', ',', 'democratic', 'attorney', 'general', 'ben', 'chandler', 'faces', 'u.s.', 'rep.', 'ernie', 'fletcher', '.'], ['previously', 'he', 'had', 'seen', 'the', 'grinning', 'mechanic', 'from', 'east', 'java', 'only', 'on', 'television', '.'], ['governor', 'gray', 'davis', 'estimated', 'yesterday', 'that', 'the', 'fires', 'could', 'cost', 'nearly', '$', '2', 'billion', '.'], ['instead', ',', 'tickets', 'for', 'the', 'jersey', 'jam', 'went', 'on', 'sale', 'last', 'night', 'through', 'ticketmaster', '.'], ['july', '18', ':', 'hurlbert', 'files', 'a', 'single', 'count', 'of', 'felony', 'sexual', 'assault', 'against', 'bryant', '.'], ['jordan', 'green', ',', 'the', 'prelate', "'s", 'private', 'lawyer', ',', 'said', 'he', 'had', 'no', 'comment', '.'], ['yes', 'filed', 'suit', 'yesterday', 'in', 'the', 'supreme', 'court', 'of', 'new', 'york', 'county', 'in', 'manhattan', '.'], ['rep.', 'peter', 'deutsch', ',', 'd-fla', '.', ',', 'criticized', 'the', 'fda', "'s", 'efforts', 'in', 'florida', '.'], ['glover', 'spoke', 'at', 'a', 'news', 'conference', 'that', 'included', 'about', '20', 'relatives', 'of', 'the', 'victims', '.'], ['eric', 'gagne', 'pitched', 'a', 'perfect', 'ninth', 'for', 'his', '23rd', 'save', 'in', 'as', 'many', 'opportunities', '.'], ['his', 'dissent', 'was', 'joined', 'by', 'chief', 'justice', 'william', 'h.', 'rehnquist', 'and', 'justice', 'clarence', 'thomas', '.'], ['the', 'company', 'has', 'expanded', 'into', 'providing', 'other', 'services', 'for', 'buyers', ',', 'including', 'payment', 'services', '.'], ['microsoft', 'acquired', 'virtual', 'pc', 'when', 'it', 'bought', 'the', 'assets', 'of', 'connectix', 'earlier', 'this', 'year', '.'], ['congress', 'twice', 'before', 'passed', 'similar', 'restrictions', 'only', 'to', 'see', 'president', 'bill', 'clinton', 'veto', 'them', '.'], ['in', 'exchange', ',', 'north', 'korea', 'would', 'be', 'required', 'to', 'end', 'its', 'nuclear', 'weapons', 'program', '.'], ['no', 'charges', 'were', 'announced', ',', 'but', 'the', 'statement', 'said', 'legal', 'proceedings', 'were', 'expected', 'monday', '.'], ['in', 'september', ',', 'a', '27-year-old', 'singapore', 'researcher', 'contracted', 'sars', 'while', 'working', 'in', 'a', 'laboratory', '.'], ['the', 'company', 'claims', 'it', "'s", 'the', 'largest', 'single', 'apple', 'var', 'xserve', 'sale', 'to', 'date', '.'], ['excluding', 'one-time', 'items', ',', 'the', 'company', 'enjoyed', 'a', 'profit', 'of', '6', 'cents', 'a', 'share', '.'], ['the', 'standard', 'edition', 'is', '$', '15,000', 'per', 'processor', 'or', '$', '300', 'per', 'named', 'user', '.'], ['one', 'had', 'also', 'pointed', 'to', 'the', 'word', 'refugee', 'in', 'an', 'english', '/', 'turkish', 'dictionary', '.'], ['the', 'measures', 'could', 'be', 'taken', 'up', 'by', 'the', 'full', 'senate', 'as', 'early', 'as', 'friday', '.'], ['advancing', 'issues', 'outnumbered', 'decliners', 'about', '8', 'to', '5', 'on', 'the', 'new', 'york', 'stock', 'exchange', '.'], ['the', 'new', 'law', 'requires', 'libraries', 'and', 'schools', 'to', 'use', 'filters', 'or', 'lose', 'federal', 'financing', '.'], ['when', 'police', 'removed', 'the', 'four', 'from', 'the', 'home', ',', 'they', 'weighed', '136', 'pounds', 'combined', '.'], ['sars', 'has', 'killed', '296', 'people', 'on', 'china', "'s", 'mainland', 'and', 'infected', 'more', 'than', '5,200', '.'], ['he', 'later', 'learned', 'that', 'the', 'incident', 'was', 'caused', 'by', 'the', 'concorde', "'s", 'sonic', 'boom', '.'], ['the', 'dollar', 'was', 'at', '117.85', 'yen', 'against', 'the', 'japanese', 'currency', ',', 'up', '0.1', 'percent', '.'], ['the', 'state', "'s", 'population', 'was', '6,080,485', 'in', '2000', ',', 'according', 'to', 'the', 'u.s.', 'census', '.'], ['for', 'the', 'full', '12-month', 'period', ',', 'high-speed', 'cable', 'modem', 'connections', 'increased', 'by', '49', 'percent', '.'], ['the', 'researchers', 'found', 'only', 'empty', 'cavities', 'and', 'scar', 'tissue', 'where', 'the', 'tumors', 'had', 'been', '.'], ['nterprise', 'linux', 'services', 'is', 'expected', 'to', 'be', 'available', 'before', 'then', 'end', 'of', 'this', 'year', '.'], ['the', 'nasdaq', 'composite', 'index', 'rose', '13.70', ',', 'or', '0.8', 'per', 'cent', ',', 'to', '1700.34', '.'], ['the', 'afl-cio', 'is', 'waiting', 'until', 'october', 'to', 'decide', 'if', 'it', 'will', 'endorse', 'a', 'candidate', '.'], ['the', 'problem', 'likely', 'will', 'mean', 'corrective', 'changes', 'before', 'the', 'shuttle', 'fleet', 'starts', 'flying', 'again', '.'], ['he', 'started', 'in', 'the', '1983', 'super', 'bowl', ',', 'which', 'miami', 'lost', '27-17', 'to', 'washington', '.'], ['the', 'announcements', 'were', 'made', 'at', 'the', 'international', 'broadcasting', 'convention', '(', 'ibc', ')', 'in', 'amsterdam', '.'], ['for', 'kathy', 'nicolo', '(', 'jennifer', 'connelly', ')', ',', 'the', 'house', 'means', 'stability', 'and', 'family', '.'], ['that', 'exploit', 'works', 'on', 'unpatched', 'windows', '2000', 'machines', 'with', 'service', 'pack', '3', 'and', '4', '.'], ['the', 'a920', 'comes', 'with', 'a', 'printer', ',', 'scanner', ',', 'and', 'copier', 'for', '$', '89', '.'], ['chapman', 'was', 'not', 'immediately', 'arrested', 'and', 'is', 'expected', 'to', 'appear', 'for', 'arraignment', 'july', '3', '.'], ['sovereign', "'s", 'shares', 'lost', '74', 'cents', ',', 'or', '4.5', '%', ',', 'to', '$', '15.68', '.'], ['the', 'trial', ',', 'featuring', '125', 'witnesses', ',', 'could', 'continue', 'until', 'the', 'start', 'of', '2004', '.'], ['the', 'woman', 'felt', 'threatened', 'and', 'went', 'to', 'the', 'magistrate', "'s", 'office', ',', 'police', 'said', '.'], ['among', 'three', 'major', 'candidates', ',', 'schwarzenegger', 'is', 'wining', 'the', 'battle', 'for', 'independents', 'and', 'crossover', 'voters', '.'], ['the', 'identical', 'rovers', 'will', 'act', 'as', 'robotic', 'geologists', ',', 'searching', 'for', 'evidence', 'of', 'past', 'water', '.'], ['mahmud', 'controlled', 'access', 'to', 'saddam', 'for', 'everyone', 'but', 'immediate', 'family', 'members', ',', 'pentagon', 'officials', 'said', '.'], ['sales', 'for', 'the', 'quarter', 'beat', 'expectations', ',', 'rising', '37', 'percent', 'year-on-year', 'to', '1.76', 'billion', 'euros', '.'], ['the', 'average', 'american', 'makes', 'four', 'trips', 'a', 'day', ',', '45', 'percent', 'for', 'shopping', 'or', 'errands', '.'], ['a', 'rebel', 'who', 'was', 'captured', 'said', 'more', 'than', '2,000', 'insurgents', 'were', 'involved', 'in', 'the', 'attack', '.'], ['the', 'findings', 'were', 'published', 'in', 'the', 'july', '1', 'issue', 'of', 'the', 'annals', 'of', 'internal', 'medicine', '.'], ['security', 'lights', 'have', 'also', 'been', 'installed', 'and', 'police', 'have', 'swept', 'the', 'grounds', 'for', 'booby', 'traps', '.'], ['the', 'dow', 'jones', 'industrial', 'average', 'fell', '98.32', ',', 'or', 'about', '1.1', 'percent', ',', 'to', '9011.53', '.'], ['south', 'asia', 'follows', ',', 'with', '1.1', 'millions', 'youths', 'infected', '—', '62', 'percent', 'of', 'them', 'female', '.'], ['the', 'next', 'court', 'session', 'will', 'be', 'when', 'the', 'three-judge', 'tribunal', 'announces', 'its', 'verdict', 'in', 'mid-february', '.'], ['officials', 'at', 'brandeis', 'said', 'this', 'was', 'an', '"', 'extremely', 'heartrending', '"', 'time', 'for', 'the', 'campus', '.'], ['prosecutors', 'contended', 'that', 'mr.', 'durst', 'had', 'plotted', 'the', 'murder', 'to', 'assume', 'mr.', 'black', "'s", 'identity', '.'], ['board', 'chancellor', 'robert', 'bennett', 'declined', 'to', 'comment', 'on', 'personnel', 'matters', 'tuesday', ',', 'as', 'did', 'mills', '.'], ['however', ',', 'commercial', 'use', 'of', 'the', '2.6.0', 'kernel', 'is', 'still', 'months', 'off', 'for', 'most', 'customers', '.'], ['there', 'are', 'a', 'total', 'of', '659', 'women', 'enrolled', 'at', 'the', 'academy', ',', 'the', 'report', 'said', '.'], ['after', '9', '/', '11', ',', 'connolly', 'said', ',', 'agents', 'spent', 'thousands', 'of', 'hours', 'investigating', 'al-bayoumi', '.'], ['mr', 'mcdevitt', 'has', 'been', 'granted', 'control', 'of', 'three', 'crucial', 'aspects', 'of', 'policing', 'in', 'the', 'solomons', '.'], ['mr.', 'bankhead', 'said', 'the', 'crime', 'scenes', 'indicated', 'that', 'the', 'killer', 'was', '"', 'very', 'methodical', '.', '"'], ['one', 'of', 'the', 'most', 'well', 'studied', 'facets', 'of', 'temperament', 'is', 'how', 'people', 'respond', 'to', 'novelty', '.'], ['"', 'they', "'ve", 'been', 'in', 'the', 'stores', 'for', 'over', 'six', 'weeks', ',', '"', 'says', 'carney', '.'], ['"', 'it', "'", "'", 's', 'absurd', ',', '"', 'funny', 'cide', "'s", 'trainer', 'barclay', 'tagg', 'said', '.'], ['he', 'was', 'arrested', 'tuesday', 'night', 'at', 'a', 'northwest', 'atlanta', 'tire', 'shop', 'after', 'a', 'national', 'manhunt', '.'], ['with', 'the', 'power', 'back', 'on', ',', 'state', 'workers', 'headed', 'back', 'to', 'their', 'jobs', 'friday', 'morning', '.'], ['some', '14,000', 'customers', 'were', 'without', 'power', 'in', 'the', 'area', ',', 'oklahoma', 'gas', 'and', 'electric', 'said', '.'], ['yesterday', ',', 'shares', 'closed', 'up', '29', 'cents', ',', 'or', '0.54', 'percent', ',', 'at', '$', '54.32', '.'], ['mailblocks', 'was', 'founded', 'five', 'years', 'after', 'webtv', 'was', 'acquired', 'by', 'microsoft', 'for', '$', '425', 'million', '.'], ['its', 'legal', 'representatives', 'began', 'meeting', 'with', 'plaintiffs', "'", 'attorneys', 'last', 'week', 'to', 'discuss', 'a', 'settlement', '.'], ['he', 'was', 'also', 'expected', 'to', 'visit', 'yemen', 'and', 'bahrain', 'during', 'his', 'tour', 'of', 'the', 'region', '.'], ['trading', 'in', 'loral', 'was', 'halted', 'yesterday', ';', 'the', 'shares', 'closed', 'on', 'monday', 'at', '$', '3.01', '.'], ['wall', 'street', 'analysts', 'had', 'expected', '22', 'cents', 'a', 'share', ',', 'according', 'to', 'thomson', 'first', 'call', '.'], ['enron', "'s", '401', '(', 'k', ')', 'plan', 'covered', 'about', '20,000', 'workers', ',', 'retirees', 'and', 'beneficiaries', '.'], ['the', 'lawyer', 'representing', 'torch', 'concepts', ',', 'rich', 'marsden', 'did', 'not', 'respond', 'to', 'repeated', 'phone', 'calls', '.'], ['prince', 'is', 'replacing', 'sanford', '"', 'sandy', '"', 'weill', ',', 'who', 'will', 'remain', 'citigroup', "'s", 'chairman', '.'], ['perry', 'has', 'since', 'called', 'two', 'special', 'legislative', 'sessions', 'to', 'try', 'force', 'the', 'redistricting', 'plan', 'through', '.'], ['the', 'rev.', 'christopher', 'j.', 'coyne', ',', 'spokesman', 'for', 'the', 'archdiocese', ',', 'wouldn', "'t", 'comment', 'friday', '.'], ['french', 'retirees', 'will', 'increase', 'from', 'one', 'fifth', 'to', 'a', 'third', 'of', 'the', 'population', 'by', '2040', '.'], ['neither', 'military', 'action', 'nor', 'large-scale', 'bribery', 'can', 'solve', 'the', 'north', 'korean', 'problem', ',', 'wolfowitz', 'said', '.'], ['it', 'is', 'a', 'national', 'concern', 'that', 'will', 'touch', 'virtually', 'every', 'american', ',', '"', 'abraham', 'said', '.'], ['morrill', "'s", 'wife', ',', 'ellie', ',', 'sobbed', 'and', 'hugged', 'bondeson', "'s", 'sister-in-law', 'during', 'the', 'service', '.'], ['gartner', 'analysts', 'said', 'that', 'businesses', 'are', 'not', 'yet', 'feeling', 'confident', 'enough', 'to', 'upgrade', 'corporate', 'pcs', '.'], ['yeager', 'said', 'the', 'suspect', 'in', 'the', 'target', 'attack', 'showed', 'tendencies', 'of', 'being', 'a', 'prior', 'offender', '.'], ['the', 'nasdaq', 'fell', 'about', '1.3', '%', 'for', 'the', 'month', ',', 'snapping', 'a', 'seven-month', 'winning', 'streak', '.'], ['the', 'consensus', 'among', 'wall', 'street', 'analysts', 'was', 'for', 'a', 'loss', 'of', '28', 'cents', 'a', 'share', '.'], ['tamil', 'parties', 'blame', 'the', 'liberation', 'tigers', 'of', 'tamil', 'eelam', '(', 'ltte', ')', 'for', 'these', 'killings', '.'], ['on', 'july', '10', ',', 'a', 'team', 'of', '32', 'singaporean', 'police', 'officers', 'was', 'sent', 'to', 'baghdad', '.'], ['"', 'we', 'have', 'found', 'the', 'smoking', 'gun', ',', '"', 'investigating', 'board', 'member', 'scott', 'hubbard', 'said', '.'], ['the', 'who', 'experts', 'didn', "'t", 'say', 'how', 'many', 'cases', 'in', 'hebei', 'were', 'in', 'rural', 'areas', '.'], ['she', 'was', 'the', 'only', 'woman', 'in', 'her', 'unit', 'and', 'a', 'member', 'of', 'teamsters', 'local', '995', '.'], ['it', 'was', 'the', 'third', 'time', 'in', 'four', 'years', 'that', 'wildfires', 'forced', 'the', 'park', 'to', 'close', '.'], ['freeman', "'s", 'civil', 'hearing', 'may', 'be', ',', 'on', 'the', 'surface', ',', 'about', 'a', 'driver', "'s", 'license', '.'], ['the', 'panel', 'ordered', 'the', 'commerce', 'department', 'to', 'issue', 'revised', 'figures', 'for', 'the', 'duties', 'within', '60', 'days', '.'], ['shares', 'of', 'mandalay', 'closed', 'down', 'eight', 'cents', 'to', '$', '29.42', ',', 'before', 'the', 'earnings', 'were', 'announced', '.'], ['seniors', 'would', 'then', 'have', 'to', 'pay', '50', 'percent', 'of', 'their', 'drug', 'costs', 'up', 'to', '$', '3,450', '.'], ['vivendi', 'shares', 'closed', '1.9', 'percent', 'at', '15.80', 'euros', 'in', 'paris', 'after', 'falling', '3.6', 'percent', 'on', 'monday', '.'], ['sony', 'said', 'the', 'psp', 'would', 'also', 'feature', 'a', '4.5-inch', 'lcd', 'screen', ',', 'memory', 'stick', 'expansion', 'slots', '.'], ['the', 'venture', 'owns', 'five', 'oil', 'refineries', 'and', 'more', 'than', '2,100', 'filling', 'stations', 'in', 'russia', 'and', 'ukraine', '.'], ['the', 'budget', 'analysis', 'comes', 'as', 'negotiations', 'between', 'the', 'city', 'council', 'and', 'the', 'bloomberg', 'administration', 'heat', 'up', '.'], ['the', 'songs', 'are', 'on', 'offer', 'for', '99', 'cents', 'each', ',', 'or', '$', '9.99', 'for', 'an', 'album', '.'], ['as', 'part', 'of', 'his', 'deal', ',', 'mr.', 'delainey', 'has', 'agreed', 'to', 'cooperate', 'in', 'the', 'continuing', 'investigation', '.'], ['that', 'truck', 'was', 'spotted', 'in', 'the', 'campbells', 'creek', 'area', ',', 'fitting', 'that', 'description', ',', 'morris', 'said', '.'], ['"', 'this', 'deal', 'makes', 'sense', 'for', 'both', 'companies', ',', '"', 'halla', 'said', 'in', 'a', 'prepared', 'statement', '.'], ['during', 'a', 'period', 'of', 'nearly', 'a', 'decade', ',', 'telemarketing', 'associates', 'raised', 'more', 'than', '$', '7', 'million', '.'], ['second', 'comes', 'hp', '(', '27', 'percent', 'with', '$', '2.9', 'billion', ',', 'up', 'just', '0.4', 'percent', ')', '.'], ['rt', 'jones', 'analyst', 'juli', 'niemann', 'said', 'grant', 'was', '"', 'the', 'one', 'we', 'were', 'all', 'pulling', 'for', '.'], ['the', 'recording', 'industry', 'association', 'of', 'america', 'says', 'it', 'plans', 'to', 'sue', 'the', 'song', 'traders', 'next', 'month', '.'], ['the', 'tech-heavy', 'nasdaq', 'stock', 'markets', 'composite', 'index', 'added', '14.17', 'points', 'or', '0.94', 'per', 'cent', 'to', '1,517.05', '.'], ['the', 'additional', 'contribution', 'brings', 'total', 'u.s.', 'food', 'aid', 'to', 'north', 'korea', 'this', 'year', 'to', '100,000', 'tonnes', '.'], ['in', 'the', 'weeks', 'leading', 'up', 'to', 'the', 'execution', ',', 'several', 'florida', 'officials', 'received', 'anonymous', 'threatening', 'letters', '.'], ['the', 'arrests', 'revealed', 'new', 'details', 'about', 'the', 'four-month', 'investigation', 'at', 'selenski', "'s", 'mount', 'olivet', 'road', 'home', '.'], ['neither', 'iowa', 'state', 'athletic', 'director', 'bruce', 'van', 'de', 'velde', 'nor', 'morgan', 'could', 'be', 'reached', 'for', 'comment', '.'], ['he', 'said', 'european', 'governments', '"', 'have', 'blocked', 'all', 'new', 'bio-crops', 'because', 'of', 'unfounded', ',', 'unscientific', 'fears', '.'], ['the', 'pound', 'also', 'made', 'progress', 'against', 'the', 'dollar', ',', 'reached', 'fresh', 'three-year', 'highs', 'at', '$', '1.6789', '.'], ['so', 'far', ',', 'georgia', 'has', 'been', 'returned', 'all', 'but', '$', '70,000', 'of', 'the', 'money', 'it', 'wired', '.'], ['evacuation', 'went', 'smoothly', ',', 'although', 'passengers', 'weren', "'t", 'told', 'what', 'was', 'going', 'on', ',', 'hunt', 'said', '.'], ['two', 'officials', 'initially', 'said', 'about', '300', 'workers', 'had', 'been', 'arrested', 'at', '61', 'stores', 'in', '21', 'states', '.'], ['shares', 'of', 'hartford', 'rose', '$', '2.88', 'to', '$', '46.50', 'in', 'new', 'york', 'stock', 'exchange', 'composite', 'trading', '.'], ['thomas', 'was', 'joined', 'in', 'full', 'by', 'rehnquist', ',', 'and', 'in', 'parts', 'by', 'o', "'connor", 'and', 'scalia', '.'], ['brian', 'brabazon', 'said', 'his', 'son', 'would', 'get', 'upset', 'but', 'then', 'turn', 'around', 'and', 'befriend', 'his', 'taunters', '.'], ['police', 'said', 'her', 'two', 'small', 'children', 'were', 'alone', 'in', 'the', 'apartment', 'for', 'up', 'to', 'two', 'days', '.'], ['cia', 'director', 'george', 'tenet', 'said', 'the', 'two', 'men', 'were', '"', 'defined', 'by', 'dedication', 'and', 'courage', '.', '"'], ['the', 'value', 'will', 'total', 'about', '$', '124', 'million', ',', 'including', 'convertible', 'securities', ',', 'according', 'to', 'corel', '.'], ['its', 'shares', 'fell', '71', 'cents', ',', 'or', '3.5', 'percent', ',', 'in', 'after-hours', 'trading', 'to', '$', '19.55', '.'], ['iac', "'s", 'stock', 'closed', 'yesterday', 'down', '$', '2.81', ',', 'or', '7.6', 'percent', ',', 'at', '$', '34.19', '.'], ['tampering', 'with', 'medicine', 'has', 'been', 'so', 'lucrative', 'that', 'many', 'of', 'those', 'arrested', 'lived', 'in', 'million-dollar', 'homes', '.'], ['yucaipa', 'owned', 'dominick', "'s", 'before', 'selling', 'the', 'chain', 'to', 'safeway', 'in', '1998', 'for', '$', '2.5', 'billion', '.'], ['the', 'driver', ',', 'eugene', 'rogers', ',', 'helped', 'to', 'remove', 'children', 'from', 'the', 'bus', ',', 'wood', 'said', '.'], ['in', 'that', 'position', ',', 'elias', 'will', 'report', 'to', 'joe', 'tucci', ',', 'president', 'and', 'ceo', 'of', 'emc', '.'], ['mr', 'howard', 'said', 'he', 'also', 'had', 'faith', 'in', 'the', 'system', 'under', 'which', 'hicks', 'could', 'be', 'tried', '.'], ['tuition', 'at', 'four-year', 'private', 'colleges', 'averaged', '$', '19,710', 'this', 'year', ',', 'up', '6', 'percent', 'from', '2002', '.'], ['preliminary', 'reports', 'were', 'that', 'the', 'men', 'were', 'not', 'seen', 'together', 'at', 'the', 'airport', ',', 'sources', 'said', '.'], ['two', 'kuwaitis', 'and', 'six', 'palestinians', 'with', 'jordanian', 'passports', 'were', 'among', 'the', 'suspects', ',', 'the', 'official', 'said', '.'], ['the', 'retailer', 'said', 'it', 'came', 'to', 'the', 'decision', 'after', 'hearing', 'the', 'opinions', 'of', 'customers', 'and', 'associates', '.'], ['the', 'dow', 'jones', 'industrial', 'average', '.dji', 'rose', '41.61', 'points', ',', 'or', '0.44', 'percent', ',', 'to', '9,415.82', '.'], ['in', 'a', 'statement', 'distributed', 'friday', ',', '28', 'chechen', 'non-governmental', 'organizations', 'said', 'they', 'would', 'boycott', 'the', 'vote', '.'], ['he', 'claims', 'it', 'may', 'seem', 'unrealistic', 'only', 'because', 'little', 'effort', 'has', 'been', 'devoted', 'to', 'the', 'concept', '.'], ['schools', 'chancellor', 'joel', 'klein', 'said', 'the', 'push', 'for', 'higher', 'standards', 'and', 'accountability', '"', 'is', 'showing', 'results', '.', '"'], ['staff', 'writers', 'selwyn', 'crawford', 'and', 'colleen', 'mccain', 'nelson', 'and', 'brian', 'anderson', 'of', 'dallasnews.com', 'contributed', 'to', 'this', 'report', '.'], ['the', 'cradle', 'of', 'liberty', 'council', 'isn', "'t", 'the', 'first', 'one', 'to', 'buck', 'the', 'national', 'group', "'s", 'stance', '.'], ['"', 'they', 'were', 'an', 'inspirational', 'couple', ',', 'selfless', 'and', 'courageous', ',', '"', 'said', 'the', 'oscar-winning', 'film', 'director', '.'], ['but', 'she', 'didn', "'t", 'say', 'whether', 'she', "'ll", 'certify', 'the', 'two-year', '$', '117.4', 'billion', 'budget', 'for', '2004-05', '.'], ['leon', 'williams', "'", 'body', 'was', 'found', 'inside', 'his', 'third-floor', 'apartment', 'at', '196', 'bay', 'st.', ',', 'in', 'tompkinsville', '.'], ['they', 'made', 'a', 'similar', 'discovery', 'in', 'houston', 'on', 'another', 'aircraft', ',', 'dallas-based', 'southwest', 'said', 'in', 'a', 'statement', '.'], ['the', 'market', "'s", 'broader', 'gauges', 'also', 'posted', 'big', 'gains', ',', 'having', 'climbed', 'higher', 'for', 'four', 'consecutive', 'weeks', '.'], ['"', 'she', 'was', 'crying', 'and', 'scared', ',', "'", 'said', 'isa', 'yasin', ',', 'the', 'owner', 'of', 'the', 'store', '.'], ['"', 'this', 'decision', 'is', 'clearly', 'incorrect', ',', '"', 'ftc', 'chairman', 'timothy', 'muris', 'said', 'in', 'a', 'written', 'statement', '.'], ['but', 'odette', 'is', 'the', 'first', 'to', 'form', 'over', 'the', 'caribbean', 'sea', 'in', 'december', ',', 'the', 'center', 'said', '.'], ['knox', 'county', 'health', 'department', 'is', 'following', 'national', 'centers', 'for', 'disease', 'control', 'and', 'prevention', 'protocol', 'to', 'contain', 'infection', '.'], ['that', 'compared', 'with', '$', '35.18', 'million', ',', 'or', '24', 'cents', 'per', 'share', ',', 'in', 'the', 'year-ago', 'period', '.'], ['the', 'flamboyant', 'entrepreneur', 'flagged', 'the', 'plan', 'after', 'a', 'meeting', 'in', 'london', 'with', 'australian', 'tourism', 'minister', 'joe', 'hockey', '.'], ['in', 'hong', 'kong', ',', '24', 'patients', 'were', 'quarantined', 'wednesday', 'after', 'seven', 'public', 'hospital', 'workers', 'developed', 'flu-like', 'symptoms', '.'], ['cmdr.', 'rod', 'gibbons', ',', 'an', 'academy', 'spokesman', ',', 'said', ',', '"', 'the', 'academy', 'is', 'shocked', 'and', 'saddened', '.'], ['the', 'dow', 'jones', 'industrial', 'average', 'ended', 'the', 'day', 'down', '10.89', 'at', '9,837.94', ',', 'after', 'advancing', '111.04', 'wednesday', '.'], ['doctors', 'have', 'speculated', 'that', 'the', 'body', "'s", 'own', 'estrogen', 'protects', 'against', 'cell', 'damage', 'and', 'improves', 'blood', 'flow', '.'], ['one', 'witness', 'told', 'anatolian', 'he', 'saw', 'the', 'plane', 'on', 'fire', 'while', 'it', 'was', 'still', 'in', 'the', 'air', '.'], ['the', 'life', 'expectancy', 'for', 'boys', 'born', 'in', '2001', 'was', '74.4', 'years', ',', 'up', 'two', 'years', 'since', '1990', '.'], ['gillespie', 'sent', 'a', 'letter', 'to', 'cbs', 'president', 'leslie', 'moonves', 'asking', 'for', 'a', 'historical', 'review', 'or', 'a', 'disclaimer', '.'], ['it', "'s", 'the', 'first', 'such', 'drill', 'since', 'the', 'september', '11', 'terrorist', 'attacks', 'on', 'new', 'york', 'and', 'washington', '.'], ['ruffner', ',', '45', ',', 'doesn', "'t", 'yet', 'have', 'an', 'attorney', 'in', 'the', 'murder', 'charge', ',', 'authorities', 'said', '.'], ['local', 'police', 'authorities', 'are', 'treating', 'the', 'explosion', 'as', 'a', 'criminal', 'matter', 'and', 'nothing', 'has', 'been', 'ruled', 'out', '.'], ['gov.', 'linda', 'lingle', 'and', 'members', 'of', 'her', 'staff', 'were', 'at', 'the', 'navy', 'base', 'and', 'watched', 'the', 'launch', '.'], ['toll', ',', 'australia', "'s", 'second-largest', 'transport', 'company', ',', 'last', 'week', 'offered', 'nz75', 'a', 'share', 'for', 'tranz', 'rail', '.'], ['agnew', 'said', 'pinellas', 'began', 'baiting', 'when', 'rabies', 'cases', '"', 'went', 'from', 'zero', 'to', '30', '"', 'in', '1995', '.'], ['"', 'this', 'child', 'was', 'literally', 'neglected', 'to', 'death', ',', '"', 'armstrong', 'county', 'district', 'attorney', 'scott', 'andreassi', 'said', '.'], ['about', '22', '%', 'of', 'twentysomethings', 'are', 'obese', ',', 'which', 'is', 'roughly', '30', 'pounds', 'over', 'a', 'healthy', 'weight', '.'], ['shares', 'closed', 'on', 'nasdaq', 'just', 'below', 'their', '52-week', 'high', ',', 'at', '$', '32.17', ',', 'up', '$', '1.54', '.'], ['the', 'institute', 'for', 'supply', 'management', "'s", 'index', 'of', 'nonmanufacturing', 'activity', 'rose', 'unexpectedly', 'in', 'april', ',', 'reports', 'said', '.'], ['mr', 'levine', 'confirmed', 'that', 'ms', 'cohen', 'alon', 'had', 'tried', 'to', 'sell', 'the', 'story', 'during', 'the', 'world', 'cup', '.'], ['microsoft', 'fell', '5', 'percent', 'before', 'the', 'open', 'to', '$', '27.45', 'from', 'thursday', "'s", 'close', 'of', '$', '28.91', '.'], ['south', 'africa', 'has', 'the', 'world', "'s", 'highest', 'caseload', 'with', '4.7', 'million', 'people', 'infected', 'with', 'hiv', 'or', 'aids', '.'], ['the', 'technology-laced', 'nasdaq', 'composite', 'index', '.ixic', 'inched', 'down', '1', 'point', ',', 'or', '0.11', 'percent', ',', 'to', '1,650', '.'], ['half', 'of', 'the', 'women', 'were', 'given', 'a', 'daily', 'dose', 'of', 'the', 'drug', 'and', 'half', 'took', 'a', 'placebo', '.'], ['the', 'new', 'york', 'senator', "'s", 'new', 'book', ',', '"', 'living', 'history', ',', '"', 'appears', 'a', 'certain', 'bestseller', '.'], ['in', '2001', ',', 'president', 'bush', 'named', 'kathy', 'gregg', 'to', 'the', 'student', 'loan', 'marketing', 'association', 'board', 'of', 'directors', '.'], ['justices', 'said', 'that', 'the', 'constitution', 'allows', 'the', 'government', 'to', 'administer', 'drugs', 'only', '"', 'in', 'limited', 'circumstances', '.', '"'], ['he', 'proposed', 'a', 'system', 'under', 'which', 'it', 'would', 'take', 'fewer', 'and', 'fewer', 'votes', 'to', 'overcome', 'a', 'filibuster', '.'], ['the', 'state', 'wants', 'to', 'kill', 'the', 'wolves', 'in', 'approximately', 'a', '1,700-square-mile', 'area', 'near', 'the', 'village', 'of', 'mcgrath', '.'], ['the', 'statewide', 'unemployment', 'rate', 'fell', 'to', '6.4', 'percent', ',', 'down', 'from', 'a', 'revised', '6.7', 'percent', 'in', 'august', '.'], ['gyorgy', 'heizler', ',', 'head', 'of', 'the', 'local', 'disaster', 'unit', ',', 'said', 'the', 'coach', 'was', 'carrying', '38', 'passengers', '.'], ['"', 'i', 'feel', 'confident', 'saying', 'that', 'hp', 'is', 'no', 'longer', 'an', 'integration', 'story', ',', '"', 'fiorina', 'said', '.'], ['"', 'this', 'is', 'extraordinarily', 'fast', ',', '"', 'said', 'matt', 'geller', ',', 'an', 'analyst', 'at', 'cibc', 'world', 'markets', '.'], ['a', 'beauty', 'contest', 'to', 'be', 'held', 'in', 'italy', 'next', 'week', 'may', 'be', 'the', 'first', 'for', 'pixel-perfect', 'pin-ups', '.'], ['they', 'passed', 'through', 'the', 'lemelson', 'medical', ',', 'educational', 'and', 'research', 'foundation', 'limited', 'partnership', 'in', '2001', 'to', 'syndia', '.'], ['washington', ',', 'however', ',', 'said', 'more', 'was', 'needed', 'to', 'prevent', 'complaints', 'being', 'filed', 'in', 'the', 'first', 'place', '.'], ['hoy', 'confirmed', 'the', 'woman', "'s", 'age', 'tuesday', 'and', 'said', 'she', 'has', 'left', 'on', 'vacation', 'with', 'her', 'family', '.'], ['in', 'addition', ',', 'it', 'offered', 'the', 'united', 'states', 'and', 'israel', 'a', 'way', 'to', 'work', 'around', 'mr.', 'arafat', '.'], ['niels', 'told', 'an', 'interviewer', 'in', '1999', 'he', 'thought', 'the', 'old', 'man', 'would', 'outlive', 'him', 'by', 'many', 'years', '.'], ['the', 'legal', 'ruling', 'follows', 'three', 'days', 'of', 'intense', 'speculation', 'hewlett-packard', 'co.', 'may', 'be', 'bidding', 'for', 'the', 'company', '.'], ['american', 'masters', ':', 'arthur', 'miller', ',', 'elia', 'kazan', 'and', 'the', 'blacklist', ':', 'none', 'without', 'sin', '(', 'wed', '.'], ['"', 'it', "'s", 'a', 'major', 'victory', 'for', 'maine', ',', 'and', 'it', "'s", 'a', 'major', 'victory', 'for', 'other', 'states', '.'], ['the', 'reading', 'for', 'both', 'august', 'and', 'july', 'is', 'the', 'best', 'seen', 'since', 'the', 'survey', 'began', 'in', 'august', '1997', '.'], ['there', 'are', 'now', '37', 'active', 'probable', 'cases', 'in', 'the', 'gta', ',', 'compared', 'with', '70', 'cases', 'on', 'june', '6', '.'], ['he', 'gave', 'a', 'surprisingly', 'impassioned', 'speech', 'to', 'the', 'united', 'steelworkers', 'of', 'america', ',', 'who', 'announced', 'their', 'endorsement', 'yesterday', '.'], ['the', 'parents', 'want', 'her', 'kept', 'alive', ';', 'her', 'husband', 'says', 'she', 'never', 'wanted', 'to', 'be', 'kept', 'alive', 'artificially', '.'], ['in', 'midafternoon', 'trading', ',', 'the', 'nasdaq', 'composite', 'index', 'was', 'up', '8.34', ',', 'or', '0.5', 'percent', ',', 'to', '1,790.47', '.'], ['the', 'nasdaq', 'had', 'a', 'weekly', 'gain', 'of', '17.27', ',', 'or', '1.2', 'percent', ',', 'closing', 'at', '1,520.15', 'on', 'friday', '.'], ['heatley', 'was', 'sixth', 'in', 'the', 'league', 'in', 'goals', 'last', 'season', 'with', '41', 'and', 'ninth', 'in', 'points', 'with', '89', '.'], ['jim', 'furyk', 'celebrated', 'his', 'first', 'father', "'s", 'day', 'as', 'a', 'father', 'by', 'winning', 'his', 'first', 'major', 'golf', 'championship', '.'], ['the', 'report', 'was', 'found', 'oct.', '23', ',', 'tucked', 'inside', 'an', 'old', 'three-ring', 'binder', 'not', 'related', 'to', 'the', 'investigation', '.'], ['costa', "'s", 'semifinal', 'opponent', 'is', 'spaniard', 'juan', 'carlos', 'ferrero', ',', 'whom', 'he', 'beat', 'in', 'last', 'year', "'s", 'final', '.'], ['he', 'will', 'be', 'paid', '$', '395,000', 'per', 'year', ',', 'up', 'from', 'atkinson', "'s", 'current', 'salary', 'of', '$', '361,400', '.'], ['it', 'would', 'be', 'difficult', 'to', 'overestimate', 'the', 'potential', 'dangers', 'of', 'the', 'remote', 'procedure', 'call', '(', 'rpc', ')', 'vulnerability', '.'], ['under', 'nasd', 'regulations', ',', 'mr.', 'young', 'can', 'file', 'a', 'response', 'and', 'request', 'a', 'hearing', 'before', 'an', 'nasd', 'panel', '.'], ['he', 'was', 'listed', 'last', 'night', 'in', 'critical', 'but', 'stable', 'condition', 'at', 'kings', 'county', 'hospital', 'center', ',', 'police', 'said', '.'], ['founded', 'in', '1996', ',', 'the', 'lendingtree', 'exchange', 'consists', 'of', 'more', 'than', '200', 'banks', ',', 'lenders', ',', 'and', 'brokers', '.'], ['under', 'kansas', 'law', ',', 'it', 'is', 'illegal', 'for', 'children', 'under', 'the', 'age', 'of', '16', 'to', 'have', 'sexual', 'relations', '.'], ['u.s.', 'forces', 'recently', 'apprehended', 'some', '40', 'suspected', 'foreign', 'militants', 'near', 'the', 'syrian', 'border', 'and', 'are', 'now', 'interrogating', 'them', '.'], ['the', 'decades-long', 'conflict', 'has', 'killed', 'more', 'than', '10,000', 'people', 'in', 'the', 'resource-rich', 'province', ',', 'most', 'of', 'them', 'civilians', '.'], ['who', 'spokeswoman', 'maria', 'cheng', 'agreed', ',', 'saying', ':', '"', 'it', 'looks', 'very', 'much', 'like', 'an', 'isolated', 'event', '.', '"'], ['a', 'number', 'below', '50', 'suggests', 'contraction', 'in', 'the', 'manufacturing', 'sector', ',', 'while', 'a', 'number', 'above', 'that', 'indicates', 'expansion', '.'], ['police', 'official', 's.k.', 'tonapi', 'told', 'reuters', 'at', 'least', '40', 'people', 'had', 'been', 'killed', 'and', 'more', 'than', '100', 'wounded', '.'], ['according', 'to', 'the', '2000', 'census', ',', 'long', 'beach', "'s", 'hispanic', 'or', 'latino', 'population', 'was', 'listed', 'at', '35.8', 'percent', '.'], ['annika', 'sorenstam', 'gets', 'her', 'first', 'opportunity', 'to', 'test', 'that', 'objective', 'this', 'week', 'at', 'the', 'mcdonald', "'s", 'lpga', 'championship', '.'], ['the', 'technology-laced', 'nasdaq', 'composite', 'index', '<', '.ixic', '>', 'jumped', '26', 'points', ',', 'or', '1.78', 'percent', ',', 'to', '1,516', '.'], ['the', 'technology-laced', 'nasdaq', 'composite', 'index', '<', '.ixic', '>', 'added', '8.27', 'points', ',', 'or', '0.51', 'percent', ',', 'to', '1,633.53', '.'], ['the', 'american', 'express', 'corp.', 'has', 'pledged', 'at', 'least', '$', '3', 'million', 'of', 'more', 'than', '$', '5', 'million', 'needed', '.'], ['kansas', 'city', 'drafted', 'outfielder', 'chris', 'lubanski', ',', 'from', 'kennedy-kenrick', 'high', 'school', 'in', 'pennsylvania', ',', 'with', 'the', 'fifth', 'pick', '.'], ['smith', 'said', 'simply', '"', 'oh', ',', 'my', 'god', ',', '"', 'in', 'the', 'seconds', 'afterward', ',', 'according', 'to', 'weinshall', '.'], ['as', 'a', 'result', ',', 'murphy', 'sought', 'to', 'substitute', 'strier', "'s", 'sister', ',', 'ethel', 'celnik', ',', 'as', 'the', 'trustee', '.'], ['the', 'ireland', 'palestine', 'solidarity', 'campaign', ',', 'of', 'which', 'mr', 'o', 'muireagáin', 'was', 'a', 'member', ',', 'welcomed', 'his', 'release', '.'], ['cooley', 'said', 'he', 'expects', 'muhammad', 'will', 'similarly', 'be', 'called', 'as', 'a', 'witness', 'at', 'a', 'pretrial', 'hearing', 'for', 'malvo', '.'], ['some', '50', 'million', 'phone', 'numbers', 'are', 'registered', 'on', 'the', 'list', ',', 'which', 'was', 'to', 'take', 'effect', 'oct.', '1', '.'], ['a', 'former', 'candidate', 'for', 'governor', 'of', 'new', 'york', ',', 'he', 'styled', 'himself', 'an', 'advocate', 'of', 'better', 'corporate', 'governance', '.'], ['mr', 'hadley', ',', 'who', 'has', 'a', 'new', 'partner', 'and', 'child', ',', 'said', ':', '"', 'i', 'am', 'absolutely', 'delighted', '.'], ['he', 'was', 'voluntarily', 'castrated', 'in', '2001', ',', 'an', 'operation', 'he', 'contends', 'removed', 'his', 'ability', 'to', 'become', 'sexually', 'aroused', '.'], ['"', 'i', 'can', 'say', 'i', 'am', 'being', 'forced', 'into', 'exile', 'by', 'the', 'world', 'superpower', ',', '"', 'he', 'said', '.'], ['a', 'total', 'of', '17', 'cases', 'have', 'been', 'confirmed', 'in', 'the', 'southern', 'city', 'of', 'basra', ',', 'the', 'organization', 'said', '.'], ['also', 'in', 'mosul', ',', 'rebel', 'gunmen', 'on', 'friday', 'assassinated', 'a', 'sunni', 'muslim', 'tribal', 'leader', 'who', 'backed', 'the', 'coalition', '.'], ['"', 'canada', 'is', 'a', 'small', 'country', ',', 'and', 'it', 'needs', 'companies', 'like', 'this', ',', 'deals', 'like', 'this', '.', '"'], ['two', 'of', 'the', 'britons', ',', 'sandy', 'mitchell', 'from', 'glasgow', 'and', 'glasgow-born', 'william', 'sampson', ',', 'face', 'the', 'death', 'penalty', '.'], ['mr.', 'manuel', 'and', 'his', 'group', 'entered', 'the', 'united', 'states', 'by', 'walking', 'across', 'the', 'bridge', 'linking', 'matamoros', 'and', 'brownsville', '.'], ['"', 'right', 'from', 'the', 'beginning', ',', 'we', 'didn', "'t", 'want', 'to', 'see', 'anyone', 'take', 'a', 'cut', 'in', 'pay', '.'], ['the', 'swedish', 'central', 'bank', 'was', 'also', 'meeting', 'on', 'wednesday', 'and', 'widely', 'expected', 'to', 'announce', 'a', 'cut', 'on', 'thursday', '.'], ['graham', ',', 'a', 'presidential', 'candidate', ',', 'was', 'criticized', 'by', 'several', 'republicans', 'as', "'", "'", 'politicizing', "'", "'", 'the', 'report', '.'], ['gm', ',', 'the', 'world', "'s", 'largest', 'automaker', ',', 'has', '115,000', 'active', 'uaw', 'workers', 'and', 'another', '340,000', 'retirees', 'and', 'spouses', '.'], ['indeed', ',', 'mr.', 'weill', ',', 'a', 'self-described', 'workaholic', ',', 'said', 'he', 'planned', 'to', 'remain', 'very', 'involved', 'in', 'the', 'company', '.'], ['the', 'operating', 'revenues', 'were', '$', '1.45', 'billion', ',', 'an', 'increase', 'over', 'last', 'year', "'s", 'result', 'of', '$', '1.38', 'billion', '.'], ['comcast', 'class', 'a', 'shares', 'were', 'up', '8', 'cents', 'at', '$', '30.50', 'in', 'morning', 'trading', 'on', 'the', 'nasdaq', 'stock', 'market', '.'], ['harris', 'and', 'klebold', 'killed', '12', 'students', 'and', 'a', 'teacher', 'before', 'taking', 'their', 'own', 'lives', 'on', 'april', '20', ',', '1999', '.'], ['no', 'americans', 'were', 'reported', 'among', 'the', 'casualties', ',', 'according', 'to', 'capt.', 'michael', 'calvert', ',', 'a', 'spokesman', 'for', 'the', 'regiment', '.'], ['the', 'broader', 'standard', '&', 'poor', "'s", '500', 'index', '.spx', 'climbed', '10', 'points', ',', 'or', '1.10', 'percent', ',', 'to', '943', '.'], ['in', 'the', 'second', 'quarter', ',', 'anadarko', 'now', 'expects', 'volume', 'of', '46', 'million', 'boe', ',', 'down', 'from', '48', 'million', 'boe', '.'], ['the', 'broader', 'standard', '&', 'poor', "'s", '500', 'index', '.spx', 'advanced', '2', 'points', ',', 'or', '0.24', 'percent', ',', 'to', '977', '.'], ['mr.', 'geoghan', 'had', 'been', 'living', 'in', 'a', 'protective', 'custody', 'unit', 'since', 'being', 'transferred', 'to', 'the', 'prison', 'on', 'april', '3', '.'], ['the', 'rate', 'of', 'skin', 'cancer', 'has', 'tripled', 'since', 'the', '1950s', 'in', 'norway', 'and', 'sweden', ',', 'according', 'to', 'the', 'study', '.'], ['but', 'software', 'license', 'revenues', ',', 'a', 'measure', 'financial', 'analysts', 'watch', 'closely', ',', 'decreased', '21', 'percent', 'to', '$', '107.6', 'million', '.'], ['the', 'broader', 'standard', '&', 'poor', "'s", '500', 'index', '.spx', 'gained', '3', 'points', ',', 'or', '0.39', 'percent', ',', 'at', '924', '.'], ['the', 'broader', 'standard', '&', 'poor', "'s", '500', 'index', '.spx', 'fell', '3', 'points', ',', 'or', '0.30', 'percent', ',', 'to', '995', '.'], ['a', '25', 'percent', 'increase', 'would', 'raise', 'undergraduate', 'tuition', 'to', 'about', '$', '5,247', 'annually', ',', 'including', 'miscellaneous', ',', 'campus-based', 'fees', '.'], ['nasa', 'satellite', 'images', 'show', 'that', 'arctic', 'ice', 'has', 'been', 'shrinking', 'at', 'the', 'rate', 'of', 'nearly', '10', 'percent', 'a', 'decade', '.'], ['page', 'said', 'it', "'s", 'also', 'possible', 'that', 'genes', 'on', 'the', 'y', 'chromosome', 'may', 'influence', 'gender-specific', 'differences', 'in', 'disease', 'susceptibility', '.'], ['his', '1996', 'chevrolet', 'tahoe', 'was', 'found', 'abandoned', 'in', 'a', 'virginia', 'beach', ',', 'va', '.', ',', 'parking', 'lot', 'june', '25', '.'], ['but', 'butterflies', 'exposed', 'to', 'an', 'earlier', 'light', 'cycle', ',', 'from', '1am', 'to', '1pm', ',', 'orientated', 'themselves', 'towards', 'the', 'south-east', '.'], ['but', 'u.s.', 'troops', 'will', 'not', 'shrink', 'from', 'mounting', 'raids', 'and', 'attacking', 'their', 'foes', 'when', 'their', 'locations', 'can', 'be', 'pinpointed', '.'], ['the', 'technology-laced', 'nasdaq', 'composite', 'index', '<', '.ixic', '>', 'was', 'up', '7.42', 'points', ',', 'or', '0.45', 'percent', ',', 'at', '1,653.44', '.'], ['five', 'more', 'human', 'cases', 'of', 'west', 'nile', 'virus', ',', 'were', 'reported', 'by', 'the', 'mesa', 'county', 'health', 'department', 'on', 'wednesday', '.'], ['activists', 'say', 'they', 'fear', 'the', 'gathering', 'is', 'an', 'attempt', 'by', 'corporate', 'farming', 'to', 'push', 'bio-engineered', 'crops', 'on', 'starving', 'countries', '.'], ['the', '30-year', 'bond', 'us30yt', '=', 'rr', 'dipped', '14', '/', '32', 'for', 'a', 'yield', 'of', '4.26', 'percent', 'from', '4.23', 'percent', '.'], ['investigators', 'have', 'meticulously', 'looked', 'into', 'virtually', 'every', 'aspect', 'of', 'campbell', "'s", 'personal', 'and', 'financial', 'affairs', ',', 'campbell', 'has', 'acknowledged', '.'], ['on', 'wednesday', ',', 'the', 'total', 'of', 'national', 'guard', 'and', 'reserve', 'members', 'called', 'to', 'active', 'duty', 'worldwide', 'stood', 'at', '154,603', '.'], ['aaa', 'spokesman', 'jerry', 'cheske', 'said', 'prices', 'may', 'have', 'affected', 'some', 'plans', ',', 'but', 'cheap', 'hotel', 'deals', 'mitigated', 'the', 'effect', '.'], ['carnival', 'corp.', 'stock', 'was', 'trading', 'at', '$', '31.29', 'during', 'midday', 'trading', 'wednesday', ',', 'down', '72', 'cents', 'or', '2.25', 'percent', '.'], ['st.', 'paul', 'chairman', 'and', 'chief', 'executive', 'jay', 's.', 'fishman', ',', '51', ',', 'will', 'be', 'ceo', 'of', 'the', 'combined', 'company', '.'], ['the', 'consensus', 'estimate', 'of', 'analysts', 'polled', 'by', 'thomson', 'first', 'call', 'called', 'for', 'a', 'profit', 'of', '$', '1.70', 'per', 'share', '.'], ['shares', 'of', 'lendingtree', 'rose', '$', '6.21', ',', 'or', '42', 'percent', ',', 'to', '$', '20.90', 'after', 'hitting', '$', '21.36', 'earlier', '.'], ['there', 'is', ',', 'however', ',', 'no', 'photo', 'of', 'peter', 'hollingworth', 'in', 'the', 'june', 'issue', 'examined', 'by', 'the', 'herald', 'yesterday', '.'], ['the', 'technology-laced', 'nasdaq', 'composite', 'index', '<', '.ixic', '>', 'tacked', 'on', '5.91', 'points', ',', 'or', '0.29', 'percent', ',', 'to', '2,053.27', '.'], ['mr', 'eddington', 'warned', 'yesterday', 'that', 'the', 'future', 'of', 'the', 'airline', 'was', '"', 'in', 'the', 'hands', 'of', 'the', 'negotiators', '"', '.'], ['doctors', 'say', 'one', 'or', 'both', 'boys', 'may', 'die', ',', 'and', 'that', 'some', 'brain', 'damage', 'is', 'possible', 'if', 'they', 'survive', '.'], ['"', 'fighting', 'continued', 'until', 'midnight', ',', '"', 'western', 'rebel', 'commander', 'ousmane', 'coulibaly', 'told', 'reuters', 'by', 'satellite', 'phone', 'on', 'sunday', '.'], ['peterson', 'told', 'police', 'he', 'fished', 'alone', 'in', 'san', 'francisco', 'bay', 'on', 'christmas', 'eve', ',', 'returning', 'to', 'an', 'empty', 'house', '.'], ['the', 'transaction', 'will', 'expand', 'callebaut', "'s", 'sales', 'revenues', 'from', 'its', 'consumer', 'products', 'business', 'to', '45', 'percent', 'from', '23', 'percent', '.'], ['under', 'the', 'program', ',', 'u.s.', 'officials', 'work', 'with', 'foreign', 'port', 'authorities', 'to', 'identify', ',', 'target', 'and', 'search', 'high-risk', 'cargo', '.'], ['cubs', 'outfielder', 'sammy', 'sosa', 'was', 'suspended', 'for', 'eight', 'games', 'by', 'major', 'league', 'baseball', 'friday', 'for', 'using', 'a', 'corked', 'bat', '.'], ['it', 'was', 'developed', 'with', 'consultation', 'from', 'more', 'than', '300', 'leaders', 'in', 'academia', ',', 'industry', ',', 'government', 'and', 'the', 'public', '.'], ['"', 'i', 'know', 'of', 'no', 'pressure', ',', '"', 'said', 'mr.', 'feith', ',', 'the', 'under', 'secretary', 'of', 'defense', 'for', 'policy', '.'], ['"', 'fewer', 'than', '10', '"', 'fbi', 'offices', 'have', 'conducted', 'investigations', 'involving', 'visits', 'to', 'mosques', ',', 'the', 'justice', 'department', 'said', '.'], ['analysts', 'expected', 'earnings', 'of', '27', 'cents', 'a', 'share', 'on', 'revenue', 'of', '$', '17.7', 'billion', ',', 'thomson', 'first', 'call', 'says', '.'], ['britz', 'earned', '$', '3.77', 'million', 'in', 'salary', ',', 'bonus', 'and', 'other', 'compensation', ',', 'and', 'kinney', 'earned', '$', '3.76', 'million', '.'], ['most', 'taxpayers', 'will', 'get', 'surprise', 'tax', 'cuts', 'of', 'between', '$', '4', 'and', '$', '11', 'a', 'week', 'from', 'july', '1', '.'], ['the', 'male', 'eagle', 'was', 'found', 'by', 'a', 'zookeeper', 'early', 'thursday', ',', 'suffering', 'from', 'severe', 'puncture', 'wounds', 'in', 'his', 'abdomen', '.'], ['devries', 'did', 'make', 'one', 'stop', 'in', 'town', 'wednesday', '-', 'registering', 'as', 'a', 'sex', 'offender', 'at', 'the', 'soledad', 'police', 'department', '.'], ['the', 'effort', 'is', 'the', 'bush', 'administration', "'s", 'latest', 'effort', 'to', 'expand', 'the', 'role', 'of', 'religious', 'organizations', 'in', 'government', 'services', '.'], ['in', 'vienna', ',', 'the', 'iaea', 'said', 'elbaradei', 'would', 'accept', 'the', 'invitation', ',', 'although', 'no', 'date', 'had', 'yet', 'been', 'set', '.'], ['"', 'if', 'that', 'ain', "'t", 'a', 'democrat', ',', 'i', 'must', 'be', 'at', 'the', 'wrong', 'meeting', ',', '"', 'he', 'said', '.'], ['both', 'rebels', 'and', 'government', 'forces', 'have', 'been', 'accused', 'of', 'pillaging', 'villages', 'in', 'liberia', "'s", 'countryside', 'despite', 'the', 'peace', 'agreement', '.'], ['several', 'black', 'democratic', 'leaders', 'were', 'attempting', 'to', 'arrange', 'a', 'meeting', 'with', 'dnc', 'chairman', 'terry', 'mcauliffe', 'to', 'discuss', 'the', 'layoffs', '.'], ['just', 'five', 'months', 'ago', ',', 'dean', 'committed', 'to', 'accepting', 'taxpayer', 'money', 'and', 'vowed', 'to', 'attack', 'any', 'democrat', 'who', 'didnt', '.'], ['the', 'report', 'ranked', '45', 'large', 'companies', 'based', 'on', 'employment', ',', 'marketing', ',', 'procurement', ',', 'community', 'reinvestment', 'and', 'charitable', 'donations', '.'], ['her', 'birthday', ',', '21', 'april', ',', 'was', 'declared', 'a', 'public', 'holiday', 'and', 'she', 'attended', 'a', 'reception', 'and', 'a', 'ball', '.'], ['although', 'obesity', 'can', 'increase', 'the', 'risk', 'of', 'health', 'problems', ',', 'skeptics', 'argue', ',', 'so', 'do', 'smoking', 'and', 'high', 'cholesterol', '.'], ['the', 'dollar', 'was', 'last', 'at', '$', '1.1149', 'to', 'the', 'euro', ',', 'close', 'to', 'its', 'strongest', 'level', 'since', 'april', '30', '.'], ['protesters', 'had', 'been', 'calling', 'for', 'an', 'end', 'to', 'the', 'country', "'s", 'hard-line', 'establishment', 'and', 'for', 'supreme', 'leader', 'khamenei', "'s", 'death', '.'], ['earlier', 'on', 'monday', ',', 'grant', 'thornton', 'spa', 'repeated', 'previous', 'statements', 'that', 'it', 'was', '"', 'a', 'victim', 'of', 'fraudulent', 'action', '"', '.'], ['gen.', 'sattler', 'heads', 'a', 'combined', 'joint', 'task', 'force', 'based', 'on', 'ship', 'in', 'the', 'gulf', 'of', 'aden', 'and', 'the', 'indian', 'ocean', '.'], ['united', 'already', 'has', 'paid', 'the', 'city', '$', '34', 'million', 'in', 'penalties', 'for', 'not', 'meeting', 'the', 'first', 'round', 'of', 'employment', 'targets', '.'], ['since', 'being', 'drafted', 'into', 'service', 'in', '1971', ',', 'it', 'has', 'racked', 'up', 'a', 'record', '45', 'accidents', ',', 'with', '393', 'deaths', '.'], ['singapore', 'reported', 'no', 'suspected', 'sars', 'cases', 'wednesday', ',', 'but', 'officials', 'quarantined', '70', 'people', 'who', 'had', 'contact', 'with', 'the', 'taiwanese', 'patient', '.'], ['shinseki', 'retires', 'wednesday', 'after', 'a', '38-year', 'career', 'that', 'included', 'combat', 'in', 'vietnam', 'and', 'head', 'of', 'u.s.', 'peacekeeping', 'efforts', 'in', 'bosnia', '.'], ['the', 'unemployment', 'rate', 'rose', 'a', 'tenth', 'of', 'a', 'percentage', 'point', 'to', '6.1', '%', ',', 'the', 'highest', 'level', 'since', 'july', '1994', '.'], ['shares', 'of', 'mony', 'were', 'gaining', '$', '2.57', ',', 'or', '9', '%', ',', 'to', '$', '31.90', 'in', 'after-hours', 'trading', 'on', 'instinet', '.'], ['in', '2002', ',', 'linksys', 'overtook', 'cisco', 'systems', 'as', 'the', 'leading', 'wireless', 'equipment', 'vendor', ',', 'accounting', 'for', '14.1', 'percent', 'of', 'revenue', '.'], ['she', 'said', 'jane', 'doe', "'s", 'lawyers', 'asked', 'verizon', 'to', 'withhold', 'her', 'name', 'because', 'she', 'was', 'planning', 'on', 'challenging', 'the', 'subpoena', '.'], ['federal', 'judge', 'william', 'barbour', 'said', 'tuesday', 'he', 'imposed', 'the', 'maximum', 'sentence', 'because', 'avants', 'showed', 'no', 'remorse', 'in', 'the', 'brutal', 'slaying', '.'], ['deaths', 'in', 'rollover', 'crashes', 'accounted', 'for', '82', 'percent', 'of', 'the', 'number', 'of', 'traffic', 'deaths', 'in', '2002', ',', 'the', 'agency', 'says', '.'], ['the', 'dow', 'jones', 'industrials', 'climbed', 'more', 'than', '140', 'points', 'to', 'above', 'the', '9,000', 'mark', ',', 'the', 'first', 'time', 'since', 'december', '.'], ['five', 'alternate', 'jurors', 'were', 'also', 'chosen', ',', 'with', 'a', 'final', 'one', 'set', 'to', 'be', 'selected', 'friday', 'morning', 'from', 'the', 'panel', '.'], ['general', 'jeffrey', 'said', 'he', 'would', 'donate', 'his', 'military', 'pension', 'to', 'charity', 'for', 'the', 'period', 'he', 'was', 'in', 'office', 'at', 'yarralumla', '.'], ['a', 'second', 'victim', ',', 'michael', 'walker', ',', '23', ',', 'was', 'struck', 'in', 'the', 'torso', ',', 'the', 'bullet', 'perforating', 'his', 'lung', '.'], ['schools', 'that', 'fail', 'to', 'meet', 'state', 'goals', 'for', 'three', 'years', 'in', 'a', 'row', 'must', 'offer', 'tutoring', 'in', 'addition', 'to', 'transfers', '.'], ['dunlap', 'won', 'both', 'the', 'swimsuit', 'competition', 'and', 'the', 'talent', 'portion', 'of', 'the', 'competition', ',', 'singing', '"', 'if', 'i', 'could', '.', '"'], ['"', 'we', 'had', 'nothing', 'to', 'do', 'with', '@', 'stake', "'s", 'internal', 'personnel', 'decision', ',', '"', 'microsoft', 'spokesman', 'sean', 'sundwell', 'said', '.'], ['shares', 'of', 'coke', 'were', 'up', '6', 'cents', 'at', '$', '44.42', 'in', 'afternoon', 'trading', 'wednesday', 'on', 'the', 'new', 'york', 'stock', 'exchange', '.'], ['"', 'now', ',', 'the', 'other', 'guy', ',', 'he', "'s", 'ashamed', 'of', 'his', 'party', ',', '"', 'street', 'said', 'to', 'crowd', 'laughter', '.'], ['the', '30-year', 'bond', 'us30yt', '=', 'rr', 'jumped', '20', '/', '32', ',', 'taking', 'its', 'yield', 'to', '4.14', 'percent', 'from', '4.18', 'percent', '.'], ['gillette', 'shares', 'rose', '$', '1.45', ',', 'or', '4.5', 'percent', ',', 'to', '$', '33.95', 'in', 'afternoon', 'new', 'york', 'stock', 'exchange', 'trading', '.'], ['the', 'bulk', 'of', 'the', 'funds', ',', 'some', '$', '65', 'billion', ',', 'will', 'go', 'for', 'military', 'operations', 'in', 'iraq', 'and', 'afghanistan', '.'], ['barbini', "'s", 'comments', 'came', 'tuesday', ',', 'the', 'second', 'day', 'of', 'the', 'ministerial', 'conference', 'and', 'expo', 'on', 'agricultural', 'science', 'and', 'technology', '.'], ['a', 'lawsuit', 'has', 'been', 'filed', 'in', 'an', 'attempt', 'to', 'block', 'the', 'removal', 'of', 'the', 'ten', 'commandments', 'monument', 'from', 'the', 'building', '.'], ['aspen', 'technology', "'s", 'shares', 'dropped', '74', 'cents', ',', 'or', '23', 'percent', ',', 'to', 'close', 'at', '$', '2.48', 'on', 'the', 'nasdaq', '.'], ['last', 'week', ',', 'his', 'lawyers', 'asked', 'gov.', 'mark', 'r.', 'warner', 'to', 'grant', 'clemency', ',', 'but', 'the', 'governor', 'declined', 'to', 'intervene', '.'], ['based', 'on', 'a', 'separate', 'survey', 'of', 'households', ',', 'the', 'unemployment', 'rate', 'fell', 'in', 'august', 'to', '6.1', 'percent', 'from', '6.2', 'percent', '.'], ['the', 'who', 'says', 'the', 'latest', 'case', 'does', 'not', 'fit', 'its', 'profile', 'of', 'sars', 'and', 'is', 'not', 'a', 'public', 'health', 'concern', '.'], ['the', 'rule', 'bars', 'groups', 'from', 'airing', 'ads', 'that', 'promote', ',', 'support', ',', 'attack', 'or', 'oppose', 'a', 'candidate', 'at', 'any', 'time', '.'], ['singapore', 'is', 'already', 'the', 'united', 'states', "'", '12th-largest', 'trading', 'partner', ',', 'with', 'two-way', 'trade', 'totaling', 'more', 'than', '$', '34', 'billion', '.'], ['det', 'chief', 'insp', 'norman', 'mckinlay', 'said', 'there', 'was', '"', 'evidence', 'a', 'body', 'or', 'bodies', 'have', 'been', 'in', 'this', 'area', '"', '.'], ['some', 'women', 'are', 'also', 'concerned', 'that', 'so', 'many', 'women', 'are', 'turning', 'to', 'cosmetic', 'surgery', 'in', 'a', 'quest', 'for', 'bodily', 'perfection', '.'], ['"', 'certainly', 'what', 'we', 'know', 'suggests', 'that', 'we', 'should', 'take', 'what', 'they', "'re", 'saying', 'very', 'seriously', ',', '"', 'he', 'added', '.'], ['about', '25', 'governments', 'have', 'signed', 'in', 'the', 'last', 'four', 'months', ',', 'about', 'half', 'of', 'those', 'in', 'the', 'last', 'three', 'weeks', '.'], ['general', 'myers', 'told', 'reporters', 'that', '"', 'at', 'first', 'blush', ',', 'it', 'doesn', "'t", 'look', 'like', 'any', 'rules', 'were', 'broken', '"', '.'], ['we', 'need', 'a', 'certifiable', 'pay', 'as', 'you', 'go', 'budget', 'by', 'mid-july', 'or', 'schools', 'wont', 'open', 'in', 'september', ',', 'strayhorn', 'said', '.'], ['it', 'can', 'store', 'more', 'than', 'a', 'gigabyte', 'of', 'information', 'equivalent', 'to', 'around', '12', 'hours', 'of', 'music', 'in', 'one', 'cubic', 'centimetre', '.'], ['the', 'veteran', 'malyasian', 'diplomat', 'met', 'suu', 'kyi', 'wednesday', 'at', 'the', 'lakeside', 'home', 'in', 'yangon', 'where', 'she', 'is', 'under', 'house', 'arrest', '.'], ['janice', 'kelly', 'said', 'her', 'husband', 'had', 'received', 'assurances', 'from', 'senior', 'mod', 'officials', 'that', 'his', 'name', 'would', 'not', 'be', 'made', 'public', '.'], ['claudette', ',', 'the', 'first', 'hurricane', 'of', 'the', 'atlantic', 'storm', 'season', ',', 'hit', 'the', 'mid-texas', 'coast', 'july', '15', 'with', '85-mph', 'wind', '.'], ['he', 'urged', 'patience', 'from', 'americans', 'eager', 'for', 'the', 'service', ',', 'which', 'is', 'intended', 'to', 'block', 'about', '80', 'percent', 'of', 'telemarketing', 'calls', '.'], ['during', 'the', 'same', 'quarter', 'last', 'year', ',', 'eds', 'declared', 'a', 'profit', 'of', '$', '354', 'million', ',', 'or', '72', 'cents', 'per', 'share', '.'], ['two', 'astronomers', 'surveying', 'the', 'region', 'around', 'jupiter', 'have', 'detected', '20', 'new', 'moons', ',', 'bringing', 'the', 'giant', 'planet', "'s", 'total', 'to', '60', '.'], ['outside', 'the', 'court', ',', 'sriyanto', ',', 'who', 'faces', 'up', 'to', '25', 'years', "'", 'jail', ',', 'denied', 'he', 'had', 'done', 'anything', 'wrong', '.'], ['his', 'daughter', ',', 'nina', 'axelrod', ',', 'told', 'the', 'associated', 'press', 'that', 'he', 'died', 'in', 'his', 'sleep', ',', 'apparently', 'of', 'heart', 'failure', '.'], ['in', 'his', 'new', 'position', ',', 'dynes', 'will', 'earn', '$', '395,000', ',', 'a', 'significant', 'increase', 'over', 'atkinson', "'s", 'salary', 'of', '$', '361,400', '.'], ['the', 'broader', 'standard', '&', 'poor', "'s", '500', 'index', '<', '.spx', '>', 'eased', '7.57', 'points', ',', 'or', '0.76', 'percent', ',', 'at', '990.94', '.'], ['the', 'tech-heavy', 'nasdaq', 'composite', 'index', 'fell', '3.99', ',', 'or', '0.2', 'percent', ',', 'to', '1,682.72', ',', 'following', 'a', 'two-day', 'win', 'of', '55.93', '.'], ['with', 'all', 'precincts', 'reporting', ',', 'fletcher', '—', 'a', 'three-term', 'congressman', 'from', 'lexington', '—', 'had', 'an', 'overwhelming', '57', 'percent', 'of', 'the', 'vote', '.'], ['business', 'week', "'s", 'online', 'edition', 'reported', 'on', 'friday', 'that', 'worldcom', 'and', 'the', 'sec', 'could', 'announce', 'a', 'settlement', 'as', 'early', 'as', 'monday', '.'], ['the', 'center', "'s", 'president', ',', 'joseph', 'torsella', ',', 'was', 'struck', 'on', 'the', 'head', 'but', 'was', 'able', 'to', 'walk', 'to', 'an', 'ambulance', '.'], ['sec', 'chairman', 'william', 'donaldson', 'said', 'there', 'is', 'a', '"', 'building', 'confidence', 'out', 'there', 'that', 'the', 'cop', 'is', 'on', 'the', 'beat', '.', '"'], ['only', '29', 'families', 'of', 'frogs', 'are', 'known', 'and', 'most', 'were', 'identified', 'and', 'described', 'in', 'the', 'mid-1800s', 'and', 'the', 'last', 'in', '1926', '.'], ['cisco', 'has', 'signed', 'similar', 'deals', 'with', 'at', '&', 't', 'corp.', 't.n', ',', 'sbc', 'communications', 'inc', '.', 'sbc.n', 'and', 'sprint', 'corp.', 'fon.n', '.'], ['boeing', "'s", 'board', 'is', 'also', 'expected', 'to', 'decide', 'at', 'its', 'december', 'meeting', 'whether', 'to', 'offer', 'the', 'airplane', 'for', 'sale', 'to', 'airlines', '.'], ['idec', 'shareholders', 'would', 'own', '50.5', 'percent', 'of', 'the', 'stock', 'of', 'the', 'combined', 'company', ',', 'and', 'biogen', 'shareholders', 'the', 'remaining', '49.5', 'percent', '.'], ['nonetheless', ',', 'the', 'open-source', 'guru', 'insisted', ',', '"', 'this', 'attack', 'was', 'wrong', ',', 'and', 'it', 'was', 'dangerous', 'to', 'our', 'goals', '.', '"'], ['paul', 'themba', 'nyathi', ',', 'an', 'mdc', 'spokesman', ',', 'said', 'the', 'avenues', 'clinic', 'was', 'being', 'targeted', 'because', 'it', 'was', 'treating', 'opposition', 'supporters', '.'], ['analysts', "'", 'consensus', 'estimate', 'from', 'thomson', 'first', 'call', 'was', 'for', 'a', 'loss', 'of', '$', '2.08', 'a', 'share', ',', 'excluding', 'one-time', 'items', '.'], ['franklin', 'county', 'judge-executive', 'teresa', 'barton', 'said', 'a', 'firefighter', 'was', 'struck', 'by', 'lightning', 'and', 'was', 'taken', 'to', 'the', 'frankfort', 'regional', 'medical', 'center', '.'], ['he', 'was', 'the', 'ace', 'of', 'diamonds', 'in', 'a', 'pack', 'of', 'cards', 'depicting', 'iraqi', 'fugitives', 'that', 'has', 'been', 'issued', 'to', 'u.s.', 'troops', '.'], ['after', 'protesters', 'rushed', 'the', 'stage', 'and', 'twice', 'cut', 'power', 'to', 'the', 'microphone', ',', 'hedges', 'drew', 'the', 'speech', 'to', 'an', 'early', 'close', '.'], ['the', 'unions', 'also', 'staged', 'a', 'five-day', 'strike', 'in', 'march', 'that', 'forced', 'all', 'but', 'one', 'of', 'yale', "'s", 'dining', 'halls', 'to', 'close', '.'], ['michelle', 'wie', 'chips', 'sunday', 'during', 'her', '1-up', 'victory', 'over', 'virada', 'nirapathpongporn', 'in', 'the', '27th', 'u.s.', 'women', "'s", 'amateur', 'public', 'links', 'championship', '.'], ['stony', 'brook', 'university', 'launched', 'the', 'study', 'in', '1996', ',', 'after', 'earlier', 'studies', 'indicated', 'a', 'possible', 'connection', 'between', 'electromagnetic', 'fields', 'and', 'cancer', '.'], ['the', 'company', 'said', 'it', 'would', 'issue', 'revised', 'guidance', 'for', 'the', 'full', 'fiscal', 'year', 'next', 'month', 'when', 'it', 'releases', 'its', 'q2', 'results', '.'], ['nick', 'markakis', ',', 'a', 'left-hander', 'from', 'young', 'harris', 'junior', 'college', 'in', 'georgia', ',', 'went', 'to', 'the', 'orioles', 'with', 'the', 'seventh', 'pick', '.'], ['kansas', 'department', 'of', 'health', 'and', 'environment', 'records', 'show', 'there', 'were', '88', 'abortions', 'performed', 'on', 'girls', 'age', '14', 'and', 'younger', 'last', 'year', '.'], ['last', 'week', ',', 'his', 'lawyers', 'asked', 'warner', 'to', 'grant', 'clemency', 'under', 'conditions', 'that', 'would', 'have', 'lead', 'to', 'a', 'new', 'sentencing', 'hearing', '.'], ['a', 'seventh', 'victim', ',', 'a', 'woman', ',', 'died', 'tuesday', 'night', 'in', 'the', 'burn', 'unit', 'at', 'childrens', 'hospital', 'medical', 'center', 'of', 'akron', '.'], ['but', 'the', 'country', 'is', 'scrambling', 'to', 'prevent', 'it', 'spreading', 'to', 'the', 'vast', 'countryside', 'where', 'most', 'of', 'its', '1.3', 'billion', 'people', 'live', '.'], ['he', 'was', 'released', 'on', '$', '3-million', 'bail', 'and', 'immediately', 'returned', 'to', 'las', 'vegas', ',', 'where', 'he', 'had', 'been', 'filming', 'a', 'video', '.'], ['british-based', 'glaxosmithkline', 'plc', 'said', 'earlier', 'this', 'year', 'it', 'would', 'cut', 'off', 'supplies', 'to', 'canadian', 'drugstores', 'that', 'ship', 'to', 'the', 'united', 'states', '.'], ['we', 'remain', 'hopeful', 'that', 'the', 'city', 'will', 'agree', 'to', 'work', 'with', 'us', 'and', 'engage', 'in', 'good-faith', 'discussions', 'on', 'this', 'issue', '.', '"'], ['the', 'injured', 'were', 'taken', 'to', 'hospitals', 'in', 'jefferson', 'city', 'and', 'columbia', ',', 'a', 'college', 'town', 'about', '30', 'miles', 'to', 'the', 'north', '.'], ['hundreds', 'of', 'reporters', 'and', 'photographers', 'swamped', 'the', 'courthouse', 'grounds', 'before', 'the', 'hearing', ',', 'which', 'was', 'carried', 'live', 'on', 'national', 'cable', 'networks', '.'], ['rosenthal', 'declined', 'comment', 'on', 'the', 'garrett', 'situation', 'tuesday', 'but', 'said', 'in', 'a', 'statement', ':', '"', 'we', 'had', 'a', 'big', 'contract', 'negotiation', '.'], ['previously', ',', 'it', 'had', 'reported', 'a', 'profit', 'of', '$', '12', 'million', ',', 'or', '0', 'cents', 'a', 'share', ',', 'for', 'that', 'period', '.'], ['"', 'we', 'condemn', 'and', 'denounce', 'the', 'governing', 'council', ',', 'which', 'is', 'headed', 'by', 'the', 'united', 'states', ',', '"', 'moqtada', 'al-sadr', 'said', '.'], ['daniels', 'said', 'he', 'doesn', "'t", 'know', 'what', 'bush', 'will', 'have', 'to', 'say', 'about', 'him', 'at', 'tonight', "'s", '$', '2,000-per-person', 'presidential', 'fund-raiser', '.'], ['on', 'monday', ',', 'the', 'dow', 'declined', '5.25', ',', 'or', '0.1', 'percent', ',', 'at', '8,983.80', ',', 'having', 'shed', '2.3', 'percent', 'last', 'week', '.'], ['glover', 'said', 'the', 'killer', 'used', 'a', '"', 'peculiar', 'slipknot', '"', 'on', 'the', 'ligature', 'he', 'used', 'to', 'strangle', 'some', 'of', 'the', 'victims', '.'], ['the', 'boston', 'archdiocese', 'has', 'faced', 'waves', 'of', 'scandal', 'that', 'have', 'not', 'only', 'angered', 'victims', "'", 'advocates', 'but', 'parishioners', 'and', 'some', 'priests', '.'], ['the', '2000', 'democratic', 'platform', 'supported', '"', 'the', 'full', 'inclusion', 'of', 'gay', 'and', 'lesbian', 'families', 'in', 'the', 'life', 'of', 'the', 'nation', '.', '"'], ['"', 'he', 'really', 'left', 'us', 'with', 'a', 'smile', 'on', 'his', 'face', 'and', 'no', 'last', 'words', ',', '"', 'daughter', 'linda', 'hope', 'said', '.'], ['on', 'saturday', ',', 'the', 'international', 'committee', 'of', 'the', 'red', 'cross', 'said', 'it', 'was', 'temporarily', 'closing', 'its', 'offices', 'in', 'baghdad', 'and', 'basra', '.'], ['the', 'civilian', 'unemployment', 'rate', 'improved', 'marginally', 'last', 'month', '--', 'slipping', 'to', '6.1', 'percent', '--', 'even', 'as', 'companies', 'slashed', 'payrolls', 'by', '93,000', '.'], ['it', 'will', 'cost', 'about', '$', '20,000', 'per', 'eight-week', 'course', 'of', 'treatment', ',', 'comparable', 'to', 'other', 'injected', 'cancer', 'therapies', ',', 'a', 'spokeswoman', 'said', '.'], ['also', 'at', 'increased', 'risk', 'are', 'those', 'whose', 'immune', 'systems', 'suppressed', 'by', 'medications', 'or', 'by', 'diseases', 'such', 'as', 'cancer', ',', 'diabetes', 'and', 'aids', '.'], ['the', 'new', 'army', 'commander', 'is', 'the', 'masaka', 'armoured', 'brigade', 'commanding', 'officer', ',', 'brigadier', 'aronda', 'nyakairima', 'who', 'is', 'now', 'promoted', 'to', 'major', 'general', '.'], ['ryan', 'harvey', ',', 'an', 'outfielder', 'from', 'dunedin', 'high', 'school', 'in', 'florida', ',', 'was', 'selected', 'with', 'the', 'sixth', 'pick', 'by', 'the', 'chicago', 'cubs', '.'], ['coupling', ',', 'an', 'american', 'version', 'of', 'a', 'hit', 'british', 'comedy', ',', 'will', 'get', 'the', 'valuable', 'thursday', '9', ':', '30', 'p.m.', 'time', 'slot', '.'], ['the', 'time', 'was', 'about', '4', 'a.m.', 'on', 'march', '18', ',', 'just', 'hours', 'before', 'the', 'first', 'pinpoint', 'missiles', 'rained', 'down', 'on', 'the', 'capital', '.'], ['under', 'the', 'u.s.', 'constitution', ',', 'the', 'other', 'chamber', 'of', 'congress', ',', 'the', 'house', 'of', 'representatives', ',', 'does', 'not', 'vote', 'on', 'foreign', 'alliances', '.'], ['the', 'broader', 'standard', '&', 'poor', "'s", '500', 'index', '<', '.spx', '>', 'was', 'up', '9.69', 'points', ',', 'or', '0.98', 'percent', ',', 'at', '994.53', '.'], ['the', 'broad', 'standard', '&', 'poor', "'s", '500', 'index', '<', '.spx', '>', 'was', 'up', '8.79', 'points', ',', 'or', '0.96', 'percent', ',', 'at', '929.06', '.'], ['earlier', 'this', 'month', ',', 'rim', 'had', 'said', 'it', 'expected', 'to', 'report', 'second-quarter', 'earnings', 'of', 'between', '7', 'cents', 'and', '11', 'cents', 'a', 'share', '.'], ['we', 'don', "'t", 'know', 'if', 'any', 'will', 'be', 'sars', ',', '"', 'said', 'dr.', 'james', 'young', ',', 'ontario', "'s", 'commissioner', 'of', 'public', 'safety', '.'], ['"', 'they', 'were', 'brutally', 'beaten', ',', 'and', 'it', "'s", 'really', 'a', 'wonder', 'that', 'porchia', 'is', 'the', 'only', 'deceased', 'in', 'this', 'case', '.', '"'], ['the', '3', '{', 'hours', 'of', 'recordings', 'include', 'conversations', 'between', 'police', ',', 'firefighters', 'and', 'other', 'emergency', 'workers', ',', 'as', 'well', 'as', 'media', 'inquiries', '.'], ['the', 'giant', 'rock', 'was', 'first', 'observed', 'on', 'august', '24', 'by', 'lincoln', 'near-earth', 'asteroid', 'research', 'program', ',', 'based', 'in', 'socorro', ',', 'new', 'mexico', '.'], ['ross', 'garber', ',', 'rowland', "'s", 'lawyer', ',', 'said', 'tuesday', 'he', 'would', 'attend', 'the', 'meeting', 'and', 'would', 'ask', 'to', 'speak', 'on', 'the', 'issue', '.'], ['shares', 'of', 'lendingtree', 'soared', '$', '5.99', ',', 'or', '40.1', 'percent', ',', 'to', '$', '20.68', 'in', 'afternoon', 'trading', 'on', 'the', 'nasdaq', 'stock', 'market', '.'], ['special', 'police', 'detained', 'khodorkovsky', 'early', 'on', 'saturday', 'in', 'the', 'siberian', 'city', 'of', 'novosibirsk', ',', 'where', 'his', 'plane', 'had', 'made', 'a', 'refuelling', 'stop', '.'], ['"', 'three', 'weeks', 'in', 'october', '"', 'goes', 'on', 'sale', 'monday', ',', 'nearly', 'a', 'year', 'after', 'the', 'washington', ',', 'd.c.-area', 'sniper', 'shootings', 'started', '.'], ['the', 'shares', 'represent', 'more', 'than', 'half', 'the', '115.9', 'million', 'shares', 'turner', 'held', 'at', 'the', 'end', 'of', 'april', ',', 'according', 'to', 'bloomberg', 'data', '.'], ['it', "'s", 'also', 'a', 'strategic', 'win', 'for', 'overture', ',', 'given', 'that', 'knight', 'ridder', 'had', 'the', 'option', 'of', 'signing', 'on', 'google', "'s", 'services', '.'], ['the', 'seven', 'other', 'nations', 'in', 'the', 'initiative', 'are', 'britain', ',', 'germany', ',', 'italy', ',', 'the', 'netherlands', ',', 'poland', ',', 'portugal', 'and', 'spain', '.'], ['north', 'american', 'markets', 'finished', 'mixed', 'in', 'directionless', 'trading', 'monday', 'as', 'earnings', 'season', 'begins', 'to', 'slow', 'and', 'economic', 'indicators', 'move', 'into', 'the', 'spotlight', '.'], ['to', 'reach', 'john', 'a.', 'dvorak', ',', 'who', 'covers', 'kansas', ',', 'call', '(', '816', ')', '234-7743', 'or', 'send', 'e-mail', 'to', 'jdvorak', '@', 'kctar.com', '.'], ['a', 'nationally', 'board', 'certified', 'teacher', 'with', 'a', 'master', "'s", 'degree', ',', 'kelley', 'makes', 'a', 'salary', 'of', '$', '65,000', 'in', 'his', '30th', 'year', '.'], ['several', 'relatives', 'of', 'australian', 'victims', 'of', 'the', 'attack', 'were', 'in', 'court', 'to', 'witness', 'the', 'proceedings', ',', 'but', 'few', 'balinese', 'attended', 'the', 'trial', '.'], ['"', 'it', "'s", 'going', 'to', 'happen', ',', '"', 'said', 'jim', 'santangelo', ',', 'president', 'of', 'the', 'teamsters', 'joint', 'council', '42', 'in', 'el', 'monte', '.'], ['earlier', ',', 'he', 'told', 'france', 'inter-radio', ',', '"', 'i', 'think', 'we', 'can', 'now', 'qualify', 'what', 'is', 'happening', 'as', 'a', 'genuine', 'epidemic', '.', '"'], ['but', 'the', 'justices', 'ruled', 'that', 'the', 'police', 'supervisor', 'who', 'repeatedly', 'questioned', 'martinez', 'did', 'not', 'violate', 'his', 'fifth', 'amendment', 'rights', 'in', 'doing', 'so', '.'], ['the', 'transaction', 'will', 'grant', 'handspring', 'stockholders', '0.09', 'of', 'a', 'share', 'of', 'palm--and', 'no', 'shares', 'of', 'palmsource--for', 'each', 'share', 'of', 'handspring', 'common', 'stock', '.'], ['artists', 'are', 'worried', 'the', 'plan', 'would', 'harm', 'those', 'who', 'need', 'help', 'most', '-', 'performers', 'who', 'have', 'a', 'difficult', 'time', 'lining', 'up', 'shows', '.'], ['if', 'convicted', ',', 'they', 'face', 'up', 'to', 'five', 'years', 'in', 'prison', 'and', 'a', '$', '250,000', 'fine', 'on', 'each', 'of', 'the', '11', 'counts', '.'], ['defense', 'lawyers', 'had', 'said', 'a', 'change', 'of', 'venue', 'was', 'needed', 'because', 'massive', 'pretrial', 'publicity', 'had', 'tainted', 'the', 'jury', 'pool', 'against', 'their', 'client', '.'], ['the', 'department', "'s", 'position', 'threatens', 'to', 'alienate', 'social', 'conservatives', ',', 'who', 'have', 'provided', 'strong', 'political', 'support', 'for', 'mr.', 'ashcroft', 'and', 'president', 'bush', '.'], ['ursel', 'reisen', 'confirmed', 'it', 'operated', 'the', 'coach', ',', 'but', 'gave', 'no', 'details', 'other', 'than', 'to', 'say', 'the', 'passengers', 'were', 'of', 'mixed', 'ages', '.'], ['when', 'you', 'crossed', 'the', 'line', ',', 'you', 'violated', 'the', 'constitutional', 'right', ',', '"', 'said', 'charles', 'weisselberg', ',', 'a', 'uc', 'berkeley', 'law', 'professor', '.'], ['"', 'let', "'s", 'show', 'we', 'won', "'t", 'let', 'them', 'off', ',', '"', 'said', 'marc', 'blondel', ',', 'leader', 'of', 'the', 'force', 'ouvriere', 'union', '.'], ['lee', 'janzen', ',', 'who', 'was', 'tied', 'atop', 'the', 'leaderboard', 'with', 'five', 'holes', 'left', ',', 'finished', 'with', 'a', '68', 'and', 'tied', 'for', 'sixth', '.'], ['diana', 'was', 'at', 'her', 'lowest', 'ebb', 'when', 'she', 'wrote', 'the', 'letter', 'to', 'paul', 'burrell', 'claiming', 'there', 'was', 'a', 'plot', 'to', 'kill', 'her', '.'], ['"', 'i', 'do', 'not', 'wish', 'to', 'be', 'present', 'during', 'this', 'witness', ',', '"', 'he', 'told', 'stanislaus', 'county', 'superior', 'court', 'judge', 'al', 'girolami', '.'], ['the', 'u.s.', 'conference', 'board', 'said', 'its', 'latest', 'measure', 'of', 'business', 'confidence', 'hit', '60', 'after', 'falling', 'to', '53', 'in', 'its', 'first', 'quarter', 'survey', '.'], ['"', 'today', ',', 'we', 'are', 'trying', 'to', 'convey', 'this', 'problem', 'to', 'russian', 'president', 'vladimir', 'putin', 'and', 'us', 'president', 'george', 'w', 'bush', '.', '"'], ['scribner', "'s", 'body', 'was', 'found', 'floating', 'in', 'the', 'city', "'s", 'portage', 'bay', ',', 'where', 'he', 'lived', 'with', 'his', 'wife', 'in', 'a', 'houseboat', '.'], ['"', 'what', 'they', 'have', 'done', 'is', 'a', 'thinly', 'veiled', 'attempt', 'to', 'do', 'an', 'end', 'run', 'around', 'the', 'constitution', ',', '"', 'she', 'said', '.'], ['the', 'blue-chip', 'dow', 'jones', 'industrial', 'average', 'eased', '44', 'points', ',', 'or', '0.47', 'percent', ',', 'to', '9,543', ',', 'after', 'scoring', 'five', 'consecutive', 'up', 'sessions', '.'], ['bush', 'said', 'resistance', 'forces', 'hostile', 'to', 'the', 'u.s.', 'presence', '"', 'feel', 'like', '...', 'the', 'conditions', 'are', 'such', 'that', 'they', 'can', 'attack', 'us', 'there', '.'], ['the', 'men', 'were', 'immediately', 'flown', 'to', 'nearby', 'botswana', 'on', 'a', 'chartered', 'air', 'malawi', 'flight', ',', 'malawi', 'intelligence', 'officials', 'said', 'on', 'condition', 'of', 'anonymity', '.'], ['prior', 'to', 'joining', 'kodak', ',', 'palumbo', 'worked', 'for', 'procter', '&', 'gamble', 'corp.', ',', 'where', 'he', 'managed', 'products', 'such', 'as', 'folgers', 'and', 'pantene', 'shampoo', '.'], ['on', 'the', 'cell', "'s", 'surface', ',', 'the', 'foreign', 'components', 'are', 'recognised', 'by', 'other', 'cells', 'that', 'then', 'realise', 'it', 'is', 'infected', 'and', 'kill', 'it', '.'], ['also', 'missing', 'is', 'al', 'larsen', ',', '31', ',', 'of', 'fort', 'worth', ',', 'texas', ',', 'who', 'was', 'in', 'another', 'vehicle', 'swept', 'off', 'the', 'turnpike', '.'], ['martin', ',', '58', ',', 'will', 'be', 'freed', 'today', 'after', 'serving', 'two', 'thirds', 'of', 'his', 'five-year', 'sentence', 'for', 'the', 'manslaughter', 'of', '16-year-old', 'fred', 'barras', '.'], ['ukrainian', 'president', 'leonid', 'kuchma', 'today', 'cut', 'short', 'a', 'visit', 'to', 'latin', 'america', 'as', 'a', 'bitter', 'border', 'wrangle', 'between', 'ukraine', 'and', 'russia', 'deteriorates', 'further', '.'], ['sorkin', ',', 'who', 'faces', 'charges', 'of', 'conspiracy', 'to', 'obstruct', 'justice', 'and', 'lying', 'to', 'a', 'grand', 'jury', ',', 'was', 'to', 'have', 'been', 'tried', 'separately', '.'], ['shares', 'of', 'guidant', 'plummeted', 'in', 'trading', 'both', 'on', 'and', 'off', 'the', 'new', 'york', 'stock', 'exchange', 'before', 'a', 'news', 'halt', 'was', 'imposed', 'after', 'midday', '.'], ['the', 'company', 'is', 'highlighting', 'the', 'addition', 'of', '.net', 'and', 'j2ee', 'support', 'to', 'its', 'enterprise', 'application', 'environment', '(', 'eae', ')', 'with', 'the', 'new', 'mainframe', '.'], ['she', 'said', 'he', 'persisted', 'and', 'she', 'eventually', 'allowed', 'him', 'to', 'fix', 'her', 'bike', 'while', 'she', 'sat', 'in', 'the', 'front', 'seat', 'of', 'his', 'car', '.'], ['the', 's', '&', 'p', '/', 'tsx', 'composite', 'rose', '87.74', 'points', 'on', 'the', 'week', ',', 'while', 'the', 'tsx', 'venture', 'exchange', 'composite', 'gained', '44.49', 'points', '.'], ['pg', '&', 'e', 'corp.', 'shares', 'were', 'up', '39', 'cents', 'or', '2.6', 'percent', 'at', '$', '15.59', 'on', 'the', 'new', 'york', 'stock', 'exchange', 'on', 'tuesday', '.'], ['"', 'we', 'condemn', 'the', 'governing', 'council', 'headed', 'by', 'the', 'united', 'states', ',', '"', 'muqtada', 'al', 'sadr', 'said', 'in', 'a', 'sermon', 'at', 'a', 'mosque', '.'], ['gov.', 'rick', 'perry', 'has', 'said', 'that', 'while', 'he', 'opposes', 'gambling', 'expansion', ',', 'he', 'would', 'be', 'reluctant', 'to', 'veto', 'continuation', 'of', 'the', 'lottery', 'commission', '.'], ['treasurys', 'long-range', 'plan', 'is', 'to', 'provide', 'retail', 'buyers', 'of', 'all', 'treasury', 'securities', 'the', 'ability', 'to', 'manage', 'their', 'holdings', 'online', 'in', 'a', 'single', 'account', '.'], ['smugglers', 'in', 'a', 'van', 'chased', 'down', 'a', 'pickup', 'and', 'suv', 'carrying', 'other', 'smugglers', 'and', 'illegal', 'immigrants', ',', 'said', 'pinal', 'county', 'sheriff', 'roger', 'vanderpool', '.'], ['the', 'national', 'denomination', 'of', 'the', 'episcopal', 'church', ',', 'with', '2.3', 'million', 'members', ',', 'is', 'the', 'u.s.', 'branch', 'of', 'the', '77', 'million-member', 'anglican', 'communion', '.'], ['tom', 'kasmer', ',', 'a', '14-year-old', 'from', 'belmont', ',', 'n.c.', ',', 'got', 'a', 'word', 'that', 'sounded', 'like', '"', 'zistee', '"', 'during', 'yesterday', "'s", 'competition', '.'], ['the', 'president', 'began', 'his', 'speech', 'by', 'acknowledging', 'the', 'terrorist', 'attacks', 'in', 'saudi', 'arabia', 'that', 'killed', 'at', 'least', '29', 'people', ',', 'including', 'seven', 'americans', '.'], ['this', 'integrates', 'with', 'rational', 'purifyplus', 'and', 'allows', 'developers', 'to', 'work', 'in', 'supported', 'versions', 'of', 'java', ',', 'visual', 'c', '#', 'and', 'visual', 'basic', '.net', '.'], ['the', 'agency', 'will', '"', 'consider', 'as', 'timely', 'any', 'tax', 'returns', 'or', 'payments', 'due', '"', 'aug.', '15', 'if', 'they', 'are', 'submitted', 'by', 'aug.', '22', '.'], ['another', 'million', 'barrels', ',', 'bought', 'by', 'spanish', 'refiner', 'cepsa', 'sa', ',', 'was', 'to', 'be', 'loaded', 'onto', 'a', 'spanish', 'tanker', ',', 'the', 'sandra', 'tapias', '.'], ['after', '26', 'hours', 'of', 'surgery', 'and', 'a', 'year', 'of', 'anticipation', ',', 'the', 'boys', 'were', 'separated', 'sunday', 'at', 'children', "'s", 'medical', 'center', 'of', 'dallas', '.'], ['the', 'government', 'has', 'chosen', 'three', 'companies', 'to', 'develop', 'plans', 'to', 'protect', 'commercial', 'aircraft', 'from', 'shoulder-fired', 'missile', 'attacks', ',', 'homeland', 'security', 'officials', 'announced', 'tuesday', '.'], ['the', 'ministry', 'of', 'defence', 'is', 'facing', 'a', '20m', 'compensation', 'claim', 'from', 'hundreds', 'of', 'kenyan', 'women', 'who', 'claim', 'they', 'were', 'raped', 'by', 'british', 'soldiers', '.'], ['but', 'in', 'the', 'first', '30', 'seconds', 'after', 'young', 'entered', 'the', 'ring', ',', 'the', 'family', 'knew', 'it', 'was', 'an', 'uneven', 'match', ',', 'meyers', 'said', '.'], ['the', '30-year', 'bond', 'us30yt', '=', 'rr', 'grew', '1-3', '/', '32', 'for', 'a', 'yield', 'of', '4.30', 'percent', ',', 'down', 'from', '4.35', 'percent', 'late', 'wednesday', '.'], ['the', 'metropolitan', 'transportation', 'authority', 'was', 'given', 'two', 'weeks', 'to', 'restore', 'the', '$', '1.50', 'fare', 'and', 'the', 'old', 'commuter', 'railroad', 'rates', ',', 'york', 'declared', '.'], ['metropolitan', 'ambulance', 'spokesman', 'james', 'howe', 'said', 'five', 'people', 'were', 'taken', 'to', 'hospital', 'and', 'three', 'were', 'treated', 'at', 'the', 'scene', 'after', 'yesterday', "'s", 'incident', '.'], ['"', 'both', 'of', 'these', 'kids', 'are', 'in', 'wonderful', 'physical', 'condition', 'right', 'now', ',', '"', 'he', 'said', 'during', 'a', 'press', 'briefing', 'at', 'the', 'hospital', '.'], ['the', 'vulnerability', 'affects', 'windows', 'nt', '4.0', ',', 'nt', '4.0', 'terminal', 'services', 'edition', ',', 'xp', 'and', '2000', ',', 'as', 'well', 'as', 'windows', 'server', '2003', '.'], ['"', 'my', 'judgment', 'is', '95', 'percent', 'of', 'that', 'information', 'should', 'be', 'declassified', ',', 'become', 'uncensored', ',', 'so', 'the', 'american', 'people', 'would', 'know', '.', '"'], ['we', 'believe', 'the', 'report', 'is', 'fully', 'consistent', 'with', 'what', 'the', 'courts', 'have', 'ruled', '...', 'that', 'the', 'department', "'s", 'actions', 'are', 'fully', 'within', 'the', 'law', '.'], ['the', 'man', ',', 'who', 'entered', 'the', 'post', 'office', 'in', 'lakeside', 'shortly', 'before', '3', 'p.m.', ',', 'gave', 'up', 'to', 'deputies', 'about', '6', ':', '30', 'p.m', '.'], ['miodrag', 'zivkovic', ',', 'of', 'the', 'liberal', 'alliance', 'of', 'montenegro', ',', 'won', '31', 'percent', 'of', 'the', 'vote', 'while', 'the', 'independent', 'dragan', 'hajdukovic', 'got', 'four', 'percent', '.'], ['"', 'these', 'are', 'dark', 'days', 'for', 'our', 'industry', ',', '"', 'giovanni', 'bisignani', ',', 'director', 'general', 'of', 'the', 'geneva-based', 'organization', ',', 'said', 'in', 'a', 'statement', '.'], ['but', 'price', 'declines', 'have', 'remained', 'below', '30', 'percent', ',', 'year', 'over', 'year', ',', 'for', 'the', 'last', 'two', 'quarters', ',', 'said', 'idc', 'analyst', 'john', 'mcarthur', '.'], ['"', 'for', 'me', ',', 'the', 'lewinsky', 'imbroglio', 'seemed', 'like', 'just', 'another', 'vicious', 'scandal', 'manufactured', 'by', 'political', 'opponents', ',', '"', 'according', 'to', 'extracts', 'released', 'yesterday', '.'], ['the', 'single', 'currency', 'rose', 'as', 'far', 'as', '135.13', 'yen', ',', 'matching', 'a', 'record', 'high', 'hit', 'shortly', 'after', 'the', 'euro', "'s", 'launch', 'in', 'january', '1999', '.'], ['the', '30-year', 'bond', 'us30yt', '=', 'rr', 'rose', '22', '/', '32', 'for', 'a', 'yield', 'of', '4.31', 'percent', ',', 'versus', '4.35', 'percent', 'at', 'wednesday', "'s", 'close', '.'], ['sony', 'ericsson', 'also', 'said', 'it', 'would', 'shut', 'down', 'its', 'gsm', '/', 'umts', 'r', '&', 'd', 'center', 'in', 'munich', ',', 'germany', ',', 'to', 'increase', 'profitability', '.'], ['north', 'american', 'futures', 'pointed', 'to', 'a', 'sub-par', 'start', 'to', 'trading', 'on', 'tuesday', ',', 'with', 'investors', 'ready', 'to', 'get', 'their', 'first', 'taste', 'of', 'quarterly', 'earnings', '.'], ['in', 'addition', 'to', 'o', "'connor", ',', 'rehnquist', "'s", 'majority', 'opinion', 'was', 'joined', 'by', 'justices', 'david', 'souter', ',', 'ruth', 'bader', 'ginsburg', ',', 'and', 'stephen', 'breyer', '.'], ['in', '2001', 'and', '2002', ',', 'wire', 'transfers', 'from', '4', 'of', 'the', 'company', "'s", '40', 'accounts', 'totaled', 'more', 'than', '$', '3.2', 'billion', ',', 'prosecutors', 'said', '.'], ['dotson', ',', '21', ',', 'was', 'arrested', 'and', 'charged', 'on', 'july', '21', 'after', 'reportedly', 'telling', 'authorities', 'he', 'shot', 'dennehy', 'after', 'dennehy', 'tried', 'to', 'shoot', 'him', '.'], ['dolores', 'mahoy', ',', '68', ',', 'of', 'colorado', 'springs', ',', 'colo', '.', ',', 'is', 'someone', 'who', 'might', 'be', 'helped', 'by', 'the', 'legislation', 'pending', 'in', 'congress', '.'], ['in', 'canada', ',', 'the', 'booming', 'dollar', 'will', 'be', 'in', 'focus', 'again', 'as', 'it', 'tries', 'to', 'stay', 'above', 'the', '75', 'cent', '(', 'u.s.', ')', 'mark', '.'], ['some', '660', 'prisoners', 'from', '42', 'countries', ',', 'including', 'canada', ',', 'are', 'held', ',', 'many', 'captured', 'during', 'the', 'war', 'against', 'in', 'afghanistan', 'the', 'al-qaida', 'network', '.'], ['the', 'euro', 'soared', 'to', 'us', '$', '1.1914', 'in', 'asian', 'trading', ',', 'before', 'slipping', 'back', 'slightly', 'to', 'us', '$', '1.1895', 'as', 'trading', 'opened', 'in', 'europe', '.'], ['teams', 'of', 'inspectors', 'walked', 'into', 'a', 'restricted', 'area', 'at', 'one', 'park', 'without', 'being', 'stopped', ';', 'others', 'took', 'pictures', 'of', 'sensitive', 'areas', 'without', 'being', 'challenged', '.'], ['lord', 'falconer', 'hailed', 'the', 'changes', 'as', '"', 'a', 'new', 'beginning', 'as', 'far', 'as', 'the', 'courts', ',', 'crown', 'prosecution', 'service', 'and', 'police', 'are', 'concerned', '"', '.'], ['luzerne', 'county', 'district', 'attorney', 'david', 'lupas', 'told', 'a', 'news', 'conference', 'monday', 'that', 'authorities', 'were', '``continuing', 'to', 'make', 'significant', 'progress', "'", "'", 'in', 'the', 'case', '.'], ['passed', 'in', '1999', 'but', 'never', 'put', 'into', 'effect', ',', 'the', 'law', 'would', 'have', 'made', 'it', 'illegal', 'for', 'bar', 'and', 'restaurant', 'patrons', 'to', 'light', 'up', '.'], ['pratt', '&', 'whitney', 'had', 'said', 'that', '75', 'per', 'cent', 'of', 'the', 'engine', 'equipment', 'would', 'be', 'outsourced', 'to', 'europe', ',', 'with', 'final', 'assembly', 'in', 'germany', '.'], ['the', 'us', 'senate', 'judiciary', 'committee', 'overcame', 'a', 'significant', 'hurdle', 'yesterday', 'in', 'the', 'battle', 'to', 'create', 'a', 'trust', 'fund', 'to', 'pay', 'victims', 'of', 'asbestos', 'exposure', '.'], ['the', 'fed', "'s", 'empire', 'state', 'survey', 'far', 'exceeded', 'expectations', 'by', 'jumping', 'to', '26.8', 'in', 'june', ',', 'a', 'record', 'high', ',', 'from', '10.6', 'in', 'may', '.'], ['the', 'report', 'shows', 'that', 'drugs', 'sold', 'in', 'canadian', 'pharmacies', 'are', 'manufactured', 'in', 'facilities', 'approved', 'by', 'health', 'canada', '-', 'the', 'fda', "'s", 'counterpart', 'in', 'canada', '.'], ['mr.', 'bush', 'had', 'sought', 'to', 'store', 'his', 'papers', 'in', 'his', 'father', "'s", 'presidential', 'library', ',', 'where', 'they', 'would', 'have', 'stayed', 'secret', 'for', 'a', 'half-century', '.'], ['she', 'countersued', 'for', '$', '125', 'million', ',', 'saying', 'gruner', '&', 'jahr', 'broke', 'its', 'contract', 'with', 'her', 'by', 'cutting', 'her', 'out', 'of', 'key', 'editorial', 'decisions', '.'], ['shelia', 'chaney', 'wilson', 'seemed', 'agitated', 'when', 'she', 'came', 'to', 'help', 'prepare', 'for', 'turner', 'monumental', 'a.m.e.', 'church', "'s", 'upcoming', '104th', 'anniversary', 'celebration', ',', 'worshippers', 'said', '.'], ['"', 'jeremy', "'s", 'a', 'good', 'guy', ',', '"', 'barber', 'said', ',', 'adding', ':', '"', 'jeremy', 'is', 'living', 'the', 'dream', 'life', 'of', 'the', 'new', 'york', 'athlete', '.'], ['the', 'european', 'union', 'banned', 'the', 'import', 'of', 'genetically', 'modified', 'food', 'in', '1998', ';', 'the', 'united', 'states', 'is', 'now', 'demanding', 'that', 'the', 'eu', 'end', 'its', 'ban', '.'], ['"', 'it', 'was', 'brazen', 'intimidation', 'to', 'keep', 'people', 'on', 'the', 'reservation', ',', '"', 'said', 'a', 'republican', 'who', 'attended', 'but', 'did', 'not', 'want', 'to', 'be', 'named', '.'], ['"', 'i', 'think', 'it', 'should', 'have', 'been', 'released', 'years', 'ago', ',', '"', 'said', 'brian', 'rohrbough', ',', 'whose', 'son', ',', 'daniel', ',', 'was', 'killed', 'at', 'columbine', '.'], ['chaudhry', 'was', 'released', '56', 'days', 'later', 'and', 'the', 'fiji', 'military', 'installed', 'an', 'all-indigenous', 'government', 'led', 'by', 'qarase', ',', 'who', 'won', 'democratic', 'elections', 'in', 'september', '2001', '.'], ['likewise', ',', 'the', '30-year', 'bond', '<', 'us30yt', '=', 'rr', '>', 'slid', '1-11', '/', '32', 'for', 'a', 'yield', 'of', '4.38', 'percent', ',', 'up', 'from', '4.30', 'percent', '.'], ['tech', 'stocks', 'were', 'hurt', 'by', 'a', 'sour', 'forecast', 'from', 'sun', 'microsystems', ',', 'which', 'was', 'viewed', 'as', 'a', 'bad', 'omen', 'for', 'the', 'upcoming', 'quarterly', 'earnings', 'season', '.'], ['"', 'we', 'do', 'not', 'want', 'to', 'stand', 'by', 'and', 'let', 'a', 'credit', 'card', 'lynching', 'take', 'place', ',', '"', 'declared', 'rep.', 'major', 'owens', '(', 'd-brooklyn', ')', '.'], ['the', 'festival', 'kicked', 'off', 'yesterday', 'one', 'day', 'after', 'the', 'competition', 'commission', 'delivered', 'its', 'final', 'verdict', 'to', 'the', 'government', 'on', 'the', 'proposed', '£', '4.1', 'billion', 'merger', '.'], ['miodrag', 'zivkovic', ',', 'of', 'the', 'liberal', 'alliance', 'of', 'montenegro', ',', 'won', '31', 'percent', 'of', 'the', 'vote', ',', 'while', 'the', 'independent', 'dragan', 'hajdukovic', 'got', '4', 'percent', '.'], ['"', 'it', "'s", 'a', 'huge', 'black', 'eye', ',', '"', 'said', 'publisher', 'arthur', 'ochs', 'sulzberger', 'jr', '.', ',', 'whose', 'family', 'has', 'controlled', 'the', 'paper', 'since', '1896', '.'], ['standing', 'with', 'reporters', 'and', 'photographers', 'about', '150', 'yards', 'from', 'the', 'prison', 'gates', 'was', 'brent', 'newbury', ',', 'president', 'of', 'the', 'rockland', 'county', 'patrolmen', "'s", 'benevolent', 'association', '.'], ['"', 'there', 'were', 'a', 'number', 'of', 'bureaucratic', 'and', 'administrative', 'missed', 'signals', '--', 'there', "'s", 'not', 'one', 'person', 'who', "'s", 'responsible', 'here', ',', '"', 'gehman', 'said', '.'], ['gilroy', 'police', 'and', 'fbi', 'agents', 'described', 'gehring', 'as', 'cooperative', ',', 'but', 'said', 'saturday', 'that', 'he', 'had', 'revealed', 'nothing', 'about', 'what', 'had', 'happened', 'to', 'the', 'children', '.'], ['u.s.', 'troops', 'killed', 'nearly', 'two', 'dozen', 'suspected', 'taliban', 'militants', 'after', 'coming', 'under', 'fire', 'in', 'southern', 'afghanistan', 'in', 'the', 'latest', 'in', 'a', 'series', 'of', 'such', 'attacks', '.'], ['but', 'the', 'fbi', 'never', 'kept', 'tabs', 'on', 'al-bayoumi', '-', 'even', 'though', 'it', 'had', 'received', 'information', 'that', 'he', 'was', 'a', 'saudi', 'agent', ',', 'the', 'document', 'says', '.'], ['that', 'compares', 'with', 'a', 'profit', 'of', '$', '1', 'million', ',', 'or', 'breakeven', ',', 'on', '$', '1.39', 'billion', 'in', 'revenue', 'during', 'the', 'same', 'period', 'last', 'year', '.'], ['while', 'the', 'ibrahims', 'had', 'one', 'separation', 'operation', ',', 'goodrich', 'and', 'dr.', 'david', 'staffenberg', 'plan', 'about', 'three', 'for', 'the', 'aguirres', ',', 'with', 'several', 'weeks', 'between', 'each', '.'], ['where', 'long-lines', 'used', 'to', 'catch', '10', 'fish', 'per', '100', 'hooks', 'in', 'past', 'decades', ',', 'they', 'are', 'now', 'lucky', 'to', 'catch', 'one', ',', 'the', 'study', 'found', '.'], ['the', 'dow', 'jones', 'industrial', 'average', 'was', 'up', '0.3', 'per', 'cent', 'at', '9,886.75', ',', 'while', 'the', 'nasdaq', 'composite', 'index', 'was', '0.4', 'per', 'cent', 'higher', 'at', '1,986.97', '.'], ['university', 'of', 'michigan', 'president', 'mary', 'sue', 'coleman', 'said', 'in', 'a', 'statement', 'on', 'the', 'university', "'s", 'web', 'site', ',', '"', 'our', 'fundamental', 'values', 'haven', "'t", 'changed', '.'], ['at', '5', 'p.m.', 'edt', ',', 'grace', "'s", 'center', 'was', 'near', 'latitude', '25.6', 'north', ',', 'longitude', '93.7', 'west', 'or', 'about', '280', 'miles', 'east-southeast', 'of', 'corpus', 'christi', '.'], ['the', 'technology-laced', 'nasdaq', 'composite', 'index', '.ixic', 'rose', '39.39', 'points', ',', 'or', '2.2', 'percent', ',', 'to', '1,826.33', ',', 'after', 'losing', 'more', 'than', '2', 'percent', 'on', 'tuesday', '.'], ['sirius', 'carries', 'national', 'public', 'radio', ',', 'although', 'it', 'doesn', "'t", 'include', 'popular', 'shows', 'such', 'as', '"', 'all', 'things', 'considered', '"', 'and', '"', 'morning', 'edition', '.', '"'], ['contrary', 'to', 'what', 'peoplesoft', 'management', 'would', 'have', 'you', 'believe', ',', 'oracle', 'intends', 'to', 'fully', 'support', 'peoplesoft', 'customers', 'and', 'products', 'for', 'many', 'years', 'to', 'come', '.', '"'], ['the', 'dollar', 'was', 'at', '116.92', 'yen', 'against', 'the', 'yen', ',', 'flat', 'on', 'the', 'session', ',', 'and', 'at', '1.2891', 'against', 'the', 'swiss', 'franc', ',', 'also', 'flat', '.'], ['police', 'spokesman', 'brig.', 'gen.', 'edward', 'aritonang', 'confirmed', 'saturday', 'that', 'another', 'two', 'had', 'been', 'arrested', ',', 'one', 'in', 'jakarta', 'and', 'another', 'in', 'magelang', ',', 'central', 'java', '.'], ['bolivia', "'s", 'army', 'fought', 'to', 'stop', 'columns', 'of', 'protesters', 'from', 'streaming', 'into', 'the', 'food-starved', 'capital', 'on', 'wednesday', 'as', 'a', 'popular', 'uprising', 'against', 'the', 'president', 'spread', '.'], ['"', 'but', 'i', 'do', 'question', 'the', 'motives', 'of', 'a', 'deskbound', 'president', 'who', 'assumes', 'the', 'garb', 'of', 'a', 'warrior', 'for', 'the', 'purposes', 'of', 'a', 'speech', '.', '"'], ['the', 'blue-chip', 'dow', 'jones', 'industrial', 'average', '.dji', 'fell', '86.56', 'points', ',', 'or', '0.94', 'percent', ',', 'to', '9,109.99', ',', 'after', 'giving', 'up', 'more', 'than', '1', 'percent', 'earlier', '.'], ['the', 'dow', 'jones', 'industrial', 'average', '<', '.dji', '>', 'added', '28', 'points', ',', 'or', '0.27', 'percent', ',', 'at', '10,557', ',', 'hitting', 'its', 'highest', 'level', 'in', '21', 'months', '.'], ['chief', 'merchandising', 'officers', 'oversee', 'the', 'buying', 'and', 'development', 'of', 'merchandise', ',', 'deciding', 'the', 'merchandising', 'direction', 'of', 'the', 'company', ',', 'or', 'basically', 'what', 'it', 'should', 'stand', 'for', '.'], ['but', 'mitsubishi', 'tokyo', 'financial', '(', 'jp', ':', '8306', ':', 'news', ',', 'chart', ',', 'profile', ')', 'declined', '3,000', 'yen', ',', 'or', '0.65', 'percent', ',', 'to', '456,000', 'yen', '.'], ['egyptologists', 'cast', 'doubt', 'tuesday', 'on', 'an', 'expedition', "'s", 'claim', 'that', 'it', 'may', 'have', 'found', 'the', 'mummy', 'of', 'queen', 'nefertiti', ',', 'one', 'of', 'the', 'best-known', 'ancient', 'egyptians', '.'], ['an', 'hour', 'later', ',', 'an', 'israeli', 'helicopter', 'fired', 'missiles', 'at', 'a', 'car', 'in', 'gaza', 'city', ',', 'killing', 'two', 'hamas', 'officials', 'and', 'at', 'least', 'five', 'other', 'people', '.'], ['and', 'now', 'it', "'s", 'anything', 'he', 'wants', 'to', 'say', ',', '"', 'alesha', 'badgley', ',', 'stone', 'county', 'nursing', 'and', 'rehabilitation', 'center', 'social', 'director', ',', 'said', 'this', 'week', '.'], ['electronic', 'data', 'systems', 'corp.', 'thursday', 'said', 'the', 'securities', 'and', 'exchange', 'commission', 'has', 'asked', 'the', 'company', 'for', 'documents', 'related', 'to', 'its', 'large', 'contract', 'with', 'the', 'u.s.', 'navy', '.'], ['iraqi', 'police', 'opened', 'fire', 'in', 'downtown', 'baghdad', 'today', 'after', 'demonstrators', 'stormed', 'a', 'police', 'station', 'demanding', 'they', 'be', 'given', 'jobs', 'they', 'claimed', 'to', 'have', 'paid', 'bribes', 'for', '.'], ['last', 'year', ',', 'cuban', 'officials', 'said', 'none', 'of', 'nearly', 'two', 'dozen', 'nominated', 'cuban', 'musicians', 'received', 'u.s.', 'visas', 'on', 'time', 'to', 'attend', 'the', 'grammys', 'in', 'los', 'angeles', '.'], ['u.s.', 'troops', 'encountered', 'no', 'resistance', 'during', 'the', 'five-hour', 'sweep', 'near', 'tikrit', ',', 'saddam', 'hussein', "'s", 'hometown', 'and', 'the', 'center', 'of', 'a', 'region', 'of', 'baath', 'party', 'supporters', '.'], ['microsoft', 'said', 'friday', 'that', 'it', 'is', 'halting', 'development', 'of', 'future', 'macintosh', 'versions', 'of', 'its', 'internet', 'explorer', 'browser', ',', 'citing', 'competition', 'from', 'apple', 'computer', "'s", 'safari', 'browser', '.'], ['downstream', 'at', 'mount', 'vernon', ',', 'the', 'skagit', 'river', 'was', 'expected', 'to', 'crest', 'at', '36', 'feet', '--', '8', 'feet', 'above', 'flood', 'stage', '--', 'tonight', ',', 'burke', 'said', '.'], ['british', 'police', 'arrested', '21', 'people', 'early', 'tuesday', 'in', 'connection', 'with', 'the', 'suspected', 'ritual', 'murder', 'of', 'an', 'african', 'boy', 'whose', 'torso', 'was', 'found', 'in', 'the', 'thames', 'river', '.'], ['in', 'turn', 'suse', 'will', 'license', 'java', '2', 'standard', 'edition', 'while', 'also', 'formalising', 'an', 'existing', 'agreement', 'to', 'distribute', 'sun', "'s", 'java', 'virtual', 'machine', 'for', 'running', 'java', 'applications', '.'], ['but', 'the', 'prime', 'minister', 'told', 'the', 'bbc', 'he', 'wasn', "'t", 'bothered', 'by', 'polls', 'and', 'that', 'he', 'would', 'continue', 'to', 'do', 'what', 'he', 'believed', 'to', 'be', 'right', '.'], ['the', 'best', 'the', 'investigators', 'can', 'do', 'is', 'nitpick', 'about', 'the', 'process', 'and', 'substance', 'of', 'isolated', 'business', 'decisions', '...', 'and', 'question', 'his', 'competence', 'as', 'a', 'manager', '.', '"'], ['but', 'white', 'house', 'spokesman', 'ari', 'fleischer', 'said', 'yesterday', ':', '"', 'the', 'steps', 'the', 'iranians', 'claim', 'to', 'have', 'taken', 'in', 'terms', 'of', 'capturing', 'al-qa', "'ida", 'are', 'insufficient', '.', '"'], ['taylor', "'s", 'aides', 'warn', 'an', 'abrupt', 'departure', 'could', 'trigger', 'more', 'chaos', ',', 'a', 'view', 'shared', 'by', 'many', 'in', 'liberia', 'and', ',', 'in', 'private', ',', 'even', 'by', 'mediators', '.'], ['executive', 'recruiters', 'say', 'that', 'finding', 'a', 'seasoned', 'chief', 'merchandising', 'officer', 'will', 'perhaps', 'be', 'the', 'trickiest', 'of', 'all', 'hires', 'because', 'there', 'is', 'such', 'a', 'dearth', 'of', 'good', 'ones', '.'], ['one', 'question', 'was', 'whether', 'france', ',', 'which', 'infuriated', 'washington', 'by', 'leading', 'the', 'charge', 'against', 'u.n.', 'authorization', 'for', 'the', 'war', ',', 'would', 'vote', '"', 'yes', '"', 'or', 'abstain', '.'], ['proving', 'that', 'the', 'millville', 'son', "'s", 'sacrifice', 'would', 'not', 'go', 'unsung', ',', 'mayor', 'james', 'quinn', 'ordered', 'all', 'city', 'flags', 'flown', 'at', 'half-mast', 'for', 'the', 'next', '30', 'days', '.'], ['"', 'but', 'that', 'does', 'not', 'clear', 'them', 'of', 'the', 'obligation', 'to', 'do', 'everything', 'possible', 'to', 'protect', 'civilians', ',', 'and', 'that', 'is', 'not', 'what', 'we', "'re", 'seeing', '.', '"'], ['instead', ',', 'weida', 'decided', 'that', 'meester', "'s", 'case', 'on', 'charges', 'of', 'rape', ',', 'sodomy', ',', 'indecent', 'assault', 'and', 'providing', 'alcohol', 'to', 'minors', 'should', 'proceed', 'to', 'a', 'court-martial', '.'], ['"', 'i', '´', 'm', 'very', 'proud', 'of', 'the', 'citizens', 'of', 'this', 'state', ',', '"', 'gov.', 'john', 'baldacci', 'said', 'after', 'votes', 'from', 'tuesday', '´', 's', 'referendum', 'were', 'counted', '.'], ['it', 'got', 'up', 'on', 'his', 'personal', 'radar', 'screen', 'this', 'past', 'week', ',', 'i', "'ve", 'given', 'him', 'my', 'pitch', 'and', 'he', 'was', 'quite', 'supportive', ',', '"', 'he', 'said', '.'], ['as', 'well', 'as', 'the', 'dolphin', 'scheme', ',', 'the', 'chaos', 'has', 'allowed', 'foreign', 'companies', 'to', 'engage', 'in', 'damaging', 'logging', 'and', 'fishing', 'operations', 'without', 'proper', 'monitoring', 'or', 'export', 'controls', '.'], ['in', 'the', 'year-ago', 'quarter', ',', 'the', 'steelmaker', 'recorded', 'a', 'profit', 'of', '$', '16.2', 'million', ',', 'or', '15', 'cents', 'per', 'share', ',', 'on', 'sales', 'of', '$', '1.14', 'billion', '.'], ['at', 'one', 'of', 'the', 'three', 'sampling', 'sites', 'at', 'huntington', 'beach', ',', 'the', 'bacteria', 'reading', 'came', 'back', 'at', '160', 'on', 'june', '16', 'and', 'at', '120', 'on', 'june', '23', '.'], ['tehran', 'has', 'made', 'clear', 'it', 'will', 'only', 'sign', 'the', 'additional', 'protocol', 'introducing', 'such', 'inspections', 'if', 'a', 'ban', 'on', 'the', 'import', 'of', 'civilian', 'western', 'nuclear', 'technology', 'is', 'lifted', '.'], ['the', 'sia', 'says', 'the', 'dram', 'market', 'is', 'expected', 'to', 'grow', '2.9', 'percent', 'to', '$', '15.7', 'billion', 'in', '2003', 'and', '43', 'percent', 'to', '$', '22.5', 'billion', 'in', '2004', '.'], ['shares', 'of', 'san', 'diego-based', 'jack', 'in', 'the', 'box', 'closed', 'at', '$', '21.49', ',', 'up', '78', 'cents', ',', 'or', '3.8', 'percent', ',', 'on', 'the', 'new', 'york', 'stock', 'exchange', '.'], ['the', 'first', 'trial', 'of', 'a', 'suspect', 'in', 'last', 'year', "'s", 'bali', 'nightclub', 'bombings', 'that', 'killed', 'more', 'than', '200', 'people', 'opened', 'in', 'indonesia', 'today', ',', 'sky', 'news', 'reported', '.'], ['calvin', 'hollins', 'jr', '.', "'", 's', 'attorney', ',', 'thomas', 'royce', ',', 'has', 'repeatedly', 'said', 'his', 'client', 'had', 'no', 'link', 'with', 'the', 'company', 'that', 'owned', 'and', 'operated', 'e2', '.'], ['results', 'from', 'no.', '2', 'u.s.', 'soft', 'drink', 'maker', 'pepsico', 'inc', '.', '(', 'nyse', ':', 'pep', '-', 'news', '-', 'people', ')', 'were', 'likely', 'to', 'be', 'in', 'the', 'spotlight', '.'], ['at', 'first', 'blush', ',', 'then', ',', 'the', 'distinction', 'drawn', 'by', 'the', 'creators', 'of', 'the', 'do-not-call', 'registry', 'would', 'seem', 'to', 'draw', 'the', 'line', 'in', 'precisely', 'the', 'right', 'place', '.'], ['a', 'new', 'castle', 'county', 'woman', 'has', 'become', 'the', 'first', 'delaware', 'patient', 'to', 'contract', 'the', 'west', 'nile', 'virus', 'this', 'year', ',', 'the', 'state', "'s", 'department', 'of', 'health', 'reported', '.'], ['a', 'passer-by', 'found', 'ben', 'hiding', 'along', 'a', 'dirt', 'road', 'in', 'spanish', 'fork', 'canyon', 'about', '7', 'p.m.', ',', 'with', 'his', 'hands', 'still', 'taped', 'together', 'but', 'his', 'feet', 'free', '.'], ['frank', 'quattrone', ',', 'the', 'former', 'credit', 'suisse', 'first', 'boston', 'technology', 'investment-banking', 'guru', ',', 'reportedly', 'pleaded', 'not', 'guilty', 'tuesday', 'to', 'charges', 'of', 'obstruction', 'of', 'justice', 'and', 'witness', 'tampering', '.'], ['microsoft', 'has', 'been', 'awarded', 'a', 'patent', 'for', 'a', 'feature', 'in', 'instant', 'messaging', 'that', 'alerts', 'a', 'user', 'when', 'the', 'person', 'they', 'are', 'communicating', 'with', 'is', 'inputting', 'a', 'message', '.'], ['terri', 'schiavo', ',', '39', ',', 'is', 'expected', 'to', 'die', 'sometime', 'in', 'the', 'next', 'two', 'weeks', 'in', 'the', 'tampa-area', 'hospice', 'where', 'she', 'has', 'spent', 'the', 'past', 'several', 'years', '.'], ['and', 'yes', ',', 'marilyn', 'monroe', 'is', 'definitely', 'part', 'of', 'the', 'story', ',', 'titled', '"', 'arthur', 'miller', ',', 'elia', 'kazan', 'and', 'the', 'blacklist', ':', 'none', 'without', 'sin', '.', '"'], ['"', 'these', 'despicable', 'acts', 'were', 'committed', 'by', 'killers', 'whose', 'only', 'faith', 'is', 'hate', ',', '"', 'bush', 'said', ',', 'adding', 'that', 'the', 'united', 'states', 'will', 'find', 'the', 'killers', '.'], ['bremer', 'said', 'one', 'initiative', 'is', 'to', 'launch', 'a', 'us', '$', '70', 'million', 'nationwide', 'program', 'in', 'the', 'next', 'two', 'weeks', 'to', 'clean', 'up', 'neighborhoods', 'and', 'build', 'community', 'projects', '.'], ['actor', 'noah', 'wyle', ',', 'who', 'has', 'played', 'dr.', 'john', 'carter', 'since', 'the', 'shows', 'inception', 'in', '1994', ',', 'extended', 'his', 'contract', 'with', 'the', 'show', 'through', 'the', '2004-2005', 'season', '.'], ['the', 'bank', 'also', 'said', 'its', 'offer', 'was', 'subject', 'to', 'the', 'agreement', 'of', 'drax', "'s", 'senior', 'banks', ',', 'senior', 'bond', 'holders', 'and', 'hedging', 'banks', 'by', '30', 'september', '2003', '.'], ['the', 'results', 'were', 'released', 'at', 'tuesday', "'s", 'meeting', 'in', 'seattle', 'of', 'the', 'american', 'thoracic', 'society', 'and', 'will', 'be', 'published', 'in', 'thursday', "'s", 'new', 'england', 'journal', 'of', 'medicine', '.'], ['"', 'i', 'started', 'crying', 'and', 'yelling', 'at', 'him', ',', "'", 'what', 'do', 'you', 'mean', ',', 'what', 'are', 'you', 'saying', ',', 'why', 'did', 'you', 'lie', 'to', 'me', '?', "'", '"'], ['us', 'special', 'forces', 'troops', 'are', 'training', 'colombian', 'soldiers', 'at', 'military', 'bases', 'in', 'the', 'region', 'to', 'protect', 'the', 'pipeline', ',', 'which', 'transports', 'crude', 'from', 'colombia', "'s", 'second-biggest', 'oil', 'field', '.'], ['a', 'key', 'question', 'was', 'whether', 'france', ',', 'which', 'infuriated', 'washington', 'by', 'leading', 'the', 'charge', 'against', 'u.n.', 'authorization', 'for', 'the', 'war', ',', 'would', 'vote', '"', 'yes', '"', 'or', 'abstain', '.'], ['government', 'bonds', 'sold', 'off', 'sharply', 'after', 'greenspan', 'told', 'congress', 'on', 'tuesday', 'that', 'the', 'u.s.', 'economy', '"', 'could', 'very', 'well', 'be', 'embarking', 'on', 'a', 'period', 'of', 'extended', 'growth', '"', '.'], ['while', 'opposition', 'parties', 'have', 'welcomed', 'the', 'cabinet', "'s", 'decision', 'on', 'anti-retroviral', 'treatment', ',', 'some', 'said', 'health', 'minister', 'manto', 'tshabalala-msimang', 'was', 'not', 'fit', 'to', 'preside', 'over', 'a', 'rollout', 'plan', '.'], ['appellate', 'courts', 'across', 'the', 'country', 'have', 'issued', 'differing', 'rulings', 'on', 'the', 'issue', ',', 'allowing', 'public', 'displays', 'of', 'the', 'ten', 'commandments', 'in', 'some', 'cases', 'and', 'banning', 'them', 'in', 'others', '.'], ['but', 'in', 'the', 'end', ',', 'all', 'the', 'worm', 'did', 'was', 'visit', 'a', 'pornography', 'site', ',', 'said', 'vincent', 'weafer', ',', 'a', 'security', 'director', 'with', 'symantec', 'security', 'response', 'in', 'california', '.'], ['the', 'losses', 'occurred', 'overnight', 'in', 'willow', 'canyon', ',', 'one', 'of', 'three', 'areas', 'in', 'the', 'santa', 'catalina', 'mountains', 'threatened', 'by', 'the', '2-week-old', 'fire', 'that', 'already', 'had', 'blackened', '70,000', 'acres', '.'], ['a', 'three-judge', 'panel', 'of', 'the', '11th', 'u.s.', 'circuit', 'court', 'of', 'appeals', 'ruled', 'late', 'friday', 'that', 'the', 'small', 'size', ',', 'limited', 'use', 'and', 'secular', 'purpose', 'make', 'the', 'image', 'legitimate', '.'], ['fbi', 'agents', 'arrested', 'a', 'former', 'partner', 'of', 'big', 'four', 'accounting', 'firm', 'ernst', '&', 'young', 'erny.ul', 'on', 'criminal', 'charges', 'of', 'obstructing', 'federal', 'investigations', ',', 'u.s.', 'officials', 'said', 'on', 'thursday', '.'], ['in', 'afternoon', 'trading', 'in', 'europe', ',', 'france', "'s", 'cac-40', 'advanced', 'and', 'britain', "'s", 'ftse', '100', 'each', 'gained', '0.7', 'percent', ',', 'while', 'germany', "'s", 'dax', 'index', 'rose', '0.6', 'percent', '.'], ['"', 'i', 'feel', 'that', 'people', 'that', 'are', 'sending', 'these', 'messages', 'are', 'infringing', 'on', 'my', 'rights', 'and', 'everyone', 'else', "'s", 'rights', 'to', 'use', 'their', 'computers', ',', '"', 'said', 'mckechnie', '.'], ['i', "'ve", 'still', 'got', 'a', 'fighting', 'chance', ',', 'though', ',', '"', 'hewitt', 'said', 'after', 'battling', 'to', 'overcome', 'davydenko', '6-3', '4-6', '6-3', '7-6', '(', '7-5', ')', 'on', 'centre', 'court', '.'], ['pines', 'said', 'he', 'would', 'convene', 'the', 'relevant', 'party', 'bodies', 'within', '10', 'days', 'to', 'discuss', 'whether', 'new', 'elections', 'would', 'be', 'held', 'or', 'whether', 'a', 'temporary', 'leader', 'would', 'be', 'appointed', '.'], ['kyi', ',', 'a', 'u.n.', 'envoy', 'says', ',', 'as', 'japan', 'adds', 'to', 'growing', 'international', 'pressure', 'by', 'saying', 'it', 'will', 'halt', 'its', 'hefty', 'economic', 'aid', 'unless', 'suu', 'kyi', 'is', 'freed', '.'], ['gold', "'s", 'rise', 'pulled', 'other', 'precious', 'metals', 'higher', ',', 'shunting', 'silver', 'to', '$', '4.81', '/', '83', 'an', 'ounce', 'from', '$', '4.77', '/', '79', 'last', 'quoted', 'in', 'new', 'york', '.'], ['chechen', 'officials', 'working', 'for', 'the', 'moscow-backed', 'government', 'are', 'a', 'frequent', 'target', 'for', 'rebels', 'and', 'tension', 'is', 'running', 'high', 'ahead', 'of', 'next', 'sunday', "'s", 'presidential', 'election', 'in', 'war-torn', 'chechnya', '.'], ['further', 'out', 'the', 'curve', ',', 'the', 'benchmark', '10-year', 'note', 'us10yt', '=', 'rr', 'shed', '25', '/', '32', 'in', 'price', ',', 'taking', 'its', 'yield', 'to', '3.26', 'percent', 'from', '3.17', 'percent', '.'], ['bush', 'wanted', '"', 'to', 'see', 'an', 'aircraft', 'landing', 'the', 'same', 'way', 'that', 'the', 'pilots', 'saw', 'an', 'aircraft', 'landing', ',', '"', 'white', 'house', 'press', 'secretary', 'ari', 'fleischer', 'said', 'yesterday', '.'], ['a', 'statement', 'released', 'by', 'dei', 'said', 'green', 'would', 'begin', 'driving', 'the', 'car', 'on', 'an', 'interim', 'basis', 'beginning', 'next', 'week', 'in', 'the', 'winston', 'open', 'at', 'lowe', "'s", 'motor', 'speedway', '.'], ['hormone', 'replacement', 'therapy', 'that', 'combines', 'estrogen', 'and', 'progestin', 'doubles', 'a', 'woman', "'s", 'risk', 'of', 'breast', 'cancer', ',', 'a', 'british', 'study', 'of', 'more', 'than', 'a', 'million', 'women', 'has', 'found', '.'], ['but', 'adora', 'obi', 'nweze', ',', 'the', 'naacp', 'state', 'president', ',', 'said', 'the', 'state', 'only', 'tried', 'to', 'prove', 'its', 'conclusion', 'of', 'suicide', ',', 'rather', 'than', 'consider', 'the', 'possibility', 'of', 'murder', '.'], ['mel', 'gibson', "'s", 'passion-stirring', 'biblical', 'epic', '"', 'the', 'passion', 'of', 'christ', '"', 'will', 'open', 'in', 'the', 'united', 'states', 'on', 'feb.', '25', '-', 'ash', 'wednesday', 'on', 'the', 'roman', 'catholic', 'calendar', '.'], ['long', 'lines', 'formed', 'outside', 'gas', 'stations', 'and', 'people', 'rushed', 'to', 'get', 'money', 'from', 'cash', 'machines', 'sunday', 'as', 'israelis', 'prepared', 'to', 'weather', 'a', 'strike', 'that', 'threatened', 'to', 'paralyze', 'the', 'country', '.'], ['the', 'epidemic', 'began', 'in', 'november', 'in', 'the', 'mainland', "'s", 'guangdong', 'province', ',', 'but', 'the', 'people', "'s", 'republic', 'of', 'china', 'refused', 'to', 'report', 'truthfully', 'about', 'its', 'spread', 'for', 'four', 'months', '.'], ['in', 'an', 'interview', ',', 'healey', ',', 'who', 'is', 'a', 'criminologist', ',', 'acknowledged', 'that', 'much', 'of', 'the', 'sentiment', 'among', 'legislators', 'here', 'and', 'across', 'the', 'country', 'was', 'wariness', 'toward', 'capital', 'punishment', '.'], ['moffitt', 'said', 'the', 'results', 'need', 'to', 'be', 'replicated', 'in', 'another', 'study', 'before', 'testing', 'of', 'individuals', 'for', 'presence', 'of', 'the', 'long', 'or', 'short', 'versions', 'of', 'the', 'gene', 'will', 'be', 'pursued', '.'], ['eu', 'ministers', 'were', 'invited', 'to', 'the', 'conference', 'but', 'canceled', 'because', 'the', 'union', 'is', 'closing', 'talks', 'on', 'agricultural', 'reform', ',', 'said', 'gerry', 'kiely', ',', 'a', 'eu', 'agriculture', 'representative', 'in', 'washington', '.'], ['the', 'report', 'cites', 'the', 'marriage', 'of', 'the', 'alleged', 'operational', 'commander', 'of', 'the', 'bali', 'bombings', ',', 'mukhlas', ',', 'to', 'farida', ',', 'a', '"', 'highly', 'cosmopolitan', 'and', 'well-educated', 'young', 'woman', '"', '.'], ['the', 'euro', '<', 'eur', '=', '>', 'was', 'up', '0.67', 'percent', 'against', 'the', 'dollar', 'at', '$', '1.1784', 'after', 'rising', 'to', 'a', 'three-month', 'high', 'earlier', 'in', 'the', 'session', 'above', '$', '1.18', '.'], ['tenet', 'has', 'been', 'under', 'scrutiny', 'since', 'november', ',', 'when', 'former', 'chief', 'executive', 'jeffrey', 'barbakow', 'said', 'the', 'company', 'used', 'aggressive', 'pricing', 'to', 'trigger', 'higher', 'payments', 'for', 'the', 'sickest', 'medicare', 'patients', '.'], ['peter', 'lyman', 'and', 'hal', 'varian', 'of', 'berkeley', "'s", 'school', 'of', 'information', 'management', 'and', 'systems', 'say', 'that', 'information', 'production', 'has', 'increased', 'by', '30', 'percent', 'each', 'year', 'between', '1999', 'and', '2002', '.'], ['spot', 'gold', 'was', 'quoted', 'at', '$', '367.90', '/', '368.60', 'an', 'ounce', 'at', '1000', 'gmt', ',', 'having', 'marched', 'up', 'to', '$', '369.50', '--', 'a', 'level', 'not', 'seen', 'since', 'february', '10', '.'], ['blue', 'chips', 'edged', 'lower', 'by', 'midday', 'on', 'wednesday', 'as', 'a', 'wave', 'of', 'profit', 'warnings', 'drained', 'some', 'of', 'the', 'optimism', 'that', 'has', 'driven', 'a', 'rally', 'over', 'the', 'past', 'three', 'months', '.'], ['the', 'priest', ',', 'the', 'rev.', 'john', 'f.', 'johnston', ',', '64', ',', 'of', '35th', 'avenue', 'in', 'jackson', 'heights', ',', 'was', 'charged', 'with', 'aggravated', 'harassment', 'and', 'criminal', 'possession', 'of', 'a', 'weapon', '.'], ['the', 'bodies', 'of', 'former', 'invercargill', 'man', 'warren', 'campbell', ',', '32', ',', 'and', 'climbing', 'companion', 'jonathan', 'smith', ',', '47', ',', 'were', 'recovered', 'from', 'below', 'mt', 'd', "'archiac", 'late', 'yesterday', 'morning', '.'], ['messagelabs', ',', 'which', 'runs', 'outsourced', 'e-mail', 'servers', 'for', '700,000', 'customers', 'around', 'the', 'world', ',', 'said', 'it', 'had', 'filtered', 'out', '27,000', 'infected', 'e-mails', 'in', '115', 'countries', 'as', 'of', 'thursday', 'morning', '.'], ['country-music', 'station', 'kkcs', 'has', 'suspended', 'two', 'disc', 'jockeys', 'for', 'playing', 'songs', 'by', 'the', 'dixie', 'chicks', 'in', 'violation', 'of', 'a', 'ban', 'imposed', 'after', 'one', 'group', 'member', 'criticized', 'president', 'george', 'bush', '.'], ['the', 'federal', 'and', 'local', 'governments', 'remained', 'largely', 'shut', 'down', 'in', 'the', 'aftermath', 'of', 'hurricane', 'isabel', ',', 'as', 'were', 'schools', ',', 'and', 'much', 'of', 'the', 'region', "'s", 'transportation', 'network', 'was', 'slowed', '.'], ['but', 'he', 'is', 'the', 'only', 'candidate', 'who', 'has', 'won', 'national', 'labor', 'endorsements', 'so', 'far', ',', 'picking', 'up', 'his', '11th', 'union', 'on', 'tuesday', 'when', 'the', 'united', 'steelworkers', 'of', 'america', 'endorsed', 'him', '.'], ['authorities', 'appealed', 'to', 'the', 'public', 'friday', 'to', 'come', 'forward', 'and', 'provide', 'what', 'could', 'be', 'the', 'key', 'to', 'solving', 'the', 'sniper-style', 'shootings', 'that', 'have', 'killed', 'three', 'people', 'outside', 'area', 'convenience', 'stores', '.'], ['tampa', 'bay', 'manager', 'lou', 'piniella', ',', 'bench', 'coach', 'john', 'mclaren', 'and', 'right', 'fielder', 'aubrey', 'huff', 'were', 'ejected', 'for', 'arguing', 'after', 'huff', 'was', 'called', 'out', 'on', 'strikes', 'to', 'end', 'the', 'ninth', '.'], ['on', 'the', 'upside', ',', 'coca-cola', 'co', '.', '(', 'nyse', ':', 'ko', '-', 'news', '-', 'people', ')', 'reported', 'higher', 'profit', 'for', 'the', 'quarter', 'early', 'on', 'thursday', ',', 'helped', 'by', 'european', 'demand', '.'], ['shares', 'of', 'usa', 'interactive', 'rose', '$', '2.28', ',', 'or', '7', 'percent', ',', 'to', '$', '34.96', 'on', 'friday', 'in', 'nasdaq', 'stock', 'market', 'composite', 'trading', 'and', 'have', 'gained', '53', 'percent', 'this', 'year', '.'], ['a', 'coalition', 'of', 'campaign', 'groups', ',', 'including', 'the', 'militant', 'national', 'students', 'body', ',', 'is', 'backing', 'the', 'protests', ',', 'which', 'the', 'nlc', 'called', 'in', 'defiance', 'of', 'a', 'court', 'order', 'banning', 'them', '.'], ['boston', 'the', 'catholic', 'archdiocese', 'in', 'boston', 'has', 'offered', '$', '55', 'million', 'to', 'settle', 'more', 'than', '500', 'clergy', 'sex', 'abuse', 'lawsuits', ',', 'according', 'to', 'a', 'document', 'obtained', 'by', 'the', 'associated', 'press', '.'], ['u.s.', 'corporate', 'bond', 'yield', 'spreads', 'tightened', 'in', 'spotty', 'trading', 'on', 'friday', 'as', 'wall', 'street', 'labored', 'to', 'get', 'back', 'on', 'its', 'feet', 'after', 'the', 'largest', 'power', 'outage', 'ever', 'in', 'north', 'america', '.'], ['"', 'generally', 'unwanted', '-', 'and', 'often', 'pornographic', 'or', 'with', 'fraudulent', 'intent', '-', 'spam', 'is', 'a', 'nuisance', 'and', 'a', 'distraction', ',', '"', 'gates', 'writes', ',', 'warming', 'us', 'up', 'for', 'the', 'punchline', '.'], ['the', 'redesigned', 'finder', 'also', 'features', 'search', ',', 'coloured', 'labels', 'for', 'customized', 'organization', 'of', 'documents', 'and', 'projects', 'and', 'dynamic', 'browsing', 'of', 'the', 'network', 'for', 'mac', ',', 'windows', 'and', 'unix', 'file', 'servers', '.'], ['labor', 'department', 'analysts', 'think', 'the', 'payroll', 'statistics', 'from', 'the', 'survey', 'of', 'businesses', 'provide', 'a', 'more', 'accurate', 'picture', 'of', 'the', 'economy', 'because', 'the', 'survey', 'figures', 'are', 'based', 'on', 'a', 'larger', 'sample', '.'], ['tom', 'kraynak', ',', 'manager', 'of', 'operations', 'and', 'resources', 'for', 'the', 'canton', ',', 'ohio-based', 'east', 'central', 'area', 'reliability', 'council', ',', 'said', 'that', 'scenario', 'is', 'one', 'among', 'many', 'that', 'investigators', 'are', 'considering', '.'], ['earlier', ',', 'they', 'had', 'defied', 'a', 'police', 'order', 'and', 'cried', '"', 'allahu', 'akbar', '"', '(', 'god', 'is', 'greatest', ')', 'as', 'bashir', 'walked', 'to', 'his', 'seat', 'in', 'the', 'tightly', 'guarded', 'courtroom', '.'], ['"', 'biotech', 'products', ',', 'if', 'anything', ',', 'may', 'be', 'safer', 'than', 'conventional', 'products', 'because', 'of', 'all', 'the', 'testing', ',', '"', 'fraley', 'said', ',', 'adding', 'that', '18', 'countries', 'have', 'adopted', 'biotechnology', '.'], ['maddox', ',', '87', ',', 'cracked', 'two', 'ribs', 'when', 'he', 'fell', 'about', '10', 'days', 'ago', 'at', 'an', 'assisted', 'living', 'home', 'where', 'he', 'was', 'recovering', 'from', 'intestinal', 'surgery', ',', 'virginia', 'carnes', 'said', '.'], ['a', 'federal', 'appeals', 'court', 'ruled', 'tuesday', 'that', 'the', 'government', 'was', 'free', 'to', 'withhold', 'the', 'names', 'of', 'more', 'than', '700', 'people', 'detained', 'in', 'the', 'aftermath', 'of', 'sept', '.', '11', ',', '2001', '.'], ['the', 'world', "'s", 'largest', 'software', 'company', 'said', 'it', 'recognized', 'the', 'difficulty', 'the', 'multiple', 'patches', 'posed', 'for', 'companies', ',', 'and', 'set', 'out', 'to', 'make', 'it', 'easier', 'for', 'them', 'to', 'apply', 'the', 'updates', '.'], ['the', 'company', 'will', 'launch', '800', 'hot', 'spots', ',', 'or', '"', 'wi-fi', 'zones', ',', '"', 'later', 'this', 'summer', ',', 'and', 'plans', 'to', 'have', 'more', 'than', '2,100', 'by', 'the', 'end', 'of', 'the', 'year', '.'], ['indonesia', "'s", 'army', 'has', 'often', 'been', 'accused', 'of', 'human', 'rights', 'abuses', 'during', 'gam', "'s", 'battle', 'for', 'independence', ',', 'charges', 'it', 'has', 'generally', 'denied', 'while', 'accusing', 'the', 'separatists', 'of', 'committing', 'rights', 'violations', '.'], ['iraq', "'s", 'economy', 'was', 'ravaged', 'during', 'years', 'of', 'u.n.', 'sanctions', 'over', 'ex-president', 'saddam', 'hussein', "'s", '1990', 'occupation', 'of', 'kuwait', 'and', 'by', 'the', 'u.s.-led', 'war', 'to', 'oust', 'him', 'which', 'ended', 'in', 'april', '.'], ['"', 'this', 'moves', 'us', 'a', 'lot', 'closer', 'to', 'saying', 'that', 'the', 'foam', 'can', 'do', 'this', 'kind', 'of', 'damage', ',', '"', 'said', 'hubbard', ',', 'a', 'member', 'of', 'the', 'columbia', 'accident', 'investigation', 'board', '.'], ['in', '1999', ',', 'the', 'building', "'s", 'owners', ',', 'the', 'port', 'authority', 'of', 'new', 'york', 'and', 'new', 'jersey', ',', 'issued', 'guidelines', 'to', 'upgrade', 'the', 'fireproofing', 'to', 'a', 'thickness', 'of', '1', '{', 'inches', '.'], ['now', ',', 'blanca', "'s", 'american', 'husband', ',', '63-year-old', 'roger', 'lawrence', 'strunk', ',', 'faces', 'a', 'murder', 'indictment', 'issued', 'in', 'february', 'by', 'the', 'philippine', 'government', ',', 'which', 'says', 'he', "'s", 'the', 'leading', 'suspect', '.'], ['byrd', ',', 'speaking', 'from', 'the', 'senate', 'floor', ',', 'said', 'he', 'questioned', '"', 'the', 'motives', 'of', 'a', 'deskbound', 'president', 'who', 'assumes', 'the', 'garb', 'of', 'a', 'warrior', 'for', 'purposes', 'of', 'a', 'speech', '.', '"'], ['wal-mart', ',', 'kohl', "'s", 'corp.', ',', 'family', 'dollar', 'stores', 'inc', '.', ',', 'and', 'big', 'lots', 'inc.', 'were', 'among', 'the', 'merchants', 'posting', 'may', 'sales', 'that', 'fell', 'below', 'wall', 'street', "'s", 'modest', 'expectations', '.'], ['the', 'standard', '&', 'poor', "'s", '500', 'stock', 'index', 'ended', 'the', 'quarter', 'up', '120', 'points', ',', 'a', 'gain', 'of', '14', 'percent', ',', 'the', 'best', 'performance', 'for', 'that', 'broad', 'market', 'benchmark', 'since', '1998', '.'], ['president', 'bush', 'raised', 'a', 'record-breaking', '$', '49.5', 'million', 'for', 'his', 're-election', 'campaign', 'over', 'the', 'last', 'three', 'months', ',', 'with', 'contributions', 'from', '262,000', 'americans', ',', 'the', 'president', "'s", 'campaign', 'chairman', 'said', 'tuesday', '.'], ['myanmar', "'s", 'pro-democracy', 'leader', 'aung', 'san', 'suu', 'kyi', 'will', 'return', 'home', 'late', 'friday', 'but', 'will', 'remain', 'in', 'detention', 'after', 'recovering', 'from', 'surgery', 'at', 'a', 'yangon', 'hospital', ',', 'her', 'personal', 'physician', 'said', '.'], ['sean', 'harrigan', ',', 'president', 'of', 'the', 'california', 'public', 'employees', "'", 'retirement', 'system', ',', 'also', 'suggested', 'paring', 'down', 'the', 'nyse', "'s", '27-member', 'board', ',', 'changing', 'its', 'composition', 'and', 'making', 'it', 'more', 'accountable', '.'], ['abbas', 'told', 'the', 'summit', 'he', 'would', 'end', 'the', '"', 'armed', 'intefadeh', ',', '"', 'renounced', '"', 'terrorism', 'against', 'the', 'israelis', 'wherever', 'they', 'might', 'be', '"', 'and', 'alluded', 'to', 'the', 'disarming', 'of', 'militants', '.'], ['their', 'contract', 'will', 'expire', 'at', '12', ':', '01', 'a.m.', 'wednesday', 'instead', 'of', '12', ':', '01', 'a.m.', 'sunday', ',', 'said', 'rian', 'wathen', ',', 'organizing', 'director', 'for', 'united', 'food', 'and', 'commercial', 'workers', 'local', '700', '.'], ['"', 'i', 'don', "'t", 'know', 'if', 'the', 'person', 'i', "'m", 'talking', 'to', 'now', 'may', 'end', 'up', 'being', 'someone', 'else', 'at', 'another', 'time', 'that', 'may', 'not', 'follow', 'the', 'rules', ',', '"', 'parrish', 'said', '.'], ['by', 'late', 'afternoon', ',', 'the', 'dow', 'jones', 'industrial', 'average', 'was', 'up', '12.81', ',', 'or', '0.1', 'percent', ',', 'at', '9,331.77', ',', 'having', 'gained', '201', 'points', 'monday', 'to', 'its', 'highest', 'level', 'since', 'july', '2002', '.'], ['"', 'we', 'continue', 'to', 'work', 'with', 'the', 'saudis', 'on', 'this', ',', 'but', 'they', 'did', 'not', ',', 'as', 'of', 'the', 'time', 'of', 'this', 'tragic', 'event', ',', 'provide', 'the', 'additional', 'security', 'we', 'requested', '.', '"'], ['"', 'we', 'believe', 'we', 'are', 'fully', 'prepared', 'to', 'roll', 'out', 'the', '[', 'touch-screen', ']', 'machines', 'for', 'the', '2004', 'presidential', 'primary', ',', '"', 'said', 'gilles', 'w.', 'burger', ',', 'state', 'board', 'of', 'elections', 'chairman', '.'], ['"', 'we', 'haven', "'t", 'decided', 'whether', 'to', 'use', 'an', 'unbroken', 'or', 'dotted', 'white', 'line', ',', 'but', 'this', 'is', 'an', 'experiment', 'worth', 'introducing', ',', '"', 'said', 'david', 'richardson', ',', 'the', 'icc', "'s", 'cricket', 'manager', '.'], ['"', 'the', 'discovery', 'that', 'the', 'map', 'bug', 'is', 'present', 'in', 'the', 'vast', 'majority', 'of', 'crohn', "'s", 'sufferers', 'means', 'it', 'is', 'almost', 'certainly', 'causing', 'the', 'intestinal', 'inflammation', ',', '"', 'it', 'said', 'in', 'a', 'statement', '.'], ['the', 'island', 'nation', 'reported', '65', 'new', 'cases', 'on', 'may', '22', ',', 'a', 'one', 'day', 'record', ',', 'and', '55', 'new', 'cases', 'on', 'may', '23', ',', 'making', 'taiwan', "'s", 'epidemic', 'the', 'fastest-growing', 'in', 'the', 'world', '.'], ['the', 'justice', 'department', 'filed', 'suit', 'thursday', 'against', 'the', 'state', 'of', 'mississippi', 'for', 'failing', 'to', 'end', 'what', 'federal', 'officials', 'call', '"', 'disturbing', '"', 'abuse', 'of', 'juveniles', 'and', '"', 'unconscionable', '"', 'conditions', 'at', 'two', 'state-run', 'facilities', '.'], ['moore', 'did', 'not', 'have', 'an', 'immediate', 'response', 'tuesday', '.'], ['msnbc.com', 'first', 'reported', 'the', 'cia', 'request', 'on', 'friday', '.'], ['conte', 'said', 'druce', 'will', 'be', 'charged', 'with', 'murder', '.'], ['in', 'response', 'to', 'sluggish', 'sales', ',', 'cisco', 'pared', 'spending', '.'], ['it', 'would', 'take', 'three', 'votes', 'to', 'kill', 'the', 'acc', "'s", 'expansion', '.'], ['"', 'we', 'think', 'peoplesoft', 'customers', 'deserve', 'the', 'right', 'to', 'choose', ',', '"', 'ellison', 'said', '.'], ['the', 'dvd', 'cca', 'appealed', 'that', 'decision', 'to', 'the', 'u.s.', 'supreme', 'court', '.'], ['under', 'the', 'agreement', ',', 'medicare', 'coverage', 'for', 'drug', 'benefits', 'would', 'begin', 'in', '2006', '.'], ['chinese', 'language', 'web', 'portals', 'also', 'support', '3721', 'keywords', ',', 'the', 'web', 'site', 'said', '.'], ['fcc', 'chief', 'michael', 'powell', 'presides', 'over', 'hearing', 'monday', '.'], ['buymusic', 'is', 'the', 'first', 'subscription-free', 'paid', 'download', 'music', 'service', 'for', 'pc', 'users', '.'], ['the', 'no-shows', 'were', 'sens.', 'john', 'kerry', 'of', 'massachusetts', 'and', 'bob', 'graham', 'of', 'florida', '.'], ['young', 'faces', 'a', 'fine', ',', 'suspension', 'or', 'being', 'permanently', 'barred', 'from', 'the', 'securities', 'industry', '.'], ['nascar', 'officials', 'could', 'not', 'be', 'reached', 'for', 'comment', 'tuesday', '.'], ['nevada', 'is', 'where', 'the', 'most', 'women', 'smoke', ',', '28', 'percent', '.'], ['"', 'i', 'was', 'in', 'the', 'water', 'for', 'about', 'eight', 'seconds', '.'], ['the', 'remaining', '1,557', 'genes', 'are', 'believed', 'to', 'be', 'all', 'functional', '.'], ['thursday', 'morning', ',', 'peoplesoft', "'s", 'board', 'rejected', 'the', 'oracle', 'takeover', 'offer', '.'], ['there', 'is', 'no', 'clear', 'successor', 'for', 'mr.', 'o', "'neill", 'at', 'hyundai', '.'], ['missouri', 'health', 'officials', 'said', 'he', 'had', 'not', 'been', 'hospitalized', 'and', 'is', 'recovering', '.'], ['grassley', 'and', 'baucus', 'said', 'they', 'had', 'rejected', 'that', 'approach', 'in', 'their', 'plan', '.'], ['puretunes', 'representatives', 'could', 'not', 'be', 'located', 'thursday', 'to', 'comment', 'on', 'the', 'suit', '.'], ['chiron', 'has', 'acceptances', 'from', 'holders', 'of', 'nearly', '20', 'percent', 'of', 'powderject', 'shares', '.'], ['products', 'featuring', 'vanderpool', 'will', 'be', 'released', 'within', 'five', 'years', ',', 'he', 'said', '.'], ['"', 'we', 'think', 'it', "'s", 'great', 'news', ',', '"', 'said', 'enron', 'spokesman', 'eric', 'thode', '.'], ['the', 'bombardier', 'and', 'embraer', 'aircraft', 'will', 'be', 'delivered', 'to', 'u.s.', 'airways', 'by', 'september', '2006', '.'], ['of', 'those', 'who', 'signed', 'the', 'letter', ',', '57', 'are', 'senators', ',', 'including', '24', 'republicans', '.'], ['whitley', "'s", 'attorney', ',', 'marc', 'garber', ',', 'called', 'the', 'ruling', '"', 'a', 'huge', 'victory', '.', '"'], ['the', 'atlanta', 'olympic', 'games', 'attack', 'killed', 'one', 'woman', 'and', 'injured', 'more', 'than', '100', 'other', 'people', '.'], ['two', 'years', 'ago', ',', 'the', 'house', 'passed', 'a', 'similar', 'version', 'that', 'floundered', 'in', 'the', 'senate', '.'], ['the', 'california', 'farm', 'bureau', 'did', 'not', 'immediately', 'have', 'an', 'official', 'response', 'to', 'tuesday', "'s", 'ruling', '.'], ['congress', 'has', 'added', 'a', 'new', 'benefit', '-', 'an', 'option', 'to', 'buy', 'insurance', 'coverage', 'for', 'prescription', 'drugs', '.'], ['in', 'addition', ',', '6,800', 'people', 'were', 'in', 'quarantine', 'and', 'thousands', 'of', 'health-care', 'workers', 'in', 'working', 'quarantine', '.'], ['district', 'superintendent', 'j.', 'chester', 'floyd', 'told', 'reporters', 'monday', 'that', 'mccrackin', 'will', 'be', 'reassigned', 'within', 'the', 'district', '.'], ['zuccarini', 'was', 'ordered', 'held', 'without', 'bail', 'wednesday', 'by', 'a', 'federal', 'judge', 'in', 'fort', 'lauderdale', ',', 'fla', '.'], ['at', 'northwest', 'medical', 'center', 'of', 'washington', 'county', 'in', 'springdale', ',', 'one', 'child', 'is', 'in', 'serious', 'condition', '.'], ['on', 'aug.', '8', ',', 'juvenile', 'court', 'judge', 'robert', 'yeates', 'ordered', 'that', 'parker', 'be', 'placed', 'in', 'protective', 'custody', '.'], ['aug.', '15', ',', '1982', ':', 'remains', 'of', 'chapman', ',', 'hinds', 'and', 'mills', 'are', 'found', 'in', 'the', 'green', 'river', '.'], ['the', 'texas', 'congressional', 'delegation', 'now', 'is', '17-15', 'democratic', '.'], ['a', 'global', 'crossing', 'representative', 'had', 'no', 'immediate', 'comment', '.'], ['greg', 'parseghian', 'was', 'appointed', 'the', 'new', 'chief', 'executive', '.'], ['this', 'report', 'was', 'cleared', 'by', 'the', 'joint', 'intelligence', 'committee', '.'], ['dittemore', 'announced', 'his', 'plans', 'to', 'resign', 'on', 'april', '23', '.'], ['pete', 'slover', 'reported', 'from', 'laredo', 'and', 'gromer', 'jeffers', 'from', 'albuquerque', '.'], ['and', 'several', 'senate', 'republicans', 'are', 'cranky', 'about', 'the', 'house', 'map', '.'], ['quattrone', 'lawyer', 'john', 'w.', 'keker', 'said', 'his', 'client', 'is', 'innocent', '.'], ['but', 'lawtey', 'is', 'the', 'first', 'entire', 'prison', 'to', 'take', 'that', 'path', '.'], ['dotson', ',', '21', ',', 'was', 'subsequently', 'charged', 'with', 'dennehy', "'s", 'murder', '.'], ['the', 'company', 'said', 'this', 'impacted', 'earnings', 'by', 'a', 'penny', 'a', 'share', '.'], ['service', 'officials', 'said', 'carlson', 'refused', 'to', 'recuse', 'himself', 'from', 'the', 'case', '.'], ['both', 'men', 'are', 'being', 'held', 'for', 'investigation', 'of', 'administrative', 'immigration', 'violations', '.'], ['plaintiffs', "'", 'attorneys', 'would', 'get', '$', '4.1', 'million', 'of', 'the', 'settlement', '.'], ['a', 'year', 'ago', 'the', 'firm', 'posted', 'a', 'profit', 'of', '26', 'million', 'pounds', '.'], ['some', '34.7', 'million', 'shares', 'have', 'been', 'tendered', ',', 'oracle', 'said', 'in', 'a', 'statement', '.'], ['many', 'of', 'the', 'changes', 'are', 'improvements', 'to', 'the', 'original', 'plan', ',', 'libeskind', 'said', '.'], ['the', 'letter', 'indicates', 'the', 'vatican', "'s", 'interest', 'in', 'bolstering', 'conservatives', 'within', 'the', 'episcopal', 'church', '.'], ['as', 'of', 'wednesday', ',', 'there', 'were', '65', 'probable', 'sars', 'cases', 'in', 'the', 'toronto', 'region', '.'], ['from', '1993', 'to', '2002', ',', 'there', 'were', '142', 'reported', 'sexual', 'assaults', 'at', 'the', 'academy', '.'], ['marcel', 'desailly', ',', 'the', 'france', 'captain', 'and', 'chelsea', 'defender', ',', 'believes', 'the', 'latter', 'is', 'true', '.'], ['we', 'think', 'this', 'was', 'bad', 'law', ',', 'and', 'we', 'still', 'expect', 'to', 'be', 'vindicated', 'ultimately', '.'], ['before', 'the', 'meeting', ',', 'putin', 'said', 'russia', 'regards', 'saudi', 'arabia', 'as', 'a', 'key', 'muslim', 'state', '.'], ['bounty', 'hunting', 'is', 'illegal', 'in', 'mexico', ',', 'and', 'the', 'bounty', 'hunter', 'was', 'charged', 'with', 'kidnapping', '.'], ['the', 'nasdaq', 'composite', 'index', ',', 'full', 'of', 'technology', 'stocks', ',', 'was', 'lately', 'up', 'around', '18', 'points', '.'], ['second-quarter', 'sales', 'grew', '7', 'percent', ',', 'to', '$', '645', 'million', 'from', '$', '600', 'million', 'in', '2002', '.'], ['so', 'far', ',', 'authorities', 'also', 'have', 'searched', 'areas', 'in', 'pennsylvania', ',', 'ohio', ',', 'indiana', ',', 'and', 'michigan', '.'], ['the', 'technology-laced', 'nasdaq', 'composite', 'index', '<', '.ixic', '>', 'added', '1.92', 'points', ',', 'or', '0.12', 'percent', ',', 'at', '1,647.94', '.'], ['"', 'it', "'s", 'a', 'blond-haired', 'woman', 'wearing', 'a', 'cartier', 'watch', 'on', 'her', 'wrist', ',', '"', 'the', 'source', 'said', '.'], ['the', 'jurors', 'were', 'taken', 'into', 'the', 'courtroom', 'in', 'groups', 'of', '40', 'and', 'asked', 'to', 'fill', 'out', 'a', 'questionnaire', '.'], ['two', 'more', 'died', 'later', ',', 'and', 'initial', 'autopsy', 'reports', 'show', 'they', 'succumbed', 'to', 'dehydration', ',', 'hypothermia', 'and', 'suffocation', '.'], ['the', 'exchange', 'also', 'said', 'its', 'five-person', 'compensation', 'committee', 'will', 'consist', 'only', 'of', 'directors', 'from', 'outside', 'the', 'securities', 'industry', '.'], ['john', 'heine', ',', 'a', 'spokesman', 'for', 'the', 'commission', 'in', 'washington', ',', 'declined', 'to', 'comment', 'on', 'mr.', 'spitzer', "'s", 'criticism', '.'], ['he', 'urged', 'employees', 'to', '"', 'avoid', 'complacency', '"', 'and', 'to', '"', 'change', 'old', 'habits', 'and', 'seriously', 'rethink', 'business-as-usual', '.', '"'], ['members', 'of', 'kathleen', 'peterson', "'s", 'family', 'were', 'not', 'present', '.'], ['peoplesoft', "'s", 'board', 'has', 'recommended', 'stockholders', 'reject', 'the', 'offer', '.'], ['he', 'added', 'that', 'prosecutors', 'will', 'seek', 'the', 'death', 'penalty', '.'], ['shares', 'of', 'redwood', 'shores-based', 'oracle', 'rose', '14', 'cents', 'to', '$', '13.62', '.'], ['the', 'study', 'appears', 'today', 'in', 'the', 'new', 'england', 'journal', 'of', 'medicine', '.'], ['selenski', 'had', 'served', 'about', 'seven', 'years', 'in', 'prison', 'for', 'bank', 'robbery', '.'], ['the', 'dollar', 'rose', '0.15', 'percent', 'against', 'the', 'japanese', 'currency', 'to', '115.97', 'yen', '.'], ['but', 'the', 'north', 'korean', 'nuclear', 'crisis', 'has', 'dominated', 'his', 'time', 'in', 'office', '.'], ['october', 'heating', 'oil', 'ended', 'down', '0.41', 'cent', 'to', '70.74', 'cents', 'a', 'gallon', '.'], ['"', 'i', 'would', 'rather', 'be', 'talking', 'about', 'positive', 'numbers', 'rather', 'than', 'negative', '.'], ['water', 'samples', 'are', 'being', 'sent', 'to', 'the', 'state', 'health', 'department', 'for', 'analysis', '.'], ['unifi', 'official', 'rob', 'o', "'neill", 'said', ':', '"', 'the', 'gloves', 'are', 'off', '.'], ['four', 'of', 'the', 'defendants', 'have', 'pleaded', 'guilty', 'to', 'weapons', 'charges', 'and', 'other', 'counts', '.'], ['the', 'pcs', 'are', 'scattered', 'across', 'the', 'united', 'states', ',', 'canada', 'and', 'south', 'korea', '.'], ['the', 'product', 'can', 'be', 'instructed', 'to', 'permit', 'business-only', 'use', ',', 'according', 'to', 'sendmail', '.'], ['he', 'also', 'objects', 'to', 'the', 'sales', 'and', 'personal', 'income', 'tax', 'increases', 'in', 'the', 'package', '.'], ['obus', 'also', 'denied', 'a', 'defense', 'motion', 'to', 'postpone', 'the', 'sept', '.', '29', 'trial', 'date', '.'], ['revenue', 'was', 'up', 'a', 'tad', ',', 'from', '$', '610.6', 'million', 'to', '$', '616.5', 'million', '.'], ['her', '20-year-old', 'sister', ',', 'allyson', ',', 'was', 'severely', 'burned', 'and', 'remained', 'hospitalized', 'in', 'critical', 'condition', '.'], ['we', "'re", 'thrilled', 'to', 'be', 'up', '3-2', ',', "'", "'", 'coach', 'gregg', 'popovich', 'said', 'wednesday', '.'], ['that', 'federal', 'investigation', 'is', 'separate', 'from', 'an', 'informal', 'inquiry', 'by', 'the', 'securities', 'and', 'exchange', 'commission', '.'], ['that', 'poll', 'had', '712', 'likely', 'voters', 'and', 'sampling', 'error', 'of', 'plus', 'or', 'minus', '3.7', 'percentage', 'points', '.'], ['oracle', 'shares', 'also', 'rose', 'on', 'the', 'news', ',', 'up', '15', 'cents', 'to', '$', '13.51', 'on', 'the', 'nasdaq', '.'], ['the', 'new', 'zealand', 'commerce', 'commission', 'had', 'given', 'westpac', 'no', 'indication', 'whether', 'it', 'would', 'have', 'approved', 'its', 'deal', '.'], ['district', 'attorney', 'dave', 'lupas', 'reminded', 'jurors', 'that', 'stubbs', 'is', ',', '"', 'a', 'cold', ',', 'calculating', 'killer', '"', '.'], ['ernst', '&', 'young', 'has', 'denied', 'the', 'sec', "'s", 'claims', ',', 'and', 'called', 'its', 'recommendations', '"', 'irresponsible', '"', '.'], ['on', 'saturday', ',', 'roddick', 'equalled', 'the', 'world', 'record', 'with', 'a', '149', 'm.p.h.', 'serve', 'in', 'beating', 'andre', 'agassi', '.'], ['and', 'all', 'of', 'the', 'samples', 'contained', 'molecules', 'that', 'can', 'only', 'be', 'produced', 'when', 'cholesterol', 'interacts', 'with', 'ozone', '.'], ['shares', 'of', 'bioreliance', 'sold', 'at', '$', '47.98', 'at', 'the', 'close', 'of', 'market', 'yesterday', ',', 'down', '2', 'cents', '.'], ['germans', 'represent', 'more', 'than', 'a', 'quarter', 'of', 'all', 'foreign', 'visitors', 'to', 'italy', 'and', '40', 'percent', 'of', 'hotel', 'bookings', '.'], ['shelley', "'s", 'office', 'released', 'signature', 'counts', 'late', 'friday', 'and', 'said', 'counties', 'had', 'reported', 'counting', '575,926', 'signatures', 'so', 'far', '.'], ['in', 'morning', 'trading', 'on', 'the', 'new', 'york', 'stock', 'exchange', ',', 'coca-cola', 'shares', 'were', 'down', '34', 'cents', 'at', '$', '43.67', '.'], ['the', 'boy', 'was', 'taken', 'to', 'nassau', 'university', 'medical', 'center', ',', 'where', 'staff', 'said', 'he', 'was', '"', 'doing', 'okay', '.', '"'], ['kentucky', ':', 'republican', 'ernie', 'fletcher', ',', 'a', 'three-term', 'house', 'member', ',', 'is', 'battling', 'democratic', 'state', 'attorney', 'general', 'ben', 'chandler', '.'], ['mr', 'hughes', 'had', 'seen', 'amrozi', "'s", 'face', 'on', 'television', ',', 'the', 'infamous', 'images', 'of', 'the', 'grinning', 'mechanic', 'from', 'east', 'java', '.'], ['state', 'officials', 'estimated', 'the', 'cost', 'at', 'nearly', '$', '2', 'billion', '.'], ['tickets', ',', 'available', 'through', 'ticketmaster', ',', 'went', 'on', 'sale', 'yesterday', '.'], ['bryant', ',', '24', ',', 'has', 'been', 'charged', 'with', 'felony', 'sexual', 'assault', '.'], ['o', "'brien", "'s", 'attorney', ',', 'jordan', 'green', ',', 'declined', 'to', 'comment', '.'], ['the', 'suit', 'was', 'filed', 'in', 'the', 'supreme', 'court', 'of', 'new', 'york', 'county', '.'], ['peter', 'deutsch', ',', 'd-fla', '.', ',', 'said', 'he', 'wanted', 'to', 'congratulate', 'the', 'fda', '.'], ['about', '20', 'family', 'members', 'of', 'the', 'victims', 'were', 'invited', 'to', 'the', 'news', 'conference', '.'], ['gagne', 'struck', 'out', 'two', 'in', 'a', 'perfect', 'ninth', 'inning', 'for', 'his', '23rd', 'save', '.'], ['chief', 'justice', 'william', 'h.', 'rehnquist', 'and', 'justices', 'antonin', 'scalia', 'and', 'clarence', 'thomas', 'dissented', '.'], ['the', 'company', 'has', 'expanded', 'those', 'basic', 'services', ',', 'offering', 'payment', 'and', 'even', 'financing', '.'], ['microsoft', 'acquired', 'virtual', 'pc', 'from', 'its', 'developer', ',', 'connectix', ',', 'earlier', 'this', 'year', '.'], ['congress', 'twice', 'passed', 'similar', 'bills', ',', 'but', 'then-president', 'clinton', 'vetoed', 'them', 'both', 'times', '.'], ['"', 'in', 'return', 'we', 'expect', 'north', 'korea', 'to', 'give', 'up', 'nuclear', 'weapons', '.', '"'], ['an', 'fbi', 'statement', 'said', 'legal', 'proceedings', 'were', 'expected', 'monday', 'in', 'federal', 'court', 'in', 'baltimore', '.'], ['in', 'a', 'case', 'in', 'august', ',', 'a', '27-year-old', 'laboratory', 'worker', 'in', 'singapore', 'developed', 'sars', '.'], ['the', 'company', 'claimed', 'it', 'is', 'the', 'largest', 'sale', 'of', 'xserves', 'by', 'an', 'apple', 'retailer', '.'], ['excluding', 'one-time', 'items', ',', 'it', 'expects', 'profit', 'of', '11', 'cents', 'to', '15', 'cents', 'a', 'share', '.'], ['the', 'standard', 'edition', 'one', 'is', 'a', 'single', 'processor', 'version', 'of', 'the', 'oracle', 'standard', 'edition', 'database', '.'], ['one', 'man', 'had', 'brandished', 'an', 'english-turkish', 'dictionary', 'and', 'pointed', 'to', 'the', 'word', '"', 'refugee', '"', '.'], ['ratliff', 'said', 'he', 'hopes', 'to', 'bring', 'the', 'two', 'measures', 'to', 'the', 'full', 'senate', 'by', 'friday', '.'], ['declining', 'issues', 'outnumbered', 'advancers', 'slightly', 'more', 'than', '3', 'to', '1', 'on', 'the', 'new', 'york', 'stock', 'exchange', '.'], ['the', '6-3', 'ruling', 'reinstates', 'a', 'law', 'that', 'told', 'libraries', 'to', 'install', 'filters', 'or', 'surrender', 'federal', 'money', '.'], ['each', 'boy', 'weighed', 'less', 'than', '50', 'pounds', 'when', 'they', 'were', 'removed', 'from', 'the', 'home', 'oct.', '10', '.'], ['throughout', 'china', "'s", 'mainland', ',', 'the', 'disease', 'has', 'killed', '300', 'people', 'and', 'infected', 'more', 'than', '5,270', '.'], ['he', 'later', 'found', 'out', 'the', 'alarming', 'incident', 'had', 'been', 'caused', 'by', 'concorde', "'s", 'powerful', 'sonic', 'boom', '.'], ['against', 'the', 'swiss', 'franc', 'the', 'dollar', 'was', 'at', '1.3289', 'francs', ',', 'up', '0.5', 'percent', 'on', 'the', 'day', '.'], ['between', '1960', 'and', '2000', ',', 'however', ',', 'the', 'state', "'s", 'population', 'grew', 'by', '30.4', 'percent', 'to', '6,080,485', '.'], ['for', 'the', 'full', 'year', 'period', 'ending', 'june', '30', ',', '2003', ',', 'high-speed', 'lines', 'increased', 'by', '45', 'percent', '.'], ['no', 'tumors', 'were', 'detected', ';', 'rather', ',', 'empty', 'cavities', 'and', 'scar', 'tissue', 'were', 'found', 'in', 'their', 'place', '.'], ['beta', 'versions', 'of', 'nterprise', 'linux', 'services', 'are', 'expected', 'to', 'be', 'available', 'on', 'certain', 'hp', 'proliant', 'servers', 'in', 'july', '.'], ['the', 'technology-laced', 'nasdaq', 'composite', 'index', '.ixic', 'gained', '13.73', 'points', ',', 'or', '0.81', 'percent', ',', 'to', 'finish', 'at', '1,700.34', '.'], ['the', 'afl-cio', 'announced', 'wednesday', 'that', 'it', 'will', 'decide', 'in', 'october', 'whether', 'to', 'endorse', 'a', 'candidate', 'before', 'the', 'primaries', '.'], ['he', 'said', 'the', 'problem', 'needs', 'to', 'be', 'corrected', 'before', 'the', 'space', 'shuttle', 'fleet', 'is', 'cleared', 'to', 'fly', 'again', '.'], ['he', 'played', 'four', 'seasons', 'in', 'miami', ',', 'including', 'the', 'dolphins', "'", '27-17', 'loss', 'to', 'washington', 'in', 'the', 'super', 'bowl', '.'], ['the', 'submission', 'was', 'on', 'monday', ',', 'but', 'the', 'announcement', 'was', 'made', 'friday', 'at', 'the', 'international', 'broadcasting', 'convention', 'in', 'amsterdam', '.'], ['kathy', 'nicolo', '(', 'jennifer', 'connelly', ')', 'is', 'desperate', 'that', 'her', 'family', 'not', 'find', 'out', 'that', 'her', 'life', 'has', 'fallen', 'apart', '.'], ['both', 'counterpane', 'and', 'idefense', 'contend', 'that', 'the', 'exploit', 'works', 'effectively', 'against', 'windows', '2000', 'systems', 'running', 'service', 'pack', '3', 'and', '4', '.'], ['dell', "'s", 'personal', 'all-in-one', 'a920', 'is', 'an', 'entry-level', 'device', 'that', 'combines', 'printer', ',', 'scanner', 'and', 'copier', 'functions', 'for', '$', '89', '.'], ['mr.', 'chapman', ',', 'who', 'hasn', "'t", 'been', 'arrested', ',', 'is', 'expected', 'to', 'appear', 'for', 'arraignment', 'next', 'thursday', ',', 'prosecutors', 'said', '.'], ['sovereign', 'shares', 'closed', 'on', 'the', 'new', 'york', 'stock', 'exchange', 'at', '$', '15.68', ',', 'down', '74', 'cents', ',', 'or', '4.5', 'percent', '.'], ['the', 'trial', ',', 'which', 'could', 'last', 'until', 'early', '2004', ',', 'is', 'expected', 'to', 'hear', 'the', 'first', 'of', '125', 'witnesses', 'friday', '.'], ['the', 'woman', 'reported', 'that', 'she', 'felt', 'threatened', 'and', 'obtained', 'a', 'warrant', 'for', 'stackhouse', "'s", 'arrest', 'from', 'the', 'local', 'magistrate', "'s", 'office', '.'], ['schwarzenegger', 'picks', 'up', 'more', 'independents', 'and', 'crossover', 'voters', 'than', 'bustamante', '.'], ['the', 'rovers', 'act', 'as', 'robotic', 'geologists', ',', 'moving', 'on', 'six', 'wheels', '.'], ['mahmud', 'controlled', 'access', 'to', 'saddam', 'and', 'was', 'frequently', 'at', 'his', 'side', '.'], ['sales', 'rose', '37', 'per', 'cent', 'year-on-year', 'to', '1.76bn', ',', 'beating', 'expectations', '.'], ['nearly', 'half', '-', '45', 'percent', '-', 'are', 'for', 'shopping', 'or', 'running', 'errands', '.'], ['a', 'captured', 'rebel', 'said', '2,100', 'combatants', 'had', 'been', 'involved', 'in', 'the', 'offensive', '.'], ['the', 'findings', 'are', 'being', 'published', 'today', 'in', 'the', 'annals', 'of', 'internal', 'medicine', '.'], ['security', 'lights', 'have', 'also', 'been', 'installed', 'on', 'a', 'barn', 'near', 'the', 'front', 'gate', '.'], ['the', 'dow', 'jones', 'industrial', 'average', 'finished', 'the', 'day', 'down', '98.32', 'points', 'at', '9,011.53', '.'], ['of', 'the', '1.1', 'million', 'infected', 'in', 'south', 'asia', ',', '62', '%', 'are', 'female', '.'], ['the', 'court', "'s", 'three-judge', 'tribunal', 'was', 'expected', 'to', 'give', 'its', 'verdict', 'next', 'february', '.'], ['"', 'this', 'is', 'an', 'extremely', 'heartrending', 'time', 'for', 'the', 'entire', 'brandeis', 'university', 'community', '.'], ['prosecutors', 'maintained', 'that', 'durst', 'murdered', 'black', 'to', 'try', 'to', 'assume', 'black', "'s", 'identity', '.'], ['mr.', 'mills', 'declined', 'to', 'comment', 'yesterday', ',', 'saying', 'that', 'he', 'never', 'discussed', 'personnel', 'matters', '.'], ['commercial', 'releases', 'of', 'the', '2.6', 'kernel', 'by', 'major', 'linux', 'distributors', 'still', 'remain', 'months', 'away', '.'], ['there', 'were', '659', 'women', 'enrolled', 'at', 'the', 'academy', 'at', 'the', 'time', 'of', 'the', 'survey', '.'], ['connolly', 'said', 'fbi', 'agents', 'in', 'san', 'diego', 'and', 'abroad', 'spent', 'thousands', 'of', 'hours', 'investigating', 'al-bayoumi', '.'], ['mr', 'mcdevitt', 'has', 'been', 'granted', 'control', 'of', 'three', 'aspects', 'of', 'policing', 'by', 'commissioner', 'william', 'morrell', '.'], ['gbi', 'spokesman', 'john', 'bankhead', 'said', 'the', 'murder', 'scenes', 'showed', 'that', 'the', 'killer', 'was', 'very', 'methodical', '.'], ['one', 'of', 'the', 'defining', 'and', 'well-studied', 'characteristics', 'of', 'temperament', 'is', 'how', 'people', 'react', 'to', 'novelty', '.'], ['the', 'quarterlies', 'usually', 'stay', 'in', 'stores', 'for', 'between', 'six', 'to', 'eight', 'weeks', ',', '"', 'carney', 'added', '.'], ['meanwhile', ',', 'funny', 'cide', "'s", 'trainer', ',', 'barclay', 'tagg', ',', 'called', 'the', 'allegations', '"', 'ridiculous', '.', '"'], ['he', 'was', 'arrested', 'in', 'atlanta', ',', 'georgia', ',', 'on', 'monday', 'night', 'by', 'police', 'acting', 'on', 'a', 'tip-off', '.'], ['pataki', 'said', 'if', 'power', 'was', 'restored', ',', 'state', 'workers', 'would', 'be', 'back', 'on', 'the', 'job', 'friday', 'morning', '.'], ['about', '38,000', 'oge', 'energy', 'corp.', 'customers', 'are', 'without', 'power', ',', 'the', 'company', 'said', 'on', 'its', 'web', 'site', '.'], ['amazon', "'s", 'shares', 'yesterday', 'closed', 'at', '$', '54.32', 'on', 'the', 'nasdaq', 'stock', 'market', ',', 'up', '29', 'cents', '.'], ['webtv', 'was', 'sold', 'to', 'microsoft', 'in', '1997', 'for', '$', '425', 'million', 'and', 'today', 'is', 'called', 'msn', 'tv', '.'], ['attorneys', 'for', 'both', 'the', 'archdiocese', 'and', 'the', 'plaintiffs', 'began', 'meeting', 'last', 'week', 'to', 'reach', 'the', 'out-of-court', 'settlement', '.'], ['khatami', ',', 'who', 'arrived', 'in', 'beirut', 'monday', ',', 'was', 'also', 'to', 'visit', 'syria', ',', 'yemen', 'and', 'bahrain', '.'], ['the', 'new', 'york', 'stock', 'exchange', 'suspended', 'trading', 'yesterday', 'in', 'loral', ',', 'which', 'closed', 'at', '$', '3.01', 'friday', '.'], ['the', 'results', 'were', '3', 'cents', 'a', 'share', 'lower', 'than', 'the', 'forecast', 'of', 'analysts', 'surveyed', 'by', 'thomson', 'first', 'call', '.'], ['enron', "'s", 'stock', 'comprised', 'as', 'much', 'as', '61', '%', 'of', 'the', 'workers', "'", '401', '(', 'k', ')', 'portfolios', '.'], ['officials', 'with', 'torch', 'and', 'jetblue', 'did', 'not', 'respond', 'to', 'repeated', 'phone', 'calls', 'from', 'the', 'mercury', 'news', 'seeking', 'comment', '.'], ['prince', ',', 'who', 'heads', 'citigroup', "'s", 'global', 'corporate', 'and', 'investment', 'bank', ',', 'is', 'replacing', 'sanford', 'weill', 'as', 'ceo', '.'], ['since', 'then', ',', 'texas', 'gov.', 'rick', 'perry', 'has', 'called', 'two', 'special', 'sessions', 'trying', 'to', 'push', 'the', 'measure', 'through', '.'], ['the', 'rev.', 'christopher', 'coyne', ',', 'spokesman', 'for', 'the', 'archdiocese', ',', 'did', 'not', 'immediately', 'return', 'several', 'calls', 'seeking', 'comment', '.'], ['by', '2040', ',', 'retirees', 'will', 'account', 'for', 'a', 'third', 'of', 'the', 'population', ',', 'up', 'from', 'a', 'fifth', 'today', '.'], ['indeed', ',', 'wolfowitz', 'admitted', 'saturday', 'that', 'neither', 'military', 'action', 'nor', '"', 'large', 'scale', 'bribery', '"', 'would', 'solve', 'the', 'issue', '.'], ['the', 'impact', 'of', 'natural-gas', 'shortages', '"', 'will', 'touch', 'virtually', 'every', 'american', ',', '"', 'energy', 'secretary', 'spencer', 'abraham', 'warned', 'yesterday', '.'], ['at', 'the', 'service', 'morrill', "'s", 'widow', ',', 'ellie', ',', 'sobbed', 'and', 'hugged', 'bondeson', "'s", 'sister-in-law', 'as', 'people', 'consoled', 'her', '.'], ['what', "'s", 'more', ',', 'companies', 'are', 'not', 'feeling', 'confident', 'enough', 'to', 'replace', 'older', 'pcs', ',', 'the', 'analyst', 'firm', 'said', '.'], ['yeager', 'said', 'the', 'incident', 'appeared', 'to', 'be', 'isolated', ',', 'but', 'the', 'suspect', 'showed', 'tendencies', 'of', 'being', 'a', 'prior', 'offender', '.'], ['the', 'nasdaq', 'is', 'down', 'roughly', '0.4', 'percent', 'for', 'the', 'month', ',', 'on', 'track', 'to', 'snap', 'a', '7-month', 'streak', 'of', 'gains', '.'], ['analysts', 'surveyed', 'by', 'first', 'call', 'were', 'expecting', 'sales', 'of', '$', '723', 'million', 'and', 'a', 'loss', 'of', '28', 'cents', 'a', 'share', '.'], ['the', 'government', 'wanted', 'to', 'revive', 'talks', 'with', 'the', 'liberation', 'tigers', 'of', 'tamil', 'eelam', '(', 'ltte', ')', 'to', 'discuss', 'the', 'interim', 'structure', '.'], ['in', 'july', '32', 'singaporean', 'police', 'officers', 'were', 'sent', 'to', 'baghdad', 'to', 'help', 'train', 'iraqi', 'police', 'forces', 'and', 'returned', 'home', 'last', 'month', '.'], ['"', 'we', 'have', 'found', 'the', 'smoking', 'gun', ',', '"', 'said', 'hubbard', ',', 'director', 'of', 'nasa', "'s", 'ames', 'research', 'center', 'in', 'california', '.'], ['hebei', 'has', 'reported', '191', 'cases', 'and', 'eight', 'deaths', ',', 'though', 'the', 'who', 'experts', 'did', 'not', 'say', 'how', 'many', 'were', 'in', 'rural', 'areas', '.'], ['she', 'was', 'the', 'only', 'woman', 'employed', 'as', 'a', 'warehouse', 'worker', 'and', 'heavy', 'equipment', 'operator', 'and', 'the', 'only', 'woman', 'in', 'her', 'teamsters', 'local', '.'], ['for', 'the', 'third', 'time', 'in', 'four', 'years', 'wildfires', 'closed', 'mesa', 'verde', 'national', 'park', ',', 'the', 'country', '’', 's', 'only', 'park', 'dedicated', 'to', 'ancient', 'ruins', '.'], ['freeman', 'said', 'not', 'having', 'a', 'driver', 'license', 'has', 'been', 'a', 'burden', '.'], ['it', 'called', 'on', 'the', 'commerce', 'department', 'to', 'respond', 'within', '60', 'days', '.'], ['shares', 'of', 'mandalay', 'closed', 'down', '8', 'cents', 'at', '$', '29.42', 'thursday', '.'], ['above', 'that', ',', 'medicare', 'would', 'pay', '90', 'percent', 'of', 'all', 'drug', 'costs', '.'], ['in', 'new', 'york', ',', 'vivendi', 'shares', 'were', '1.4', 'percent', 'down', 'at', '$', '18.29', '.'], ['it', 'also', 'features', 'a', '4.5', 'in', 'back-lit', 'lcd', 'screen', 'and', 'memory', 'expansion', 'facilities', '.'], ['tnk-bp', 'has', 'six', 'oil', 'producing', 'units', ',', 'five', 'refineries', 'and', '2,100', 'filling', 'stations', '.'], ['budget', 'negotiations', 'between', 'the', 'mayor', 'and', 'the', 'city', 'council', 'are', 'entering', 'high', 'gear', '.'], ['the', 'company', 'will', 'offer', 'songs', 'for', '99', 'cents', 'and', 'albums', 'for', '$', '9.95', '.'], ['dave', 'delainey', 'agreed', 'to', 'cooperate', 'with', 'federal', 'prosecutors', 'in', 'exchange', 'for', 'the', 'plea', '.'], ['a', 'black', 'pickup', 'truck', 'was', 'also', 'seen', 'in', 'the', 'campbells', 'creek', 'area', ',', 'morris', 'said', '.'], ['brian', 'halla', ',', 'ceo', 'of', 'natsemi', ',', 'claimed', 'the', 'deal', 'made', 'sense', 'for', 'both', 'companies', '.'], ['telemarketing', 'associates', ',', 'hired', 'by', 'vietnow', ',', 'raised', '$', '7.1', 'million', 'from', '1987', 'to', '1995', '.'], ['hp', 'fell', 'to', 'second', 'place', 'with', 'server', 'sales', 'growing', '0.4', 'percent', 'to', '$', '2.9', 'billion', '.'], ['he', 'has', 'a', 'very', 'good', 'reputation', ',', '"', 'rt', 'jones', 'analyst', 'juli', 'niemann', 'said', 'of', 'grant', '.'], ['that', 'is', ',', 'if', 'the', 'recording', 'industry', 'association', 'of', 'america', 'has', 'anything', 'to', 'say', 'about', 'it', '.'], ['the', 'nasdaq', 'composite', 'index', ',', 'full', 'of', 'technology', 'stocks', ',', 'was', 'lately', 'up', 'around', '18', 'points', '.'], ['the', 'donation', 'of', '60,000', 'tons', 'brings', 'the', 'total', 'of', 'u.s.', 'contributions', 'for', 'the', 'year', 'to', '100,000', '.'], ['several', 'florida', 'officials', 'connected', 'to', 'the', 'case', 'have', 'received', 'threatening', 'letters', ',', 'accompanied', 'by', 'rifle', 'bullets', '.'], ['kerkowski', 'and', 'fassett', "'s", 'bodies', 'were', 'unearthed', 'at', 'selenski', "'s", 'mount', 'olivet', 'road', 'home', 'on', 'june', '5', '.'], ['iowa', 'state', 'athletic', 'director', 'bruce', 'van', 'de', 'velde', 'would', 'not', 'confirm', 'an', 'offer', 'was', 'made', 'to', 'lebo', '.'], ['"', 'they', 'have', 'blocked', 'all', 'new', 'bio-crops', 'because', 'of', 'unfounded', ',', 'unscientific', 'fears', ',', '"', 'bush', 'said', '.'], ['the', 'british', 'pound', 'flexed', 'its', 'muscle', 'against', 'the', 'dollar', ',', 'last', 'up', '1', 'percent', 'at', '$', '1.6672', '.'], ['georgia', 'has', 'received', 'all', 'but', '$', '70,000', 'of', 'its', 'money', 'back', 'with', 'the', 'help', 'of', 'the', 'fbi', '.'], ['passengers', 'were', 'evacuated', 'smoothly', ',', 'although', 'they', 'were', 'not', 'told', 'what', 'was', 'going', 'on', ',', 'he', 'said', '.'], ['but', 'the', 'officials', 'later', 'revised', 'the', 'numbers', 'and', 'said', 'about', '250', 'had', 'been', 'arrested', 'at', 'some', '60', 'stores', '.'], ['shares', 'of', 'hartford', 'were', 'up', '$', '2.28', ',', 'or', '5.2', 'percent', ',', 'to', '$', '45.90', 'in', 'midday', 'trading', '.'], ['he', 'was', 'joined', 'by', 'chief', 'justice', 'william', 'h.', 'rehnquist', 'and', 'justices', 'sandra', 'day', 'o', "'connor", 'and', 'antonin', 'scalia', '.'], ['her', 'son', 'would', 'get', 'upset', ',', 'his', 'mom', 'said', ',', 'but', 'then', 'turn', 'around', 'and', 'befriend', 'his', 'taunters', '.'], ['police', 'said', 'her', 'two', 'small', 'children', 'were', 'alone', 'there', 'for', 'up', 'to', 'two', 'days', 'as', 'she', 'lay', 'dead', '.'], ['"', '[', 'they', ']', 'were', 'defined', 'by', 'dedication', 'and', 'courage', ',', '"', 'said', 'the', 'cia', "'s", 'director', 'george', 'tenet', '.'], ['including', 'convertible', 'securities', ',', 'the', 'total', 'estimated', 'value', 'of', 'the', 'deal', 'is', '$', '124', 'million', ',', 'according', 'to', 'corel', '.'], ['the', 'stock', 'had', 'risen', '63', 'cents', ',', 'or', '3', 'percent', ',', 'to', 'close', 'at', '$', '20.26', 'in', 'regular-session', 'nasdaq', 'trading', '.'], ['interactivecorp', "'s", 'shares', 'closed', 'at', '$', '34.19', ',', 'down', '$', '2.81', ',', 'or', '7.6', 'percent', 'on', 'the', 'nasdaq', 'stock', 'market', '.'], ['tampering', 'with', 'medicine', 'apparently', 'was', 'so', 'lucrative', ',', 'investigators', 'said', ',', 'that', 'many', 'of', 'the', 'suspects', 'lived', 'in', 'million-dollar', 'homes', '.'], ['yucaipa', 'bought', 'dominick', "'s", 'in', '1995', 'for', '$', '693', 'million', 'and', 'sold', 'it', 'to', 'safeway', 'for', '$', '1.8', 'billion', 'in', '1998', '.'], ['at', 'the', 'accident', 'scene', ',', 'the', 'driver', 'was', '"', 'covered', 'in', 'blood', '"', 'but', 'helped', 'to', 'remove', 'children', ',', 'wood', 'said', '.'], ['as', 'executive', 'vice', 'president', 'of', 'new', 'ventures', ',', 'elias', 'will', 'report', 'to', 'joe', 'tucci', ',', 'emc', "'s", 'president', 'and', 'chief', 'executive', '.'], ['mr', 'howard', 'also', 'said', 'yesterday', 'that', 'hicks', 'would', 'be', 'fairly', 'treated', ',', 'saying', 'he', 'had', 'faith', 'in', 'the', 'us', 'justice', 'system', '.'], ['for', 'the', 'current', 'academic', 'year', ',', 'tuition', 'at', 'public', 'colleges', 'averaged', '$', '4,694', ',', 'up', 'almost', '$', '600', 'from', 'the', 'year', 'before', '.'], ['the', 'men', 'had', 'pakistani', 'passports', 'and', 'reportedly', 'were', 'seen', 'together', 'at', 'the', 'airport', 'earlier', 'in', 'the', 'evening', ',', 'law', 'enforcement', 'sources', 'said', '.'], ['they', 'include', 'two', 'kuwaitis', 'and', 'six', 'palestinians', 'with', 'jordanian', 'passports', ',', 'and', 'the', 'remainder', 'are', 'iraqis', 'and', 'saudis', ',', 'the', 'official', 'said', '.'], ['the', 'decision', 'came', 'after', '"', 'listening', 'to', 'our', 'customers', 'and', 'associates', ',', '"', 'melissa', 'berryhill', ',', 'a', 'spokeswoman', 'for', 'wal-mart', ',', 'said', '.'], ['the', 'dow', 'jones', 'rose', '41.61', 'points', 'friday', ',', 'a', 'gain', 'of', '0.4', '%', 'for', 'the', 'day', 'and', '0.7', '%', 'for', 'the', 'week', '.'], ['in', 'a', 'statement', 'distributed', 'today', ',', '28', 'chechen', 'non-governmental', 'organisations', 'said', 'they', 'would', 'boycott', 'the', 'vote', 'and', 'warned', 'voters', 'to', 'expect', 'fraud', '.'], ['"', 'this', 'proposal', 'is', 'modest', 'compared', 'with', 'the', 'space', 'programme', ',', 'and', 'may', 'seem', 'unrealistic', 'only', 'because', 'little', 'effort', 'has', 'been', 'devoted', 'to', 'it', '.'], ['schools', 'chancellor', 'joel', 'klein', 'said', 'he', 'was', 'pleased', 'by', 'the', 'results', '.'], ['staff', 'writers', 'brandon', 'formby', 'and', 'colleen', 'mccain', 'nelson', 'contributed', 'to', 'this', 'report', '.'], ['philadelphia', "'s", 'council', 'is', 'not', 'the', 'first', 'to', 'defy', 'the', 'national', 'policy', '.'], ['he', 'said', ':', '“', 'they', 'were', 'an', 'inspirational', 'couple', ',', 'selfless', 'and', 'courageous', '.'], ['she', 'said', 'she', 'didn', "'t", 'know', 'yet', 'whether', 'she', 'would', 'certify', 'the', 'budget', '.'], ['the', 'dead', 'man', ',', 'leon', 'williams', ',', 'was', 'found', 'in', 'his', 'third-floor', 'apartment', '.'], ['the', 'airline', 'said', 'a', 'similar', 'discovery', 'was', 'made', 'on', 'the', 'plane', 'in', 'houston', '.'], ['the', 'broader', 'market', 'also', 'retreated', ',', 'having', 'climbed', 'higher', 'for', 'four', 'consecutive', 'weeks', '.'], ['"', 'she', 'was', 'crying', 'and', 'she', 'was', 'really', 'scared', ',', '"', 'said', 'yasin', '.'], ['the', 'decision', 'is', '"', 'clearly', 'incorrect', ',', '"', 'ftc', 'chairman', 'tim', 'muris', 'said', '.'], ['it', 'is', 'the', 'first', 'named', 'storm', 'to', 'develop', 'in', 'the', 'caribbean', 'in', 'december', '.'], ['the', 'health', 'department', 'spokesperson', 'added', 'the', 'department', 'is', 'following', 'centers', 'for', 'disease', 'control', 'protocol', '.'], ['earnings', 'were', 'affected', 'by', 'a', 'non-recurring', '$', '8', 'million', 'tax', 'benefit', 'in', 'the', 'year-ago', 'period', '.'], ['sir', 'richard', 'was', 'speaking', 'after', 'a', 'meeting', 'in', 'london', 'with', 'australian', 'tourism', 'minister', 'joe', 'hockey', '.'], ['this', 'after', 'seven', 'health', 'care', 'workers', 'at', 'a', 'public', 'hospital', 'fell', 'ill', 'with', 'flu-like', 'symptoms', '.'], ['naval', 'academy', 'spokesman', 'cmdr.', 'rod', 'gibbons', 'said', ':', '``today', "'s", 'announcement', 'came', 'as', 'a', 'surprise', '.'], ['the', 'dow', 'jones', 'industrial', 'average', 'fell', '10.89', 'points', ',', 'or', '0.11', 'percent', ',', 'to', '9,837.94', '.'], ['their', 'belief', 'was', 'based', 'on', 'speculation', 'that', 'estrogen', 'prevents', 'cell', 'damage', 'and', 'improves', 'blood', 'flow', '.'], ['one', 'witness', 'told', 'the', 'anatolian', 'news', 'agency', 'he', 'saw', 'the', 'plane', 'on', 'fire', 'in', 'mid-air', '.'], ['according', 'to', 'the', 'report', ',', 'average', 'life', 'expectancy', 'has', 'increased', 'by', 'nearly', 'two', 'years', 'since', '1990', '.'], ['republican', 'national', 'committee', 'chairman', 'ed', 'gillespie', 'issued', 'a', 'letter', 'friday', 'to', 'cbs', 'television', 'president', 'leslie', 'moonves', '.'], ['it', 'is', 'the', 'nation', "'s", 'first', 'large-scale', 'counterterrorism', 'exercise', 'since', 'the', 'sept', '.', '11', 'terrorist', 'attacks', '.'], ['ruffner', ',', '45', ',', 'does', 'not', 'have', 'a', 'lawyer', 'on', 'the', 'murder', 'charge', ',', 'authorities', 'said', '.'], ['acting', 'new', 'haven', 'police', 'chief', 'francisco', 'ortiz', 'said', 'police', 'were', 'treating', 'the', 'explosion', 'as', 'a', 'criminal', 'matter', '.'], ['gov.', 'linda', 'lingle', 'and', 'other', 'dignitaries', 'are', 'scheduled', 'to', 'be', 'at', 'the', 'base', 'to', 'watch', 'the', 'launch', '.'], ['toll', 'last', 'week', 'offered', 'to', 'buy', 'the', 'company', 'for', 'nz75c', 'a', 'share', ',', 'or', '$', 'nz158', 'million', '.'], ['still', ',', 'pinellas', 'rabies', 'cases', 'fell', 'sharply', 'to', '17', 'in', '1996', 'and', '3', 'in', '1997', ',', 'agnew', 'said', '.'], ['armstrong', 'county', 'district', 'attorney', 'scott', 'andreassi', 'said', 'the', 'many', 'family', 'photos', 'in', 'the', 'home', 'did', 'not', 'include', 'kristen', '.'], ['about', '31', 'percent', 'of', 'americans', 'are', 'now', 'obese', '—', 'roughly', '30', 'or', 'more', 'pounds', 'over', 'a', 'healthy', 'weight', '.'], ['trading', 'of', 'echostar', "'s", 'stock', 'closed', 'tuesday', 'at', 'a', '52-week', 'high', 'of', '$', '32.17', ',', 'up', '$', '1.54', '.'], ['the', 'institute', 'for', 'supply', 'management', 'said', 'its', 'index', 'of', 'non-manufacturing', 'activity', 'rose', 'to', '50.7', 'from', '47.9', 'in', 'march', '.'], ['ms', 'cohen', 'alon', "'s", 'lawyer', ',', 'donald', 'levine', ',', 'confirmed', 'she', 'tried', 'to', 'sell', 'the', 'story', 'in', 'february', '.'], ['shares', 'in', 'microsoft', 'slipped', '4.7', 'percent', 'in', 'after-hours', 'trade', 'to', '$', '27.54', 'from', 'a', 'nasdaq', 'close', 'of', '$', '28.91', '.'], ['with', '4.7', 'million', 'people', 'infected', 'with', 'hiv', 'or', 'aids', ',', 'south', 'africa', 'has', 'the', 'world', "'s", 'highest', 'aids', 'caseload', '.'], ['the', 'broad', 'standard', '&', 'poor', "'s", '500', 'index', '.spx', 'inched', 'up', '3', 'points', ',', 'or', '0.32', 'percent', ',', 'to', '970', '.'], ['half', 'the', 'women', 'received', 'a', 'daily', 'tablet', 'of', 'the', 'combination', 'oestrogen', 'plus', 'progestin', 'while', 'the', 'rest', 'were', 'given', 'a', 'placebo', '.'], ['hillary', 'clinton', ',', 'the', 'new', 'york', 'senator', 'and', 'former', 'first', 'lady', ',', 'has', 'a', 'book', 'out', 'monday', 'titled', 'living', 'history', '.'], ['in', '2001', ',', 'president', 'bush', 'named', 'her', 'to', 'the', 'student', 'loan', 'marketing', 'association', ',', 'the', 'largest', 'u.s.', 'lender', 'for', 'students', '.'], ['in', 'a', '6-3', 'ruling', ',', 'the', 'justices', 'said', 'such', 'anti-psychotic', 'drugs', 'can', 'be', 'used', 'only', 'in', '"', 'limited', 'circumstances', '.', '"'], ['frist', 'proposed', 'a', 'process', 'in', 'which', 'it', 'would', 'take', 'gradually', 'fewer', 'votes', 'to', 'overcome', 'filibusters', 'preventing', 'final', 'votes', 'on', 'judicial', 'confirmations', '.'], ['the', 'state', 'wants', 'to', 'kill', 'the', 'wolves', 'in', 'approximately', 'a', '1,700-square-mile', 'area', 'near', 'mcgrath', 'to', 'establish', 'a', 'moose', 'nursery', 'of', 'sorts', '.'], ['the', 'unemployment', 'rate', 'in', 'san', 'joaquin', 'county', 'dipped', 'last', 'month', 'to', '8.5', 'percent', ',', 'down', 'nearly', 'a', 'full', 'percentage', 'point', 'from', 'august', '.'], ['the', 'head', 'of', 'the', 'local', 'disaster', 'unit', ',', 'gyorgy', 'heizler', ',', 'said', 'the', 'coach', 'driver', 'had', 'failed', 'to', 'heed', 'red', 'stop', 'lights', '.'], ['"', 'we', 'still', 'have', 'a', 'lot', 'to', 'do', ',', 'but', 'i', 'feel', 'confident', 'that', 'hp', 'is', 'no', 'longer', 'an', 'integration', 'story', '.', '"'], ['they', 'are', 'actually', 'going', 'to', 'record', 'sales', 'for', 'velcade', 'this', 'year', ',', '"', 'said', 'matt', 'geller', ',', 'an', 'analyst', 'at', 'cibc', 'world', 'markets', '.'], ['a', 'new', 'beauty', 'contest', 'kicking', 'off', 'in', 'italy', 'next', 'week', 'will', 'give', 'pixel-perfect', 'pin-ups', 'the', 'chance', 'to', 'steal', 'sultry', 'sophia', "'s", 'sex-symbol', 'status', '.'], ['it', 'said', 'the', 'patents', 'were', '"', 'allegedly', '"', 'assigned', 'to', 'syndia', 'in', '2001', 'through', 'the', 'lemelson', 'medical', ',', 'educational', 'and', 'research', 'foundation', 'limited', 'partnership', '.'], ['but', 'the', 'us', 'insisted', 'it', 'wanted', 'more', 'done', 'to', 'prevent', 'complaints', 'being', 'filed', 'in', 'the', 'first', 'place', ',', 'preferably', 'by', 'repealing', 'the', 'entire', 'law', '.'], ['hoy', 'confirmed', 'the', 'woman', "'s", 'age', 'tuesday', 'and', 'said', 'she', 'was', 'on', 'vacation', 'with', 'her', 'family', ',', 'but', 'is', 'expected', 'to', 'return', 'this', 'week', '.'], ['when', 'the', 'prime', 'minister', "'s", 'position', 'was', 'established', 'in', 'the', 'spring', ',', 'it', 'offered', 'the', 'united', 'states', 'and', 'israel', 'a', 'way', 'to', 'work', 'around', 'arafat', '.'], ['in', '1999', ',', 'two', 'years', 'before', 'his', 'death', ',', 'mr.', 'nielsen', 'told', 'an', 'interviewer', 'that', 'he', 'thought', 'the', 'rock', 'face', 'would', 'outlive', 'him', 'by', 'many', 'years', '.'], ['the', 'legal', 'ruling', 'follows', 'three', 'days', 'of', 'wild', 'volatility', 'in', 'rim', "'s", 'stock', 'over', 'speculation', 'that', 'pc', 'giant', 'hewlett-packard', 'co.', 'may', 'be', 'bidding', 'for', 'the', 'company', '.'], ['note', 'the', 'subheading', 'of', 'this', 'terrible', 'parable', 'in', 'the', '"', 'american', 'masters', '"', 'series', ',', '"', 'arthur', 'miller', ',', 'elia', 'kazan', 'and', 'the', 'blacklist', ':', 'none', 'without', 'sin', '.', '"'], ['the', 'maine', 'program', 'could', 'be', 'a', 'model', 'for', 'other', 'states', '.'], ['it', 'is', 'the', 'highest', 'reading', 'since', 'the', 'index', 'was', 'created', 'in', 'august', '1997', '.'], ['and', ',', 'globally', ',', 'the', 'number', 'of', 'active', 'probable', 'cases', 'has', 'declined', 'to', '573', '.'], ['on', 'tuesday', ',', 'he', 'picked', 'up', 'the', 'endorsement', 'of', 'the', 'united', 'steelworkers', 'of', 'america', '.'], ['michael', 'schiavo', 'has', 'argued', 'that', 'his', 'wife', 'never', 'wanted', 'to', 'be', 'kept', 'alive', 'artificially', '.'], ['the', 'nasdaq', 'composite', 'index', '.ixic', 'dipped', '8.59', 'points', ',', 'or', '0.48', 'percent', ',', 'to', '1,773.54', '.'], ['the', 'tech-laced', 'nasdaq', 'composite', '.ixic', 'rallied', '30.46', 'points', ',', 'or', '2.04', 'percent', ',', 'to', '1,520.15', '.'], ['heatley', 'led', 'atlanta', 'in', 'scoring', 'last', 'season', 'with', 'a', 'team-record', '41', 'goals', 'and', '48', 'assists', '.'], ['his', 'first', 'father', "'s", 'day', 'as', 'a', 'dad', ',', 'his', 'first', 'major', 'as', 'a', 'champion', '.'], ['the', 'report', 'was', 'found', 'last', 'week', 'tucked', 'inside', 'a', 'training', 'manual', 'that', 'belonged', 'to', 'hicks', '.'], ['costa', 'will', 'play', 'juan', 'carlos', 'ferrero', 'next', 'in', 'a', 'rematch', 'of', 'last', 'year', "'s", 'final', '.'], ['dynes', 'will', 'be', 'paid', '$', '395,000', 'a', 'year', ';', 'atkinson', "'s", 'salary', 'is', '$', '361,400', '.'], ['the', 'flaw', 'involves', 'the', 'remote', 'procedure', 'call', '(', 'rpc', ')', 'protocol', ',', 'which', 'deals', 'with', 'inter-computer', 'communications', '.'], ['the', 'analyst', ',', 'already', 'let', 'go', 'by', 'merrill', ',', 'can', 'request', 'a', 'hearing', 'before', 'a', 'nasd', 'panel', '.'], ['that', 'victim', ',', 'a', 'new', 'jersey', 'resident', ',', 'was', 'in', 'critical', 'but', 'stable', 'condition', 'at', 'the', 'hospital', '.'], ['lendingtree', 'matches', 'borrowers', 'via', 'the', 'internet', 'with', 'more', 'than', '200', 'mortgage', 'brokers', ',', 'banks', 'and', 'other', 'lenders', '.'], ['in', 'the', 'opinion', ',', 'kline', 'noted', 'that', 'it', 'is', 'illegal', 'for', 'children', 'that', 'young', 'to', 'have', 'sexual', 'relations', '.'], ['the', 'third', 'armored', 'cavalry', 'regiment', 'recently', 'apprehended', 'about', '40', 'suspected', 'fighters', 'near', 'the', 'syrian', 'border', ',', 'officials', 'said', '.'], ['more', 'than', '10,000', 'people', ',', 'most', 'of', 'them', 'civilians', ',', 'have', 'been', 'killed', 'in', 'three', 'decades', 'of', 'insurgency', '.'], ['"', 'it', 'looks', 'very', 'much', 'like', 'an', 'isolated', 'event', ',', '"', 'world', 'health', 'organization', 'spokeswoman', 'maria', 'cheng', 'said', '.'], ['a', 'reading', 'above', '50', 'suggests', 'growth', 'in', 'the', 'sector', ',', 'while', 'a', 'number', 'below', 'that', 'level', 'indicates', 'contraction', '.'], ['state', 'interior', 'ministry', 'spokesman', 'vasant', 'pitke', 'told', 'reuters', 'that', 'at', 'least', '42', 'people', 'had', 'been', 'killed', 'and', '112', 'injured', '.'], ['according', 'to', 'the', 'census', 'bureau', ',', 'the', 'hispanic', 'population', 'increased', 'by', '9.8', 'percent', 'from', 'the', 'april', '2000', 'census', 'figures', '.'], ['the', 'appetizer', 'devoured', ',', 'annika', 'sorenstam', 'moves', 'on', 'to', 'the', 'entree', 'this', 'week', 'at', 'the', 'mcdonald', "'s", 'lpga', 'championship', '.'], ['the', 'broad', 'standard', '&', 'poor', "'s", '500', 'index', '.spx', 'was', 'up', '8.79', 'points', ',', 'or', '0.96', 'percent', ',', 'at', '929.06', '.'], ['the', 'broader', 'standard', '&', 'poor', "'s", '500', 'index', '.spx', 'crept', 'up', '4.3', 'points', ',', 'or', '0.44', 'percent', ',', 'to', '980.52', '.'], ['the', 'city', 'had', 'requested', 'federal', 'funds', ',', 'but', 'withdrew', 'that', 'request', 'when', 'american', 'express', 'pledged', 'at', 'least', '$', '3', 'million', '.'], ['the', 'royals', 'chose', 'chris', 'lubanski', ',', 'a', 'high', 'school', 'outfielder', 'from', 'pennsylvania', 'who', 'hit', '.528', ',', 'with', 'the', 'fifth', 'pick', '.'], ['in', 'the', 'seconds', 'after', 'the', 'crash', ',', 'she', 'added', ',', 'captain', 'smith', 'said', 'simply', ',', '"', 'oh', 'my', 'god', '.', '"'], ['murphy', 'said', 'strier', "'s", 'sister', ',', 'ethel', 'celnik', ',', 'was', 'in', 'the', 'courtroom', 'at', 'the', 'time', ',', 'but', 'strier', 'was', 'not', '.'], ['the', 'ireland', 'palestine', 'solidarity', 'campaign', ',', 'of', 'which', 'he', 'was', 'a', 'member', ',', 'said', 'he', 'was', 'researching', 'a', 'schools', 'exchange', 'project', '.'], ['lee', 'boyd', 'malvo', 'will', 'be', 'called', 'as', 'a', 'witness', 'wednesday', 'in', 'a', 'pretrial', 'hearing', 'for', 'fellow', 'sniper', 'suspect', 'john', 'allen', 'muhammad', '.'], ['the', 'ftc', 'signed', 'up', '50', 'million', 'phone', 'numbers', 'for', 'the', 'list', ',', 'which', 'was', 'due', 'to', 'go', 'into', 'effect', 'october', '1', '.'], ['mr.', 'mccall', ',', 'who', 'ran', 'unsuccessfully', 'for', 'governor', 'of', 'new', 'york', 'in', '2002', ',', 'styled', 'himself', 'an', 'advocate', 'of', 'better', 'corporate', 'governance', '.'], ['in', 'a', 'statement', 'issued', 'by', 'his', 'solicitors', ',', 'mr', 'hadley', 'said', ':', '"', 'i', 'am', 'absolutely', 'delighted', 'with', 'the', 'outcome', 'of', 'proceedings', '.'], ['devries', ',', 'who', 'was', 'voluntarily', 'castrated', 'in', 'august', '2001', ',', 'has', 'said', 'the', 'surgery', 'took', 'away', 'his', 'ability', 'to', 'become', 'sexually', 'aroused', '.'], ['"', 'i', 'am', 'being', 'forced', 'into', 'exile', 'by', 'the', 'world', "'s", 'superpower', ',', '"', 'taylor', 'said', 'in', 'an', 'address', 'videotaped', 'at', 'his', 'home', '.'], ['a', 'total', 'of', '17', 'confirmed', 'cases', 'of', 'cholera', 'were', 'reported', 'yesterday', 'by', 'the', 'world', 'health', 'organisation', 'in', 'the', 'southern', 'iraqi', 'city', 'of', 'basra', '.'], ['near', 'a', 'mosque', 'in', 'the', 'northern', 'town', 'of', 'mosul', ',', 'rebel', 'gunmen', 'also', 'assassinated', 'a', 'sunni', 'muslim', 'tribal', 'leader', 'who', 'backed', 'the', 'coalition', '.'], ['a', 'passionate', 'nationalist', ',', 'mr.', 'd', "'alessandro", 'said', ':', '"', 'canada', 'is', 'a', 'small', 'country', ',', 'and', 'we', 'need', 'more', 'companies', 'like', 'this', '.', '"'], ['sandy', 'mitchell', ',', '44', ',', 'from', 'kirkintilloch', ',', 'glasgow', ',', 'and', 'william', 'sampson', ',', 'a', 'british', 'citizen', 'born', 'in', 'glasgow', ',', 'face', 'beheading', 'in', 'public', '.'], ['manuel', 'and', 'his', 'group', 'entered', 'the', 'us', 'the', 'same', 'way', 'thousands', 'of', 'people', 'do', 'every', 'day', ',', 'by', 'walking', 'across', 'the', 'bridge', 'between', 'matamoros', 'and', 'brownsville', '.'], ['but', 'mr.', 'crosby', 'told', 'the', 'associated', 'press', ':', '"', 'right', 'from', 'the', 'beginning', ',', 'we', 'didn', "'t", 'want', 'to', 'see', 'anyone', 'take', 'a', 'cut', 'in', 'pay', '.'], ['the', 'swedish', 'central', 'bank', ',', 'also', 'meeting', 'on', 'wednesday', ',', 'is', 'widely', 'expected', 'to', 'announce', 'a', 'cut', 'on', 'thursday', 'of', 'as', 'much', 'as', 'half', 'a', 'percentage', 'point', '.'], ['graham', ',', 'who', 'co-chaired', 'the', 'inquiry', ',', 'is', 'a', 'presidential', 'candidate', '.'], ['they', 'cover', 'more', 'than', '300,000', 'uaw', 'workers', 'and', '500,000', 'retirees', 'and', 'spouses', '.'], ['still', ',', 'mr.', 'weill', 'will', 'remain', 'very', 'much', 'a', 'power', 'at', 'the', 'company', '.'], ['operating', 'revenues', 'rose', 'to', '$', '1.45', 'billion', 'from', '$', '1.38', 'billion', 'last', 'year', '.'], ['the', 'stock', 'rose', '48', 'cents', 'to', '$', '30', 'yesterday', 'in', 'nasdaq', 'stock', 'market', 'trading', '.'], ['harris', 'and', 'klebold', 'killed', '12', 'students', ',', 'a', 'teacher', 'and', 'themselves', 'at', 'the', 'school', '.'], ['none', 'of', 'the', 'casualties', 'was', 'americans', ',', 'said', 'capt.', 'michael', 'calvert', ',', 'regiment', 'spokesman', '.'], ['the', 'nasdaq', 'composite', 'index', '.ixic', 'jumped', '43.64', 'points', ',', 'or', '2.89', 'percent', ',', 'to', '1,553.73', '.'], ['production', 'for', 'the', 'second', 'quarter', 'was', 'cut', 'to', '46', 'million', 'barrels', 'from', '48', 'million', 'barrels', '.'], ['the', 'nasdaq', 'composite', 'index', '.ixic', 'was', 'off', '6.52', 'points', ',', 'or', '0.39', 'percent', ',', 'at', '1,645.66', '.'], ['he', 'had', 'been', 'in', 'protective', 'custody', 'since', 'being', 'transferred', 'to', 'souza-baranowski', 'in', 'april', ',', 'officials', 'said', '.'], ['the', 'study', 'also', 'found', 'that', 'skin', 'cancer', 'nearly', 'tripled', 'in', 'norway', 'and', 'sweden', 'since', 'the', '1950s', '.'], ['license', 'sales', ',', 'a', 'key', 'measure', 'of', 'demand', ',', 'fell', '21', 'percent', 'to', '$', '107.6', 'million', '.'], ['the', 'technology-laced', 'nasdaq', 'composite', 'index', '<', '.ixic', '>', 'rose', '6', 'points', ',', 'or', '0.41', 'percent', ',', 'to', '1,498', '.'], ['the', 'technology-laced', 'nasdaq', 'composite', 'index', '<', '.ixic', '>', 'added', '1.92', 'points', ',', 'or', '0.12', 'percent', ',', 'at', '1,647.94', '.'], ['the', '25', 'percent', 'hike', 'takes', 'annual', 'uc', 'undergraduate', 'tuition', 'to', '$', '4,794', 'and', 'graduate', 'fees', 'to', '$', '5,019', '.'], ['researchers', 'found', 'that', 'sea', 'ice', 'in', 'the', 'arctic', 'is', 'disappearing', 'at', 'a', 'rate', 'of', '9', 'percent', 'each', 'decade', '.'], ['he', 'added', 'that', 'genes', 'on', 'the', 'y', 'might', 'play', 'a', 'role', 'in', 'influencing', 'gender-specific', 'differences', 'in', 'disease', 'susceptibility', '.'], ['his', 'sport', 'utility', 'vehicle', 'was', 'found', 'june', '25', ',', 'abandoned', 'without', 'its', 'license', 'plates', 'in', 'virginia', 'beach', ',', 'va', '.'], ['but', 'butterflies', 'housed', 'under', 'an', 'earlier', 'light', 'cycle', ',', '1', 'a.m.', 'to', '1', 'p.m.', ',', 'flew', 'toward', 'the', 'southeast', '.'], ['but', 'american', 'troops', 'will', 'not', 'shrink', 'from', 'mounting', 'raids', 'in', 'the', 'locations', 'of', 'their', 'foes', 'that', 'can', 'be', 'pinpointed', '.'], ['the', 'broader', 'standard', '&', 'poor', "'s", '500', 'index', '.spx', 'was', '0.46', 'points', 'lower', ',', 'or', '0.05', 'percent', ',', 'at', '997.02', '.'], ['as', 'of', 'this', 'week', ',', '103', 'human', 'west', 'nile', 'cases', 'in', '45', 'counties', 'had', 'been', 'reported', 'to', 'the', 'health', 'department', '.'], ['instead', ',', 'they', 'fear', 'the', 'conference', 'is', 'an', 'attempt', 'by', 'corporate', 'farming', 'and', 'biotech', 'interests', 'to', 'push', 'into', 'new', 'markets', '.'], ['the', '30-year', 'bond', 'us30yt', '=', 'rr', 'lost', '16', '/', '32', ',', 'taking', 'its', 'yield', 'to', '4.20', 'percent', 'from', '4.18', 'percent', '.'], ['investigators', 'continue', 'to', 'probe', 'virtually', 'every', 'aspect', 'of', 'campbell', "'s", 'personal', 'and', 'financial', 'affairs', ',', 'the', 'former', 'mayor', 'has', 'said', '.'], ['as', 'of', 'yesterday', ',', 'the', 'total', 'number', 'of', 'national', 'guard', 'and', 'reserve', 'troops', 'called', 'to', 'duty', 'worldwide', 'stood', 'at', '154,603', '.'], ['aaa', 'spokesman', 'jerry', 'cheske', 'said', 'prices', 'might', 'have', 'affected', 'some', 'plans', ',', 'but', 'cheap', 'hotel', 'deals', 'made', 'up', 'for', 'it', '.'], ['carnival', 'corp.', 'stock', 'was', 'down', '2.5', 'percent', ',', 'or', '81', 'cents', ',', 'at', '$', '31.20', 'on', 'the', 'new', 'york', 'stock', 'exchange', '.'], ['jay', 'fishman', ',', '51', ',', 'chairman', 'and', 'chief', 'executive', 'of', 'st', 'paul', ',', 'will', 'be', 'chief', 'executive', 'of', 'the', 'combined', 'company', '.'], ['analysts', 'polled', 'by', 'reuters', 'research', ',', 'a', 'unit', 'of', 'reuters', 'group', 'plc', ',', 'on', 'average', 'forecast', 'profit', 'of', '$', '1.69', 'per', 'share', '.'], ['lendingtree', 'shares', 'rose', '43', 'percent', ',', 'or', '$', '6.31', ',', 'to', '$', '21', ',', 'more', 'than', 'doubling', 'in', 'the', 'last', 'two', 'months', '.'], ['there', 'is', ',', 'however', ',', 'no', 'photograph', 'of', 'dr', 'hollingworth', 'in', 'the', 'june', 'issue', 'of', 'the', 'magazine', 'examined', 'by', 'the', 'age', 'yesterday', '.'], ['the', 'technology-focused', 'nasdaq', 'composite', 'index', '<', '.ixic', '>', 'advanced', '6', 'points', ',', 'or', '0.30', 'percent', ',', 'to', '2,053', ',', 'erasing', 'earlier', 'losses', '.'], ['"', 'the', 'future', 'of', 'ba', 'is', 'in', 'the', 'hands', 'of', 'the', 'negotiators', ',', '"', 'eddington', 'told', 'british', 'broadcasting', 'corp.', 'television', 'on', 'sunday', '.'], ['doctors', 'said', 'that', 'one', 'or', 'both', 'of', 'the', 'boys', 'may', 'die', 'and', 'that', 'if', 'they', 'survive', ',', 'some', 'brain', 'damage', 'is', 'possible', '.'], ['"', 'they', 'ransacked', 'and', 'burned', 'the', 'two', 'villages', 'completely', ',', '"', 'rebel', 'commander', 'ousmane', 'coulibaly', 'told', 'reuters', 'by', 'satellite', 'phone', 'from', 'the', 'bush', '.'], ['peterson', 'told', 'police', 'he', 'left', 'his', 'wife', 'at', 'about', '9', ':', '30', 'a.m.', 'on', 'dec.', '24', 'to', 'fish', 'alone', 'in', 'san', 'francisco', 'bay', '.'], ['the', 'transaction', 'will', 'expand', 'callebaut', "'s", 'sales', 'revenues', 'from', 'its', 'consumer', 'products', 'business', 'by', 'around', '45', 'percent', 'to', 'some', 'one-third', 'of', 'total', 'sales', '.'], ['under', 'the', 'program', ',', 'us', 'officials', 'work', 'with', 'foreign', 'port', 'authorities', 'to', 'seek', 'out', 'high-risk', 'cargo', 'before', 'it', 'can', 'reach', 'the', 'united', 'states', '.'], ['sammy', 'sosa', 'was', 'suspended', 'for', 'eight', 'games', 'by', 'major', 'league', 'baseball', 'friday', 'for', 'using', 'a', 'corked', 'bat', ',', 'and', 'he', 'immediately', 'appealed', 'the', 'decision', '.'], ['the', 'plan', ',', 'called', 'the', 'nih', 'roadmap', ',', 'was', 'developed', 'over', '14', 'months', 'with', 'help', 'from', 'more', 'than', '300', 'consultants', 'in', 'industry', 'and', 'academia', '.'], ['"', 'i', 'know', 'of', 'nobody', 'who', 'pressured', 'anybody', ',', '"', 'douglas', 'feith', ',', 'undersecretary', 'of', 'defense', 'for', 'policy', ',', 'said', 'at', 'a', 'pentagon', 'briefing', '.'], ['in', 'addition', ',', 'the', 'justice', 'department', 'said', 'that', 'the', 'fbi', 'has', 'conducted', "'", "'", 'fewer', 'than', '10', "'", "'", 'investigations', 'involving', 'visits', 'to', 'mosques', '.'], ['hewlett-packard', 'is', 'putting', 'in', 'for', 'a', 'second-quarter', 'profit', 'of', '$', '659', 'million', ',', 'or', '29', 'cents', 'a', 'share', ',', 'on', 'revenue', 'of', '$', '18', 'billion', '.'], ['johnston', 'made', '$', '5.8', 'million', 'in', 'salary', 'and', 'bonus', 'in', '2001', 'as', 'president', ',', 'and', 'as', 'a', 'part-time', 'adviser', 'earned', '$', '1', 'million', 'in', '2002', '.'], ['australia', "'s", '9', 'million', 'income', 'taxpayers', 'received', 'surprise', 'tax', 'cuts', 'of', 'between', '$', '3', 'and', '$', '11', 'a', 'week', 'from', 'treasurer', 'peter', 'costello', 'last', 'night', '.'], ['the', '21-year-old', 'eagle', ',', 'found', 'by', 'a', 'zookeeper', 'early', 'thursday', ',', 'had', 'severe', 'puncture', 'wounds', 'to', 'his', 'abdomen', 'and', 'back', ',', 'spokeswoman', 'julie', 'mason', 'said', '.'], ['convicted', 'child', 'molester', 'brian', 'devries', 'spoke', 'out', 'in', 'response', 'to', 'community', 'outrage', 'wednesday', 'afternoon', 'after', 'registering', 'as', 'a', 'sex', 'offender', 'at', 'the', 'soledad', 'police', 'department', '.'], ['july', '10', '-', 'the', 'bush', 'administration', "'s", 'latest', 'effort', 'to', 'expand', 'the', 'role', 'of', 'religious', 'organizations', 'in', 'government', 'services', 'enlists', 'church-based', 'youth', 'groups', 'in', 'anti-drug', 'programs', '.'], ['in', 'vienna', ',', 'the', 'international', 'atomic', 'energy', 'agency', 'said', 'on', 'monday', 'elbaradei', 'had', 'accepted', 'iran', "'s", 'invitation', ',', 'but', 'said', 'no', 'date', 'had', 'yet', 'been', 'set', '.'], ['and', 'if', 'that', 'ain', "'t", 'a', 'democrat', ',', 'then', 'i', 'must', 'be', 'in', 'the', 'wrong', 'meeting', ',', '"', 'he', 'said', 'to', 'thunderous', 'applause', 'from', 'his', 'supporters', '.'], ['both', 'rebels', 'and', 'government', 'forces', ',', 'including', 'those', 'loyald', 'to', 'taylor', ',', 'have', 'been', 'accused', 'of', 'pillaging', 'villages', 'in', 'liberia', "'s", 'countryside', 'despite', 'a', 'peace', 'deal', '.'], ['black', 'democratic', 'leaders', 'were', 'trying', 'to', 'arrange', 'a', 'meeting', 'with', 'democratic', 'national', 'committee', 'chairman', 'terry', 'mcauliffe', 'to', 'discuss', 'the', 'layoffs', 'of', '10', 'minority', 'staffers', 'at', 'party', 'headquarters', '.'], ['just', 'five', 'months', 'ago', ',', 'dean', 'committed', 'to', 'accepting', 'taxpayer', 'money', 'and', 'the', 'spending', 'limits', 'that', 'come', 'with', 'it', 'and', 'vowed', 'to', 'attack', 'any', 'democrat', 'who', 'didnt', '.'], ['of', 'those', 'three', ',', 'only', 'dillard', "'s", 'responded', 'to', 'the', 'survey', 'that', 'ranked', '45', 'large', 'companies', 'on', 'employment', ',', 'marketing', ',', 'procurement', ',', 'community', 'reinvestment', 'and', 'charitable', 'donations', '.'], ['when', 'his', 'grandmother', ',', 'then-princess', 'elizabeth', ',', 'turned', '21', 'in', '1947', ',', 'the', 'day', 'was', 'declared', 'a', 'public', 'holiday', ',', 'and', 'she', 'attended', 'a', 'reception', 'and', 'a', 'ball', '.'], ['although', 'obesity', 'can', 'increase', 'the', 'risk', 'of', 'a', 'host', 'of', 'health', 'problems', ',', 'skeptics', 'argue', ',', 'so', 'do', 'smoking', 'and', 'high', 'cholesterol', ',', 'which', 'are', 'not', 'considered', 'diseases', '.'], ['the', 'dollar', 'pushed', 'as', 'high', 'as', '$', '1.1115', 'to', 'the', 'euro', 'in', 'early', 'trade', ',', 'extending', 'tuesday', "'s", 'one', 'percent', 'rally', 'to', 'hit', 'its', 'strongest', 'level', 'since', 'april', '30', '.'], ['the', 'protesters', 'denounced', 'the', 'country', "'s", 'supreme', 'leader', ',', 'hard-liner', 'ayatollah', 'ali', 'khamenei', '.'], ['grant', 'thornton', 'claims', 'that', 'it', 'too', 'was', 'the', '“', 'victim', '”', 'of', 'a', 'fraud', '.'], ['general', 'sattler', 'heads', 'a', 'combined', 'joint', 'task', 'force', 'aboard', 'the', 'command', 'ship', 'mount', 'whitney', '.'], ['united', 'has', 'paid', '$', '34', 'million', 'in', 'penalties', 'for', 'failing', 'to', 'meet', 'employment', 'targets', '.'], ['it', 'has', 'a', 'chequered', 'safety', 'record', ',', 'including', '47', 'accidents', 'that', 'resulted', 'in', '668', 'deaths', '.'], ['still', ',', 'singapore', 'quarantined', '70', 'people', 'who', 'had', 'been', 'in', 'close', 'contact', 'with', 'the', 'scientist', '.'], ['shinseki', "'s", 'career', 'included', 'combat', 'in', 'vietnam', 'and', 'head', 'of', 'u.s.', 'peacekeeping', 'efforts', 'in', 'bosnia', '.'], ['the', 'unemployment', 'rate', 'is', 'predicted', 'to', 'have', 'ticked', 'up', 'a', 'percentage', 'point', 'to', '6.1', '%', '.'], ['mony', 'shares', 'rose', '8.76', 'per', 'cent', 'to', '$', '31.90', 'in', 'after-hours', 'trading', 'in', 'new', 'york', '.'], ['rolfe', 'said', 'linksys', 'overtook', 'cisco', 'systems', 'last', 'year', 'as', 'the', 'leading', 'supplier', 'of', 'wlan', 'equipment', '.'], ['jane', 'doe', ',', 'deciding', 'to', 'fight', 'the', 'subpoena', ',', 'asked', 'verizon', 'to', 'withhold', 'her', 'name', '.'], ['u.s.', 'district', 'judge', 'william', 'barbour', 'said', 'he', 'imposed', 'the', 'maximum', 'because', 'avants', 'showed', 'no', 'remorse', '.'], ['fatalities', 'in', 'rollover', 'crashes', 'accounted', 'for', '82', 'percent', 'of', 'the', 'increase', 'in', '2002', ',', 'nhtsa', 'said', '.'], ['the', 'dow', 'jones', 'industrials', 'briefly', 'surpassed', 'the', '9,000', 'mark', 'for', 'the', 'first', 'time', 'since', 'december', '.', '"'], ['five', 'alternate', 'jurors', 'also', 'were', 'selected', ',', 'with', 'a', 'sixth', 'alternate', 'to', 'be', 'picked', 'on', 'friday', '.'], ['maj-gen', 'jeffery', 'said', 'he', 'would', 'give', 'his', 'military', 'pension', 'to', 'charity', 'while', 'he', 'served', 'at', 'yarralumla', '.'], ['the', 'other', 'victim', ',', 'michael', 'walker', ',', 'of', '550', 'barbey', 'street', ',', 'was', 'struck', 'in', 'the', 'neck', '.'], ['schools', 'that', 'don', "'t", 'meet', 'the', 'testing', 'goals', 'for', 'two', 'years', 'in', 'a', 'row', 'must', 'offer', 'transfers', '.'], ['she', 'won', 'the', 'talent', 'portion', 'singing', '"', 'if', 'i', 'could', '"', 'and', 'also', 'won', 'in', 'evening', 'wear', '.'], ['"', 'microsoft', 'had', 'absolutely', 'nothing', 'to', 'do', 'with', 'atstake', "'s", 'internal', 'personnel', 'decision', ',', '"', 'sundwall', 'said', '.'], ['shares', 'of', 'coke', 'were', 'down', '26', 'cents', 'to', 'close', 'at', '$', '44.10', 'on', 'the', 'new', 'york', 'stock', 'exchange', '.'], ['"', 'the', 'other', 'guy', ',', 'he', "'s", 'ashamed', 'of', 'his', 'party', ',', 'and', 'i', 'don', "'t", 'blame', 'him', '.'], ['the', '30-year', 'bond', 'us30yt', '=', 'rr', 'dipped', '14', '/', '32', 'for', 'a', 'yield', 'of', '4.26', 'percent', 'from', '4.23', 'percent', '.'], ['shares', 'of', 'gillette', 'closed', 'down', '45', 'cents', 'at', '$', '33.70', 'in', 'trading', 'wednesday', 'on', 'the', 'new', 'york', 'stock', 'exchange', '.'], ['the', 'bulk', 'of', 'the', 'bill', '-', '$', '64.7', 'billion', '-', 'goes', 'for', 'military', 'operations', 'primarily', 'in', 'iraq', 'and', 'afghanistan', '.'], ['u.s.', 'agriculture', 'secretary', 'ann', 'veneman', 'kicks', 'off', 'the', 'three-day', 'ministerial', 'conference', 'and', 'expo', 'on', 'agricultural', 'science', 'and', 'technology', 'on', 'monday', '.'], ['supporters', 'asked', 'a', 'federal', 'court', 'monday', 'to', 'block', 'the', 'removal', 'of', 'a', 'ten', 'commandments', 'monument', 'from', 'the', 'alabama', 'judicial', 'building', '.'], ['in', 'afternoon', 'trading', ',', 'aspen', "'s", 'shares', 'were', 'off', '89', 'cents', 'or', 'more', 'than', '27', 'percent', 'at', '$', '2.33', 'per', 'share', '.'], ['last', 'week', ',', 'his', 'lawyers', 'asked', 'warner', 'to', 'grant', 'clemency', 'under', 'certain', 'conditions', 'that', 'would', 'have', 'lead', 'to', 'a', 'new', 'sentencing', 'hearing', '.'], ['labor', 'department', 'analysts', 'discounted', 'a', 'slight', 'improvement', 'in', 'the', 'national', 'unemployment', 'rate', ',', 'which', 'fell', 'in', 'august', 'to', '6.1', 'percent', 'from', '6.2', 'percent', '.'], ['the', 'who', 'has', 'said', 'the', 'singapore', 'case', 'did', 'not', 'fit', 'its', 'profile', 'of', 'sars', 'and', 'was', '"', 'not', 'an', 'international', 'public', 'health', 'concern', '.', '"'], ['it', 'upheld', 'fallback', 'rules', 'that', 'bar', 'the', 'same', 'groups', 'from', 'airing', 'ads', 'that', 'promote', ',', 'support', ',', 'attack', 'or', 'oppose', 'a', 'candidate', 'at', 'any', 'time', '.'], ['although', 'a', 'small', 'city-state', ',', 'singapore', 'is', 'the', '12th-largest', 'trading', 'partner', 'of', 'the', 'united', 'states', ',', 'with', 'trade', 'volume', 'of', '$', '33.4', 'billion', 'last', 'year', '.'], ['detective', 'chief', 'inspector', 'norman', 'mckinlay', ',', 'leading', 'the', 'investigation', ',', 'said', ':', '"', 'there', 'is', 'evidence', 'a', 'body', 'or', 'bodies', 'have', 'been', 'in', 'this', 'area', '.'], ['some', 'women', 'also', 'express', 'concern', 'about', 'a', 'society', 'in', 'which', 'more', 'people', 'than', 'ever', 'are', 'turning', 'to', 'cosmetic', 'surgery', 'in', 'a', 'quest', 'for', 'bodily', 'perfection', '.'], ['"', 'we', 'don', "'t", 'know', 'everything', '[', 'but', ']', 'what', 'we', 'know', 'suggests', 'that', 'we', 'should', 'take', 'what', 'they', "'re", 'saying', 'seriously', ',', '"', 'he', 'said', '.'], ['according', 'to', 'the', 'state', 'department', 'list', ',', 'about', '25', 'nations', 'have', 'signed', 'bilateral', 'agreements', 'in', 'the', 'past', 'four', 'months', ',', 'about', 'half', 'in', 'the', 'past', 'three', 'weeks', '.'], ['"', 'at', 'first', 'blush', ',', 'it', 'doesn', "'t", 'look', 'like', 'any', 'rules', 'were', 'broken', ',', '"', 'said', 'gen.', 'richard', 'myers', ',', 'chairman', 'of', 'the', 'joint', 'chiefs', 'of', 'staff', '.'], ['texas', 'lawmakers', 'must', 'close', 'a', '$', '185.9', 'million', 'budget', 'gap', 'by', 'the', 'middle', 'of', 'july', 'or', 'the', 'schools', 'wont', 'open', 'in', 'september', ',', 'comptroller', 'carole', 'keeton', 'strayhorn', 'said', 'thursday', '.'], ['and', 'it', 'can', 'store', 'more', 'than', 'a', 'gigabyte', 'of', 'information', '-', 'equivalent', 'to', '1,000', 'high', 'quality', 'images', 'or', 'around', '12', 'hours', 'of', 'music', '-', 'in', 'just', 'one', 'cubic', 'centimetre', '.'], ['razali', 'ismail', 'met', 'for', '90', 'minutes', 'with', 'suu', 'kyi', ',', 'a', '1991', 'winner', 'of', 'the', 'nobel', 'peace', 'prize', ',', 'at', 'her', 'lakeside', 'home', ',', 'where', 'she', 'is', 'under', 'house', 'arrest', '.'], ['she', 'told', 'the', 'inquiry', 'that', 'her', 'husband', 'had', 'received', 'assurances', 'from', 'his', 'line', 'manager', 'and', 'senior', 'ministry', 'officials', 'that', 'when', 'he', 'came', 'forward', 'his', 'name', 'would', 'not', 'be', 'made', 'public', '.'], ['claudette', ',', 'the', 'first', 'hurricane', 'of', 'the', 'atlantic', 'storm', 'season', ',', 'hit', 'the', 'mid-texas', 'coast', 'july', '15', ',', 'classified', 'as', 'a', 'category', '1', 'hurricane', 'with', 'maximum', 'sustained', 'winds', 'of', '85', 'mph', '.'], ['the', 'free', 'service', 'was', 'originally', 'intended', 'to', 'block', 'about', '80', 'percent', 'of', 'telemarketer', 'calls', '.'], ['eds', 'reported', 'a', 'first-quarter', 'loss', 'of', '$', '126', 'million', ',', 'or', '26', 'cents', 'per', 'share', '.'], ['astronomers', 'have', 'clocked', 'eight', 'more', 'moons', 'orbiting', 'jupiter', ',', 'bringing', 'its', 'known', 'total', 'to', '60', '.'], ['outside', 'the', 'court', ',', 'sriyanto', 'denied', 'to', 'the', 'age', 'that', 'he', 'had', 'done', 'anything', 'wrong', '.'], ['axelrod', 'died', 'in', 'his', 'sleep', 'of', 'heart', 'failure', ',', 'said', 'his', 'daughter', ',', 'nina', 'axelrod', '.'], ['dynes', 'will', 'be', 'paid', '$', '395,000', 'a', 'year', ';', 'atkinson', "'s", 'salary', 'is', '$', '361,400', '.'], ['the', 'technology-laced', 'nasdaq', 'composite', 'index', 'was', 'down', '25.36', 'points', ',', 'or', '1.53', 'percent', ',', 'at', '1,628.26', '.'], ['the', 'technology-laced', 'nasdaq', 'composite', 'index', '.ixic', 'eased', '8.52', 'points', ',', 'or', '0.51', 'percent', ',', 'to', '1,670.21', '.'], ['with', 'all', 'precincts', 'reporting', ',', 'fletcher', 'had', '88,747', 'votes', ',', 'or', '57', 'percent', 'of', 'the', 'total', '.'], ['businessweek', 'online', 'has', 'learned', 'that', 'the', 'settlement', 'could', 'come', 'as', 'early', 'as', 'monday', ',', 'may', '19', '.'], ['national', 'constitution', 'center', 'president', 'joseph', 'torsella', 'was', 'hit', 'in', 'the', 'head', 'and', 'knocked', 'to', 'his', 'knees', '.'], ['"', 'i', 'think', 'there', "'s", 'a', 'building', 'confidence', 'that', 'the', 'cop', 'is', 'on', 'the', 'beat', '.', '"'], ['only', '29', 'families', 'of', 'frogs', 'are', 'known', ',', 'most', 'of', 'which', 'were', 'named', 'by', 'the', 'mid-1800s', '.'], ['cisco', 'has', 'similar', 'relationships', 'with', 'bellsouth', 'competitors', 'sbc', 'communications', ',', 'at', '&', 't', 'and', 'sprint', 'communications', '.'], ['it', 'is', 'widely', 'expected', 'to', 'give', 'bair', 'formal', 'approval', 'to', 'offer', 'the', 'plane', 'for', 'sale', 'to', 'airlines', '.'], ['the', 'companies', 'said', 'idec', 'holders', 'would', 'own', 'about', '50.5', 'percent', 'of', 'the', 'stock', 'of', 'the', 'combined', 'company', '.'], ['"', 'this', 'attack', 'was', 'wrong', ',', 'and', 'it', 'was', 'dangerous', 'to', 'our', 'goals', ',', '"', 'raymond', 'said', '.'], ['mdc', 'spokesperson', 'paul', 'themba', 'nyathi', 'said', 'the', 'avenues', 'clinic', 'was', 'targeted', 'because', 'it', 'was', 'handling', 'opposition', 'supporters', '.'], ['the', 'estimate', 'of', 'analysts', 'surveyed', 'by', 'thomson', 'first', 'call', 'was', 'for', 'a', 'loss', 'of', '$', '2.75', 'a', 'share', '.'], ['a', 'county', 'firefighter', ',', 'was', 'struck', 'by', 'lightning', 'and', 'was', 'in', 'stable', 'condition', 'at', 'frankfort', 'regional', 'medical', 'center', '.'], ['he', 'was', 'the', 'ace', 'of', 'diamonds', 'in', 'a', 'u.s.', 'deck', 'of', 'cards', 'showing', 'pictures', 'of', 'most-wanted', 'iraqi', 'leaders', '.'], ['after', 'protesters', 'rushed', 'the', 'stage', 'and', 'twice', 'cut', 'power', 'to', 'the', 'microphone', ',', 'hedges', 'cut', 'his', 'speech', 'short', '.'], ['the', 'unions', 'also', 'staged', 'a', 'five-day', 'strike', 'in', 'march', ';', 'strikes', 'have', 'preceded', 'eight', 'of', 'the', 'last', '10', 'contracts', '.'], ['now', 'she', 'has', 'her', 'first', 'trophy', 'after', 'victory', 'in', 'the', 'us', 'women', "'s", 'amateur', 'public', 'links', 'here', 'on', 'sunday', '.'], ['the', 'state', 'university', 'at', 'stony', 'brook', 'launched', 'the', 'study', 'in', '1996', ',', 'after', 'earlier', 'studies', 'indicated', 'a', 'possible', 'connection', '.'], ['the', 'company', 'said', 'it', 'would', 'renew', 'its', 'guidance', 'for', '2003', 'when', 'it', 'announces', 'its', 'second', 'quarter', 'results', 'in', 'mid-july', '.'], ['next', ',', 'baltimore', 'took', 'nick', 'markakis', ',', 'a', 'left-handed', 'pitcher', 'and', 'outfielder', 'from', 'young', 'harris', 'junior', 'college', 'in', 'georgia', '.'], ['statistics', 'from', 'the', 'kansas', 'department', 'of', 'health', 'and', 'environment', 'show', 'that', '11,844', 'abortions', 'were', 'performed', 'in', 'the', 'state', 'last', 'year', '.'], ['last', 'week', ',', 'his', 'lawyers', 'asked', 'gov.', 'mark', 'r.', 'warner', 'to', 'grant', 'clemency', ',', 'but', 'the', 'governor', 'declined', 'to', 'intervene', '.'], ['a', 'seventh', 'person', ',', 'a', '21-year-old', 'woman', ',', 'was', 'in', 'critical', 'condition', 'in', 'the', 'burn', 'unit', 'at', 'akron', 'children', "'s", 'hospital', '.'], ['china', 'is', 'scrambling', 'to', 'stop', 'the', 'disease', 'from', 'spreading', 'to', 'the', 'countryside', ',', 'where', 'most', 'of', 'its', '1.3', 'billion', 'people', 'live', '.'], ['after', 'posting', '$', '3', 'million', 'bail', ',', 'jackson', 'flew', 'to', 'las', 'vegas', ',', 'where', 'he', 'had', 'been', 'working', 'on', 'a', 'video', '.'], ['glaxosmithkline', ',', 'the', 'uk', 'drugmaker', ',', 'has', 'said', 'it', 'would', 'cut', 'off', 'supplies', 'to', 'canadian', 'stores', 'shipping', 'drugs', 'to', 'the', 'us', '.'], ['alhart', 'said', 'the', 'governor', '"', 'remains', 'hopeful', 'that', 'the', 'city', 'will', 'continue', 'to', 'work', 'with', 'us', 'and', 'engage', 'in', 'good-faith', 'discussions', '.', '"'], ['the', 'injured', 'were', 'taken', 'to', 'hospitals', 'in', 'jefferson', 'city', 'and', 'columbia', ',', 'about', '48km', 'north', ';', 'their', 'conditions', 'were', 'not', 'immediately', 'available', '.'], ['hundreds', 'of', 'reporters', 'and', 'photographers', 'swamped', 'the', 'town', 'and', 'the', 'short', 'hearing', 'involving', 'the', 'five-time', 'all-star', 'was', 'carried', 'live', 'on', 'national', 'cable', 'networks', '.'], ['the', 'show', "'s", 'creator', 'and', 'executive', 'producer', ',', 'phil', 'rosenthal', ',', 'quipped', 'in', 'a', 'statement', ':', '"', 'we', 'had', 'a', 'big', 'contract', 'negotiation', '.'], ['previously', ',', 'it', 'had', 'reported', 'a', 'small', 'profit', 'of', '$', '12', 'million', ',', 'or', 'break-even', 'on', 'a', 'per-share', 'basis', ',', 'for', 'the', 'period', '.'], ['"', 'we', 'condemn', 'the', 'governing', 'council', 'headed', 'by', 'the', 'united', 'states', ',', '"', 'sadr', 'said', 'in', 'a', 'fiery', 'sermon', 'at', 'koufa', 'mosque', 'near', 'najaf', '.'], ['daniels', 'said', 'he', 'doesn', "'t", 'know', 'what', 'bush', 'will', 'say', 'at', 'tonight', "'s", '$', '2,000-per-person', 'reception', 'to', 'raise', 'funds', 'for', 'his', 'presidential', 're-election', 'bid', '.'], ['in', 'early', 'trading', ',', 'the', 'dow', 'jones', 'industrial', 'average', 'was', 'down', '39.94', ',', 'or', '0.4', 'percent', ',', 'at', '8,945.50', ',', 'having', 'slipped', '3.61', 'points', 'monday', '.'], ['glover', 'said', 'the', 'killer', 'used', 'a', '``peculiar', 'slipknot', ',', "'", "'", 'including', 'a', 'coaxial', 'cable', 'and', 'an', 'extension', 'cord', ',', 'to', 'strangle', 'some', 'of', 'the', 'victims', '.'], ['the', 'waves', 'of', 'scandal', 'angered', 'not', 'only', 'victims', "'", 'advocates', 'but', 'parishioners', 'and', 'some', 'priests', ',', 'to', 'the', 'point', 'that', 'law', 'could', 'no', 'longer', 'run', 'the', 'archdiocese', '.'], ['the', 'democrat', "'s", '2000', 'platform', 'didn', "'t", 'explicitly', 'support', 'gay', 'marriages', 'but', 'backed', '"', 'the', 'full', 'inclusion', 'of', 'gay', 'and', 'lesbian', 'families', 'into', 'the', 'life', 'of', 'the', 'nation', '.', '"'], ['"', 'he', 'really', 'left', 'us', 'with', 'a', 'smile', 'on', 'his', 'face', 'and', 'no', 'last', 'words', '...', 'he', 'gave', 'us', 'each', 'a', 'kiss', 'and', 'that', 'was', 'it', ',', '"', 'she', 'said', '.'], ['the', 'international', 'committee', 'of', 'the', 'red', 'cross', ',', 'fearful', 'for', 'the', 'safety', 'of', 'its', 'staff', 'operating', 'in', 'iraq', ',', 'announced', 'it', 'was', 'temporarily', 'shutting', 'its', 'offices', 'in', 'baghdad', 'and', 'basra', '.'], ['the', 'civilian', 'unemployment', 'rate', 'improved', 'marginally', 'last', 'month', '_', 'sliding', 'down', 'to', '6.1', 'percent', '_', 'as', 'companies', 'slashed', 'payrolls', 'by', '93,000', 'amid', 'continuing', 'mixed', 'signals', 'about', 'the', 'nation', "'s", 'economic', 'health', '.'], ['it', 'will', 'cost', 'about', '$', '20,000', 'per', 'average', 'course', 'of', 'treatment', '-16', 'to', '17', 'weeks', '.'], ['also', 'at', 'increased', 'risk', 'are', 'those', 'with', 'suppressed', 'immune', 'systems', 'due', 'to', 'illness', 'or', 'medicines', '.'], ['president', 'yoweri', 'museveni', 'has', 'promoted', 'brigadier', 'aronda', 'nyakairima', 'to', 'major', 'general', 'and', 'named', 'him', 'army', 'commander', '.'], ['ryan', 'harvey', ',', 'a', 'high', 'school', 'outfielder', 'from', 'florida', ',', 'was', 'chosen', 'sixth', 'by', 'the', 'cubs', '.'], ['coupling', ',', 'an', 'american', 'version', 'of', 'a', 'hit', 'british', 'comedy', ',', 'will', 'couple', 'with', 'friends', 'on', 'thursdays', '.'], ['the', 'seizure', 'occurred', 'at', '4am', 'on', 'march', '18', ',', 'just', 'hours', 'before', 'the', 'first', 'american', 'air', 'assault', '.'], ['the', 'u.s.', 'house', 'of', 'representatives', ',', 'the', 'other', 'chamber', 'of', 'congress', ',', 'does', 'not', 'vote', 'on', 'treaties', '.'], ['the', 'tech-laced', 'nasdaq', 'composite', 'index', '<', '.ixic', '>', 'gained', '18.35', 'points', ',', 'or', '1.13', 'percent', ',', 'to', '1,646.02', '.'], ['the', 'technology-laced', 'nasdaq', 'composite', 'index', '<', '.ixic', '>', 'jumped', '26', 'points', ',', 'or', '1.78', 'percent', ',', 'to', '1,516', '.'], ['excluding', 'legal', 'fees', 'and', 'other', 'charges', 'it', 'expected', 'a', 'loss', 'of', 'between', '1', 'and', '4', 'cents', 'a', 'share', '.'], ['"', 'we', "'re", 'being', 'hyper-vigilant', ',', '"', 'said', 'dr.', 'james', 'young', ',', 'ontario', "'s", 'commissioner', 'of', 'public', 'safety', '.'], ['"', 'it', "'s", 'a', 'wonder', 'that', 'porchia', 'was', 'the', 'only', 'deceased', 'in', 'this', 'case', ',', '"', 'sax', 'added', '.'], ['among', 'them', 'are', 'conversations', 'between', 'police', ',', 'firefighters', 'and', 'other', 'emergency', 'workers', ',', 'as', 'well', 'as', 'media', 'inquiries', '.'], ['the', 'rock', 'was', 'first', 'observed', 'by', 'the', 'lincoln', 'near', 'earth', 'asteroid', 'research', 'program', ',', 'also', 'known', 'as', 'linear', '.'], ['ross', 'garber', ',', 'rowland', "'s", 'legal', 'counsel', ',', 'said', 'the', 'governor', 'would', 'have', 'no', 'comment', 'on', 'the', 'condo', 'deal', '.'], ['shares', 'of', 'lendingtree', 'rose', '$', '6.21', ',', 'or', '42', 'percent', ',', 'to', '$', '20.90', 'after', 'hitting', '$', '21.36', 'earlier', '.'], ['police', 'detained', 'khodorkovsky', 'early', 'on', 'saturday', 'when', 'his', 'jet', 'made', 'a', 'refueling', 'stop', 'in', 'the', 'siberian', 'city', 'of', 'novosibirsk', '.'], ['moose', "'s", 'book', 'goes', 'on', 'sale', 'monday', ',', 'nearly', 'a', 'year', 'after', 'the', 'sniper', 'shootings', 'started', 'in', 'the', 'washington', 'area', '.'], ['the', 'sale', 'represents', 'about', '52', 'per', 'cent', 'of', 'the', '115.9', 'million', 'shares', 'mr', 'turner', 'held', 'at', 'the', 'end', 'of', 'april', '.'], ['it', "'s", 'also', 'a', 'strategic', 'win', 'for', 'overture', ',', 'given', 'that', 'knight', 'ridder', 'had', 'been', 'using', 'google', "'s", 'advertising', 'services', '.'], ['other', 'nations', 'involved', 'in', 'the', 'psi', 'include', 'germany', ',', 'italy', ',', 'the', 'netherlands', ',', 'poland', ',', 'portugal', ',', 'spain', 'and', 'britain', '.'], ['north', 'american', 'markets', 'grabbed', 'early', 'gains', 'monday', 'morning', ',', 'as', 'earnings', 'season', 'begins', 'to', 'slow', 'and', 'economic', 'indicators', 'take', 'the', 'spotlight', '.'], ['to', 'reach', 'brad', 'cooper', ',', 'johnson', 'county', 'municipal', 'reporter', ',', 'call', '(', '816', ')', '234-7724', 'or', 'send', 'e-mail', 'to', 'bcooper', '@', 'kcstar.com', '.'], ['a', 'nationally', 'board', 'certified', 'teacher', 'with', 'a', 'master', "'s", 'degree', ',', 'kelley', ',', 'in', 'his', '30th', 'year', 'teaching', ',', 'makes', '$', '65,000', '.'], ['several', 'relatives', 'of', 'australian', 'victims', 'of', 'the', 'attack', 'sat', 'in', 'the', 'front', 'row', 'of', 'the', 'court', ',', 'but', 'few', 'balinese', 'attended', 'the', 'trial', '.'], ['"', 'that', 'really', 'affects', 'the', 'companies', ',', 'big', 'time', ',', '"', 'said', 'jim', 'santangelo', ',', 'president', 'of', 'the', 'teamsters', 'joint', 'council', '42', 'in', 'el', 'monte', '.'], ['"', 'i', 'think', 'we', 'can', 'now', 'qualify', 'what', 'is', 'happening', 'as', 'a', 'genuine', 'epidemic', ',', '"', 'health', 'minister', 'jean-francois', 'mattei', 'said', 'on', 'france', 'inter', 'radio', '.'], ['but', 'the', 'justices', 'ruled', 'that', 'the', 'police', 'supervisor', 'who', 'repeatedly', 'questioned', 'mr', 'martinez', 'as', 'he', 'screamed', 'in', 'pain', 'did', 'not', 'violate', 'his', 'fifth', 'amendment', 'rights', '.'], ['under', 'the', 'proposed', 'terms', 'of', 'the', 'transaction', ',', 'and', 'following', 'the', 'spinoff', 'of', 'palmsource', ',', 'palm', 'will', 'exchange', '0.09', 'shares', 'for', 'each', 'share', 'of', 'handspring', '.'], ['the', 'artists', 'say', 'the', 'plan', 'will', 'harm', 'french', 'culture', 'and', 'punish', 'those', 'who', 'need', 'help', 'most', '-', 'performers', 'who', 'have', 'a', 'hard', 'time', 'lining', 'up', 'work', '.'], ['if', 'convicted', ',', 'each', 'faces', 'up', 'to', 'five', 'years', 'in', 'prison', 'and', 'a', 'fine', 'of', '$', '250,000', 'or', 'more', 'based', 'on', 'the', 'amount', 'of', 'gains', 'involved', '.'], ['defense', 'lawyers', 'had', 'requested', 'a', 'change', 'of', 'venue', 'for', 'two', 'reasons', ':', 'they', 'argued', 'that', 'massive', 'pretrial', 'publicity', 'had', 'tainted', 'the', 'jury', 'pool', 'against', 'their', 'client', '.'], ['the', 'department', "'s", 'stance', 'disappointed', 'some', 'abortion', 'opponents', ',', 'and', 'it', 'threatens', 'to', 'alienate', 'social', 'conservatives', 'who', 'have', 'provided', 'strong', 'political', 'support', 'for', 'ashcroft', 'and', 'president', 'bush', '.'], ['ursel', 'reisen', 'confirmed', 'it', 'had', 'been', 'operating', 'the', 'coach', ',', 'but', 'declined', 'to', 'give', 'details', 'of', 'the', 'passengers', 'other', 'than', 'to', 'say', 'they', 'were', 'of', 'mixed', 'ages', '.'], ['when', 'you', 'crossed', 'the', 'line', ',', 'you', 'violated', 'the', 'constitutional', 'right', ',', '"', 'said', 'charles', 'weisselberg', ',', 'who', 'teaches', 'law', 'at', 'the', 'university', 'of', 'california', ',', 'berkeley', '.'], ['"', 'the', 'movement', 'to', 'defend', 'pensions', 'is', 'gaining', 'momentum', ',', 'and', 'we', 'won', "'t", 'give', 'up', ',', '"', 'said', 'marc', 'blondel', ',', 'leader', 'of', 'the', 'force', 'ouvriere', 'union', '.'], ['lee', 'janzen', ',', 'tied', 'atop', 'the', 'leaderboard', 'with', 'five', 'holes', 'left', ',', 'finished', 'with', 'a', '68', 'to', 'tie', 'for', 'sixth', 'with', 'bob', 'crane', '(', '67', ')', 'at', '269', '.'], ['burrell', 'said', 'diana', 'wrote', 'a', 'letter', 'in', 'october', '1996', 'claiming', 'there', 'was', 'a', 'plot', 'to', 'kill', 'her', 'in', 'a', 'smash', 'and', 'gave', 'it', 'to', 'him', 'to', 'keep', 'as', 'insurance', '.'], ['"', 'yes', ',', 'i', 'do', 'not', 'wish', 'to', 'be', 'present', 'during', 'this', 'witness', ',', '"', 'peterson', ',', '31', ',', 'calmly', 'told', 'the', 'judge', 'as', 'he', 'was', 'returned', 'to', 'his', 'cell', '.'], ['the', 'conference', 'board', 'said', 'its', 'measure', 'of', 'business', 'confidence', ',', 'which', 'had', 'fallen', 'to', '53', 'in', 'the', 'first', 'quarter', 'of', '2003', ',', 'improved', 'to', '60', 'in', 'the', 'most', 'recent', 'second', 'quarter', '.'], ['"', 'today', ',', 'we', 'are', 'trying', 'to', 'convey', 'this', 'problem', 'to', 'russian', 'president', 'vladimir', 'putin', '(', 'news', '-', 'web', 'sites', ')', 'and', 'president', 'bush', '(', 'news', '-', 'web', 'sites', ')', '.', '"'], ['a', 'kayaker', 'found', 'dr.', 'scribner', "'s", 'body', 'floating', 'near', 'the', 'doctor', "'s", 'houseboat', 'in', 'portage', 'bay', ',', 'where', 'he', 'was', 'eating', 'lunch', 'when', 'his', 'wife', ',', 'ethel', ',', 'left', 'for', 'an', 'appointment', '.'], ['the', 'church', "'s", 'lawyer', ',', 'shirley', 'phelps-roper', ',', 'was', 'clear', ':', '"', 'what', 'they', 'have', 'done', 'is', 'a', 'thinly', 'veiled', 'attempt', 'to', 'do', 'an', 'end', 'run', 'around', 'the', 'constitution', ',', '"', 'she', 'said', '.'], ['the', 'dow', 'jones', 'industrial', 'average', '.dji', 'rose', '18.25', 'points', ',', 'or', '0.19', 'percent', ',', 'to', '9,586.71', '.'], ['there', 'are', 'some', 'who', 'feel', 'like', 'the', 'conditions', 'are', 'such', 'that', 'they', 'can', 'attack', 'us', 'there', '.'], ['the', 'men', 'were', 'flown', 'to', 'nearby', 'botswana', 'on', 'an', 'air', 'malawi', 'flight', ',', 'the', 'officials', 'said', '.'], ['prior', 'to', 'joining', 'eastman', 'kodak', ',', 'mr.', 'palumbo', 'held', 'senior', 'marketing', 'positions', 'for', 'procter', '&', 'gamble', '.'], ['the', 'foreign', 'bits', 'are', 'recognized', 'by', 'other', 'cells', 'that', 'realize', 'it', 'is', 'infected', 'and', 'kill', 'it', '.'], ['the', 'other', 'victim', 'recovered', 'tuesday', 'morning', 'was', 'al', 'larsen', ',', '31', ',', 'of', 'fort', 'worth', ',', 'texas', '.'], ['martin', 'served', 'two', 'thirds', 'of', 'a', 'five-year', 'sentence', 'for', 'the', 'manslaughter', 'of', 'barras', 'and', 'for', 'wounding', 'fearon', '.'], ['the', 'dispute', 'has', 'led', 'ukrainian', 'president', 'leonid', 'kuchma', 'to', 'cut', 'short', 'a', 'state', 'visit', 'to', 'latin', 'america', '.'], ['sorkin', 'was', 'to', 'have', 'been', 'tried', 'separately', 'on', 'charges', 'of', 'conspiracy', 'and', 'lying', 'to', 'a', 'grand', 'jury', '.'], ['shares', 'of', 'guidant', 'were', 'down', '5.3', 'percent', 'at', '$', '40.92', 'in', 'afternoon', 'trading', 'on', 'the', 'new', 'york', 'stock', 'exchange', '.'], ['the', 'company', 'is', 'also', 'adding', '.net', 'and', 'j2ee', 'support', 'to', 'its', 'enterprise', 'application', 'environment', 'development', 'toolset', ',', 'sapp', 'said', '.'], ['she', 'said', 'she', 'eventually', 'allowed', 'him', 'to', 'fix', 'her', 'bike', 'while', 'she', 'sat', 'in', 'his', 'car', ',', 'and', 'they', 'chatted', '.'], ['on', 'the', 'week', ',', 'the', 'dow', 'jones', 'industrial', 'average', 'rose', '11.56', 'points', ',', 'while', 'the', 'nasdaq', 'stock', 'market', 'gained', '39.42', 'points', '.'], ['pg', '&', 'e', "'s", 'shares', 'gained', '24', 'cents', 'to', '$', '15.44', 'during', 'tuesday', "'s", 'trading', 'on', 'the', 'new', 'york', 'stock', 'exchange', '.'], ['"', 'we', 'condemn', 'and', 'denounce', 'the', 'governing', 'council', ',', 'which', 'is', 'headed', 'by', 'the', 'united', 'states', ',', '"', 'moqtada', 'al-sadr', 'said', '.'], ['gov.', 'rick', 'perry', 'opposes', 'expansion', 'of', 'gambling', 'but', 'has', 'said', 'he', 'would', 'be', '"', 'hard-pressed', '"', 'to', 'veto', 'the', 'lottery', 'sunset', 'bill', '.'], ['the', 'treasury', 'said', 'wants', 'to', 'give', 'retail', 'buyers', 'of', 'all', 'treasury', 'securities', 'the', 'ability', 'eventually', 'to', 'manage', 'holdings', 'online', 'in', 'a', 'single', 'account', '.'], ['people', 'in', 'a', 'van', 'opened', 'fire', 'on', 'a', 'pickup', 'and', 'suv', 'believed', 'to', 'be', 'transporting', 'illegal', 'immigrants', ',', 'said', 'pinal', 'county', 'sheriff', 'roger', 'vanderpool', '.'], ['the', 'episcopal', 'church', ',', 'with', '2.3', 'million', 'members', ',', 'is', 'the', 'american', 'branch', 'of', 'the', 'worldwide', 'anglican', 'communion', ',', 'which', 'has', '77', 'million', 'adherents', '.'], ['the', '14-year-old', 'national', 'spelling', 'finalist', 'who', 'attends', 'school', 'in', 'belmont', ',', 'n.c.', ',', 'got', 'a', 'word', 'that', 'sounded', 'like', '"', 'zistee', '"', 'during', 'competition', 'wednesday', '.'], ['bush', ',', 'however', ',', 'also', 'lashed', 'out', 'at', 'the', 'terrorist', 'bombings', 'in', 'saudi', 'arabia', 'that', 'killed', 'at', 'least', '29', 'people', '--', 'including', 'a', 'number', 'of', 'americans', '.'], ['ibm', 'said', 'the', 'rational', 'products', 'were', 'also', 'integrated', 'with', 'rational', 'purifyplus', ',', 'which', 'allows', 'developers', 'to', 'work', 'in', 'java', ',', 'visual', 'c', '#', 'and', 'visualbasic', '.net', '.'], ['the', 'agency', 'said', 'it', 'would', 'consider', 'as', 'timely', 'any', 'tax', 'returns', 'or', 'payments', 'due', 'from', 'friday', 'through', 'aug.', '22', 'if', 'they', 'are', 'completed', 'by', 'aug.', '22', '.'], ['a', 'spanish', 'tanker', ',', 'sandra', 'tapias', ',', 'was', 'to', 'be', 'loaded', 'with', 'another', 'million', 'barrels', ',', 'bought', 'by', 'spanish', 'refiner', 'cepsa', 'sa', ',', 'in', 'the', 'afternoon', '.'], ['two-year-old', 'egyptian', 'twin', 'boys', ',', 'born', 'with', 'their', 'heads', 'conjoined', ',', 'have', 'been', 'separated', 'after', '26', 'hours', 'of', 'surgery', 'at', 'the', 'children', "'s", 'medical', 'centre', 'in', 'dallas', '.'], ['the', 'department', 'of', 'homeland', 'security', 'announced', 'tuesday', 'that', 'it', 'has', 'selected', 'three', 'companies', 'to', 'continue', 'research', 'into', 'ways', 'to', 'thwart', 'shoulder-fired', 'missile', 'attacks', 'on', 'u.s.', 'commercial', 'aircraft', '.'], ['hundreds', 'of', 'kenyan', 'women', 'claiming', 'they', 'were', 'attacked', 'and', 'raped', 'by', 'british', 'soldiers', 'were', 'granted', 'government', 'legal', 'aid', 'wednesday', 'to', 'pursue', 'their', 'case', 'against', 'the', 'ministry', 'of', 'defense', '.'], ['but', 'in', 'the', 'first', '30', 'seconds', 'of', 'the', 'bout', ',', 'family', 'members', 'knew', 'it', 'was', 'an', 'uneven', 'match', ',', 'jodie', 'meyers', ',', 'stacy', 'young', "'s", 'sister', ',', 'said', '.'], ['at', '11', 'a.m.', '(', '1500', 'gmt', ')', ',', 'the', '10-year', 'note', 'us10yt', '=', 'rr', 'was', 'up', '11', '/', '32', 'for', 'a', 'yield', 'of', '3.36', 'percent', 'from', '3.40', 'percent', 'wednesday', '.'], ['the', 'metropolitan', 'transportation', 'authority', ',', 'which', 'plans', 'to', 'appeal', ',', 'was', 'given', 'until', 'may', '28', 'to', 'restore', 'the', 'old', 'subway', 'and', 'bus', 'fare', 'and', 'the', 'old', 'commuter', 'railroad', 'rates', '.'], ['spokesman', 'james', 'howe', 'said', 'five', 'children', 'aged', 'between', '4', 'and', '17', 'were', 'taken', 'to', 'hospital', 'with', 'neck', 'and', 'chest', 'injuries', ',', 'while', 'three', 'others', 'were', 'treated', 'at', 'the', 'scene', '.'], ['"', 'both', 'of', 'these', 'kids', 'are', 'in', 'wonderful', 'physical', 'condition', 'right', 'now', ',', '"', 'said', 'goodrich', ',', 'also', 'director', 'of', 'pediatric', 'neurosurgery', 'at', 'the', 'children', "'s", 'hospital', 'at', 'montefiore', '.'], ['microsoft', 'issued', 'a', 'patch', 'for', 'the', 'vulnerability', ',', 'which', 'affects', 'windows', 'nt', '4.0', ',', 'windows', 'nt', '4.0', 'terminal', 'services', 'edition', ',', 'windows', '2000', ',', 'windows', 'xp', 'and', 'windows', 'server', '2003', '.'], ['my', 'judgment', 'is', '95', 'percent', 'of', 'that', 'information', 'could', 'be', 'declassified', ',', 'become', 'uncensored', 'so', 'the', 'american', 'people', 'would', 'know', ',', '"', 'mr.', 'shelby', 'said', 'on', 'nbc', "'s", '"', 'meet', 'the', 'press', '.', '"'], ['comstock', 'said', 'several', 'federal', 'courts', 'have', 'ruled', 'that', 'the', 'detentions', 'are', 'fully', 'within', 'the', 'law', '.'], ['the', 'man', 'had', 'entered', 'the', 'post', 'office', 'on', 'woodside', 'avenue', 'at', 'maine', 'avenue', 'shortly', 'before', '3', 'p.m', '.'], ['miodrag', 'zivkovic', ',', 'the', 'leader', 'of', 'the', 'pro-independence', 'opposition', 'liberal', 'alliance', ',', 'came', 'in', 'second', 'with', '31', 'percent', 'of', 'the', 'vote', '.'], ['"', 'these', 'are', 'dark', 'days', 'for', 'our', 'industry', ',', '"', 'the', 'geneva-based', 'international', 'air', 'transport', 'association', "'s", 'director-general', 'giovanni', 'bisignani', 'said', '.'], ['mcarthur', 'told', 'internetnews.com', 'price', 'declines', 'have', 'moderated', 'and', 'remained', 'below', '30', 'percent', 'from', 'the', 'previous', 'year', 'for', 'the', 'last', 'two', 'quarters', '.'], ['"', 'for', 'me', ',', 'the', 'lewinsky', 'imbroglio', 'seemed', 'like', 'just', 'another', 'vicious', 'scandal', 'manufactured', 'by', 'political', 'opponents', ',', '"', 'mrs', 'clinton', 'writes', '.'], ['the', 'single', 'currency', 'rose', 'as', 'far', 'as', '135.26', 'yen', ',', 'its', 'highest', 'level', 'since', 'the', 'introduction', 'of', 'the', 'single', 'currency', 'in', 'january', '1999', '.'], ['the', '30-year', 'bond', 'us30yt', '=', 'rr', 'grew', '1-3', '/', '32', 'for', 'a', 'yield', 'of', '4.30', 'percent', ',', 'down', 'from', '4.35', 'percent', 'late', 'wednesday', '.'], ['sony', 'ericsson', 'also', 'said', 'it', 'plans', 'to', 'close', 'its', 'r', '&', 'd', 'site', 'in', 'munich', ',', 'germany', ',', 'for', 'gsm', 'and', 'umts', 'handsets', '.'], ['north', 'american', 'stock', 'markets', 'got', 'off', 'to', 'a', 'slow', 'start', 'tuesday', ',', 'with', 'investors', 'ready', 'to', 'get', 'their', 'first', 'taste', 'of', 'quarterly', 'earnings', '.'], ['joining', 'him', 'in', 'the', 'majority', 'opinion', 'were', 'justices', 'sandra', 'day', 'o', "'connor", ',', 'david', 'h.', 'souter', ',', 'ruth', 'bader', 'ginsburg', 'and', 'stephen', 'g.', 'breyer', '.'], ['wire', 'transfers', 'from', 'four', 'of', 'the', '40', 'accounts', 'open', 'at', 'beacon', 'hill', 'totaled', 'more', 'than', '$', '3.2', 'billion', 'from', '2001', 'to', '2002', ',', 'morgenthau', 'said', '.'], ['dotson', 'was', 'arrested', 'july', '21', 'after', 'telling', 'fbi', 'agents', 'he', 'shot', 'dennehy', 'when', 'dennehy', 'tried', 'to', 'shoot', 'him', ',', 'according', 'to', 'the', 'arrest', 'warrant', 'affidavit', '.'], ['dolores', 'e.', 'mahoy', ',', '68', ',', 'of', 'colorado', 'springs', 'is', 'just', 'the', 'type', 'of', 'person', 'who', 'might', 'be', 'helped', 'by', 'the', 'legislation', 'pending', 'in', 'congress', '.'], ['in', 'canada', ',', 'the', 'surging', 'dollar', 'was', 'in', 'focus', 'again', 'as', 'it', 'struggled', 'and', 'just', 'failed', 'to', 'stay', 'above', 'the', '75', 'cent', '(', 'u.s.', ')', 'mark', '.'], ['some', '660', 'prisoners', 'from', '42', 'countries', 'are', 'being', 'held', 'at', 'the', 'naval', 'base', 'at', 'guantanamo', 'bay', ',', 'many', 'captured', 'during', 'the', 'war', 'against', 'al', 'qaeda', 'in', 'afghanistan', '.'], ['the', 'euro', 'soared', 'to', '$', '1.1914', 'in', 'asian', 'trading', ',', 'before', 'slipping', 'back', 'to', '$', '1.1884', 'in', 'late', 'european', 'trading', ',', 'up', 'from', '$', '1.1857', 'late', 'monday', '.'], ['teams', 'of', 'inspectors', 'walked', 'into', 'a', 'restricted', 'area', 'at', 'one', 'park', 'without', 'being', 'stopped', 'by', 'employees', ';', 'others', 'took', 'pictures', 'of', 'security-sensitive', 'areas', 'without', 'anyone', 'challenging', 'them', '.'], ['"', 'it', "'s", 'a', 'new', 'beginning', 'as', 'far', 'as', 'the', 'courts', ',', 'crown', 'prosecution', 'service', 'and', 'police', 'are', 'concerned', ',', 'making', 'the', 'criminal', 'justice', 'system', 'work', 'better', '.', '"'], ['no', 'charges', 'have', 'been', 'filed', 'in', 'those', 'deaths', ',', 'but', 'luzerne', 'county', 'district', 'attorney', 'david', 'lupas', 'said', 'authorities', 'were', '"', 'continuing', 'to', 'make', 'significant', 'progress', '"', 'in', 'the', 'case', '.'], ['passed', 'in', '1999', 'but', 'never', 'put', 'into', 'effect', ',', 'the', 'smoking', 'law', 'would', 'have', 'prevented', 'bar', 'and', 'restaurant', 'patrons', 'from', 'lighting', 'up', ',', 'but', 'exempted', 'private', 'clubs', 'from', 'the', 'regulation', '.'], ['pratt', '&', 'whitney', 'had', 'said', 'that', 'if', 'it', 'won', 'the', 'contract', '75', 'per', 'cent', 'of', 'the', 'engine', 'equipment', 'would', 'be', 'outsourced', 'to', 'european', 'suppliers', ',', 'with', 'final', 'assembly', 'in', 'germany', '.'], ['the', 'senate', 'judiciary', 'committee', 'on', 'tuesday', 'overcame', 'a', 'significant', 'hurdle', 'in', 'the', 'battle', 'to', 'create', 'a', 'trust', 'fund', 'to', 'pay', 'victims', 'of', 'asbestos', 'exposure', 'but', 'the', 'most', 'difficult', 'obstacles', 'remain', '.'], ['prices', 'had', 'pulled', 'back', 'from', 'offshore', 'highs', 'when', 'the', 'empire', 'state', 'survey', 'far', 'exceeded', 'expectations', 'by', 'jumping', 'to', '26.8', 'in', 'june', ',', 'a', 'record', 'high', ',', 'from', '10.6', 'in', 'may', '.'], ['the', 'report', 'shows', 'that', 'drugs', 'sold', 'in', 'canadian', 'pharmacies', 'are', 'manufactured', 'in', 'facilities', 'approved', 'by', 'health', 'canada', ',', 'which', 'serves', 'a', 'similar', 'role', 'as', 'the', 'fda', 'for', 'the', 'canadian', 'government', '.'], ['in', 'texas', ',', 'public', 'watchdog', 'groups', 'opposed', 'mr.', 'bush', "'s", 'efforts', 'to', 'house', 'his', 'papers', 'at', 'his', 'father', "'s", 'presidential', 'library', ',', 'where', 'they', 'would', 'have', 'remained', 'secret', 'for', 'a', 'half-century', '.'], ['she', 'countersued', 'for', '$', '125', 'million', ',', 'saying', 'g', '+', 'j', 'broke', 'its', 'contract', 'with', 'her', 'by', 'cutting', 'her', 'out', 'of', 'key', 'editorial', 'decisions', 'and', 'manipulated', 'the', 'magazine', "'s", 'financial', 'figures', '.'], ['congregants', 'of', 'turner', 'monumental', 'ame', 'church', 'said', 'shelia', 'chaney', 'wilson', ',', '43', ',', 'was', 'agitated', 'when', 'she', 'came', 'to', 'the', 'church', ',', 'in', 'the', 'kirkwood', 'neighborhood', 'on', 'the', 'city', "'s", 'east', 'side', '.'], ['he', 'also', 'said', 'shockey', 'is', '"', 'living', 'the', 'dream', 'life', 'of', 'a', 'new', 'york', 'athlete', '.'], ['the', 'union', 'banned', 'the', 'import', 'of', 'genetically', 'modified', 'food', 'in', '1998', 'after', 'many', 'consumers', 'feared', 'health', 'risks', '.'], ['"', 'it', 'was', 'a', 'brazen', 'attempt', 'at', 'intimidation', ',', '"', 'said', 'a', 'republican', 'legislator', 'who', 'attended', 'the', 'meeting', '.'], ['brian', 'rohrbough', ',', 'whose', 'son', ',', 'daniel', ',', 'was', 'killed', 'at', 'columbine', ',', 'said', 'the', 'tape', 'raises', 'disturbing', 'issues', '.'], ['the', 'fijian', 'military', 'installed', 'an', 'all-indigenous', 'government', 'led', 'by', 'mr', 'qarase', ',', 'who', 'won', 'democratic', 'elections', 'in', 'september', '2001', '.'], ['the', '30-year', 'bond', 'us30yt', '=', 'rr', 'jumped', '20', '/', '32', ',', 'taking', 'its', 'yield', 'to', '4.14', 'percent', 'from', '4.18', 'percent', '.'], ['a', 'sour', 'forecast', 'from', 'sun', 'microsystems', 'inc', '.', 'sunw.o', 'put', 'more', 'pressure', 'on', 'wall', 'street', 'before', 'the', 'quarterly', 'earnings', 'season', '.'], ['"', 'we', 'don', "'t", 'want', 'to', 'stand', 'by', 'and', 'see', 'a', 'credit', 'card', 'lynching', 'take', 'place', ',', '"', 'he', 'said', '.'], ['the', 'competition', 'commission', 'delivered', 'its', 'verdict', 'yesterday', 'on', 'the', 'proposed', 'merger', 'of', 'the', 'two', 'big', 'itv', 'players', ',', 'carlton', 'and', 'granada', '.'], ['a', 'pro-independence', 'radical', ',', 'miodrag', 'zivkovic', ',', 'of', 'the', 'liberal', 'alliance', ',', 'came', 'in', 'second', 'with', '31', 'percent', 'of', 'the', 'vote', '.'], ['"', 'it', "'s", 'a', 'huge', 'black', 'eye', ',', '"', 'arthur', 'sulzberger', ',', 'the', 'newspaper', "'s", 'publisher', ',', 'said', 'of', 'the', 'scandal', '.'], ['"', 'the', 'whole', 'thing', 'is', 'a', 'travesty', ',', '"', 'fumed', 'brent', 'newbury', ',', 'president', 'of', 'the', 'rockland', 'county', 'patrolmen', "'s", 'benevolent', 'association', '.'], ['in', 'turning', 'down', 'the', 'nima', 'offer', ',', 'gehman', 'said', ',', "'", "'", 'there', 'were', 'a', 'number', 'of', 'bureaucratic', 'and', 'administrative', 'missed', 'signals', 'here', '.'], ['saturday', ',', 'officials', 'in', 'california', 'described', 'gehring', 'as', 'cooperative', '—', 'but', 'said', 'he', 'has', 'revealed', 'nothing', 'about', 'what', 'has', 'happened', 'to', 'the', 'children', '.'], ['u.s.', 'soldiers', 'killed', 'about', 'two', 'dozen', 'suspected', 'taliban', 'militants', 'in', 'southern', 'afghanistan', 'after', 'their', 'convoy', 'came', 'under', 'attack', ',', 'the', 'military', 'said', 'sunday', '.'], ['but', 'the', 'bureau', 'never', 'kept', 'tabs', 'on', 'al-bayoumi', '—', 'despite', 'receiving', 'prior', 'information', 'he', 'was', 'a', 'secret', 'saudi', 'agent', ',', 'the', 'report', 'says', '.'], ['total', 'revenue', 'for', 'the', 'second', 'quarter', 'was', '$', '1.48', 'billion', ',', 'up', '7', '%', 'from', '$', '1.39', 'billion', 'in', 'the', 'same', 'period', 'last', 'year', '.'], ['instead', 'of', 'one', 'long', 'operation', 'to', 'separate', 'the', 'twins', ',', 'goodrich', 'and', 'dr.', 'david', 'staffenberg', 'plan', 'about', 'three', ',', 'with', 'several', 'weeks', 'between', 'each', '.'], ['"', 'whereas', 'long', 'lines', 'used', 'to', 'catch', '10', 'fish', 'per', '100', 'hooks', ',', 'now', 'they', 'are', 'lucky', 'to', 'catch', 'one', ',', '"', 'myers', 'said', '.'], ['on', 'wall', 'street', ',', 'the', 'dow', 'jones', 'industrial', 'average', 'rose', '0.5', 'per', 'cent', 'at', '9,905.8', 'and', 'the', 'nasdaq', 'composite', 'added', '0.7', 'per', 'cent', 'at', '1,995.1', '.'], ['"', 'our', 'fundamental', 'values', 'haven', "'t", 'changed', ',', '"', 'mary', 'sue', 'coleman', ',', 'president', 'of', 'the', 'university', ',', 'said', 'in', 'a', 'statement', 'in', 'ann', 'arbor', '.'], ['at', '11', 'a.m.', 'edt', ',', 'fabian', "'s", 'center', 'was', 'near', 'latitude', '18.1', 'north', 'and', 'longitude', '53.2', 'west', ',', 'or', 'about', '550', 'miles', 'east', 'of', 'the', 'leewards', '.'], ['the', 'blue-chip', 'dow', 'jones', 'industrial', 'average', '.dji', 'jumped', '194.14', 'points', ',', 'or', '2.09', 'percent', ',', 'to', '9,469.20', 'after', 'sinking', 'more', 'than', '1', 'percent', 'a', 'day', 'earlier', '.'], ['sirius', 'recently', 'began', 'carrying', 'national', 'public', 'radio', ',', 'a', 'deal', 'pooh-poohed', 'by', 'xm', 'because', 'it', 'doesn', "'t", 'include', 'popular', 'shows', 'like', 'all', 'things', 'considered', 'and', 'morning', 'edition', '.'], ['ellison', 'said', 'that', 'contrary', 'to', 'the', 'contentions', 'of', 'peoplesoft', 'management', ',', 'oracle', 'intends', 'to', '"', 'fully', 'support', 'peoplesoft', 'customers', 'and', 'products', '"', 'for', 'many', 'years', 'to', 'come', '.'], ['the', 'dollar', 'was', 'at', '116.78', 'yen', 'jpy', '=', ',', 'virtually', 'flat', 'on', 'the', 'session', ',', 'and', 'at', '1.2871', 'against', 'the', 'swiss', 'franc', 'chf', '=', ',', 'down', '0.1', 'percent', '.'], ['brigadier', 'general', 'edward', 'aritonang', ',', 'a', 'police', 'spokesman', ',', 'confirmed', 'yesterday', 'that', 'another', 'two', 'had', 'been', 'arrested', ',', 'one', 'in', 'jakarta', 'and', 'another', 'in', 'magelang', ',', 'central', 'java', '.'], ['bolivia', "'s", 'army', 'fought', 'to', 'stop', 'a', 'column', 'of', 'dynamite-wielding', 'miners', 'from', 'streaming', 'into', 'the', 'besieged', 'capital', 'on', 'wednesday', ',', 'leaving', 'two', 'dead', 'as', 'a', 'popular', 'uprising', 'against', 'the', 'president', 'spread', '.'], ['byrd', ',', 'speaking', 'from', 'the', 'senate', 'floor', ',', 'said', 'he', 'questioned', '"', 'the', 'motives', 'of', 'a', 'deskbound', 'president', 'who', 'assumes', 'the', 'garb', 'of', 'a', 'warrior', 'for', 'purposes', 'of', 'a', 'speech', '.', '"'], ['the', 'dow', 'jones', 'industrial', 'average', '.dji', 'fell', '79.43', 'points', ',', 'or', '0.86', 'percent', ',', 'to', '9,117.12', 'on', 'friday', '.'], ['the', 'dow', 'jones', 'industrial', 'average', '<', '.dji', '>', 'rose', '49', 'points', ',', 'or', '0.47', 'percent', ',', 'to', '10,578', '.'], ['he', 'or', 'she', 'decides', 'the', 'merchandising', 'direction', 'of', 'the', 'company', ',', 'or', 'basically', 'what', 'it', 'should', 'stand', 'for', '.'], ['sumitomo', 'mitsui', 'financial', '(', 'jp', ':', '8316', ':', 'news', ',', 'chart', ',', 'profile', ')', 'was', 'down', '2.5', 'percent', 'at', '198,000', 'yen', '.'], ['egyptologists', 'think', 'they', 'may', 'have', 'identified', 'the', 'long-sought', 'mummy', 'of', 'queen', 'nefertiti', ',', 'one', 'of', 'the', 'ancient', 'world', "'s", 'legendary', 'beauties', '.'], ['an', 'hour', 'later', 'israeli', 'attack', 'helicopters', 'rained', 'missiles', 'on', 'a', 'car', 'in', 'gaza', 'city', ',', 'killing', 'seven', 'people', ',', 'palestinian', 'sources', 'said', '.'], ['and', 'now', 'it', "'s", 'anything', 'he', 'wants', 'to', 'say', ',', '"', 'confirmed', 'stone', 'county', 'nursing', 'and', 'rehabilitation', 'center', 'social', 'director', 'alesha', 'badgley', '.'], ['in', 'a', 'regulatory', 'filing', ',', 'eds', 'said', 'the', 'sec', 'had', 'asked', 'for', 'information', 'related', 'to', 'its', 'troubled', 'it', 'outsourcing', 'contract', 'with', 'the', 'us', 'navy', '.'], ['iraqi', 'police', 'opened', 'fire', 'in', 'downtown', 'baghdad', 'wednesday', 'after', 'demonstrators', 'demanding', 'jobs', 'stormed', 'a', 'police', 'station', 'and', 'threw', 'stones', 'at', 'officers', ',', 'police', 'said', '.'], ['cuban', 'officials', 'said', 'that', 'last', 'year', 'none', 'of', 'about', 'two', 'dozen', 'nominated', 'cuban', 'musicians', 'received', 'u.s.', 'visas', 'in', 'time', 'to', 'attend', 'the', 'latin', 'grammys', '.'], ['tikrit', 'is', 'saddam', 'hussein', "'s", 'hometown', 'and', 'the', 'region', 'around', 'it', 'is', 'known', 'as', 'a', 'hotbed', 'of', 'baath', 'party', 'supporters', 'and', 'former', 'high-ranking', 'iraqi', 'military', 'officials', '.'], ['microsoft', 'will', 'stop', 'developing', 'versions', 'of', 'its', 'internet', 'explorer', 'browser', 'software', 'for', 'macintosh', 'computers', ',', 'saying', 'that', 'apple', "'s", 'safari', 'is', 'now', 'all', 'that', 'apple', 'needs', '.'], ['the', 'skagit', 'was', 'expected', 'to', 'crest', 'during', 'the', 'night', 'at', '38', 'feet', 'at', 'mount', 'vernon', ',', '10', 'feet', 'above', 'flood', 'stage', ',', 'the', 'national', 'weather', 'service', 'said', '.'], ['police', 'have', 'arrested', '21', 'people', 'in', 'connection', 'with', 'the', 'murder', 'of', 'a', 'young', 'nigerian', 'child', 'whose', 'headless', 'and', 'limbless', 'torso', 'was', 'found', 'floating', 'in', 'the', 'river', 'thames', '.'], ['nuremberg', ',', 'germany-based', 'suse', 'will', 'license', 'sun', "'s", 'java', '2', 'standard', 'edition', '(', 'j2se', ')', 'and', 'ship', 'sun', "'s", 'java', 'virtual', 'machine', 'across', 'its', 'linux', 'software', 'line', '.'], ['he', 'said', 'he', 'wasn', "'t", 'bothered', 'by', 'polls', 'and', 'that', 'he', 'would', 'continue', 'to', 'do', 'what', 'he', 'believed', 'to', 'be', 'right', ',', 'in', 'both', 'foreign', 'and', 'domestic', 'policy', '.'], ['ebbers', "'", 'lawyer', ',', 'reid', 'weingarten', ',', 'said', 'despite', 'the', 'thorough', 'investigation', ',', '"', 'the', 'best', 'the', 'investigators', 'can', 'do', 'is', 'nitpick', 'about', 'the', 'process', 'and', 'substance', 'of', 'isolated', 'business', 'decisions', '.', '"'], ['"', 'the', 'steps', 'that', 'the', 'iranians', 'claim', 'to', 'have', 'taken', 'in', 'terms', 'of', 'capturing', 'al-qaida', 'are', 'insufficient', '.'], ['his', 'aides', 'warn', 'an', 'abrupt', 'departure', 'could', 'trigger', 'more', 'bloodshed', ',', 'a', 'view', 'shared', 'by', 'many', 'in', 'liberia', '.'], ['executive', 'recruiters', 'say', 'that', 'finding', 'a', 'seasoned', 'chief', 'merchandising', 'officer', 'perhaps', 'will', 'be', 'the', 'trickiest', 'of', 'all', 'hires', '.'], ['france', ',', 'which', 'infuriated', 'washington', 'by', 'leading', 'the', 'charge', 'against', 'u.n.', 'approval', 'for', 'the', 'war', ',', 'also', 'sought', 'changes', '.'], ['in', 'millville', 'yesterday', ',', 'mayor', 'james', 'quinn', 'ordered', 'all', 'city', 'flags', 'flown', 'at', 'half-staff', 'for', 'the', 'next', '30', 'days', '.'], ['"', 'but', 'that', 'does', 'not', 'clear', 'them', 'of', 'the', 'responsibility', 'to', 'do', 'everything', 'possible', 'to', 'minimize', 'civilian', 'harm', '.', '"'], ['douglas', 'meester', ',', '20', ',', 'is', 'charged', 'with', 'rape', ',', 'sodomy', ',', 'indecent', 'assault', ',', 'and', 'providing', 'alcohol', 'to', 'minors', '.'], ['"', 'i', "'m", 'very', 'proud', 'of', 'the', 'citizens', 'of', 'this', 'state', ',', '"', 'said', 'gov.', 'john', 'baldacci', ',', 'a', 'casino', 'foe', '.'], ['"', 'it', 'got', 'up', 'on', 'his', 'personal', 'radar', 'screen', 'in', 'the', 'past', 'week', ',', 'and', 'i', 'had', 'to', 'get', 'my', 'pitch', '.'], ['internal', 'chaos', 'has', 'allowed', 'foreign', 'companies', 'to', 'set', 'up', 'damaging', 'commercial', 'logging', 'and', 'fishing', 'operations', 'without', 'proper', 'monitoring', 'or', 'export', 'controls', '.'], ['in', 'the', 'second', 'quarter', 'last', 'year', ',', 'ak', 'steel', 'reported', 'a', 'profit', 'of', '$', '16.2', 'million', ',', 'or', '15', 'cents', 'a', 'share', '.'], ['the', 'readings', 'came', 'back', 'at', '160', 'on', 'june', '16', 'and', '120', 'at', 'june', '23', 'at', 'one', 'of', 'three', 'sampling', 'sites', 'at', 'huntington', 'beach', '.'], ['tehran', 'has', 'made', 'it', 'clear', 'it', 'will', 'only', 'sign', 'the', 'protocol', 'if', 'a', 'ban', 'on', 'imports', 'of', 'peaceful', 'western', 'nuclear', 'technology', 'is', 'lifted', '.'], ['the', 'americas', 'market', 'will', 'decline', '2.1', 'percent', 'to', '$', '30.6', 'billion', 'in', '2003', ',', 'and', 'then', 'grow', '15.7', 'percent', 'to', '$', '35.4', 'billion', 'in', '2004', '.'], ['shares', 'of', 'tampa-based', 'outback', 'steakhouse', 'inc.', 'closed', 'at', '$', '44.50', ',', 'up', '$', '1.78', ',', 'or', '4.2', 'percent', ',', 'on', 'the', 'new', 'york', 'stock', 'exchange', '.'], ['one', 'of', 'the', 'key', 'suspects', 'in', 'the', 'october', 'nightclub', 'bombings', 'that', 'killed', 'more', 'than', '200', 'people', 'in', 'bali', 'went', 'on', 'trial', 'monday', 'amid', 'tight', 'security', '.'], ['the', 'lawyer', 'for', 'hollins', 'jr', '.', ',', 'thomas', 'royce', ',', 'has', 'said', 'his', 'client', 'had', 'no', 'link', 'to', 'the', 'company', 'that', 'owned', 'and', 'operated', 'e2', '.'], ['wall', 'street', 'was', 'also', 'waiting', 'for', 'aluminum', 'maker', 'alcoa', 'inc', '.', '(', 'nyse', ':', 'pep', '-', 'news', '-', 'people', ')', 'to', 'report', 'earnings', 'after', 'the', 'close', '.'], ['at', 'first', 'blush', ',', 'then', ',', 'the', 'creators', 'of', 'the', 'registry', 'would', 'seem', 'to', 'have', 'drawn', 'the', 'line', 'between', 'exempted', 'and', 'banned', 'calls', 'in', 'the', 'right', 'place', '.'], ['a', '62-year-old', 'west', 'babylon', 'man', 'has', 'contracted', 'the', 'west', 'nile', 'virus', ',', 'the', 'first', 'human', 'case', 'in', 'suffolk', 'county', 'this', 'year', ',', 'according', 'to', 'the', 'county', 'health', 'department', '.'], ['a', 'passer-by', 'found', 'the', 'boy', 'hiding', 'along', 'a', 'dirt', 'road', 'in', 'spanish', 'fork', 'canyon', 'about', '7', 'p.m.', 'thursday', ',', 'with', 'his', 'feet', 'free', 'but', 'his', 'hands', 'still', 'taped', '.'], ['new', 'york', '-', '(', 'dow', 'jones', ')', '-', 'former', 'star', 'investment', 'banker', 'frank', 'quattrone', 'pleaded', 'not', 'guilty', 'tuesday', 'to', 'criminal', 'charges', 'of', 'obstruction', 'of', 'justice', 'and', 'witness', 'tampering', '.'], ['microsoft', 'has', 'been', 'awarded', 'a', 'patent', 'on', 'a', 'popular', 'instant-messaging', 'feature', 'that', 'shows', 'users', 'when', 'the', 'person', 'on', 'the', 'other', 'end', 'of', 'the', 'conversation', 'is', 'typing', 'a', 'message', '.'], ['terri', 'schiavo', ',', '39', ',', 'underwent', 'the', 'procedure', 'at', 'the', 'tampa', 'bay', 'area', 'hospice', 'where', 'she', 'has', 'been', 'living', 'for', 'several', 'years', ',', 'said', 'her', 'father', ',', 'bob', 'schindler', '.'], ['note', 'the', 'subheading', 'of', 'this', 'terrible', 'parable', 'in', 'the', '"', 'american', 'masters', '"', 'series', ',', '"', 'arthur', 'miller', ',', 'elia', 'kazan', 'and', 'the', 'blacklist', ':', 'none', 'without', 'sin', '.', '"'], ['these', 'despicable', 'acts', 'were', 'committed', 'by', 'killers', 'whose', 'only', 'faith', 'is', 'hate', 'and', 'the', 'united', 'states', 'will', 'find', 'the', 'killers', 'and', 'they', 'will', 'learn', 'the', 'meaning', 'of', 'american', 'justice', '.'], ['bremer', 'said', 'he', 'would', 'launch', 'a', '$', '70-million', 'program', 'in', 'the', 'next', 'two', 'weeks', 'to', 'clean', 'up', 'neighborhoods', 'across', 'iraq', 'and', 'build', 'community', 'projects', ',', 'but', 'gave', 'no', 'details', '.'], ['noah', 'wyle', ',', 'who', 'has', 'played', 'dr.', 'john', 'carter', 'since', 'the', 'series', 'launched', 'in', '1994', ',', 'has', 'inked', 'a', 'one-year', 'extension', 'to', 'his', 'deal', 'with', 'producer', 'warner', 'bros.', 'tv', '.'], ['the', 'offer', 'is', 'also', 'subject', 'to', 'goldman', 'signing', 'an', 'agreement', 'with', 'drax', "'s", 'senior', 'banks', ',', 'senior', 'bond', 'holders', 'and', 'hedging', 'banks', 'by', 'sept', '.', '30', ',', 'it', 'said', '.'], ['the', 'study', 'results', 'were', 'released', 'at', 'a', 'meeting', 'in', 'seattle', 'of', 'the', 'american', 'thoracic', 'society', 'and', 'also', 'will', 'be', 'published', 'in', 'tomorrow', "'s", 'issue', 'of', 'the', 'new', 'england', 'journal', 'of', 'medicine', '.'], ['gulping', 'for', 'air', ',', 'i', 'started', 'crying', 'and', 'yelling', 'at', 'him', ',', "'", 'what', 'do', 'you', 'mean', '?'], ['u.s.', 'special', 'forces', 'troops', 'are', 'training', 'colombian', 'soldiers', 'at', 'military', 'bases', 'in', 'the', 'region', 'to', 'protect', 'the', 'pipeline', '.'], ['france', ',', 'which', 'infuriated', 'washington', 'by', 'leading', 'the', 'charge', 'against', 'u.n.', 'approval', 'for', 'the', 'war', ',', 'also', 'sought', 'changes', '.'], ['greenspan', 'told', 'congress', 'on', 'tuesday', 'the', 'u.s.', 'economy', '"', 'could', 'very', 'well', 'be', 'embarking', 'on', 'a', 'period', 'of', 'sustained', 'growth', '"', '.'], ['health', 'minister', 'manto-tshabalala', 'msimang', 'is', 'not', 'fit', 'to', 'preside', 'over', 'an', 'anti-retroviral', 'treatment', 'rollout', 'plan', ',', 'according', 'to', 'some', 'opposition', 'parties', '.'], ['lower', 'courts', 'have', 'splintered', 'on', 'the', 'issue', ',', 'allowing', 'depictions', 'of', 'the', 'ten', 'commandments', 'in', 'some', 'instances', 'and', 'not', 'in', 'others', '.'], ['but', 'vincent', 'weafer', ',', 'security', 'director', 'with', 'symantec', 'security', 'response', ',', 'said', 'all', 'the', 'virus', 'did', 'was', 'visit', 'a', 'pornography', 'site', '.'], ['the', 'losses', 'occurred', 'overnight', 'in', 'willow', 'canyon', ',', 'one', 'of', 'three', 'areas', 'in', 'the', 'santa', 'catalina', 'mountains', 'threatened', 'by', 'the', 'resurgent', 'aspen', 'fire', '.'], ['a', 'three-judge', 'panel', 'of', 'the', '8th', 'u.s.', 'circuit', 'court', 'of', 'appeals', 'in', 'st.', 'louis', 'overturned', 'a', 'ruling', 'issued', 'last', 'year', 'that', 'supported', 'the', 'ordinance', '.'], ['a', 'former', 'partner', 'of', 'accountancy', 'firm', 'ernst', '&', 'young', 'was', 'yesterday', 'arrested', 'by', 'fbi', 'agents', 'in', 'the', 'us', 'on', 'charges', 'of', 'obstructing', 'federal', 'investigations', '.'], ['in', 'europe', ',', 'france', "'s", 'cac-40', 'rose', '1.3', 'percent', ',', 'britain', "'s", 'ftse', '100', 'declined', '0.2', 'percent', 'and', 'germany', "'s", 'dax', 'index', 'gained', '0.6', 'percent', '.'], ['the', 'people', 'who', 'are', 'sending', 'these', 'messages', 'are', 'infringing', 'on', 'my', 'rights', 'and', 'everyone', 'else', "'s", 'rights', 'to', 'use', 'your', 'computer', ',', '"', 'mckechnie', 'said', '.'], ['"', 'i', "'m", 'still', 'not', 'red-hot', 'favourite', ',', '"', 'hewitt', 'said', 'after', 'battling', 'to', 'down', 'davydenko', '6-3', '4-6', '6-3', '7-6', '(', '7-5', ')', 'on', 'centre', 'court', '.'], ['mitzna', 'and', 'party', 'secretary', 'ophir', 'pines', 'agreed', 'to', 'convene', 'party', 'organizations', 'within', '10', 'days', 'to', 'discuss', 'whether', 'new', 'primaries', 'would', 'be', 'held', 'or', 'a', 'temporary', 'leader', 'appointed', '.'], ['japan', 'added', 'to', 'growing', 'international', 'pressure', 'on', 'burma', 'yesterday', ',', 'threatening', 'to', 'halt', 'its', 'hefty', 'economic', 'aid', 'unless', 'the', 'military', 'government', 'released', 'pro-democracy', 'leader', 'aung', 'san', 'suu', 'kyi', '.'], ['gold', "'s", 'bounce', 'pulled', 'other', 'precious', 'metals', 'higher', ',', 'sending', 'silver', 'to', '$', '4.83', '/', '85', 'an', 'ounce', 'from', '$', '4.80', '/', '82', 'last', 'quoted', 'in', 'new', 'york', '.'], ['officials', 'in', 'chechnya', "'s", 'moscow-backed', 'government', 'are', 'a', 'frequent', 'target', 'for', 'rebels', ',', 'and', 'tension', 'is', 'running', 'high', 'ahead', 'of', 'sunday', "'s", 'presidential', 'election', 'in', 'the', 'war-ravaged', 'region', '.'], ['early', 'wednesday', ',', 'the', 'benchmark', '10-year', 'us10yt', '=', 'rr', 'had', 'lost', '16', '/', '32', 'in', 'price', ',', 'driving', 'its', 'yield', 'up', 'to', '3.33', 'percent', 'from', '3.26', 'percent', 'late', 'tuesday', '.'], ['on', 'tuesday', ',', 'before', 'byrd', "'s", 'speech', ',', 'fleischer', 'said', 'bush', 'wanted', "'", "'", 'to', 'see', 'an', 'aircraft', 'landing', 'the', 'same', 'way', 'that', 'the', 'pilots', 'saw', 'an', 'aircraft', 'landing', '.'], ['it', 'was', 'announced', 'that', 'green', 'will', 'replace', 'park', 'in', 'the', 'dale', 'earnhardt', 'inc', '.', 'no.', '1', 'chevrolet', ',', 'beginning', 'with', 'the', 'winston', 'open', 'at', 'lowe', "'s", 'motor', 'speedway', 'next', 'week', '.'], ['a', 'major', 'british', 'study', 'has', 'added', 'to', 'the', 'evidence', 'that', 'hormone', 'replacement', 'therapy', 'increases', 'the', 'risk', 'of', 'breast', 'cancer', ',', 'especially', 'when', 'women', 'receive', 'a', 'combination', 'of', 'estrogen', 'and', 'progestin', '.'], ['adora', 'obi', 'nweze', ',', 'the', 'naacp', 'state', 'president', ',', 'said', 'the', 'state', 'has', 'refused', 'to', 'consider', 'the', 'possibility', 'of', 'murder', '.'], ['mel', 'gibson', 'is', 'negotiating', 'with', 'newmarket', 'films', 'to', 'distribute', 'his', 'embattled', 'biblical', 'epic', '"', 'the', 'passion', 'of', 'christ', '"', 'in', 'the', 'united', 'states', '.'], ['long', 'lines', 'formed', 'sunday', 'outside', 'gas', 'stations', 'and', 'people', 'rushed', 'to', 'get', 'money', 'from', 'cash', 'machines', 'as', 'israelis', 'braced', 'for', 'the', 'strike', "'s", 'effects', '.'], ['the', 'prc', 'epidemic', 'began', 'in', 'guandong', 'province', 'in', 'november', ',', 'but', 'the', 'communist', 'party', 'government', 'refused', 'to', 'report', 'truthfully', 'about', 'its', 'spread', 'for', 'four', 'months', '.'], ['in', 'an', 'interview', ',', 'ms.', 'healey', ',', 'who', 'is', 'a', 'criminologist', ',', 'said', 'many', 'lawmakers', 'here', 'and', 'across', 'the', 'country', 'shared', 'a', 'wariness', 'toward', 'capital', 'punishment', '.'], ['professor', 'moffitt', 'said', 'the', 'results', 'needed', 'to', 'be', 'replicated', 'before', 'pursuing', 'testing', 'of', 'individuals', 'for', 'the', 'presence', 'of', 'the', 'long', 'or', 'short', 'versions', 'of', 'the', 'gene', '.'], ['gerry', 'kiely', ',', 'a', 'eu', 'agriculture', 'representative', 'in', 'washington', ',', 'said', 'eu', 'ministers', 'were', 'invited', 'but', 'canceled', 'because', 'the', 'union', 'is', 'closing', 'talks', 'on', 'agricultural', 'reform', '.'], ['the', 'report', 'cites', 'the', 'marriage', 'of', 'the', 'alleged', 'operational', 'commander', 'of', 'the', 'bali', 'bombings', ',', 'mukhlas', ',', 'as', 'an', 'example', 'of', 'how', 'ji', 'uses', 'marriage', 'to', 'grow', '.'], ['in', 'early', 'u.s.', 'trading', ',', 'the', 'euro', '<', 'eur', '=', '>', 'was', 'down', '0.6', 'percent', 'to', '$', '1.1741', ',', 'after', 'rising', 'to', 'a', 'four-month', 'high', 'of', '$', '1.1860', '.'], ['in', 'november', ',', 'jeffrey', 'brabakow', ',', 'the', 'chief', 'executive', 'at', 'the', 'time', ',', 'said', 'the', 'company', 'used', 'aggressive', 'pricing', 'to', 'get', 'higher', 'payments', 'for', 'the', 'sickest', 'medicare', 'patients', '.'], ['that', 'task', 'appealed', 'to', 'two', 'masters', 'of', 'the', 'megabyte', ',', 'peter', 'lyman', 'and', 'hal', 'varian', ',', 'professors', 'at', 'the', 'university', 'of', 'california-berkeley', "'s", 'school', 'of', 'information', 'management', 'and', 'systems', '.'], ['spot', 'gold', 'was', 'quoted', 'at', '$', '358.65', '/', '359.15', 'an', 'ounce', 'at', '0500', 'gmt', ',', 'having', 'darted', 'as', 'high', 'as', '$', '359.25', '--', 'a', 'level', 'not', 'seen', 'since', 'february', '25', '.'], ['u.s.', 'stocks', 'fell', 'in', 'early', 'morning', 'trade', 'on', 'wednesday', 'as', 'a', 'wave', 'of', 'profit', 'warnings', 'drained', 'some', 'of', 'the', 'optimism', 'that', 'has', 'driven', 'a', 'rally', 'over', 'the', 'past', 'three', 'months', '.'], ['the', 'rev.', 'john', 'johnston', ',', '64', ',', 'was', 'charged', 'with', 'aggravated', 'harassment', 'in', 'the', 'phone', 'call', 'case', 'and', 'with', 'criminal', 'possession', 'of', 'a', 'weapon', ',', 'according', 'to', 'a', 'police', 'statement', '.'], ['warren', 'david', 'campbell', ',', '32', ',', 'and', 'climbing', 'companion', 'jonathan', 'smith', ',', '47', ',', 'did', 'not', 'return', 'as', 'expected', 'on', 'tuesday', 'from', 'climbing', 'on', 'mt', 'd', "'archiac", 'in', 'the', 'mckenzie', 'district', '.'], ['messagelabs', ',', 'which', 'runs', 'outsourced', 'e-mail', 'servers', 'for', '700,000', 'customers', 'around', 'the', 'world', ',', 'has', 'labeled', 'the', 'worm', '"', 'high', 'risk', '"', 'and', 'reports', 'more', 'than', '31,000', 'infections', 'in', '120', 'countries', '.'], ['a', 'radio', 'station', 'has', 'suspended', 'two', 'disc', 'jockeys', 'for', 'locking', 'themselves', 'in', 'the', 'studio', 'and', 'continuously', 'playing', 'dixie', 'chicks', 'songs', ',', 'violating', 'the', 'station', "'s", 'two-month-old', 'ban', 'on', 'the', 'group', "'s", 'music', '.'], ['in', 'the', 'aftermath', 'of', 'hurricane', 'isabel', ',', 'federal', 'and', 'local', 'governments', 'remained', 'shut', 'down', ',', 'as', 'did', 'schools', 'and', 'many', 'businesses', '.'], ['but', 'he', 'is', 'the', 'only', 'candidate', 'who', 'has', 'won', 'national', 'labor', 'endorsements', 'so', 'far', ',', 'picking', 'up', 'his', '11th', 'on', 'tuesday', '.'], ['authorities', 'looked', 'to', 'the', 'public', 'yesterday', 'for', 'help', 'in', 'solving', 'the', 'deadly', 'sniper-style', 'shootings', 'of', 'three', 'people', 'outside', 'charleston-area', 'convenience', 'stores', '.'], ['tampa', 'bay', 'manager', 'lou', 'piniella', ',', 'bench', 'coach', 'john', 'mclaren', 'and', 'huff', 'all', 'were', 'ejected', 'in', 'the', 'middle', 'of', 'the', 'ninth', '.'], ['coca-cola', 'co', '.', '(', 'ko', ')', 'reported', 'higher', 'profit', 'for', 'the', 'quarter', ',', 'helped', 'by', 'european', 'demand', ',', 'early', 'on', 'thursday', '.'], ['shares', 'of', 'lendingtree', 'rose', '$', '6.03', ',', 'or', '41', 'percent', ',', 'to', 'close', 'at', '$', '20.72', 'on', 'the', 'nasdaq', 'stock', 'market', 'yesterday', '.'], ['a', 'coalition', 'of', 'campaign', 'groups', ',', 'including', 'the', 'militant', 'national', 'students', 'body', ',', 'is', 'backing', 'the', 'action', 'and', 'is', 'mobilising', 'for', 'protests', '.'], ['the', 'american', 'roman', 'catholic', 'archdiocese', 'of', 'boston', 'has', 'offered', '$', '55', 'million', 'to', 'settle', 'more', 'than', '500', 'sex', 'abuse', 'lawsuits', 'involving', 'priests', '.'], ['u.s.', 'stocks', 'rose', 'slightly', 'on', 'feather-light', 'volume', 'on', 'friday', ',', 'as', 'wall', 'street', 'regrouped', 'after', 'the', 'biggest-ever', 'power', 'outage', 'in', 'north', 'america', '.'], ['generally', 'unwanted', '—', 'and', 'often', 'pornographic', 'or', 'with', 'fraudulent', 'intent', '—', 'spam', 'is', 'a', 'nuisance', 'and', 'a', 'distraction', ',', '"', 'gates', 'writes', '.'], ['it', 'also', 'supports', 'coloured', 'labels', 'to', 'better', 'organise', 'documents', ',', 'and', 'dynamic', 'browsing', 'of', 'the', 'network', 'for', 'mac', ',', 'windows', 'and', 'unix', 'file', 'servers', '.'], ['the', 'analysts', 'said', 'they', 'believe', 'the', 'payroll', 'statistics', 'provide', 'a', 'more', 'accurate', 'picture', 'of', 'the', 'economy', 'because', 'they', 'are', 'based', 'on', 'a', 'larger', 'sample', '.'], ['tom', 'kraynak', ',', 'manager', 'of', 'operations', 'and', 'resources', 'for', 'the', 'canton', ',', 'ohio-based', 'east', 'central', 'area', 'reliability', 'council', ',', 'said', 'investigators', 'are', 'considering', 'the', 'scenario', '.'], ['earlier', ',', 'bashir', "'s", 'supporters', 'had', 'defied', 'a', 'police', 'order', 'and', 'cried', '"', 'allahu', 'akbar', '(', 'god', 'is', 'greatest', ')', '"', 'as', 'he', 'walked', 'to', 'his', 'seat', '.'], ['"', 'biotech', 'products', ',', 'if', 'anything', ',', 'may', 'be', 'safer', 'than', 'conventional', 'products', 'because', 'of', 'all', 'the', 'testing', ',', '"', 'said', 'robert', 'fraley', ',', 'monsanto', "'s", 'executive', 'vice', 'president', '.'], ['maddox', ',', 'who', 'had', 'battled', 'cancer', 'since', '1983', ',', 'cracked', 'two', 'ribs', 'earlier', 'this', 'month', 'when', 'he', 'fell', 'at', 'an', 'assisted', 'living', 'home', 'where', 'he', 'was', 'recovering', 'from', 'surgery', '.'], ['a', 'federal', 'appeals', 'court', 'ruled', 'tuesday', 'that', 'the', 'u.s.', 'government', 'can', 'withhold', 'the', 'names', 'of', 'people', 'detained', 'as', 'part', 'of', 'the', 'september', '11', 'investigation', ',', 'reversing', 'a', 'lower', 'court', 'decision', '.'], ['the', 'world', "'s", 'largest', 'software', 'company', 'said', 'it', 'recognized', 'the', 'difficulty', 'the', 'multiple', 'patches', 'posed', 'for', 'companies', 'trying', 'to', 'apply', 'them', '.'], ['the', 'service', 'will', 'launch', 'later', 'this', 'summer', 'with', '800', 'locations', ',', 'and', 'sprint', 'plans', '2,100', 'locations', 'by', 'the', 'end', 'of', 'the', 'year', '.'], ['indonesia', "'s", 'army', 'has', 'been', 'accused', 'of', 'human', 'rights', 'abuses', 'during', 'its', 'earlier', 'battles', 'with', 'gam', ',', 'charges', 'it', 'has', 'generally', 'denied', '.'], ['iraq', "'s", 'economy', 'was', 'shattered', 'under', 'former', 'president', 'saddam', 'hussein', 'and', 'by', 'the', 'u.s.-led', 'war', 'to', 'oust', 'him', 'which', 'ended', 'in', 'april', '.'], ['"', 'i', 'think', 'this', 'moves', 'us', 'a', 'lot', 'closer', 'toward', 'saying', 'that', 'foam', 'can', 'do', 'this', 'kind', 'of', 'damage', ',', '"', 'hubbard', 'said', '.'], ['the', 'nist', 'discovered', 'that', 'in', '1999', 'the', 'port', 'authority', 'issued', 'guidelines', 'to', 'upgrade', 'the', 'fireproofing', 'to', 'a', 'thickness', 'of', '1', '1', '/', '2', 'inches', '.'], ['now', ',', 'blanca', "'s", 'husband', ',', '63-year-old', 'roger', 'lawrence', 'strunk', 'of', 'tracy', ',', 'faces', 'a', 'murder', 'indictment', 'issued', 'by', 'the', 'philippine', 'government', 'in', 'february', '.'], ['"', 'but', 'i', 'do', 'question', 'the', 'motives', 'of', 'a', 'desk-bound', 'president', 'who', 'assumes', 'the', 'garb', 'of', 'a', 'warrior', 'for', 'the', 'purposes', 'of', 'a', 'speech', '.', '"'], ['wal-', 'mart', ',', 'kohl', "'s", 'corp.', ',', 'family', 'dollar', 'stores', 'inc', '.', ',', 'and', 'big', 'lots', 'inc.', 'posted', 'may', 'sales', 'that', 'fell', 'below', 'wall', 'street', "'s", 'modest', 'expectations', '.'], ['the', 'standard', 'and', 'poor', "'s", '500-stock', 'index', ',', 'a', 'broad', 'collection', 'of', 'equities', 'representing', 'leading', 'companies', ',', 'finished', 'its', 'best', 'quarter', 'yesterday', 'since', 'the', 'last', 'three', 'months', 'of', '1998', '.'], ['president', 'bush', 'has', 'raised', '$', '83.9', 'million', 'since', 'beginning', 'his', 're-election', 'campaign', 'in', 'may', ',', 'and', 'has', '$', '70', 'million', 'of', 'that', 'left', 'to', 'spend', ',', 'his', 'campaign', 'said', 'tuesday', '.'], ['myanmar', "'s", 'pro-democracy', 'leader', 'aung', 'san', 'suu', 'kyi', 'will', 'be', 'kept', 'under', 'house', 'arrest', 'following', 'her', 'release', 'from', 'a', 'hospital', 'where', 'she', 'underwent', 'surgery', ',', 'her', 'personal', 'physician', 'said', 'friday', '.'], ['sean', 'harrigan', ',', 'president', 'of', 'the', 'california', 'public', 'employees', "'", 'retirement', 'system', ',', 'also', 'suggested', 'paring', 'the', 'nyse', "'s", '27-member', 'board', 'and', 'allotting', 'more', 'seats', 'to', 'investors', 'outside', 'the', 'securities', 'industry', '.'], ['at', 'wednesday', "'s", 'summit', ',', 'abbas', 'pledged', 'to', 'end', 'the', '"', 'armed', 'intefadeh', ',', '"', 'renounced', '"', 'terrorism', 'against', 'the', 'israelis', 'wherever', 'they', 'might', 'be', '"', 'and', 'alluded', 'to', 'the', 'disarming', 'of', 'militants', '.'], ['"', 'it', 'has', 'outraged', 'the', 'membership', ',', '"', 'said', 'rian', 'wathen', ',', 'organizing', 'director', 'of', 'united', 'food', 'and', 'commercial', 'workers', 'local', '700', '.'], ['"', 'i', 'don', "'t", 'know', 'whether', 'the', 'person', 'i', "'m", 'talking', 'to', 'now', 'may', 'end', 'up', 'being', 'someone', 'else', ',', '"', 'parrish', 'said', '.'], ['in', 'morning', 'trading', ',', 'the', 'dow', 'jones', 'industrial', 'average', 'was', 'down', '8.76', ',', 'or', '0.1', 'percent', ',', 'at', '9,310.20', ',', 'having', 'gained', '201', 'points', 'monday', '.'], ['"', 'but', 'they', 'did', 'not', ',', 'as', 'of', 'the', 'time', 'of', 'this', 'particular', 'tragic', 'event', ',', 'provide', 'the', 'security', 'we', 'had', 'requested', ',', '"', 'mr', 'jordan', 'said', '.'], ['"', 'we', 'believe', 'we', 'are', 'fully', 'prepared', 'to', 'roll', 'out', 'the', 'revised', 'diebold', 'machines', ',', '"', 'said', 'gilles', 'w.', 'burger', ',', 'chairman', 'of', 'the', 'maryland', 'state', 'board', 'of', 'elections', '.'], ['"', 'we', 'haven', "'t", 'decided', 'whether', 'to', 'use', 'an', 'unbroken', 'or', 'dotted', 'white', 'line', ',', '"', 'said', 'dave', 'richardson', ',', 'the', 'icc', "'s", 'cricket', 'manager', '.'], ['the', 'researchers', 'say', 'that', 'the', 'fact', 'the', 'map', 'bug', 'is', 'present', 'in', 'the', 'vast', 'majority', 'of', 'crohns', 'sufferers', 'means', 'it', 'is', 'almost', 'certainly', 'causing', 'the', 'intestinal', 'inflammation', '.'], ['taiwan', 'reported', '65', 'new', 'cases', 'on', 'may', '22', ',', 'a', 'one-day', 'record', ',', 'and', '55', 'new', 'cases', 'on', 'may', '23', ',', 'making', 'its', 'epidemic', 'the', 'world', "'s", 'fastest-growing', '.'], ['the', 'justice', 'department', 'filed', 'a', 'civil', 'rights', 'lawsuit', 'thursday', 'against', 'the', 'state', 'of', 'mississippi', ',', 'alleging', 'abuse', 'of', 'juvenile', 'offenders', 'at', 'two', 'state-run', 'facilities', '.'], ['the', 'missile', 'exploded', '.'], ['a', 'man', 'jumping', 'rope'], ['raccoons', 'are', 'eating', '.'], ['fish', 'are', 'swimming', '.'], ['three', 'boys', 'are', 'dancing', '.'], ['two', 'zebras', 'are', 'playing', '.'], ['a', 'woman', 'is', 'dancing', '.'], ['the', 'animal', 'is', 'eating', '.'], ['a', 'couple', 'are', 'walking', '.'], ['the', 'elephants', 'are', 'bathing', '.'], ['a', 'woman', 'is', 'dancing', '.'], ['someone', 'is', 'mixing', 'ingrediants', '.'], ['a', 'man', 'is', 'smoking', '.'], ['a', 'woman', 'is', 'writing', '.'], ['a', 'man', 'is', 'fishing', '.'], ['a', 'man', 'is', 'cycling', '.'], ['a', 'man', 'is', 'dancing', '.'], ['two', 'boys', 'are', 'driving', '.'], ['a', 'man', 'is', 'crying', '.'], ['a', 'man', 'is', 'speaking', '.'], ['two', 'men', 'are', 'fighting', '.'], ['a', 'man', 'is', 'levitating', '.'], ['a', 'man', 'is', 'speaking', '.'], ['a', 'man', 'is', 'spinning', '.'], ['three', 'women', 'are', 'dancing', '.'], ['the', 'ballerina', 'is', 'dancing', '.'], ['people', 'are', 'playing', 'cricket', '.'], ['some', 'men', 'are', 'sawing', '.'], ['a', 'woman', 'swims', 'underwater', '.'], ['some', 'men', 'are', 'fighting', '.'], ['a', 'man', 'is', 'swimming', '.'], ['a', 'train', 'is', 'moving', '.'], ['a', 'woman', 'is', 'dancing', '.'], ['a', 'woman', 'is', 'dancing', '.'], ['a', 'woman', 'is', 'dancing', '.'], ['a', 'man', 'is', 'dancing', '.'], ['a', 'woman', 'is', 'dancing', '.'], ['a', 'woman', 'is', 'reading', '.'], ['two', 'men', 'are', 'talking', '.'], ['a', 'man', 'swims', 'underwater', '.'], ['a', 'plane', 'is', 'landing', '.'], ['a', 'person', 'peels', 'shrimp', '.'], ['kittens', 'are', 'eating', 'food', '.'], ['a', 'woman', 'peels', 'shrimp', '.'], ['a', 'woman', 'slices', 'tofu', '.'], ['people', 'are', 'playing', 'baseball', '.'], ['a', 'man', 'is', 'dancing', '.'], ['a', 'guy', 'is', 'playing', 'hackysack'], ['a', 'panda', 'is', 'climbing', '.'], ['a', 'man', 'is', 'dancing', '.'], ['a', 'man', 'is', 'cycling', '.'], ['people', 'are', 'dancing', 'outside', '.'], ['a', 'baby', 'is', 'laughing', '.'], ['a', 'lion', 'is', 'walking', '.'], ['a', 'man', 'is', 'running', '.'], ['a', 'person', 'is', 'writing', '.'], ['a', 'cheetah', 'is', 'running', '.'], ['two', 'women', 'are', 'wrestling', '.'], ['the', 'woman', 'is', 'waterskiing', '.'], ['a', 'man', 'is', 'smoking', '.'], ['two', 'men', 'are', 'fighting', '.'], ['someone', 'typed', 'on', 'a', 'keyboard', '.'], ['a', 'woman', 'opens', 'a', 'window', '.'], ['someone', 'is', 'greasing', 'a', 'pan', '.'], ['a', 'woman', 'is', 'drinking', 'vodka', '.'], ['a', 'woman', 'is', 'slicing', 'butter', '.'], ['a', 'kangroo', 'is', 'eating', 'something', '.'], ['a', 'man', 'cuts', 'up', 'potatos', '.'], ['a', 'group', 'of', 'people', 'sing', '.'], ['someone', 'is', 'feeding', 'a', 'animal', '.'], ['someone', 'is', 'slicing', 'tortila', "'s", '.'], ['someone', 'is', 'holding', 'a', 'hedgehog', '.'], ['a', 'woman', 'is', 'peeling', 'shrimp', '.'], ['a', 'man', 'is', 'playing', 'guitar', '.'], ['a', 'woman', 'is', 'water', 'skiing', '.'], ['a', 'man', 'is', 'finding', 'something', '.'], ['a', 'man', 'is', 'mowing', 'grass', '.'], ['a', 'man', 'is', 'playing', 'soccer', '.'], ['a', 'man', 'is', 'dancing', 'outside', '.'], ['someone', 'is', 'slicing', 'an', 'onion', '.'], ['a', 'woman', 'peels', 'a', 'potato', '.'], ['a', 'woman', 'is', 'wrapping', 'tofu', '.'], ['a', 'man', 'plays', 'a', 'guitar', '.'], ['a', 'woman', 'is', 'slicing', 'tomato', '.'], ['a', 'woman', 'is', 'cutting', 'onions', '.'], ['a', 'girl', 'is', 'making', 'eyelid', '.'], ['the', 'men', 'are', 'playing', 'cricket', '.'], ['a', 'man', 'is', 'slicing', 'potatoes', '.'], ['three', 'men', 'are', 'playing', 'chess', '.'], ['a', 'slow', 'lori', 'walks', 'around', '.'], ['a', 'monkey', 'is', 'playing', 'drums', '.'], ['a', 'woman', 'is', 'eating', 'something', '.'], ['a', 'woman', 'is', 'cooking', 'eggs', '.'], ['someone', 'is', 'slicing', 'a', 'vegetable', '.'], ['two', 'baby', 'pandas', 'are', 'playing', '.'], ['runners', 'race', 'around', 'a', 'track', '.'], ['a', 'man', 'attacks', 'a', 'woman', '.'], ['an', 'animal', 'run', 'in', 'circles', '.'], ['someone', 'is', 'slicing', 'an', 'onion', '.'], ['a', 'woman', 'is', 'frying', 'fish', '.'], ['the', 'lady', 'sliced', 'a', 'tomatoe', '.'], ['a', 'man', 'is', 'eating', 'food', '.'], ['a', 'man', 'is', 'playing', 'guitar', '.'], ['a', 'man', 'plays', 'the', 'violin', '.'], ['a', 'person', 'plays', 'a', 'keyboard', '.'], ['a', 'woman', 'is', 'deveining', 'shrimp', '.'], ['the', 'woman', 'is', 'peeling', 'shrimp', '.'], ['a', 'man', 'playing', 'the', 'guitar', '.'], ['a', 'woman', 'is', 'swimming', 'underwater', '.'], ['a', 'woman', 'is', 'slicing', 'octopus', '.'], ['animals', 'are', 'playing', 'in', 'water', '.'], ['a', 'man', 'is', 'doing', 'push-ups', '.'], ['a', 'man', 'shooting', 'a', 'car', '.'], ['a', 'man', 'is', 'hugging', 'someone', '.'], ['a', 'man', 'shoots', 'a', 'basket', '.'], ['a', 'man', 'is', 'folding', 'paper', '.'], ['a', 'woman', 'is', 'frying', 'something', '.'], ['a', 'man', 'is', 'selling', 'items', '.'], ['a', 'man', 'plays', 'a', 'piano', '.'], ['someone', 'is', 'playing', 'a', 'piano', '.'], ['a', 'man', 'is', 'playing', 'guitar', '.'], ['a', 'woman', 'is', 'slicing', 'garlics', '.'], ['a', 'man', 'is', 'playing', 'keyboard', '.'], ['a', 'woman', 'is', 'cutting', 'tofu', '.'], ['a', 'woman', 'is', 'flipping', 'pancakes', '.'], ['a', 'person', 'is', 'slicing', 'onions', '.'], ['a', 'man', 'is', 'slicing', 'something', '.'], ['kids', 'are', 'dancing', 'on', 'stage', '.'], ['a', 'cat', 'is', 'playing', 'keyboards', '.'], ['a', 'boy', 'is', 'playing', 'guitar', '.'], ['a', 'man', 'is', 'playing', 'guitar', '.'], ['someone', 'is', 'carving', 'a', 'statue', '.'], ['the', 'man', 'played', 'his', 'guitar', '.'], ['a', 'kid', 'is', 'playing', 'guitar', '.'], ['two', 'cartoon', 'rabbits', 'are', 'kissing', '.'], ['a', 'man', 'plays', 'a', 'keyboard', '.'], ['a', 'man', 'drove', 'his', 'car', '.'], ['a', 'woman', 'is', 'playing', 'flute', '.'], ['a', 'dog', 'licks', 'a', 'baby', '.'], ['the', 'lady', 'peeled', 'the', 'potatoe', '.'], ['a', 'monkey', 'pushes', 'another', 'monkey', '.'], ['a', 'plane', 'is', 'taking', 'off', '.'], ['a', 'woman', 'peels', 'a', 'potato', '.'], ['a', 'woman', 'peels', 'a', 'potato', '.'], ['a', 'dog', 'rides', 'a', 'skateboard', '.'], ['a', 'person', 'cuts', 'an', 'onion', '.'], ['a', 'woman', 'rides', 'a', 'horse', '.'], ['a', 'man', 'plays', 'the', 'flute', '.'], ['a', 'man', 'plays', 'the', 'piano', '.'], ['a', 'woman', 'peels', 'an', 'apple', '.'], ['a', 'man', 'is', 'playing', 'guitar', '.'], ['a', 'woman', 'is', 'cracking', 'eggs', '.'], ['a', 'woman', 'is', 'chopping', 'onion', '.'], ['a', 'woman', 'slices', 'a', 'lemon', '.'], ['a', 'woman', 'is', 'slicing', 'ginger', '.'], ['the', 'woman', 'is', 'dicing', 'onions', '.'], ['a', 'man', 'is', 'playing', 'with', 'frogs'], ['a', 'baby', 'is', 'crawling', 'happily', '.'], ['a', 'woman', 'plays', 'the', 'guitar', '.'], ['someone', 'is', 'beating', 'an', 'egg', '.'], ['trees', 'are', 'bending', 'in', 'storm', '.'], ['a', 'man', 'plays', 'the', 'guitar', '.'], ['the', 'turtle', 'followed', 'the', 'fish', '.'], ['a', 'woman', 'holds', 'a', 'kangaroo', '.'], ['three', 'men', 'are', 'playing', 'guitars', '.'], ['the', 'woman', 'is', 'rolling', 'dough', '.'], ['two', 'robots', 'share', 'a', 'kiss', '.'], ['a', 'person', 'is', 'boiling', 'noodles', '.'], ['a', 'pankda', 'is', 'eating', 'bamboo', '.'], ['a', 'boy', 'studies', 'a', 'calendar', '.'], ['a', 'woman', 'holds', 'a', 'kangaroo', '.'], ['a', 'man', 'is', 'playing', 'guitar', '.'], ['a', 'person', 'is', 'boiling', 'soup', '.'], ['a', 'woman', 'is', 'picking', 'tomatoes', '.'], ['a', 'woman', 'mixes', 'up', 'vegetables', '.'], ['some', 'men', 'are', 'playing', 'guitars', '.'], ['a', 'woman', 'is', 'mixing', 'ingrediants', '.'], ['someone', 'fell', 'off', 'a', 'horse', '.'], ['a', 'person', 'is', 'cutting', 'mushrooms', '.'], ['a', 'woman', 'is', 'chopping', 'broccoli', '.'], ['a', 'man', 'shoots', 'a', 'man', '.'], ['people', 'step', 'off', 'a', 'train', '.'], ['the', 'man', 'is', 'eating', 'pizza', '.'], ['a', 'cow', 'is', 'eating', 'grass', '.'], ['a', 'man', 'is', 'cutting', 'bread', '.'], ['a', 'woman', 'is', 'cooking', 'something', '.'], ['the', 'cat', 'is', 'drinking', 'milk', '.'], ['two', 'teams', 'are', 'playing', 'soccer', '.'], ['a', 'woman', 'splits', 'garlic', 'cloves', '.'], ['a', 'cat', 'is', 'licking', 'himself', '.'], ['a', 'monkey', 'is', 'playing', 'drums', '.'], ['a', 'man', 'is', 'shooting', 'a', 'gun', '.'], ['a', 'person', 'is', 'slicing', 'some', 'onions', '.'], ['a', 'person', 'checks', 'some', 'rice', 'pudding', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'boy', 'takes', 'off', 'his', 'sunglasses', '.'], ['the', 'man', 'cut', 'up', 'an', 'onion', '.'], ['a', 'person', 'is', 'slicing', 'an', 'onion', '.'], ['a', 'group', 'of', 'people', 'are', 'dancing', '.'], ['the', 'couple', 'danced', 'in', 'the', 'church', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'is', 'drawing', 'a', 'picture', '.'], ['a', 'black', 'horse', 'is', 'running', 'around', '.'], ['a', 'cat', 'is', 'licking', 'a', 'sucker', '.'], ['a', 'person', 'is', 'cutting', 'a', 'meat', '.'], ['a', 'man', 'is', 'holding', 'a', 'leaf', '.'], ['a', 'person', 'is', 'riding', 'a', 'horse', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'woman', 'is', 'peeling', 'a', 'potato', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'cat', 'is', 'on', 'a', 'robot', '.'], ['a', 'man', 'is', 'riding', 'a', 'bike', '.'], ['a', 'submarine', 'is', 'going', 'through', 'water', '.'], ['the', 'woman', 'is', 'styling', 'her', 'hair', '.'], ['a', 'man', 'is', 'breaking', 'water', 'jugs', '.'], ['a', 'woman', 'is', 'firing', 'a', 'rifle', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'is', 'jumping', 'into', 'water', '.'], ['a', 'man', 'is', 'seasoning', 'some', 'carrots', '.'], ['an', 'overweight', 'man', 'is', 'skipping', 'rope', '.'], ['a', 'woman', 'is', 'feeding', 'a', 'man', '.'], ['a', 'woman', 'is', 'deboning', 'a', 'fish', '.'], ['a', 'man', 'is', 'playing', 'the', 'piano', '.'], ['a', 'man', 'is', 'playing', 'a', 'flute', '.'], ['a', 'woman', 'is', 'slicing', 'a', 'leek', '.'], ['a', 'woman', 'is', 'playing', 'the', 'guitar', '.'], ['a', 'woman', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'woman', 'is', 'peeling', 'a', 'potato', '.'], ['a', 'woman', 'is', 'cutting', 'an', 'onion', '.'], ['a', 'woman', 'is', 'slicing', 'some', 'vegetables', '.'], ['a', 'woman', 'is', 'cutting', 'some', 'herbs', '.'], ['a', 'man', 'is', 'cutting', 'a', 'fish', '.'], ['a', 'person', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'turtle', 'is', 'swimming', 'in', 'water', '.'], ['the', 'elephants', 'sprayed', 'water', 'on', 'themselves', '.'], ['a', 'man', 'is', 'peeling', 'a', 'potato', '.'], ['the', 'woman', 'is', 'cutting', 'cooked', 'octopus', '.'], ['a', 'chef', 'is', 'preparing', 'some', 'food', '.'], ['a', 'girl', 'is', 'riding', 'a', 'horse', '.'], ['a', 'woman', 'is', 'cutting', 'an', 'onion', '.'], ['a', 'woman', 'is', 'chopping', 'up', 'garlic', '.'], ['a', 'man', 'is', 'climbing', 'a', 'rope', '.'], ['a', 'man', 'is', 'playing', 'the', 'drums', '.'], ['a', 'man', 'is', 'cutting', 'an', 'onion', '.'], ['a', 'man', 'is', 'tying', 'his', 'shoe', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['the', 'boy', 'is', 'playing', 'the', 'piano', '.'], ['the', 'man', 'is', 'riding', 'a', 'horse', '.'], ['a', 'man', 'is', 'making', 'a', 'bed', '.'], ['a', 'man', 'is', 'riding', 'a', 'horse', '.'], ['a', 'man', 'is', 'riding', 'a', 'skateboard', '.'], ['a', 'man', 'is', 'holding', 'a', 'leaf', '.'], ['a', 'chimpanzee', 'is', 'hurting', 'a', 'woman', '.'], ['a', 'woman', 'is', 'cutting', 'a', 'cucumber', '.'], ['a', 'woman', 'is', 'filing', 'her', 'nails', '.'], ['a', 'cat', 'is', 'watching', 'a', 'tv', '.'], ['a', 'girl', 'is', 'eating', 'a', 'cupcake', '.'], ['a', 'boy', 'is', 'pushing', 'a', 'van', '.'], ['a', 'man', 'is', 'jumping', 'rope', 'outside', '.'], ['plastic', 'toys', 'are', 'floating', 'in', 'water', '.'], ['a', 'woman', 'is', 'mincing', 'some', 'garlic', '.'], ['a', 'girl', 'is', 'playing', 'a', 'piano', '.'], ['three', 'children', 'are', 'eating', 'their', 'food', '.'], ['a', 'woman', 'is', 'slicing', 'big', 'pepper', '.'], ['a', 'man', 'is', 'cutting', 'a', 'potato', '.'], ['a', 'woman', 'is', 'picking', 'a', 'can', '.'], ['a', 'man', 'is', 'mowing', 'a', 'lawn', '.'], ['a', 'man', 'is', 'riding', 'a', 'horse', '.'], ['a', 'man', 'is', 'smelling', 'two', 'pizzas', '.'], ['a', 'woman', 'is', 'slicing', 'a', 'meat', '.'], ['a', 'woman', 'is', 'taking', 'a', 'picture', '.'], ['a', 'girl', 'is', 'playing', 'a', 'violin', '.'], ['a', 'woman', 'is', 'cutting', 'an', 'onion', '.'], ['a', 'man', 'is', 'rowing', 'a', 'boat', '.'], ['a', 'man', 'is', 'punching', 'through', 'boards', '.'], ['a', 'woman', 'is', 'slicing', 'a', 'pumpkin', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'woman', 'is', 'doing', 'weight', 'exercises', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'woman', 'is', 'taking', 'a', 'bath', '.'], ['a', 'cat', 'is', 'playing', 'a', 'piano', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'is', 'rinsing', 'a', 'steak', '.'], ['a', 'man', 'is', 'paddling', 'a', 'canoe', '.'], ['a', 'man', 'is', 'riding', 'a', 'motorcycle', '.'], ['a', 'man', 'is', 'opening', 'a', 'door', '.'], ['a', 'man', 'is', 'opening', 'a', 'door', '.'], ['a', 'man', 'is', 'slicing', 'a', 'tomato', '.'], ['a', 'man', 'is', 'playing', 'a', 'flute', '.'], ['a', 'man', 'is', 'playing', 'the', 'piano', '.'], ['a', 'woman', 'is', 'playing', 'the', 'flute', '.'], ['a', 'woman', 'is', 'picking', 'a', 'can', '.'], ['a', 'man', 'is', 'assembling', 'speaker', 'parts', '.'], ['a', 'man', 'is', 'driving', 'a', 'car', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['the', 'boy', 'is', 'playing', 'the', 'piano', '.'], ['a', 'man', 'is', 'riding', 'a', 'motorcycle', '.'], ['a', 'man', 'is', 'riding', 'a', 'motorcycle', '.'], ['a', 'man', 'is', 'lifting', 'a', 'car', '.'], ['a', 'man', 'is', 'playing', 'a', 'basketball', '.'], ['a', 'woman', 'is', 'slicing', 'some', 'tomatoes', '.'], ['a', 'man', 'is', 'slicing', 'a', 'potato', '.'], ['a', 'man', 'is', 'playing', 'a', 'harp', '.'], ['a', 'man', 'is', 'playing', 'the', 'guitar', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['the', 'man', 'is', 'playing', 'the', 'piano', '.'], ['a', 'man', 'is', 'playing', 'the', 'piano', '.'], ['a', 'man', 'and', 'woman', 'are', 'talking', '.'], ['a', 'man', 'is', 'playing', 'a', 'flute', '.'], ['the', 'man', 'is', 'playing', 'the', 'guitar', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'person', 'is', 'slicing', 'a', 'tomato', '.'], ['a', 'man', 'is', 'playing', 'the', 'piano', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'girl', 'is', 'riding', 'a', 'horse', '.'], ['a', 'woman', 'is', 'peeling', 'a', 'potato', '.'], ['a', 'person', 'is', 'peeling', 'an', 'onion', '.'], ['a', 'man', 'is', 'slicing', 'a', 'tomato', '.'], ['a', 'man', 'is', 'riding', 'a', 'bicycle', '.'], ['a', 'man', 'is', 'playing', 'a', 'piano', '.'], ['a', 'man', 'is', 'playing', 'the', 'drums', '.'], ['a', 'man', 'is', 'slicing', 'an', 'onion', '.'], ['a', 'woman', 'is', 'carrying', 'a', 'boy', '.'], ['a', 'man', 'is', 'slicing', 'a', 'bun', '.'], ['the', 'girl', 'sang', 'into', 'a', 'microphone', '.'], ['the', 'lady', 'cut', 'up', 'a', 'potatoe', '.'], ['a', 'girl', 'is', 'styling', 'her', 'hair', '.'], ['a', 'boy', 'is', 'playing', 'an', 'instrument', '.'], ['the', 'cook', 'is', 'kneading', 'the', 'flour', '.'], ['a', 'woman', 'is', 'playing', 'the', 'flute', '.'], ['a', 'woman', 'is', 'slicing', 'an', 'onion', '.'], ['people', 'flip', 'into', 'a', 'swimming', 'pool', '.'], ['a', 'man', 'is', 'pressing', 'microwave', 'buttons', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'woman', 'is', 'slicing', 'some', 'tuna', '.'], ['a', 'plane', 'rides', 'on', 'a', 'road', '.'], ['a', 'woman', 'is', 'slicing', 'an', 'onion', '.'], ['a', 'boy', 'is', 'playing', 'a', 'key-board', '.'], ['a', 'girl', 'is', 'playing', 'a', 'flute', '.'], ['the', 'woman', 'picked', 'up', 'the', 'kangaroo', '.'], ['a', 'man', 'is', 'playing', 'the', 'guitar', '.'], ['the', 'person', 'is', 'riding', 'the', 'horse', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'is', 'sitting', 'and', 'smoking', '.'], ['two', 'dogs', 'swim', 'in', 'a', 'pool', '.'], ['a', 'squirrel', 'runs', 'around', 'in', 'circles', '.'], ['a', 'man', 'is', 'playing', 'a', 'piano', '.'], ['a', 'man', 'is', 'cutting', 'a', 'potato', '.'], ['a', 'badger', 'is', 'burrowing', 'a', 'hole', '.'], ['the', 'boy', 'fell', 'off', 'his', 'bike', '.'], ['the', 'man', 'is', 'playing', 'the', 'guitar', '.'], ['the', 'man', 'is', 'playing', 'the', 'guitar', '.'], ['the', 'woman', 'is', 'playing', 'the', 'flute', '.'], ['the', 'cougar', 'is', 'chasing', 'the', 'bear', '.'], ['a', 'man', 'is', 'climbing', 'a', 'rope', '.'], ['a', 'woman', 'is', 'taking', 'a', 'shower', '.'], ['a', 'man', 'is', 'playing', 'a', 'violin', '.'], ['a', 'man', 'is', 'playing', 'a', 'flute', '.'], ['a', 'person', 'is', 'peeling', 'a', 'potato', '.'], ['the', 'man', 'is', 'playing', 'the', 'drums', '.'], ['a', 'cat', 'is', 'drinking', 'some', 'water', '.'], ['a', 'man', 'plays', 'an', 'acoustic', 'guitar', '.'], ['a', 'person', 'is', 'slicing', 'an', 'onion', '.'], ['a', 'man', 'is', 'riding', 'a', 'bike', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'walks', 'down', 'a', 'path', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'is', 'rowing', 'a', 'boat', '.'], ['a', 'person', 'is', 'petting', 'a', 'monkey', '.'], ['the', 'boy', 'is', 'riding', 'a', 'tricycle', '.'], ['a', 'girl', 'is', 'playing', 'a', 'flute', '.'], ['a', 'dog', 'biting', 'on', 'a', 'can', '.'], ['a', 'woman', 'is', 'dicing', 'a', 'potato', '.'], ['a', 'man', 'is', 'riding', 'a', 'bike', '.'], ['people', 'ride', 'and', 'paddle', 'a', 'raft', '.'], ['the', 'cat', 'is', 'licking', 'a', 'bottle', '.'], ['the', 'baby', 'crawled', 'through', 'the', 'doorway', '.'], ['someone', 'is', 'sitring', 'noodles', 'in', 'water', '.'], ['a', 'man', 'is', 'jumping', 'a', 'wall', '.'], ['a', 'lion', 'is', 'playing', 'with', 'people', '.'], ['a', 'small', 'monkey', 'walks', 'through', 'water', '.'], ['the', 'hedgehog', 'chewed', 'on', 'the', 'towel', '.'], ['the', 'man', 'talked', 'on', 'the', 'telephone', '.'], ['an', 'orangutan', 'is', 'riding', 'a', 'bus', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['the', 'woman', 'picked', 'up', 'the', 'kangaroo', '.'], ['a', 'turtle', 'walks', 'over', 'the', 'ground', '.'], ['a', 'boy', 'is', 'checking', 'a', 'calender', '.'], ['a', 'man', 'is', 'playing', 'a', 'flute', '.'], ['a', 'woman', 'is', 'slicing', 'a', 'pepper', '.'], ['a', 'girl', 'is', 'flying', 'a', 'kite', '.'], ['paper', 'is', 'being', 'cut', 'with', 'scissors', '.'], ['the', 'rhino', 'grazed', 'on', 'the', 'grass', '.'], ['a', 'man', 'is', 'playing', 'the', 'cello', '.'], ['the', 'man', 'cut', 'a', 'paper', 'plate', '.'], ['the', 'man', 'is', 'riding', 'a', 'horse', '.'], ['a', 'man', 'is', 'tying', 'his', 'shoe', '.'], ['the', 'woman', 'is', 'seasoning', 'the', 'oil', '.'], ['a', 'person', 'is', 'chopping', 'coriander', 'leaves', '.'], ['an', 'animal', 'is', 'chewing', 'on', 'something', '.'], ['a', 'man', 'is', 'playing', 'the', 'guitar', '.'], ['the', 'lady', 'cracked', 'two', 'raw', 'eggs', '.'], ['a', 'man', 'is', 'eating', 'a', 'food', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'toddler', 'walks', 'down', 'a', 'hallway', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['butter', 'is', 'melting', 'in', 'a', 'pan', '.'], ['a', 'machine', 'is', 'sharpening', 'a', 'pencil', '.'], ['a', 'man', 'is', 'slicing', 'a', 'pepper', '.'], ['the', 'woman', 'is', 'walking', 'her', 'dog', '.'], ['a', 'dog', 'is', 'eating', 'water', 'melon', '.'], ['a', 'cat', 'is', 'eating', 'some', 'corn', '.'], ['a', 'woman', 'is', 'powdering', 'her', 'face', '.'], ['a', 'woman', 'cuts', 'up', 'some', 'vegetables', '.'], ['the', 'people', 'got', 'off', 'the', 'train', '.'], ['elephants', 'are', 'walking', 'down', 'a', 'trail', '.'], ['a', 'man', 'is', 'carrying', 'another', 'man', '.'], ['a', 'woman', 'is', 'peeling', 'an', 'orange', '.'], ['a', 'man', 'is', 'playing', 'a', 'football', '.'], ['a', 'group', 'of', 'penguins', 'is', 'walking', '.'], ['a', 'woman', 'is', 'chopping', 'an', 'onion', '.'], ['a', 'man', 'is', 'playing', 'the', 'guitar', '.'], ['a', 'woman', 'is', 'peeling', 'some', 'fish', '.'], ['a', 'woman', 'is', 'slicing', 'some', 'tofu', '.'], ['a', 'boy', 'is', 'playing', 'a', 'key-board', '.'], ['a', 'helicopter', 'is', 'landing', 'on', 'a', 'helipad', '.'], ['a', 'person', 'is', 'surfing', 'a', 'large', 'wave', '.'], ['a', 'little', 'boy', 'is', 'vacuuming', 'the', 'floor', '.'], ['a', 'cat', 'is', 'walking', 'around', 'a', 'house', '.'], ['a', 'ballet', 'dancer', 'is', 'performing', 'on', 'stage', '.'], ['a', 'girl', 'is', 'dancing', 'in', 'a', 'cage', '.'], ['a', 'cat', 'is', 'playing', 'on', 'the', 'floor', '.'], ['two', 'puppies', 'are', 'playing', 'with', 'each', 'other', '.'], ['a', 'man', 'is', 'kicking', 'pots', 'of', 'water', '.'], ['a', 'woman', 'is', 'dancing', 'with', 'other', 'women', '.'], ['a', 'woman', 'is', 'speaking', 'in', 'sign', 'language', '.'], ['a', 'man', 'is', 'running', 'on', 'a', 'street', '.'], ['a', 'woman', 'peels', 'garlic', 'with', 'her', 'hands', '.'], ['a', 'bear', 'is', 'running', 'from', 'two', 'dogs', '.'], ['a', 'tiger', 'cub', 'is', 'making', 'a', 'sound', '.'], ['a', 'woman', 'is', 'cutting', 'up', 'a', 'chicken', '.'], ['a', 'person', 'is', 'playing', 'an', 'electronic', 'keyboard', '.'], ['two', 'men', 'pushed', 'carts', 'through', 'the', 'woods', '.'], ['the', 'panda', 'bear', 'lay', 'on', 'the', 'logs', '.'], ['a', 'person', 'is', 'cutting', 'some', 'burrata', 'cheese', '.'], ['a', 'man', 'and', 'a', 'woman', 'are', 'kissing', '.'], ['a', 'band', 'is', 'performing', 'on', 'a', 'stage', '.'], ['a', 'young', 'man', 'is', 'playing', 'the', 'piano', '.'], ['a', 'man', 'is', 'taking', 'some', 'math', 'classes', '.'], ['a', 'woman', 'is', 'riding', 'on', 'a', 'horse', '.'], ['a', 'man', 'is', 'moving', 'a', 'tree', 'limb', '.'], ['a', 'cat', 'is', 'pouncing', 'on', 'a', 'trampoline', '.'], ['a', 'dog', 'is', 'jumping', 'on', 'a', 'trampoline', '.'], ['a', 'boy', 'is', 'playing', 'violin', 'on', 'stage', '.'], ['a', 'badger', 'is', 'digging', 'in', 'the', 'dirt', '.'], ['a', 'woman', 'is', 'sitting', 'at', 'a', 'desk', '.'], ['the', 'man', 'is', 'driving', 'a', 'white', 'car', '.'], ['a', 'man', 'is', 'walking', 'on', 'a', 'rope', '.'], ['a', 'man', 'is', 'kicking', 'a', 'soccer', 'ball', '.'], ['a', 'woman', 'is', 'sewing', 'on', 'a', 'machine', '.'], ['a', 'person', 'pours', 'oil', 'on', 'a', 'skillet', '.'], ['a', 'man', 'is', 'running', 'on', 'a', 'field', '.'], ['a', 'man', 'is', 'playing', 'an', 'electronic', 'keyboard', '.'], ['a', 'woman', 'is', 'dancing', 'by', 'a', 'fire', '.'], ['a', 'cat', 'is', 'playing', 'a', 'key', 'board', '.'], ['a', 'man', 'is', 'cutting', 'tomatoes', 'into', 'pieces', '.'], ['the', 'man', 'is', 'slicing', 'a', 'fish', 'open', '.'], ['a', 'woman', 'is', 'slicing', 'up', 'some', 'meat', '.'], ['a', 'man', 'is', 'cutting', 'up', 'a', 'potato', '.'], ['a', 'skunk', 'is', 'looking', 'here', 'and', 'there', '.'], ['the', 'military', 'officer', 'barked', 'at', 'the', 'recruits', '.'], ['a', 'man', 'is', 'riding', 'on', 'a', 'horse', '.'], ['the', 'cats', 'are', 'playing', 'with', 'each', 'other', '.'], ['the', 'person', 'lit', 'the', 'cameras', 'on', 'fire', '.'], ['the', 'polar', 'bears', 'fought', 'over', 'the', 'kill', '.'], ['a', 'man', 'is', 'riding', 'an', 'electric', 'bicycle', '.'], ['a', 'woman', 'is', 'playing', 'an', 'electric', 'guitar', '.'], ['a', 'man', 'is', 'playing', 'a', 'large', 'flute', '.'], ['the', 'man', 'is', 'shooting', 'an', 'automatic', 'rifle', '.'], ['a', 'man', 'is', 'riding', 'a', 'mechanical', 'bull', '.'], ['a', 'squirrel', 'is', 'spinning', 'around', 'in', 'circles', '.'], ['a', 'man', 'is', 'cutting', 'up', 'a', 'cucumber', '.'], ['a', 'little', 'boy', 'is', 'playing', 'a', 'keyboard', '.'], ['a', 'baby', 'panda', 'goes', 'down', 'a', 'slide', '.'], ['a', 'panda', 'is', 'sliding', 'down', 'a', 'slide', '.'], ['a', 'young', 'child', 'is', 'riding', 'a', 'horse', '.'], ['a', 'dog', 'is', 'jumping', 'on', 'a', 'trampoline', '.'], ['a', 'dog', 'is', 'running', 'on', 'the', 'ground', '.'], ['a', 'man', 'is', 'talking', 'on', 'a', 'phone', '.'], ['a', 'woman', 'is', 'playing', 'in', 'the', 'ocean', '.'], ['a', 'man', 'is', 'running', 'on', 'the', 'road', '.'], ['a', 'man', 'is', 'riding', 'on', 'a', 'horse', '.'], ['a', 'car', 'is', 'driven', 'down', 'the', 'road', '.'], ['a', 'dog', 'is', 'barking', 'at', 'a', 'ball', '.'], ['a', 'dog', 'is', 'looking', 'into', 'swimming', 'pool', '.'], ['a', 'rat', 'like', 'creature', 'is', 'smelling', 'around', '.'], ['a', 'woman', 'is', 'adding', 'oil', 'on', 'fishes', '.'], ['a', 'young', 'asian', 'girl', 'is', 'applying', 'eyeliner', '.'], ['a', 'young', 'boy', 'is', 'holding', 'a', 'guitar', '.'], ['someone', 'is', 'pouring', 'ingrediants', 'into', 'a', 'pot', '.'], ['a', 'man', 'pours', 'oil', 'into', 'a', 'pot', '.'], ['an', 'explosion', 'is', 'booming', 'in', 'the', 'desert', '.'], ['a', 'girl', 'is', 'communicating', 'with', 'sign', 'language', '.'], ['a', 'man', 'is', 'playing', 'on', 'his', 'keyboard', '.'], ['a', 'car', 'is', 'moving', 'through', 'a', 'road', '.'], ['a', 'woman', 'puts', 'cosmetics', 'on', 'her', 'eyelid', '.'], ['a', 'rabbit', 'is', 'running', 'from', 'an', 'eagle', '.'], ['two', 'zebras', 'play', 'in', 'an', 'open', 'field', '.'], ['a', 'man', 'is', 'slicing', 'open', 'a', 'fish', '.'], ['a', 'man', 'rides', 'off', 'on', 'a', 'motorcycle', '.'], ['the', 'lady', 'chopped', 'up', 'the', 'green', 'onions', '.'], ['a', 'woman', 'is', 'cutting', 'meat', 'into', 'pieces', '.'], ['a', 'man', 'is', 'erasing', 'a', 'chalk', 'board', '.'], ['a', 'woman', 'is', 'using', 'a', 'sewing', 'machine', '.'], ['a', 'woman', 'is', 'riding', 'on', 'a', 'horse', '.'], ['a', 'cat', 'is', 'jumping', 'into', 'a', 'box', '.'], ['a', 'boy', 'is', 'making', 'some', 'dance', 'steps', '.'], ['a', 'cat', 'is', 'playing', 'in', 'a', 'box', '.'], ['a', 'man', 'is', 'running', 'on', 'the', 'road', '.'], ['three', 'teenage', 'girls', 'danced', 'in', 'the', 'room', '.'], ['five', 'men', 'jump', 'into', 'a', 'swimming', 'pool', '.'], ['an', 'animal', 'is', 'biting', 'a', 'persons', 'finger', '.'], ['medics', 'transport', 'a', 'patient', 'on', 'a', 'stretcher', '.'], ['a', 'kid', 'plays', 'with', 'a', 'toy', 'phone', '.'], ['a', 'woman', 'is', 'chopping', 'a', 'hard', 'egg', '.'], ['a', 'man', 'mixes', 'vegetables', 'in', 'a', 'pot', '.'], ['a', 'dog', 'is', 'barking', 'at', 'a', 'toy', '.'], ['the', 'shirtless', 'man', 'sat', 'in', 'a', 'chair', '.'], ['a', 'puppy', 'plays', 'with', 'a', 'plastic', 'container', '.'], ['a', 'man', 'pours', 'oil', 'on', 'some', 'tomatos', '.'], ['lionel', 'richie', 'is', 'dancing', 'on', 'the', 'ceiling', '.'], ['a', 'rooster', 'pecks', 'at', 'a', 'dead', 'mouse', '.'], ['butter', 'is', 'being', 'put', 'into', 'a', 'bowl', '.'], ['a', 'cheetah', 'is', 'running', 'behind', 'its', 'prey', '.'], ['a', 'woman', 'is', 'dancing', 'in', 'the', 'rain', '.'], ['two', 'sumo', 'ringers', 'are', 'fighting', 'each', 'other', '.'], ['a', 'man', 'is', 'fishing', 'in', 'the', 'river', '.'], ['a', 'man', 'is', 'dragging', 'a', 'dead', 'deer', '.'], ['a', 'car', 'backs', 'out', 'of', 'a', 'space', '.'], ['a', 'child', 'is', 'playing', 'a', 'computer', 'game', '.'], ['a', 'woman', 'is', 'swimming', 'in', 'the', 'beach', '.'], ['a', 'man', 'is', 'getting', 'inside', 'a', 'car', '.'], ['meat', 'is', 'being', 'put', 'into', 'a', 'skillet', '.'], ['someone', 'is', 'boiling', 'okra', 'in', 'a', 'pot', '.'], ['a', 'dog', 'is', 'looking', 'into', 'swimming', 'pool', '.'], ['the', 'woman', 'is', 'pencilling', 'on', 'eye', 'shadow', '.'], ['someone', 'is', 'stirring', 'chili', 'in', 'a', 'kettle', '.'], ['a', 'man', 'is', 'getting', 'into', 'a', 'car', '.'], ['a', 'couple', 'are', 'running', 'towards', 'the', 'ocean', '.'], ['an', 'airplane', 'is', 'flying', 'in', 'the', 'air', '.'], ['a', 'man', 'is', 'walking', 'with', 'a', 'horse', '.'], ['a', 'dog', 'is', 'playing', 'in', 'the', 'snow', '.'], ['a', 'small', 'baby', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'seated', 'is', 'playing', 'the', 'guitar', '.'], ['someone', 'touched', 'the', 'kitten', 'chewing', 'on', 'a', 'string', '.'], ['a', 'woman', 'is', 'adding', 'sauce', 'to', 'some', 'food', '.'], ['a', 'dog', 'licks', 'up', 'a', 'baby', "'s", 'drool', '.'], ['a', 'man', 'is', 'tying', 'on', 'a', 'stenographers', 'machine', '.'], ['the', 'man', 'stirred', 'the', 'sauce', 'for', 'the', 'chicken', '.'], ['batman', 'and', 'robin', 'fly', 'a', 'helicopter', 'over', 'water', '.'], ['men', 'are', 'jumping', 'into', 'water', 'from', 'a', 'bridge', '.'], ['a', 'person', 'is', 'folding', 'a', 'square', 'paper', 'piece', '.'], ['a', 'man', 'is', 'lifting', 'weights', 'in', 'a', 'garage', '.'], ['a', 'woman', 'is', 'water', 'skiing', 'on', 'a', 'lake', '.'], ['a', 'woman', 'is', 'frying', 'something', 'in', 'the', 'pan', '.'], ['a', 'baby', 'elephant', 'is', 'eating', 'a', 'small', 'tree', '.'], ['a', 'woman', 'is', 'slicing', 'meat', 'with', 'a', 'knife', '.'], ['a', 'man', 'is', 'cutting', 'paper', 'with', 'a', 'sword', '.'], ['a', 'person', 'is', 'frying', 'meat', 'in', 'a', 'pan', '.'], ['a', 'man', 'making', 'a', 'bed', 'in', 'a', 'hotel', '.'], ['the', 'lady', 'cracked', 'an', 'egg', 'for', 'the', 'mixer', '.'], ['a', 'woman', 'is', 'adding', 'spices', 'on', 'a', 'meat', '.'], ['a', 'man', 'is', 'looking', 'at', 'a', 'computer', 'monitor', '.'], ['five', 'kittens', 'are', 'eating', 'out', 'of', 'five', 'dishes', '.'], ['a', 'man', 'is', 'playing', 'the', 'guitar', 'and', 'singing', '.'], ['a', 'boy', 'is', 'singing', 'and', 'playing', 'the', 'piano', '.'], ['a', 'man', 'is', 'running', 'across', 'a', 'high', 'wire', '.'], ['a', 'group', 'of', 'people', 'dance', 'on', 'a', 'hill', '.'], ['a', 'man', 'is', 'singing', 'and', 'playing', 'a', 'guitar', '.'], ['a', 'woman', 'is', 'playing', 'a', 'flute', 'on', 'stage', '.'], ['a', 'puppy', 'is', 'sliding', 'backwards', 'along', 'the', 'floor', '.'], ['a', 'small', 'dog', 'is', 'chasing', 'a', 'yoga', 'ball', '.'], ['a', 'man', 'is', 'slicing', 'a', 'potato', 'into', 'pieces', '.'], ['someone', 'is', 'shredding', 'cabbage', 'leaves', 'with', 'a', 'knife', '.'], ['a', 'man', 'is', 'playing', 'the', 'guitar', 'and', 'singing', '.'], ['a', 'man', 'sharpens', 'a', 'pencil', 'using', 'a', 'knife', '.'], ['a', 'boy', 'is', 'crawling', 'into', 'a', 'dog', 'house', '.'], ['a', 'man', 'is', 'buttering', 'a', 'piece', 'of', 'bread', '.'], ['a', 'man', 'is', 'looking', 'out', 'of', 'a', 'window', '.'], ['a', 'boy', 'is', 'crawling', 'into', 'a', 'dog', 'house', '.'], ['a', 'baby', 'tiger', 'is', 'playing', 'with', 'a', 'ball', '.'], ['a', 'person', 'is', 'adding', 'food', 'in', 'a', 'pan', '.'], ['a', 'man', 'riding', 'a', 'horse', 'in', 'slow', 'motion', '.'], ['a', 'cat', 'is', 'rubbing', 'against', 'baby', "'s", 'face', '.'], ['a', 'woman', 'is', 'resting', 'in', 'a', 'floating', 'raft', '.'], ['two', 'women', 'mix', 'something', 'in', 'a', 'food', 'processor', '.'], ['a', 'big', 'green', 'ball', 'knocks', 'a', 'potato', 'over', '.'], ['a', 'rider', 'is', 'running', 'and', 'jumping', 'her', 'horse', '.'], ['a', 'person', 'is', 'folding', 'a', 'piece', 'of', 'paper', '.'], ['one', 'woman', 'is', 'measuring', 'another', 'woman', "'s", 'ankle', '.'], ['a', 'man', 'puts', 'some', 'cheese', 'on', 'a', 'pizza', '.'], ['a', 'man', 'is', 'buttering', 'a', 'piece', 'of', 'bread', '.'], ['a', 'man', 'is', 'doing', 'trick', 'with', 'play', 'cards', '.'], ['a', 'man', 'is', 'screwing', 'wood', 'to', 'a', 'wall', '.'], ['a', 'man', 'is', 'dieing', 'in', 'an', 'operating', 'room', '.'], ['two', 'little', 'girls', 'are', 'talking', 'on', 'the', 'phone', '.'], ['a', 'woman', 'is', 'cracking', 'eggs', 'into', 'a', 'bowl', '.'], ['a', 'man', 'is', 'preparing', 'a', 'snake', 'for', 'cooking', '.'], ['a', 'man', 'is', 'cutting', 'a', 'pipe', 'with', 'scissors', '.'], ['a', 'man', 'is', 'cutting', 'apple', 'by', 'his', 'hand', '.'], ['the', 'man', 'is', 'putting', 'chicken', 'into', 'the', 'container', '.'], ['the', 'lady', 'cracked', 'an', 'egg', 'into', 'a', 'bowl', '.'], ['two', 'men', 'compete', 'in', 'a', 'tire', 'rolling', 'race', '.'], ['a', 'herd', 'of', 'deer', 'are', 'crossing', 'the', 'street', '.'], ['a', 'woman', 'beats', 'two', 'eggs', 'in', 'a', 'bowl', '.'], ['a', 'monkey', 'is', 'swinging', 'from', 'limb', 'to', 'limb', '.'], ['the', 'puppy', 'played', 'with', 'a', 'blue', 'tennis', 'ball', '.'], ['the', 'woman', 'is', 'frying', 'a', 'breaded', 'pork', 'chop', '.'], ['a', 'man', 'is', 'pouring', 'oil', 'into', 'a', 'pan', '.'], ['a', 'rabbit', 'is', 'playing', 'with', 'a', 'toy', 'rabbit', '.'], ['the', 'man', 'cut', 'some', 'carpet', 'with', 'a', 'knife', '.'], ['the', 'woman', 'is', 'cracking', 'eggs', 'into', 'a', 'bowl', '.'], ['a', 'man', 'with', 'a', 'hard', 'hat', 'is', 'dancing', '.'], ['the', 'polar', 'bear', 'is', 'sliding', 'on', 'the', 'snow', '.'], ['the', 'man', 'is', 'kissing', 'and', 'hugging', 'the', 'woman', '.'], ['a', 'man', 'is', 'petting', 'a', 'tied', 'up', 'dog', '.'], ['a', 'girl', 'is', 'singing', 'and', 'playing', 'a', 'guitar', '.'], ['a', 'monkey', 'pulls', 'the', 'tail', 'of', 'a', 'dog', '.'], ['a', 'woman', 'is', 'talking', 'on', 'a', 'cell', 'phone', '.'], ['a', 'cat', 'is', 'trying', 'to', 'touch', 'a', 'dog', '.'], ['an', 'elderly', 'man', 'is', 'speaking', 'at', 'a', 'podium', '.'], ['a', 'man', 'is', 'fitting', 'silencer', 'on', 'a', 'pistol', '.'], ['a', 'person', 'is', 'drawing', 'on', 'a', 'large', 'touchscreen', '.'], ['the', 'man', 'sat', 'in', 'his', 'chair', 'and', 'stared', '.'], ['the', 'woman', 'is', 'putting', 'paste', 'into', 'a', 'dish', '.'], ['a', 'man', 'and', 'woman', 'is', 'riding', 'in', 'a', 'car', '.'], ['a', 'woman', 'is', 'playing', 'a', 'game', 'with', 'a', 'man', '.'], ['a', 'young', 'man', 'is', 'singing', 'and', 'playing', 'the', 'piano', '.'], ['a', 'man', 'is', 'putting', 'a', 'pan', 'into', 'an', 'oven', '.'], ['a', 'woman', 'is', 'riding', 'a', 'horse', 'along', 'a', 'perimeter', '.'], ['a', 'person', 'is', 'frying', 'a', 'meat', 'in', 'the', 'pan', '.'], ['the', 'men', 'played', 'follow', 'the', 'leader', 'on', 'the', 'grass', '.'], ['a', 'woman', 'is', 'slicing', 'an', 'onion', 'with', 'a', 'knife', '.'], ['a', 'man', 'is', 'playing', 'a', 'flute', 'in', 'a', 'band', '.'], ['the', 'little', 'boy', 'is', 'singing', 'and', 'playing', 'the', 'guitar', '.'], ['a', 'man', 'and', 'a', 'woman', 'walk', 'through', 'the', 'woods', '.'], ['a', 'man', 'is', 'eating', 'a', 'banana', 'by', 'a', 'tree', '.'], ['a', 'person', 'is', 'doing', 'pencil', 'tricks', 'with', 'his', 'hand', '.'], ['a', 'woman', 'is', 'opening', 'a', 'soda', 'and', 'drinking', 'it', '.'], ['a', 'man', 'holds', 'a', 'small', 'animal', 'in', 'one', 'hand', '.'], ['a', 'woman', 'dances', 'while', 'a', 'man', 'plays', 'the', 'keyboard', '.'], ['a', 'man', 'is', 'walking', 'along', 'a', 'path', 'through', 'wilderness', '.'], ['a', 'dog', 'sat', 'at', 'the', 'counter', 'of', 'a', 'store', '.'], ['a', 'cat', 'rapidly', 'swats', 'a', 'kid', 'in', 'the', 'head', '.'], ['a', 'man', 'is', 'walking', 'slowly', 'across', 'a', 'rope', 'bridge', '.'], ['an', 'old', 'woman', 'is', 'shaking', 'hand', 'with', 'a', 'man', '.'], ['the', 'man', 'spread', 'butter', 'on', 'the', 'loaf', 'of', 'bread', '.'], ['the', 'man', 'is', 'slicing', 'the', 'tape', 'from', 'the', 'box', '.'], ['the', 'dog', 'walked', 'along', 'the', 'edge', 'of', 'the', 'pool', '.'], ['the', 'lady', 'stirred', 'up', 'raw', 'eggs', 'in', 'the', 'bowl', '.'], ['a', 'little', 'boy', 'is', 'singing', 'and', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'and', 'woman', 'are', 'eating', 'at', 'a', 'table', '.'], ['a', 'man', 'chops', 'down', 'a', 'tree', 'with', 'an', 'axe', '.'], ['a', 'woman', 'puts', 'flour', 'on', 'a', 'piece', 'of', 'meat', '.'], ['a', 'woman', 'is', 'spreading', 'mustard', 'on', 'a', 'bread', 'roll', '.'], ['a', 'woman', 'is', 'riding', 'a', 'horse', 'on', 'a', 'beach', '.'], ['a', 'man', 'is', 'playing', 'on', 'a', 'guitar', 'and', 'singing', '.'], ['a', 'woman', 'is', 'dancing', 'and', 'singing', 'with', 'other', 'women', '.'], ['a', 'woman', 'cracks', 'two', 'eggs', 'into', 'a', 'glass', 'bowl', '.'], ['the', 'man', 'hit', 'the', 'other', 'man', 'with', 'a', 'stick', '.'], ['a', 'man', 'is', 'feeding', 'a', 'mouse', 'to', 'a', 'snake', '.'], ['the', 'man', 'cut', 'down', 'a', 'tree', 'with', 'an', 'axe', '.'], ['a', 'man', 'puts', 'seasoning', 'in', 'a', 'bowl', 'of', 'water', '.'], ['a', 'man', 'and', 'a', 'woman', 'talk', 'in', 'a', 'diner', '.'], ['the', 'man', 'used', 'a', 'chopper', 'to', 'chop', 'the', 'onion', '.'], ['a', 'chef', 'is', 'cleaning', 'up', 'a', 'bowl', 'of', 'soup', '.'], ['a', 'group', 'of', 'men', 'play', 'soccer', 'on', 'the', 'beach', '.'], ['a', 'man', 'is', 'spreading', 'shreded', 'cheese', 'on', 'a', 'pizza', '.'], ['a', 'man', 'is', 'drilling', 'a', 'hole', 'in', 'a', 'board', '.'], ['a', 'man', 'is', 'patting', 'an', 'alligator', 'on', 'the', 'mouth', '.'], ['a', 'woman', 'mixes', 'a', 'beauty', 'treatment', 'in', 'a', 'glass', '.'], ['a', 'woman', 'picks', 'up', 'and', 'holds', 'a', 'baby', 'kangaroo', '.'], ['a', 'man', 'is', 'putting', 'garlic', 'on', 'some', 'bread', 'slices', '.'], ['a', 'cat', 'is', 'sliding', 'upside', 'down', 'under', 'a', 'couch', '.'], ['a', 'woman', 'and', 'man', 'are', 'riding', 'in', 'a', 'car', '.'], ['a', 'man', 'sitting', 'on', 'the', 'floor', 'plays', 'a', 'guitar', '.'], ['a', 'woman', 'is', 'adding', 'pasta', 'to', 'a', 'pot', 'of', 'water', '.'], ['a', 'man', 'is', 'opening', 'a', 'box', 'and', 'taking', 'out', 'paper', '.'], ['a', 'man', 'is', 'cleaning', 'a', 'fish', 'on', 'a', 'kitchen', 'counter', '.'], ['the', 'dog', 'pulled', 'the', 'dogs', 'tail', 'and', 'then', 'his', 'leg', '.'], ['an', 'oriental', 'lady', 'is', 'cutting', 'a', 'carrot', 'into', 'thin', 'pieces', '.'], ['soccer', 'players', 'are', 'kicking', 'a', 'soccer', 'ball', 'into', 'the', 'goal', '.'], ['the', 'lady', 'cut', 'the', 'tail', 'and', 'body', 'of', 'a', 'shrimp', '.'], ['a', 'man', 'is', 'talking', 'with', 'other', 'women', 'on', 'the', 'beach', '.'], ['an', 'elderly', 'woman', 'is', 'pouring', 'oil', 'into', 'a', 'frying', 'pan', '.'], ['a', 'man', 'is', 'riding', 'a', 'horse', 'and', 'jumping', 'over', 'obstacles', '.'], ['a', 'woman', 'is', 'riding', 'a', 'motorized', 'scooter', 'down', 'a', 'road', '.'], ['a', 'man', 'is', 'shaking', 'a', 'container', 'over', 'a', 'kitchen', 'sink', '.'], ['a', 'dog', 'lies', 'on', 'his', 'back', 'on', 'a', 'wooden', 'floor', '.'], ['a', 'woman', 'is', 'putting', 'her', 'baby', 'in', 'a', 'waste', 'bin', '.'], ['a', 'man', 'lays', 'on', 'the', 'ground', 'and', 'aims', 'a', 'rifle', '.'], ['a', 'woman', 'is', 'stirring', 'two', 'eggs', 'in', 'a', 'glass', 'bowl', '.'], ['a', 'woman', 'puts', 'a', 'garland', 'over', 'a', 'man', "'s", 'shoulders', '.'], ['a', 'person', 'is', 'peeling', 'a', 'potato', 'with', 'a', 'potato', 'peeler', '.'], ['one', 'man', 'is', 'breaking', 'cement', 'on', 'another', 'man', "'s", 'chest', '.'], ['the', 'boy', 'is', 'stuck', 'in', 'the', 'door', 'of', 'the', 'doghouse', '.'], ['someone', 'pours', 'olive', 'oil', 'into', 'a', 'bowl', 'of', 'tomato', 'wedges', '.'], ['a', 'man', 'puts', 'three', 'pieces', 'of', 'meat', 'into', 'a', 'pan', '.'], ['a', 'young', 'woman', 'is', 'putting', 'stickers', 'all', 'over', 'her', 'face', '.'], ['a', 'person', 'is', 'throwing', 'a', 'cat', 'on', 'to', 'the', 'ceiling', '.'], ['the', 'man', 'drove', 'his', 'little', 'red', 'car', 'around', 'the', 'traffic', '.'], ['a', 'woman', 'is', 'filing', 'her', 'fingernails', 'with', 'an', 'emery', 'board', '.'], ['the', 'lady', 'put', 'the', 'skewered', 'shimp', 'in', 'the', 'hot', 'water', '.'], ['the', 'man', 'is', 'using', 'a', 'camera', 'to', 'hammer', 'a', 'nail', '.'], ['the', 'man', 'talked', 'to', 'a', 'girl', 'over', 'the', 'internet', 'camera', '.'], ['the', 'man', 'used', 'a', 'sword', 'to', 'slice', 'a', 'plastic', 'bottle', '.'], ['a', 'man', 'spreads', 'butter', 'on', 'a', 'big', 'piece', 'of', 'bread', '.'], ['a', 'dog', 'is', 'trying', 'to', 'get', 'bacon', 'off', 'his', 'back', '.'], ['four', 'boys', 'are', 'standing', 'in', 'front', 'of', 'the', 'burning', 'car', '.'], ['a', 'man', 'and', 'woman', 'are', 'walking', 'together', 'through', 'the', 'woods', '.'], ['a', 'woman', 'is', 'placing', 'food', 'in', 'a', 'pot', 'of', 'hot', 'water', '.'], ['the', 'girl', 'flew', 'her', 'kite', 'while', 'she', 'ran', 'up', 'a', 'hill', '.'], ['a', 'man', 'is', 'riding', 'a', 'white', 'horse', 'on', 'an', 'enclosed', 'ground', '.'], ['a', 'woman', 'rides', 'an', 'elephant', 'in', 'front', 'of', 'an', 'adoring', 'crowd', '.'], ['two', 'foxes', 'are', 'eating', 'from', 'a', 'plate', 'on', 'a', 'brick', 'patio', '.'], ['two', 'women', 'are', 'dancing', 'and', 'singing', 'in', 'front', 'of', 'a', 'crowd', '.'], ['a', 'man', 'is', 'swinging', 'from', 'a', 'rope', 'attached', 'to', 'the', 'ceiling', '.'], ['a', 'man', 'is', 'crushing', 'garlic', 'with', 'the', 'back', 'of', 'a', 'knife', '.'], ['a', 'baby', 'rhino', 'is', 'walking', 'around', 'his', 'pen', 'with', 'his', 'mother', '.'], ['a', 'woman', 'pours', 'a', 'mixture', 'into', 'a', 'bowl', 'and', 'mixes', 'it', '.'], ['a', 'man', 'is', 'cutting', 'pieces', 'of', 'butter', 'into', 'a', 'mixing', 'bowl', '.'], ['two', 'men', 'are', 'packing', 'suitcases', 'into', 'the', 'trunk', 'of', 'a', 'car', '.'], ['a', 'person', 'is', 'putting', 'salt', 'and', 'pepper', 'on', 'a', 'piece', 'of', 'pork', '.'], ['a', 'man', 'is', 'standing', 'in', 'front', 'of', 'the', 'window', 'and', 'looking', 'outside', '.'], ['a', 'woman', 'is', 'talking', 'to', 'her', 'pet', 'bunny', 'as', 'she', 'fondles', 'it', '.'], ['a', 'man', 'and', 'woman', 'are', 'driving', 'down', 'the', 'street', 'in', 'a', 'jeep', '.'], ['a', 'man', 'is', 'holding', 'up', 'and', 'talking', 'about', 'a', 'gray', 'you', 'tube', 't-shirt', '.'], ['a', 'woman', 'is', 'applying', 'eye', 'liner', 'to', 'her', 'eyelid', 'using', 'an', 'eye', 'pencil', '.'], ['the', 'little', 'girl', 'played', 'on', 'the', 'grass', 'with', 'a', 'dog', 'and', 'a', 'ball', '.'], ['a', 'kid', 'runs', 'away', 'from', 'a', 'fish', 'swinging', 'around', 'on', 'a', 'fishing', 'pole', '.'], ['someone', 'drills', 'a', 'hole', 'in', 'a', 'strip', 'of', 'wood', 'with', 'a', 'power', 'drill', '.'], ['a', 'man', 'is', 'playing', 'a', 'wooden', 'flute', 'while', 'several', 'other', 'men', 'play', 'bongo', 'drums', '.'], ['three', 'young', 'men', 'run', ',', 'jump', ',', 'and', 'kick', 'off', 'of', 'a', 'coke', 'machine', '.'], ['a', 'woman', 'is', 'lying', 'on', 'a', 'blanket', 'on', 'a', 'rock', 'and', 'reading', 'a', 'book', '.'], ['a', 'small', 'guinea', 'pig', 'is', 'gnawing', 'and', 'eating', 'a', 'piece', 'of', 'carrot', 'on', 'the', 'floor', '.'], ['a', 'woman', 'pours', 'oil', 'into', 'a', 'skillet', 'from', 'a', 'plastic', 'bottle', 'while', 'she', 'is', 'talking', '.'], ['a', 'woman', 'is', 'walking', 'on', 'an', 'open', 'ground', 'with', 'a', 'pet', 'dog', 'as', 'some', 'other', 'dogs', 'follow', 'them', '.'], ['a', 'rocket', 'exploded', '.'], ['a', 'man', 'is', 'talking', '.'], ['a', 'man', 'is', 'eating', '.'], ['a', 'fish', 'is', 'swimming', '.'], ['kids', 'are', 'dancing', '.'], ['zebras', 'are', 'socializing', '.'], ['a', 'man', 'is', 'talking', '.'], ['a', 'woman', 'is', 'dancing', '.'], ['some', 'men', 'are', 'sawing', '.'], ['two', 'women', 'are', 'singing', '.'], ['a', 'man', 'is', 'eating', '.'], ['someone', 'is', 'sewing', 'things', '.'], ['a', 'man', 'is', 'skating', '.'], ['a', 'woman', 'is', 'swimming', '.'], ['a', 'man', 'is', 'exercising', '.'], ['a', 'man', 'is', 'talking', '.'], ['a', 'man', 'is', 'talking', '.'], ['two', 'bays', 'are', 'dancing', '.'], ['a', 'woman', 'is', 'dancing', '.'], ['a', 'man', 'is', 'spitting', '.'], ['the', 'man', 'is', 'exercising', '.'], ['a', 'man', 'is', 'talking', '.'], ['a', 'man', 'is', 'cooking', '.'], ['a', 'man', 'is', 'dancing', '.'], ['the', 'man', 'is', 'dancing', '.'], ['a', 'man', 'is', 'dancing', '.'], ['men', 'are', 'playing', 'cricket', '.'], ['men', 'are', 'sawing', 'logs', '.'], ['a', 'woman', 'is', 'swimming', '.'], ['two', 'men', 'are', 'fighting', '.'], ['a', 'woman', 'is', 'slicing', 'meat', '.'], ['a', 'man', 'is', 'doing', 'yoga', '.'], ['a', 'man', 'breaks', 'a', 'stick', '.'], ['a', 'woman', 'is', 'kneading', 'dough', '.'], ['a', 'woman', 'is', 'playing', 'violin', '.'], ['a', 'man', 'is', 'eating', 'bread', '.'], ['a', 'woman', 'plays', 'the', 'clarinet', '.'], ['a', 'woman', 'is', 'kneading', 'dough', '.'], ['two', 'men', 'are', 'playing', 'guitar', '.'], ['a', 'woman', 'is', 'swimming', 'underwater', '.'], ['a', 'animated', 'airplane', 'is', 'landing', '.'], ['the', 'lady', 'peeled', 'the', 'shrimp', '.'], ['kittens', 'are', 'eating', 'from', 'dishes', '.'], ['a', 'woman', 'is', 'peeling', 'shrimp', '.'], ['a', 'woman', 'is', 'cutting', 'tofu', '.'], ['the', 'cricket', 'player', 'hit', 'the', 'ball', '.'], ['a', 'man', 'is', 'riding', 'a', 'horse', '.'], ['a', 'man', 'is', 'playing', 'a', 'key-board', '.'], ['a', 'man', 'is', 'climbing', 'a', 'rope', '.'], ['a', 'man', 'and', 'woman', 'is', 'dancing', '.'], ['a', 'boy', 'is', 'riding', 'a', 'bicycle', '.'], ['a', 'group', 'of', 'people', 'are', 'dancing', '.'], ['a', 'baby', 'is', 'laughing', 'and', 'giggling', '.'], ['a', 'lion', 'is', 'pacing', 'in', 'a', 'pen', '.'], ['a', 'man', 'is', 'running', 'down', 'a', 'road', '.'], ['a', 'person', 'is', 'writing', 'with', 'a', 'pencil', '.'], ['a', 'cheetah', 'is', 'running', 'across', 'a', 'field', '.'], ['two', 'women', 'are', 'wrestling', 'on', 'the', 'floor', '.'], ['a', 'woman', 'is', 'wake', 'boarding', 'on', 'a', 'lake', '.'], ['the', 'man', 'sat', 'in', 'his', 'suit', 'and', 'smoked', '.'], ['two', 'men', 'are', 'fighting', 'using', 'a', 'form', 'of', 'martial', 'arts', '.'], ['someone', 'is', 'typing', '.'], ['a', 'man', 'is', 'crawling', '.'], ['smeone', 'is', 'laying', 'down', '.'], ['a', 'woman', 'is', 'exercising', '.'], ['a', 'woman', 'slices', 'tofu', '.'], ['a', 'kangaroo', 'is', 'eating', '.'], ['a', 'man', 'slices', 'potatoes', '.'], ['some', 'people', 'are', 'singing', '.'], ['someone', 'is', 'playing', 'a', 'piano', '.'], ['someone', 'is', 'riding', 'a', 'horse', '.'], ['a', 'onion', 'is', 'being', 'sliced', '.'], ['a', 'man', 'is', 'squeezing', 'water', '.'], ['a', 'man', 'is', 'draining', 'pasta', '.'], ['a', 'woman', 'is', 'slicing', 'fish', '.'], ['a', 'woman', 'is', 'slicing', 'something', '.'], ['a', 'man', 'is', 'busting', 'concrete', '.'], ['a', 'man', 'is', 'playing', 'flute', '.'], ['a', 'man', 'is', 'playing', 'flute', '.'], ['someoen', 'is', 'peeling', 'a', 'potato', '.'], ['a', 'woman', 'is', 'grilling', 'pineapples', '.'], ['a', 'woman', 'is', 'balling', 'dough', '.'], ['a', 'man', 'plays', 'the', 'piano', '.'], ['a', 'man', 'is', 'slicing', 'onion', '.'], ['a', 'woman', 'is', 'cutting', 'tofu', '.'], ['a', 'woman', 'is', 'applying', 'eyeshadow', '.'], ['the', 'men', 'are', 'playing', 'basketball', '.'], ['a', 'woman', 'is', 'peeling', 'potato', '.'], ['two', 'men', 'are', 'playing', 'chess', '.'], ['a', 'animal', 'is', 'walking', 'around', '.'], ['a', 'gorilla', 'plays', 'the', 'drums', '.'], ['a', 'woman', 'is', 'eating', 'meat', '.'], ['a', 'woman', 'is', 'cooking', 'something', '.'], ['someone', 'is', 'slicing', 'a', 'tomato', '.'], ['two', 'pandas', 'are', 'laying', 'together', '.'], ['runners', 'compete', 'in', 'a', 'race', '.'], ['a', 'man', 'slaps', 'a', 'woman', '.'], ['a', 'squirrel', 'runs', 'in', 'circles', '.'], ['a', 'woman', 'is', 'cutting', 'onion', '.'], ['a', 'woman', 'is', 'cooking', 'fish', '.'], ['someone', 'is', 'cutting', 'a', 'tomato', '.'], ['a', 'man', 'is', 'eating', 'something', '.'], ['a', 'man', 'plays', 'a', 'guitar', '.'], ['a', 'man', 'is', 'playing', 'violin', '.'], ['someone', 'is', 'playing', 'a', 'keyboard', '.'], ['a', 'man', 'is', 'dancing', 'in', 'rain', '.'], ['the', 'man', 'is', 'holding', 'a', 'lemur', '.'], ['a', 'woman', 'is', 'painting', 'her', 'lips', '.'], ['a', 'man', 'is', 'slicing', 'some', 'carrots', '.'], ['a', 'man', 'is', 'playing', 'the', 'guitar', '.'], ['two', 'men', 'are', 'playing', 'ping', 'pong', '.'], ['a', 'woman', 'is', 'cutting', 'some', 'octopus', '.'], ['a', 'man', 'is', 'cutting', 'a', 'potato', '.'], ['a', 'man', 'is', 'taking', 'a', 'picture', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'woman', 'is', 'slicing', 'a', 'pepper', '.'], ['a', 'woman', 'is', 'riding', 'a', 'horse', '.'], ['a', 'man', 'is', 'cleaning', 'the', 'house', '.'], ['a', 'woman', 'is', 'playing', 'the', 'violin', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'lady', 'is', 'playing', 'the', 'guitar', '.'], ['a', 'woman', 'is', 'slicing', 'an', 'onion', '.'], ['a', 'person', 'is', 'playing', 'a', 'piano', '.'], ['a', 'woman', 'is', 'cutting', 'an', 'onion', '.'], ['a', 'cartoon', 'chef', 'is', 'flipping', 'pancakes', '.'], ['a', 'person', 'is', 'peeling', 'an', 'onion', '.'], ['a', 'man', 'is', 'slicing', 'a', 'bun', '.'], ['some', 'people', 'are', 'dancing', 'on', 'stage', '.'], ['a', 'cat', 'is', 'playing', 'a', 'piano', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'boy', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'is', 'carving', 'a', 'statue', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'boy', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'rabbit', 'is', 'kissing', 'another', 'rabbit', '.'], ['a', 'person', 'is', 'playing', 'the', 'keyboard', '.'], ['a', 'man', 'is', 'driving', 'a', 'car', '.'], ['a', 'woman', 'plays', 'on', 'a', 'flute', '.'], ['a', 'dog', 'is', 'licking', 'a', 'baby', '.'], ['a', 'woman', 'is', 'peeling', 'a', 'potato', '.'], ['the', 'monkey', 'pushed', 'the', 'other', 'monkey', '.'], ['an', 'air', 'plane', 'is', 'taking', 'off', '.'], ['a', 'woman', 'is', 'peeling', 'a', 'potato', '.'], ['a', 'woman', 'is', 'peeling', 'a', 'potato', '.'], ['a', 'dog', 'is', 'riding', 'a', 'skateboard', '.'], ['a', 'person', 'is', 'cutting', 'an', 'onion', '.'], ['a', 'woman', 'is', 'riding', 'a', 'horse', '.'], ['a', 'man', 'is', 'playing', 'a', 'flute', '.'], ['a', 'man', 'is', 'playing', 'a', 'piano', '.'], ['a', 'man', 'pops', 'wheelies', 'on', 'a', 'motorcycle', '.'], ['a', 'woman', 'is', 'slicing', 'up', 'some', 'pork', '.'], ['a', 'man', 'is', 'talking', 'to', 'a', 'woman', '.'], ['a', 'man', 'is', 'doing', 'a', 'card', 'trick', '.'], ['a', 'man', 'is', 'talking', 'into', 'a', 'microphone', '.'], ['a', 'person', 'is', 'holding', 'two', 'baby', 'lorises', '.'], ['a', 'woman', 'is', 'dancing', 'in', 'railway', 'station', '.'], ['a', 'cat', 'is', 'playing', 'with', 'a', 'camera', '.'], ['a', 'baby', 'sneezed', 'and', 'scared', 'another', 'baby', '.'], ['a', 'man', 'sings', 'and', 'plays', 'the', 'guitar', '.'], ['a', 'woman', 'stirs', 'eggs', 'in', 'a', 'bowl', '.'], ['trees', 'are', 'hit', 'by', 'very', 'hard', 'winds', '.'], ['the', 'man', 'sang', 'and', 'played', 'his', 'guitar', '.'], ['a', 'sea', 'turtle', 'is', 'hunting', 'for', 'fish', '.'], ['a', 'woman', 'is', 'picking', 'up', 'a', 'kangaroo', '.'], ['three', 'men', 'are', 'on', 'stage', 'playing', 'guitars', '.'], ['a', 'woman', 'is', 'flouring', 'and', 'rolling', 'dough', '.'], ['a', 'male', 'and', 'female', 'robot', 'are', 'kissing', '.'], ['a', 'woman', 'is', 'boiling', 'noodles', 'in', 'water', '.'], ['a', 'panda', 'bear', 'is', 'eating', 'some', 'bamboo', '.'], ['a', 'boy', 'is', 'looking', 'at', 'a', 'calendar', '.'], ['a', 'man', 'in', 'a', 'hat', 'sings', 'a', 'song', '.'], ['a', 'boy', 'is', 'talking', 'to', 'a', 'giant', 'dog', '.'], ['a', 'woman', 'is', 'placing', 'eggs', 'into', 'a', 'pan', '.'], ['a', 'woman', 'is', 'pouring', 'batter', 'into', 'a', 'bowl', '.'], ['a', 'woman', 'is', 'adding', 'ingredients', 'to', 'a', 'skillet', '.'], ['three', 'men', 'are', 'playing', 'guitars', 'and', 'singing', 'together', '.'], ['a', 'woman', 'is', 'mixing', 'food', 'in', 'a', 'bowl', '.'], ['a', 'man', 'gets', 'knocked', 'off', 'of', 'a', 'horse', '.'], ['a', 'person', 'is', 'cutting', 'mushrooms', 'with', 'a', 'knife', '.'], ['a', 'woman', 'is', 'chopping', 'broccoli', 'with', 'a', 'knife', '.'], ['a', 'man', 'with', 'a', 'pistol', 'shoots', 'another', 'man', '.'], ['a', 'group', 'of', 'people', 'get', 'off', 'a', 'train', '.'], ['a', 'man', 'is', 'eating', 'a', 'slice', 'of', 'pizza', '.'], ['a', 'dog', 'is', 'pulling', 'a', 'girl', 'down', 'a', 'hill', '.'], ['a', 'man', 'is', 'doing', 'tricks', 'with', 'a', 'soccer', 'ball', '.'], ['a', 'woman', 'is', 'pouring', 'ingredients', 'into', 'a', 'frying', 'pan', '.'], ['a', 'cat', 'is', 'licking', 'from', 'a', 'saucer', 'of', 'milk', '.'], ['different', 'sets', 'of', 'teams', 'are', 'playing', 'football', 'on', 'a', 'field', '.'], ['a', 'woman', 'is', 'breaking', 'a', 'head', 'of', 'garlic', 'into', 'cloves', '.'], ['a', 'man', 'is', 'slicing', 'a', 'white', 'onion', 'with', 'a', 'mandolin', 'slicer', '.'], ['someone', 'in', 'a', 'gorilla', 'costume', 'is', 'playing', 'a', 'set', 'of', 'drums', '.'], ['a', 'man', 'is', 'spitting', '.'], ['a', 'hamster', 'is', 'singing', '.'], ['a', 'horse', 'is', 'running', '.'], ['a', 'woman', 'is', 'dancing', '.'], ['a', 'boy', 'is', 'screaming', '.'], ['a', 'man', 'is', 'dancing', '.'], ['a', 'person', 'cuts', 'ginger', '.'], ['women', 'are', 'dancing', 'outside', '.'], ['a', 'couple', 'slow', 'dances', '.'], ['someoen', 'is', 'playing', 'guitar', '.'], ['a', 'person', 'is', 'drawing', '.'], ['the', 'horse', 'is', 'running', '.'], ['a', 'woman', 'is', 'slicing', 'cucumber', '.'], ['a', 'person', 'riding', 'a', 'mechanical', 'bull'], ['a', 'woman', 'is', 'watching', 'television', '.'], ['a', 'woman', 'is', 'slicing', 'potatoes', '.'], ['a', 'woman', 'is', 'cutting', 'vegetable', '.'], ['a', 'lion', 'is', 'walking', 'around', '.'], ['someone', 'is', 'handling', 'a', 'bun', '.'], ['a', 'man', 'is', 'eating', 'bread', '.'], ['a', 'panda', 'bear', 'is', 'eating', '.'], ['a', 'baby', 'is', 'falling', 'asleep', '.'], ['the', 'woman', 'is', 'slicing', 'herbs', '.'], ['a', 'woman', 'is', 'peeling', 'garlic', '.'], ['a', 'man', 'is', 'performing', 'stunts', '.'], ['a', 'man', 'is', 'eating', 'pasta', '.'], ['a', 'man', 'is', 'cutting', 'paper', '.'], ['a', 'woman', 'is', 'slicing', 'garlic', '.'], ['a', 'young', 'man', 'is', 'dancing', '.'], ['a', 'woman', 'is', 'mixing', 'meat', '.'], ['a', 'man', 'catches', 'a', 'fish', '.'], ['a', 'man', 'played', 'the', 'guitar', '.'], ['a', 'man', 'is', 'playing', 'guitar', '.'], ['a', 'woman', 'is', 'slicing', 'ginger', '.'], ['a', 'man', 'is', 'playing', 'guitar', '.'], ['a', 'man', 'plays', 'a', 'guitar', '.'], ['a', 'man', 'is', 'slicing', 'potato', '.'], ['a', 'man', 'is', 'cutting', 'onions', '.'], ['the', 'woman', 'is', 'dicing', 'garlic', '.'], ['a', 'woman', 'is', 'chopping', 'cilantro', '.'], ['a', 'man', 'guts', 'a', 'fish', '.'], ['the', 'man', 'played', 'his', 'guitar', '.'], ['a', 'turtle', 'is', 'walking', 'underwater', '.'], ['elephants', 'spray', 'themselves', 'with', 'water', '.'], ['a', 'man', 'peeled', 'a', 'potatoe', '.'], ['a', 'woman', 'is', 'chopping', 'octopus', '.'], ['a', 'chef', 'prepared', 'a', 'meal', '.'], ['the', 'girl', 'trotted', 'the', 'horse', '.'], ['a', 'woman', 'is', 'chopping', 'onions', '.'], ['the', 'woman', 'is', 'dicing', 'garlic', '.'], ['a', 'man', 'climbs', 'a', 'rope', '.'], ['a', 'man', 'plays', 'the', 'drum', '.'], ['a', 'man', 'cuts', 'an', 'onion', '.'], ['a', 'man', 'ties', 'his', 'shoe', '.'], ['a', 'man', 'plays', 'the', 'guitar', '.'], ['a', 'boy', 'plays', 'a', 'piano', '.'], ['a', 'woman', 'is', 'using', 'a', 'hoe', '.'], ['a', 'woman', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'woman', 'is', 'washing', 'a', 'freezer', '.'], ['a', 'woman', 'is', 'cutting', 'a', 'potato', '.'], ['a', 'monkey', 'is', 'fighting', 'a', 'man', '.'], ['a', 'man', 'is', 'driving', 'a', 'car', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'is', 'peeling', 'a', 'carrot', '.'], ['a', 'man', 'is', 'slicing', 'a', 'leave', '.'], ['a', 'man', 'is', 'slicing', 'a', 'cucumber', '.'], ['a', 'person', 'is', 'slicing', 'a', 'tomato', '.'], ['a', 'woman', 'is', 'slicing', 'a', 'cucumber', '.'], ['the', 'ladies', 'are', 'performing', 'a', 'dance', '.'], ['a', 'person', 'is', 'cleaning', 'a', 'disk', '.'], ['a', 'boy', 'is', 'doing', 'push', 'ups', '.'], ['the', 'deer', 'jumped', 'over', 'the', 'fence', '.'], ['a', 'dog', 'is', 'moving', 'its', 'mouth', '.'], ['a', 'woman', 'plays', 'an', 'electric', 'guitar', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'woman', 'is', 'cutting', 'a', 'lemon', '.'], ['a', 'man', 'is', 'playing', 'a', 'keyboard', '.'], ['a', 'woman', 'is', 'feeding', 'a', 'man', '.'], ['a', 'woman', 'is', 'singing', 'on', 'stage', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'baby', 'is', 'laughing', 'and', 'giggling', '.'], ['a', 'woman', 'is', 'cleaning', 'a', 'garden', '.'], ['a', 'man', 'is', 'slicing', 'a', 'tomato', '.'], ['a', 'dog', 'is', 'running', 'through', 'water', '.'], ['a', 'woman', 'is', 'riding', 'an', 'elephant', '.'], ['a', 'man', 'is', 'driving', 'a', 'car', '.'], ['a', 'woman', 'is', 'doing', 'her', 'hair', '.'], ['a', 'woman', 'is', 'riding', 'a', 'horse', '.'], ['a', 'woman', 'is', 'riding', 'a', 'horse', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'is', 'eating', 'a', 'banana', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'is', 'playing', 'a', 'harp', '.'], ['a', 'woman', 'is', 'riding', 'a', 'horse', '.'], ['a', 'man', 'is', 'cutting', 'an', 'onion', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'is', 'riding', 'a', 'horse', '.'], ['a', 'man', 'is', 'riding', 'a', 'scooter', '.'], ['a', 'woman', 'is', 'playing', 'the', 'violin', '.'], ['a', 'man', 'is', 'playing', 'the', 'guitar', '.'], ['a', 'woman', 'is', 'slicing', 'a', 'tomato', '.'], ['a', 'man', 'is', 'playing', 'the', 'guitar', '.'], ['a', 'man', 'is', 'riding', 'a', 'horse', '.'], ['a', 'man', 'is', 'stabbing', 'a', 'target', '.'], ['a', 'boy', 'is', 'playing', 'a', 'piano', '.'], ['a', 'band', 'is', 'playing', 'on', 'stage', '.'], ['a', 'man', 'is', 'riding', 'a', 'horse', '.'], ['a', 'man', 'is', 'riding', 'a', 'horse', '.'], ['a', 'man', 'is', 'climbing', 'a', 'wall', '.'], ['a', 'man', 'is', 'playing', 'a', 'piano', '.'], ['a', 'woman', 'is', 'chopping', 'a', 'potato', '.'], ['a', 'woman', 'is', 'cutting', 'a', 'jalepeno', '.'], ['a', 'man', 'is', 'playing', 'a', 'keyboard', '.'], ['a', 'man', 'is', 'playing', 'the', 'drums', '.'], ['a', 'man', 'is', 'playing', 'a', 'flute', '.'], ['the', 'man', 'is', 'playing', 'the', 'guitar', '.'], ['a', 'man', 'is', 'playing', 'the', 'trumpet', '.'], ['a', 'man', 'and', 'woman', 'is', 'eating', '.'], ['a', 'man', 'is', 'playing', 'a', 'piano', '.'], ['a', 'man', 'is', 'playing', 'the', 'flute', '.'], ['a', 'man', 'is', 'playing', 'a', 'trumpet', '.'], ['a', 'man', 'is', 'playing', 'a', 'trumpet', '.'], ['a', 'man', 'is', 'playing', 'a', 'trumpet', '.'], ['a', 'person', 'is', 'slicing', 'some', 'meat', '.'], ['the', 'man', 'is', 'playing', 'the', 'violin', '.'], ['a', 'man', 'is', 'playing', 'a', 'keyboard', '.'], ['a', 'girl', 'is', 'riding', 'a', 'bicycle', '.'], ['a', 'woman', 'is', 'peeling', 'an', 'apple', '.'], ['a', 'person', 'is', 'peeling', 'an', 'eggplant', '.'], ['a', 'man', 'is', 'slicing', 'a', 'bun', '.'], ['a', 'monkey', 'is', 'riding', 'a', 'bike', '.'], ['a', 'man', 'is', 'playing', 'a', 'flute', '.'], ['a', 'man', 'is', 'playing', 'the', 'guitar', '.'], ['a', 'woman', 'is', 'slicing', 'a', 'pumpkin', '.'], ['a', 'woman', 'is', 'carrying', 'her', 'baby', '.'], ['a', 'man', 'is', 'slicing', 'an', 'onion', '.'], ['the', 'lady', 'sang', 'into', 'the', 'microphone', '.'], ['a', 'man', 'is', 'cutting', 'a', 'potato', '.'], ['a', 'girl', 'is', 'brushing', 'her', 'hair', '.'], ['a', 'man', 'is', 'playing', 'a', 'trumpet', '.'], ['a', 'woman', 'is', 'kneading', 'tortilla', 'dough', '.'], ['a', 'man', 'is', 'playing', 'a', 'flute', '.'], ['a', 'man', 'is', 'cutting', 'an', 'onion', '.'], ['men', 'are', 'falling', 'into', 'a', 'pool', '.'], ['a', 'man', 'turns', 'on', 'the', 'microwave', '.'], ['a', 'girl', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'woman', 'is', 'cutting', 'raw', 'fish', '.'], ['an', 'airplane', 'moves', 'along', 'a', 'runway', '.'], ['a', 'man', 'is', 'cutting', 'and', 'onion', '.'], ['a', 'boy', 'is', 'playing', 'the', 'piano', '.'], ['a', 'girl', 'plays', 'a', 'wind', 'instrument', '.'], ['a', 'woman', 'is', 'carrying', 'a', 'kangaroo', '.'], ['a', 'man', 'plays', 'an', 'acoustic', 'guitar', '.'], ['a', 'man', 'is', 'riding', 'a', 'horse', '.'], ['a', 'guy', 'is', 'playing', 'an', 'instrument', '.'], ['a', 'man', 'is', 'smoking', 'a', 'cigarette', '.'], ['dogs', 'are', 'swimming', 'in', 'a', 'pool', '.'], ['a', 'squirrel', 'is', 'moving', 'in', 'circles', '.'], ['a', 'person', 'is', 'playing', 'the', 'piano', '.'], ['a', 'man', 'is', 'slicing', 'some', 'potato', '.'], ['a', 'badger', 'is', 'digging', 'the', 'earth', '.'], ['a', 'boy', 'falls', 'off', 'his', 'bike', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'woman', 'is', 'playing', 'a', 'flute', '.'], ['a', 'cougar', 'is', 'chasing', 'a', 'bear', '.'], ['a', 'man', 'climbs', 'up', 'a', 'rope', '.'], ['a', 'man', 'is', 'riding', 'a', 'white', 'horse', '.'], ['a', 'woman', 'is', 'using', 'a', 'sewing', 'machine', '.'], ['a', 'dog', 'is', 'barking', 'at', 'a', 'fly', '.'], ['a', 'cat', 'is', 'licking', 'a', 'small', 'bottle', '.'], ['a', 'man', 'pours', 'cleaner', 'in', 'his', 'sink', '.'], ['a', 'truck', 'is', 'going', 'down', 'a', 'hill', '.'], ['a', 'woman', 'and', 'dog', 'are', 'walking', 'together', '.'], ['a', 'cat', 'is', 'pooping', 'into', 'a', 'toilet', '.'], ['a', 'woman', 'is', 'dancing', 'in', 'the', 'street', '.'], ['a', 'man', 'is', 'cleaning', 'a', 'glass', 'door', '.'], ['a', 'man', 'walks', 'across', 'a', 'tight', 'rope', '.'], ['a', 'man', 'tries', 'to', 'read', 'the', 'paper', '.'], ['a', 'man', 'is', 'shooting', 'off', 'a', 'gun', '.'], ['a', 'man', 'is', 'holding', 'a', 'slow', 'loris', '.'], ['a', 'woman', 'is', 'riding', 'a', 'water', 'scooter', '.'], ['a', 'band', 'is', 'playing', 'on', 'a', 'stage', '.'], ['a', 'dog', 'opens', 'a', 'can', 'of', 'food', '.'], ['a', 'woman', 'cuts', 'tofu', 'into', 'tiny', 'pieces', '.'], ['a', 'man', 'is', 'riding', 'on', 'a', 'horse', '.'], ['four', 'people', 'are', 'floating', 'on', 'a', 'raft', '.'], ['a', 'cat', 'plays', 'with', 'a', 'small', 'bottle', '.'], ['a', 'baby', 'is', 'crawling', 'across', 'the', 'floor', '.'], ['a', 'woman', 'is', 'boiling', 'noodles', 'in', 'water', '.'], ['a', 'man', 'jumps', 'onto', 'a', 'short', 'wall', '.'], ['a', 'lion', 'is', 'playing', 'with', 'two', 'men', '.'], ['a', 'monkey', 'is', 'wading', 'across', 'a', 'marsh', '.'], ['a', 'hedgehog', 'is', 'biting', 'at', 'a', 'blanket', '.'], ['the', 'man', 'is', 'talking', 'on', 'the', 'phone', '.'], ['a', 'monkey', 'is', 'riding', 'on', 'a', 'bus', '.'], ['a', 'man', 'is', 'playing', 'an', 'electric', 'guitar', '.'], ['a', 'woman', 'picks', 'up', 'a', 'baby', 'kangaroo', '.'], ['a', 'large', 'turtle', 'crawls', 'in', 'the', 'grass', '.'], ['a', 'boy', 'is', 'looking', 'at', 'a', 'calendar', '.'], ['a', 'man', 'is', 'playing', 'a', 'bamboo', 'flute', '.'], ['a', 'woman', 'is', 'cutting', 'a', 'red', 'pepper', '.'], ['a', 'girl', 'running', 'is', 'flying', 'a', 'kite', '.'], ['a', 'piece', 'of', 'paper', 'is', 'being', 'cut', '.'], ['a', 'rhino', 'is', 'grazing', 'in', 'a', 'field', '.'], ['a', 'man', 'seated', 'is', 'playing', 'the', 'cello', '.'], ['a', 'man', 'is', 'cutting', 'a', 'paper', 'plate', '.'], ['a', 'man', 'is', 'riding', 'on', 'a', 'horse', '.'], ['a', 'dog', 'is', 'riding', 'on', 'a', 'bicyclists', 'shoulders', '.'], ['a', 'woman', 'is', 'pouring', 'ingredients', 'in', 'a', 'bowl', '.'], ['a', 'woman', 'is', 'slicing', 'up', 'some', 'green', 'leaves', '.'], ['an', 'animal', 'is', 'chewing', 'on', 'a', 'key', 'chain', '.'], ['the', 'man', 'played', 'his', 'guitar', 'with', 'a', 'friend', '.'], ['the', 'woman', 'is', 'cracking', 'eggs', 'into', 'a', 'bowl', '.'], ['a', 'man', 'is', 'eating', 'a', 'piece', 'of', 'bread', '.'], ['a', 'man', 'is', 'singing', 'and', 'playing', 'a', 'guitar', '.'], ['a', 'little', 'girl', 'is', 'walking', 'down', 'a', 'hallway', '.'], ['a', 'man', 'is', 'singing', 'while', 'playing', 'the', 'guitar', '.'], ['a', 'man', 'is', 'melting', 'butter', 'in', 'a', 'pan', '.'], ['the', 'machine', 'shaved', 'the', 'end', 'of', 'the', 'pencil', '.'], ['a', 'person', 'cuts', 'a', 'big', 'green', 'pepper', 'apart', '.'], ['a', 'woman', 'walks', 'her', 'dog', 'down', 'the', 'street', '.'], ['a', 'dog', 'is', 'eating', 'a', 'piece', 'of', 'watermelon', '.'], ['a', 'cat', 'is', 'eating', 'corn', 'on', 'the', 'cob', '.'], ['the', 'woman', 'is', 'tapping', 'her', 'fingers', 'on', 'the', 'table', '.'], ['a', 'woman', 'is', 'placing', 'skewered', 'food', 'onto', 'a', 'cooker', '.'], ['the', 'people', 'got', 'off', 'the', 'train', 'at', 'the', 'station', '.'], ['a', 'herd', 'of', 'elephants', 'are', 'walking', 'along', 'a', 'trail', '.'], ['a', 'man', 'sprays', 'another', 'man', 'with', 'water', 'from', 'a', 'hose', '.'], ['a', 'woman', 'cuts', 'off', 'a', 'small', 'piece', 'of', 'lemon', 'rind', '.'], ['a', 'man', 'is', 'maneuvering', 'a', 'soccer', 'ball', 'with', 'his', 'feet', '.'], ['a', 'large', 'amount', 'of', 'penguins', 'are', 'heading', 'to', 'the', 'ocean', '.'], ['a', 'woman', 'is', 'slicing', 'partially', 'into', 'half', 'of', 'an', 'onion', '.'], ['a', 'man', 'seated', 'in', 'a', 'room', 'is', 'playing', 'the', 'guitar', '.'], ['a', 'woman', 'is', 'pouring', 'a', 'yellow', 'mixture', 'on', 'a', 'frying', 'pan', '.'], ['a', 'woman', 'is', 'cutting', 'a', 'block', 'of', 'tofu', 'into', 'small', 'cubes', '.'], ['a', 'boy', 'is', 'sitting', 'in', 'a', 'room', 'playing', 'a', 'piano', 'by', 'lamp', 'light', '.'], ['a', 'helicopter', 'lands', '.'], ['a', 'person', 'is', 'surfing', '.'], ['a', 'boy', 'is', 'vacuuming', '.'], ['a', 'woman', 'is', 'peeling', 'potato', '.'], ['the', 'beaver', 'is', 'being', 'petted', '.'], ['a', 'onion', 'is', 'being', 'chopped', '.'], ['a', 'man', 'is', 'slicing', 'garlic', '.'], ['some', 'men', 'are', 'playing', 'soccer', '.'], ['a', 'man', 'is', 'picking', 'flowers', '.'], ['a', 'woman', 'is', 'shredding', 'chicken', '.'], ['a', 'woman', 'spices', 'some', 'seafood', '.'], ['a', 'man', 'plays', 'the', 'guitar', '.'], ['the', 'woman', 'is', 'slicing', 'herbs', '.'], ['a', 'bear', 'is', 'fighting', 'wolves', '.'], ['a', 'tiger', 'is', 'walking', 'around', '.'], ['a', 'woman', 'is', 'slicing', 'meat', '.'], ['a', 'kid', 'is', 'playing', 'keyboard', '.'], ['two', 'men', 'are', 'pushing', 'carts', '.'], ['a', 'panda', 'is', 'lying', 'down', '.'], ['the', 'person', 'is', 'slicing', 'cheese', '.'], ['a', 'man', 'and', 'woman', 'kiss', '.'], ['a', 'band', 'is', 'playing', 'onstage', '.'], ['a', 'woman', 'is', 'peeling', 'a', 'prawn', '.'], ['a', 'man', 'is', 'rinsing', 'a', 'steak', '.'], ['a', 'man', 'is', 'shooting', 'off', 'guns', '.'], ['the', 'woman', 'is', 'playing', 'the', 'violin', '.'], ['a', 'man', 'is', 'slicing', 'a', 'tomato', '.'], ['a', 'boy', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'person', 'is', 'mixing', 'a', 'pot', '.'], ['a', 'man', 'is', 'moving', 'large', 'rocks', '.'], ['a', 'woman', 'is', 'riding', 'a', 'donkey', '.'], ['a', 'man', 'is', 'giving', 'an', 'interview', '.'], ['a', 'man', 'is', 'riding', 'a', 'mo-pad', '.'], ['a', 'man', 'is', 'mixing', 'a', 'bowl', '.'], ['a', 'woman', 'is', 'breading', 'some', 'meat', '.'], ['a', 'person', 'is', 'dicing', 'an', 'onion', '.'], ['a', 'man', 'climbs', 'up', 'a', 'rope', '.'], ['a', 'man', 'is', 'playing', 'a', 'flute', '.'], ['a', 'woman', 'is', 'taking', 'a', 'bath', '.'], ['a', 'man', 'is', 'playing', 'two', 'keyboards', '.'], ['a', 'woman', 'is', 'cutting', 'some', 'vegetables', '.'], ['a', 'man', 'is', 'slicing', 'a', 'potato', '.'], ['a', 'woman', 'is', 'breading', 'some', 'meat', '.'], ['a', 'man', 'is', 'cutting', 'up', 'carrots', '.'], ['a', 'skunk', 'looks', 'at', 'the', 'camera', '.'], ['an', 'officer', 'is', 'talking', 'to', 'recruits', '.'], ['a', 'girl', 'is', 'riding', 'a', 'horse', '.'], ['two', 'cats', 'are', 'fighting', 'each', 'other', '.'], ['a', 'person', 'is', 'blow', 'torching', 'cameras', '.'], ['polar', 'bears', 'are', 'fighting', 'each', 'other', '.'], ['a', 'man', 'is', 'riding', 'a', 'bicycle', '.'], ['a', 'woman', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'is', 'playing', 'a', 'flute', '.'], ['a', 'man', 'is', 'shooting', 'a', 'gun', '.'], ['a', 'man', 'rode', 'a', 'mechanical', 'bull', '.'], ['a', 'squirrel', 'runs', 'around', 'in', 'circles', '.'], ['a', 'man', 'is', 'slicing', 'a', 'cucumber', '.'], ['a', 'boy', 'is', 'playing', 'key', 'board', '.'], ['a', 'panda', 'slides', 'down', 'a', 'slide', '.'], ['a', 'panda', 'slides', 'down', 'a', 'slide', '.'], ['a', 'child', 'is', 'riding', 'a', 'horse', '.'], ['a', 'dog', 'bounces', 'on', 'a', 'trampoline', '.'], ['a', 'girl', 'is', 'tapping', 'her', 'finger', 'nails', '.'], ['a', 'man', 'is', 'singing', 'to', 'a', 'woman', '.'], ['a', 'woman', 'is', 'preparing', 'shrimp', 'to', 'cook', '.'], ['a', 'car', 'is', 'driving', 'down', 'the', 'road', '.'], ['a', 'man', 'is', 'talking', 'on', 'a', 'phone', '.'], ['a', 'girl', 'is', 'walking', 'down', 'a', 'road', '.'], ['a', 'dog', 'swims', 'after', 'a', 'tennis', 'ball', '.'], ['a', 'dog', 'is', 'walking', 'around', 'a', 'pool', '.'], ['an', 'animal', 'is', 'walking', 'on', 'the', 'ground', '.'], ['a', 'woman', 'is', 'pouring', 'water', 'on', 'fish', '.'], ['a', 'girl', 'is', 'putting', 'on', 'eye', 'makeup', '.'], ['a', 'little', 'boy', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'man', 'puts', 'vegetables', 'into', 'a', 'pot', '.'], ['a', 'man', 'pours', 'wine', 'in', 'a', 'pot', '.'], ['a', 'nuclear', 'weapon', 'explodes', 'in', 'a', 'desert', '.'], ['a', 'young', 'girl', 'is', 'using', 'sign', 'language', '.'], ['a', 'man', 'is', 'playing', 'a', 'keyboard', 'piano', '.'], ['a', 'car', 'is', 'driving', 'down', 'the', 'road', '.'], ['the', 'woman', 'is', 'pencilling', 'on', 'eye', 'shadow', '.'], ['a', 'hare', 'is', 'running', 'from', 'a', 'eagle', '.'], ['two', 'zebras', 'are', 'playing', 'in', 'a', 'field', '.'], ['a', 'man', 'is', 'cutting', 'up', 'a', 'fish', '.'], ['a', 'man', 'is', 'riding', 'on', 'a', 'motorcycle', '.'], ['the', 'lady', 'cut', 'up', 'a', 'green', 'onion', '.'], ['a', 'woman', 'is', 'cutting', 'up', 'some', 'meat', '.'], ['the', 'man', 'is', 'erasing', 'the', 'chalk', 'board', '.'], ['a', 'woman', 'is', 'sewing', 'on', 'a', 'machine', '.'], ['a', 'man', 'is', 'turning', 'over', 'tables', 'in', 'anger', '.'], ['a', 'young', 'man', 'is', 'talking', 'to', 'a', 'leaf', '.'], ['a', 'boy', 'is', 'pouring', 'something', 'from', 'a', 'pitcher', '.'], ['the', 'dog', 'is', 'playing', 'with', 'the', 'floor', 'cleaner', '.'], ['a', 'panda', 'dog', 'is', 'running', 'on', 'the', 'road', '.'], ['some', 'teenage', 'girls', 'are', 'dancing', 'for', 'the', 'camera', '.'], ['a', 'group', 'of', 'swimmers', 'fall', 'into', 'a', 'pool', '.'], ['a', 'slow', 'loris', 'is', 'biting', 'a', 'persons', 'finger', '.'], ['a', 'doctor', 'rushes', 'a', 'patient', 'on', 'a', 'stretcher', '.'], ['a', 'little', 'boy', 'plays', 'with', 'a', 'toy', 'phone', '.'], ['a', 'person', 'is', 'cutting', 'boiled', 'egg', 'into', 'pieces', '.'], ['a', 'person', 'is', 'stirring', 'vegetables', 'in', 'a', 'pot', '.'], ['the', 'little', 'dog', 'barked', 'at', 'the', 'electronic', 'toy', '.'], ['a', 'shirtless', 'man', 'is', 'sitting', 'on', 'a', 'chair', '.'], ['the', 'dog', 'is', 'playing', 'with', 'a', 'plastic', 'container', '.'], ['the', 'man', 'poured', 'oil', 'on', 'the', 'cut', 'tomatoes', '.'], ['lionel', 'ritchie', 'dances', 'while', 'standing', 'on', 'the', 'ceiling', '.'], ['a', 'chicken', 'is', 'pecking', 'at', 'a', 'dead', 'mouse', '.'], ['a', 'man', 'cutting', 'butter', 'into', 'a', 'mixing', 'bowl', '.'], ['a', 'cheetah', 'chases', 'prey', 'on', 'across', 'a', 'field', '.'], ['a', 'woman', 'dances', 'in', 'the', 'rain', 'out', 'side', '.'], ['a', 'man', 'rides', 'a', 'water', 'toy', 'in', 'the', 'water', '.'], ['a', 'man', 'is', 'singing', 'while', 'walking', 'down', 'the', 'sidewalk', '.'], ['a', 'man', 'is', 'jumping', 'into', 'water', 'from', 'a', 'cliff', '.'], ['a', 'car', 'is', 'backing', 'out', 'of', 'a', 'blocked', 'lane', '.'], ['a', 'boy', 'is', 'playing', 'a', 'game', 'on', 'a', 'laptop', '.'], ['a', 'woman', 'splashes', 'around', 'in', 'the', 'ocean', "'s", 'waves', '.'], ['an', 'elderly', 'man', 'is', 'getting', 'into', 'a', 'red', 'car', '.'], ['a', 'woman', 'is', 'putting', 'a', 'baby', 'into', 'a', 'trash', 'can', '.'], ['someone', 'pours', 'olive', 'oil', 'into', 'a', 'bowl', 'of', 'tomato', 'wedges', '.'], ['a', 'dog', 'is', 'walking', 'along', 'the', 'edge', 'of', 'a', 'pool', '.'], ['the', 'girl', 'is', 'using', 'an', 'eye', 'pencil', 'on', 'her', 'eyelid', '.'], ['someone', 'is', 'stirring', 'a', 'pot', 'of', 'chili', 'with', 'a', 'spoon', '.'], ['a', 'man', 'is', 'getting', 'into', 'a', 'car', 'in', 'a', 'garage', '.'], ['a', 'man', 'and', 'and', 'woman', 'are', 'running', 'together', ',', 'holding', 'hands', '.'], ['a', 'south', 'african', 'airways', 'plane', 'is', 'flying', 'in', 'a', 'blue', 'sky', '.'], ['a', 'man', 'is', 'leading', 'a', 'horse', 'around', 'by', 'it', "'s", 'halter', '.'], ['a', 'dog', 'is', 'pawing', 'at', 'a', 'fly', 'in', 'the', 'carpet', 'before', 'it', '.'], ['a', 'boy', 'sits', 'on', 'a', 'bed', ',', 'sings', 'and', 'plays', 'a', 'guitar', '.'], ['a', 'man', 'is', 'playing', 'an', 'acoustic', 'guitar', 'and', 'singing', 'into', 'a', 'microphone', '.'], ['someone', 'is', 'cutting', 'paper', '.'], ['a', 'person', 'is', 'cutting', 'leaves', '.'], ['a', 'dog', 'licks', 'a', 'baby', '.'], ['the', 'man', 'used', 'a', 'stenograph', '.'], ['the', 'man', 'is', 'stirring', 'oil', '.'], ['a', 'helicopter', 'flies', 'over', 'water', '.'], ['men', 'jump', 'in', 'a', 'river', '.'], ['a', 'man', 'is', 'folding', 'paper', '.'], ['a', 'man', 'is', 'lifting', 'weights', '.'], ['a', 'man', 'is', 'playing', 'a', 'flute', '.'], ['a', 'man', 'is', 'playing', 'his', 'guitar', '.'], ['a', 'little', 'girl', 'peddling', 'a', 'scooter', '.'], ['a', 'man', 'is', 'spreading', 'out', 'dough', '.'], ['a', 'woman', 'is', 'cutting', 'a', 'tomato', '.'], ['a', 'person', 'is', 'holding', 'a', 'skunk', '.'], ['a', 'man', 'is', 'holding', 'a', 'animal', '.'], ['the', 'lady', 'sliced', 'up', 'the', 'meat', '.'], ['woman', 'is', 'adding', 'sugar', 'to', 'meat', '.'], ['a', 'man', 'is', 'using', 'a', 'laptop', '.'], ['kittens', 'are', 'eating', 'food', 'on', 'trays', '.'], ['a', 'man', 'is', 'playing', 'the', 'guitar', '.'], ['a', 'boy', 'is', 'playing', 'the', 'piano', '.'], ['a', 'man', 'is', 'running', 'on', 'rope', '.'], ['a', 'group', 'of', 'people', 'are', 'dancing', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['the', 'woman', 'is', 'playing', 'the', 'flute', '.'], ['a', 'puppy', 'is', 'pushing', 'itself', 'backwards', '.'], ['a', 'dog', 'is', 'chasing', 'a', 'ball', '.'], ['a', 'man', 'is', 'cutting', 'a', 'potato', '.'], ['someone', 'is', 'chopping', 'some', 'cabbage', 'leaves', '.'], ['a', 'man', 'sings', 'with', 'a', 'guitar', '.'], ['a', 'woman', 'is', 'dancing', 'in', 'the', 'rain', '.'], ['a', 'cat', 'is', 'playing', 'on', 'the', 'floor', '.'], ['a', 'slow', 'loris', 'hanging', 'on', 'a', 'cord', '.'], ['a', 'woman', 'is', 'swimming', 'in', 'the', 'water', '.'], ['a', 'boy', 'is', 'playing', 'a', 'wooden', 'flute', '.'], ['a', 'baby', 'is', 'playing', 'with', 'a', 'doll', '.'], ['a', 'woman', 'puts', 'rice', 'into', 'a', 'pan', '.'], ['a', 'man', 'is', 'riding', 'on', 'a', 'horse', '.'], ['a', 'cat', 'is', 'rubbing', 'against', 'a', 'baby', '.'], ['a', 'woman', 'relaxes', 'in', 'an', 'inner', 'tube', '.'], ['two', 'women', 'make', 'something', 'in', 'a', 'blender', '.'], ['a', 'large', 'green', 'ball', 'hits', 'a', 'potato', '.'], ['a', 'woman', 'jumps', 'hurdles', 'on', 'a', 'horse', '.'], ['someone', 'is', 'folding', 'a', 'piece', 'of', 'paper', '.'], ['a', 'woman', 'measures', 'another', 'woman', "'s", 'ankle', '.'], ['the', 'man', 'sprinkled', 'cheese', 'on', 'the', 'pizza', '.'], ['a', 'man', 'is', 'applying', 'butter', 'to', 'bread', '.'], ['a', 'man', 'is', 'performing', 'a', 'card', 'trick', '.'], ['a', 'man', 'is', 'giving', 'a', 'woman', 'a', 'massage', '.'], ['a', 'man', 'is', 'kissing', 'a', 'woman', 'in', 'stage', '.'], ['a', 'little', 'girl', 'is', 'walking', 'down', 'the', 'street', '.'], ['a', 'woman', 'is', 'placing', 'skewers', 'onto', 'a', 'rack', '.'], ['the', 'man', 'is', 'dropping', 'sliced', 'cucumbers', 'into', 'water', '.'], ['a', 'man', 'is', 'cutting', 'carpet', 'with', 'a', 'knife', '.'], ['a', 'man', 'is', 'cutting', 'carpet', 'with', 'a', 'knife', '.'], ['a', 'man', 'is', 'putting', 'foods', 'in', 'a', 'box', '.'], ['the', 'man', 'is', 'cracking', 'eggs', 'into', 'a', 'bowl', '.'], ['two', 'people', 'flipped', 'tractor', 'tires', 'in', 'a', 'race', '.'], ['a', 'herd', 'of', 'caribou', 'are', 'crossing', 'a', 'road', '.'], ['a', 'person', 'is', 'mixing', 'ingredients', 'in', 'a', 'bowl', '.'], ['a', 'monkey', 'is', 'swinging', 'from', 'tree', 'to', 'tree', '.'], ['a', 'puppy', 'is', 'playing', 'with', 'a', 'tennis', 'ball', '.'], ['a', 'woman', 'is', 'cooking', 'a', 'breaded', 'pork', 'chop', '.'], ['a', 'man', 'is', 'pouring', 'oil', 'into', 'a', 'skillet', '.'], ['a', 'bunny', 'is', 'playing', 'with', 'a', 'stuffed', 'bunny', '.'], ['a', 'man', 'is', 'cutting', 'carpet', 'with', 'a', 'knife', '.'], ['the', 'lady', 'broke', 'raw', 'eggs', 'into', 'a', 'bowl', '.'], ['a', 'man', 'wearing', 'a', 'hard', 'hat', 'is', 'dancing', '.'], ['a', 'polar', 'bear', 'is', 'sliding', 'across', 'the', 'snow', '.'], ['a', 'man', 'is', 'hugging', 'and', 'kissing', 'a', 'woman', '.'], ['a', 'man', 'is', 'showing', 'his', 'dog', 'to', 'the', 'camera', '.'], ['a', 'young', 'man', 'is', 'singing', 'while', 'playing', 'the', 'guitar', '.'], ['a', 'monkey', 'is', 'teasing', 'a', 'dog', 'at', 'the', 'zoo', '.'], ['a', 'man', 'and', 'woman', 'are', 'talking', 'on', 'the', 'phone', '.'], ['the', 'cat', 'teased', 'the', 'dog', 'by', 'touching', 'his', 'head', '.'], ['a', 'man', 'is', 'making', 'a', 'speech', 'at', 'the', 'podium', '.'], ['a', 'man', 'is', 'adding', 'a', 'silencer', 'to', 'a', 'gun', '.'], ['a', 'man', 'is', 'drawing', 'on', 'a', 'digital', 'dry', 'erase', 'board', '.'], ['the', 'little', 'girl', 'is', 'putting', 'her', 'hair', 'into', 'a', 'pony', 'tail', '.'], ['a', 'woman', 'is', 'pulling', 'some', 'kind', 'of', 'sticky', 'goo', 'out', 'of', 'a', 'bowl', '.'], ['a', 'woman', 'is', 'frying', 'shrimp', '.'], ['a', 'man', 'is', 'playing', 'piano', '.'], ['the', 'man', 'is', 'writing', 'a', 'letter', '.'], ['a', 'woman', 'is', 'cutting', 'an', 'onion', '.'], ['a', 'dog', 'is', 'riding', 'a', 'skateboard', '.'], ['a', 'man', 'is', 'cutting', 'a', 'vegetable', '.'], ['the', 'rhino', 'grazed', 'on', 'the', 'grass', '.'], ['a', 'man', 'is', 'preparing', 'some', 'dish', '.'], ['a', 'man', 'is', 'playing', 'a', 'guitar', '.'], ['a', 'baby', 'is', 'playing', 'a', 'guitar', '.'], ['the', 'man', 'and', 'woman', 'are', 'walking', '.'], ['a', 'man', 'is', 'eating', 'a', 'banana', '.'], ['a', 'person', 'is', 'slicing', 'carrots', 'in', 'half', '.'], ['a', 'woman', 'is', 'riding', 'on', 'an', 'elephant', '.'], ['a', 'man', 'shows', 'off', 'a', 'small', 'monkey', '.'], ['a', 'woman', 'is', 'dancing', 'on', 'a', 'stage', '.'], ['a', 'man', 'is', 'walking', 'down', 'a', 'road', '.'], ['the', 'dog', 'is', 'standing', 'at', 'the', 'counter', '.'], ['a', 'cat', 'and', 'a', 'boy', 'play', 'together', '.'], ['a', 'boy', 'is', 'walking', 'across', 'a', 'bridge', '.'], ['a', 'man', 'and', 'a', 'woman', 'shake', 'hands', '.'], ['a', 'man', 'is', 'spreading', 'butter', 'onto', 'bread', '.'], ['a', 'man', 'is', 'cutting', 'open', 'a', 'box', '.'], ['a', 'dog', 'is', 'walking', 'along', 'a', 'pool', '.'], ['a', 'woman', 'is', 'pouring', 'eyes', 'into', 'a', 'bowl', '.'], ['a', 'man', 'is', 'singing', 'and', 'playing', 'the', 'guitar', '.'], ['a', 'couple', 'are', 'eating', 'a', 'meal', 'and', 'talking', '.'], ['a', 'man', 'cut', 'a', 'tree', 'with', 'an', 'axe', '.'], ['a', 'woman', 'is', 'putting', 'flour', 'onto', 'some', 'meat', '.'], ['a', 'kitten', 'is', 'playing', 'with', 'a', 'woman', "'s", 'finger', '.'], ['a', 'man', 'is', 'carrying', 'another', 'man', 'on', 'his', 'back', '.'], ['a', 'woman', 'is', 'playing', 'an', 'acoustic', 'guitar', 'and', 'singing', '.'], ['a', 'woman', 'is', 'dancing', 'and', 'singing', 'in', 'the', 'rain', '.'], ['a', 'woman', 'is', 'cracking', 'an', 'egg', 'into', 'a', 'bowl', '.'], ['the', 'man', 'spanked', 'the', 'other', 'man', 'with', 'a', 'stick', '.'], ['the', 'man', 'is', 'feeding', 'a', 'mouse', 'to', 'the', 'snake', '.'], ['a', 'man', 'chops', 'down', 'a', 'tree', 'with', 'an', 'axe', '.'], ['the', 'man', 'added', 'seasoning', 'to', 'water', 'in', 'a', 'bowl', '.'], ['a', 'man', 'and', 'a', 'woman', 'are', 'riding', 'on', 'a', 'bicycle', '.'], ['the', 'man', 'used', 'a', 'machine', 'to', 'chop', 'up', 'a', 'potatoe', '.'], ['a', 'man', 'is', 'wiping', 'the', 'edge', 'of', 'a', 'bowl', 'off', '.'], ['a', 'group', 'of', 'boys', 'are', 'playing', 'soccer', 'on', 'the', 'beach', '.'], ['a', 'man', 'is', 'spreading', 'shredded', 'cheese', 'on', 'an', 'uncooked', 'pizza', '.'], ['someone', 'is', 'drilling', 'a', 'hole', 'in', 'a', 'piece', 'of', 'wood', '.'], ['a', 'person', 'is', 'dumping', 'chopped', 'vegetables', 'into', 'a', 'pot', 'on', 'the', 'stove', '.'], ['a', 'woman', 'is', 'mixing', 'a', 'cream', 'in', 'a', 'glass', 'with', 'a', 'spoon', '.'], ['a', 'woman', 'picks', 'up', 'and', 'holds', 'a', 'baby', 'kangaroo', 'in', 'her', 'arms', '.'], ['a', 'man', 'is', 'sprinkling', 'seasoning', 'on', 'several', 'split', 'and', 'buttered', 'loaves', 'of', 'bread', '.'], ['a', 'cat', 'is', 'crawling', 'under', 'a', 'piece', 'of', 'furniture', 'on', 'it', "'s", 'back', '.'], ['a', 'woman', 'driving', 'a', 'car', 'is', 'talking', 'to', 'the', 'man', 'seated', 'beside', 'her', '.'], ['a', 'man', 'sitting', 'on', 'the', 'floor', 'in', 'a', 'room', 'is', 'strumming', 'a', 'guitar', '.'], ['a', 'woman', 'is', 'making', 'noodles', '.'], ['a', 'woman', 'is', 'peeling', 'a', 'potato', '.'], ['a', 'man', 'is', 'playing', 'a', 'flute', '.'], ['a', 'monkey', 'pulled', 'a', 'dogs', 'tail', '.'], ['a', 'woman', 'is', 'slicing', 'a', 'carrot', '.'], ['a', 'soccer', 'player', 'scores', 'a', 'goal', '.'], ['a', 'woman', 'is', 'cleaning', 'a', 'shrimp', '.'], ['a', 'man', 'is', 'walking', 'down', 'the', 'street', '.'], ['a', 'tied', 'man', 'is', 'putting', 'into', 'water', '.'], ['a', 'man', 'is', 'kicking', 'a', 'soccer', 'ball', '.'], ['a', 'man', 'is', 'riding', 'a', 'motor', 'scooter', '.'], ['a', 'man', 'is', 'emptying', 'a', 'plastic', 'container', '.'], ['a', 'dog', 'is', 'laying', 'on', 'the', 'floor', '.'], ['the', 'woman', 'is', 'poking', 'holes', 'in', 'the', 'potato', '.'], ['a', 'man', 'is', 'spreading', 'butter', 'on', 'a', 'tortilla', '.'], ['a', 'young', 'girl', 'is', 'singing', 'on', 'a', 'stage', '.'], ['a', 'man', 'pours', 'water', 'over', 'a', 'pilots', 'head', '.'], ['a', 'man', 'is', 'cutting', 'tomatoes', 'with', 'a', 'cleaver', '.'], ['a', 'man', 'breaks', 'cinder', 'blocks', 'on', 'another', 'man', '.'], ['a', 'child', 'is', 'stuck', 'in', 'a', 'dog', 'house', '.'], ['the', 'man', 'poured', 'oil', 'on', 'the', 'tomato', 'slices', '.'], ['a', 'man', 'is', 'putting', 'meat', 'in', 'a', 'pan', '.'], ['a', 'woman', 'is', 'applying', 'stickers', 'to', 'her', 'face', '.'], ['a', 'person', 'throws', 'a', 'cat', 'on', 'the', 'ceiling', '.'], ['the', 'dog', 'ran', 'in', 'the', 'water', 'at', 'the', 'beach', '.'], ['a', 'woman', 'is', 'feeding', 'her', 'baby', 'with', 'a', 'bottle', '.'], ['the', 'lady', 'fried', 'the', 'breaded', 'meat', 'in', 'hot', 'oil', '.'], ['someone', 'is', 'banging', 'a', 'camera', 'lense', 'against', 'a', 'nail', '.'], ['a', 'teenager', 'talks', 'to', 'a', 'girl', 'over', 'a', 'webcam', '.'], ['a', 'man', 'sliced', 'a', 'plastic', 'bottle', 'with', 'a', 'sword', '.'], ['a', 'man', 'is', 'smearing', 'butter', 'on', 'a', 'slice', 'of', 'garlic', 'bread', '.'], ['a', 'dog', 'is', 'trying', 'to', 'eat', 'the', 'bacon', 'on', 'its', 'back', '.'], ['four', 'young', 'men', 'stand', 'still', 'as', 'a', 'car', 'explodes', 'behind', 'them', '.'], ['a', 'man', 'and', 'woman', 'are', 'walking', 'arm', 'in', 'arm', 'through', 'a', 'wooded', 'area', '.'], ['a', 'woman', 'is', 'frying', 'meat', '.'], ['a', 'girl', 'loses', 'her', 'kite', '.'], ['someone', 'is', 'riding', 'a', 'horse', '.'], ['a', 'woman', 'rides', 'an', 'elephant', '.'], ['foxes', 'are', 'eating', 'from', 'a', 'plate', '.'], ['the', 'women', 'are', 'singing', 'and', 'dancing', '.'], ['a', 'man', 'is', 'swinging', 'on', 'a', 'rope', '.'], ['a', 'woman', 'is', 'slicing', 'meat', 'with', 'a', 'knife', '.'], ['a', 'baby', 'rhino', 'is', 'following', 'an', 'adult', 'rhino', '.'], ['a', 'woman', 'is', 'preparing', 'an', 'egg', 'in', 'a', 'skillet', '.'], ['pieces', 'of', 'butter', 'is', 'added', 'in', 'the', 'stand', 'mixer', '.'], ['the', 'men', 'are', 'putting', 'suitcases', 'into', 'the', 'car', "'s", 'trunk', '.'], ['a', 'youngster', 'is', 'sitting', 'on', 'a', 'piano', '.'], ['a', 'man', 'is', 'staring', 'out', 'the', 'window', '.'], ['a', 'woman', 'walks', 'her', 'dog', 'onto', 'astro', 'turf', '.'], ['a', 'man', 'and', 'woman', 'are', 'driving', 'down', 'the', 'road', 'in', 'an', 'open', 'air', 'vehicle', '.'], ['a', 'man', 'is', 'showing', 'a', 't', 'shirt', '.'], ['a', 'woman', 'is', 'applying', 'cosmetics', 'to', 'her', 'eyelid', '.'], ['a', 'baby', 'plays', 'with', 'a', 'dog', 'and', 'ball', '.'], ['a', 'little', 'kid', 'is', 'running', 'around', 'a', 'fishing', 'pole', '.'], ['a', 'man', 'drills', 'a', 'hole', 'in', 'a', 'piece', 'of', 'wood', '.'], ['a', 'man', 'is', 'playing', 'a', 'flute', '.'], ['three', 'men', 'are', 'jumping', 'off', 'a', 'wall', '.'], ['a', 'woman', 'seated', 'is', 'singing', 'a', 'song', 'and', 'playing', 'the', 'guitar', 'too', '.'], ['a', 'guinea', 'pig', 'eats', 'a', 'carrot', '.'], ['an', 'older', 'woman', 'is', 'pouring', 'oil', 'into', 'a', 'skillet', 'on', 'the', 'stove', '.'], ['a', 'man', 'is', 'walking', 'with', 'several', 'dogs', '.'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['there', 'must', 'be', 'a', 'balance', 'as', 'a', 'whole', '.'], ['there', 'must', 'be', 'a', 'balance', 'as', 'a', 'whole', '.'], ['there', 'must', 'be', 'a', 'balance', 'as', 'a', 'whole', '.'], ['there', 'must', 'be', 'a', 'balance', 'as', 'a', 'whole', '.'], ['there', 'must', 'be', 'a', 'balance', 'as', 'a', 'whole', '.'], ['there', 'must', 'be', 'a', 'balance', 'as', 'a', 'whole', '.'], ['there', 'must', 'be', 'a', 'balance', 'as', 'a', 'whole', '.'], ['there', 'must', 'be', 'a', 'balance', 'as', 'a', 'whole', '.'], ['there', 'must', 'be', 'a', 'balance', 'as', 'a', 'whole', '.'], ['there', 'must', 'be', 'a', 'balance', 'as', 'a', 'whole', '.'], ['there', 'must', 'be', 'a', 'balance', 'as', 'a', 'whole', '.'], ['there', 'must', 'be', 'a', 'balance', 'as', 'a', 'whole', '.'], ['there', 'must', 'be', 'a', 'balance', 'as', 'a', 'whole', '.'], ['there', 'must', 'be', 'a', 'balance', 'as', 'a', 'whole', '.'], ['there', 'must', 'be', 'a', 'balance', 'as', 'a', 'whole', '.'], ['there', 'must', 'be', 'a', 'balance', 'as', 'a', 'whole', '.'], ['there', 'must', 'be', 'a', 'balance', 'as', 'a', 'whole', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['then', 'perhaps', 'we', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['then', 'perhaps', 'we', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['then', 'perhaps', 'we', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['then', 'perhaps', 'we', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['then', 'perhaps', 'we', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['then', 'perhaps', 'we', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['then', 'perhaps', 'we', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['then', 'perhaps', 'we', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['then', 'perhaps', 'we', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['then', 'perhaps', 'we', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['then', 'perhaps', 'we', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['then', 'perhaps', 'we', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['then', 'perhaps', 'we', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['then', 'perhaps', 'we', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['then', 'perhaps', 'we', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['then', 'perhaps', 'we', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['then', 'perhaps', 'we', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['the', 'vote', 'will', 'take', 'place', 'today', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'today', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'today', 'at', '5.30', 'p.m', '.'], ['that', 'provision', 'could', 'open', 'the', 'door', 'wide', 'to', 'arbitrariness', '.'], ['the', 'vote', 'will', 'take', 'place', 'today', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'today', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'today', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'today', 'at', '5.30', 'p.m', '.'], ['the', 'standards', 'are', 'scarcely', 'comparable', ',', 'let', 'alone', 'transferable', '.'], ['that', 'provision', 'could', 'open', 'the', 'door', 'wide', 'to', 'arbitrariness', '.'], ['the', 'vote', 'will', 'take', 'place', 'today', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'today', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'today', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'today', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'today', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'today', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'today', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'today', 'at', '5.30', 'p.m', '.'], ['that', 'provision', 'could', 'open', 'the', 'door', 'wide', 'to', 'arbitrariness', '.'], ['the', 'vote', 'will', 'take', 'place', 'today', 'at', '5.30', 'p.m', '.'], ['the', 'standards', 'are', 'scarcely', 'comparable', ',', 'let', 'alone', 'transferable', '.'], ['the', 'standards', 'are', 'scarcely', 'comparable', ',', 'let', 'alone', 'transferable', '.'], ['the', 'standards', 'are', 'scarcely', 'comparable', ',', 'let', 'alone', 'transferable', '.'], ['the', 'standards', 'are', 'scarcely', 'comparable', ',', 'let', 'alone', 'transferable', '.'], ['the', 'standards', 'are', 'scarcely', 'comparable', ',', 'let', 'alone', 'transferable', '.'], ['the', 'standards', 'are', 'scarcely', 'comparable', ',', 'let', 'alone', 'transferable', '.'], ['the', 'standards', 'are', 'scarcely', 'comparable', ',', 'let', 'alone', 'transferable', '.'], ['the', 'standards', 'are', 'scarcely', 'comparable', ',', 'let', 'alone', 'transferable', '.'], ['the', 'standards', 'are', 'scarcely', 'comparable', ',', 'let', 'alone', 'transferable', '.'], ['the', 'standards', 'are', 'scarcely', 'comparable', ',', 'let', 'alone', 'transferable', '.'], ['the', 'standards', 'are', 'scarcely', 'comparable', ',', 'let', 'alone', 'transferable', '.'], ['the', 'standards', 'are', 'scarcely', 'comparable', ',', 'let', 'alone', 'transferable', '.'], ['the', 'standards', 'are', 'scarcely', 'comparable', ',', 'let', 'alone', 'transferable', '.'], ['the', 'standards', 'are', 'scarcely', 'comparable', ',', 'let', 'alone', 'transferable', '.'], ['the', 'standards', 'are', 'scarcely', 'comparable', ',', 'let', 'alone', 'transferable', '.'], ['the', 'vote', 'will', 'take', 'place', 'today', 'at', '5.30', 'p.m', '.'], ['that', 'provision', 'could', 'open', 'the', 'door', 'wide', 'to', 'arbitrariness', '.'], ['that', 'provision', 'could', 'open', 'the', 'door', 'wide', 'to', 'arbitrariness', '.'], ['the', 'standards', 'are', 'scarcely', 'comparable', ',', 'let', 'alone', 'transferable', '.'], ['that', 'provision', 'could', 'open', 'the', 'door', 'wide', 'to', 'arbitrariness', '.'], ['that', 'provision', 'could', 'open', 'the', 'door', 'wide', 'to', 'arbitrariness', '.'], ['that', 'provision', 'could', 'open', 'the', 'door', 'wide', 'to', 'arbitrariness', '.'], ['that', 'provision', 'could', 'open', 'the', 'door', 'wide', 'to', 'arbitrariness', '.'], ['that', 'provision', 'could', 'open', 'the', 'door', 'wide', 'to', 'arbitrariness', '.'], ['that', 'provision', 'could', 'open', 'the', 'door', 'wide', 'to', 'arbitrariness', '.'], ['that', 'provision', 'could', 'open', 'the', 'door', 'wide', 'to', 'arbitrariness', '.'], ['that', 'provision', 'could', 'open', 'the', 'door', 'wide', 'to', 'arbitrariness', '.'], ['that', 'provision', 'could', 'open', 'the', 'door', 'wide', 'to', 'arbitrariness', '.'], ['that', 'provision', 'could', 'open', 'the', 'door', 'wide', 'to', 'arbitrariness', '.'], ['that', 'provision', 'could', 'open', 'the', 'door', 'wide', 'to', 'arbitrariness', '.'], ['that', 'provision', 'could', 'open', 'the', 'door', 'wide', 'to', 'arbitrariness', '.'], ['unfortunately', ',', 'others', 'separate', 'on', 'the', 'basis', 'of', 'accumulated', 'hatred', '.'], ['unfortunately', ',', 'others', 'separate', 'on', 'the', 'basis', 'of', 'accumulated', 'hatred', '.'], ['unfortunately', ',', 'others', 'separate', 'on', 'the', 'basis', 'of', 'accumulated', 'hatred', '.'], ['unfortunately', ',', 'others', 'separate', 'on', 'the', 'basis', 'of', 'accumulated', 'hatred', '.'], ['unfortunately', ',', 'others', 'separate', 'on', 'the', 'basis', 'of', 'accumulated', 'hatred', '.'], ['unfortunately', ',', 'others', 'separate', 'on', 'the', 'basis', 'of', 'accumulated', 'hatred', '.'], ['unfortunately', ',', 'others', 'separate', 'on', 'the', 'basis', 'of', 'accumulated', 'hatred', '.'], ['unfortunately', ',', 'others', 'separate', 'on', 'the', 'basis', 'of', 'accumulated', 'hatred', '.'], ['unfortunately', ',', 'others', 'separate', 'on', 'the', 'basis', 'of', 'accumulated', 'hatred', '.'], ['unfortunately', ',', 'others', 'separate', 'on', 'the', 'basis', 'of', 'accumulated', 'hatred', '.'], ['unfortunately', ',', 'others', 'separate', 'on', 'the', 'basis', 'of', 'accumulated', 'hatred', '.'], ['unfortunately', ',', 'others', 'separate', 'on', 'the', 'basis', 'of', 'accumulated', 'hatred', '.'], ['unfortunately', ',', 'others', 'separate', 'on', 'the', 'basis', 'of', 'accumulated', 'hatred', '.'], ['unfortunately', ',', 'others', 'separate', 'on', 'the', 'basis', 'of', 'accumulated', 'hatred', '.'], ['unfortunately', ',', 'others', 'separate', 'on', 'the', 'basis', 'of', 'accumulated', 'hatred', '.'], ['unfortunately', ',', 'others', 'separate', 'on', 'the', 'basis', 'of', 'accumulated', 'hatred', '.'], ['unfortunately', ',', 'others', 'separate', 'on', 'the', 'basis', 'of', 'accumulated', 'hatred', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'attends', 'sittings', 'quite', 'faithfully', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'attends', 'sittings', 'quite', 'faithfully', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'attends', 'sittings', 'quite', 'faithfully', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'attends', 'sittings', 'quite', 'faithfully', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'attends', 'sittings', 'quite', 'faithfully', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'attends', 'sittings', 'quite', 'faithfully', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'attends', 'sittings', 'quite', 'faithfully', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'attends', 'sittings', 'quite', 'faithfully', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'attends', 'sittings', 'quite', 'faithfully', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'attends', 'sittings', 'quite', 'faithfully', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'attends', 'sittings', 'quite', 'faithfully', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'attends', 'sittings', 'quite', 'faithfully', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'attends', 'sittings', 'quite', 'faithfully', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'attends', 'sittings', 'quite', 'faithfully', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'attends', 'sittings', 'quite', 'faithfully', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'attends', 'sittings', 'quite', 'faithfully', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'attends', 'sittings', 'quite', 'faithfully', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['the', 'european', 'union', 'has', 'got', 'to', 'do', 'something', 'and', 'do', 'it', 'quickly', '.'], ['the', 'european', 'union', 'has', 'got', 'to', 'do', 'something', 'and', 'do', 'it', 'quickly', '.'], ['the', 'european', 'union', 'has', 'got', 'to', 'do', 'something', 'and', 'do', 'it', 'quickly', '.'], ['the', 'european', 'union', 'has', 'got', 'to', 'do', 'something', 'and', 'do', 'it', 'quickly', '.'], ['the', 'european', 'union', 'has', 'got', 'to', 'do', 'something', 'and', 'do', 'it', 'quickly', '.'], ['the', 'european', 'union', 'has', 'got', 'to', 'do', 'something', 'and', 'do', 'it', 'quickly', '.'], ['the', 'european', 'union', 'has', 'got', 'to', 'do', 'something', 'and', 'do', 'it', 'quickly', '.'], ['the', 'european', 'union', 'has', 'got', 'to', 'do', 'something', 'and', 'do', 'it', 'quickly', '.'], ['the', 'european', 'union', 'has', 'got', 'to', 'do', 'something', 'and', 'do', 'it', 'quickly', '.'], ['the', 'european', 'union', 'has', 'got', 'to', 'do', 'something', 'and', 'do', 'it', 'quickly', '.'], ['the', 'european', 'union', 'has', 'got', 'to', 'do', 'something', 'and', 'do', 'it', 'quickly', '.'], ['the', 'european', 'union', 'has', 'got', 'to', 'do', 'something', 'and', 'do', 'it', 'quickly', '.'], ['the', 'european', 'union', 'has', 'got', 'to', 'do', 'something', 'and', 'do', 'it', 'quickly', '.'], ['the', 'european', 'union', 'has', 'got', 'to', 'do', 'something', 'and', 'do', 'it', 'quickly', '.'], ['the', 'european', 'union', 'has', 'got', 'to', 'do', 'something', 'and', 'do', 'it', 'quickly', '.'], ['the', 'european', 'union', 'has', 'got', 'to', 'do', 'something', 'and', 'do', 'it', 'quickly', '.'], ['the', 'european', 'union', 'has', 'got', 'to', 'do', 'something', 'and', 'do', 'it', 'quickly', '.'], ['we', 'often', 'pontificate', 'here', 'about', 'being', 'the', 'representatives', 'of', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'pontificate', 'here', 'about', 'being', 'the', 'representatives', 'of', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'pontificate', 'here', 'about', 'being', 'the', 'representatives', 'of', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'pontificate', 'here', 'about', 'being', 'the', 'representatives', 'of', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'pontificate', 'here', 'about', 'being', 'the', 'representatives', 'of', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'pontificate', 'here', 'about', 'being', 'the', 'representatives', 'of', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'pontificate', 'here', 'about', 'being', 'the', 'representatives', 'of', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'pontificate', 'here', 'about', 'being', 'the', 'representatives', 'of', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'pontificate', 'here', 'about', 'being', 'the', 'representatives', 'of', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'pontificate', 'here', 'about', 'being', 'the', 'representatives', 'of', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'pontificate', 'here', 'about', 'being', 'the', 'representatives', 'of', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'pontificate', 'here', 'about', 'being', 'the', 'representatives', 'of', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'pontificate', 'here', 'about', 'being', 'the', 'representatives', 'of', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'pontificate', 'here', 'about', 'being', 'the', 'representatives', 'of', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'pontificate', 'here', 'about', 'being', 'the', 'representatives', 'of', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'pontificate', 'here', 'about', 'being', 'the', 'representatives', 'of', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'pontificate', 'here', 'about', 'being', 'the', 'representatives', 'of', 'the', 'citizens', 'of', 'europe', '.'], ['mr', 'president', ',', 'i', 'rise', 'in', 'defence', 'of', 'the', 'workers', 'in', 'the', 'industry', '.'], ['mr', 'president', ',', 'i', 'rise', 'in', 'defence', 'of', 'the', 'workers', 'in', 'the', 'industry', '.'], ['let', 'me', 'remind', 'you', 'that', 'our', 'allies', 'include', 'fervent', 'supporters', 'of', 'this', 'tax', '.'], ['mr', 'president', ',', 'i', 'rise', 'in', 'defence', 'of', 'the', 'workers', 'in', 'the', 'industry', '.'], ['mr', 'president', ',', 'i', 'rise', 'in', 'defence', 'of', 'the', 'workers', 'in', 'the', 'industry', '.'], ['let', 'me', 'remind', 'you', 'that', 'our', 'allies', 'include', 'fervent', 'supporters', 'of', 'this', 'tax', '.'], ['mr', 'president', ',', 'i', 'rise', 'in', 'defence', 'of', 'the', 'workers', 'in', 'the', 'industry', '.'], ['mr', 'president', ',', 'i', 'rise', 'in', 'defence', 'of', 'the', 'workers', 'in', 'the', 'industry', '.'], ['mr', 'president', ',', 'i', 'rise', 'in', 'defence', 'of', 'the', 'workers', 'in', 'the', 'industry', '.'], ['mr', 'president', ',', 'i', 'rise', 'in', 'defence', 'of', 'the', 'workers', 'in', 'the', 'industry', '.'], ['mr', 'president', ',', 'i', 'rise', 'in', 'defence', 'of', 'the', 'workers', 'in', 'the', 'industry', '.'], ['mr', 'president', ',', 'i', 'rise', 'in', 'defence', 'of', 'the', 'workers', 'in', 'the', 'industry', '.'], ['mr', 'president', ',', 'i', 'rise', 'in', 'defence', 'of', 'the', 'workers', 'in', 'the', 'industry', '.'], ['mr', 'president', ',', 'i', 'rise', 'in', 'defence', 'of', 'the', 'workers', 'in', 'the', 'industry', '.'], ['mr', 'president', ',', 'i', 'rise', 'in', 'defence', 'of', 'the', 'workers', 'in', 'the', 'industry', '.'], ['mr', 'president', ',', 'i', 'rise', 'in', 'defence', 'of', 'the', 'workers', 'in', 'the', 'industry', '.'], ['let', 'me', 'remind', 'you', 'that', 'our', 'allies', 'include', 'fervent', 'supporters', 'of', 'this', 'tax', '.'], ['let', 'me', 'remind', 'you', 'that', 'our', 'allies', 'include', 'fervent', 'supporters', 'of', 'this', 'tax', '.'], ['mr', 'president', ',', 'i', 'rise', 'in', 'defence', 'of', 'the', 'workers', 'in', 'the', 'industry', '.'], ['mr', 'president', ',', 'i', 'rise', 'in', 'defence', 'of', 'the', 'workers', 'in', 'the', 'industry', '.'], ['let', 'me', 'remind', 'you', 'that', 'our', 'allies', 'include', 'fervent', 'supporters', 'of', 'this', 'tax', '.'], ['let', 'me', 'remind', 'you', 'that', 'our', 'allies', 'include', 'fervent', 'supporters', 'of', 'this', 'tax', '.'], ['let', 'me', 'remind', 'you', 'that', 'our', 'allies', 'include', 'fervent', 'supporters', 'of', 'this', 'tax', '.'], ['let', 'me', 'remind', 'you', 'that', 'our', 'allies', 'include', 'fervent', 'supporters', 'of', 'this', 'tax', '.'], ['let', 'me', 'remind', 'you', 'that', 'our', 'allies', 'include', 'fervent', 'supporters', 'of', 'this', 'tax', '.'], ['let', 'me', 'remind', 'you', 'that', 'our', 'allies', 'include', 'fervent', 'supporters', 'of', 'this', 'tax', '.'], ['let', 'me', 'remind', 'you', 'that', 'our', 'allies', 'include', 'fervent', 'supporters', 'of', 'this', 'tax', '.'], ['let', 'me', 'remind', 'you', 'that', 'our', 'allies', 'include', 'fervent', 'supporters', 'of', 'this', 'tax', '.'], ['mr', 'president', ',', 'i', 'rise', 'in', 'defence', 'of', 'the', 'workers', 'in', 'the', 'industry', '.'], ['let', 'me', 'remind', 'you', 'that', 'our', 'allies', 'include', 'fervent', 'supporters', 'of', 'this', 'tax', '.'], ['let', 'me', 'remind', 'you', 'that', 'our', 'allies', 'include', 'fervent', 'supporters', 'of', 'this', 'tax', '.'], ['let', 'me', 'remind', 'you', 'that', 'our', 'allies', 'include', 'fervent', 'supporters', 'of', 'this', 'tax', '.'], ['let', 'me', 'remind', 'you', 'that', 'our', 'allies', 'include', 'fervent', 'supporters', 'of', 'this', 'tax', '.'], ['let', 'me', 'remind', 'you', 'that', 'our', 'allies', 'include', 'fervent', 'supporters', 'of', 'this', 'tax', '.'], ['selective', 'aid', ',', 'such', 'as', 'market', 'support', 'and', 'a', 'grass', 'subsidy', ',', 'are', 'essential', '.'], ['selective', 'aid', ',', 'such', 'as', 'market', 'support', 'and', 'a', 'grass', 'subsidy', ',', 'are', 'essential', '.'], ['selective', 'aid', ',', 'such', 'as', 'market', 'support', 'and', 'a', 'grass', 'subsidy', ',', 'are', 'essential', '.'], ['selective', 'aid', ',', 'such', 'as', 'market', 'support', 'and', 'a', 'grass', 'subsidy', ',', 'are', 'essential', '.'], ['neither', 'was', 'there', 'a', 'qualified', 'majority', 'within', 'this', 'house', 'to', 'revert', 'to', 'article', '272', '.'], ['neither', 'was', 'there', 'a', 'qualified', 'majority', 'within', 'this', 'house', 'to', 'revert', 'to', 'article', '272', '.'], ['neither', 'was', 'there', 'a', 'qualified', 'majority', 'within', 'this', 'house', 'to', 'revert', 'to', 'article', '272', '.'], ['neither', 'was', 'there', 'a', 'qualified', 'majority', 'within', 'this', 'house', 'to', 'revert', 'to', 'article', '272', '.'], ['neither', 'was', 'there', 'a', 'qualified', 'majority', 'within', 'this', 'house', 'to', 'revert', 'to', 'article', '272', '.'], ['neither', 'was', 'there', 'a', 'qualified', 'majority', 'within', 'this', 'house', 'to', 'revert', 'to', 'article', '272', '.'], ['neither', 'was', 'there', 'a', 'qualified', 'majority', 'within', 'this', 'house', 'to', 'revert', 'to', 'article', '272', '.'], ['neither', 'was', 'there', 'a', 'qualified', 'majority', 'within', 'this', 'house', 'to', 'revert', 'to', 'article', '272', '.'], ['neither', 'was', 'there', 'a', 'qualified', 'majority', 'within', 'this', 'house', 'to', 'revert', 'to', 'article', '272', '.'], ['selective', 'aid', ',', 'such', 'as', 'market', 'support', 'and', 'a', 'grass', 'subsidy', ',', 'are', 'essential', '.'], ['selective', 'aid', ',', 'such', 'as', 'market', 'support', 'and', 'a', 'grass', 'subsidy', ',', 'are', 'essential', '.'], ['selective', 'aid', ',', 'such', 'as', 'market', 'support', 'and', 'a', 'grass', 'subsidy', ',', 'are', 'essential', '.'], ['selective', 'aid', ',', 'such', 'as', 'market', 'support', 'and', 'a', 'grass', 'subsidy', ',', 'are', 'essential', '.'], ['selective', 'aid', ',', 'such', 'as', 'market', 'support', 'and', 'a', 'grass', 'subsidy', ',', 'are', 'essential', '.'], ['selective', 'aid', ',', 'such', 'as', 'market', 'support', 'and', 'a', 'grass', 'subsidy', ',', 'are', 'essential', '.'], ['neither', 'was', 'there', 'a', 'qualified', 'majority', 'within', 'this', 'house', 'to', 'revert', 'to', 'article', '272', '.'], ['selective', 'aid', ',', 'such', 'as', 'market', 'support', 'and', 'a', 'grass', 'subsidy', ',', 'are', 'essential', '.'], ['selective', 'aid', ',', 'such', 'as', 'market', 'support', 'and', 'a', 'grass', 'subsidy', ',', 'are', 'essential', '.'], ['selective', 'aid', ',', 'such', 'as', 'market', 'support', 'and', 'a', 'grass', 'subsidy', ',', 'are', 'essential', '.'], ['selective', 'aid', ',', 'such', 'as', 'market', 'support', 'and', 'a', 'grass', 'subsidy', ',', 'are', 'essential', '.'], ['selective', 'aid', ',', 'such', 'as', 'market', 'support', 'and', 'a', 'grass', 'subsidy', ',', 'are', 'essential', '.'], ['neither', 'was', 'there', 'a', 'qualified', 'majority', 'within', 'this', 'house', 'to', 'revert', 'to', 'article', '272', '.'], ['neither', 'was', 'there', 'a', 'qualified', 'majority', 'within', 'this', 'house', 'to', 'revert', 'to', 'article', '272', '.'], ['neither', 'was', 'there', 'a', 'qualified', 'majority', 'within', 'this', 'house', 'to', 'revert', 'to', 'article', '272', '.'], ['neither', 'was', 'there', 'a', 'qualified', 'majority', 'within', 'this', 'house', 'to', 'revert', 'to', 'article', '272', '.'], ['selective', 'aid', ',', 'such', 'as', 'market', 'support', 'and', 'a', 'grass', 'subsidy', ',', 'are', 'essential', '.'], ['selective', 'aid', ',', 'such', 'as', 'market', 'support', 'and', 'a', 'grass', 'subsidy', ',', 'are', 'essential', '.'], ['neither', 'was', 'there', 'a', 'qualified', 'majority', 'within', 'this', 'house', 'to', 'revert', 'to', 'article', '272', '.'], ['neither', 'was', 'there', 'a', 'qualified', 'majority', 'within', 'this', 'house', 'to', 'revert', 'to', 'article', '272', '.'], ['neither', 'was', 'there', 'a', 'qualified', 'majority', 'within', 'this', 'house', 'to', 'revert', 'to', 'article', '272', '.'], ['it', 'increases', 'the', 'power', 'of', 'the', 'big', 'countries', 'at', 'the', 'expense', 'of', 'the', 'small', 'countries', '.'], ['it', 'increases', 'the', 'power', 'of', 'the', 'big', 'countries', 'at', 'the', 'expense', 'of', 'the', 'small', 'countries', '.'], ['it', 'increases', 'the', 'power', 'of', 'the', 'big', 'countries', 'at', 'the', 'expense', 'of', 'the', 'small', 'countries', '.'], ['it', 'increases', 'the', 'power', 'of', 'the', 'big', 'countries', 'at', 'the', 'expense', 'of', 'the', 'small', 'countries', '.'], ['it', 'increases', 'the', 'power', 'of', 'the', 'big', 'countries', 'at', 'the', 'expense', 'of', 'the', 'small', 'countries', '.'], ['it', 'increases', 'the', 'power', 'of', 'the', 'big', 'countries', 'at', 'the', 'expense', 'of', 'the', 'small', 'countries', '.'], ['it', 'increases', 'the', 'power', 'of', 'the', 'big', 'countries', 'at', 'the', 'expense', 'of', 'the', 'small', 'countries', '.'], ['it', 'increases', 'the', 'power', 'of', 'the', 'big', 'countries', 'at', 'the', 'expense', 'of', 'the', 'small', 'countries', '.'], ['it', 'increases', 'the', 'power', 'of', 'the', 'big', 'countries', 'at', 'the', 'expense', 'of', 'the', 'small', 'countries', '.'], ['it', 'increases', 'the', 'power', 'of', 'the', 'big', 'countries', 'at', 'the', 'expense', 'of', 'the', 'small', 'countries', '.'], ['it', 'increases', 'the', 'power', 'of', 'the', 'big', 'countries', 'at', 'the', 'expense', 'of', 'the', 'small', 'countries', '.'], ['it', 'increases', 'the', 'power', 'of', 'the', 'big', 'countries', 'at', 'the', 'expense', 'of', 'the', 'small', 'countries', '.'], ['it', 'increases', 'the', 'power', 'of', 'the', 'big', 'countries', 'at', 'the', 'expense', 'of', 'the', 'small', 'countries', '.'], ['it', 'increases', 'the', 'power', 'of', 'the', 'big', 'countries', 'at', 'the', 'expense', 'of', 'the', 'small', 'countries', '.'], ['it', 'increases', 'the', 'power', 'of', 'the', 'big', 'countries', 'at', 'the', 'expense', 'of', 'the', 'small', 'countries', '.'], ['it', 'increases', 'the', 'power', 'of', 'the', 'big', 'countries', 'at', 'the', 'expense', 'of', 'the', 'small', 'countries', '.'], ['it', 'increases', 'the', 'power', 'of', 'the', 'big', 'countries', 'at', 'the', 'expense', 'of', 'the', 'small', 'countries', '.'], ['the', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.'], ['the', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.'], ['the', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.'], ['the', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.'], ['the', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.'], ['the', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.'], ['the', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.'], ['the', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.'], ['the', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.'], ['the', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.'], ['the', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.'], ['the', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.'], ['the', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.'], ['it', 'is', 'our', 'job', 'to', 'continue', 'to', 'support', 'latvia', 'with', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['the', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.'], ['as', 'i', 'already', 'explained', 'during', 'second', 'reading', ',', 'there', 'is', 'a', 'crisis', 'underlying', 'this', 'directive', 'amendment', '.'], ['as', 'i', 'already', 'explained', 'during', 'second', 'reading', ',', 'there', 'is', 'a', 'crisis', 'underlying', 'this', 'directive', 'amendment', '.'], ['the', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.'], ['the', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.'], ['the', 'leaders', 'have', 'now', 'been', 'given', 'a', 'new', 'chance', 'and', 'let', 'us', 'hope', 'they', 'seize', 'it', '.'], ['it', 'is', 'our', 'job', 'to', 'continue', 'to', 'support', 'latvia', 'with', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['as', 'i', 'already', 'explained', 'during', 'second', 'reading', ',', 'there', 'is', 'a', 'crisis', 'underlying', 'this', 'directive', 'amendment', '.'], ['it', 'is', 'our', 'job', 'to', 'continue', 'to', 'support', 'latvia', 'with', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'our', 'job', 'to', 'continue', 'to', 'support', 'latvia', 'with', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['as', 'i', 'already', 'explained', 'during', 'second', 'reading', ',', 'there', 'is', 'a', 'crisis', 'underlying', 'this', 'directive', 'amendment', '.'], ['it', 'is', 'our', 'job', 'to', 'continue', 'to', 'support', 'latvia', 'with', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'our', 'job', 'to', 'continue', 'to', 'support', 'latvia', 'with', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'our', 'job', 'to', 'continue', 'to', 'support', 'latvia', 'with', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'our', 'job', 'to', 'continue', 'to', 'support', 'latvia', 'with', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'our', 'job', 'to', 'continue', 'to', 'support', 'latvia', 'with', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'our', 'job', 'to', 'continue', 'to', 'support', 'latvia', 'with', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['as', 'i', 'already', 'explained', 'during', 'second', 'reading', ',', 'there', 'is', 'a', 'crisis', 'underlying', 'this', 'directive', 'amendment', '.'], ['as', 'i', 'already', 'explained', 'during', 'second', 'reading', ',', 'there', 'is', 'a', 'crisis', 'underlying', 'this', 'directive', 'amendment', '.'], ['it', 'is', 'our', 'job', 'to', 'continue', 'to', 'support', 'latvia', 'with', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'our', 'job', 'to', 'continue', 'to', 'support', 'latvia', 'with', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'our', 'job', 'to', 'continue', 'to', 'support', 'latvia', 'with', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'our', 'job', 'to', 'continue', 'to', 'support', 'latvia', 'with', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['as', 'i', 'already', 'explained', 'during', 'second', 'reading', ',', 'there', 'is', 'a', 'crisis', 'underlying', 'this', 'directive', 'amendment', '.'], ['as', 'i', 'already', 'explained', 'during', 'second', 'reading', ',', 'there', 'is', 'a', 'crisis', 'underlying', 'this', 'directive', 'amendment', '.'], ['it', 'is', 'our', 'job', 'to', 'continue', 'to', 'support', 'latvia', 'with', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['as', 'i', 'already', 'explained', 'during', 'second', 'reading', ',', 'there', 'is', 'a', 'crisis', 'underlying', 'this', 'directive', 'amendment', '.'], ['as', 'i', 'already', 'explained', 'during', 'second', 'reading', ',', 'there', 'is', 'a', 'crisis', 'underlying', 'this', 'directive', 'amendment', '.'], ['as', 'i', 'already', 'explained', 'during', 'second', 'reading', ',', 'there', 'is', 'a', 'crisis', 'underlying', 'this', 'directive', 'amendment', '.'], ['as', 'i', 'already', 'explained', 'during', 'second', 'reading', ',', 'there', 'is', 'a', 'crisis', 'underlying', 'this', 'directive', 'amendment', '.'], ['it', 'is', 'our', 'job', 'to', 'continue', 'to', 'support', 'latvia', 'with', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['as', 'i', 'already', 'explained', 'during', 'second', 'reading', ',', 'there', 'is', 'a', 'crisis', 'underlying', 'this', 'directive', 'amendment', '.'], ['as', 'i', 'already', 'explained', 'during', 'second', 'reading', ',', 'there', 'is', 'a', 'crisis', 'underlying', 'this', 'directive', 'amendment', '.'], ['as', 'i', 'already', 'explained', 'during', 'second', 'reading', ',', 'there', 'is', 'a', 'crisis', 'underlying', 'this', 'directive', 'amendment', '.'], ['it', 'is', 'our', 'job', 'to', 'continue', 'to', 'support', 'latvia', 'with', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['as', 'i', 'already', 'explained', 'during', 'second', 'reading', ',', 'there', 'is', 'a', 'crisis', 'underlying', 'this', 'directive', 'amendment', '.'], ['as', 'i', 'already', 'explained', 'during', 'second', 'reading', ',', 'there', 'is', 'a', 'crisis', 'underlying', 'this', 'directive', 'amendment', '.'], ['consumers', 'will', 'lose', 'out', ',', 'employees', 'will', 'lose', 'out', ',', 'europe', 'will', 'lose', 'competitive', 'strength', 'and', 'growth', '.'], ['consumers', 'will', 'lose', 'out', ',', 'employees', 'will', 'lose', 'out', ',', 'europe', 'will', 'lose', 'competitive', 'strength', 'and', 'growth', '.'], ['consumers', 'will', 'lose', 'out', ',', 'employees', 'will', 'lose', 'out', ',', 'europe', 'will', 'lose', 'competitive', 'strength', 'and', 'growth', '.'], ['consumers', 'will', 'lose', 'out', ',', 'employees', 'will', 'lose', 'out', ',', 'europe', 'will', 'lose', 'competitive', 'strength', 'and', 'growth', '.'], ['consumers', 'will', 'lose', 'out', ',', 'employees', 'will', 'lose', 'out', ',', 'europe', 'will', 'lose', 'competitive', 'strength', 'and', 'growth', '.'], ['consumers', 'will', 'lose', 'out', ',', 'employees', 'will', 'lose', 'out', ',', 'europe', 'will', 'lose', 'competitive', 'strength', 'and', 'growth', '.'], ['consumers', 'will', 'lose', 'out', ',', 'employees', 'will', 'lose', 'out', ',', 'europe', 'will', 'lose', 'competitive', 'strength', 'and', 'growth', '.'], ['consumers', 'will', 'lose', 'out', ',', 'employees', 'will', 'lose', 'out', ',', 'europe', 'will', 'lose', 'competitive', 'strength', 'and', 'growth', '.'], ['consumers', 'will', 'lose', 'out', ',', 'employees', 'will', 'lose', 'out', ',', 'europe', 'will', 'lose', 'competitive', 'strength', 'and', 'growth', '.'], ['consumers', 'will', 'lose', 'out', ',', 'employees', 'will', 'lose', 'out', ',', 'europe', 'will', 'lose', 'competitive', 'strength', 'and', 'growth', '.'], ['consumers', 'will', 'lose', 'out', ',', 'employees', 'will', 'lose', 'out', ',', 'europe', 'will', 'lose', 'competitive', 'strength', 'and', 'growth', '.'], ['consumers', 'will', 'lose', 'out', ',', 'employees', 'will', 'lose', 'out', ',', 'europe', 'will', 'lose', 'competitive', 'strength', 'and', 'growth', '.'], ['consumers', 'will', 'lose', 'out', ',', 'employees', 'will', 'lose', 'out', ',', 'europe', 'will', 'lose', 'competitive', 'strength', 'and', 'growth', '.'], ['consumers', 'will', 'lose', 'out', ',', 'employees', 'will', 'lose', 'out', ',', 'europe', 'will', 'lose', 'competitive', 'strength', 'and', 'growth', '.'], ['consumers', 'will', 'lose', 'out', ',', 'employees', 'will', 'lose', 'out', ',', 'europe', 'will', 'lose', 'competitive', 'strength', 'and', 'growth', '.'], ['consumers', 'will', 'lose', 'out', ',', 'employees', 'will', 'lose', 'out', ',', 'europe', 'will', 'lose', 'competitive', 'strength', 'and', 'growth', '.'], ['consumers', 'will', 'lose', 'out', ',', 'employees', 'will', 'lose', 'out', ',', 'europe', 'will', 'lose', 'competitive', 'strength', 'and', 'growth', '.'], ['unanimous', 'decisions', ',', 'and', 'hence', 'an', 'inherent', 'incapacity', 'to', 'act', ',', 'remain', 'largely', 'the', 'norm', 'in', 'the', 'council', '.'], ['unanimous', 'decisions', ',', 'and', 'hence', 'an', 'inherent', 'incapacity', 'to', 'act', ',', 'remain', 'largely', 'the', 'norm', 'in', 'the', 'council', '.'], ['unanimous', 'decisions', ',', 'and', 'hence', 'an', 'inherent', 'incapacity', 'to', 'act', ',', 'remain', 'largely', 'the', 'norm', 'in', 'the', 'council', '.'], ['unanimous', 'decisions', ',', 'and', 'hence', 'an', 'inherent', 'incapacity', 'to', 'act', ',', 'remain', 'largely', 'the', 'norm', 'in', 'the', 'council', '.'], ['unanimous', 'decisions', ',', 'and', 'hence', 'an', 'inherent', 'incapacity', 'to', 'act', ',', 'remain', 'largely', 'the', 'norm', 'in', 'the', 'council', '.'], ['unanimous', 'decisions', ',', 'and', 'hence', 'an', 'inherent', 'incapacity', 'to', 'act', ',', 'remain', 'largely', 'the', 'norm', 'in', 'the', 'council', '.'], ['unanimous', 'decisions', ',', 'and', 'hence', 'an', 'inherent', 'incapacity', 'to', 'act', ',', 'remain', 'largely', 'the', 'norm', 'in', 'the', 'council', '.'], ['unanimous', 'decisions', ',', 'and', 'hence', 'an', 'inherent', 'incapacity', 'to', 'act', ',', 'remain', 'largely', 'the', 'norm', 'in', 'the', 'council', '.'], ['unanimous', 'decisions', ',', 'and', 'hence', 'an', 'inherent', 'incapacity', 'to', 'act', ',', 'remain', 'largely', 'the', 'norm', 'in', 'the', 'council', '.'], ['unanimous', 'decisions', ',', 'and', 'hence', 'an', 'inherent', 'incapacity', 'to', 'act', ',', 'remain', 'largely', 'the', 'norm', 'in', 'the', 'council', '.'], ['unanimous', 'decisions', ',', 'and', 'hence', 'an', 'inherent', 'incapacity', 'to', 'act', ',', 'remain', 'largely', 'the', 'norm', 'in', 'the', 'council', '.'], ['unanimous', 'decisions', ',', 'and', 'hence', 'an', 'inherent', 'incapacity', 'to', 'act', ',', 'remain', 'largely', 'the', 'norm', 'in', 'the', 'council', '.'], ['unanimous', 'decisions', ',', 'and', 'hence', 'an', 'inherent', 'incapacity', 'to', 'act', ',', 'remain', 'largely', 'the', 'norm', 'in', 'the', 'council', '.'], ['unanimous', 'decisions', ',', 'and', 'hence', 'an', 'inherent', 'incapacity', 'to', 'act', ',', 'remain', 'largely', 'the', 'norm', 'in', 'the', 'council', '.'], ['unanimous', 'decisions', ',', 'and', 'hence', 'an', 'inherent', 'incapacity', 'to', 'act', ',', 'remain', 'largely', 'the', 'norm', 'in', 'the', 'council', '.'], ['unanimous', 'decisions', ',', 'and', 'hence', 'an', 'inherent', 'incapacity', 'to', 'act', ',', 'remain', 'largely', 'the', 'norm', 'in', 'the', 'council', '.'], ['unanimous', 'decisions', ',', 'and', 'hence', 'an', 'inherent', 'incapacity', 'to', 'act', ',', 'remain', 'largely', 'the', 'norm', 'in', 'the', 'council', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'four', 'words', ':', 'citizens', "'", 'power', 'over', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'four', 'words', ':', 'citizens', "'", 'power', 'over', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'four', 'words', ':', 'citizens', "'", 'power', 'over', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'four', 'words', ':', 'citizens', "'", 'power', 'over', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'four', 'words', ':', 'citizens', "'", 'power', 'over', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'four', 'words', ':', 'citizens', "'", 'power', 'over', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'four', 'words', ':', 'citizens', "'", 'power', 'over', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'four', 'words', ':', 'citizens', "'", 'power', 'over', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'four', 'words', ':', 'citizens', "'", 'power', 'over', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'four', 'words', ':', 'citizens', "'", 'power', 'over', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'four', 'words', ':', 'citizens', "'", 'power', 'over', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'four', 'words', ':', 'citizens', "'", 'power', 'over', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'four', 'words', ':', 'citizens', "'", 'power', 'over', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'four', 'words', ':', 'citizens', "'", 'power', 'over', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'four', 'words', ':', 'citizens', "'", 'power', 'over', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'four', 'words', ':', 'citizens', "'", 'power', 'over', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'four', 'words', ':', 'citizens', "'", 'power', 'over', 'bureaucracy', '.'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia'], ['tunisia', 'was'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'commissioner', '.'], ['thank', 'you', ',', 'mister', 'commissioner', '.'], ['thank-you', ',', 'mister', 'the', 'commissioner', '.'], ['thank', 'you', ',', 'mr', 'commissioner', '.'], ['thank', 'you', ',', 'mr', 'commissioner', '.'], ['thank', 'you', ',', 'mister', 'the', 'commissioner', '.'], ['thank', 'you', ',', 'mister', 'the', 'police', 'chief', '.'], ['maij-weggen', 'report', '(', '5', '/', '2000', ')'], ['relation', 'maij-weggen', '(', 'a5-0323', '/', '2000', ')'], ['(', 'the', 'parliament', 'adopts', 'legislative', 'resolution', ')'], ['report', 'maij-weggen', '(', 'a5-0323', '/', '2000', ')'], ['report', 'maij-weggen', '(', 'a5-0323', '/', '2000', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['(', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['(', 'the', 'parliament', 'adopts', 'the', 'legislative', 'resolution', ')'], ['report', 'maij-weggen', 'in', 'particular', 'a5-0323', '--', '2000', ')'], ['mrs', 'maij-weggen', 'report', '(', 'a5-0323', '/', '2000', ')'], ['(', 'the', 'parliament', 'adopts', 'the', 'legislative', 'resolution', ')'], ['(', 'the', 'parliament', 'adopts', 'the', 'legislative', 'resolution', ')'], ['the', 'report', 'maij-weggen', ',', 'a5-0323', '/', '2000', '--'], [',', 'the', 'parliament', 'adopted', 'the', 'legislative', 'resolution', '--'], ['in', 'particular', 'parliament', 'adopted', 'the', 'legislative', 'resolution', ')'], ['mrs', 'maij-weggen', "'s", 'report', '(', '5', '/', '2000', ')'], ['report', '/', 'ratio', 'maij-weggen', '(', 'a5-0323', '/', '2000', ')'], ['report', 'horsebox', 'orden', '(', 'a5-0241', '/', '2000', ')'], ['relation', 'horse-trailer', 'orden', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', '5', '/', '2000', ')'], ['van', 'orden', 'report', '(', '5', '/', '2000', ')'], ['report', 'van', 'orden', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['report', 'van', 'orden', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['report', 'van', 'orden', '(', 'a5-0241', '/', '2000', ')'], ['report', 'van', 'orden', ',', 'a5-0241', '/', '2000', '--'], ['van', 'orden', 'report', 'in', 'particular', 'a5-0241', '--', '2000', ')'], ['mr', 'van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')'], ['report', '/', 'ratio', 'van', 'orden', '(', 'a5-0241', '/', '2000', ')'], ['all', 'must', 'be', 'balanced', '.'], ['all', 'must', 'be', 'balanced', '.'], ['the', 'whole', 'must', 'be', 'balanced', '.'], ['group', 'must', 'be', 'in', 'equilibrium', '.'], ['all', 'must', 'be', 'in', 'balance', '.'], ['the', 'together', 'must', 'be', 'in', 'balance', '.'], ['the', 'body', 'must', 'be', 'in', 'balance', '.'], ['the', 'unity', 'must', 'be', 'in', 'balance', '.'], ['the', 'whole', 'must', 'be', 'in', 'balance', '.'], ['the', 'whole', 'must', 'be', 'in', 'balance', '.'], ['the', 'whole', 'must', 'be', 'in', 'balance', '.'], ['the', 'whole', 'must', 'be', 'in', 'balance', '.'], ['the', 'whole', 'must', 'be', 'in', 'balance', '.'], ['the', 'whole', 'must', 'be', 'in', 'balance', '.'], ['the', 'whole', 'must', 'be', 'in', 'balance', '.'], ['the', 'whole', 'must', 'be', 'in', 'balance', '.'], ['the', 'unit', 'must', 'be', 'in', 'balance', '.'], ['fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointment', '.'], ['we', 'might', 'then', 'have', 'to', 'avoid', 'a', 'disaster', '.'], ['we', 'might', 'then', 'have', 'to', 'avoid', 'a', 'disaster', '.'], ['we', 'might', 'then', 'have', 'to', 'avoid', 'a', 'disaster', '.'], ['we', 'would', 'perhaps', 'then', 'able', 'prevent', 'a', 'disaster', '.'], ['we', 'would', 'perhaps', 'then', 'could', 'prevent', 'a', 'disaster', '.'], ['the', 'fishermen', 'are', 'unemployed', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishers', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['perhaps', 'then', 'we', 'would', 'have', 'avoided', 'a', 'catastrophe', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['we', 'maybe', 'then', 'could', 'have', 'avoided', 'a', 'catastrophe', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'inactive', ',', 'are', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['we', 'could', 'perhaps', 'then', 'have', 'avoided', 'a', 'catastrophe', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'idle', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['the', 'fishermen', 'are', 'inactive', ',', 'tired', 'and', 'disappointed', '.'], ['perhaps', 'we', 'would', 'then', 'have', 'to', 'avoid', 'a', 'disaster', '.'], ['we', 'might', 'have', 'been', 'able', 'to', 'prevent', 'a', 'disaster', '.'], ['we', 'could', 'perhaps', 'have', 'been', 'able', 'to', 'prevent', 'a', 'disaster', '.'], ['we', 'would', 'perhaps', 'have', 'been', 'able', 'to', 'prevent', 'a', 'disaster', '.'], ['perhaps', 'we', 'should', 'have', 'been', 'able', 'to', 'prevent', 'a', 'disaster', '.'], ['then', 'we', 'might', 'have', 'been', 'able', 'to', 'avoid', 'a', 'disaster', '.'], ['then', 'we', 'might', 'have', 'been', 'able', 'to', 'avoid', 'a', 'disaster', '.'], ['we', 'would', 'maybe', 'then', 'have', 'been', 'able', 'to', 'avoid', 'a', 'disaster', '.'], ['we', 'would', 'perhaps', 'have', 'been', 'able', 'to', 'then', 'avoid', 'a', 'disaster', '.'], ['the', 'vote', 'will', 'take', 'place', 'at', '5.30pm'], ['the', 'vote', 'will', 'place', 'to', '17h30', '.'], ['the', 'vote', 'will', 'be', 'to', '17h30', '.'], ['this', 'regulation', 'opens', 'the', 'door', 'to', 'arbitrariness', '.'], ['the', 'vote', 'will', 'take', 'place', 'with', '17h30', '.'], ['the', 'vote', 'will', 'take', 'place', 'at', '17h30', '.'], ['the', 'vote', 'will', 'take', 'place', 'at', '17h30', '.'], ['the', 'vote', 'will', 'take', 'place', 'at', '17h30', '.'], ['norms', 'are', 'barely', 'comparable', 'and', 'even', 'less', 'alienable', '.'], ['this', 'regulation', 'opens', 'the', 'door', 'to', 'the', 'arbitrary', '.'], ['the', 'vote', 'will', 'take', 'place', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'at', '5.30', 'p.m', '.'], ['the', 'vote', 'will', 'take', 'place', 'at', '5.30', 'p.m', '.'], ['this', 'regulation', 'opens', 'the', 'door', 'to', 'the', 'arbitrary', '.'], ['vote', 'will', 'take', 'place', 'at', '17', 'h', '30', '.'], ['the', 'rules', 'are', 'comparable', 'only', 'good', ',', 'let', 'alone', '.'], ['the', 'norms', 'are', 'scarcely', 'comparable', 'and', 'again', 'less', 'transférables', '.'], ['the', 'norms', 'are', 'hardly', 'comparable', 'and', 'still', 'less', 'transferable', '.'], ['the', 'standards', 'are', 'hardly', 'comparable', 'and', 'even', 'less', 'transferable', '.'], ['the', 'standards', 'are', 'hardly', 'comparable', 'and', 'even', 'less', 'transferable', '.'], ['the', 'standards', 'are', 'hardly', 'comparable', 'and', 'even', 'less', 'transferable', '.'], ['the', 'standards', 'are', 'hardly', 'comparable', 'and', 'even', 'less', 'transferable', '.'], ['the', 'standards', 'are', 'hardly', 'comparable', 'and', 'even', 'less', 'transferable', '.'], ['the', 'standards', 'are', 'hardly', 'comparable', 'and', 'even', 'less', 'transferable', '.'], ['the', 'standards', 'are', 'hardly', 'comparable', 'and', 'even', 'less', 'transferable', '.'], ['the', 'standards', 'are', 'hardly', 'comparable', 'and', 'still', 'less', 'transferable', '.'], ['the', 'standards', 'are', 'hardly', 'comparable', 'and', 'still', 'less', 'transferable', '.'], ['the', 'standards', 'are', 'hardly', 'comparable', 'and', 'still', 'less', 'transferable', '.'], ['the', 'standards', 'are', 'hardly', 'comparable', 'and', 'still', 'less', 'transferable', '.'], ['the', 'standards', 'are', 'hardly', 'comparable', 'and', 'still', 'less', 'transferable', '.'], ['the', 'vote', 'will', 'take', 'place', 'on', '17', ':', '30', '.'], ['this', 'point', 'of', 'the', 'regulation', 'opens', 'the', 'door', 'to', 'arbitrariness', '.'], ['this', 'point', 'of', 'the', 'regulation', 'opens', 'the', 'door', 'to', 'arbitrariness', '.'], ['the', 'standards', 'are', 'to', 'death', 'similar', 'and', 'even', 'less', 'transferable', '.'], ['this', 'point', 'of', 'procedure', 'opens', 'the', 'door', 'to', 'the', 'arbitrary', '.'], ['this', 'regulation', 'point', 'opens', 'the', 'door', 'to', 'the', 'arbitrary', 'one', '.'], ['this', 'point', 'of', 'procedure', 'opens', 'the', 'door', 'to', 'the', 'arbitrary', '.'], ['this', 'point', 'in', 'the', 'regulation', 'opens', 'the', 'door', 'to', 'the', 'arbitrary', '.'], ['this', 'point', 'of', 'the', 'regulation', 'opens', 'the', 'door', 'to', 'the', 'arbitrary', '.'], ['this', 'point', 'of', 'the', 'regulation', 'opens', 'the', 'door', 'to', 'the', 'arbitrary', '.'], ['this', 'point', 'of', 'the', 'regulation', 'opens', 'the', 'door', 'to', 'the', 'arbitrary', '.'], ['this', 'point', 'of', 'the', 'regulations', 'opens', 'the', 'door', 'to', 'arbitrary', 'power', '.'], ['this', 'paves', 'the', 'way', 'for', 'the', 'of', 'the', 'rules', 'of', 'procedure', 'here', '.'], ['this', 'point', 'of', 'the', 'payment', 'opens', 'the', 'door', 'with', 'the', 'arbitrary', 'one', '.'], ['this', 'point', 'of', 'the', 'regulations', 'opens', 'the', 'door', 'to', 'the', 'arbitrary', 'one', '.'], ['this', 'point', 'of', 'the', 'rules', 'of', 'procedure', 'opens', 'the', 'door', 'to', 'the', 'arbitrary', '.'], ['other', 'separate', 'unfortunately', 'a', 'accumulated', 'resentment', '.'], ['other', 'separate', 'unfortunately', 'because', 'of', 'resentment', 'accumulated', '.'], ['other', 'diverge', 'unfortunately', 'owing', 'to', 'an', 'accumulated', 'vengeance', '.'], ['other', 'divide', 'unfortunately', 'because', 'of', 'a', 'grudge', 'accumulated', '.'], ['other', 'divide', 'unfortunately', 'because', 'of', 'a', 'grudge', 'accumulated', '.'], ['others', 'separate', 'unfortunately', 'because', 'of', 'an', 'accumulated', 'resentment', '.'], ['other', 'divide', 'unfortunately', 'due', 'to', 'a', 'accumulated', 'ill-will', '.'], ['others', 'unfortunately', 'diverge', 'in', 'response', 'to', 'rancune', 'gained', '.'], ['other', 'separate', 'unfortunately', 'because', 'of', 'an', 'accumulated', 'resentment', '.'], ['other', 'separate', 'unfortunately', 'because', 'of', 'a', 'grudge', 'accumulated', '.'], ['other', 'separate', 'unfortunately', 'because', 'of', 'a', 'grudge', 'accumulated', '.'], ['others', 'are', 'parting', 'unfortunately', 'because', 'of', 'a', 'rancune', 'gained', '.'], ['others', 'separate', 'themselves', 'unfortunately', 'because', 'of', 'an', 'amassed', 'grudge', '.'], ['some', 'separate', 'themselves', 'unfortunately', 'in', 'view', 'of', 'a', 'grudge', 'gained', '.'], ['d', "'", 'other', 'separate', 'themselves', 'unfortunately', 'because', 'of', 'an', 'accumulated', 'grudge', '.'], ['d', "'", 'others', 'part', 'unfortunately', 'in', 'reason', 'd', "'", 'a', 'collected', 'resentment', '.'], ['on', 'others', 'who', 'are', 'unfortunately', 'in', 'the', 'reason', 'of', 'a', 'vengeance', 'gained', '.'], ['amendment', '7', 'changes', 'in', 'references', 'to', 'paragraphs', '.'], ['amendment', 'no', '7', 'proposes', 'changes', 'to', 'certain', 'paragraphs', 'references', '.'], ['amendment', '7', 'offers', 'certain', 'modifications', 'in', 'references', 'to', 'paragraphs', '.'], ['i', 'am', 'part', 'of', 'members', 'who', 'are', 'not', 'properly', 'present', '.'], ['i', 'am', 'part', 'of', 'members', 'who', 'are', 'not', 'properly', 'present', '.'], ['i', 'am', 'part', 'of', 'parliamentarians', 'who', 'are', 'rather', 'present', 'honourably', '.'], ['i', 'belong', 'to', 'the', 'members', 'who', 'are', 'present', 'rather', 'honourably', '.'], ['i', 'am', 'one', 'of', 'the', 'members', 'who', 'are', 'rather', 'present', '.'], ['amendment', '7', 'proposes', 'certain', 'modifications', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'are', 'quite', 'properly', 'here', '.'], ['i', 'am', 'one', 'of', 'the', 'members', 'who', 'are', 'quite', 'honourably', 'here', '.'], ['i', 'am', 'one', 'of', 'the', 'members', 'who', 'are', 'quite', 'honourably', 'here', '.'], ['i', 'am', 'one', 'of', 'the', 'members', 'who', 'are', 'quite', 'honourably', 'here', '.'], ['i', 'am', 'one', 'of', 'the', 'members', 'who', 'are', 'quite', 'honourably', 'here', '.'], ['i', 'am', 'one', 'of', 'the', 'members', 'who', 'are', 'quite', 'honourably', 'here', '.'], ['i', 'am', 'one', 'of', 'those', 'members', 'who', 'are', 'rather', 'honourably', 'here', '.'], ['i', 'am', 'one', 'of', 'the', 'members', 'who', 'are', 'rather', 'honourably', 'here', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['amendment', '7', 'proposes', 'certain', 'modifications', 'in', 'the', 'references', 'to', 'the', 'paragraphs', '.'], ['amendment', '7', 'proposes', 'certain', 'modifications', 'in', 'the', 'references', 'to', 'the', 'paragraphs', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['i', 'am', 'one', 'of', 'the', 'members', 'of', 'parliament', 'who', 'are', 'rather', 'present', '.'], ['i', 'am', 'part', 'of', 'the', 'parliamentary', 'ones', 'that', 'are', 'rather', 'present', 'honorably', '.'], ['the', 'amendment', '7', 'proposes', 'some', 'changes', 'in', 'the', 'references', 'to', 'the', 'paragraphs', '.'], ['the', 'amendment', '7', 'proposes', 'some', 'changes', 'in', 'the', 'references', 'to', 'the', 'paragraphs', '.'], ['amendment', 'no', '7', 'is', 'proposing', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['amendment', 'no', '7', 'is', 'proposing', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['amendment', 'no', '7', 'is', 'proposing', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['the', 'amendment', '7', 'proposes', 'some', 'modifications', 'in', 'the', 'references', 'to', 'the', 'paragraphs', '.'], ['amendment', 'no', '7', 'proposes', 'certain', 'changes', 'in', 'the', 'references', 'to', 'the', 'paragraphs', '.'], ['amendment', 'no', '7', 'is', 'proposing', 'certain', 'changes', 'in', 'the', 'references', 'to', 'paragraphs', '.'], ['i', 'am', 'part', 'of', 'the', 'members', 'of', 'parliament', 'who', 'are', 'rather', 'honorably', 'present', '.'], ['i', 'form', 'part', 'of', 'the', 'members', 'of', 'parliament', 'who', 'are', 'rather', 'honourably', 'present', '.'], ['amendment', 'no', '7', 'is', 'proposing', 'certain', 'amendments', 'in', 'the', 'references', 'to', 'the', 'paragraphs', '.'], ['the', 'european', 'union', 'itself', 'and', 'do', 'so', 'quickly', '.'], ['the', 'european', 'union', 'must', 'be', 'involved', 'and', 'do', 'so', 'quickly', '.'], ['it', 'suits', 'that', 'the', 'european', 'union', 'is', 'implied', 'and', 'that', 'it', 'makesit', 'rapidly', '.'], ['it', 'is', 'that', 'the', 'european', 'union', 'is', 'involved', 'and', 'that', 'it', 'will', 'soon', '.'], ['it', 'is', 'important', 'that', 'the', 'european', 'union', 'itself', 'and', 'to', 'do', 'it', 'quickly', '.'], ['it', 'suits', 'that', 'the', 'european', 'union', 'implies', 'itself', 'and', 'that', 'she', 'do', 'it', 'quickly', '.'], ['it', 'is', 'that', 'the', 'european', 'union', 'is', 'involved', 'and', 'that', 'it', 'do', 'so', 'quickly', '.'], ['it', 'is', 'right', 'that', 'the', 'european', 'union', 'is', 'it', 'entails', ',', 'and', 'do', 'so', 'quickly', '.'], ['it', 'is', 'appropriate', 'that', 'the', 'european', 'union', 'is', 'implied', 'and', 'that', 'it', 'does', 'it', 'quickly', '.'], ['it', 'is', 'that', 'the', 'european', 'union', 'is', 'means', 'and', 'that', 'it', 'be', 'done', 'very', 'soon', '.'], ['it', 'is', 'appropriate', 'that', 'the', 'european', 'union', 'is', 'and', 'what', 'it', 'implies', 'does', 'it', 'quickly', '.'], ['it', 'is', 'appropriate', 'that', 'the', 'european', 'union', 'is', 'involved', 'and', 'that', 'it', 'makes', 'it', 'quickly', '.'], ['it', 'is', 'appropriate', 'that', 'the', 'european', 'union', 'means', 'and', 'that', 'it', 'should', 'do', 'so', 'quickly', '.'], ['it', 'is', 'appropriate', 'that', 'the', 'european', 'union', 'means', 'and', 'that', 'it', 'should', 'do', 'so', 'quickly', '.'], ['it', 'is', 'important', 'that', 'the', 'european', 'union', 'to', 'be', 'involved', 'and', 'to', 'do', 'it', 'quickly', '.'], ['it', 'is', 'necessary', 'that', 'the', 'european', 'union', 'gets', 'involved', 'and', 'that', 'it', 'makes', 'it', 'fast', '.'], ['it', 'is', 'right', 'that', 'the', 'european', 'union', 'is', 'involved', ',', 'and', 'for', 'this', 'to', 'be', 'done', 'quickly', '.'], ['we', 'ourselves', 'often', 'represent', 'european', 'citizens', '.'], ['we', 'are', 'often', 'here', 'to', 'represent', 'the', 'european', 'citizens', '.'], ['often', 'we', 'are', 'proud', 'of', 'representing', 'europe', "'s", 'citizens', '.'], ['we', 'are', 'often', 'here', 'to', 'represent', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'take', 'pride', 'often', 'here', 'to', 'represent', 'the', 'european', 'citizens', '.'], ['we', 'enorgueillissons', 'ourselves', 'often', 'here', 'to', 'represent', 'the', 'european', 'citizens', '.'], ['we', 'we', 'pride', 'often', 'here', 'of', 'representing', 'the', 'european', 'citizens', '.'], ['we', 'are', 'proud', 'often', 'of', 'representing', 'the', 'european', 'citizens', 'here', '.'], ['we', 'often', 'enorgueillissons', 'ourselves', 'here', 'to', 'represent', 'the', 'european', 'citizens', '.'], ['we', 'often', 'take', 'pride', 'here', 'in', 'representing', 'the', 'european', 'citizens', '.'], ['we', 'are', 'proud', 'often', 'here', 'to', 'represent', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'take', 'pride', 'here', 'to', 'represent', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'take', 'pride', 'here', 'to', 'represent', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'take', 'pride', 'here', 'to', 'represent', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'often', 'take', 'pride', 'here', 'to', 'represent', 'the', 'citizens', 'of', 'europe', '.'], ['we', 'we', 'take', 'pride', 'often', 'here', 'of', 'representing', 'the', 'european', 'citizens', '.'], ['often', 'we', 'pride', 'ourselves', 'here', 'to', 'represent', 'the', 'people', 'of', 'europe', '.'], ['mr', 'president', ',', 'i', 'want', 'to', 'take', 'the', 'protection', 'of', 'workers', '.'], ['mr', 'president', ',', 'i', 'would', 'like', 'to', 'defend', 'workers', 'in', 'the', 'sector', '.'], ['i', 'would', 'remind', 'you', 'that', 'our', 'allies', ',', 'devotees', 'of', 'this', 'tax', '.'], ['mr', 'president', ',', 'i', 'hold', 'to', 'the', 'defence', 'of', 'workers', 'in', 'this', 'sector', '.'], ['mr', 'president', ',', 'i', 'want', 'to', 'stand', 'up', 'for', 'workers', 'of', 'this', 'area', '.'], ['i', 'hold', 'you', 'recall', 'that', 'our', 'allies', ',', 'there', 'are', 'enthusiastic', 'about', 'this', 'tax', '.'], ['mr', 'president', ',', 'i', 'wish', 'to', 'take', 'the', 'protection', 'of', 'workers', 'of', 'this', 'sector', '.'], ['mr', 'president', ',', 'i', 'wish', 'to', 'take', 'the', 'protection', 'of', 'workers', 'of', 'the', 'sector', '.'], ['mr', 'president', ',', 'i', 'would', 'like', 'to', 'make', 'the', 'defence', 'of', 'workers', 'in', 'this', 'sector', '.'], ['mr', 'president', ',', 'i', 'would', 'like', 'to', 'take', 'the', 'protection', 'of', 'workers', 'in', 'this', 'sector', '.'], ['mr', 'president', ',', 'i', 'should', 'like', 'to', 'take', 'the', 'defence', 'of', 'workers', 'in', 'this', 'sector', '.'], ['mr', 'president', ',', 'i', 'would', 'like', 'to', 'take', 'the', 'defence', 'of', 'workers', 'in', 'this', 'sector', '.'], ['mr', 'president', ',', 'i', 'would', 'like', 'to', 'take', 'the', 'defence', 'of', 'workers', 'in', 'this', 'sector', '.'], ['mr', 'president', ',', 'i', 'would', 'like', 'to', 'take', 'the', 'defence', 'of', 'workers', 'in', 'this', 'sector', '.'], ['mr', 'president', ',', 'i', 'would', 'like', 'to', 'take', 'the', 'defence', 'of', 'workers', 'in', 'this', 'sector', '.'], ['mr', 'president', ',', 'i', 'would', 'like', 'to', 'take', 'the', 'defence', 'of', 'workers', 'in', 'this', 'sector', '.'], ['i', 'insist', 'on', 'reminding', 'you', 'that', 'among', 'our', 'allies', ',', 'there', 'are', 'devotees', 'of', 'this', 'tax', '.'], ['i', 'want', 'to', 'remind', 'you', 'that', 'of', 'our', 'allies', ',', 'it', 'is', 'of', 'major', 'of', 'tax', '.'], ['mister', 'the', 'president', ',', 'i', 'am', 'anxious', 'to', 'take', 'the', 'defense', 'of', 'the', 'workers', 'of', 'this', 'sector', '.'], ['mister', 'preside', 'over', 'it', ',', 'i', 'insist', 'on', 'taking', 'the', 'defense', 'of', 'the', 'workers', 'of', 'this', 'sector', '.'], ['i', 'would', 'like', 'to', 'remind', 'you', 'that', 'among', 'our', 'allies', ',', 'there', 'are', 'strong', 'of', 'this', 'tax', '.'], ['i', 'am', 'anxious', 'to', 'recall', 'you', 'that', 'among', 'our', 'allies', ',', 'there', 'is', 'fervent', 'of', 'this', 'tax', '.'], ['i', 'would', 'like', 'to', 'remind', 'you', 'that', 'one', 'of', 'our', 'allies', ',', 'there', 'are', 'strong', 'this', 'tax', '.'], ['i', 'would', 'remind', 'you', 'that', 'one', 'of', 'our', 'allies', ',', 'it', 'is', 'very', 'much', 'of', 'this', 'tax', '.'], ['i', 'want', 'to', 'remind', 'you', 'that', 'of', 'our', 'allies', ',', 'it', 'is', 'of', 'much', 'of', 'this', 'tax', '.'], ['i', 'would', 'like', 'to', 'remind', 'you', 'that', 'our', 'allies', ',', 'there', 'are', 'ardent', 'supporters', 'of', 'this', 'tax', '.'], ['i', 'would', 'like', 'to', 'remind', 'you', 'that', 'among', 'our', 'allies', ',', 'there', 'are', 'enthusiasts', 'of', 'this', 'tax', '.'], ['i', 'want', 'to', 'say', 'to', 'you', 'that', 'among', 'our', 'allies', ',', 'there', 'are', 'fervent', 'of', 'this', 'tax', '.'], ['mr.', 'president', ',', 'i', 'make', 'a', 'point', 'of', 'taking', 'the', 'defense', 'of', 'the', 'workers', 'of', 'this', 'sector', '.'], ['i', 'would', 'like', 'to', 'remind', 'you', 'that', 'among', 'our', 'allies', ',', 'there', 'are', 'ardent', 'supporters', 'of', 'this', 'tax', '.'], ['i', 'would', 'like', 'to', 'remind', 'you', 'that', 'among', 'our', 'allies', ',', 'there', 'are', 'ardent', 'supporters', 'of', 'this', 'tax', '.'], ['i', 'would', 'like', 'to', 'remind', 'you', 'that', 'among', 'our', 'allies', ',', 'there', 'are', 'ardent', 'supporters', 'of', 'this', 'tax', '.'], ['i', 'make', 'a', 'point', 'of', 'recalling', 'you', 'that', 'among', 'our', 'allies', ',', 'there', 'are', 'enthusiasts', 'of', 'this', 'tax', '.'], ['i', 'would', 'like', 'to', 'remind', 'you', 'that', 'one', 'of', 'our', 'allies', ',', 'there', 'are', 'ardent', 'supporters', 'of', 'this', 'tax', '.'], ['specific', 'aid', ',', 'support', 'or', 'market', 'at', 'grass', 'is', 'essential', '.'], ['the', 'occasional', 'aid', 'as', 'the', 'market', 'or', 'the', 'grass', 'is', 'essential', '.'], ['specific', 'aid', 'as', 'the', 'market', 'or', 'the', 'premium', 'on', 'grass', 'are', 'essential', '.'], ['specific', 'aid', 'as', 'the', 'market', 'or', 'the', 'premium', 'on', 'grass', 'are', 'essential', '.'], ['there', 'was', 'not', 'a', 'majority', 'voting', 'in', 'parliament', 'to', 'go', 'back', 'to', 'article', '272', '.'], ['there', 'was', 'no', 'qualified', 'majority', 'in', 'this', 'parliament', 'to', 'go', 'back', 'to', 'article', '272', '.'], ['there', 'was', 'no', 'qualified', 'majority', 'in', 'this', 'parliament', 'to', 'go', 'back', 'to', 'article', '272', '.'], ['there', 'was', 'no', 'qualified', 'majority', 'in', 'this', 'parliament', 'to', 'go', 'back', 'to', 'article', '272', '.'], ['there', 'was', 'no', 'qualified', 'majority', 'in', 'this', 'parliament', 'to', 'go', 'back', 'to', 'article', '272', '.'], ['there', 'was', 'no', 'qualified', 'majority', 'in', 'this', 'parliament', 'to', 'go', 'back', 'to', 'article', '272', '.'], ['there', 'was', 'no', 'qualified', 'majority', 'in', 'this', 'parliament', 'to', 'go', 'back', 'to', 'article', '272', '.'], ['there', 'was', 'no', 'qualified', 'majority', 'in', 'this', 'parliament', 'to', 'go', 'back', 'to', 'article', '272', '.'], ['there', 'was', 'no', 'qualified', 'majority', 'in', 'this', 'parliament', 'to', 'go', 'back', 'to', 'article', '272', '.'], ['punctual', 'assistance', 'as', 'the', 'support', 'of', 'the', 'market', 'or', 'the', 'premium', 'to', 'grass', 'is', 'essential', '.'], ['specific', 'aid', 'as', 'the', 'support', 'of', 'the', 'market', 'or', 'the', 'premium', 'to', 'grass', 'is', 'essential', '.'], ['specific', 'assistances', 'like', 'the', 'support', 'of', 'the', 'market', 'or', 'the', 'premium', 'with', 'grass', 'are', 'essential', '.'], ['punctual', 'helps', 'as', 'the', 'support', 'of', 'the', 'market', 'or', 'bonus', 'in', 'the', 'grass', 'are', 'necessary', '.'], ['specific', 'aid', 'as', 'the', 'support', 'of', 'the', 'market', 'or', 'the', 'premium', 'on', 'grass', 'are', 'essential', '.'], ['specific', 'aid', 'as', 'the', 'support', 'of', 'the', 'market', 'or', 'the', 'premium', 'on', 'grass', 'are', 'essential', '.'], ['it', 'did', 'not', 'exist', 'either', 'qualified', 'majority', 'in', 'this', 'parliament', 'to', 'return', 'to', 'article', '272', '.'], ['punctual', 'help', 'as', 'the', 'support', 'of', 'the', 'market', 'or', 'the', 'bonus', 'at', 'the', 'grass', 'is', 'essential', '.'], ['a', 'specific', 'aid', 'as', 'the', 'support', 'of', 'the', 'market', 'or', 'the', 'premium', 'on', 'grass', 'are', 'essential', '.'], ['the', 'specific', 'aid', 'as', 'the', 'support', 'of', 'the', 'market', 'or', 'the', 'premium', 'on', 'grass', 'are', 'essential', '.'], ['the', 'occasional', 'aid', 'as', 'the', 'support', 'of', 'the', 'market', 'or', 'the', 'premium', 'to', 'grass', 'is', 'essential', '.'], ['support', 'specific', 'as', 'the', 'support', 'of', 'the', 'market', 'or', 'the', 'premium', 'to', 'the', 'grass', 'is', 'necessary', '.'], ['there', 'was', 'not', 'majority', 'qualified', 'within', 'this', 'parliament', 'to', 'bring', 'back', 'to', 'the', 'article', '272', 'either', '.'], ['there', 'did', 'not', 'exist', 'either', 'of', 'majority', 'qualified', 'within', 'this', 'parliament', 'to', 'return', 'in', 'article', '272', '.'], ['there', 'was', 'not', 'either', 'qualified', 'majority', 'within', 'this', 'parliament', 'to', 'come', 'back', 'to', 'the', 'article', '272', '.'], ['there', 'was', 'no', 'more', 'qualified', 'majority', 'voting', 'in', 'this', 'parliament', 'to', 'come', 'back', 'to', 'article', '272', '.'], ['the', 'specific', 'aid', 'as', 'the', 'support', 'of', 'the', 'market', 'or', 'the', 'premium', 'to', 'the', 'grass', 'is', 'necessary', '.'], ['the', 'specific', 'aid', 'as', 'the', 'support', 'of', 'the', 'market', 'or', 'the', 'premium', 'to', 'the', 'grass', 'are', 'essential', '.'], ['it', 'did', 'not', 'not', 'more', 'of', 'the', 'qualified', 'majority', 'in', 'this', 'parliament', 'to', 'return', 'to', 'the', 'article', '272', '.'], ['it', 'did', 'not', 'exist', 'not', 'more', 'qualified', 'majority', 'within', 'this', 'parliament', 'to', 'come', 'back', 'to', 'the', 'item', '272', '.'], ['it', 'does', 'not', 'was', 'not', 'no', 'longer', 'a', 'majority', 'voting', 'in', 'this', 'parliament', 'to', 'return', 'to', 'the', 'article', '272', '.'], ['it', 'has', 'increased', 'the', 'power', 'of', 'the', 'large', 'countries', 'to', 'the', 'detriment', 'of', 'small', '.'], ['that', 'causes', 'to', 'increase', 'the', 'power', 'of', 'the', 'large', 'countries', 'to', 'the', 'detriment', 'of', 'small', '.'], ['that', 'was', 'to', 'increase', 'the', 'power', 'of', 'the', 'large', 'countries', 'to', 'the', 'detriment', 'of', 'small', '.'], ['it', 'has', 'in', 'fact', 'increased', 'the', 'power', 'of', 'large', 'countries', 'to', 'the', 'detriment', 'of', 'small', '.'], ['this', 'is', 'to', 'increase', 'the', 'power', 'of', 'the', 'large', 'countries', 'at', 'the', 'expense', 'of', 'small', '.'], ['has', 'the', 'effect', 'of', 'increasing', 'the', 'power', 'of', 'the', 'large', 'at', 'the', 'expense', 'of', 'smaller', 'countries', '.'], ['this', 'has', 'the', 'effect', 'of', 'increasing', 'the', 'power', 'of', 'large', 'countries', 'at', 'the', 'expense', 'of', 'smaller', '.'], ['it', 'has', 'the', 'effect', 'of', 'augmenting', 'the', 'potency', 'of', 'the', 'big', 'countries', 'to', 'the', 'detriment', 'of', 'babies', '.'], ['this', 'has', 'the', 'effect', 'of', 'increasing', 'the', 'power', 'of', 'the', 'large', 'countries', 'to', 'the', 'detriment', 'of', 'small', '.'], ['this', 'has', 'the', 'effect', 'of', 'increasing', 'the', 'power', 'of', 'the', 'large', 'countries', 'at', 'the', 'expense', 'of', 'small', '.'], ['this', 'has', 'the', 'effect', 'of', 'increasing', 'the', 'power', 'of', 'the', 'large', 'countries', 'at', 'the', 'expense', 'of', 'small', '.'], ['this', 'has', 'the', 'effect', 'of', 'increasing', 'the', 'power', 'of', 'the', 'large', 'countries', 'at', 'the', 'expense', 'of', 'small', '.'], ['it', 'has', 'to', 'effect', 'of', 'increasing', 'the', 'power', 'of', 'the', 'large', 'countries', 'to', 'the', 'detriment', 'of', 'small', '.'], ['has', 'the', 'effect', 'of', 'increasing', 'the', 'power', 'of', 'the', 'large', 'countries', 'at', 'the', 'expense', 'of', 'the', 'smaller', 'countries', '.'], ['that', 'has', 'for', 'effect', 'to', 'increase', 'the', 'power', 'of', 'the', 'big', 'countries', 'to', 'the', 'detriment', 'of', 'the', 'small', 'ones', '.'], ['that', 'has', 'as', 'effect', 'to', 'increase', 'the', 'power', 'of', 'the', 'great', 'countries', 'to', 'the', 'detriment', 'of', 'the', 'small', 'ones', '.'], ['it', 'has', 'the', 'effect', 'of', 'increasing', 'the', 'power', 'of', 'the', 'large', 'countries', ',', 'to', 'the', 'detriment', 'of', 'the', 'small', 'ones', '.'], ['the', 'leaders', 'are', 'today', 'a', 'new', 'chance', 'and', 'thus', 'let', '.'], ['leaders', 'now', 'have', 'another', 'chance', 'to', 'let', 'them', 'and', 'therefore', 'take', '.'], ['given', 'the', 'leaders', 'today', 'a', 'new', 'chance', 'and', 'thus', 'let', 'the', 'approach', '.'], ['the', 'leaders', 'now', 'have', 'a', 'new', 'chance', 'and', 'let', 'them', 'therefore', 'take', '.'], ['the', 'leaders', 'benefit', 'today', 'of', 'a', 'new', 'chance', 'and', 'thus', 'let', 'the', 'matter', '.'], ['the', 'leaders', 'have', 'today', 'a', 'new', 'chance', 'and', 'let', 'them', 'therefore', 'seize', 'it', '.'], ['the', 'leaders', 'have', 'today', 'a', 'new', 'chance', 'and', 'let', 'them', 'therefore', 'seize', 'it', '.'], ['the', 'leaders', 'have', 'today', 'of', 'a', 'new', 'opportunity', 'and', 'let', 'them', 'therefore', 'seized', '.'], ['the', 'leaders', 'now', 'have', 'another', 'chance', 'to', 'let', 'them', 'and', 'therefore', 'seize', 'it', '.'], ['the', 'leaders', 'now', 'have', 'a', 'new', 'chance', 'and', 'let', 'them', 'therefore', 'seize', 'it', '.'], ['the', 'leaders', 'are', 'today', 'of', 'a', 'new', 'chance', 'and', 'therefore', 'let', 'us', 'take', 'it', '.'], ['the', 'leaders', 'have', 'now', 'of', 'a', 'new', 'chance', 'and', 'let', '--', 'therefore', 'the', 'take', '.'], ['the', 'leaders', 'profit', 'today', 'from', 'a', 'new', 'chance', 'and', 'thus', 'let', 'them', 'seize', 'it', '.'], ['it', 'is', 'our', 'duty', 'to', 'continue', 'supporting', 'latvia', 'on', 'the', 'question', 'of', 'the', 'russian', 'population', '.'], ['the', 'leaders', 'are', 'here', 'today', 'to', 'a', 'new', 'chance', 'and', 'let', 'them', 'therefore', 'the', 'take', '.'], ['as', 'i', 'have', 'already', 'explained', 'in', 'second', 'reading', ',', 'is', 'at', 'the', 'root', 'of', 'this', 'directive', '.'], ['as', 'i', 'have', 'explained', ',', 'a', 'crisis', 'at', 'second', 'reading', 'is', 'the', 'basis', 'of', 'amendment', 'of', 'the', 'directive', '.'], ['the', 'leaders', 'benefit', 'aujourd', "'", 'hui', 'of', 'a', 'new', 'luck', 'and', 'let', "'s", 'let', 'them', 'therefore', 'seize', 'it', '.'], ['the', 'leaders', 'benefit', 'aujourd', "'", 'hui', 'd', "'", 'a', 'new', 'chance', 'and', 'therefore', 'let', 'us', 'let', 'them', 'it', 'grab', '.'], ['the', 'leaders', 'profit', 'aujourd', "'", 'hui', 'of', 'a', 'new', 'luck', 'and', 'let', 'us', 'leave', 'them', 'therefore', 'to', 'seize', 'it', '.'], ['it', 'is', 'of', 'our', 'duty', 'of', 'continuing', 'press', 'latvia', 'on', 'the', 'question', 'of', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['as', 'i', 'already', 'explained', 'in', 'second', 'reading', ',', 'a', 'crisis', 'is', 'at', 'the', 'base', 'of', 'this', 'modification', 'of', 'directive', '.'], ['it', 'is', 'our', 'duty', 'to', 'continue', 'to', 'support', 'latvia', 'on', 'the', 'issue', 'of', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'our', 'duty', 'to', 'continue', 'to', 'support', 'latvia', 'on', 'the', 'issue', 'of', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['as', 'i', 'have', 'already', 'explained', 'at', 'second', 'reading', ',', 'a', 'crisis', 'is', 'on', 'the', 'basis', 'of', 'this', 'amended', 'directive', '.'], ['it', 'is', 'our', 'duty', 'to', 'continue', 'to', 'support', 'latvia', 'on', 'the', 'issue', 'of', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'our', 'duty', 'to', 'continue', 'to', 'support', 'latvia', 'on', 'the', 'issue', 'of', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'our', 'duty', 'to', 'continue', 'to', 'support', 'latvia', 'on', 'the', 'issue', 'of', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'our', 'duty', 'to', 'continue', 'to', 'support', 'latvia', 'on', 'the', 'issue', 'of', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'our', 'duty', 'to', 'continue', 'to', 'support', 'latvia', 'on', 'the', 'question', 'of', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'our', 'duty', 'to', 'continue', 'to', 'press', 'for', 'latvia', 'on', 'the', 'issue', 'of', 'the', 'integration', 'of', 'the', 'russian', 'people', '.'], ['as', 'i', 'have', 'already', 'explained', 'at', 'second', 'reading', ',', 'a', 'crisis', 'is', 'on', 'the', 'basis', 'of', 'this', 'amendment', 'of', 'directive', '.'], ['as', 'i', 'have', 'already', 'explained', 'at', 'second', 'reading', ',', 'a', 'crisis', 'is', 'on', 'the', 'basis', 'of', 'this', 'amendment', 'of', 'directive', '.'], ['it', 'is', 'of', 'our', 'duty', 'to', 'continue', 'supporting', 'the', 'latvia', 'on', 'the', 'question', 'of', 'the', 'incorporation', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'our', 'duty', 'to', 'continue', 'to', 'support', 'the', 'latvia', 'on', 'the', 'question', 'of', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'our', 'duty', 'to', 'continue', 'to', 'support', 'the', 'latvia', 'on', 'the', 'issue', 'of', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['it', 'is', 'of', 'our', 'duty', 'to', 'continue', 'to', 'support', 'latvia', 'on', 'the', 'question', 'of', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['as', 'i', 'have', 'already', 'explained', 'in', 'second', 'reading', ',', 'a', 'crisis', 'is', 'at', 'the', 'basis', 'of', 'this', 'modification', 'of', 'directive', '.'], ['as', 'i', 'have', 'already', 'explained', 'at', 'second', 'reading', ',', 'a', 'crisis', 'is', 'at', 'the', 'root', 'of', 'this', 'amendment', 'to', 'directive', '.'], ['it', 'is', 'our', 'duty', 'of', 'continue', 'to', 'support', 'the', 'latvia', 'on', 'the', 'issue', 'of', 'the', 'integration', 'of', 'the', 'russian', 'people', '.'], ['as', 'i', 'have', 'already', 'explained', 'in', 'second', 'reading', ',', 'a', 'crisis', 'is', 'on', 'the', 'basis', 'of', 'this', 'amendment', 'of', 'directive', '.'], ['as', 'i', 'have', 'already', 'explained', 'at', 'second', 'reading', ',', 'a', 'crisis', 'is', 'at', 'the', 'root', 'of', 'this', 'amendment', 'of', 'directive', '.'], ['as', 'i', 'have', 'already', 'explained', ',', 'at', 'second', 'reading', ',', 'a', 'crisis', 'is', 'at', 'the', 'root', 'of', 'this', 'modification', 'of', 'directive', '.'], ['as', 'i', 'have', 'already', 'said', 'in', 'the', 'second', 'reading', ',', 'a', 'crisis', 'is', 'at', 'the', 'basis', 'of', 'this', 'amendment', 'of', 'directive', '.'], ['it', 'is', 'of', 'our', 'duty', 'to', 'continue', 'to', 'support', 'the', 'latvia', 'on', 'the', 'question', 'of', 'the', 'integration', 'of', 'the', 'russian', 'population', '.'], ['as', 'i', 'already', 'explained', 'it', 'in', 'second', 'reading', ',', 'a', 'crisis', 'is', 'located', 'to', 'the', 'basis', 'of', 'this', 'modification', 'of', 'directive', '.'], ['as', 'i', 'have', 'already', 'explained', 'it', 'in', 'second', 'reading', ',', 'a', 'crisis', 'is', 'at', 'the', 'base', 'of', 'this', 'modification', 'of', 'directive', '.'], ['as', 'i', 'have', 'already', 'explained', 'in', 'second', 'reading', ',', 'a', 'crisis', 'is', 'to', 'the', 'basis', 'of', 'this', 'amendment', 'of', 'the', 'directive', '.'], ['it', 'is', 'of', 'our', 'duty', 'of', 'continue', 'to', 'support', 'the', 'country', 'on', 'the', 'issue', 'of', 'the', 'integration', 'of', 'the', 'russian', 'people', '.'], ['as', 'i', 'have', 'already', 'explained', 'to', 'us', 'at', 'second', 'reading', ',', 'a', 'crisis', 'is', 'at', 'the', 'root', 'of', 'this', 'amendment', 'to', 'directive', '.'], ['as', 'i', 'l', "'", 'have', 'already', 'explained', 'in', 'second', 'reading', ',', 'a', 'crisis', 'is', 'at', 'the', 'root', 'of', 'this', 'modification', 'of', 'directive', '.'], ['consumers', 'will', 'be', 'the', 'losers', ',', 'with', 'workers', 'and', 'competitiveness', 'and', 'growth', 'regress', '.'], ['users', 'are', 'the', 'losers', ',', 'with', 'workers', 'and', 'european', 'competitiveness', 'and', 'innovation', 'régresseront', '.'], ['the', 'users', 'are', 'the', 'losers', ',', 'employees', ',', 'and', 'european', 'competitiveness', 'and', 'growth', 'régresseront', '.'], ['users', 'will', 'be', 'the', 'losers', ',', 'with', 'employees', ',', 'and', 'european', 'competitiveness', 'and', 'growth', 'régresseront', '.'], ['users', 'will', 'be', 'the', 'losers', ',', 'with', 'employees', ',', 'and', 'european', 'competitiveness', 'and', 'growth', 'régresseront', '.'], ['users', 'will', 'be', 'the', 'losers', ',', 'with', 'employees', ',', 'and', 'régresseront', 'european', 'competitiveness', 'and', 'growth', '.'], ['consumers', 'will', 'be', 'the', 'losers', ',', 'with', 'employees', ',', 'and', 'competitiveness', 'and', 'growth', 'régresseront', 'european', '.'], ['the', 'users', 'are', 'the', 'losers', ',', 'with', 'the', 'employees', ',', 'and', 'the', 'competitiveness', 'and', 'growth', 'european', 'régresseront', '.'], ['users', 'will', 'be', 'the', 'losers', ',', 'with', 'the', 'employees', ',', 'and', 'the', 'competitiveness', 'and', 'european', 'growth', 'régresseront', '.'], ['the', 'users', 'will', 'be', 'the', 'losers', ',', 'with', 'employees', ',', 'and', 'european', 'competitiveness', 'and', 'growth', 'will', 'diminish', '.'], ['the', 'users', 'will', 'be', 'the', 'losers', ',', 'with', 'employees', ',', 'and', 'european', 'competitiveness', 'and', 'growth', 'will', 'diminish', '.'], ['the', 'users', 'will', 'be', 'the', 'losers', ',', 'with', 'the', 'employees', ',', 'and', 'the', 'competitiveness', 'and', 'growth', 'regress', '.'], ['the', 'users', 'will', 'be', 'the', 'losers', ',', 'with', 'the', 'employees', ',', 'and', 'european', 'competitiveness', 'and', 'growth', 'will', 'regress', '.'], ['the', 'consumers', 'are', 'the', 'losers', ',', 'with', 'the', 'employees', ',', 'and', 'the', 'competitiveness', 'and', 'the', 'growth', 'european', 'régresseront', '.'], ['the', 'users', 'will', 'be', 'the', 'losers', ',', 'with', 'the', 'employees', ',', 'and', 'european', 'competitiveness', 'and', 'the', 'growth', 'will', 'regress', '.'], ['the', 'users', 'will', 'be', 'the', 'losers', ',', 'with', 'the', 'employees', ',', 'and', 'the', 'european', 'competitiveness', 'and', 'growth', 'will', 'recede', '.'], ['the', 'users', 'will', 'be', 'the', 'losers', ',', 'with', 'the', 'employees', ',', 'and', 'the', 'competitiveness', 'and', 'the', 'european', 'growths', 'will', 'decline', '.'], ['we', 'keep', 'unanimity', 'in', 'council', 'and', 'therefore', 'a', 'latent', 'incapacity', 'to', 'act', '.'], ['we', 'maintain', 'unanimity', 'in', 'council', 'and', 'therefore', 'a', 'latent', 'inability', 'to', 'act', '.'], ['we', 'keep', 'unanimity', 'in', 'the', 'council', 'and', 'thus', 'a', 'latent', 'failure', 'to', 'act', '.'], ['we', 'preserve', 'unanimity', 'in', 'the', 'council', 'and', 'thus', 'a', 'latent', 'incapacity', 'to', 'act', '.'], ['we', 'keep', 'unanimity', 'in', 'the', 'council', 'and', 'therefore', 'a', 'latent', 'inability', 'to', 'act', '.'], ['we', 'keep', 'unanimity', 'in', 'the', 'council', 'and', 'therefore', 'a', 'latent', 'inability', 'to', 'act', '.'], ['we', 'maintain', 'unanimity', 'in', 'the', 'council', 'and', 'therefore', 'a', 'latent', 'inability', 'to', 'act', '.'], ['we', 'maintain', 'unanimity', 'in', 'the', 'council', 'and', 'therefore', 'a', 'latent', 'inability', 'to', 'act', '.'], ['we', 'maintain', 'unanimity', 'in', 'the', 'council', 'and', 'therefore', 'a', 'latent', 'inability', 'to', 'act', '.'], ['we', 'maintain', 'unanimity', 'in', 'the', 'council', 'and', 'therefore', 'a', 'latent', 'inability', 'to', 'act', '.'], ['we', 'keep', 'l', "'", 'unanimity', 'in', 'council', 'and', 'therefore', 'a', 'latent', 'inability', 'to', 'act', '.'], ['we', 'keep', 'unanimity', 'in', 'the', 'council', ',', 'and', 'therefore', 'a', 'latent', 'inability', 'to', 'act', '.'], ['we', 'preserve', 'the', 'unanimity', 'at', 'the', 'council', 'and', 'thus', 'a', 'latent', 'incapacity', 'to', 'act', '.'], ['we', 'preserve', 'the', 'unanimity', 'to', 'the', 'advice', 'and', 'therefore', 'a', 'latent', 'inability', 'to', 'act', '.'], ['we', 'maintain', 'the', 'unanimity', 'in', 'the', 'council', 'and', 'therefore', 'a', 'latent', 'capacity', 'to', 'act', '.'], ['we', 'have', 'the', 'unanimity', 'in', 'the', 'council', 'and', 'therefore', 'a', 'covert', 'inability', 'to', 'act', '.'], ['we', 'preserve', 'the', 'unanimity', 'to', 'the', 'counsel', 'and', 'therefore', 'a', 'latent', 'inability', 'to', 'act', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'as', 'follows', ':', 'the', 'citizens', 'on', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarized', 'briefly', ':', 'the', 'power', 'of', 'the', 'people', 'on', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'report', 'cashman', 'can', 'come', 'down', 'in', 'some', 'words', ':', 'the', 'power', 'of', 'the', 'citizens', 'on', 'officialism', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarised', 'in', 'a', 'few', 'words', ':', 'the', 'power', 'of', 'citizens', 'on', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarized', 'in', 'a', 'few', 'words', ':', 'the', 'power', 'of', 'citizens', 'on', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summarized', 'in', 'two', 'words', ':', 'the', 'power', 'of', 'the', 'people', 'on', 'the', 'tape', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summed', 'up', 'in', 'a', 'few', 'words', ':', 'the', 'power', 'of', 'citizens', 'on', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summed', 'up', 'in', 'a', 'few', 'words', ':', 'the', 'power', 'of', 'citizens', 'on', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summed', 'up', 'in', 'a', 'few', 'words', ':', 'the', 'power', 'of', 'citizens', 'on', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summed', 'up', 'in', 'a', 'few', 'words', ':', 'the', 'power', 'of', 'citizens', 'on', 'bureaucracy', '.'], ['mister', 'the', 'president', ',', 'the', 'report', 'cashman', 'can', 'summarize', 'itself', 'in', 'some', 'words', ':', 'the', 'strength', 'of', 'the', 'citizens', 'on', 'the', 'bureaucracy', '.'], ['mister', 'preside', 'over', 'it', ',', 'the', 'relation', 'cashman', 'can', 'amount', 'in', 'some', 'words', ':', 'the', 'power', 'of', 'the', 'citizens', 'on', 'the', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summed', 'up', 'in', 'a', 'few', 'words', ':', 'the', 'power', 'of', 'citizens', 'on', 'the', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summed', 'up', 'in', 'a', 'few', 'words', ':', 'the', 'power', 'of', 'the', 'citizens', 'on', 'bureaucracy', '.'], ['mr', 'president', ',', 'the', 'report', 'mr', 'cashman', "'s", 'can', 'be', 'summarised', 'in', 'a', 'few', 'words', ':', 'the', 'power', 'of', 'people', 'on', 'the', 'tape', '.'], ['mr', 'president', ',', 'the', 'cashman', 'report', 'can', 'be', 'summed', 'up', 'in', 'a', 'few', 'words', ':', 'the', 'power', 'of', 'the', 'citizens', 'on', 'the', 'bureaucracy', '.'], ['mr.', 'president', ',', 'the', 'cashman', 'report', '/', 'ratio', 'can', 'summarize', 'himself', 'in', 'a', 'few', 'words', ':', 'capacity', 'of', 'the', 'citizens', 'on', 'the', 'bureaucracy', '.'], ['end'], ['arrangement'], ['revolve'], ['medicines'], ['physically', 'inflate'], ['become', 'due'], ['culminating', 'event'], ['forcefully', 'request'], ['marriage', 'offer'], ['funeral', 'direction'], ['make', 'shine'], ['personnel', 'casualty'], ['per', 'centum'], ['the', 'money'], ['indistinct', 'vocal', 'articulations'], ['hunt', 'for', 'hawks'], ['system', 'of', 'arrangement'], ['duplicate', ',', 'match'], ['provide', 'lodging', 'for'], ['to', 'hunt', 'snipe'], ['give', 'structure', 'to'], ['have', 'sex', 'with'], ['mark', 'with', 'scars'], ['a', 'physical', 'entity'], ['time', 'to', 'come'], ['become', 'pregnant', '.'], ['shoot', 'down', 'birds'], ['activity', 'of', 'selling'], ['act', 'of', 'constructing'], ['throw', 'with', 'force'], ['line', ',', 'cover'], ['flog', 'or', 'whip'], ['to', 'grow', 'rapidly'], ['act', 'of', 'distributing'], ['a', 'large', 'room'], ['observe', ',', 'perceive'], ['restrict', 'or', 'confine'], ['prove', 'or', 'corroborate'], ['remove', 'by', 'rubbing'], ['the', 'armed', 'forces'], ['performance', ',', 'routine'], ['a', 'partial', 'denture'], ['storage', 'cabinet', 'furniture'], ['perform', 'a', 'duty'], ['forbid', ',', 'prohibit'], ['goal', 'or', 'objective'], ['computer', 'text', 'files'], ['a', 'computer', 'server'], ['orient', ',', 'be', 'positioned'], ['start', 'abruptly', ',', 'ignite'], ['throw', 'quickly', ',', 'flick'], ['issue', 'bonds', 'on', 'something'], ['travel', 'on', 'foot', '.'], ['an', 'unforseen', 'development', '.'], ['destruction', 'of', 'a', 'ship'], ['a', 'quantity', 'of', 'something'], ['clean', ',', 'wash', 'thoroughly'], ['sniff', 'at', 'inquiringly', '.'], ['make', 'or', 'become', 'harder'], ['the', 'act', 'of', 'suction'], ['make', 'or', 'become', 'longer'], ['make', 'or', 'become', 'thick'], ['adherence', 'to', 'moral', 'principles'], ['make', 'or', 'become', 'harder'], ['the', 'alleviation', 'of', 'distress'], ['an', 'abundance', 'of', 'something'], ['persuade', 'or', 'achieve', 'acceptance'], ['covering', 'for', 'a', 'bed'], ['an', 'expanse', 'of', 'land'], ['have', 'a', 'logical', 'result'], ['committee', 'with', 'supervisory', 'powers'], ['women', "'s", 'sleeveless', 'undergarment'], ['actors', 'in', 'a', 'play'], ['the', 'day', 'before', 'today'], ['state', 'of', 'being', 'active'], ['appoint', 'to', 'clerical', 'posts'], ['cause', 'to', 'be', 'interested'], ['beat', 'or', 'pound', 'rapidly'], ['a', 'loss', 'of', 'function'], ['head', 'of', 'a', 'country'], ['be', 'against', ',', 'resist'], ['communicate', 'with', 'a', 'hiss'], ['act', 'of', 'applying', 'force'], ['be', 'relevant', 'or', 'pertinent'], ['hold', ',', 'contain', 'within'], ['provide', 'with', 'a', 'toggle'], ['make', 'a', 'ticking', 'sound'], ['guide', ',', 'draw', 'through'], ['something', 'that', 'attains', 'victory'], ['lock', 'with', 'one', 'another'], ['highly', 'infectious', 'viral', 'disease'], ['furnish', 'with', 'a', 'guideline'], ['fail', 'or', 'suffer', 'failure'], ['making', 'real', 'or', 'concrete'], ['the', 'quality', 'of', 'dissimilarity'], ['act', 'of', 'verbally', 'informing'], ['use', 'resources', 'or', 'materials'], ['possibility', 'for', 'the', 'future'], ['position', 'or', 'location', 'change'], ['forked', 'form', 'or', 'shape'], ['fill', 'with', 'high', 'spirits'], ['a', 'concern', 'or', 'affair'], ['a', 'computer-generated', 'visual', 'image'], ['change', 'one', 'for', 'another'], ['secrete', 'or', 'form', 'water'], ['state', 'of', 'being', 'unified'], ['spend', 'time', 'in', 'prison'], ['equip', 'with', 'a', 'fuse'], ['a', 'project', 'or', 'undertaking'], ['pass', 'through', 'the', 'esophagus'], ['a', 'sudden', 'emotional', 'vocalization'], ['bring', 'in', 'to', 'consonance'], ['cause', 'animals', 'to', 'move'], ['move', 'back', 'and', 'forth'], ['something', 'that', 'is', 'craved'], ['make', 'sure', 'of', '.'], ['move', 'along', 'on', 'wheels'], ['submit', ',', 'send', 'to'], ['run', 'or', 'flow', 'slowly'], ['social', 'ranking', 'or', 'position'], ['an', 'objective', 'or', 'goal'], ['a', 'constructed', 'underground', 'passageway'], ['make', 'impure', ';', 'make', 'radioactive'], ['a', 'military', 'fortification', ',', 'stronghold'], ['examine', 'minutely', ';', 'search', 'diligently'], ['to', 'interrupt', ',', 'enter', 'uninvited'], ['do', 'over', ',', 'make', 'new'], ['look', 'furtively', 'at', 'something', '.'], ['a', 'problem', ',', 'something', 'wrong'], ['furnish', 'with', 'metal', 'spurs', '.'], ['produce', 'a', 'phonemic', 'click', '.'], ['move', 'oneself', 'very', 'fast', '.'], ['decree', ',', 'issue', 'an', 'order'], ['overcome', 'with', 'amazement', ';', 'flabbergast'], ['hurt', ',', 'feel', 'physical', 'pain'], ['rehearse', ',', 'learn', 'by', 'repetition'], ['make', 'reference', 'to', 'something', '.'], ['stretch', 'out', 'or', 'over', 'completely'], ['put', 'a', 'harness', 'on', '.'], ['provide', 'medical', 'care', 'for', '.'], ['work', 'material', 'with', 'a', 'tool'], ['physically', 'attach', 'to', 'something', '.'], ['the', 'act', 'of', 'swallowing', 'liquid'], ['write', 'a', 'computer', 'programme', '.'], ['an', 'electronic', 'storage', 'device', '.'], ['make', 'or', 'be', 'a', 'summary'], ['open', 'again', ',', 'open', 'anew'], ['a', 'unit', 'cost', 'of', 'something'], ['(', 'cause', 'to', ')', 'survive'], ['a', 'sporting', 'match', ',', 'boxing'], ['human', 'legs', ',', 'informal', 'usage'], ['the', 'state', 'of', 'being', 'retained'], ['assign', 'to', 'a', 'particular', 'task'], ['an', 'onerous', 'worry', 'or', 'concern'], ['admit', 'to', 'testing', '/', 'proof'], ['a', 'person', 'who', 'cleans', 'chimneys'], ['characteristic', 'call', 'of', 'an', 'animal'], ['provide', 'a', 'feast', 'or', 'banquet'], ['partake', 'or', 'provide', 'a', 'banquet'], ['take', 'apart', 'into', 'constituent', 'pieces'], ['allow', 'the', 'passage', 'of', 'air'], ['a', 'serving', 'of', 'a', 'beverage'], ['accounting', 'period', 'of', '12', 'months'], ['a', 'measurable', 'or', 'quantitative', 'decrease'], ['give', 'something', 'for', 'a', 'cause'], ['a', 'stovetop', ',', 'cooking', 'appliance'], ['behave', 'in', 'a', 'condescending', 'manner'], ['relative', 'darkness', 'of', 'a', 'color'], ['speed', 'up', 'the', 'process', 'of'], ['put', 'up', 'with', 'something', 'unpleasant'], ['make', '(', 'more', ')', 'comprehensible'], ['improve', 'by', 'pruning', 'or', 'polishing'], ['register', 'formally', 'as', 'a', 'participant'], ['idolize', '/', 'show', 'devotion', 'to'], ['the', 'activity', 'of', 'providing', 'supplies'], ['give', 'a', 'detailed', 'account', 'of'], ['end', 'of', 'the', 'american', 'revolution'], ['reason', 'or', 'establish', 'by', 'deduction'], ['create', 'an', 'edge', 'or', 'border'], ['a', 'creased', 'or', 'bent', 'form'], ['be', 'or', 'come', 'into', 'conflict'], ['make', 'more', 'complex', 'or', 'intricate'], ['conscious', 'subjective', 'emotion', 'or', 'desire'], ['luggage', 'compartment', 'of', 'a', 'car'], ['state', 'or', 'explain', 'the', 'meaning'], ['one', 'fourth', 'of', 'a', 'year'], ['a', 'written', 'message', 'of', 'nonacceptance'], ['any', 'tall', ',', 'vertical', 'shape'], ['injure', ',', 'scratch', 'the', 'surface'], ['acknowledge', 'a', 'contribution', 'or', 'cause'], ['give', ',', 'dedicate', ',', 'pledge'], ['cancel', ',', 'reverse', ',', 'annul'], ['festive', 'social', 'event', ',', 'celebration'], ['aerodynamic', 'force', 'that', 'opposed', 'gravity'], ['a', 'caress', 'with', 'the', 'lips'], ['incite', 'some', 'act', 'of', 'insubordination'], ['something', 'that', 'provides', 'advice', '.'], ['move', 'in', 'an', 'irregular', 'course'], ['flow', ',', 'as', 'of', 'liquids'], ['a', 'stiffener', 'in', 'a', 'garment'], ['capital', 'stock', 'in', 'a', 'corporation'], ['touch', 'with', 'the', 'lips', '.'], ['seal', ',', 'insulate', 'or', 'protect'], ['to', 'ignite', 'or', 'catch', 'on', 'fire'], ['remove', 'water', 'from', 'a', 'substance', '.'], ['hit', 'a', 'fly', '(', 'baseball', ')'], ['maintain', ',', 'assert', ',', 'or', 'claim'], ['give', 'out', ',', 'allow', 'to', 'have'], ['fluctuate', ',', 'move', 'back', 'and', 'forth'], ['sharpen', 'a', 'razor', 'with', 'a', 'strap'], ['a', 'brief', 'attempt', 'at', 'some', 'endeavor'], ['increase', 'the', 'acoustic', 'volume', 'of', '.'], ['make', 'a', 'short', ',', 'sharp', 'sound'], ['tell', 'or', 'spread', 'rumors', ';', 'say'], ['disregard', ',', 'fail', 'to', 'attend', 'to'], ['declare', 'or', 'make', 'unfit', 'or', 'unsuitable'], ['to', 'collect', ',', 'acquire', 'or', 'gather'], ['unrestricted', 'freedom', 'to', 'use', 'something', '.'], ['make', 'or', 'become', 'brighter', 'or', 'lighter'], ['the', 'state', 'of', 'being', 'in', 'contact'], ['have', 'faith', 'in', ',', 'bet', 'on'], ['mold', 'for', 'casting', ',', 'setting', 'concrete'], ['recover', ',', 'return', 'to', 'former', 'state'], ['define', ';', 'determine', 'essential', 'qualities', 'of'], ['feel', 'the', 'strong', 'need', 'to', 'eat'], ['an', 'intense', 'surprise', ',', 'often', 'unpleasant'], ['administrator', ',', 'one', 'who', 'controls', 'resources'], ['emit', 'or', 'cause', 'to', 'emit', 'noise'], ['go', 'back', ',', 'restore', ',', 'revive'], ['(', 'cause', 'to', ')', 'move', 'around'], ['to', 'act', 'as', 'the', 'host', 'of'], ['cause', 'to', 'be', 'alert', 'or', 'energetic'], ['have', 'unlawful', 'sex', 'with', 'a', 'prostitute'], ['rot', ';', 'become', 'unfit', 'for', 'consumption'], ['the', 'political', 'head', 'of', 'city', 'government'], ['make', 'up', 'something', 'fictional', 'or', 'untrue'], ['a', 'competitor', 'most', 'likely', 'to', 'win'], ['make', 'something', 'more', 'diverse', 'or', 'varied'], ['the', 'introductory', 'portion', 'of', 'a', 'story'], ['an', 'accounting', 'period', 'of', '12', 'months'], ['interfere', 'with', 'someone', 'else', "'s", 'activity'], ['cause', 'to', 'become', 'awake', 'or', 'alert'], ['oppose', 'with', 'equal', 'weight', 'or', 'force'], ['the', 'status', 'of', 'being', 'champion', '.'], ['having', 'responsibility', 'for', 'managing', 'or', 'supervising'], ['a', 'continuous', 'horizontal', 'layer', 'of', 'brick'], ['a', 'method', 'for', 'solving', 'a', 'problem'], ['burst', 'into', 'flames', '(', 'telic', ')'], ['return', 'to', 'in', 'thought', 'or', 'speech'], ['engrave', 'by', 'means', 'of', 'dots', '.'], ['give', 'a', 'thrashing', 'to', ',', 'beat'], ['the', 'front', 'face', 'of', 'a', 'building'], ['take', 'someone', "'s", 'soul', 'into', 'heaven'], ['heat', 'metal', 'prior', 'to', 'working', 'it'], ['the', 'soft', 'padding', 'under', 'a', 'saddle'], ['pass', 'into', 'a', 'state', 'or', 'condition'], ['become', 'aware', 'of', 'through', 'the', 'senses'], ['make', 'the', 'surface', 'level', 'or', 'smooth'], ['a', 'cause', 'or', 'source', 'of', 'harm'], ['put', 'or', 'store', 'in', 'a', 'bottle'], ['clear', 'mucus', 'from', 'one', "'s", 'throat'], ['protect', 'or', 'strengthen', 'with', 'sandbags', '.'], ['a', 'region', 'allocated', 'to', 'hold', 'something'], ['hold', 'back', 'in', 'uncertainty', 'or', 'unwillingness'], ['scrape', 'or', 'rub', 'to', 'relieve', 'itching'], ['have', 'a', 'strong', 'sexual', 'attraction', 'to'], ['elevated', 'wooden', 'porch', 'of', 'a', 'house'], ['be', 'in', 'direct', 'physical', 'contact', 'with'], ['a', 'judgement', 'or', 'opinion', 'about', 'something'], ['turn', 'on', 'an', 'axis', ';', 'spin'], ['recover', 'posession', 'or', 'get', 'back', 'again'], ['touch', 'or', 'hold', 'with', 'the', 'hands'], ['the', 'legal', 'state', 'of', 'owing', 'money'], ['exercise', 'or', 'have', 'power', 'of', 'memory'], ['an', 'open', 'motorboat', 'used', 'for', 'transport'], ['a', 'change', 'which', 'is', 'a', 'decrease'], ['reduce', 'to', 'a', 'simpler', 'molecular', 'compound'], ['secondary', 'public', 'school', ',', 'grades', '9-12'], ['render', 'one', 'language', 'in', 'another', 'language'], ['the', 'processing', 'or', 'operation', 'of', 'something'], ['the', 'forceful', 'hit', 'of', 'a', 'ball'], ['the', 'act', 'of', 'shoving', 'something', 'away'], ['a', 'journey', 'for', 'some', 'purpose', '.'], ['a', 'style', 'and', 'size', 'of', 'typeset'], ['insert', 'or', 'close', 'with', 'a', 'plug'], ['recite', 'as', 'a', 'chant', ',', 'intone'], ['either', 'half', 'of', 'an', 'archery', 'bow'], ['baseball', ':', 'strike', 'out', 'a', 'batter'], ['cause', 'something', 'to', 'be', 'the', 'case'], ['a', 'sharp', 'side', 'of', 'an', 'object'], ['a', 'social', 'gesture', 'of', 'deferential', 'greeting'], ['meshed', 'frame', 'over', 'windows', 'and', 'doors'], ['be', 'affected', 'with', 'smut', 'or', 'mildew'], ['an', 'instance', 'of', 'visual', 'perception', '.'], ['an', 'allotment', 'or', 'percentage', 'of', 'something'], ['ponder', ',', 'consider', ',', 'observe', 'carefully'], ['a', 'mental', 'state', 'of', 'sadness', '.'], ['find', ',', 'determine', 'the', 'place', 'of'], ['effectively', 'wield', 'or', 'manipulate', 'a', 'concrete', 'entity'], ['enjoy', ',', 'be', 'fond', 'of', 'or', 'approve'], ['require', 'or', 'need', 'something', 'that', 'is', 'missing'], ['cause', 'to', 'come', 'into', 'existence', ';', 'create'], ['the', 'fashionable', 'elite', 'of', 'a', 'given', 'community'], ['cast', 'off', ',', 'get', 'rid', 'of', 'something'], ['give', 'suck', 'to', ';', 'receive', 'suck', '.'], ['a', 'sudden', ',', 'brief', 'burst', 'of', 'light'], ['draw', 'advantages', 'from', ';', 'take', 'advantage', '.'], ['abstain', 'from', 'eating', 'or', 'from', 'certain', 'foods'], ['a', 'sudden', '(', 'emotional', ')', 'outburst', '.'], ['fool', 'or', 'hoax', '(', 'someone', ')', '.'], ['ride', '(', 'a', 'bicycle', 'or', 'motorcycle', ')'], ['(', 'literally', ')', 'sound', 'with', 'resonance', '.'], ['employ', 'or', 'utilize', ',', 'put', 'into', 'service'], ['take', 'something', 'or', 'someone', 'away', 'from', 'somewhere'], ['learn', ',', 'assimilate', 'a', 'body', 'of', 'knowledge'], ['cause', 'friction', ';', 'wear', 'away', 'or', 'erode'], ['the', 'act', 'of', 'leading', 'or', 'guiding', 'others'], ['remove', 'the', 'pod', ',', 'husk', 'or', 'shell'], ['remove', 'tree', 'stumps', 'from', 'an', 'area', '.'], ['face', 'an', 'event', 'and', 'survive', ';', 'endure'], ['a', 'casual', ',', 'brief', 'meeting', 'with', 'someone'], ['anything', 'illusory', ',', 'not', 'concrete', 'or', 'real'], ['an', 'approximate', 'attribute', 'or', 'quality', 'of', 'something'], ['the', 'legal', 'granting', 'or', 'bestowment', 'of', 'rights'], ['take', 'into', 'one', "'s", 'family', 'or', 'group'], ['take', 'or', 'capture', 'by', 'force', 'or', 'authority'], ['make', 'or', 'become', 'less', 'strict', 'or', 'severe'], ['stimulating', 'or', 'demanding', 'factors', 'of', 'a', 'situation'], ['income', 'collected', 'as', 'tax', 'by', 'a', 'government'], ['provide', 'or', 'equip', 'specifically', 'with', 'furniture', '.'], ['the', 'termination', ',', 'ending', 'of', 'a', 'meeting'], ['stick', 'out', ',', 'jut', 'out', ',', 'protrude'], ['fruit', 'cooked', 'down', 'with', 'sugar', ',', 'jam'], ['manipulate', 'the', 'registers', 'of', 'a', 'musical', 'instrument'], ['have', 'a', 'monopoly', 'of', ';', 'control', 'fully'], ['an', 'indicator', 'that', 'orients', 'one', 'generally', '.'], ['discipline', 'in', 'personal', 'or', 'social', 'activities', '.'], ['wrinkled', ',', 'crumpled', ',', 'or', 'creased', '.'], ['preserve', 'in', 'a', 'tin', 'or', 'jar', '.'], ['start', 'suddenly', ',', 'as', 'if', 'from', 'fright'], ['a', 'mental', 'activity', 'of', 'construing', 'or', 'interpreting'], ['germinate', ',', 'as', 'in', 'a', 'plant', '.'], ['the', 'act', 'of', 'annoying', 'someone', 'or', 'something'], ['remove', 'the', 'pod', ',', 'husk', 'or', 'shell'], ['cause', 'to', 'vibrate', 'in', 'a', 'different', 'pattern'], ['people', 'assembled', 'together', 'in', 'a', 'rhythmic', 'movement'], ['coating', 'used', 'as', 'a', 'repellant', 'to', 'moisture'], ['drive', 'at', 'an', 'excessive', 'or', 'illegal', 'velocity'], ['(', 'cause', 'to', ')', 'fall', 'in', 'drops'], ['move', 'or', 'execute', 'very', 'quickly', 'or', 'hastily'], ['the', 'process', 'of', 'becoming', 'less', 'or', 'smaller'], ['make', 'or', 'become', 'wider', 'or', 'more', 'extensive'], ['escape', 'or', 'avoid', 'a', 'situation', 'or', 'responsibility'], ['formally', 'announce', 'the', 'termination', 'of', 'an', 'agreement'], ['become', 'master', 'of', ',', 'overcome', ',', 'dominate'], ['play', 'a', 'stringed', 'instrument', 'with', 'a', 'bow'], ['indicate', ',', 'be', 'signs', 'or', 'symptoms', 'of'], ['say', ',', 'state', ',', 'or', 'perform', 'again'], ['domesticated', 'animals', 'kept', 'for', 'use', 'or', 'profit'], ['the', 'bestowing', 'of', 'a', 'franchise', ',', 'enfranchisment'], ['add', 'embellishments', '(', 'to', 'medieval', 'manuscripts', ')'], ['twice', 'the', 'quantity', 'or', 'amount', 'of', 'something'], ['a', 'quantity', 'that', 'is', 'an', 'approximate', 'calculation'], ['the', 'people', 'who', 'dwell', 'in', 'a', 'camp'], ['a', 'person', 'who', 'delivers', 'information', 'or', 'goods'], ['a', 'sweeping', 'cut', 'with', 'a', 'sharp', 'instrument'], ['make', 'or', 'become', 'more', 'acute', 'or', 'intense'], ['beat', 'thoroughly', 'in', 'a', 'competition', 'or', 'fight'], ['have', 'a', 'strong', 'desire', 'to', 'do', 'something'], ['fly', 'in', 'or', 'on', 'a', 'glider', 'plane'], ['assign', 'a', 'new', 'time', 'for', 'an', 'event'], ['(', 'cause', 'to', ')', 'smell', 'bad', '.'], ['employ', 'or', 'become', 'employed', 'in', 'a', 'position'], ['emphasize', ';', 'draw', 'attention', 'to', ';', 'underline'], ['the', 'public', 'submission', 'of', 'a', 'legal', 'record'], ['a', 'knapsack', ',', 'bundle', 'on', 'the', 'back'], ['the', 'price', 'of', 'a', 'good', 'or', 'service'], ['a', 'portion', 'or', 'percentage', 'of', 'a', 'whole'], ['enjoy', ',', 'be', 'fond', 'of', 'or', 'approve'], ['a', 'discipline', ',', 'a', 'branch', 'of', 'knowledge'], ['close', 'within', 'bounds', ';', 'deprive', 'of', 'freedom'], ['a', 'standardized', 'system', 'of', 'measurements', 'for', 'heaviness'], ['decide', 'a', 'specific', 'time', 'for', 'an', 'event'], ['do', 'better', 'in', 'a', 'competition', ';', 'win'], ['the', 'longer', 'telegraphic', 'signal', 'of', 'morse', 'code'], ['form', 'an', 'aesthetic', 'or', 'conceptual', 'whole', '.'], ['entail', 'as', 'a', 'necessary', 'accompaniment', 'or', 'result'], ['jolt', 'of', 'electrical', 'current', 'to', 'the', 'body'], ['make', 'a', 'hole', 'with', 'a', 'pointed', 'tool'], ['enter', 'a', 'realm', ',', 'become', 'involved', 'with'], ['initiation', ',', 'founding', ',', 'introduction', 'of', 'something'], ['a', 'reference', 'line', 'used', 'to', 'align', 'lettering'], ['a', 'relative', 'loss', 'in', 'price', ',', 'value'], ['a', 'physical', 'protuberance', ',', 'large', 'or', 'small'], ['incorporate', 'or', 'enclose', 'one', 'thing', 'in', 'another'], ['a', 'slight', 'cut', 'in', 'a', 'physical', 'surface'], ['take', 'a', 'digestion-aiding', 'walk', 'after', 'a', 'meal'], ['propagation', 'of', 'waves', 'back', 'from', 'a', 'surface'], ['recognize', ',', 'be', 'familar', 'or', 'well-versed', 'with'], ['the', 'ground', 'level', 'of', 'a', 'multi-storied', 'building'], ['bring', 'into', 'general', 'popular', '/', 'common', 'use'], ['an', 'electrical', 'connection', 'between', 'conductor', 'and', 'earth'], ['settlement', 'of', 'people', 'distant', 'from', 'their', 'homeland'], ['preserve', ',', 'keep', 'in', 'safety', ',', 'use', 'frugally'], ['a', 'statement', ',', 'an', 'idea', 'expressed', 'in', 'language'], ['provide', 'with', 'rails', ',', 'as', 'by', 'laying', '.'], ['provide', 'with', 'rails', ',', 'as', 'by', 'laying', '.'], ['take', 'in', 'marriage', ';', 'perform', 'a', 'marriage', 'ceremony'], ['deliver', 'a', 'formal', 'talk', 'or', 'reprimand', 'at', 'length'], ['a', 'speech', 'act', 'of', 'acknowledging', 'gratitude', 'for', 'something'], ['move', 'to', 'a', 'lower', 'position', ',', 'terminate', 'something'], ['increase', 'in', 'amount', ',', 'extent', ',', 'or', 'intensity'], ['make', 'an', 'abstract', 'thing', 'tight', 'or', 'more', 'restrictive'], ['feel', 'hot', 'or', 'painful', ',', 'as', 'if', 'burning'], ['cause', 'to', 'be', '/', 'show', 'to', 'be', 'invalid'], ['move', 'faster', ',', 'or', 'cause', 'to', 'move', 'faster'], ['impress', 'or', 'affect', 'deeply', '(', 'emotionally', ')', '.'], ['lessen', 'in', 'force', ',', 'impact', ',', 'or', 'effect'], ['a', 'natural', 'physical', 'force', 'of', 'drawing', 'towards', 'something'], ['become', 'widespread', 'as', 'an', 'idea', 'or', 'feeling', '.'], ['a', 'human', 'utters', 'words', 'loudly', 'and', 'forcefully', '.'], ['a', 'sharp', 'bladed', 'instrument', 'for', 'shaving', 'off', 'hair'], ['having', 'an', 'intuitive', 'understanding', 'or', 'appreciation', 'of', 'something'], ['the', 'act', 'of', 'damaging', 'something', ',', 'causing', 'breakage'], ['throw', 'with', 'force', 'or', 'recklessness', ';', 'cast', 'away'], ['an', 'attempt', 'or', 'effort', 'to', 'achieve', 'a', 'goal'], ['give', 'an', 'incentive', 'or', 'reason', 'for', 'action', '.'], ['(', 'cause', 'to', ')', 'return', 'after', 'being', 'refused'], ['envelop', 'completely', ';', 'form', 'a', 'cover', 'over', '.'], ['a', 'feeling', 'or', 'state', 'of', 'extreme', 'anger', '.'], ['supply', 'or', 'furnish', 'with', 'something', 'needed', 'or', 'wanted'], ['the', 'act', 'of', 'physically', 'affixing', 'or', 'connecting', 'things'], ['act', 'of', 'becoming', 'distant', ',', 'moving', 'away', 'from'], ['a', 'feeling', 'of', 'aversion', ',', 'distaste', ',', 'antipathy'], ['cause', 'to', 'reproduce', '(', 'plants', 'or', 'animals', ')'], ['leave', 'out', ',', 'omit', ',', 'prevent', 'from', 'entering'], ['the', 'finger', 'next', 'to', 'the', 'thumb', ',', 'forefinger'], ['travel', 'in', 'or', 'be', 'shaped', 'like', 'a', 'zigzag'], ['social', 'sphere', ',', 'scope', 'of', 'activity', 'or', 'interest'], ['the', 'rescue', 'or', 'release', 'of', 'someone', ',', 'something'], ['continue', 'to', 'live', ',', 'exist', ',', 'or', 'prosper'], ['the', 'act', 'of', 'officially', 'gaining', 'entrance', 'to', 'somewhere'], ['travel', ';', 'make', 'a', 'tour', 'of', 'a', 'place'], ['fishing-exhaust', 'by', 'allowing', 'to', 'pull', 'on', 'the', 'line'], ['the', 'act', 'of', 'having', 'and', 'controlling', 'property', '.'], ['rinse', 'one', "'s", 'mouth', 'with', 'mouthwash', ',', 'gargle'], ['the', 'cognitive', 'processes', 'and', 'capacity', 'to', 'remember', '.'], ['the', 'close', 'approach', 'of', 'one', 'object', 'to', 'another'], ['support', 'by', 'placing', 'on', 'or', 'against', 'something', 'solid'], ['connect', 'or', 'fasten', 'into', 'a', 'chain', 'by', 'linking'], ['physical', 'matter', 'left', 'behind', 'after', 'a', 'removal', 'process'], ['unite', ';', 'merge', 'with', 'something', 'already', 'in', 'existence'], ['make', 'a', 'loud', 'noise', ',', 'as', 'an', 'animal'], ['compress', 'out', 'of', 'shape', 'or', 'into', 'small', 'pieces'], ['the', 'quality', 'of', 'dedication', ',', 'fixity', 'of', 'purpose'], ['adapt', 'oneself', 'to', 'new', 'or', 'different', 'conditions', '.'], ['go', 'on', 'a', 'campaign', 'or', 'off', 'to', 'war'], ['cook', 'slowly', 'for', 'a', 'long', 'time', 'in', 'liquid'], ['connect', 'with', ';', 'reach', 'a', 'target', 'or', 'goal'], ['group', 'of', 'people', 'defined', 'by', 'a', 'specific', 'profession'], ['put', 'under', 'the', 'control', 'of', 'the', 'federal', 'government'], ['reduce', 'to', 'small', 'pieces', 'by', 'pounding', 'or', 'abrading'], ['apply', 'conditioner', 'to', 'make', 'smooth', 'and', 'shiny', '.'], ['intend', 'something', 'to', 'move', 'toward', 'a', 'certain', 'goal'], ['a', 'local', 'announcement', 'made', 'within', 'a', 'public', 'broadcast'], ['the', 'hard', 'upper', 'part', 'of', 'the', 'human', 'nose'], ['express', 'protest', 'or', 'dissent', ',', 'raise', 'an', 'objection'], ['a', 'written', 'list', 'of', 'particular', 'things', 'being', 'offered'], ['a', 'social', 'entreaty', ',', 'as', 'courtship', 'or', 'pleading'], ['a', 'process', 'of', 'development', ',', 'increase', 'or', 'advancement'], ['the', 'human', 'act', 'of', 'making', 'something', 'new', '.'], ['(', 'the', 'content', 'of', ')', 'a', 'communication', '.'], ['an', 'authoritative', 'text', 'or', 'the', 'corresponding', 'citation', '.'], ['the', 'breakup', 'or', 'termination', 'of', 'a', 'social', 'relationship'], ['apply', 'a', 'thin', 'coating', 'of', 'paint', '/', 'metal'], ['measure', 'the', 'depth', 'of', 'a', 'body', 'of', 'water'], ['the', 'medical', 'procedure', 'of', 'manually', 'turning', 'the', 'uterus'], ['estimate', 'or', 'calculate', 'the', 'numerical', 'value', 'of', 'something'], ['deprive', 'of', 'courage', ',', 'hope', ',', 'or', 'optimism'], ['a', 'decorative', 'container', ',', 'often', 'for', 'holding', 'flowers'], ['warn', 'or', 'call', 'to', 'a', 'sense', 'of', 'preparedness'], ['a', 'state', 'of', 'confinement', 'in', 'a', 'small', 'space'], ['a', 'notational', 'mark', 'used', 'in', 'writing', ',', 'diacritic'], ['a', 'home', 'furnishing', 'that', 'is', 'a', 'fabric', 'cover'], ['capture', ',', 'ensnare', ',', 'discover', 'in', 'the', 'act'], ['heavy', 'radioactive', 'metallic', 'element', ',', 'atomic', 'number', '92'], ['a', 'spoken', 'or', 'written', 'reply', 'to', 'a', 'question'], ['a', 'full', 'stop', ',', 'as', 'a', 'punctuation', 'mark'], ['a', 'person', 'who', 'is', 'aggressively', 'pursued', ',', 'chased'], ['lightest', 'most', 'abundant', 'element', ',', 'atomic', 'number', '1'], ['move', 'or', 'change', 'one', "'s", 'own', 'position', 'or', 'quality'], ['rescue', ',', 'redeem', ',', 'deliver', 'from', 'harm', 'or', 'danger'], ['an', 'indication', 'or', 'manifestation', 'of', 'something', ',', 'often', 'physical'], ['sound', 'sharply', 'or', 'shrilly', ',', 'as', 'if', 'stabbing', '.'], ['determine', 'a', 'standard', ';', 'estimate', 'a', 'capacity', 'or', 'measurement'], ['reach', 'an', 'objective', ',', 'attain', 'a', 'benchmark', 'or', 'goal'], ['a', 'large', 'approximate', 'quantity', 'of', 'some', 'attribute', 'or', 'thing'], ['be', 'equivalent', ',', 'have', 'the', 'same', 'effect', 'as', 'something'], ['[', 'provide', 'with', 'a', 'new', 'or', 'different', 'resting', 'place'], ['make', 'pregnant', ';', 'fertilize', 'and', 'cause', 'to', 'grow', '.'], ['notable', 'or', 'famous', 'persons', '(', 'real', 'or', 'fictional', ')'], ['a', 'feeling', 'for', 'something', 'that', 'is', 'desired', 'intensely', '.'], ['bring', 'back', 'to', 'life', ',', 'return', 'from', 'the', 'dead'], ['the', 'act', 'of', 'sorting', 'one', 'thing', 'from', 'others', '.'], ['(', 'medicine', ')', 'apply', 'a', 'plaster', 'cast', 'to', '.'], ['fail', 'in', 'strength', 'or', 'health', 'and', 'cease', 'to', 'funtion'], ['the', 'act', 'of', 'choosing', ',', 'picking', 'one', 'among', 'many'], ['make', 'a', 'causal', 'or', 'logical', 'connection', ';', 'group', 'together'], ['move', 'about', 'in', 'a', 'confused', 'or', 'purposeless', 'manner', '.'], ['support', ';', 'hold', 'up', ';', 'nurture', 'or', 'provide', 'for'], ['nations', 'unified', 'by', 'shared', 'interests', ',', 'history', 'or', 'institutions'], ['alter', 'by', 'chemical', 'means', ',', 'to', 'make', 'images', '.'], ['a', 'short', 'lyric', 'or', 'poem', 'intended', 'to', 'be', 'sung'], ['informal', 'usage', 'for', 'a', 'domestic', 'cat', ',', 'often', 'young'], ['a', 'section', 'of', 'a', 'composition', ',', 'textual', 'or', 'musical'], ['put', 'up', 'with', 'something', 'or', 'someone', '(', 'unpleasant', ')'], ['a', 'pair', 'of', 'mated', 'people', ',', 'e.g.', ',', 'married'], ['talk', 'or', 'behave', 'amorously', ';', 'have', 'sex', 'with', '.'], ['oppose', 'or', 'approach', ',', 'as', 'in', 'hostility', 'or', 'competition'], ['be', 'a', 'factor', 'or', 'play', 'a', 'role', 'in', 'something'], ['a', 'static', 'vertical', 'layout', ',', 'e.g.', 'text', 'or', 'numbers'], ['change', 'the', 'purpose', 'or', 'functioning', 'of', 'a', 'system', '.'], ['any', 'activity', 'that', 'acquires', 'a', 'person', "'s", 'attention', '.'], ['write', 'as', 'if', 'with', 'print', ',', 'not', 'in', 'cursive'], ['a', 'quantity', 'of', 'money', 'added', 'to', 'a', 'bank', 'account'], ['limit', ',', 'keep', 'under', 'control', ',', 'deter', ',', 'forbid'], ['the', 'temperature', 'at', 'which', 'a', 'liquid', 'boils', ',', 'bubbles'], ['make', 'a', 'certain', 'size', 'or', 'sort', 'according', 'to', 'size'], ['travel', 'in', 'front', 'of', ',', 'be', 'first', ',', 'precede'], ['provide', 'with', 'tools', 'or', 'items', 'for', 'a', 'specific', 'purpose'], ['move', 'freely', '(', 'usually', 'within', 'a', 'bounded', 'space', ')'], ['elecate', 'or', 'idealize', 'in', 'allusion', 'to', 'christ', "'s", 'transfiguration'], ['facility', 'for', 'secure', 'storage', 'of', 'things', ',', 'often', 'money'], ['anyone', 'who', 'judges', 'or', 'finds', 'fault', 'with', 'something', '.'], ['pat', 'or', 'squeeze', ',', 'especially', 'under', 'the', 'chin', '.'], ['a', 'surrogate', 'expression', ',', 'utterance', 'for', 'a', 'taboo', 'word'], ['a', 'ceremony', 'marking', 'admission', 'into', 'a', 'religious', 'community', '.'], ['the', 'pursuit', 'and', 'killing', 'or', 'capture', 'of', 'animals', '.'], ['a', 'person', 'under', 'the', 'guardianship', 'or', 'custody', 'of', 'another'], ['open', 'to', 'members', 'of', 'all', 'races', 'and', 'ethnic', 'groups'], ['consider', 'again', ',', 'usually', 'with', 'a', 'view', 'to', 'changing'], ['coat', ',', 'cover', ',', 'or', 'treat', 'the', 'surface', 'of'], ['be', 'on', 'base', 'at', 'the', 'end', 'of', 'an', 'inning'], ['benjamin', 'rush', ',', 'physician', 'and', 'american', 'revolutionary', 'leader', '.'], ['praise', 'the', 'qualities', 'in', 'order', 'to', 'sell', 'or', 'promote'], ['find', 'the', 'solution', 'to', 'or', 'understand', 'the', 'meaning', 'of'], ['consider', 'again', ',', 'usually', 'with', 'a', 'view', 'to', 'changing'], ['scar', 'physically', ';', 'create', 'a', 'mark', 'on', 'someone', 'or', 'something'], ['complain', 'about', 'something', 'or', 'say', 'mean', 'things', 'to', 'someone', '.'], ['protect', 'from', 'heat', ',', 'cold', ',', 'or', 'other', 'detrimental', 'conditions'], ['pass', 'the', 'tongue', 'over', ',', 'take', 'up', 'with', 'the', 'tongue'], ['move', 'an', 'implement', 'through', 'with', 'a', 'circular', 'motion', ',', 'mix'], ['restart', 'or', 'continue', 'an', 'activity', 'after', 'an', 'interruption', 'or', 'pause'], ['pass', 'the', 'tongue', 'over', ',', 'take', 'up', 'with', 'the', 'tongue'], ['a', 'removable', 'device', 'that', 'is', 'an', 'accessory', 'to', 'larger', 'object'], ['an', 'agent', "'s", 'act', 'of', 'terminating', 'an', 'activity', 'or', 'event'], ['beat', '(', 'as', 'if', ')', 'with', 'a', 'whip', 'or', 'rod'], ['fake', 'or', 'falsify', ';', 'control', 'in', 'a', 'fraudulent', 'manner', '.'], ['a', 'conceptual', 'formulation', ';', 'a', 'way', 'of', 'conceiving', 'something', '.'], ['(', 'cause', 'to', ')', 'be', 'firmly', 'attached', 'or', 'closed', '.'], ['(', 'cause', 'to', ')', 'strike', 'the', 'air', 'in', 'flight', '.'], ['depict', 'or', 'portray', 'in', 'art', ',', 'words', ',', 'or', 'model'], ['annoy', ',', 'harass', ',', 'or', 'mock', ',', 'perhaps', 'playfully', '.'], ['the', 'state', 'of', 'being', 'more', 'than', 'satisfied', 'or', 'satiated', '.'], ['create', 'a', 'new', 'entity', 'by', 'putting', 'components', 'or', 'members', 'together'], ['improve', 'by', 'restoring', 'to', 'a', 'previous', 'or', 'better', 'condition', '.'], ['the', 'act', 'of', 'giving', 'someone', 'a', 'job', ',', 'hiring', 'someone'], ['strike', 'or', 'hit', 'sharply', ',', 'perhaps', 'as', 'in', 'sport', '.'], ['form', 'a', 'spiral', 'or', 'move', 'in', 'a', 'spiral', 'course', '.'], ['move', 'or', 'drive', 'forcefully', 'as', 'if', 'by', 'a', 'punch', '.'], ['area', 'of', 'damaged', ',', 'cut', 'tissue', 'on', 'a', 'living', 'body'], ['a', 'person', 'who', 'manipulate', 'or', 'controls', 'a', 'device', 'or', 'machine'], ['literally', 'and', 'quantifiably', 'raise', 'in', 'social', 'rank', 'or', 'status', '.'], ['a', 'long', ',', 'thin', 'implement', ',', 'usually', 'made', 'of', 'wood'], ['take', 'away', 'a', 'vital', 'or', 'essential', 'part', 'of', 'something', '.'], ['an', 'air', 'force', 'military', 'unit', ',', 'smaller', 'than', 'a', 'squadron'], ['edit', 'so', 'as', 'to', 'block', 'the', 'truth', ';', 'prevent', 'distribution'], ['any', 'address', 'at', 'which', 'one', 'dwells', 'more', 'than', 'temporarily', '.'], ['the', 'feeling', 'of', 'annoyance', 'at', 'being', 'hindered', 'or', 'thwarted', '.'], ['make', 'accessible', 'to', 'some', 'action', ',', 'influence', ',', 'or', 'condition'], ['the', 'act', 'of', 'coming', 'to', 'rest', 'after', 'a', 'voyage', '.'], ['the', 'act', 'of', 'hindering', 'someone', "'s", 'plans', 'or', 'efforts', '.'], ['manual', 'stimulation', 'fo', 'the', 'genital', 'area', 'for', 'sexual', 'pleasure', '.'], ['role', 'or', 'function', 'of', 'the', 'head', 'of', 'a', 'government', 'department'], ['move', 'to', 'a', 'new', 'location', ',', 'state', ',', 'or', 'situation'], ['a', 'state', 'of', 'pretending', 'to', 'be', 'someone', ',', 'something', 'else'], ['indicate', 'contempt', 'by', 'breathing', 'noisily', 'and', 'forcefully', 'through', 'the', 'news'], ['strike', 'heavily', ',', 'especially', 'with', 'a', 'fist', 'or', 'implement', '.'], ['an', 'area', 'that', 'is', 'central', 'within', 'some', 'larger', 'region', '.'], ['angle', 'with', 'a', 'hook', 'and', 'line', 'and', 'draw', 'through', 'water'], ['the', 'act', 'of', 'copying', 'the', 'actions', 'of', 'someone', 'else', '.'], ['a', 'cyclical', 'span', 'of', 'time', 'defined', 'by', 'a', 'calendar', 'system'], ['activity', 'of', 'extracting', 'ore', 'or', 'other', 'matter', 'from', 'the', 'ground'], ['a', 'person', 'eating', 'a', 'meal', ',', 'often', 'in', 'a', 'restaurant'], ['make', 'receptive', 'or', 'willing', 'toward', 'an', 'action', 'or', 'belief', '.'], ['a', 'state', 'of', 'social', 'order', 'in', 'conformance', 'with', 'existing', 'law'], ['a', 'means', 'or', 'agency', 'by', 'which', 'something', 'is', 'communicated', '.'], ['make', 'out', 'of', 'components', '(', 'in', 'an', 'improvising', 'manner', ')'], ['(', 'cause', 'to', ')', 'stop', 'speaking', 'or', 'bring', 'about', 'silence'], ['the', 'principal', 'activity', 'that', 'one', 'does', 'to', 'earn', 'money', '.'], ['soldier', 'who', 'is', 'a', 'member', 'of', 'a', 'special', 'forces', 'group'], ['care', 'about', ',', 'be', 'bothered', 'by', ',', 'be', 'concerned', 'by'], ['line', 'up', 'or', 'make', 'parallel', ';', 'bring', 'into', 'proper', 'adjustment'], ['an', 'evaluation', ',', 'an', 'estimation', 'of', 'worth', ',', 'a', 'grade'], ['a', 'verbal', 'or', 'written', 'request', 'for', 'access', 'to', 'something', '.'], ['a', 'plant', 'structure', 'near', 'the', 'top', 'of', 'the', 'plant', '.'], ['a', 'male', 'bee', 'whose', 'function', 'is', 'to', 'fertilize', 'the', 'queen'], ['a', 'measure', 'of', 'the', 'likelihood', 'or', 'probability', 'of', 'an', 'event'], ['keep', 'away', ',', 'prevent', 'from', 'happening', ',', 'or', 'not', 'do', 'something'], ['gain', 'points', 'in', 'a', 'game', ';', 'add', 'up', '(', 'numbers', ')'], ['provide', '(', 'a', 'person', 'or', 'group', ')', 'with', 'power', 'or', 'authority'], ['discern', ',', 'identify', ',', 'or', 'know', 'because', 'of', 'a', 'previous', 'experience'], ['to', 'remain', 'emotionally', 'or', 'intellectually', 'attached', 'without', 'necessary', 'physical', 'contact', '.'], ['run', 'at', 'a', 'moderately', 'swift', 'pace', ',', 'as', 'for', 'exercise', '.'], ['make', 'less', 'emotionally', 'hostile', ';', 'win', 'over', 'mentally', 'or', 'emotionally', '.'], ['be', 'in', 'command', 'of', ';', 'exercise', 'authority', 'or', 'control', 'over', '.'], ['value', 'a', 'thing', 'or', 'course', 'of', 'action', 'more', 'highly', 'than', 'another'], ['give', 'pleasure', ',', 'satifaction', ',', 'or', 'happiness', ';', 'be', 'pleasing', 'to'], ['take', 'vows', ',', 'join', 'or', 'allow', 'to', 'join', 'a', 'religious', 'order'], ['a', 'portrayal', 'of', 'someone', 'or', 'something', ',', 'through', 'words', 'or', 'behavior'], ['maintain', 'or', 'preserve', 'a', 'particular', 'condition', ',', 'state', ',', 'or', 'activity'], ['seize', 'and', 'take', 'captive', ';', 'take', 'or', 'gain', 'possession', 'of', 'something'], ['train', ',', 'tutor', 'or', 'polish', 'social', 'behavior', ',', 'taste', ',', 'judgement'], ['someone', 'or', 'something', 'that', 'is', 'the', 'agent', 'of', 'fulfilling', 'desired', 'expectations'], ['stand', 'with', 'arms', 'or', 'forelegs', 'raised', ',', 'as', 'of', 'menacing', '.'], ['a', 'city', 'on', 'the', 'river', 'thames', 'in', 'bershire', ',', 'southern', 'england'], ['(', 'cause', 'to', ')', 'to', 'turn', 'to', 'the', 'left', 'side', '.'], ['inability', 'of', 'the', 'heart', 'to', 'pump', 'blood', 'to', 'sustain', 'life', 'functions'], ['the', 'act', 'of', 'proposing', 'someone', 'as', 'a', 'candidate', 'for', 'something', '.'], ['a', 'chain', 'of', 'atoms', 'in', 'a', 'molecule', 'forming', 'a', 'closed', 'loop'], ['the', 'release', 'of', 'pressure', 'built', 'up', 'during', 'a', 'stop', 'consonant', '.'], ['demand', ',', 'ask', 'for', ',', 'or', 'take', 'as', 'one', "'s", 'due'], ['the', 'tallied', 'points', 'of', 'a', 'game', ',', 'at', 'a', 'given', 'time'], ['a', 'band', 'of', 'leather', 'or', 'rope', 'made', 'to', 'identify', 'animals', '.'], ['mechanism', 'that', 'controls', 'the', 'water', 'level', 'in', 'a', 'section', 'of', 'canal'], ['come', 'or', 'bring', 'back', 'to', 'life', ',', 'health', ',', 'existence', 'or', 'use'], ['try', 'to', 'gain', 'favor', 'by', 'flattering', 'or', 'showing', 'submission', 'or', 'fear', '.'], ['cause', 'to', 'be', 'distrusted', 'or', 'rejected', ';', 'damage', 'the', 'reputation', 'of', '.'], ['excuse', 'some', 'minor', 'social', 'infraction', ';', 'accept', 'an', 'excuse', 'for', 'something', '.'], ['regain', 'or', 'make', 'up', 'for', ',', 'as', 'of', 'a', 'financial', 'loss', '.'], ['behave', 'in', 'a', 'certain', 'way', ';', 'have', 'a', 'specific', 'effect', 'or', 'outcome'], ['be', 'a', 'sign', 'or', 'indication', 'of', ';', 'have', 'as', 'a', 'meaning', '.'], ['reverse', 'the', 'winding', 'or', 'twisting', 'of', ';', 'separate', 'the', 'tangles', 'of', '.'], ['(', 'cause', 'to', ')', 'move', 'back', 'and', 'forth', 'or', 'side', 'to', 'side'], ['react', ',', 'or', 'respond', 'to', 'a', 'call', ',', 'command', ',', 'or', 'stimulus'], ['be', 'the', 'deciding', 'factor', 'in', 'a', 'state', 'of', 'affairs', 'or', 'an', 'outcome'], ['act', ',', 'form', ',', 'or', 'progress', 'according', 'to', 'a', 'plan', 'or', 'ideal'], ['cook', 'on', 'a', 'hot', 'surface', 'using', 'fat', ';', 'cook', 'in', 'oil', '.'], ['part', ',', 'cease', ',', 'or', 'break', 'association', 'with', '(', 'something', ')', '.'], ['devote', 'or', 'adapt', 'exclusively', 'to', 'an', 'skill', ',', 'study', ',', 'or', 'work'], ['a', 'name', 'or', 'trait', 'that', 'identifies', 'a', 'product', 'or', 'type', 'of', 'thing'], ['doubt', 'or', 'an', 'impression', 'that', 'something', 'might', 'not', 'be', 'the', 'case', '.'], ['(', 'baseball', ')', 'tag', 'the', 'base', 'runner', 'to', 'get', 'him', 'out', '.'], ['give', 'equal', 'rights', 'to', ',', 'as', 'of', 'the', 'right', 'to', 'vote', '.'], ['fix', '(', 'something', 'old', ')', 'with', 'a', 'new', 'part', ';', 'improve', '.'], ['an', 'interpretation', ',', 'an', 'account', 'or', 'reading', 'of', 'an', 'event', 'or', 'situation'], ['an', 'imperfection', ';', 'the', 'quality', 'of', 'being', 'inadequate', 'or', 'falling', 'short', '.'], ['have', 'the', 'financial', 'means', 'to', 'obtain', ',', 'buy', ',', 'or', 'do', 'something'], ['supply', 'food', 'ready', 'to', 'eat', ',', 'as', 'for', 'parties', 'and', 'banquets', '.'], ['treat', 'a', 'substance', 'with', 'some', 'agent', ';', 'add', 'an', 'agent', 'to', '.'], ['an', 'injury', 'to', 'the', 'skin', 'caused', 'by', 'radiation', ',', 'heat', 'or', 'chemicals'], ['a', 'successful', 'throw', 'or', 'try', 'for', 'a', 'point', 'after', 'a', 'touchdown', '.'], ['a', 'short', 'theatrical', 'program', 'that', 'is', 'part', 'of', 'a', 'longer', 'program', '.'], ['the', 'stiching', 'that', 'forms', 'the', 'rim', 'of', 'a', 'shoe', 'or', 'boot', '.'], ['be', 'reflected', 'as', 'heat', ',', 'light', ',', 'or', 'shock', 'waves', ';', '.'], ['to', 'engage', 'in', 'plotting', 'or', 'activities', 'of', 'an', 'illegal', 'or', 'deceitful', 'nature'], ['the', 'act', 'or', 'event', 'of', 'ending', 'a', 'process', ',', 'state', ',', 'situation'], ['a', 'person', 'or', 'thing', 'equal', 'to', 'another', 'in', 'value', 'or', 'significane', '.'], ['avoid', 'or', 'try', 'to', 'avoid', 'fulfilling', ',', 'answering', ',', 'or', 'performing', '.'], ['the', 'primary', 'information-processing', 'component', 'of', 'a', 'computer', ',', 'of', 'a', 'microprocessor', 'chip'], ['put', 'an', 'end', 'to', ',', 'make', 'invalid', 'a', 'process', 'that', "'s", 'already', 'begun'], ['move', 'to', 'and', 'fro', 'around', 'a', 'particular', 'point', 'or', 'points', ',', 'remain', 'near'], ['shrivel', ',', 'wither', ',', 'or', 'mature', 'imperfectly', ',', 'as', 'of', 'a', 'flower', '.'], ['make', 'physical', 'contact', 'with', ',', 'possibly', 'with', 'the', 'effect', 'of', 'physically', 'manipulating', '.'], ['shine', 'intensely', ';', 'emit', 'a', 'steady', ',', 'even', 'light', 'from', 'a', 'source', '.'], ['the', 'act', 'of', 'transporting', 'goods', 'commerically', 'at', 'rates', 'cheaper', 'than', 'express', 'rates', '.'], ['join', 'or', 'bring', 'together', '(', 'two', ')', 'objects', ',', 'ideas', ',', 'or', 'people'], ['find', 'out', 'or', 'be', 'informed', 'of', 'a', 'new', 'information', ',', 'understand', 'or', 'realize'], ['(', 'cause', 'to', ')', 'lengthen', 'or', 'expand', 'in', 'time', 'or', 'some', 'abstract', 'realm'], ['express', 'audibly', ';', 'utter', 'sounds', '(', 'not', 'necessarily', 'words', ')', ';', 'indicate', '.'], ['fine', 'powdery', 'material', ';', 'the', 'remains', 'of', 'something', 'destroyed', 'or', 'broken', 'up', '.'], ['give', 'oneself', 'entirely', 'to', 'a', 'specific', 'person', ',', 'activity', ',', 'or', 'cause', '.'], ['(', 'medicine', ')', 'the', 'capacity', 'of', 'an', 'organism', 'to', 'defend', 'against', 'disease', '.'], ['formally', 'reject', 'or', 'disavow', 'a', 'formerly', 'held', 'belief', ';', 'retract', 'a', 'statement', '.'], ['(', 'an', 'occurance', 'of', ')', 'the', 'preservation', 'or', 'management', 'of', 'natural', 'resources', '.'], ['(', 'cause', 'to', ')', 'swing', 'and', 'miss', 'the', 'third', 'pitch', ',', 'striking', 'out'], ['a', 'gear', 'for', 'changing', 'the', 'motion', 'of', 'a', 'machine', 'to', 'the', 'opposite', 'direction'], ['raising', 'the', 'stakes', 'or', 'bet', 'of', 'a', 'game', 'by', 'a', 'factor', 'of', '2'], ['a', 'chart', 'or', 'map', 'showing', 'the', 'dimensions', ',', 'movements', 'or', 'progress', 'of', 'something'], ['value', 'measured', 'by', 'what', 'must', 'be', 'given', 'or', 'done', 'to', 'obtain', 'something', '.'], ['the', 'period', 'of', 'time', 'during', 'which', 'a', 'leasing', 'contract', 'is', 'in', 'effect', '.'], ['point', 'or', 'direct', 'object', ',', 'weapon', ',', 'or', 'blow', 'at', 'something', 'or', 'someone'], ['surround', 'completely', ';', 'close', 'in', ',', 'as', 'if', 'with', 'a', 'covering', 'or', 'border', '.'], ['signal', '(', 'approval', 'or', 'interest', ')', 'by', 'winking', ';', 'briefly', 'shut', 'one', 'eye', '.'], ['cite', 'as', 'an', 'authority', ';', 'resort', 'to', '(', 'as', 'in', 'an', 'argument', ')', '.'], ['look', 'through', 'a', 'book', 'or', 'some', 'other', 'written', 'material', '(', 'for', 'information', ')', '.'], ['(', 'chemistry', ')', 'undergo', 'sequestration', 'by', 'forming', 'a', 'stable', 'compound', 'with', 'an', 'ion', '.'], ['(', 'mathematics', ')', 'the', 'smallest', 'whole', 'number', 'or', 'a', 'numeral', 'representing', 'this', 'number', '.'], ['the', 'state', 'of', 'believing', 'in', ',', 'or', 'having', 'confidence', 'in', ',', 'something', 'or', 'someone'], ['reimburse', 'or', 'compensate', '(', 'someone', ')', ',', 'as', 'for', 'a', 'loss', 'or', 'sacrifice', '.'], ['make', 'fine', 'adjustments', 'to', 'for', 'optimal', 'operation', ';', 'adjust', 'to', 'a', 'certain', 'scale', '.'], ['convert', 'into', 'absorbable', 'substances', ',', '(', 'as', 'if', ')', 'with', 'heat', 'or', 'chemical', 'process'], ['(', 'theology', ')', 'the', 'act', 'of', 'deliving', 'from', 'sin', 'or', 'saving', 'from', 'evil', '.'], ['furnish', 'with', 'an', 'opening', 'to', 'allow', 'air', 'to', 'circulate', 'or', 'gase', 'to', 'escape', '.'], ['a', 'list', 'of', 'candidates', 'nominated', 'by', 'a', 'political', 'party', 'to', 'run', 'for', 'election', '.'], ['make', 'a', 'line', 'or', 'marks', 'on', 'a', 'surface', ';', 'copy', 'by', 'following', 'the', 'lines', 'of'], ['allocate', 'or', 'dedicate', ';', 'set', 'aside', 'or', 'apart', 'for', 'a', 'specific', 'purpose', 'or', 'use', '.'], ['lead', 'someone', 'in', 'the', 'wrong', 'direction', '(', 'physically', ')', ';', 'give', 'wrong', 'spatial', 'directions', '.'], ['supply', 'with', 'water', 'for', 'crops', ',', 'as', 'with', 'channels', ',', 'ditches', ',', 'or', 'streams', '.'], ['remove', 'by', 'erasing', 'or', 'crossing', 'out', ',', 'as', 'if', 'by', 'drawing', 'a', 'line', 'through', '.'], ['(', 'computer', 'science', ')', 'the', 'occurence', 'of', 'an', 'incorrect', 'result', 'produced', 'by', 'a', 'computer', '.'], ['grip', ',', 'penetrate', 'or', 'tear', 'off', 'with', 'or', 'as', 'if', 'with', 'the', 'teeth', 'or', 'jaws'], ['a', 'device', 'to', 'control', 'the', 'rate', 'of', 'some', 'activity', ',', 'e.g.', ',', 'chemical', 'or', 'mechanical'], ['convert', 'the', 'genetic', 'information', 'in', 'a', 'strand', 'of', 'dna', 'into', 'a', 'strand', 'of', 'rna', '.'], ['the', 'ability', 'of', 'computers', 'to', 'exchange', 'digital', 'information', 'between', 'them', 'and', 'make', 'use', 'of', 'it'], ['spin', ',', 'wind', ',', 'braid', ',', 'or', 'twist', 'together', ',', 'perhaps', 'as', 'when', 'performing', 'lacework', '.'], ['reply', 'verbally', 'or', 'in', 'written', 'form', 'to', 'a', 'question', ',', 'comment', ',', 'letter', ',', 'or', 'speech'], ['speak', 'in', 'a', 'high-pitched', 'tone', 'of', 'voice', ',', 'as', 'either', 'an', 'animal', 'or', 'human', 'might', '.'], ['(', 'cause', 'to', ')', 'become', 'different', 'or', 'aquire', 'a', 'different', 'character', ',', 'possibly', 'during', 'development', '.'], ['put', 'up', 'with', 'something', 'or', 'someone', 'unpleasant', ';', 'show', 'deference', 'towards', 'someone', 'or', 'some', 'authority', '.'], ['reproduce', 'or', 'make', 'an', 'exact', 'copy', 'of', ';', 'make', ',', 'do', ',', 'or', 'perform', 'again', '.'], ['pass', '(', 'a', 'thread', ')', 'through', 'or', 'into', ',', 'on', 'or', 'as', 'if', 'on', 'a', 'string'], ['laying', 'claim', ',', 'the', 'act', 'of', 'taking', 'possession', 'of', ',', 'or', 'power', 'over', ',', 'something', '.'], ['immerse', 'or', 'be', 'immersed', 'in', 'a', 'boiling', 'liquid', ',', 'for', 'purposes', 'of', 'cooking', 'or', 'processing', '.'], ['the', 'social', 'force', 'that', 'binds', 'one', 'to', 'the', 'courses', 'of', 'action', 'demanded', 'by', 'that', 'force', '.'], ['(', 'the', 'discovery', 'or', 'exhibition', 'of', ')', 'factual', 'evidence', 'that', 'establishes', 'the', 'truth', 'of', 'something', '.'], ['a', 'variant', 'of', 'a', 'musical', 'theme', 'which', 'employs', 'notes', 'of', 'reduced', 'duration', 'from', 'the', 'original', 'theme'], ['change', 'the', 'key', ',', 'pitch', ',', 'tone', ',', 'or', 'volume', 'of', ',', 'likely', 'for', 'aesthetic', 'reasons', '.'], ['approach', 'a', 'limit', 'as', 'the', 'number', 'of', 'terms', 'increases', 'without', 'limit', ';', 'specifically', 'a', 'mathematical', 'term', '.'], ['behave', 'violently', ',', 'as', 'if', 'in', 'a', 'state', 'of', 'great', 'anger', ';', 'break', 'or', 'tear', 'violently', '.'], ['skip', ',', 'leap', ',', 'or', 'move', 'up', 'and', 'down', 'or', 'sideways', 'as', 'a', 'manner', 'of', 'motion', '.'], ['the', '(', 'period', 'of', ')', 'control', 'of', 'a', 'country', 'by', 'military', 'forces', 'of', 'a', 'foreign', 'power', '.'], ['to', 'move', '(', 'the', 'head', 'or', 'body', ')', 'quickly', 'downwards', 'or', 'away', '(', 'from', 'something', ')', '.'], ['the', 'part', 'of', 'the', 'body', 'between', 'the', 'neck', 'and', 'upper', 'arm', ',', 'or', 'any', 'of', 'its', 'subparts', '.'], ['try', 'to', 'cure', 'by', 'special', 'care', 'or', 'treatment', ',', 'of', 'an', 'illness', 'or', 'injury', ';', 'treat', 'carefully', '.'], ['(', 'cause', 'to', ')', 'move', 'through', 'a', 'space', ',', 'circuit', 'or', 'system', ',', 'usu.', 'returning', 'to', 'the', 'starting', 'point'], ['give', 'an', 'incentive', 'for', 'action', ',', 'provoke', 'or', 'stir', 'up', ',', 'urge', 'on', ',', 'or', 'cause', 'to', 'act', '.'], ['a', 'coefficient', 'assigned', 'to', 'elements', 'in', 'a', 'frequency', 'distribution', 'in', 'order', 'to', 'indicate', 'the', 'relative', 'importance', 'of', 'each', 'element'], ['be', 'larger', 'in', 'number', ',', 'quantity', ',', 'power', ',', 'status', ',', 'or', 'importance', ',', 'without', 'personally', 'having', 'sovereign', 'power', '.'], ['supply', 'with', 'a', 'constant', 'flow', 'of', 'some', 'liquid', ',', 'for', 'the', 'purpose', 'of', 'cooling', ',', 'cleansing', ',', 'or', 'disinfecting', '.'], ['droop', ',', 'sink', ',', 'or', 'settle', 'from', 'or', 'as', 'if', 'from', 'pressure', 'or', 'loss', 'of', 'tautness', ';', 'become', 'less', 'intense', '.'], ['rain', ',', 'hail', ',', 'or', 'snow', 'hard', 'and', 'be', 'very', 'windy', ',', 'often', 'with', 'thunder', 'and', 'lightning', ';', 'blow', 'hard', '.'], ['amplify', 'an', 'electron', 'current', 'by', 'causing', 'part', 'of', 'the', 'power', 'on', 'the', 'output', 'circuit', 'to', 'act', 'upon', 'the', 'input', 'circuit', '.'], ['(', 'contract', 'bridge', ')', 'the', 'highest', 'bid', 'becomes', 'the', 'contract', 'setting', 'in', 'the', 'number', 'of', 'tricks', 'that', 'the', 'bidder', 'must', 'make', '.'], ['release', '(', 'gas', 'or', 'energy', ')', 'as', 'a', 'result', 'of', 'a', 'chemical', 'reaction', 'or', 'physical', 'decomposition', '(', 'perhaps', 'in', 'a', 'metaphorical', 'sense', ')'], ['(', 'science', ')', 'the', 'atomic', 'weight', 'of', 'an', 'element', 'that', 'has', 'the', 'same', 'combining', 'capacity', 'as', 'a', 'given', 'weight', 'of', 'another', 'element', '.'], ['make', 'a', 'map', 'of', ';', 'show', 'or', 'establish', 'the', 'features', 'or', 'details', 'of', ';', 'explore', 'or', 'survey', 'for', 'the', 'purposes', 'of', 'making', 'a', 'map', '.'], ['either', 'of', 'the', 'two', 'categories', '(', 'male', 'or', 'female', ')', 'into', 'which', 'most', 'organisms', 'are', 'divided', ',', 'or', 'key', 'properties', 'that', 'distinguish', 'the', 'categories', '.'], ['a', 'physical', 'portion', 'of', 'all', 'or', 'part', 'of', 'a', 'natural', 'object', 'that', 'is', 'collected', 'and', 'preserved', 'as', 'an', 'example', 'of', 'its', 'class', 'or', 'total', '.'], ['botany', '--', 'the', 'usually', 'underground', 'organ', 'of', 'a', 'plant', 'that', 'lacks', 'buds', 'or', 'leaves', 'or', 'nodes', ',', 'which', 'absorbs', 'water', 'and', 'mineral', 'salts', ';', 'usually', 'it', 'anchors', 'the', 'plant', 'to', 'the', 'ground', '.'], ['put', 'an', 'end', 'to', '.'], ['logical', 'or', 'comprehensible', 'arrangement', 'of', 'separate', 'elements', '.'], ['change', 'directions', 'as', 'if', 'revolving', 'on', 'a', 'pivot', '.'], ['a', 'substance', 'that', 'is', 'used', 'as', 'a', 'medicine', 'or', 'narcotic', '.'], ['become', 'inflated', '.'], ['become', 'due', 'for', 'repayment', '.'], ['be', 'the', 'culminating', 'event', '.'], ['request', 'urgently', 'and', 'forcefully', '.'], ['an', 'offer', 'of', 'marriage', '.'], ['the', 'trade', 'of', 'a', 'funeral', 'director', '.'], ['make', '(', 'a', 'surface', ')', 'shine', '.'], ['military', 'personnel', 'lost', 'by', 'death', 'or', 'capture', '.'], ['a', 'proportion', 'in', 'relation', 'to', 'a', 'whole', '(', 'which', 'is', 'usually', 'the', 'amount', 'per', 'hundred', ')', '.'], ['a', 'voluntary', 'gift', '(', 'as', 'of', 'money', 'or', 'service', 'or', 'ideas', ')', 'made', 'to', 'some', 'worthwhile', 'cause', '.'], ['indistinct', 'articulation', '.'], ['hunt', 'with', 'hawks', '.'], ['an', 'arrangement', 'scheme', '.'], ['duplicate', 'or', 'match', '.'], ['provide', 'housing', 'for', '.'], ['hunt', 'or', 'shoot', 'snipe', '.'], ['give', 'a', 'structure', 'to', '.'], ['have', 'sexual', 'intercourse', 'with', '.'], ['mark', 'with', 'a', 'scar', '.'], ['a', 'separate', 'and', 'self-contained', 'entity', '.'], ['the', 'time', 'yet', 'to', 'come', '.'], ['become', 'pregnant', ';', 'undergo', 'conception', '.'], ['shoot', 'down', ',', 'of', 'birds', '.'], ['the', 'general', 'activity', 'of', 'selling', '.'], ['the', 'act', 'of', 'constructing', 'something', '.'], ['utter', 'with', 'force', ';', 'utter', 'vehemently', '.'], ['cover', 'the', 'front', 'or', 'surface', 'of', '.'], ['beat', 'severely', 'with', 'a', 'whip', 'or', 'rod', '.'], ['increase', 'rapidly', 'and', 'in', 'an', 'uncontrolled', 'manner', '.'], ['the', 'act', 'of', 'apportioning', 'or', 'distributing', 'something', '.'], ['the', 'large', 'room', 'of', 'a', 'manor', 'or', 'castle', '.'], ['observe', 'with', 'care', 'or', 'pay', 'close', 'attention', 'to', '.'], ['place', 'limits', 'on', '(', 'extent', 'or', 'access', ')', '.'], ['establish', 'or', 'strengthen', 'as', 'with', 'new', 'evidence', 'or', 'facts', '.'], ['remove', 'by', 'or', 'as', 'if', 'by', 'rubbing', 'or', 'erasing', '.'], ['a', 'force', 'that', 'is', 'a', 'branch', 'of', 'the', 'armed', 'forces', '.'], ['a', 'short', 'theatrical', 'performance', 'that', 'is', 'part', 'of', 'a', 'longer', 'program', '.'], ['a', 'denture', 'anchored', 'to', 'teeth', 'on', 'either', 'side', 'of', 'missing', 'teeth', '.'], ['office', 'furniture', 'consisting', 'of', 'a', 'container', 'for', 'keeping', 'papers', 'in', 'order', '.'], ['do', 'duty', 'or', 'hold', 'offices', ';', 'serve', 'in', 'a', 'specific', 'function', '.'], ['forbid', 'the', 'public', 'distribution', 'of', '(', 'a', 'movie', 'or', 'a', 'newspaper', ')', '.'], ['the', 'goal', 'intended', 'to', 'be', 'attained', '(', 'and', 'which', 'is', 'believed', 'to', 'be', 'attainable', ')', '.'], ['(', 'computer', 'science', ')', 'a', 'computer', 'file', 'that', 'contains', 'text', '(', 'and', 'possibly', 'formatting', 'instructions', ')', 'using', 'seven-bit', 'ascii', 'characters', '.'], ['(', 'computer', 'science', ')', 'a', 'computer', 'that', 'provides', 'client', 'stations', 'with', 'access', 'to', 'files', 'and', 'printers', 'as', 'shared', 'resources', 'to', 'a', 'computer', 'network', '.'], ['be', 'opposite', '.'], ['start', 'abruptly', '.'], ['throw', 'quickly', '.'], ['issue', 'bonds', 'on', '.'], ['cross', 'on', 'foot', '.'], ['an', 'unforeseen', 'development', '.'], ['destroy', 'a', 'ship', '.'], ['a', 'quantity', 'of', 'money', '.'], ['clean', 'with', 'hard', 'rubbing', '.'], ['sniff', 'or', 'smell', 'inquiringly', '.'], ['make', 'hard', 'or', 'harder', '.'], ['the', 'act', 'of', 'sucking', '.'], ['become', 'long', 'or', 'longer', '.'], ['make', 'thick', 'or', 'thicker', '.'], ['adhering', 'to', 'moral', 'principles', '.'], ['become', 'hard', 'or', 'harder', '.'], ['a', 'change', 'for', 'the', 'better', '.'], ['the', 'property', 'of', 'copious', 'abundance', '.'], ['persuade', 'somebody', 'to', 'accept', 'something', '.'], ['decorative', 'cover', 'for', 'a', 'bed', '.'], ['an', 'extended', 'area', 'of', 'land', '.'], ['have', 'as', 'a', 'logical', 'consequence', '.'], ['a', 'committee', 'having', 'supervisory', 'powers', '.'], ['a', 'woman', "'s", 'sleeveless', 'undergarment', '.'], ['the', 'actors', 'in', 'a', 'play', '.'], ['the', 'day', 'immediately', 'before', 'today', '.'], ['the', 'state', 'of', 'being', 'active', '.'], ['appoint', 'to', 'a', 'clerical', 'posts', '.'], ['attract', ';', 'cause', 'to', 'be', 'enamored', '.'], ['cause', 'to', 'throb', 'or', 'beat', 'rapidly', '.'], ['loss', 'of', 'ability', 'to', 'function', 'normally', '.'], ['the', 'chief', 'executive', 'of', 'a', 'republic', '.'], ['act', 'against', 'or', 'in', 'opposition', 'to', '.'], ['express', 'or', 'utter', 'with', 'a', 'hiss', '.'], ['the', 'act', 'of', 'applying', 'force', 'suddenly', '.'], ['be', 'pertinent', 'or', 'relevant', 'or', 'applicable', '.'], ['contain', 'or', 'hold', ';', 'have', 'within', '.'], ['provide', 'with', 'a', 'toggle', 'or', 'toggles', '.'], ['make', 'a', 'clicking', 'or', 'ticking', 'sound', '.'], ['pass', 'over', ',', 'across', ',', 'or', 'through', '.'], ['an', 'event', 'that', 'accomplishes', 'its', 'intended', 'purpose', '.'], ['become', 'engaged', 'or', 'intermeshed', 'with', 'one', 'another', '.'], ['an', 'acute', 'febrile', 'highly', 'contagious', 'viral', 'disease', '.'], ['furnish', 'with', 'rubrics', 'or', 'regulate', 'by', 'rubrics', '.'], ['suffer', 'failure', ',', 'as', 'in', 'some', 'enterprise', '.'], ['something', 'that', 'is', 'made', 'real', 'or', 'concrete', '.'], ['the', 'quality', 'of', 'being', 'unlike', 'or', 'dissimilar', '.'], ['the', 'act', 'of', 'informing', 'by', 'verbal', 'report', '.'], ['use', 'up', '(', 'resources', 'or', 'materials', ')', '.'], ['a', 'prediction', 'of', 'the', 'course', 'of', 'a', 'disease', '.'], ['the', 'act', 'of', 'changing', 'the', 'location', 'of', 'something', '.'], ['a', 'part', 'of', 'a', 'forked', 'or', 'branching', 'shape', '.'], ['fill', 'with', 'high', 'spirits', ';', 'fill', 'with', 'optimism', '.'], ['some', 'situation', 'or', 'event', 'that', 'is', 'thought', 'about', '.'], ['an', 'image', 'that', 'is', 'generated', 'by', 'a', 'computer', '.'], ['give', 'to', ',', 'and', 'receive', 'from', ',', 'one', 'another', '.'], ['secrete', 'or', 'form', 'water', ',', 'as', 'tears', 'or', 'saliva', '.'], ['the', 'state', 'of', 'being', 'joined', 'or', 'united', 'or', 'linked', '.'], ['spend', 'time', 'in', 'prison', 'or', 'in', 'a', 'labor', 'camp', '.'], ['equip', 'with', 'a', 'fuse', ';', 'provide', 'with', 'a', 'fuse', '.'], ['earnest', 'and', 'conscientious', 'activity', 'intended', 'to', 'do', 'or', 'accomplish', 'something', '.'], ['pass', 'through', 'the', 'esophagus', 'as', 'part', 'of', 'eating', 'or', 'drinking', '.'], ['a', 'loud', 'utterance', 'of', 'emotion', '(', 'especially', 'when', 'inarticulate', ')', '.'], ['bring', '(', 'several', 'things', ')', 'into', 'consonance', 'or', 'relate', 'harmoniously', '.'], ['pursue', 'for', 'food', 'or', 'sport', '(', 'as', 'of', 'wild', 'animals', ')', '.'], ['move', 'or', 'sway', 'in', 'a', 'rising', 'and', 'falling', 'or', 'wavelike', 'pattern', '.'], ['something', 'craved', ',', 'especially', 'an', 'intravenous', 'injection', 'of', 'a', 'narcotic', 'drug', '.'], ['be', 'careful', 'or', 'certain', 'to', 'do', 'something', ';', 'make', 'certain', 'of', 'something', '.'], ['move', 'along', 'on', 'or', 'as', 'if', 'on', 'wheels', 'or', 'a', 'wheeled', 'vehicle', '.'], ['cause', 'to', 'move', ';', 'cause', 'to', 'be', 'in', 'a', 'certain', 'position', 'or', 'condition', '.'], ['run', 'or', 'flow', 'slowly', ',', 'as', 'in', 'drops', 'or', 'in', 'an', 'unsteady', 'stream', '.'], ['the', 'relative', 'position', 'or', 'standing', 'of', 'things', 'or', 'especially', 'persons', 'in', 'a', 'society', '.'], ['the', 'goal', 'intended', 'to', 'be', 'attained', '(', 'and', 'which', 'is', 'believed', 'to', 'be', 'attainable', ')', '.'], ['a', 'passageway', 'through', 'or', 'under', 'something', ',', 'usually', 'underground', '(', 'especially', 'one', 'for', 'trains', 'or', 'cars', ')', '.'], ['make', 'impure', '.'], ['a', 'stronghold', '.'], ['examine', 'minutely', '.'], ['enter', 'uninvited', '.'], ['make', 'new', '.'], ['look', 'furtively', '.'], ['a', 'problem', '.'], ['furnish', 'with', 'spars', '.'], ['produce', 'a', 'click', '.'], ['move', 'very', 'fast', '.'], ['issue', 'an', 'order', '.'], ['overcome', 'with', 'amazement', '.'], ['feel', 'physical', 'pain', '.'], ['learn', 'by', 'repetition', '.'], ['make', 'reference', 'to', '.'], ['stretch', 'out', 'completely', '.'], ['put', 'a', 'harness', '.'], ['provide', 'veterinary', 'care', 'for', '.'], ['work', 'with', 'a', 'tool', '.'], ['fix', 'to', ';', 'attach', '.'], ['the', 'act', 'of', 'swallowing', '.'], ['write', 'a', 'computer', 'program', '.'], ['an', 'electronic', 'memory', 'device', '.'], ['be', 'a', 'summary', 'of', '.'], ['open', 'again', 'or', 'anew', '.'], ['a', 'single', 'chance', 'or', 'instance', '.'], ['cause', 'to', 'move', 'by', 'pulling', '.'], ['a', 'boxing', 'or', 'wrestling', 'match', '.'], ['informal', 'terms', 'for', 'the', 'leg', '.'], ['the', 'act', 'of', 'retaining', 'something', '.'], ['assign', 'to', 'a', 'specific', 'task', '.'], ['an', 'onerous', 'or', 'difficult', 'concern', '.'], ['admit', 'to', 'testing', 'or', 'proof', '.'], ['someone', 'who', 'cleans', 'soot', 'from', 'chimneys', '.'], ['the', 'characteristic', 'utterance', 'of', 'an', 'animal', '.'], ['provide', 'a', 'feast', 'or', 'banquet', 'for', '.'], ['partake', 'in', 'a', 'feast', 'or', 'banquet', '.'], ['take', 'apart', 'into', 'its', 'constituent', 'pieces', '.'], ['allow', 'the', 'passage', 'of', 'air', 'through', '.'], ['a', 'single', 'serving', 'of', 'a', 'beverage', '.'], ['any', 'accounting', 'period', 'of', '12', 'months', '.'], ['a', 'sudden', 'sharp', 'decrease', 'in', 'some', 'quantity', '.'], ['give', 'to', 'a', 'charity', 'or', 'good', 'cause', '.'], ['a', 'kitchen', 'appliance', 'used', 'for', 'cooking', 'food', '.'], ['behave', 'in', 'a', 'patronizing', 'and', 'condescending', 'manner', '.'], ['relative', 'darkness', 'or', 'lightness', 'of', 'a', 'color', '.'], ['speed', 'up', 'the', 'progress', 'of', ';', 'facilitate', '.'], ['put', 'up', 'with', 'something', 'or', 'somebody', 'unpleasant', '.'], ['make', 'clear', 'and', '(', 'more', ')', 'comprehensible', '.'], ['improve', 'or', 'perfect', 'by', 'pruning', 'or', 'polishing', '.'], ['register', 'formally', 'as', 'a', 'participant', 'or', 'member', '.'], ['show', 'devotion', 'to', '(', 'a', 'deity', ')', '.'], ['the', 'activity', 'of', 'supplying', 'or', 'providing', 'something', '.'], ['narrate', 'or', 'give', 'a', 'detailed', 'account', 'of', '.'], ['the', 'successful', 'ending', 'of', 'the', 'american', 'revolution', '.'], ['reason', 'by', 'deduction', ';', 'establish', 'by', 'deduction', '.'], ['enclose', 'in', 'or', 'as', 'if', 'in', 'a', 'frame', '.'], ['an', 'angular', 'or', 'rounded', 'shape', 'made', 'by', 'folding', '.'], ['be', 'incompatible', ';', 'be', 'or', 'come', 'into', 'conflict', '.'], ['make', 'more', 'complex', ',', 'intricate', ',', 'or', 'richer', '.'], ['the', 'conscious', 'subjective', 'aspect', 'of', 'feeling', 'or', 'emotion', '.'], ['british', 'term', 'for', 'the', 'luggage', 'compartment', 'in', 'a', 'car', '.'], ['give', 'a', 'definition', 'for', 'the', 'meaning', 'of', 'a', 'word', '.'], ['a', 'fourth', 'part', 'of', 'a', 'year', ';', 'three', 'months', '.'], ['a', 'message', 'refusing', 'to', 'accept', 'something', 'that', 'is', 'offered', '.'], ['anything', 'that', 'approximates', 'the', 'shape', 'of', 'a', 'column', 'or', 'tower', '.'], ['cut', 'the', 'surface', 'of', ';', 'wear', 'away', 'the', 'surface', 'of', '.'], ['have', 'trust', 'in', ';', 'trust', 'in', 'the', 'truth', 'or', 'veracity', 'of', '.'], ['give', 'entirely', 'to', 'a', 'specific', 'person', ',', 'activity', ',', 'or', 'cause', '.'], ['cancel', ',', 'annul', ',', 'or', 'reverse', 'an', 'action', 'or', 'its', 'effect', '.'], ['an', 'occasion', 'on', 'which', 'people', 'can', 'assemble', 'for', 'social', 'interaction', 'and', 'entertainment', '.'], ['the', 'component', 'of', 'the', 'aerodynamic', 'forces', 'acting', 'on', 'an', 'airfoil', 'that', 'opposes', 'gravity', '.'], ['the', 'act', 'of', 'caressing', 'with', 'the', 'lips', '(', 'or', 'an', 'instance', 'thereof', ')', '.'], ['incite', ',', 'move', ',', 'or', 'persuade', 'to', 'some', 'act', 'of', 'lawlessness', 'or', 'insubordination', '.'], ['something', 'that', 'provides', 'direction', 'or', 'advice', 'as', 'to', 'a', 'decision', 'or', 'course', 'of', 'action', '.'], ['move', 'to', 'and', 'fro', 'or', 'from', 'place', 'to', 'place', 'usually', 'in', 'an', 'irregular', 'course', '.'], ['reduce', 'or', 'cause', 'to', 'be', 'reduced', 'from', 'a', 'solid', 'to', 'a', 'liquid', 'state', ',', 'usually', 'by', 'heating', '.'], ['(', 'nautical', ')', 'brace', 'consisting', 'of', 'a', 'heavy', 'rope', 'or', 'wire', 'cable', 'used', 'as', 'a', 'support', 'for', 'a', 'mast', 'or', 'spar', '.'], ['any', 'of', 'the', 'equal', 'portions', 'into', 'which', 'the', 'capital', 'stock', 'of', 'a', 'corporation', 'is', 'divided', 'and', 'ownership', 'of', 'which', 'is', 'evidenced', 'by', 'a', 'stock', 'certificate', '.'], ['touch', 'with', 'the', 'lips', 'or', 'press', 'the', 'lips', '(', 'against', 'someone', "'s", 'mouth', 'or', 'other', 'body', 'part', ')', 'as', 'an', 'expression', 'of', 'love', ',', 'greeting', ',', 'etc', '..'], ['treat', 'the', 'body', 'or', 'any', 'part', 'of', 'it', 'by', 'wrapping', 'it', ',', 'as', 'with', 'blankets', 'or', 'sheets', ',', 'and', 'applying', 'compresses', 'to', 'it', ',', 'or', 'stuffing', 'it', 'to', 'provide', 'cover', ',', 'containment', ',', 'or', 'therapy', ',', 'or', 'to', 'absorb', 'blood', '.'], ['catch', 'fire', '.'], ['remove', 'water', 'from', '.'], ['hit', 'a', 'fly', '.'], ['maintain', 'or', 'assert', '.'], ['allow', 'to', 'have', '.'], ['sway', 'to', 'and', 'fro', '.'], ['sharpen', 'with', 'a', 'strap', '.'], ['a', 'usually', 'brief', 'attempt', '.'], ['increase', 'the', 'volume', 'of', '.'], ['make', 'a', 'sharp', 'sound', '.'], ['tell', 'or', 'spread', 'rumors', '.'], ['fail', 'to', 'attend', 'to', '.'], ['make', 'unfit', 'or', 'unsuitable', '.'], ['get', 'or', 'gather', 'together', '.'], ['unrestricted', 'freedom', 'to', 'use', '.'], ['make', 'lighter', 'or', 'brighter', '.'], ['the', 'state', 'of', 'being', 'connected', '.'], ['have', 'faith', 'or', 'confidence', 'in', '.'], ['a', 'mold', 'for', 'setting', 'concrete', '.'], ['return', 'to', 'a', 'former', 'condition', '.'], ['determine', 'the', 'essential', 'quality', 'of', '.'], ['feel', 'the', 'need', 'to', 'eat', '.'], ['an', 'unpleasant', 'or', 'disappointing', 'surprise', '.'], ['someone', 'who', 'controls', 'resources', 'and', 'expenditures', '.'], ['make', 'a', 'certain', 'noise', 'or', 'sound', '.'], ['go', 'back', 'to', 'a', 'previous', 'state', '.'], ['cause', 'to', 'move', 'round', 'and', 'round', '.'], ['be', 'the', 'host', 'of', 'or', 'for', '.'], ['cause', 'to', 'be', 'alert', 'and', 'energetic', '.'], ['have', 'unlawful', 'sex', 'with', 'a', 'whore', '.'], ['become', 'unfit', 'for', 'consumption', 'or', 'use', '.'], ['the', 'head', 'of', 'a', 'city', 'government', '.'], ['make', 'up', 'something', 'artificial', 'or', 'untrue', '.'], ['a', 'competitor', 'thought', 'likely', 'to', 'win', '.'], ['make', 'something', 'more', 'diverse', 'and', 'varied', '.'], ['the', 'introductory', 'section', 'of', 'a', 'story', '.'], ['any', 'accounting', 'period', 'of', '12', 'months', '.'], ['interfere', 'in', 'someone', 'else', "'s", 'activity', '.'], ['cause', 'to', 'become', 'awake', 'or', 'conscious', '.'], ['contrast', 'with', 'equal', 'weight', 'or', 'force', '.'], ['the', 'status', 'of', 'being', 'a', 'champion', '.'], ['an', 'impetuous', 'rush', 'toward', 'someone', 'or', 'something', '.'], ['(', 'construction', ')', 'a', 'layer', 'of', 'masonry', '.'], ['the', 'successful', 'action', 'of', 'solving', 'a', 'problem', '.'], ['start', 'to', 'burn', 'or', 'burst', 'into', 'flames', '.'], ['return', 'in', 'thought', 'or', 'speech', 'to', 'something', '.'], ['engrave', 'by', 'means', 'of', 'dots', 'and', 'flicks', '.'], ['give', 'a', 'thrashing', 'to', ';', 'beat', 'hard', '.'], ['the', 'face', 'or', 'front', 'of', 'a', 'building', '.'], ['take', 'up', 'someone', "'s", 'soul', 'into', 'heaven', '.'], ['heat', 'a', 'metal', 'prior', 'to', 'working', 'it', '.'], ['a', 'soft', 'pad', 'placed', 'under', 'a', 'saddle', '.'], ['pass', 'into', 'a', 'specified', 'state', 'or', 'condition', '.'], ['to', 'become', 'aware', 'of', 'through', 'the', 'senses', '.'], ['make', 'the', 'surface', 'of', 'level', 'or', 'smooth', '.'], ['a', 'cause', 'of', 'pain', 'or', 'injury', 'or', 'loss', '.'], ['store', '(', 'liquids', 'or', 'gases', ')', 'in', 'bottles', '.'], ['clear', 'mucus', 'or', 'food', 'from', 'one', "'s", 'throat', '.'], ['protect', 'or', 'strengthen', 'with', 'sandbags', ';', 'stop', 'up', '.'], ['the', 'particular', 'portion', 'of', 'space', 'occupied', 'by', 'something', '.'], ['pause', 'or', 'hold', 'back', 'in', 'uncertainty', 'or', 'unwillingness', '.'], ['scrape', 'or', 'rub', 'as', 'if', 'to', 'relieve', 'itching', '.'], ['feel', 'or', 'have', 'a', 'desire', 'for', ';', 'want', 'strongly', '.'], ['a', 'porch', 'that', 'resembles', 'the', 'deck', 'on', 'a', 'ship', '.'], ['be', 'in', 'direct', 'physical', 'contact', 'with', ';', 'make', 'contact', '.'], ['a', 'judgment', 'of', 'the', 'qualities', 'of', 'something', 'or', 'somebody', '.'], ['turn', 'on', 'or', 'around', 'an', 'axis', 'or', 'a', 'center', '.'], ['get', 'or', 'find', 'back', ';', 'recover', 'the', 'use', 'of', '.'], ['touch', ',', 'lift', ',', 'or', 'hold', 'with', 'the', 'hands', '.'], ['the', 'state', 'of', 'owing', 'something', '(', 'especially', 'money', ')', '.'], ['exercise', ',', 'or', 'have', 'the', 'power', 'of', ',', 'memory', '.'], ['a', 'motorboat', 'with', 'an', 'open', 'deck', 'or', 'a', 'half', 'deck', '.'], ['a', 'gradual', 'decrease', ';', 'as', 'of', 'stored', 'charge', 'or', 'current', '.'], ['reduce', '(', 'petroleum', ')', 'to', 'a', 'simpler', 'compound', 'by', 'cracking', '.'], ['a', 'public', 'secondary', 'school', 'usually', 'including', 'grades', '9', 'through', '12', '.'], ['restate', '(', 'words', ')', 'from', 'one', 'language', 'into', 'another', 'language', '.'], ['a', 'set', 'sequence', 'of', 'steps', ',', 'part', 'of', 'larger', 'computer', 'program', '.'], ['hitting', 'a', 'golf', 'ball', 'off', 'of', 'a', 'tee', 'with', 'a', 'driver', '.'], ['the', 'act', 'of', 'applying', 'force', 'in', 'order', 'to', 'move', 'something', 'away', '.'], ['a', 'journey', 'for', 'some', 'purpose', '(', 'usually', 'including', 'the', 'return', ')', '.'], ['a', 'specific', 'size', 'and', 'style', 'of', 'type', 'within', 'a', 'type', 'family', '.'], ['fill', 'or', 'close', 'tightly', 'with', 'or', 'as', 'if', 'with', 'a', 'plug', '.'], ['recite', 'with', 'musical', 'intonation', ';', 'recite', 'as', 'a', 'chant', 'or', 'a', 'psalm', '.'], ['either', 'of', 'the', 'two', 'halves', 'of', 'a', 'bow', 'from', 'handle', 'to', 'tip', '.'], ['strike', 'out', '(', 'a', 'batter', ')', ',', '(', 'of', 'a', 'pitcher', ')', '.'], ['cause', 'to', 'move', ';', 'cause', 'to', 'be', 'in', 'a', 'certain', 'position', 'or', 'condition', '.'], ['a', 'sharp', 'side', 'formed', 'by', 'the', 'intersection', 'of', 'two', 'surfaces', 'of', 'an', 'object', '.'], ['a', 'courteous', 'expression', '(', 'by', 'word', 'or', 'deed', ')', 'of', 'esteem', 'or', 'regard', '.'], ['a', 'protective', 'covering', 'consisting', 'of', 'netting', ';', 'can', 'be', 'mounted', 'in', 'a', 'frame', '.'], ['affect', 'with', 'smut', 'or', 'mildew', ',', 'as', 'of', 'a', 'crop', 'such', 'as', 'corn', '.'], ['(', 'often', 'followed', 'by', '`of', "'", ')', 'a', 'large', 'number', 'or', 'amount', 'or', 'extent', '.'], ['assets', 'belonging', 'to', 'or', 'due', 'to', 'or', 'contributed', 'by', 'an', 'individual', 'person', 'or', 'group', '.'], ['consider', 'in', 'detail', 'and', 'subject', 'to', 'an', 'analysis', 'in', 'order', 'to', 'discover', 'essential', 'features', 'or', 'meaning', '.'], ['a', 'mental', 'state', 'characterized', 'by', 'a', 'pessimistic', 'sense', 'of', 'inadequacy', 'and', 'a', 'despondent', 'lack', 'of', 'activity', '.'], ['determine', 'or', 'indicate', 'the', 'place', ',', 'site', ',', 'or', 'limits', 'of', ',', 'as', 'if', 'by', 'an', 'instrument', 'or', 'by', 'a', 'survey', '.'], ['handle', 'effectively', '.'], ['be', 'fond', 'of', '.'], ['have', 'need', 'of', '.'], ['bring', 'into', 'being', '.'], ['the', 'fashionable', 'elite', '.'], ['get', 'rid', 'of', '.'], ['give', 'suck', 'to', '.'], ['a', 'momentary', 'brightness', '.'], ['draw', 'advantages', 'from', '.'], ['abstain', 'from', 'eating', '.'], ['a', 'sudden', 'outburst', '.'], ['fool', 'or', 'hoax', '.'], ['ride', 'a', 'motorcycle', '.'], ['sound', 'with', 'resonance', '.'], ['give', 'or', 'convey', 'physically', '.'], ['go', 'away', 'or', 'leave', '.'], ['learn', 'by', 'reading', 'books', '.'], ['wear', 'away', 'or', 'erode', '.'], ['the', 'activity', 'of', 'leading', '.'], ['remove', 'the', 'husks', 'from', '.'], ['remove', 'tree', 'stumps', 'from', '.'], ['face', 'and', 'withstand', 'with', 'courage', '.'], ['a', 'casual', 'or', 'unexpected', 'convergence', '.'], ['something', 'with', 'no', 'concrete', 'substance', '.'], ['an', 'approximate', 'definition', 'or', 'example', '.'], ['the', 'act', 'of', 'granting', 'rights', '.'], ['take', 'into', 'one', "'s", 'family', '.'], ['take', 'or', 'capture', 'by', 'force', '.'], ['make', 'less', 'severe', 'or', 'strict', '.'], ['a', 'demanding', 'or', 'stimulating', 'situation', '.'], ['government', 'income', 'due', 'to', 'taxation', '.'], ['provide', 'or', 'equip', 'with', 'furniture', '.'], ['the', 'termination', 'of', 'a', 'meeting', '.'], ['extend', 'out', 'or', 'project', 'in', 'space', '.'], ['fruit', 'preserved', 'by', 'cooking', 'with', 'sugar', '.'], ['manipulate', 'the', 'registers', 'of', 'an', 'organ', '.'], ['have', 'or', 'exploit', 'a', 'monopoly', 'of', '.'], ['an', 'indicator', 'that', 'orients', 'you', 'generally', '.'], ['discipline', 'in', 'personal', 'and', 'social', 'activities', '.'], ['become', 'wrinkled', 'or', 'crumpled', 'or', 'creased', '.'], ['preserve', 'in', 'a', 'can', 'or', 'tin', '.'], ['start', 'suddenly', ',', 'as', 'from', 'fright', '.'], ['an', 'interpretation', 'of', 'a', 'text', 'or', 'action', '.'], ['produce', 'buds', ',', 'branches', ',', 'or', 'germinate', '.'], ['the', 'act', 'of', 'troubling', 'or', 'annoying', 'someone', '.'], ['fall', 'out', 'of', 'the', 'pod', 'or', 'husk', '.'], ['cause', 'to', 'vibrate', 'in', 'a', 'definite', 'pattern', '.'], ['a', 'party', 'of', 'people', 'assembled', 'for', 'dancing', '.'], ['a', 'finishing', 'coat', 'applied', 'to', 'exclude', 'moisture', '.'], ['travel', 'at', 'an', 'excessive', 'or', 'illegal', 'velocity', '.'], ['let', 'or', 'cause', 'to', 'fall', 'in', 'drops', '.'], ['run', 'or', 'move', 'very', 'quickly', 'or', 'hastily', '.'], ['a', 'process', 'of', 'becoming', 'smaller', 'or', 'shorter', '.'], ['become', 'broader', 'or', 'wider', 'or', 'more', 'extensive', '.'], ['use', 'cunning', 'or', 'deceit', 'to', 'escape', 'or', 'avoid', '.'], ['announce', 'the', 'termination', 'of', ',', 'as', 'of', 'treaties', '.'], ['get', 'on', 'top', 'of', ';', 'deal', 'with', 'successfully', '.'], ['play', 'on', 'a', 'string', 'instrument', 'with', 'a', 'bow', '.'], ['be', 'a', 'signal', 'for', 'or', 'a', 'symptom', 'of', '.'], ['to', 'say', ',', 'state', ',', 'or', 'perform', 'again', '.'], ['a', 'special', 'variety', 'of', 'domesticated', 'animals', 'within', 'a', 'species', '.'], ['the', 'act', 'of', 'certifying', 'or', 'bestowing', 'a', 'franchise', 'on', '.'], ['add', 'embellishments', 'and', 'paintings', 'to', '(', 'medieval', 'manuscripts', ')', '.'], ['a', 'quantity', 'that', 'is', 'twice', 'as', 'great', 'as', 'another', '.'], ['an', 'approximate', 'calculation', 'of', 'quantity', 'or', 'degree', 'or', 'worth', '.'], ['a', 'group', 'of', 'people', 'living', 'together', 'in', 'a', 'camp', '.'], ['a', 'person', 'who', 'conveys', '(', 'carries', 'or', 'transmits', ')', '.'], ['a', 'strong', 'sweeping', 'cut', 'made', 'with', 'a', 'sharp', 'instrument', '.'], ['make', 'more', 'intense', ',', 'stronger', ',', 'or', 'more', 'marked', '.'], ['beat', 'thoroughly', 'and', 'conclusively', 'in', 'a', 'competition', 'or', 'fight', '.'], ['have', 'a', 'strong', 'desire', 'or', 'urge', 'to', 'do', 'something', '.'], ['fly', 'in', 'or', 'as', 'if', 'in', 'a', 'glider', 'plane', '.'], ['assign', 'a', 'new', 'time', 'and', 'place', 'for', 'an', 'event', '.'], ['be', 'extremely', 'bad', 'in', 'quality', 'or', 'in', 'one', "'s", 'performance', '.'], ['appoint', 'someone', 'to', '(', 'a', 'position', 'or', 'a', 'job', ')', '.'], ['draw', 'a', 'line', 'or', 'lines', 'underneath', 'to', 'call', 'attention', 'to', '.'], ['the', 'entering', 'of', 'a', 'legal', 'document', 'into', 'the', 'public', 'record', '.'], ['a', 'bundle', '(', 'especially', 'one', 'carried', 'on', 'the', 'back', ')', '.'], ['an', 'assertion', 'that', 'someone', 'is', 'guilty', 'of', 'a', 'fault', 'or', 'offence', '.'], ['the', 'part', 'played', 'by', 'a', 'person', 'in', 'bringing', 'about', 'a', 'result', '.'], ['feel', 'about', 'or', 'towards', ';', 'consider', ',', 'evaluate', ',', 'or', 'regard', '.'], ['the', 'discipline', 'that', 'records', 'and', 'interprets', 'past', 'events', 'involving', 'human', 'beings', '.'], ['to', 'close', 'within', 'bounds', ',', 'limit', 'or', 'hold', 'back', 'from', 'movement', '.'], ['a', 'system', 'of', 'units', 'used', 'to', 'express', 'the', 'weight', 'of', 'something', '.'], ['make', 'a', 'schedule', ';', 'plan', 'the', 'time', 'and', 'place', 'for', 'events', '.'], ['come', 'out', 'better', 'in', 'a', 'competition', ',', 'race', ',', 'or', 'conflict', '.'], ['the', 'longer', 'of', 'the', 'two', 'telegraphic', 'signals', 'used', 'in', 'morse', 'code', '.'], ['cause', 'to', 'form', 'a', 'united', ',', 'orderly', ',', 'and', 'aesthetically', 'consistent', 'whole', '.'], ['impose', ',', 'involve', ',', 'or', 'imply', 'as', 'a', 'necessary', 'accompaniment', 'or', 'result', '.'], ['a', 'reflex', 'response', 'to', 'the', 'passage', 'of', 'electric', 'current', 'through', 'the', 'body', '.'], ['make', 'a', 'hole', ',', 'especially', 'with', 'a', 'pointed', 'power', 'or', 'hand', 'tool', '.'], ['cause', 'to', 'move', ';', 'cause', 'to', 'be', 'in', 'a', 'certain', 'position', 'or', 'condition', '.'], ['the', 'act', 'of', 'starting', 'something', 'for', 'the', 'first', 'time', ';', 'introducing', 'something', 'new', '.'], ['a', 'light', 'line', 'that', 'is', 'used', 'in', 'lettering', 'to', 'help', 'align', 'the', 'letters', '.'], ['a', 'loss', 'entailed', 'by', 'giving', 'up', 'or', 'selling', 'something', 'at', 'less', 'than', 'its', 'value', '.'], ['a', 'natural', 'elevation', '(', 'especially', 'a', 'rocky', 'one', 'that', 'juts', 'out', 'into', 'the', 'sea', ')', '.'], ['incorporate', 'a', 'food', 'ingredient', 'into', 'a', 'mixture', 'by', 'repeatedly', 'turning', 'it', 'over', 'without', 'stirring', 'or', 'beating', '.'], ['a', 'slight', 'surface', 'cut', '(', 'especially', 'a', 'notch', 'that', 'is', 'made', 'to', 'keep', 'a', 'tally', ')', '.'], ['take', 'a', 'walk', 'for', 'one', "'s", 'health', 'or', 'to', 'aid', 'digestion', ',', 'as', 'after', 'a', 'meal', '.'], ['the', 'phenomenon', 'of', 'a', 'propagating', 'wave', '(', 'light', 'or', 'sound', ')', 'being', 'thrown', 'back', 'from', 'a', 'surface', '.'], ['be', 'cognizant', 'or', 'aware', 'of', 'a', 'fact', 'or', 'a', 'specific', 'piece', 'of', 'information', ';', 'possess', 'knowledge', 'or', 'information', 'about', '.'], ['the', 'floor', 'of', 'a', 'building', 'that', 'is', 'at', 'or', 'nearest', 'to', 'the', 'level', 'of', 'the', 'ground', 'around', 'the', 'building', '.'], ['cater', 'to', 'popular', 'taste', 'to', 'make', 'popular', 'and', 'present', 'to', 'the', 'general', 'public', ';', 'bring', 'into', 'general', 'or', 'common', 'use', '.'], ['a', 'connection', 'between', 'an', 'electrical', 'device', 'and', 'a', 'large', 'conducting', 'body', ',', 'such', 'as', 'the', 'earth', '(', 'which', 'is', 'taken', 'to', 'be', 'at', 'zero', 'voltage', ')', '.'], ['a', 'body', 'of', 'people', 'who', 'settle', 'far', 'from', 'home', 'but', 'maintain', 'ties', 'with', 'their', 'homeland', ';', 'inhabitants', 'remain', 'nationals', 'of', 'their', 'home', 'state', 'but', 'are', 'not', 'literally', 'under', 'the', 'home', 'state', "'s", 'system', 'of', 'government', '.'], ['preserve', 'with', 'sugar', '.'], ['a', 'brief', 'statement', '.'], ['provide', 'with', 'rails', '.'], ['lay', 'with', 'rails', '.'], ['take', 'in', 'marriage', '.'], ['censure', 'severely', 'or', 'angrily', '.'], ['an', 'acknowledgment', 'of', 'appreciation', '.'], ['terminate', 'an', 'association', 'with', '.'], ['go', 'up', 'or', 'advance', '.'], ['make', 'tight', 'or', 'tighter', '.'], ['feel', 'hot', 'or', 'painful', '.'], ['show', 'to', 'be', 'invalid', '.'], ['cause', 'to', 'move', 'faster', '.'], ['impress', 'or', 'affect', 'deeply', '.'], ['make', 'less', 'severe', 'or', 'harsh', '.'], ['the', 'force', 'used', 'in', 'pulling', '.'], ['cause', 'to', 'become', 'widely', 'known', '.'], ['utter', 'words', 'loudly', 'and', 'forcefully', '.'], ['edge', 'tool', 'used', 'in', 'shaving', '.'], ['an', 'intuitive', 'understanding', 'of', 'something', '.'], ['the', 'act', 'of', 'breaking', 'something', '.'], ['throw', 'with', 'force', 'or', 'recklessness', '.'], ['an', 'attempt', 'to', 'get', 'something', '.'], ['give', 'an', 'incentive', 'for', 'action', '.'], ['come', 'back', 'after', 'being', 'refused', '.'], ['form', 'an', 'impenetrable', 'cover', 'over', '.'], ['a', 'state', 'of', 'extreme', 'anger', '.'], ['give', 'something', 'useful', 'or', 'necessary', 'to', '.'], ['the', 'act', 'of', 'fastening', 'things', 'together', '.'], ['the', 'act', 'of', 'becoming', 'more', 'distant', '.'], ['a', 'feeling', 'of', 'aversion', 'or', 'antipathy', '.'], ['cause', 'to', 'procreate', '(', 'animals', ')', '.'], ['prevent', 'from', 'entering', ';', 'keep', 'out', '.'], ['the', 'finger', 'next', 'to', 'the', 'thumb', '.'], ['bend', 'into', 'the', 'shape', 'of', 'a', 'crank', '.'], ['a', 'particular', 'environment', 'or', 'walk', 'of', 'life', '.'], ['recovery', 'or', 'preservation', 'from', 'loss', 'or', 'danger', '.'], ['continue', 'to', 'live', 'through', 'hardship', 'or', 'adversity', '.'], ['the', 'act', 'of', 'admitting', 'someone', 'to', 'enter', '.'], ['make', 'a', 'tour', 'of', 'a', 'certain', 'place', '.'], ['exhaust', 'by', 'allowing', 'to', 'pull', 'on', 'the', 'line', '.'], ['the', 'state', 'or', 'fact', 'of', 'being', 'an', 'owner', '.'], ['rinse', 'one', "'s", 'mouth', 'and', 'throat', 'with', 'mouthwash', '.'], ['the', 'cognitive', 'processes', 'whereby', 'past', 'experience', 'is', 'remembered', '.'], ['the', 'motion', 'of', 'one', 'object', 'relative', 'to', 'another', '.'], ['support', 'by', 'placing', 'against', 'something', 'solid', 'or', 'rigid', '.'], ['connect', 'or', 'arrange', 'into', 'a', 'chain', 'by', 'linking', '.'], ['matter', 'that', 'remains', 'after', 'something', 'has', 'been', 'removed', '.'], ['unite', 'or', 'merge', 'with', 'something', 'already', 'in', 'existence', '.'], ['make', 'a', 'loud', 'noise', ',', 'as', 'of', 'animal', '.'], ['become', 'injured', ',', 'broken', ',', 'or', 'distorted', 'by', 'pressure', '.'], ['the', 'trait', 'of', 'sincere', 'and', 'steadfast', 'fixity', 'of', 'purpose', '.'], ['adapt', 'or', 'conform', 'oneself', 'to', 'new', 'or', 'different', 'conditions', '.'], ['go', 'on', 'a', 'campaign', ';', 'go', 'off', 'to', 'war', '.'], ['cook', 'slowly', 'and', 'for', 'a', 'long', 'time', 'in', 'liquid', '.'], ['perceive', 'with', 'the', 'senses', 'quickly', ',', 'suddenly', ',', 'or', 'momentarily', '.'], ['organization', 'of', 'performers', 'and', 'associated', 'personnel', '(', 'especially', 'theatrical', ')', '.'], ['put', 'under', 'the', 'control', 'and', 'authority', 'of', 'a', 'federal', 'government', '.'], ['reduce', 'to', 'small', 'pieces', 'or', 'particles', 'by', 'pounding', 'or', 'abrading', '.'], ['apply', 'conditioner', 'to', 'in', 'order', 'to', 'make', 'smooth', 'and', 'shiny', '.'], ['intend', '(', 'something', ')', 'to', 'move', 'towards', 'a', 'certain', 'goal', '.'], ['(', 'broadcasting', ')', 'a', 'local', 'announcement', 'inserted', 'into', 'a', 'network', 'program', '.'], ['the', 'hard', 'ridge', 'that', 'forms', 'the', 'upper', 'part', 'of', 'the', 'nose', '.'], ['express', 'or', 'raise', 'an', 'objection', 'or', 'protest', 'or', 'criticism', 'or', 'express', 'dissent', '.'], ['a', 'list', 'of', 'particulars', '(', 'as', 'a', 'playbill', 'or', 'bill', 'of', 'fare', ')', '.'], ['a', 'petition', 'or', 'appeal', 'made', 'to', 'a', 'person', 'of', 'superior', 'status', 'or', 'rank', '.'], ['a', 'process', 'of', 'becoming', 'larger', 'or', 'longer', 'or', 'more', 'numerous', 'or', 'more', 'important', '.'], ['the', 'act', 'of', 'starting', 'something', 'for', 'the', 'first', 'time', ';', 'introducing', 'something', 'new', '.'], ['a', 'communication', '(', 'usually', 'brief', ')', 'that', 'is', 'written', 'or', 'spoken', 'or', 'signaled', '.'], ['a', 'short', 'note', 'recognizing', 'a', 'source', 'of', 'information', 'or', 'of', 'a', 'quoted', 'passage', '.'], ['the', 'termination', 'or', 'disintegration', 'of', 'a', 'relationship', '(', 'between', 'persons', 'or', 'nations', ')', '.'], ['apply', 'a', 'thin', 'coating', 'of', 'paint', ',', 'metal', ',', 'etc', '.', ',', 'to', '.'], ['measure', 'the', 'depth', 'of', '(', 'a', 'body', 'of', 'water', ')', 'with', 'a', 'sounding', 'line', '.'], ['manual', 'turning', 'of', 'a', 'fetus', 'in', 'the', 'uterus', '(', 'usually', 'to', 'aid', 'delivery', ')', '.'], ['evaluate', 'or', 'estimate', 'the', 'nature', ',', 'quality', ',', 'ability', ',', 'extent', ',', 'or', 'significance', 'of', '.'], ['deprive', 'of', 'courage', 'or', 'hope', ';', 'take', 'away', 'hope', 'from', ';', 'cause', 'to', 'feel', 'discouraged', '.'], ['an', 'open', 'jar', 'of', 'glass', 'or', 'porcelain', 'used', 'as', 'an', 'ornament', 'or', 'to', 'hold', 'flowers', '.'], ['warn', 'or', 'arouse', 'to', 'a', 'sense', 'of', 'danger', 'or', 'call', 'to', 'a', 'state', 'of', 'preparedness', '.'], ['the', 'act', 'of', 'forcing', 'yourself', '(', 'or', 'being', 'forced', ')', 'into', 'or', 'through', 'a', 'restricted', 'space', '.'], ['a', 'diacritical', 'mark', 'used', 'to', 'indicate', 'stress', 'or', 'placed', 'above', 'a', 'vowel', 'to', 'indicate', 'a', 'special', 'pronunciation', '.'], ['bedclothes', 'consisting', 'of', 'a', 'lightweight', 'cloth', 'covering', '(', 'an', 'afghan', 'or', 'bedspread', ')', 'that', 'is', 'casually', 'thrown', 'over', 'something', '.'], ['discover', 'or', 'come', 'upon', 'accidentally', ',', 'suddenly', ',', 'or', 'unexpectedly', ';', 'catch', 'somebody', 'doing', 'something', 'or', 'in', 'a', 'certain', 'state', '.'], ['a', 'heavy', 'toxic', 'silvery-white', 'radioactive', 'metallic', 'element', ';', 'occurs', 'in', 'many', 'isotopes', ';', 'used', 'for', 'nuclear', 'fuels', 'and', 'nuclear', 'weapons', '.'], ['a', 'statement', '(', 'either', 'spoken', 'or', 'written', ')', 'that', 'is', 'made', 'to', 'reply', 'to', 'a', 'question', 'or', 'request', 'or', 'criticism', 'or', 'accusation', '.'], ['a', 'punctuation', 'mark', '(', '.', ')', 'placed', 'at', 'the', 'end', 'of', 'a', 'declarative', 'sentence', 'to', 'indicate', 'a', 'full', 'stop', 'or', 'after', 'abbreviations', '.'], ['a', 'person', 'who', 'is', 'the', 'aim', 'of', 'an', 'attack', '(', 'especially', 'a', 'victim', 'of', 'ridicule', 'or', 'exploitation', ')', 'by', 'some', 'hostile', 'person', 'or', 'influence', '.'], ['a', 'nonmetallic', 'univalent', 'element', 'that', 'is', 'normally', 'a', 'colorless', 'and', 'odorless', 'highly', 'flammable', 'diatomic', 'gas', ';', 'the', 'simplest', 'and', 'lightest', 'and', 'most', 'abundant', 'element', 'in', 'the', 'universe', '.'], ['move', 'very', 'slightly', '.'], ['refrain', 'from', 'harming', '.'], ['an', 'indication', 'of', 'damage', '.'], ['sound', 'sharply', 'or', 'shrilly', '.'], ['estimate', 'the', 'value', 'of', '.'], ['develop', 'in', 'a', 'positive', 'way', '.'], ['a', 'concentrated', 'example', 'of', 'something', '.'], ['be', 'tantamount', 'or', 'equivalent', 'to', '.'], ['provide', 'with', 'a', 'new', 'seat', '.'], ['fertilize', 'and', 'cause', 'to', 'grow', '.'], ['a', 'well-known', 'or', 'notable', 'person', '.'], ['something', 'that', 'is', 'desired', 'intensely', '.'], ['cause', 'to', 'become', 'alive', 'again', '.'], ['sorting', 'one', 'thing', 'from', 'others', '.'], ['apply', 'a', 'plaster', 'cast', 'to', '.'], ['break', 'down', ',', 'literally', 'or', 'metaphorically', '.'], ['the', 'act', 'of', 'choosing', 'or', 'selecting', '.'], ['make', 'a', 'logical', 'or', 'causal', 'connection', '.'], ['move', 'about', 'in', 'a', 'confused', 'manner', '.'], ['lengthen', 'or', 'extend', 'in', 'duration', 'or', 'space', '.'], ['a', 'group', 'of', 'nations', 'having', 'common', 'interests', '.'], ['make', 'visible', 'by', 'means', 'of', 'chemical', 'solutions', '.'], ['a', 'narrative', 'song', 'with', 'a', 'recurrent', 'refrain', '.'], ['informal', 'terms', 'referring', 'to', 'a', 'domestic', 'cat', '.'], ['a', 'short', 'section', 'of', 'a', 'musical', 'composition', '.'], ['put', 'up', 'with', 'something', 'or', 'somebody', 'unpleasant', '.'], ['a', 'pair', 'of', 'people', 'who', 'live', 'together', '.'], ['talk', 'or', 'behave', 'amorously', ',', 'without', 'serious', 'intentions', '.'], ['oppose', ',', 'as', 'in', 'hostility', 'or', 'a', 'competition', '.'], ['be', 'or', 'play', 'a', 'part', 'of', 'or', 'in', '.'], ['a', 'vertical', 'array', 'of', 'numbers', 'or', 'other', 'information', '.'], ['change', 'the', 'inherent', 'purpose', 'or', 'function', 'of', 'something', '.'], ['any', 'activity', 'that', 'occupies', 'a', 'person', "'s", 'attention', '.'], ['write', 'as', 'if', 'with', 'print', ';', 'not', 'cursive', '.'], ['money', 'deposited', 'in', 'a', 'bank', 'or', 'some', 'similar', 'institution', '.'], ['to', 'compel', 'or', 'deter', 'by', 'or', 'as', 'if', 'by', 'threats', '.'], ['the', 'temperature', 'at', 'which', 'a', 'liquid', 'boils', 'at', 'sea', 'level', '.'], ['make', 'to', 'a', 'size', ';', 'bring', 'to', 'a', 'suitable', 'size', '.'], ['travel', 'in', 'front', 'of', ';', 'go', 'in', 'advance', 'of', 'others', '.'], ['provide', 'with', '(', 'something', ')', 'usually', 'for', 'a', 'specific', 'purpose', '.'], ['cause', 'to', 'move', 'or', 'operate', 'freely', 'within', 'a', 'bounded', 'space', '.'], ['elevate', 'or', 'idealize', ',', 'in', 'allusion', 'to', 'christ', "'s", 'transfiguration', '.'], ['a', 'facility', 'where', 'things', 'can', 'be', 'deposited', 'for', 'storage', 'or', 'safekeeping', '.'], ['someone', 'who', 'frequently', 'finds', 'fault', 'or', 'makes', 'harsh', 'and', 'unfair', 'judgments', '.'], ['pat', 'or', 'squeeze', 'fondly', 'or', 'playfully', ',', 'especially', 'under', 'the', 'chin', '.'], ['a', 'blank', 'character', 'used', 'to', 'separate', 'successive', 'words', 'in', 'writing', 'or', 'printing', '.'], ['a', 'sacrament', 'admitting', 'a', 'baptized', 'person', 'to', 'full', 'participation', 'in', 'the', 'church', '.'], ['the', 'pursuit', 'and', 'killing', 'or', 'capture', 'of', 'wild', 'animals', 'regarded', 'as', 'a', 'sport', '.'], ['a', 'person', 'who', 'is', 'under', 'the', 'protection', 'or', 'in', 'the', 'custody', 'of', 'another', '.'], ['open', '(', 'a', 'place', ')', 'to', 'members', 'of', 'all', 'races', 'and', 'ethnic', 'groups', '.'], ['consider', 'again', ';', 'give', 'new', 'consideration', 'to', ';', 'usually', 'with', 'a', 'view', 'to', 'changing', '.'], ['put', 'a', 'coat', 'on', ';', 'cover', 'the', 'surface', 'of', ';', 'furnish', 'with', 'a', 'surface', '.'], ['to', 'be', 'on', 'base', 'at', 'the', 'end', 'of', 'an', 'inning', ',', 'of', 'a', 'player', '.'], ['physician', 'and', 'american', 'revolutionary', 'leader', ';', 'signer', 'of', 'the', 'declaration', 'of', 'independence', '(', '1745-1813', ')', '.'], ['make', 'a', 'plug', 'for', ';', 'praise', 'the', 'qualities', 'or', 'in', 'order', 'to', 'sell', 'or', 'promote', '.'], ['find', 'the', 'solution', 'to', '(', 'a', 'problem', 'or', 'question', ')', 'or', 'understand', 'the', 'meaning', 'of', '.'], ['consider', 'again', '(', 'a', 'bill', ')', 'that', 'had', 'been', 'voted', 'upon', 'before', ',', 'with', 'a', 'view', 'to', 'altering', 'it', '.'], ['make', 'underscoring', 'marks', '.'], ['say', 'mean', 'things', '.'], ['place', 'or', 'set', 'apart', '.'], ['pass', 'the', 'tongue', 'over', '.'], ['move', 'an', 'implement', 'through', '.'], ['take', 'up', 'or', 'begin', 'anew', '.'], ['take', 'up', 'with', 'the', 'tongue', '.'], ['a', 'supplementary', 'part', 'or', 'accessory', '.'], ['the', 'act', 'of', 'ending', 'something', '.'], ['strike', 'as', 'if', 'by', 'whipping', '.'], ['manipulate', 'in', 'a', 'fraudulent', 'manner', '.'], ['a', 'way', 'of', 'conceiving', 'something', '.'], ['cause', 'to', 'be', 'firmly', 'attached', '.'], ['strike', 'the', 'air', 'in', 'flight', '.'], ['create', 'an', 'image', 'or', 'likeness', 'of', '.'], ['mock', 'or', 'make', 'fun', 'of', 'playfully', '.'], ['the', 'state', 'of', 'being', 'more', 'than', 'full', '.'], ['create', 'by', 'putting', 'components', 'or', 'members', 'together', '.'], ['restore', 'to', 'a', 'previous', 'or', 'better', 'condition', '.'], ['the', 'act', 'of', 'giving', 'someone', 'a', 'job', '.'], ['strike', 'sharply', ',', 'as', 'in', 'some', 'sports', '.'], ['move', 'in', 'a', 'spiral', 'or', 'zigzag', 'course', '.'], ['drive', 'forcibly', 'as', 'if', 'by', 'a', 'punch', '.'], ['a', 'casualty', 'to', 'military', 'personnel', 'resulting', 'from', 'combat', '.'], ['an', 'agent', 'that', 'operates', 'some', 'apparatus', 'or', 'machine', '.'], ['raise', 'in', 'rank', ',', 'character', ',', 'or', 'status', '.'], ['an', 'implement', 'consisting', 'of', 'a', 'length', 'of', 'wood', '.'], ['take', 'away', 'a', 'vital', 'or', 'essential', 'part', 'of', '.'], ['an', 'air', 'force', 'unit', 'smaller', 'than', 'a', 'squadron', '.'], ['subject', 'to', 'political', ',', 'religious', ',', 'or', 'moral', 'censorship', '.'], ['any', 'address', 'at', 'which', 'you', 'dwell', 'more', 'than', 'temporarily', '.'], ['a', 'feeling', 'of', 'annoyance', 'at', 'being', 'hindered', 'or', 'criticized', '.'], ['expose', 'or', 'make', 'accessible', 'to', 'some', 'action', 'or', 'influence', '.'], ['the', 'act', 'of', 'coming', 'to', 'land', 'after', 'a', 'voyage', '.'], ['an', 'act', 'of', 'hindering', 'someone', "'s", 'plans', 'or', 'efforts', '.'], ['manual', 'stimulation', 'of', 'the', 'genital', 'area', 'for', 'sexual', 'pleasure', '.'], ['the', 'role', 'of', 'the', 'head', 'of', 'a', 'government', 'department', '.'], ['cause', '(', 'a', 'computer', ')', 'to', 'execute', 'a', 'single', 'command', '.'], ['an', 'outward', 'semblance', 'that', 'misrepresents', 'the', 'true', 'nature', 'of', 'something', '.'], ['indicate', 'contempt', 'by', 'breathing', 'noisily', 'and', 'forcefully', 'through', 'the', 'nose', '.'], ['strike', 'heavily', ',', 'especially', 'with', 'the', 'fist', 'or', 'a', 'bat', '.'], ['an', 'area', 'that', 'is', 'approximately', 'central', 'within', 'some', 'larger', 'region', '.'], ['angle', 'with', 'a', 'hook', 'and', 'line', 'drawn', 'through', 'the', 'water', '.'], ['a', 'representation', 'of', 'a', 'person', 'that', 'is', 'exaggerated', 'for', 'comic', 'effect', '.'], ['a', 'period', 'of', 'time', 'containing', '365', '(', 'or', '366', ')', 'days', '.'], ['the', 'act', 'of', 'extracting', 'ores', 'or', 'coal', 'etc', 'from', 'the', 'earth', '.'], ['a', 'person', 'eating', 'a', 'meal', '(', 'especially', 'in', 'a', 'restaurant', ')', '.'], ['make', 'receptive', 'or', 'willing', 'towards', 'an', 'action', 'or', 'attitude', 'or', 'belief', '.'], ['a', 'state', 'of', 'order', 'in', 'which', 'events', 'conform', 'to', 'the', 'law', '.'], ['a', 'means', 'or', 'agency', 'by', 'which', 'something', 'is', 'expressed', 'or', 'communicated', '.'], ['make', 'out', 'of', 'components', '(', 'often', 'in', 'an', 'improvising', 'manner', ')', '.'], ['move', 'so', 'that', 'an', 'opening', 'or', 'passage', 'is', 'obstructed', ';', 'make', 'shut', '.'], ['the', 'principal', 'activity', 'in', 'your', 'life', 'that', 'you', 'do', 'to', 'earn', 'money', '.'], ['a', 'member', 'of', 'a', 'military', 'unit', 'trained', 'as', 'shock', 'troops', 'for', 'hit-and-run', 'raids', '.'], ['be', 'offended', 'or', 'bothered', 'by', ';', 'take', 'offense', 'with', ',', 'be', 'bothered', 'by', '.'], ['place', 'in', 'a', 'line', 'or', 'arrange', 'so', 'as', 'to', 'be', 'parallel', 'or', 'straight', '.'], ['a', 'number', 'or', 'letter', 'indicating', 'quality', '(', 'especially', 'of', 'a', 'student', "'s", 'performance', ')', '.'], ['a', 'verbal', 'or', 'written', 'request', 'for', 'assistance', 'or', 'employment', 'or', 'admission', 'to', 'a', 'school', '.'], ['a', 'fruiting', 'structure', 'resembling', 'an', 'umbrella', 'or', 'a', 'cone', 'that', 'forms', 'the', 'top', 'of', 'a', 'stalked', 'fleshy', 'fungus', 'such', 'as', 'a', 'mushroom', '.'], ['stingless', 'male', 'bee', 'in', 'a', 'colony', 'of', 'social', 'bees', '(', 'especially', 'honeybees', ')', 'whose', 'sole', 'function', 'is', 'to', 'mate', 'with', 'the', 'queen', '.'], ['a', 'measure', 'of', 'how', 'likely', 'it', 'is', 'that', 'some', 'event', 'will', 'occur', ';', 'a', 'number', 'expressing', 'the', 'ratio', 'of', 'favorable', 'cases', 'to', 'the', 'whole', 'number', 'of', 'cases', 'possible', '.'], ['refrain', 'from', 'doing', 'something', '.'], ['gain', 'points', 'in', 'a', 'game', '.'], ['provide', 'with', 'power', 'and', 'authority', '.'], ['be', 'fully', 'aware', 'or', 'cognizant', 'of', '.'], ['to', 'remain', 'emotionally', 'or', 'intellectually', 'attached', '.'], ['run', 'at', 'a', 'moderately', 'swift', 'pace', '.'], ['make', 'less', 'hostile', ';', 'win', 'over', '.'], ['exercise', 'authoritative', 'control', 'or', 'power', 'over', '.'], ['like', 'better', ';', 'value', 'more', 'highly', '.'], ['give', 'pleasure', 'to', 'or', 'be', 'pleasing', 'to', '.'], ['take', 'vows', ',', 'as', 'in', 'religious', 'order', '.'], ['any', 'likeness', 'of', 'a', 'person', ',', 'in', 'any', 'medium', '.'], ['continue', 'a', 'certain', 'state', ',', 'condition', ',', 'or', 'activity', '.'], ['take', 'possession', 'of', 'by', 'force', ',', 'as', 'after', 'an', 'invasion', '.'], ['teach', 'or', 'refine', 'to', 'be', 'discriminative', 'in', 'taste', 'or', 'judgment', '.'], ['someone', '(', 'or', 'something', ')', 'on', 'which', 'expectations', 'are', 'centered', '.'], ['stand', 'with', 'arms', 'or', 'forelegs', 'raised', ',', 'as', 'if', 'menacing', '.'], ['a', 'city', 'on', 'the', 'river', 'thames', 'in', 'berkshire', 'in', 'southern', 'england', '.'], ['turn', 'or', 'go', 'to', 'the', 'port', 'or', 'left', 'side', ',', 'of', 'a', 'ship', '.'], ['inability', 'of', 'the', 'heart', 'to', 'pump', 'enough', 'blood', 'to', 'sustain', 'normal', 'bodily', 'functions', '.'], ['the', 'condition', 'of', 'having', 'been', 'proposed', 'as', 'a', 'suitable', 'candidate', 'for', 'appointment', 'or', 'election', '.'], ['(', 'chemistry', ')', 'a', 'chain', 'of', 'atoms', 'in', 'a', 'molecule', 'that', 'forms', 'a', 'closed', 'loop', '.'], ['the', 'terminal', 'forced', 'release', 'of', 'pressure', 'built', 'up', 'during', 'the', 'occlusive', 'phase', 'of', 'a', 'stop', 'consonant', '.'], ['demand', 'as', 'being', 'one', "'s", 'due', 'or', 'property', ';', 'assert', 'one', "'s", 'right', 'or', 'title', 'to', '.'], ['a', 'number', 'that', 'expresses', 'the', 'accomplishment', 'of', 'a', 'team', 'or', 'an', 'individual', 'in', 'a', 'game', 'or', 'contest', '.'], ['a', 'band', 'of', 'leather', 'or', 'rope', 'that', 'is', 'placed', 'around', 'an', 'animal', "'s", 'neck', 'as', 'a', 'harness', 'or', 'to', 'identify', 'it', '.'], ['enclosure', 'consisting', 'of', 'a', 'section', 'of', 'canal', 'that', 'can', 'be', 'closed', 'to', 'control', 'the', 'water', 'level', ';', 'used', 'to', 'raise', 'or', 'lower', 'vessels', 'that', 'pass', 'through', 'it', '.'], ['cause', 'to', 'regain', 'consciousness', '.'], ['show', 'submission', 'or', 'fear', '.'], ['damage', 'the', 'reputation', 'of', '.'], ['accept', 'an', 'excuse', 'for', '.'], ['regain', 'or', 'make', 'up', 'for', '.'], ['behave', 'in', 'a', 'certain', 'way', '.'], ['be', 'a', 'sign', 'or', 'indication', 'of', '.'], ['reverse', 'the', 'winding', 'or', 'twisting', 'of', '.'], ['cause', 'to', 'move', 'back', 'and', 'forth', '.'], ['react', 'to', 'a', 'stimulus', 'or', 'command', '.'], ['shape', 'or', 'influence', ';', 'give', 'direction', 'to', '.'], ['shape', 'or', 'influence', ';', 'give', 'direction', 'to', '.'], ['cook', 'on', 'a', 'hot', 'surface', 'using', 'fat', '.'], ['part', ';', 'cease', 'or', 'break', 'association', 'with', '.'], ['devote', 'oneself', 'to', 'a', 'special', 'area', 'of', 'work', '.'], ['a', 'name', 'given', 'to', 'a', 'product', 'or', 'service', '.'], ['an', 'impression', 'that', 'something', 'might', 'be', 'the', 'case', '.'], ['tag', 'the', 'base', 'runner', 'to', 'get', 'him', 'out', '.'], ['give', 'equal', 'rights', 'to', ';', 'of', 'women', 'and', 'minorities', '.'], ['piece', '(', 'something', 'old', ')', 'with', 'a', 'new', 'part', '.'], ['a', 'mental', 'representation', 'of', 'the', 'meaning', 'or', 'significance', 'of', 'something', '.'], ['the', 'quality', 'of', 'being', 'inadequate', 'or', 'falling', 'short', 'of', 'perfection', '.'], ['have', 'the', 'financial', 'means', 'to', 'do', 'something', 'or', 'buy', 'something', '.'], ['supply', 'food', 'ready', 'to', 'eat', ';', 'for', 'parties', 'and', 'banquets', '.'], ['treat', 'with', 'an', 'agent', ';', 'add', '(', 'an', 'agent', ')', 'to', '.'], ['an', 'injury', 'caused', 'by', 'exposure', 'to', 'heat', 'or', 'chemicals', 'or', 'radiation', '.'], ['a', 'successful', 'free', 'throw', 'or', 'try', 'for', 'point', 'after', 'a', 'touchdown', '.'], ['a', 'short', 'theatrical', 'performance', 'that', 'is', 'part', 'of', 'a', 'longer', 'program', '.'], ['the', 'stitching', 'that', 'forms', 'the', 'rim', 'of', 'a', 'shoe', 'or', 'boot', '.'], ['be', 'reflected', 'as', 'heat', ',', 'sound', ',', 'or', 'light', 'or', 'shock', 'waves', '.'], ['act', 'in', 'unison', 'or', 'agreement', 'and', 'in', 'secret', 'towards', 'a', 'deceitful', 'or', 'illegal', 'purpose', '.'], ['the', 'end', 'of', 'a', 'word', '(', 'a', 'suffix', 'or', 'inflectional', 'ending', 'or', 'final', 'morpheme', ')', '.'], ['a', 'person', 'or', 'thing', 'equal', 'to', 'another', 'in', 'value', 'or', 'measure', 'or', 'force', 'or', 'effect', 'or', 'significance', 'etc', '.'], ['avoid', 'or', 'try', 'to', 'avoid', 'fulfilling', ',', 'answering', ',', 'or', 'performing', '(', 'duties', ',', 'questions', ',', 'or', 'issues', ')', '.'], ['(', 'computer', 'science', ')', 'the', 'part', 'of', 'a', 'computer', '(', 'a', 'microprocessor', 'chip', ')', 'that', 'does', 'most', 'of', 'the', 'data', 'processing', '.'], ['make', 'invalid', 'for', 'use', '.'], ['move', 'to', 'and', 'fro', '.'], ['shrivel', 'or', 'wither', 'or', 'mature', 'imperfectly', '.'], ['cause', 'to', 'be', 'in', 'brief', 'contact', 'with', '.'], ['emit', 'a', 'steady', 'even', 'light', 'without', 'flames', '.'], ['transporting', 'goods', 'commercially', 'at', 'rates', 'cheaper', 'than', 'express', 'rates', '.'], ['bring', 'two', 'objects', ',', 'ideas', ',', 'or', 'people', 'together', '.'], ['get', 'to', 'know', 'or', 'become', 'aware', 'of', ',', 'usually', 'accidentally', '.'], ['lengthen', 'in', 'time', ';', 'cause', 'to', 'be', 'or', 'last', 'longer', '.'], ['express', 'audibly', ';', 'utter', 'sounds', '(', 'not', 'necessarily', 'words', ')', '.'], ['the', 'remains', 'of', 'something', 'that', 'has', 'been', 'destroyed', 'or', 'broken', 'up', '.'], ['give', 'entirely', 'to', 'a', 'specific', 'person', ',', 'activity', ',', 'or', 'cause', '.'], ['(', 'medicine', ')', 'the', 'condition', 'in', 'which', 'an', 'organism', 'can', 'resist', 'disease', '.'], ['formally', 'reject', 'or', 'disavow', 'a', 'formerly', 'held', 'belief', ',', 'usually', 'under', 'pressure', '.'], ['the', 'preservation', 'and', 'careful', 'management', 'of', 'the', 'environment', 'and', 'of', 'natural', 'resources', '.'], ['strike', 'out', 'by', 'swinging', 'and', 'missing', 'the', 'pitch', 'charged', 'as', 'the', 'third', '.'], ['the', 'gears', 'by', 'which', 'the', 'motion', 'of', 'a', 'machine', 'can', 'be', 'reversed', '.'], ['raising', 'the', 'stakes', 'in', 'a', 'card', 'game', 'by', 'a', 'factor', 'of', '2', '.'], ['a', 'chart', 'or', 'map', 'showing', 'the', 'movements', 'or', 'progress', 'of', 'an', 'object', '.'], ['value', 'measured', 'by', 'what', 'must', 'be', 'given', 'or', 'done', 'or', 'undergone', 'to', 'obtain', 'something', '.'], ['the', 'period', 'of', 'time', 'during', 'which', 'a', 'contract', 'conveying', 'property', 'to', 'a', 'person', 'is', 'in', 'effect', '.'], ['point', 'or', 'cause', 'to', 'go', '(', 'blows', ',', 'weapons', ',', 'or', 'objects', 'such', 'as', 'photographic', 'equipment', ')', 'towards', '.'], ['surround', 'completely', '.'], ['signal', 'by', 'winking', '.'], ['cite', 'as', 'an', 'authority', ';', 'resort', 'to', '.'], ['look', 'through', 'a', 'book', 'or', 'other', 'written', 'material', '.'], ['undergo', 'sequestration', 'by', 'forming', 'a', 'stable', 'compound', 'with', 'an', 'ion', '.'], ['the', 'smallest', 'whole', 'number', 'or', 'a', 'numeral', 'representing', 'this', 'number', '.'], ['the', 'trait', 'of', 'believing', 'in', 'the', 'honesty', 'and', 'reliability', 'of', 'others', '.'], ['reimburse', 'or', 'compensate', '(', 'someone', ')', ',', 'as', 'for', 'a', 'loss', '.'], ['make', 'fine', 'adjustments', 'or', 'divide', 'into', 'marked', 'intervals', 'for', 'optimal', 'measuring', '.'], ['soften', 'or', 'disintegrate', 'by', 'means', 'of', 'chemical', 'action', ',', 'heat', ',', 'or', 'moisture', '.'], ['(', 'theology', ')', 'the', 'act', 'of', 'delivering', 'from', 'sin', 'or', 'saving', 'from', 'evil', '.'], ['furnish', 'with', 'an', 'opening', 'to', 'allow', 'air', 'to', 'circulate', 'or', 'gas', 'to', 'escape', '.'], ['a', 'list', 'of', 'candidates', 'nominated', 'by', 'a', 'political', 'party', 'to', 'run', 'for', 'election', 'to', 'public', 'offices', '.'], ['make', 'a', 'mark', 'or', 'lines', 'on', 'a', 'surface', '.'], ['set', 'aside', 'or', 'apart', 'for', 'a', 'specific', 'purpose', 'or', 'use', '.'], ['lead', 'someone', 'in', 'the', 'wrong', 'direction', 'or', 'give', 'someone', 'wrong', 'directions', '.'], ['supply', 'with', 'water', ',', 'as', 'with', 'channels', 'or', 'ditches', 'or', 'streams', '.'], ['remove', 'by', 'erasing', 'or', 'crossing', 'out', 'or', 'as', 'if', 'by', 'drawing', 'a', 'line', '.'], ['(', 'computer', 'science', ')', 'the', 'occurrence', 'of', 'an', 'incorrect', 'result', 'produced', 'by', 'a', 'computer', '.'], ['to', 'grip', ',', 'cut', 'off', ',', 'or', 'tear', 'with', 'or', 'as', 'if', 'with', 'the', 'teeth', 'or', 'jaws', '.'], ['any', 'of', 'various', 'controls', 'or', 'devices', 'for', 'regulating', 'or', 'controlling', 'fluid', 'flow', ',', 'pressure', ',', 'temperature', ',', 'etc', '..'], ['convert', 'the', 'genetic', 'information', 'in', '(', 'a', 'strand', 'of', 'dna', ')', 'into', 'a', 'strand', 'of', 'rna', ',', 'especially', 'messenger', 'rna', '.'], ['(', 'computer', 'science', ')', 'the', 'ability', 'to', 'exchange', 'and', 'use', 'information', '(', 'usually', 'in', 'a', 'large', 'heterogeneous', 'network', 'made', 'up', 'of', 'several', 'local', 'area', 'networks', ')', '.'], ['do', 'lacework', '.'], ['react', 'verbally', '.'], ['make', 'high-pitched', 'sounds', '.'], ['become', 'distinct', 'and', 'acquire', 'a', 'different', 'character', '.'], ['put', 'up', 'with', 'something', 'or', 'somebody', 'unpleasant', '.'], ['reproduce', 'or', 'make', 'an', 'exact', 'copy', 'of', '.'], ['thread', 'on', 'or', 'as', 'if', 'on', 'a', 'string', '.'], ['the', 'act', 'of', 'taking', 'possession', 'of', 'or', 'power', 'over', 'something', '.'], ['immerse', 'or', 'be', 'immersed', 'in', 'a', 'boiling', 'liquid', ',', 'often', 'for', 'cooking', 'purposes', '.'], ['the', 'social', 'force', 'that', 'binds', 'you', 'to', 'the', 'courses', 'of', 'action', 'demanded', 'by', 'that', 'force', '.'], ['a', 'formal', 'series', 'of', 'statements', 'showing', 'that', 'if', 'one', 'thing', 'is', 'true', 'something', 'else', 'necessarily', 'follows', 'from', 'it', '.'], ['the', 'statement', 'of', 'a', 'theme', 'in', 'notes', 'of', 'lesser', 'duration', '(', 'usually', 'half', 'the', 'length', 'of', 'the', 'original', ')', '.'], ['change', 'the', 'key', 'of', ',', 'in', 'music', '.'], ['approach', 'a', 'limit', 'as', 'the', 'number', 'of', 'terms', 'increases', 'without', 'limit', '.'], ['behave', 'violently', ',', 'as', 'if', 'in', 'state', 'of', 'a', 'great', 'anger', '.'], ['skip', ',', 'leap', ',', 'or', 'move', 'up', 'and', 'down', 'or', 'sideways', '.'], ['the', 'control', 'of', 'a', 'country', 'by', 'military', 'forces', 'of', 'a', 'foreign', 'power', '.'], ['to', 'move', '(', 'the', 'head', 'or', 'body', ')', 'quickly', 'downwards', 'or', 'away', '.'], ['the', 'part', 'of', 'the', 'body', 'between', 'the', 'neck', 'and', 'the', 'upper', 'arm', '.'], ['try', 'to', 'cure', 'by', 'special', 'care', 'of', 'treatment', ',', 'of', 'an', 'illness', 'or', 'injury', '.'], ['move', 'in', 'circles', '.'], ['provoke', 'or', 'stir', 'up', '.'], ['(', 'statistics', ')', 'a', 'coefficient', 'assigned', 'to', 'elements', 'of', 'a', 'frequency', 'distribution', 'in', 'order', 'to', 'represent', 'their', 'relative', 'importance', '.'], ['be', 'larger', 'in', 'number', ',', 'quantity', ',', 'power', ',', 'status', 'or', 'importance', '.'], ['supply', 'with', 'a', 'constant', 'flow', 'or', 'sprinkling', 'of', 'some', 'liquid', ',', 'for', 'the', 'purpose', 'of', 'cooling', ',', 'cleansing', ',', 'or', 'disinfecting', '.'], ['droop', ',', 'sink', ',', 'or', 'settle', 'from', 'or', 'as', 'if', 'from', 'pressure', 'or', 'loss', 'of', 'tautness', '.'], ['rain', ',', 'hail', ',', 'or', 'snow', 'hard', 'and', 'be', 'very', 'windy', ',', 'often', 'with', 'thunder', 'or', 'lightning', '.'], ['amplify', '(', 'an', 'electron', 'current', ')', 'by', 'causing', 'part', 'of', 'the', 'power', 'in', 'the', 'output', 'circuit', 'to', 'act', 'upon', 'the', 'input', 'circuit', '.'], ['(', 'contract', 'bridge', ')', 'the', 'highest', 'bid', 'becomes', 'the', 'contract', 'setting', 'the', 'number', 'of', 'tricks', 'that', 'the', 'bidder', 'must', 'make', '.'], ['release', '(', 'gas', 'or', 'energy', ')', 'as', 'a', 'result', 'of', 'a', 'chemical', 'reaction', 'or', 'physical', 'decomposition', '.'], ['the', 'atomic', 'weight', 'of', 'an', 'element', 'that', 'has', 'the', 'same', 'combining', 'capacity', 'as', 'a', 'given', 'weight', 'of', 'another', 'element', ';', 'the', 'standard', 'is', '8', 'for', 'oxygen', '.'], ['make', 'a', 'map', 'of', ';', 'show', 'or', 'establish', 'the', 'features', 'of', 'details', 'of', '.'], ['either', 'of', 'the', 'two', 'categories', '(', 'male', 'or', 'female', ')', 'into', 'which', 'most', 'organisms', 'are', 'divided', '.'], ['all', 'or', 'part', 'of', 'a', 'natural', 'object', 'that', 'is', 'collected', 'and', 'preserved', 'as', 'an', 'example', 'of', 'its', 'class', '.'], ['(', 'botany', ')', 'the', 'usually', 'underground', 'organ', 'that', 'lacks', 'buds', 'or', 'leaves', 'or', 'nodes', ';', 'absorbs', 'water', 'and', 'mineral', 'salts', ';', 'usually', 'it', 'anchors', 'the', 'plant', 'to', 'the', 'ground', '.'], ['a', 'europe', 'for', 'all'], ['the', 'people', 'versus', 'putin'], ['the', 'people', 'versus', 'putin'], ['a', 'europe', 'for', 'all'], ['a', 'europe', 'for', 'all'], ['a', 'europe', 'for', 'all'], ['a', 'europe', 'for', 'all'], ['a', 'europe', 'for', 'all'], ['the', 'people', 'versus', 'putin'], ['the', 'people', 'versus', 'putin'], ['the', 'people', 'versus', 'putin'], ['the', 'people', 'versus', 'putin'], ['a', 'europe', 'for', 'all'], ['the', 'people', 'versus', 'putin'], ['the', 'political', 'stock', 'market'], ['the', 'political', 'stock', 'market'], ['the', 'political', 'stock', 'market'], ['the', 'political', 'stock', 'market'], ['the', 'political', 'stock', 'market'], ['the', 'political', 'stock', 'market'], ['the', 'political', 'stock', 'market'], ['some', 'results', 'are', 'remarkable', '.'], ['some', 'results', 'are', 'remarkable', '.'], ['the', 'questions', 'are', 'many', '.'], ['but', 'they', 'were', 'necessary', '.'], ['the', 'questions', 'are', 'many', '.'], ['some', 'results', 'are', 'remarkable', '.'], ['some', 'results', 'are', 'remarkable', '.'], ['the', 'questions', 'are', 'many', '.'], ['the', 'questions', 'are', 'many', '.'], ['the', 'questions', 'are', 'many', '.'], ['some', 'results', 'are', 'remarkable', '.'], ['but', 'they', 'were', 'necessary', '.'], ['the', 'questions', 'are', 'many', '.'], ['the', 'questions', 'are', 'many', '.'], ['some', 'results', 'are', 'remarkable', '.'], ['but', 'they', 'were', 'necessary', '.'], ['but', 'they', 'were', 'necessary', '.'], ['but', 'they', 'were', 'necessary', '.'], ['but', 'they', 'were', 'necessary', '.'], ['some', 'results', 'are', 'remarkable', '.'], ['but', 'they', 'were', 'necessary', '.'], ['other', 'ways', 'are', 'needed', '.'], ['other', 'ways', 'are', 'needed', '.'], ['other', 'ways', 'are', 'needed', '.'], ['other', 'ways', 'are', 'needed', '.'], ['other', 'ways', 'are', 'needed', '.'], ['other', 'ways', 'are', 'needed', '.'], ['other', 'ways', 'are', 'needed', '.'], ['this', 'tendency', 'extends', 'deeper', 'than', 'headscarves', '.'], ['this', 'tendency', 'extends', 'deeper', 'than', 'headscarves', '.'], ['this', 'tendency', 'extends', 'deeper', 'than', 'headscarves', '.'], ['this', 'tendency', 'extends', 'deeper', 'than', 'headscarves', '.'], ['this', 'tendency', 'extends', 'deeper', 'than', 'headscarves', '.'], ['this', 'tendency', 'extends', 'deeper', 'than', 'headscarves', '.'], ['this', 'tendency', 'extends', 'deeper', 'than', 'headscarves', '.'], ['last', 'year', 'he', 'was', 'wanted', 'for', 'murder', '.'], ['last', 'year', 'he', 'was', 'wanted', 'for', 'murder', '.'], ['last', 'year', 'he', 'was', 'wanted', 'for', 'murder', '.'], ['last', 'year', 'he', 'was', 'wanted', 'for', 'murder', '.'], ['last', 'year', 'he', 'was', 'wanted', 'for', 'murder', '.'], ['last', 'year', 'he', 'was', 'wanted', 'for', 'murder', '.'], ['last', 'year', 'he', 'was', 'wanted', 'for', 'murder', '.'], ['they', 'are', 'ignorant', ',', 'poor', ',', 'and', 'sick', '.'], ['they', 'are', 'ignorant', ',', 'poor', ',', 'and', 'sick', '.'], ['they', 'are', 'ignorant', ',', 'poor', ',', 'and', 'sick', '.'], ['they', 'are', 'ignorant', ',', 'poor', ',', 'and', 'sick', '.'], ['they', 'are', 'ignorant', ',', 'poor', ',', 'and', 'sick', '.'], ['they', 'are', 'ignorant', ',', 'poor', ',', 'and', 'sick', '.'], ['they', 'are', 'ignorant', ',', 'poor', ',', 'and', 'sick', '.'], ['this', 'gross', 'error', 'is', 'leading', 'russia', 'to', 'political', 'ruin', '.'], ['this', 'gross', 'error', 'is', 'leading', 'russia', 'to', 'political', 'ruin', '.'], ['this', 'gross', 'error', 'is', 'leading', 'russia', 'to', 'political', 'ruin', '.'], ['this', 'gross', 'error', 'is', 'leading', 'russia', 'to', 'political', 'ruin', '.'], ['this', 'gross', 'error', 'is', 'leading', 'russia', 'to', 'political', 'ruin', '.'], ['this', 'gross', 'error', 'is', 'leading', 'russia', 'to', 'political', 'ruin', '.'], ['this', 'gross', 'error', 'is', 'leading', 'russia', 'to', 'political', 'ruin', '.'], ['ahmadinejad', 'is', 'embarking', 'on', 'an', 'adventure', ';', 'bernanke', 'is', 'not', '.'], ['this', 'has', 'left', 'the', 'conservative', 'opposition', 'without', 'an', 'effective', 'response', '.'], ['ahmadinejad', 'is', 'embarking', 'on', 'an', 'adventure', ';', 'bernanke', 'is', 'not', '.'], ['ahmadinejad', 'is', 'embarking', 'on', 'an', 'adventure', ';', 'bernanke', 'is', 'not', '.'], ['this', 'has', 'left', 'the', 'conservative', 'opposition', 'without', 'an', 'effective', 'response', '.'], ['ahmadinejad', 'is', 'embarking', 'on', 'an', 'adventure', ';', 'bernanke', 'is', 'not', '.'], ['this', 'has', 'left', 'the', 'conservative', 'opposition', 'without', 'an', 'effective', 'response', '.'], ['this', 'has', 'left', 'the', 'conservative', 'opposition', 'without', 'an', 'effective', 'response', '.'], ['ahmadinejad', 'is', 'embarking', 'on', 'an', 'adventure', ';', 'bernanke', 'is', 'not', '.'], ['this', 'has', 'left', 'the', 'conservative', 'opposition', 'without', 'an', 'effective', 'response', '.'], ['ahmadinejad', 'is', 'embarking', 'on', 'an', 'adventure', ';', 'bernanke', 'is', 'not', '.'], ['this', 'has', 'left', 'the', 'conservative', 'opposition', 'without', 'an', 'effective', 'response', '.'], ['this', 'has', 'left', 'the', 'conservative', 'opposition', 'without', 'an', 'effective', 'response', '.'], ['ahmadinejad', 'is', 'embarking', 'on', 'an', 'adventure', ';', 'bernanke', 'is', 'not', '.'], ['gays', 'and', 'other', '"', 'modern', '"', 'practices', 'are', 'rejected', 'as', 'self-indulgent', '.'], ['in', 'other', 'areas', ',', 'sharia', 'law', 'is', 'being', 'introduced', 'by', 'force', '.'], ['in', 'other', 'areas', ',', 'sharia', 'law', 'is', 'being', 'introduced', 'by', 'force', '.'], ['in', 'other', 'areas', ',', 'sharia', 'law', 'is', 'being', 'introduced', 'by', 'force', '.'], ['in', 'other', 'areas', ',', 'sharia', 'law', 'is', 'being', 'introduced', 'by', 'force', '.'], ['in', 'other', 'areas', ',', 'sharia', 'law', 'is', 'being', 'introduced', 'by', 'force', '.'], ['in', 'other', 'areas', ',', 'sharia', 'law', 'is', 'being', 'introduced', 'by', 'force', '.'], ['in', 'other', 'areas', ',', 'sharia', 'law', 'is', 'being', 'introduced', 'by', 'force', '.'], ['he', 'did', ',', 'but', 'the', 'initiative', 'did', 'not', 'get', 'very', 'far', '.'], ['each', 'time', ',', 'radical', 'improvements', 'in', 'technology', 'made', 'the', 'threat', 'evaporate', '.'], ['he', 'did', ',', 'but', 'the', 'initiative', 'did', 'not', 'get', 'very', 'far', '.'], ['each', 'time', ',', 'radical', 'improvements', 'in', 'technology', 'made', 'the', 'threat', 'evaporate', '.'], ['he', 'did', ',', 'but', 'the', 'initiative', 'did', 'not', 'get', 'very', 'far', '.'], ['he', 'did', ',', 'but', 'the', 'initiative', 'did', 'not', 'get', 'very', 'far', '.'], ['he', 'did', ',', 'but', 'the', 'initiative', 'did', 'not', 'get', 'very', 'far', '.'], ['he', 'did', ',', 'but', 'the', 'initiative', 'did', 'not', 'get', 'very', 'far', '.'], ['each', 'time', ',', 'radical', 'improvements', 'in', 'technology', 'made', 'the', 'threat', 'evaporate', '.'], ['each', 'time', ',', 'radical', 'improvements', 'in', 'technology', 'made', 'the', 'threat', 'evaporate', '.'], ['each', 'time', ',', 'radical', 'improvements', 'in', 'technology', 'made', 'the', 'threat', 'evaporate', '.'], ['each', 'time', ',', 'radical', 'improvements', 'in', 'technology', 'made', 'the', 'threat', 'evaporate', '.'], ['he', 'did', ',', 'but', 'the', 'initiative', 'did', 'not', 'get', 'very', 'far', '.'], ['gays', 'and', 'other', '"', 'modern', '"', 'practices', 'are', 'rejected', 'as', 'self-indulgent', '.'], ['gays', 'and', 'other', '"', 'modern', '"', 'practices', 'are', 'rejected', 'as', 'self-indulgent', '.'], ['gays', 'and', 'other', '"', 'modern', '"', 'practices', 'are', 'rejected', 'as', 'self-indulgent', '.'], ['each', 'time', ',', 'radical', 'improvements', 'in', 'technology', 'made', 'the', 'threat', 'evaporate', '.'], ['gays', 'and', 'other', '"', 'modern', '"', 'practices', 'are', 'rejected', 'as', 'self-indulgent', '.'], ['none', 'of', 'this', 'absolves', 'rich', 'countries', 'of', 'their', 'responsibility', 'to', 'help', '.'], ['none', 'of', 'this', 'absolves', 'rich', 'countries', 'of', 'their', 'responsibility', 'to', 'help', '.'], ['none', 'of', 'this', 'absolves', 'rich', 'countries', 'of', 'their', 'responsibility', 'to', 'help', '.'], ['gays', 'and', 'other', '"', 'modern', '"', 'practices', 'are', 'rejected', 'as', 'self-indulgent', '.'], ['none', 'of', 'this', 'absolves', 'rich', 'countries', 'of', 'their', 'responsibility', 'to', 'help', '.'], ['gays', 'and', 'other', '"', 'modern', '"', 'practices', 'are', 'rejected', 'as', 'self-indulgent', '.'], ['none', 'of', 'this', 'absolves', 'rich', 'countries', 'of', 'their', 'responsibility', 'to', 'help', '.'], ['none', 'of', 'this', 'absolves', 'rich', 'countries', 'of', 'their', 'responsibility', 'to', 'help', '.'], ['none', 'of', 'this', 'absolves', 'rich', 'countries', 'of', 'their', 'responsibility', 'to', 'help', '.'], ['putin', ',', 'too', ',', 'also', 'seems', 'to', 'suspect', 'his', 'regime', "'s", 'impotence', '.'], ['being', 'a', 'muslim', 'and', 'being', 'an', 'islamist', 'are', 'not', 'the', 'same', 'thing', '.'], ['more', 'than', 'that', ',', 'the', 'divisions', 'are', 'reproduced', 'at', 'the', 'local', 'level', '.'], ['more', 'than', 'that', ',', 'the', 'divisions', 'are', 'reproduced', 'at', 'the', 'local', 'level', '.'], ['iraq', "'s", 'future', 'depends', 'directly', 'on', 'the', 'fate', 'of', 'iraqi', 'oil', 'production', '.'], ['more', 'than', 'that', ',', 'the', 'divisions', 'are', 'reproduced', 'at', 'the', 'local', 'level', '.'], ['being', 'a', 'muslim', 'and', 'being', 'an', 'islamist', 'are', 'not', 'the', 'same', 'thing', '.'], ['indeed', ',', 'intolerance', 'goes', 'right', 'to', 'the', 'top', 'of', 'the', 'turkish', 'government', '.'], ['being', 'a', 'muslim', 'and', 'being', 'an', 'islamist', 'are', 'not', 'the', 'same', 'thing', '.'], ['being', 'a', 'muslim', 'and', 'being', 'an', 'islamist', 'are', 'not', 'the', 'same', 'thing', '.'], ['iraq', "'s", 'future', 'depends', 'directly', 'on', 'the', 'fate', 'of', 'iraqi', 'oil', 'production', '.'], ['being', 'a', 'muslim', 'and', 'being', 'an', 'islamist', 'are', 'not', 'the', 'same', 'thing', '.'], ['today', 'there', 'are', 'two', 'plausible', 'ways', 'to', 'proceed', 'against', 'a', 'deposed', 'tyrant', '.'], ['today', 'there', 'are', 'two', 'plausible', 'ways', 'to', 'proceed', 'against', 'a', 'deposed', 'tyrant', '.'], ['more', 'than', 'that', ',', 'the', 'divisions', 'are', 'reproduced', 'at', 'the', 'local', 'level', '.'], ['more', 'than', 'that', ',', 'the', 'divisions', 'are', 'reproduced', 'at', 'the', 'local', 'level', '.'], ['putin', ',', 'too', ',', 'also', 'seems', 'to', 'suspect', 'his', 'regime', "'s", 'impotence', '.'], ['being', 'a', 'muslim', 'and', 'being', 'an', 'islamist', 'are', 'not', 'the', 'same', 'thing', '.'], ['being', 'a', 'muslim', 'and', 'being', 'an', 'islamist', 'are', 'not', 'the', 'same', 'thing', '.'], ['iraq', "'s", 'future', 'depends', 'directly', 'on', 'the', 'fate', 'of', 'iraqi', 'oil', 'production', '.'], ['more', 'than', 'that', ',', 'the', 'divisions', 'are', 'reproduced', 'at', 'the', 'local', 'level', '.'], ['putin', ',', 'too', ',', 'also', 'seems', 'to', 'suspect', 'his', 'regime', "'s", 'impotence', '.'], ['putin', ',', 'too', ',', 'also', 'seems', 'to', 'suspect', 'his', 'regime', "'s", 'impotence', '.'], ['more', 'than', 'that', ',', 'the', 'divisions', 'are', 'reproduced', 'at', 'the', 'local', 'level', '.'], ['today', 'there', 'are', 'two', 'plausible', 'ways', 'to', 'proceed', 'against', 'a', 'deposed', 'tyrant', '.'], ['that', 'is', 'the', 'spirit', 'in', 'which', 'the', 'european', 'constitution', 'must', 'be', 'written', '.'], ['today', 'there', 'are', 'two', 'plausible', 'ways', 'to', 'proceed', 'against', 'a', 'deposed', 'tyrant', '.'], ['putin', ',', 'too', ',', 'also', 'seems', 'to', 'suspect', 'his', 'regime', "'s", 'impotence', '.'], ['indeed', ',', 'intolerance', 'goes', 'right', 'to', 'the', 'top', 'of', 'the', 'turkish', 'government', '.'], ['iraq', "'s", 'future', 'depends', 'directly', 'on', 'the', 'fate', 'of', 'iraqi', 'oil', 'production', '.'], ['that', 'is', 'the', 'spirit', 'in', 'which', 'the', 'european', 'constitution', 'must', 'be', 'written', '.'], ['that', 'is', 'the', 'spirit', 'in', 'which', 'the', 'european', 'constitution', 'must', 'be', 'written', '.'], ['indeed', ',', 'intolerance', 'goes', 'right', 'to', 'the', 'top', 'of', 'the', 'turkish', 'government', '.'], ['indeed', ',', 'intolerance', 'goes', 'right', 'to', 'the', 'top', 'of', 'the', 'turkish', 'government', '.'], ['that', 'is', 'the', 'spirit', 'in', 'which', 'the', 'european', 'constitution', 'must', 'be', 'written', '.'], ['putin', ',', 'too', ',', 'also', 'seems', 'to', 'suspect', 'his', 'regime', "'s", 'impotence', '.'], ['that', 'is', 'the', 'spirit', 'in', 'which', 'the', 'european', 'constitution', 'must', 'be', 'written', '.'], ['iraq', "'s", 'future', 'depends', 'directly', 'on', 'the', 'fate', 'of', 'iraqi', 'oil', 'production', '.'], ['indeed', ',', 'intolerance', 'goes', 'right', 'to', 'the', 'top', 'of', 'the', 'turkish', 'government', '.'], ['today', 'there', 'are', 'two', 'plausible', 'ways', 'to', 'proceed', 'against', 'a', 'deposed', 'tyrant', '.'], ['tocqueville', 'believed', 'that', 'there', 'are', 'no', 'effective', 'long-term', 'constraints', 'on', 'this', 'tendency', '.'], ['that', 'is', 'the', 'spirit', 'in', 'which', 'the', 'european', 'constitution', 'must', 'be', 'written', '.'], ['today', 'there', 'are', 'two', 'plausible', 'ways', 'to', 'proceed', 'against', 'a', 'deposed', 'tyrant', '.'], ['indeed', ',', 'intolerance', 'goes', 'right', 'to', 'the', 'top', 'of', 'the', 'turkish', 'government', '.'], ['indeed', ',', 'intolerance', 'goes', 'right', 'to', 'the', 'top', 'of', 'the', 'turkish', 'government', '.'], ['that', 'is', 'the', 'spirit', 'in', 'which', 'the', 'european', 'constitution', 'must', 'be', 'written', '.'], ['iraq', "'s", 'future', 'depends', 'directly', 'on', 'the', 'fate', 'of', 'iraqi', 'oil', 'production', '.'], ['iraq', "'s", 'future', 'depends', 'directly', 'on', 'the', 'fate', 'of', 'iraqi', 'oil', 'production', '.'], ['putin', ',', 'too', ',', 'also', 'seems', 'to', 'suspect', 'his', 'regime', "'s", 'impotence', '.'], ['tocqueville', 'believed', 'that', 'there', 'are', 'no', 'effective', 'long-term', 'constraints', 'on', 'this', 'tendency', '.'], ['tocqueville', 'believed', 'that', 'there', 'are', 'no', 'effective', 'long-term', 'constraints', 'on', 'this', 'tendency', '.'], ['today', 'there', 'are', 'two', 'plausible', 'ways', 'to', 'proceed', 'against', 'a', 'deposed', 'tyrant', '.'], ['tocqueville', 'believed', 'that', 'there', 'are', 'no', 'effective', 'long-term', 'constraints', 'on', 'this', 'tendency', '.'], ['tocqueville', 'believed', 'that', 'there', 'are', 'no', 'effective', 'long-term', 'constraints', 'on', 'this', 'tendency', '.'], ['tocqueville', 'believed', 'that', 'there', 'are', 'no', 'effective', 'long-term', 'constraints', 'on', 'this', 'tendency', '.'], ['tocqueville', 'believed', 'that', 'there', 'are', 'no', 'effective', 'long-term', 'constraints', 'on', 'this', 'tendency', '.'], ['will', 'it', 'give', 'us', 'the', 'right', 'to', 'divorce', 'the', 'husbands', 'who', 'abandon', 'us', '?'], ['but', 'if', 'europe', "'s", 'performance', 'continues', 'to', 'decline', ',', 'would', 'this', 'perception', 'survive', '?'], ['but', 'if', 'europe', "'s", 'performance', 'continues', 'to', 'decline', ',', 'would', 'this', 'perception', 'survive', '?'], ['but', 'if', 'europe', "'s", 'performance', 'continues', 'to', 'decline', ',', 'would', 'this', 'perception', 'survive', '?'], ['but', 'if', 'europe', "'s", 'performance', 'continues', 'to', 'decline', ',', 'would', 'this', 'perception', 'survive', '?'], ['but', 'if', 'europe', "'s", 'performance', 'continues', 'to', 'decline', ',', 'would', 'this', 'perception', 'survive', '?'], ['but', 'if', 'europe', "'s", 'performance', 'continues', 'to', 'decline', ',', 'would', 'this', 'perception', 'survive', '?'], ['will', 'it', 'give', 'us', 'the', 'right', 'to', 'divorce', 'the', 'husbands', 'who', 'abandon', 'us', '?'], ['will', 'it', 'give', 'us', 'the', 'right', 'to', 'divorce', 'the', 'husbands', 'who', 'abandon', 'us', '?'], ['but', 'if', 'europe', "'s", 'performance', 'continues', 'to', 'decline', ',', 'would', 'this', 'perception', 'survive', '?'], ['each', 'side', 'had', 'quite', 'different', 'theories', 'and', 'explanations', 'of', 'what', 'was', 'going', 'on', '.'], ['will', 'it', 'give', 'us', 'the', 'right', 'to', 'divorce', 'the', 'husbands', 'who', 'abandon', 'us', '?'], ['each', 'side', 'had', 'quite', 'different', 'theories', 'and', 'explanations', 'of', 'what', 'was', 'going', 'on', '.'], ['each', 'side', 'had', 'quite', 'different', 'theories', 'and', 'explanations', 'of', 'what', 'was', 'going', 'on', '.'], ['each', 'side', 'had', 'quite', 'different', 'theories', 'and', 'explanations', 'of', 'what', 'was', 'going', 'on', '.'], ['each', 'side', 'had', 'quite', 'different', 'theories', 'and', 'explanations', 'of', 'what', 'was', 'going', 'on', '.'], ['each', 'side', 'had', 'quite', 'different', 'theories', 'and', 'explanations', 'of', 'what', 'was', 'going', 'on', '.'], ['each', 'side', 'had', 'quite', 'different', 'theories', 'and', 'explanations', 'of', 'what', 'was', 'going', 'on', '.'], ['will', 'it', 'give', 'us', 'the', 'right', 'to', 'divorce', 'the', 'husbands', 'who', 'abandon', 'us', '?'], ['today', "'s", 'great', 'pax', 'europa', 'and', 'today', "'s", 'pan-european', 'prosperity', 'depend', 'on', 'this', '.'], ['will', 'it', 'give', 'us', 'the', 'right', 'to', 'divorce', 'the', 'husbands', 'who', 'abandon', 'us', '?'], ['today', "'s", 'great', 'pax', 'europa', 'and', 'today', "'s", 'pan-european', 'prosperity', 'depend', 'on', 'this', '.'], ['today', "'s", 'great', 'pax', 'europa', 'and', 'today', "'s", 'pan-european', 'prosperity', 'depend', 'on', 'this', '.'], ['today', "'s", 'great', 'pax', 'europa', 'and', 'today', "'s", 'pan-european', 'prosperity', 'depend', 'on', 'this', '.'], ['will', 'it', 'give', 'us', 'the', 'right', 'to', 'divorce', 'the', 'husbands', 'who', 'abandon', 'us', '?'], ['today', "'s", 'great', 'pax', 'europa', 'and', 'today', "'s", 'pan-european', 'prosperity', 'depend', 'on', 'this', '.'], ['today', "'s", 'great', 'pax', 'europa', 'and', 'today', "'s", 'pan-european', 'prosperity', 'depend', 'on', 'this', '.'], ['today', "'s", 'great', 'pax', 'europa', 'and', 'today', "'s", 'pan-european', 'prosperity', 'depend', 'on', 'this', '.'], ['yet', 'a', 'process', 'of', 'de-institutionalization', 'of', 'religious', 'experience', 'is', 'also', 'taking', 'place', 'within', 'islam', '.'], ['western', 'europeans', ',', 'who', 'have', 'been', 'spared', 'this', 'legacy', ',', 'should', 'heed', 'our', 'warnings', '.'], ['yet', 'a', 'process', 'of', 'de-institutionalization', 'of', 'religious', 'experience', 'is', 'also', 'taking', 'place', 'within', 'islam', '.'], ['yet', 'a', 'process', 'of', 'de-institutionalization', 'of', 'religious', 'experience', 'is', 'also', 'taking', 'place', 'within', 'islam', '.'], ['western', 'europeans', ',', 'who', 'have', 'been', 'spared', 'this', 'legacy', ',', 'should', 'heed', 'our', 'warnings', '.'], ['yet', 'a', 'process', 'of', 'de-institutionalization', 'of', 'religious', 'experience', 'is', 'also', 'taking', 'place', 'within', 'islam', '.'], ['yet', 'a', 'process', 'of', 'de-institutionalization', 'of', 'religious', 'experience', 'is', 'also', 'taking', 'place', 'within', 'islam', '.'], ['western', 'europeans', ',', 'who', 'have', 'been', 'spared', 'this', 'legacy', ',', 'should', 'heed', 'our', 'warnings', '.'], ['yet', 'a', 'process', 'of', 'de-institutionalization', 'of', 'religious', 'experience', 'is', 'also', 'taking', 'place', 'within', 'islam', '.'], ['the', 'bottom', 'line', 'is', 'that', 'when', 'interest', 'rates', 'rise', ',', 'asset', 'values', 'will', 'fall', '.'], ['the', 'bottom', 'line', 'is', 'that', 'when', 'interest', 'rates', 'rise', ',', 'asset', 'values', 'will', 'fall', '.'], ['yet', 'a', 'process', 'of', 'de-institutionalization', 'of', 'religious', 'experience', 'is', 'also', 'taking', 'place', 'within', 'islam', '.'], ['western', 'europeans', ',', 'who', 'have', 'been', 'spared', 'this', 'legacy', ',', 'should', 'heed', 'our', 'warnings', '.'], ['but', 'us', 'stock', 'prices', 'fell', 'only', '5.2', '%', 'between', 'may', '9', 'and', 'may', '24', '.'], ['but', 'us', 'stock', 'prices', 'fell', 'only', '5.2', '%', 'between', 'may', '9', 'and', 'may', '24', '.'], ['the', 'bottom', 'line', 'is', 'that', 'when', 'interest', 'rates', 'rise', ',', 'asset', 'values', 'will', 'fall', '.'], ['western', 'europeans', ',', 'who', 'have', 'been', 'spared', 'this', 'legacy', ',', 'should', 'heed', 'our', 'warnings', '.'], ['western', 'europeans', ',', 'who', 'have', 'been', 'spared', 'this', 'legacy', ',', 'should', 'heed', 'our', 'warnings', '.'], ['the', 'bottom', 'line', 'is', 'that', 'when', 'interest', 'rates', 'rise', ',', 'asset', 'values', 'will', 'fall', '.'], ['but', 'us', 'stock', 'prices', 'fell', 'only', '5.2', '%', 'between', 'may', '9', 'and', 'may', '24', '.'], ['the', 'bottom', 'line', 'is', 'that', 'when', 'interest', 'rates', 'rise', ',', 'asset', 'values', 'will', 'fall', '.'], ['but', 'us', 'stock', 'prices', 'fell', 'only', '5.2', '%', 'between', 'may', '9', 'and', 'may', '24', '.'], ['but', 'us', 'stock', 'prices', 'fell', 'only', '5.2', '%', 'between', 'may', '9', 'and', 'may', '24', '.'], ['indeed', ',', 'the', 'french', 'philosopher', 'marcel', 'gauchet', 'entitled', 'a', 'recent', 'book', 'democracy', 'against', 'itself', '.'], ['the', 'bottom', 'line', 'is', 'that', 'when', 'interest', 'rates', 'rise', ',', 'asset', 'values', 'will', 'fall', '.'], ['western', 'europeans', ',', 'who', 'have', 'been', 'spared', 'this', 'legacy', ',', 'should', 'heed', 'our', 'warnings', '.'], ['indeed', ',', 'the', 'french', 'philosopher', 'marcel', 'gauchet', 'entitled', 'a', 'recent', 'book', 'democracy', 'against', 'itself', '.'], ['but', 'us', 'stock', 'prices', 'fell', 'only', '5.2', '%', 'between', 'may', '9', 'and', 'may', '24', '.'], ['the', 'bottom', 'line', 'is', 'that', 'when', 'interest', 'rates', 'rise', ',', 'asset', 'values', 'will', 'fall', '.'], ['indeed', ',', 'the', 'french', 'philosopher', 'marcel', 'gauchet', 'entitled', 'a', 'recent', 'book', 'democracy', 'against', 'itself', '.'], ['indeed', ',', 'the', 'french', 'philosopher', 'marcel', 'gauchet', 'entitled', 'a', 'recent', 'book', 'democracy', 'against', 'itself', '.'], ['indeed', ',', 'the', 'french', 'philosopher', 'marcel', 'gauchet', 'entitled', 'a', 'recent', 'book', 'democracy', 'against', 'itself', '.'], ['indeed', ',', 'the', 'french', 'philosopher', 'marcel', 'gauchet', 'entitled', 'a', 'recent', 'book', 'democracy', 'against', 'itself', '.'], ['but', 'us', 'stock', 'prices', 'fell', 'only', '5.2', '%', 'between', 'may', '9', 'and', 'may', '24', '.'], ['indeed', ',', 'the', 'french', 'philosopher', 'marcel', 'gauchet', 'entitled', 'a', 'recent', 'book', 'democracy', 'against', 'itself', '.'], ['and', ',', 'perhaps', 'most', 'importantly', ',', 'ahmadinejad', 'is', 'a', 'destabilizing', 'influence', ';', 'bernanke', 'is', 'not', '.'], ['the', 'book', "'s", 'success', 'is', 'itself', 'a', 'sign', 'of', 'a', 'kind', 'of', '"', 'malaise', '.', '"'], ['the', 'book', "'s", 'success', 'is', 'itself', 'a', 'sign', 'of', 'a', 'kind', 'of', '"', 'malaise', '.', '"'], ['and', ',', 'perhaps', 'most', 'importantly', ',', 'ahmadinejad', 'is', 'a', 'destabilizing', 'influence', ';', 'bernanke', 'is', 'not', '.'], ['and', ',', 'perhaps', 'most', 'importantly', ',', 'ahmadinejad', 'is', 'a', 'destabilizing', 'influence', ';', 'bernanke', 'is', 'not', '.'], ['the', 'book', "'s", 'success', 'is', 'itself', 'a', 'sign', 'of', 'a', 'kind', 'of', '"', 'malaise', '.', '"'], ['and', ',', 'perhaps', 'most', 'importantly', ',', 'ahmadinejad', 'is', 'a', 'destabilizing', 'influence', ';', 'bernanke', 'is', 'not', '.'], ['and', ',', 'perhaps', 'most', 'importantly', ',', 'ahmadinejad', 'is', 'a', 'destabilizing', 'influence', ';', 'bernanke', 'is', 'not', '.'], ['the', 'book', "'s", 'success', 'is', 'itself', 'a', 'sign', 'of', 'a', 'kind', 'of', '"', 'malaise', '.', '"'], ['the', 'book', "'s", 'success', 'is', 'itself', 'a', 'sign', 'of', 'a', 'kind', 'of', '"', 'malaise', '.', '"'], ['the', 'book', "'s", 'success', 'is', 'itself', 'a', 'sign', 'of', 'a', 'kind', 'of', '"', 'malaise', '.', '"'], ['but', 'the', 'returns', 'that', 'make', 'us', 'stock', 'markets', 'so', 'attractive', 'reflect', 'a', 'delicate', 'political', 'balance', '.'], ['and', ',', 'perhaps', 'most', 'importantly', ',', 'ahmadinejad', 'is', 'a', 'destabilizing', 'influence', ';', 'bernanke', 'is', 'not', '.'], ['the', 'book', "'s", 'success', 'is', 'itself', 'a', 'sign', 'of', 'a', 'kind', 'of', '"', 'malaise', '.', '"'], ['and', ',', 'perhaps', 'most', 'importantly', ',', 'ahmadinejad', 'is', 'a', 'destabilizing', 'influence', ';', 'bernanke', 'is', 'not', '.'], ['only', 'a', 'month', 'ago', ',', 'mubarak', 'dismissed', 'demands', 'for', 'constitutional', 'reform', 'as', '"', 'futile', '.', '"'], ['the', 'answer', 'lies', 'in', 'the', 'support', 'that', 'indonesian', 'pro-democracy', 'activists', 'can', 'generate', 'domestically', 'and', 'internationally', '.'], ['only', 'a', 'month', 'ago', ',', 'mubarak', 'dismissed', 'demands', 'for', 'constitutional', 'reform', 'as', '"', 'futile', '.', '"'], ['the', 'answer', 'lies', 'in', 'the', 'support', 'that', 'indonesian', 'pro-democracy', 'activists', 'can', 'generate', 'domestically', 'and', 'internationally', '.'], ['but', 'the', 'returns', 'that', 'make', 'us', 'stock', 'markets', 'so', 'attractive', 'reflect', 'a', 'delicate', 'political', 'balance', '.'], ['but', 'the', 'returns', 'that', 'make', 'us', 'stock', 'markets', 'so', 'attractive', 'reflect', 'a', 'delicate', 'political', 'balance', '.'], ['the', 'answer', 'lies', 'in', 'the', 'support', 'that', 'indonesian', 'pro-democracy', 'activists', 'can', 'generate', 'domestically', 'and', 'internationally', '.'], ['only', 'a', 'month', 'ago', ',', 'mubarak', 'dismissed', 'demands', 'for', 'constitutional', 'reform', 'as', '"', 'futile', '.', '"'], ['only', 'a', 'month', 'ago', ',', 'mubarak', 'dismissed', 'demands', 'for', 'constitutional', 'reform', 'as', '"', 'futile', '.', '"'], ['the', 'answer', 'lies', 'in', 'the', 'support', 'that', 'indonesian', 'pro-democracy', 'activists', 'can', 'generate', 'domestically', 'and', 'internationally', '.'], ['but', 'the', 'returns', 'that', 'make', 'us', 'stock', 'markets', 'so', 'attractive', 'reflect', 'a', 'delicate', 'political', 'balance', '.'], ['the', 'answer', 'lies', 'in', 'the', 'support', 'that', 'indonesian', 'pro-democracy', 'activists', 'can', 'generate', 'domestically', 'and', 'internationally', '.'], ['a', 'defeat', 'on', 'alstom', 'would', 'have', 'profound', 'consequences', 'in', 'europe--and', 'not', 'only', 'for', 'competition', 'policy', '.'], ['the', 'answer', 'lies', 'in', 'the', 'support', 'that', 'indonesian', 'pro-democracy', 'activists', 'can', 'generate', 'domestically', 'and', 'internationally', '.'], ['only', 'a', 'month', 'ago', ',', 'mubarak', 'dismissed', 'demands', 'for', 'constitutional', 'reform', 'as', '"', 'futile', '.', '"'], ['but', 'the', 'returns', 'that', 'make', 'us', 'stock', 'markets', 'so', 'attractive', 'reflect', 'a', 'delicate', 'political', 'balance', '.'], ['but', 'the', 'returns', 'that', 'make', 'us', 'stock', 'markets', 'so', 'attractive', 'reflect', 'a', 'delicate', 'political', 'balance', '.'], ['only', 'a', 'month', 'ago', ',', 'mubarak', 'dismissed', 'demands', 'for', 'constitutional', 'reform', 'as', '"', 'futile', '.', '"'], ['only', 'a', 'month', 'ago', ',', 'mubarak', 'dismissed', 'demands', 'for', 'constitutional', 'reform', 'as', '"', 'futile', '.', '"'], ['a', 'defeat', 'on', 'alstom', 'would', 'have', 'profound', 'consequences', 'in', 'europe--and', 'not', 'only', 'for', 'competition', 'policy', '.'], ['a', 'defeat', 'on', 'alstom', 'would', 'have', 'profound', 'consequences', 'in', 'europe--and', 'not', 'only', 'for', 'competition', 'policy', '.'], ['but', 'the', 'returns', 'that', 'make', 'us', 'stock', 'markets', 'so', 'attractive', 'reflect', 'a', 'delicate', 'political', 'balance', '.'], ['the', 'answer', 'lies', 'in', 'the', 'support', 'that', 'indonesian', 'pro-democracy', 'activists', 'can', 'generate', 'domestically', 'and', 'internationally', '.'], ['a', 'defeat', 'on', 'alstom', 'would', 'have', 'profound', 'consequences', 'in', 'europe--and', 'not', 'only', 'for', 'competition', 'policy', '.'], ['a', 'defeat', 'on', 'alstom', 'would', 'have', 'profound', 'consequences', 'in', 'europe--and', 'not', 'only', 'for', 'competition', 'policy', '.'], ['a', 'defeat', 'on', 'alstom', 'would', 'have', 'profound', 'consequences', 'in', 'europe--and', 'not', 'only', 'for', 'competition', 'policy', '.'], ['a', 'defeat', 'on', 'alstom', 'would', 'have', 'profound', 'consequences', 'in', 'europe--and', 'not', 'only', 'for', 'competition', 'policy', '.'], ['what', 'we', 'are', 'witnessing', 'today', 'is', 'a', 'shift', 'from', 'a', 'muslim', 'identity', 'to', 'an', 'islamist', 'identity', '.'], ['but', ',', 'like', 'the', 'union', 'itself', ',', 'it', 'will', 'be', 'built', 'and', 'it', 'will', 'be', 'done', '.'], ['what', 'we', 'are', 'witnessing', 'today', 'is', 'a', 'shift', 'from', 'a', 'muslim', 'identity', 'to', 'an', 'islamist', 'identity', '.'], ['after', 'all', ',', 'they', 'recommend', 'the', 'policies', 'that', 'politicians', 'may', 'or', 'may', 'not', 'want', 'to', 'consider', '.'], ['but', ',', 'like', 'the', 'union', 'itself', ',', 'it', 'will', 'be', 'built', 'and', 'it', 'will', 'be', 'done', '.'], ['but', ',', 'like', 'the', 'union', 'itself', ',', 'it', 'will', 'be', 'built', 'and', 'it', 'will', 'be', 'done', '.'], ['after', 'all', ',', 'they', 'recommend', 'the', 'policies', 'that', 'politicians', 'may', 'or', 'may', 'not', 'want', 'to', 'consider', '.'], ['after', 'all', ',', 'they', 'recommend', 'the', 'policies', 'that', 'politicians', 'may', 'or', 'may', 'not', 'want', 'to', 'consider', '.'], ['jobs', ',', 'in', 'particular', ',', 'will', 'help', 'young', 'people', 'to', 'create', 'new', 'lives', 'through', 'serious', 'work', '.'], ['what', 'we', 'are', 'witnessing', 'today', 'is', 'a', 'shift', 'from', 'a', 'muslim', 'identity', 'to', 'an', 'islamist', 'identity', '.'], ['what', 'we', 'are', 'witnessing', 'today', 'is', 'a', 'shift', 'from', 'a', 'muslim', 'identity', 'to', 'an', 'islamist', 'identity', '.'], ['after', 'all', ',', 'they', 'recommend', 'the', 'policies', 'that', 'politicians', 'may', 'or', 'may', 'not', 'want', 'to', 'consider', '.'], ['what', 'we', 'are', 'witnessing', 'today', 'is', 'a', 'shift', 'from', 'a', 'muslim', 'identity', 'to', 'an', 'islamist', 'identity', '.'], ['what', 'we', 'are', 'witnessing', 'today', 'is', 'a', 'shift', 'from', 'a', 'muslim', 'identity', 'to', 'an', 'islamist', 'identity', '.'], ['what', 'we', 'are', 'witnessing', 'today', 'is', 'a', 'shift', 'from', 'a', 'muslim', 'identity', 'to', 'an', 'islamist', 'identity', '.'], ['those', 'who', 'know', 'little', 'about', 'modern', 'factory', 'farming', 'may', 'wonder', 'why', 'such', 'legislation', 'would', 'be', 'necessary', '.'], ['but', ',', 'like', 'the', 'union', 'itself', ',', 'it', 'will', 'be', 'built', 'and', 'it', 'will', 'be', 'done', '.'], ['but', ',', 'like', 'the', 'union', 'itself', ',', 'it', 'will', 'be', 'built', 'and', 'it', 'will', 'be', 'done', '.'], ['after', 'all', ',', 'they', 'recommend', 'the', 'policies', 'that', 'politicians', 'may', 'or', 'may', 'not', 'want', 'to', 'consider', '.'], ['after', 'all', ',', 'they', 'recommend', 'the', 'policies', 'that', 'politicians', 'may', 'or', 'may', 'not', 'want', 'to', 'consider', '.'], ['this', 'is', 'a', 'clear', 'if', 'implicit', 'repudiation', 'of', 'mubarak', ',', 'the', 'sole', 'ruler', 'for', '24', 'years', '.'], ['but', ',', 'like', 'the', 'union', 'itself', ',', 'it', 'will', 'be', 'built', 'and', 'it', 'will', 'be', 'done', '.'], ['jobs', ',', 'in', 'particular', ',', 'will', 'help', 'young', 'people', 'to', 'create', 'new', 'lives', 'through', 'serious', 'work', '.'], ['putin', "'s", 'russia', 'has', 'already', 'lost', '12', 'leading', 'journalists', 'to', 'murder', 'in', 'the', 'past', 'six', 'years', '.'], ['after', 'all', ',', 'they', 'recommend', 'the', 'policies', 'that', 'politicians', 'may', 'or', 'may', 'not', 'want', 'to', 'consider', '.'], ['putin', "'s", 'russia', 'has', 'already', 'lost', '12', 'leading', 'journalists', 'to', 'murder', 'in', 'the', 'past', 'six', 'years', '.'], ['putin', "'s", 'russia', 'has', 'already', 'lost', '12', 'leading', 'journalists', 'to', 'murder', 'in', 'the', 'past', 'six', 'years', '.'], ['those', 'who', 'know', 'little', 'about', 'modern', 'factory', 'farming', 'may', 'wonder', 'why', 'such', 'legislation', 'would', 'be', 'necessary', '.'], ['pro-market', 'economists', 'don', "'t", 'object', 'to', 'corporations', 'that', 'blatantly', 'use', 'snob', 'appeal', 'to', 'promote', 'their', 'products', '.'], ['jobs', ',', 'in', 'particular', ',', 'will', 'help', 'young', 'people', 'to', 'create', 'new', 'lives', 'through', 'serious', 'work', '.'], ['pro-market', 'economists', 'don', "'t", 'object', 'to', 'corporations', 'that', 'blatantly', 'use', 'snob', 'appeal', 'to', 'promote', 'their', 'products', '.'], ['putin', "'s", 'russia', 'has', 'already', 'lost', '12', 'leading', 'journalists', 'to', 'murder', 'in', 'the', 'past', 'six', 'years', '.'], ['putin', "'s", 'russia', 'has', 'already', 'lost', '12', 'leading', 'journalists', 'to', 'murder', 'in', 'the', 'past', 'six', 'years', '.'], ['jobs', ',', 'in', 'particular', ',', 'will', 'help', 'young', 'people', 'to', 'create', 'new', 'lives', 'through', 'serious', 'work', '.'], ['but', 'america', "'s", 'interest', 'in', 'iraqi', 'oil', 'was', 'not', 'driven', 'either', 'by', 'economics', 'or', 'energy', 'policy', '.'], ['but', 'america', "'s", 'interest', 'in', 'iraqi', 'oil', 'was', 'not', 'driven', 'either', 'by', 'economics', 'or', 'energy', 'policy', '.'], ['but', ',', 'like', 'the', 'union', 'itself', ',', 'it', 'will', 'be', 'built', 'and', 'it', 'will', 'be', 'done', '.'], ['moreover', ',', 'the', 'main', 'oil', 'exporters', 'are', 'unwilling', 'to', 'subordinate', 'their', 'investment', 'policies', 'to', 'market', 'requirements', '.'], ['pro-market', 'economists', 'don', "'t", 'object', 'to', 'corporations', 'that', 'blatantly', 'use', 'snob', 'appeal', 'to', 'promote', 'their', 'products', '.'], ['putin', "'s", 'russia', 'has', 'already', 'lost', '12', 'leading', 'journalists', 'to', 'murder', 'in', 'the', 'past', 'six', 'years', '.'], ['this', 'is', 'a', 'clear', 'if', 'implicit', 'repudiation', 'of', 'mubarak', ',', 'the', 'sole', 'ruler', 'for', '24', 'years', '.'], ['this', 'is', 'a', 'clear', 'if', 'implicit', 'repudiation', 'of', 'mubarak', ',', 'the', 'sole', 'ruler', 'for', '24', 'years', '.'], ['this', 'is', 'a', 'clear', 'if', 'implicit', 'repudiation', 'of', 'mubarak', ',', 'the', 'sole', 'ruler', 'for', '24', 'years', '.'], ['jobs', ',', 'in', 'particular', ',', 'will', 'help', 'young', 'people', 'to', 'create', 'new', 'lives', 'through', 'serious', 'work', '.'], ['moreover', ',', 'the', 'main', 'oil', 'exporters', 'are', 'unwilling', 'to', 'subordinate', 'their', 'investment', 'policies', 'to', 'market', 'requirements', '.'], ['but', 'america', "'s", 'interest', 'in', 'iraqi', 'oil', 'was', 'not', 'driven', 'either', 'by', 'economics', 'or', 'energy', 'policy', '.'], ['putin', "'s", 'russia', 'has', 'already', 'lost', '12', 'leading', 'journalists', 'to', 'murder', 'in', 'the', 'past', 'six', 'years', '.'], ['this', 'is', 'a', 'clear', 'if', 'implicit', 'repudiation', 'of', 'mubarak', ',', 'the', 'sole', 'ruler', 'for', '24', 'years', '.'], ['jobs', ',', 'in', 'particular', ',', 'will', 'help', 'young', 'people', 'to', 'create', 'new', 'lives', 'through', 'serious', 'work', '.'], ['but', 'america', "'s", 'interest', 'in', 'iraqi', 'oil', 'was', 'not', 'driven', 'either', 'by', 'economics', 'or', 'energy', 'policy', '.'], ['but', 'america', "'s", 'interest', 'in', 'iraqi', 'oil', 'was', 'not', 'driven', 'either', 'by', 'economics', 'or', 'energy', 'policy', '.'], ['those', 'who', 'know', 'little', 'about', 'modern', 'factory', 'farming', 'may', 'wonder', 'why', 'such', 'legislation', 'would', 'be', 'necessary', '.'], ['those', 'who', 'know', 'little', 'about', 'modern', 'factory', 'farming', 'may', 'wonder', 'why', 'such', 'legislation', 'would', 'be', 'necessary', '.'], ['those', 'who', 'know', 'little', 'about', 'modern', 'factory', 'farming', 'may', 'wonder', 'why', 'such', 'legislation', 'would', 'be', 'necessary', '.'], ['pro-market', 'economists', 'don', "'t", 'object', 'to', 'corporations', 'that', 'blatantly', 'use', 'snob', 'appeal', 'to', 'promote', 'their', 'products', '.'], ['jobs', ',', 'in', 'particular', ',', 'will', 'help', 'young', 'people', 'to', 'create', 'new', 'lives', 'through', 'serious', 'work', '.'], ['pro-market', 'economists', 'don', "'t", 'object', 'to', 'corporations', 'that', 'blatantly', 'use', 'snob', 'appeal', 'to', 'promote', 'their', 'products', '.'], ['a', 'prominent', 'chinese', 'businessman', 'who', 'divides', 'his', 'time', 'between', 'hong-kong', 'and', 'london', 'was', 'even', 'more', 'specific', '.'], ['pro-market', 'economists', 'don', "'t", 'object', 'to', 'corporations', 'that', 'blatantly', 'use', 'snob', 'appeal', 'to', 'promote', 'their', 'products', '.'], ['moreover', ',', 'the', 'main', 'oil', 'exporters', 'are', 'unwilling', 'to', 'subordinate', 'their', 'investment', 'policies', 'to', 'market', 'requirements', '.'], ['this', 'is', 'a', 'clear', 'if', 'implicit', 'repudiation', 'of', 'mubarak', ',', 'the', 'sole', 'ruler', 'for', '24', 'years', '.'], ['this', 'is', 'a', 'clear', 'if', 'implicit', 'repudiation', 'of', 'mubarak', ',', 'the', 'sole', 'ruler', 'for', '24', 'years', '.'], ['a', 'prominent', 'chinese', 'businessman', 'who', 'divides', 'his', 'time', 'between', 'hong-kong', 'and', 'london', 'was', 'even', 'more', 'specific', '.'], ['but', 'america', "'s", 'interest', 'in', 'iraqi', 'oil', 'was', 'not', 'driven', 'either', 'by', 'economics', 'or', 'energy', 'policy', '.'], ['moreover', ',', 'the', 'main', 'oil', 'exporters', 'are', 'unwilling', 'to', 'subordinate', 'their', 'investment', 'policies', 'to', 'market', 'requirements', '.'], ['but', 'america', "'s", 'interest', 'in', 'iraqi', 'oil', 'was', 'not', 'driven', 'either', 'by', 'economics', 'or', 'energy', 'policy', '.'], ['those', 'who', 'know', 'little', 'about', 'modern', 'factory', 'farming', 'may', 'wonder', 'why', 'such', 'legislation', 'would', 'be', 'necessary', '.'], ['a', 'prominent', 'chinese', 'businessman', 'who', 'divides', 'his', 'time', 'between', 'hong-kong', 'and', 'london', 'was', 'even', 'more', 'specific', '.'], ['moreover', ',', 'the', 'main', 'oil', 'exporters', 'are', 'unwilling', 'to', 'subordinate', 'their', 'investment', 'policies', 'to', 'market', 'requirements', '.'], ['a', 'prominent', 'chinese', 'businessman', 'who', 'divides', 'his', 'time', 'between', 'hong-kong', 'and', 'london', 'was', 'even', 'more', 'specific', '.'], ['a', 'prominent', 'chinese', 'businessman', 'who', 'divides', 'his', 'time', 'between', 'hong-kong', 'and', 'london', 'was', 'even', 'more', 'specific', '.'], ['a', 'prominent', 'chinese', 'businessman', 'who', 'divides', 'his', 'time', 'between', 'hong-kong', 'and', 'london', 'was', 'even', 'more', 'specific', '.'], ['moreover', ',', 'the', 'main', 'oil', 'exporters', 'are', 'unwilling', 'to', 'subordinate', 'their', 'investment', 'policies', 'to', 'market', 'requirements', '.'], ['those', 'who', 'know', 'little', 'about', 'modern', 'factory', 'farming', 'may', 'wonder', 'why', 'such', 'legislation', 'would', 'be', 'necessary', '.'], ['it', 'then', 'appointed', 'a', 'task', 'force', 'to', 'formulate', 'the', 'necessary', 'changes', 'in', 'tax', 'and', 'spending', 'policies', '.'], ['it', 'then', 'appointed', 'a', 'task', 'force', 'to', 'formulate', 'the', 'necessary', 'changes', 'in', 'tax', 'and', 'spending', 'policies', '.'], ['it', 'then', 'appointed', 'a', 'task', 'force', 'to', 'formulate', 'the', 'necessary', 'changes', 'in', 'tax', 'and', 'spending', 'policies', '.'], ['it', 'then', 'appointed', 'a', 'task', 'force', 'to', 'formulate', 'the', 'necessary', 'changes', 'in', 'tax', 'and', 'spending', 'policies', '.'], ['a', 'prominent', 'chinese', 'businessman', 'who', 'divides', 'his', 'time', 'between', 'hong-kong', 'and', 'london', 'was', 'even', 'more', 'specific', '.'], ['pro-market', 'economists', 'don', "'t", 'object', 'to', 'corporations', 'that', 'blatantly', 'use', 'snob', 'appeal', 'to', 'promote', 'their', 'products', '.'], ['it', 'then', 'appointed', 'a', 'task', 'force', 'to', 'formulate', 'the', 'necessary', 'changes', 'in', 'tax', 'and', 'spending', 'policies', '.'], ['foremost', 'among', 'these', 'is', 'that', 'economic', 'development', 'is', 'largely', 'in', 'the', 'hands', 'of', 'poor', 'nations', 'themselves', '.'], ['it', 'then', 'appointed', 'a', 'task', 'force', 'to', 'formulate', 'the', 'necessary', 'changes', 'in', 'tax', 'and', 'spending', 'policies', '.'], ['foremost', 'among', 'these', 'is', 'that', 'economic', 'development', 'is', 'largely', 'in', 'the', 'hands', 'of', 'poor', 'nations', 'themselves', '.'], ['foremost', 'among', 'these', 'is', 'that', 'economic', 'development', 'is', 'largely', 'in', 'the', 'hands', 'of', 'poor', 'nations', 'themselves', '.'], ['foremost', 'among', 'these', 'is', 'that', 'economic', 'development', 'is', 'largely', 'in', 'the', 'hands', 'of', 'poor', 'nations', 'themselves', '.'], ['foremost', 'among', 'these', 'is', 'that', 'economic', 'development', 'is', 'largely', 'in', 'the', 'hands', 'of', 'poor', 'nations', 'themselves', '.'], ['moreover', ',', 'the', 'main', 'oil', 'exporters', 'are', 'unwilling', 'to', 'subordinate', 'their', 'investment', 'policies', 'to', 'market', 'requirements', '.'], ['foremost', 'among', 'these', 'is', 'that', 'economic', 'development', 'is', 'largely', 'in', 'the', 'hands', 'of', 'poor', 'nations', 'themselves', '.'], ['foremost', 'among', 'these', 'is', 'that', 'economic', 'development', 'is', 'largely', 'in', 'the', 'hands', 'of', 'poor', 'nations', 'themselves', '.'], ['it', 'then', 'appointed', 'a', 'task', 'force', 'to', 'formulate', 'the', 'necessary', 'changes', 'in', 'tax', 'and', 'spending', 'policies', '.'], ['but', 'from', 'the', 'american', 'point', 'of', 'view', ',', 'the', 'international', 'role', 'of', 'the', 'dollar', 'was', 'a', 'trap', '.'], ['but', 'from', 'the', 'american', 'point', 'of', 'view', ',', 'the', 'international', 'role', 'of', 'the', 'dollar', 'was', 'a', 'trap', '.'], ['but', 'from', 'the', 'american', 'point', 'of', 'view', ',', 'the', 'international', 'role', 'of', 'the', 'dollar', 'was', 'a', 'trap', '.'], ['but', 'from', 'the', 'american', 'point', 'of', 'view', ',', 'the', 'international', 'role', 'of', 'the', 'dollar', 'was', 'a', 'trap', '.'], ['but', 'from', 'the', 'american', 'point', 'of', 'view', ',', 'the', 'international', 'role', 'of', 'the', 'dollar', 'was', 'a', 'trap', '.'], ['but', 'from', 'the', 'american', 'point', 'of', 'view', ',', 'the', 'international', 'role', 'of', 'the', 'dollar', 'was', 'a', 'trap', '.'], ['but', 'from', 'the', 'american', 'point', 'of', 'view', ',', 'the', 'international', 'role', 'of', 'the', 'dollar', 'was', 'a', 'trap', '.'], ['the', 'president', 'has', 'fewer', 'powers', 'than', 'it', 'seems', ',', 'overshadowed', 'particularly', 'by', 'supreme', 'guide', 'ayatollah', 'ali', 'khamenei', '.'], ['the', 'president', 'has', 'fewer', 'powers', 'than', 'it', 'seems', ',', 'overshadowed', 'particularly', 'by', 'supreme', 'guide', 'ayatollah', 'ali', 'khamenei', '.'], ['the', 'president', 'has', 'fewer', 'powers', 'than', 'it', 'seems', ',', 'overshadowed', 'particularly', 'by', 'supreme', 'guide', 'ayatollah', 'ali', 'khamenei', '.'], ['the', 'president', 'has', 'fewer', 'powers', 'than', 'it', 'seems', ',', 'overshadowed', 'particularly', 'by', 'supreme', 'guide', 'ayatollah', 'ali', 'khamenei', '.'], ['the', 'president', 'has', 'fewer', 'powers', 'than', 'it', 'seems', ',', 'overshadowed', 'particularly', 'by', 'supreme', 'guide', 'ayatollah', 'ali', 'khamenei', '.'], ['the', 'president', 'has', 'fewer', 'powers', 'than', 'it', 'seems', ',', 'overshadowed', 'particularly', 'by', 'supreme', 'guide', 'ayatollah', 'ali', 'khamenei', '.'], ['the', 'president', 'has', 'fewer', 'powers', 'than', 'it', 'seems', ',', 'overshadowed', 'particularly', 'by', 'supreme', 'guide', 'ayatollah', 'ali', 'khamenei', '.'], ['the', 'old', 'version', 'of', 'the', 'european', 'response--what', 'psychologists', 'might', 'call', '"', 'dollar', 'envy', '"', '--will', 'only', 'become', 'more', 'acute', '.'], ['the', 'old', 'version', 'of', 'the', 'european', 'response--what', 'psychologists', 'might', 'call', '"', 'dollar', 'envy', '"', '--will', 'only', 'become', 'more', 'acute', '.'], ['the', 'old', 'version', 'of', 'the', 'european', 'response--what', 'psychologists', 'might', 'call', '"', 'dollar', 'envy', '"', '--will', 'only', 'become', 'more', 'acute', '.'], ['the', 'old', 'version', 'of', 'the', 'european', 'response--what', 'psychologists', 'might', 'call', '"', 'dollar', 'envy', '"', '--will', 'only', 'become', 'more', 'acute', '.'], ['the', 'old', 'version', 'of', 'the', 'european', 'response--what', 'psychologists', 'might', 'call', '"', 'dollar', 'envy', '"', '--will', 'only', 'become', 'more', 'acute', '.'], ['the', 'old', 'version', 'of', 'the', 'european', 'response--what', 'psychologists', 'might', 'call', '"', 'dollar', 'envy', '"', '--will', 'only', 'become', 'more', 'acute', '.'], ['the', 'old', 'version', 'of', 'the', 'european', 'response--what', 'psychologists', 'might', 'call', '"', 'dollar', 'envy', '"', '--will', 'only', 'become', 'more', 'acute', '.'], ['all', 'europe'], ['putin', 'against', 'people'], ['people', 'against', 'putin'], ['europe', 'for', 'all'], ['a', 'europe', 'for', 'all'], ['a', 'europe', 'for', 'all'], ['a', 'europe', 'for', 'all'], ['a', 'europe', 'for', 'all'], ['the', 'people', 'against', 'putin'], ['the', 'people', 'against', 'putin'], ['the', 'people', 'against', 'putin'], ['the', 'people', 'against', 'putin'], ['a', 'europe', 'for', 'all'], ['the', 'people', 'against', 'putin'], ['the', 'politicization', 'of', 'securities', 'market'], ['politicization', 'of', 'the', 'securities', 'markets'], ['the', 'politicization', 'of', 'securities', 'markets'], ['the', 'politicization', 'of', 'securities', 'markets'], ['the', 'politicization', 'of', 'securities', 'markets'], ['the', 'politicisation', 'of', 'the', 'securities', 'markets'], ['the', 'politization', 'of', 'the', 'markets', 'of', 'the', 'transferable', 'securities'], ['some', 'results', '.'], ['some', 'results', '.'], ['many', 'questions', 'are', '.'], ['but', 'they', 'are', 'necessary', '.'], ['there', 'are', 'many', 'questions', '.'], ['some', 'are', 'noteworthy', 'results', '.'], ['some', 'are', 'noteworthy', 'results', '.'], ['there', 'are', 'many', 'questions', '.'], ['there', 'are', 'many', 'questions', '.'], ['there', 'are', 'many', 'questions', '.'], ['certain', 'results', 'are', 'remarkable', '.'], ['but', 'they', 'were', 'necessary', '.'], ['the', 'questions', 'are', 'many', '.'], ['the', 'questions', 'are', 'numerous', '.'], ['some', 'results', 'are', 'remarkable', '.'], ['but', 'they', 'were', 'necessary', '.'], ['but', 'they', 'were', 'necessary', '.'], ['but', 'they', 'were', 'needed', '.'], ['but', 'they', 'were', 'necessary', '.'], ['some', 'results', 'are', 'remarkable', '.'], ['but', 'they', 'were', 'necessary', '.'], ['we', 'must', 'find', 'other', 'ways', '.'], ['we', 'must', 'find', 'other', 'ways', '.'], ['we', 'must', 'find', 'other', 'ways', '.'], ['we', 'must', 'find', 'other', 'ways', '.'], ['other', 'means', 'should', 'be', 'found', '.'], ['other', 'means', 'should', 'be', 'found', '.'], ['it', 'is', 'necessary', 'to', 'find', 'other', 'means', '.'], ['this', 'tendency', 'goes', 'well', 'beyond', 'simple', '.'], ['this', 'trend', 'goes', 'well', 'beyond', 'simple', 'headscarves', '.'], ['this', 'trend', 'goes', 'well', 'beyond', 'simple', 'scarves', '.'], ['this', 'trend', 'goes', 'well', 'beyond', 'simple', 'headscarves', '.'], ['this', 'trend', 'goes', 'well', 'beyond', 'simple', 'scarves', '.'], ['this', 'trend', 'goes', 'well', 'beyond', 'simple', 'scarves', '.'], ['this', 'tendency', 'goes', 'well', 'beyond', 'the', 'simple', 'scarves', '.'], ['last', 'year', 'it', 'was', 'sought', 'for', 'murder', '.'], ['last', 'year', 'it', 'was', 'sought', 'for', 'murder', '.'], ['last', 'year', 'it', 'was', 'sought', 'for', 'murder', '.'], ['last', 'year', 'it', 'was', 'sought', 'for', 'murder', '.'], ['last', 'year', 'it', 'was', 'required', 'for', 'murder', '.'], ['last', 'year', 'it', 'was', 'required', 'for', 'murder', '.'], ['last', 'year', 'it', 'was', 'sought', 'to', 'murder', '.'], ['they', 'are', 'for', 'the', 'moment', '.'], ['they', 'are', 'culture', ',', 'poor', 'and', 'sick', '.'], ['they', 'are', 'uncultivated', ',', 'poor', 'and', 'sick', '.'], ['they', 'are', 'uncultivated', ',', 'poor', 'and', 'sick', '.'], ['they', 'are', 'poor', ',', 'sick', 'and', 'uncultivated', '.'], ['they', 'are', 'poor', ',', 'sick', 'and', 'uncultivated', '.'], ['they', 'are', 'poor', ',', 'sick', 'and', 'uncultivated', '.'], ['and', 'this', 'great', 'mistake', 'is', 'russia', 'to', 'ruin', 'its', 'policy', '.'], ['and', 'this', 'mistake', 'mistake', 'is', 'in', 'the', 'process', 'of', 'its', 'political', '.'], ['and', 'this', 'gross', 'error', 'is', 'conducting', 'russia', 'to', 'his', 'downfall', 'policy', '.'], ['and', 'this', 'gross', 'mistake', 'is', 'conducting', 'russia', 'policy', 'to', 'his', 'downfall', '.'], ['and', 'this', 'big', 'mistake', 'is', 'leading', 'russia', 'to', 'its', 'political', 'ruin', '.'], ['and', 'this', 'important', 'mistaken', 'is', 'leading', 'russia', 'to', 'its', 'political', 'ruin', '.'], ['and', 'this', 'serious', 'error', 'is', 'taking', 'to', 'russia', 'to', 'its', 'ruin', 'policy', '.'], ['ahmadinejad', 'boards', 'in', 'an', 'adventure', ',', 'then', '.'], ['this', 'leaves', 'opposition', 'conservative', 'without', 'effective', 'response', '.'], ['ahmadinejad', 'board', 'in', 'an', 'adventure', ',', 'not', 'bernanke'], ['ahmadinejad', 'is', 'embarking', 'upon', 'an', 'adventure', ',', 'bernanke', '.'], ['that', 'leaves', 'the', 'conservative', 'opposition', 'without', 'effective', 'response', '.'], ['ahmadinejad', 'embarks', 'in', 'an', 'adventure', ',', 'not', 'bernanke', '.'], ['that', 'leaves', 'the', 'opposition', 'and', 'without', 'any', 'effective', 'response', '.'], ['that', 'leaves', 'the', 'conservative', 'opposition', 'without', 'any', 'effective', 'answer', '.'], ['ahmadinejad', 'has', 'embarque', 'in', 'an', 'adventure', ',', 'not', 'bernanke', '.'], ['this', 'leaves', 'the', 'opposition', 'conservative', 'without', 'any', 'effective', 'response', '.'], ['ahmadinejad', 'are', 'embarking', 'on', 'an', 'adventure', ',', 'not', 'bernanke', '.'], ['this', 'leaves', 'the', 'conservative', 'opposition', 'without', 'any', 'effective', 'response', '.'], ['this', 'leaves', 'the', 'conservative', 'opposition', 'without', 'any', 'effective', 'response', '.'], ['ahmadinejad', 'are', 'embarking', 'on', 'an', 'adventure', ',', 'not', 'bernanke', '.'], ['the', 'catch', 'and', 'other', 'practices', ',', 'because', '.'], ['in', 'other', 'areas', ',', 'the', 'charia', 'is', 'imposed', '.'], ['in', 'other', 'areas', ',', 'the', 'sharia', 'is', 'imposed', '.'], ['in', 'other', 'areas', ',', 'sharia', 'law', 'is', 'imposed', '.'], ['in', 'other', 'regions', ',', 'the', 'sharia', 'is', 'imposed', '.'], ['in', 'other', 'regions', ',', 'the', 'sharia', 'is', 'imposed', '.'], ['in', 'other', 'areas', ',', 'the', 'sharia', 'is', 'imposed', '.'], ['in', 'other', 'regions', ',', 'the', 'sharia', 'is', 'imposed', '.'], ['what', 'happened', 'the', 'initiative', 'does', 'not', 'go', 'very', 'far', '.'], ['every', 'time', ',', 'technological', 'progress', 'to', 'fly', 'these', 'threats', '.'], ['what', 'it', 'has', 'done', 'without', 'initiative', 'go', 'very', 'far', '.'], ['each', 'time', ',', 'radical', 'technological', 'progress', 'off', 'these', 'threats', '.'], ['what', 'he', 'has', 'done', 'without', 'the', 'initiative', 'goes', 'too', 'far', '.'], ['what', 'he', 'has', 'done', 'without', 'the', 'initiative', 'goes', 'very', 'far', '.'], ['what', 'he', 'has', 'done', 'without', 'the', 'initiative', 'goes', 'very', 'far', '.'], ['what', 'it', 'did', 'without', 'the', 'initiative', 'not', 'going', 'very', 'far', '.'], ['each', 'time', ',', 'of', 'radical', 'technological', 'progress', 'leave', 'these', 'threats', '.'], ['each', 'time', ',', 'radical', 'technological', 'progress', 'made', 'slip', 'these', 'threats', '.'], ['each', 'time', ',', 'technological', 'progress', 'radicals', 'have', 'made', 'these', 'threats', 'slip', '.'], ['each', 'time', ',', 'radical', 'technological', 'progress', 'made', 'it', 'off', 'these', 'threats', '.'], ['he', 'has', 'done', ',', 'that', 'the', 'initiative', 'not', 'go', 'very', 'far', '.'], ['the', 'gay', 'and', 'other', '‘', 'modern', '’', 'practices', 'are', 'rejected', 'because', 'nombrilistes', '.'], ['the', 'gay', 'and', 'other', 'practices', 'modern', "'", 'are', 'rejected', ',', 'because', 'navel-gazing', '.'], ['gays', 'and', 'other', 'practices', '"', 'modern', '’', 'are', 'rejected', 'because', 'among', 'selfcentred', '.'], ['each', 'time', ',', 'of', 'radicals', 'technological', 'advancements', 'made', 'fly', 'away', 'these', 'threats', '.'], ['the', 'gay', 'and', 'other', 'practical', '"', 'modern', '"', 'are', 'rejected', ',', 'laws', '.'], ['this', 'does', 'not', 'however', 'emerging', 'rich', 'countries', 'from', 'their', 'obligation', 'to', 'help', '.'], ['this', 'does', 'not', 'release', 'but', 'rich', 'countries', 'of', 'their', 'obligation', 'to', 'help', '.'], ['but', 'that', 'does', 'not', 'detract', 'rich', 'countries', 'from', 'their', 'obligation', 'to', 'help', '.'], ['the', 'gay', 'and', 'other', 'practices', '"', 'modern', '"', 'are', 'rejected', ',', 'because', 'nombrilistes', '.'], ['that', 'does', 'not', 'release', 'however', 'the', 'rich', 'countries', 'of', 'their', 'obligation', 'to', 'help', '.'], ['the', 'gays', 'and', 'others', 'practical', '“', 'modern', '”', 'are', 'rejected', ',', 'because', 'nombrilists', '.'], ['this', 'does', 'not', 'however', 'absolve', 'the', 'rich', 'countries', 'of', 'their', 'obligation', 'to', 'help', '.'], ['that', 'does', 'not', ',', 'however', ',', 'the', 'rich', 'countries', 'of', 'their', 'obligation', 'to', 'help', '.'], ['this', 'does', 'not', ',', 'however', ',', 'rich', 'countries', 'emerging', 'from', 'their', 'obligation', 'to', 'help', '.'], ['and', ',', 'also', ',', 'of', 'his', 'regime', '.'], ['it', 'is', 'necessary', 'to', 'are', 'two', 'different', 'things', '.'], ['beyond', 'the', 'geographical', 'division', ',', 'at', 'local', 'level', '.'], ['beyond', 'the', 'geographical', 'division', ',', 'divisions', 'recur', 'locally', '.'], ['the', 'future', 'of', 'iraq', 'is', 'linked', 'to', 'the', 'country', '.'], ['beyond', 'the', 'geographical', 'division', ',', 'the', 'divisions', 'occur', 'locally', '.'], ['etre', 'muslim', 'and', 'be', 'islamic', 'are', 'two', 'different', 'things', '.'], ['it', 'is', 'indisputable', 'that', 'reaches', 'to', 'the', 'turkish', 'government', '.'], ['to', 'be', 'muslim', 'and', 'islamiste', 'are', 'two', 'different', 'things', '.'], ['to', 'be', 'muslim', 'and', 'islamiste', 'are', 'two', 'different', 'things', '.'], ['the', 'future', 'of', 'iraq', 'is', 'linked', 'to', 'oil', 'production', '.'], ['to', 'be', 'muslim', 'and', 'islamic', 'are', 'two', 'different', 'things', '.'], ['today', 'there', 'are', 'two', 'ways', 'of', 'proceeding', 'to', 'the', 'origins', '.'], ['today', 'there', 'are', 'two', 'plausible', 'ways', 'to', 'a', 'tyrant', 'deposed', '.'], ['beyond', 'geographical', 'division', ',', 'divisions', 'reproduce', 'at', 'the', 'local', 'level', '.'], ['beyond', 'geographical', 'division', ',', 'divisions', 'reproduce', 'at', 'the', 'local', 'level', '.'], ['putin', ',', 'too', ',', 'seems', 'suspicion', 'powerlessness', 'of', 'his', 'regime', '.'], ['to', 'be', 'muslim', 'and', 'be', 'islamiste', 'are', 'two', 'different', 'things', '.'], ['to', 'be', 'moslem', 'and', 'be', 'islamiste', 'are', 'two', 'different', 'things', '.'], ['the', 'future', 'of', 'iraq', 'is', 'linked', 'to', 'the', 'oil', 'production', '.'], ['beyond', 'the', 'geographical', 'division', ',', 'the', 'divisions', 'recur', 'at', 'local', 'level', '.'], ['putin', ',', 'too', ',', 'seems', 'suspect', 'the', 'powerlessness', 'of', 'his', 'regime', '.'], ['putin', ',', 'too', ',', 'seems', 'suspect', 'the', 'powerlessness', 'of', 'his', 'regime', '.'], ['beyond', 'the', 'geographical', 'division', ',', 'the', 'divisions', 'occur', 'at', 'local', 'level', '.'], ['today', 'there', 'are', 'two', 'ways', 'plausible', 'proceeding', 'against', 'a', 'tyrant', 'deposed', '.'], ['it', 'is', 'in', 'that', 'spirit', 'must', 'be', 'developed', 'the', 'european', 'constitution', '.'], ['today', 'there', 'are', 'two', 'ways', 'plausible', 'to', 'undertake', 'a', 'tyrant', 'deposed', '.'], ['putin', ',', 'too', ',', 'seems', 'accuse', 'the', 'powerlessness', 'of', 'his', 'regime', '.'], ['it', 'is', 'undeniable', 'that', 'intolerance', 'reached', 'summits', 'of', 'the', 'turkish', 'government', '.'], ['iraq', "'s", 'future', 'is', 'linked', 'to', 'oil', 'production', 'of', 'the', 'country', '.'], ['that', 'is', 'the', 'spirit', 'in', 'which', 'must', 'be', 'drawn', 'up', 'the', 'constitution', '.'], ['that', 'is', 'the', 'spirit', 'in', 'which', 'the', 'european', 'constitution', 'should', 'be', 'developed', '.'], ['it', 'is', 'undeniable', 'that', 'intolerance', 'reached', 'at', 'summits', 'of', 'the', 'turkish', 'government', '.'], ['it', 'is', 'undeniable', 'that', 'intolerance', 'reached', 'to', 'the', 'heights', 'of', 'turkish', 'government', '.'], ['such', 'is', 'the', 'spirit', 'in', 'which', 'must', 'be', 'elaborate', 'the', 'european', 'constitution', '.'], ['putin', ',', 'too', ',', 'seems', 'to', 'suspect', 'the', 'powerlessness', 'of', 'his', 'regime', '.'], ['that', 'is', 'the', 'spirit', 'in', 'which', 'must', 'be', 'developed', 'the', 'european', 'constitution', '.'], ['the', 'future', 'of', 'iraq', 'is', 'linked', 'to', 'oil', 'production', 'of', 'the', 'country', '.'], ['it', 'is', 'undeniable', 'that', 'intolerance', 'reached', 'until', 'the', 'meetings', 'the', 'turkish', 'government', '.'], ['today', 'there', 'are', 'two', 'ways', 'to', 'proceed', 'plausible', 'towards', 'a', 'tyrant', 'deposed', '.'], ['thought', 'that', 'in', 'the', 'long', 'term', ',', 'nothing', 'can', 'actually', 'to', 'this', 'trend', '.'], ['that', 'is', 'the', 'spirit', 'in', 'which', 'must', 'be', 'worked', 'out', 'the', 'european', 'constitution', '.'], ['today', 'there', 'are', 'two', 'plausible', 'ways', 'to', 'proceed', 'as', 'regards', 'a', 'burden', 'tyrant', '.'], ['it', 'is', 'undeniable', 'that', 'intolerance', 'reached', 'until', 'the', 'tops', 'of', 'the', 'turkish', 'government', '.'], ['it', 'is', 'undeniable', 'that', 'intolerance', 'reached', 'until', 'the', 'heights', 'of', 'the', 'turkish', 'government', '.'], ['this', 'is', 'the', 'spirit', 'in', 'which', 'must', 'be', 'drawn', 'up', 'the', 'european', 'constitution', '.'], ['the', 'future', 'of', 'iraq', 'is', 'related', 'to', 'the', 'oil', 'production', 'of', 'the', 'country', '.'], ['the', 'future', 'of', 'iraq', 'is', 'linked', 'to', 'the', 'oil', 'production', 'in', 'the', 'country', '.'], ['putin', ',', 'also', ',', 'seems', 'to', 'him', 'to', 'suspect', 'the', 'impotence', 'of', 'its', 'mode', '.'], ['tocqueville', 'believed', 'that', 'in', 'the', 'long', 'term', ',', 'nothing', 'can', 'oppose', 'really', 'this', 'trend', '.'], ['tocqueville', 'thought', 'in', 'the', 'long', 'run', ',', 'nothing', 'can', 'be', 'really', 'opposed', 'this', 'trend', '.'], ['today', 'there', 'are', 'two', 'plausible', 'ways', 'to', 'proceed', 'with', 'regard', 'to', 'a', 'relieved', 'tyrant', '.'], ['tocqueville', 'believed', 'that', 'in', 'the', 'long', 'term', ',', 'nothing', 'can', 'actually', 'oppose', 'this', 'trend', '.'], ['tocqueville', 'believed', 'that', 'in', 'the', 'long', 'term', ',', 'nothing', 'can', 'really', 'object', 'to', 'this', 'trend', '.'], ['tocqueville', 'believed', 'that', 'in', 'the', 'long', 'term', ',', 'nothing', 'can', 'oppose', 'really', 'to', 'this', 'trend', '.'], ['tocqueville', 'thought', 'that', 'on', 'the', 'long', 'run', ',', 'nothing', 'can', 'be', 'really', 'opposed', 'to', 'this', 'tendency', '.'], ['a', 'couple', 'who', 'have', 'left', '?'], ['but', 'this', 'image', 'further', 'decline', 'can', 'survive', '?'], ['but', 'this', 'picture', 'can', 'survive', 'in', 'a', 'european', '?'], ['but', 'this', 'image', 'can', 'survive', 'a', 'further', 'decline', '?'], ['but', 'this', 'image', 'can', 'survive', 'a', 'further', 'decline', '?'], ['but', 'this', 'picture', 'can', 'survive', 'a', 'continuing', 'decline', '?'], ['but', 'can', 'this', 'image', 'survive', 'a', 'pursuit', 'of', 'decline', '?'], ['their', 'donnerait-elle', 'divorce', 'law', 'spouses', 'who', 'have', 'abandoned', 'them', '?'], ['their', 'donnerait-elle', 'divorce', 'law', 'spouses', 'who', 'would', 'have', 'abandoned', '?'], ['but', 'can', 'this', 'image', 'survive', 'a', 'continuation', 'of', 'the', 'decline', '?'], ['each', 'player', 'had', 'the', 'theories', 'and', 'explanations', 'markedly', 'different', 'situation', '.'], ['their', 'donnerait-elle', 'the', 'right', 'to', 'divorce', 'spouses', 'who', 'would', 'have', 'abandoned', '?'], ['every', 'leader', 'had', 'markedly', 'different', 'explanations', 'and', 'theories', 'on', 'the', 'situation', '.'], ['each', 'player', 'had', 'markedly', 'different', 'theories', 'and', 'explanations', 'on', 'the', 'situation', '.'], ['every', 'originator', 'had', 'theories', 'and', 'explanations', 'significantly', 'different', 'on', 'the', 'situation', '.'], ['every', 'originator', 'had', 'theories', 'and', 'explanations', 'appreciably', 'different', 'on', 'the', 'situation', '.'], ['each', 'protagonist', 'had', 'appreciably', 'different', 'theories', 'and', 'explanations', 'on', 'the', 'situation', '.'], ['every', 'leader', 'had', 'theories', 'and', 'markedly', 'different', 'explanations', 'regarding', 'the', 'situation', '.'], ['they', 'give', 'it', 'the', 'right', 'to', 'divorce', 'the', 'husband', 'who', 'have', 'abandoned', '?'], ['the', 'great', 'pax', 'europa', 'today', ',', 'as', 'the', 'current', 'pan-european', 'prosperity', ',', 'depends', '.'], ['they', 'would', 'give', 'the', 'right', 'to', 'divorce', 'the', 'husbands', 'who', 'would', 'have', 'abandoned', 'them', '?'], ['the', 'large', 'europa', 'of', 'today', ',', 'just', 'as', 'the', 'prosperity', ',', 'depends', 'on', 'it', '.'], ['big', 'pax', 'europe', 'of', 'today', ',', 'like', 'current', 'prosperity', 'pan-european', ',', 'depends', 'on', 'it', '.'], ['large', 'pax', 'europa', 'of', 'today', ',', 'just', 'like', 'current', 'prosperity', 'paneuropéenne', ',', 'depends', 'on', 'it', '.'], ['their', 'would', 'give', 'the', 'right', 'to', 'divorce', 'the', 'husbands', 'who', 'would', 'have', 'given', 'up', 'them', '?'], ['the', 'great', 'pax', 'europa', 'today', ',', 'as', 'the', 'current', 'pan-european', 'prosperity', ',', 'depends', 'on', 'it', '.'], ['the', 'great', 'pax', 'europe', 'of', 'today', ',', 'as', 'prosperity', 'pan-european', 'today', ',', 'depends', 'on', 'it', '.'], ['the', 'large', 'europa', 'pax', 'today', ',', 'just', 'as', 'the', 'current', 'pan-european', 'prosperity', ',', 'depends', 'on', 'it', '.'], ['however', ',', 'a', 'process', 'of', 'the', 'united', 'states', 'and', 'also', 'within', 'islam', '.'], ['western', 'europeans', ',', 'who', 'have', 'forgotten', 'this', 'history', ',', 'should', 'heed', 'our', 'warnings', '.'], ['however', ',', 'a', 'process', 'of', 'désinstitutionalisation', 'experience', 'of', 'religious', 'also', 'happens', 'within', 'islam', '.'], ['however', ',', 'a', 'process', 'of', 'de-institutionalisation', 'religious', 'experience', 'is', 'also', 'happening', 'within', 'islam', '.'], ['western', 'europeans', ',', 'who', 'have', 'forgotten', 'this', 'story', ',', 'should', 'heed', 'our', 'warnings', '.'], ['however', ',', 'a', 'process', 'of', 'deinstitutionalisation', 'of', 'religious', 'experience', 'also', 'happens', 'within', 'islam', '.'], ['however', ',', 'a', 'process', 'of', 'desinstitutionalisation', 'of', 'the', 'religious', 'experiment', 'also', 'occurs', 'within', 'islam', '.'], ['the', 'western', 'europeans', ',', 'who', 'have', 'forgotten', 'this', 'history', ',', 'should', 'heed', 'our', 'warnings', '.'], ['however', ',', 'a', 'process', 'of', 'de-institutionalisation', 'of', 'religious', 'experience', 'is', 'also', 'happening', 'within', 'islam', '.'], ['the', 'key', 'is', 'that', 'when', 'interest', 'rates', 'are', 'falling', ',', 'the', 'values', 'of', 'the', 'active', '.'], ['the', 'key', 'is', 'that', 'when', 'interest', 'rates', 'go', 'up', ',', 'the', 'values', 'of', 'active', 'fall', '.'], ['however', ',', 'a', 'process', 'of', 'de-institutionalisation', 'of', 'the', 'religious', 'experience', 'is', 'also', 'happening', 'within', 'islam', '.'], ['western', 'europeans', ',', 'who', 'have', 'forgotten', 'this', 'history', ',', 'should', 'take', 'account', 'of', 'our', 'warnings', '.'], ['however', ',', 'the', 'americans', 'have', 'accused', 'that', 'lower', '5.2', '%', 'between', '9', 'and', 'on', '24', 'may', '.'], ['yet', ',', 'the', 'americans', 'have', 'accused', 'a', 'lowering', 'of', '5.2', '%', 'between', '9', 'and', '24', 'may', '.'], ['the', 'important', 'thing', 'is', 'that', 'when', 'interest', 'rates', 'amount', ',', 'the', 'values', 'of', 'the', 'active', 'fall', '.'], ['the', 'western', 'europeans', ',', 'who', 'have', 'forgotten', 'this', 'story', ',', 'should', 'take', 'account', 'of', 'our', 'warnings', '.'], ['the', 'western', 'europeans', ',', 'who', 'have', 'forgotten', 'this', 'history', ',', 'should', 'take', 'account', 'of', 'our', 'warnings', '.'], ['the', 'bottom', 'line', 'is', 'that', 'when', 'interest', 'rates', 'go', 'up', ',', 'the', 'values', 'of', 'credit', 'fall', '.'], ['however', ',', 'the', 'american', 'courts', 'accused', 'a', 'reduction', 'of', '5.2', '%', 'between', '9', 'and', 'on', '24', 'may', '.'], ['the', 'most', 'important', 'thing', ',', 'however', ',', 'is', 'that', 'when', 'the', 'rate', 'of', 'interest', ',', 'the', 'values', '.'], ['yet', 'the', 'us', 'courts', 'have', 'indicted', 'that', 'a', 'reduction', 'of', '5.2', '%', 'between', '9', 'and', '24', 'may', '.'], ['however', ',', 'the', 'americans', 'have', 'only', 'a', 'fall', 'of', '5.2', '%', 'between', '9', 'and', 'on', '24', 'may', '.'], ['we', 'can', 'also', 'note', 'that', 'the', 'french', 'philosopher', 'marcel', 'gauchet', 'entitled', 'the', 'recent', 'books', 'against', 'democracy', 'itself', '.'], ['essence', ',', 'it', 'is', 'that', 'when', 'interest', 'rates', 'go', 'up', ',', 'the', 'values', 'of', 'the', 'credit', 'fall', '.'], ['europeans', 'of', 'the', 'west', ',', 'who', 'forgot', 'this', 'history', ',', 'should', 'take', 'account', 'of', 'our', 'warning', 'statements', '.'], ['we', 'can', 'also', 'see', 'that', 'the', 'french', 'philosopher', 'marcel', 'gauchet', 'entitled', 'one', 'recent', 'books', 'democracy', 'against', 'itself', '.'], ['but', 'the', 'us', 'prices', 'did', 'not', 'show', 'that', 'a', 'decline', 'in', '5.2', '%', 'between', '9', 'and', '24', 'may', '.'], ['the', 'most', 'important', 'thing', 'is', 'that', 'when', 'interest', 'rates', 'go', 'up', ',', 'the', 'values', 'of', 'the', 'active', 'fall', '.'], ['they', 'may', 'also', 'note', 'that', 'the', 'french', 'philosopher', 'marcel', 'gauchet', 'called', 'one', 'of', 'these', 'recent', 'its', 'democracy', 'against', 'itself', '.'], ['we', 'can', 'also', 'note', 'the', 'french', 'philosopher', 'marcel', 'gauchet', 'entitled', ',', 'one', 'of', 'these', 'recent', 'books', 'against', 'democracy', 'itself', '.'], ['one', 'can', 'also', 'note', 'that', 'the', 'french', 'philosopher', 'marcel', 'gauchet', 'entitled', 'one', 'of', 'these', 'recent', 'books', 'the', 'democracy', 'against', 'itself', '.'], ['we', 'can', 'also', 'note', 'that', 'the', 'french', 'philosopher', 'marcel', 'gauchet', 'has', 'called', 'one', 'of', 'these', 'recent', 'books', 'democracy', 'against', 'itself', '.'], ['however', ',', 'the', 'american', 'courses', 'did', 'not', 'show', 'that', 'a', 'fall', 'of', '5,2', '%', 'between', 'the', '9', 'and', 'on', 'may', '24', '.'], ['we', 'can', 'also', 'note', 'that', 'the', 'french', 'philosopher', ',', 'marcel', 'gauchet', 'has', 'called', 'one', 'of', 'these', 'recent', 'books', 'democracy', 'against', 'itself', '.'], ['and', ',', 'perhaps', 'the', 'most', 'important', ',', 'ahmadinejad', 'affects', 'destabilizing', ',', 'bernanke', '.'], ['the', 'success', 'of', 'the', 'paper', 'is', 'itself', 'a', 'sign', 'of', 'malaise', '"', '.'], ['the', 'success', 'of', 'books', 'is', 'itself', 'a', 'sign', 'of', '‘', 'malaise', '’', '.'], ['and', ',', 'perhaps', 'most', 'important', ',', 'ahmadinejad', 'has', 'destabilizing', 'influence', ',', 'not', 'bernanke'], ['and', ',', 'perhaps', 'most', 'importantly', ',', 'ahmadinejad', 'influence', 'destabilizing', ',', 'not', 'bernanke', '.'], ['the', 'success', 'of', 'the', 'book', 'is', 'in', 'itself', 'a', 'sign', 'of', 'israel', '"', '.'], ['and', ',', 'perhaps', 'the', 'most', 'important', ',', 'the', 'destabilising', 'influence', ',', 'not', 'then', '.'], ['and', ',', 'perhaps', 'the', 'most', 'important', ',', 'ahmadinejad', 'destabilising', 'influence', ',', 'not', 'bernanke', '.'], ['the', 'success', 'of', 'the', 'book', 'is', 'in', 'itself', 'a', 'sign', 'of', '"', 'malaise', '.'], ['the', 'success', 'of', 'the', 'book', 'is', 'in', 'itself', 'a', 'sign', 'of', "'", 'malaise', '.'], ['the', 'success', 'of', 'the', 'book', 'constitutes', 'in', 'itself', 'a', 'sign', 'of', '“', 'faintness', '”', '.'], ['but', 'returns', 'which', 'make', 'american', 'securities', 'markets', 'if', 'attractive', 'reflect', 'a', 'fragile', 'political', 'balance', '.'], ['and', ',', 'perhaps', 'most', 'importantly', ',', 'ahmadinejad', 'has', 'an', 'influence', ',', 'not', 'destabilizing', 'bernanke', '.'], ['the', 'success', 'of', 'the', 'book', 'is', 'in', 'itself', 'a', 'sign', 'of', '‘', 'malaise', '.', '"'], ['and', ',', 'perhaps', 'most', 'important', ',', 'ahmadinejad', 'exerts', 'a', 'destabilizing', 'influence', ',', 'not', 'bernanke', '.'], ['only', 'a', 'month', 'before', ',', ',', 'the', 'demands', 'of', 'constitutional', 'reform', 'in', 'the', 'five', '"', '.'], ['the', 'answer', 'lies', 'in', 'the', 'support', 'that', 'democratic', 'activists', 'indonesia', 'can', 'generate', 'nationalement', 'and', 'internationally', '.'], ['only', 'a', 'month', 'before', ',', 'muybarak', 'applications', 'of', 'constitutional', 'reform', 'disrupted', 'were', '"', 'futile', '.', '"'], ['the', 'answer', 'lies', 'in', 'the', 'support', 'that', 'democratic', 'activists', 'indonesians', 'may', 'generate', 'drive', 'and', 'internationally', '.'], ['but', 'the', 'returns', 'that', 'make', 'american', 'securities', 'markets', 'so', 'attractive', 'reflect', 'a', 'fragile', 'political', 'balance', '.'], ['but', 'the', 'returns', 'which', 'make', 'markets', 'american', 'securities', 'so', 'attractive', 'reflect', 'a', 'fragile', 'political', 'balance', '.'], ['the', 'answer', 'lies', 'in', 'the', 'support', 'that', 'all', 'will', 'be', 'able', 'to', 'generate', 'nationally', 'and', 'internationally', '.'], ['only', 'one', 'month', 'before', ',', 'muybarak', 'disrupted', 'the', 'demands', 'of', 'constitutional', 'reform', 'by', 'taxing', 'futile', "'", '.'], ['only', 'a', 'month', 'ago', ',', 'muybarak', 'écartait', 'for', 'constitutional', 'reform', 'in', 'the', 'taxing', 'of', '"', 'futile', '.'], ['the', 'answer', 'lies', 'in', 'the', 'support', 'that', 'the', 'indonesian', 'democrats', 'will', 'generate', 'activists', 'domestically', 'and', 'internationally', '.'], ['but', 'the', 'returns', 'which', 'make', 'the', 'american', 'securities', 'markets', 'if', 'attractive', 'reflect', 'a', 'fragile', 'political', 'balance', '.'], ['the', 'answer', 'lies', 'in', 'the', 'support', 'that', 'the', 'indonesian', 'democrats', 'activists', 'will', 'generate', 'domestically', 'and', 'internationally', '.'], ['losing', 'of', 'alstom', 'would', 'have', 'serious', 'repercussions', 'in', 'europe', 'and', 'not', 'just', 'for', 'policies', 'on', 'competition', '.'], ['the', 'answer', 'lies', 'in', 'the', 'support', 'that', 'the', 'indonesian', 'democrats', 'activists', 'will', 'generate', 'national', 'and', 'internationally', '.'], ['only', 'a', 'month', 'ago', ',', 'disrupted', 'muybarak', 'applications', 'of', 'constitutional', 'reform', 'were', 'a', '‘', 'futile', '’', '.'], ['but', 'the', 'returns', 'that', 'make', 'us', 'markets', 'of', 'securities', 'if', 'attractive', 'reflect', 'a', 'fragile', 'political', 'balance', '.'], ['but', 'the', 'returns', 'which', 'make', 'the', 'markets', 'do', 'not', 'so', 'attractive', 'to', 'reflect', 'a', 'fragile', 'political', 'equilibrium', '.'], ['only', 'a', 'month', 'before', ',', 'muybarak', 'refused', 'the', 'demands', 'of', 'constitutional', 'reform', 'by', 'taxing', 'them', 'with', '"', 'futile', '.'], ['only', 'one', 'month', 'before', ',', 'muybarak', 'refused', 'the', 'demands', 'of', 'constitutional', 'reform', 'by', 'taxing', 'them', 'with', '“', 'futile', '”', '.'], ['losing', 'on', 'the', 'question', 'of', 'alstom', 'would', 'have', 'serious', 'repercussions', 'in', 'europe', 'and', 'not', 'just', 'for', 'policies', 'on', 'competition', '.'], ['losing', 'on', 'the', 'question', 'of', 'alstom', 'would', 'have', 'serious', 'repercussions', 'in', 'europe', 'and', 'not', 'just', 'for', 'policies', 'on', 'competition', '.'], ['but', 'the', 'returns', 'which', 'return', 'the', 'us', 'markets', 'of', 'the', 'transferable', 'securities', 'if', 'attracting', 'reflect', 'a', 'fragile', 'political', 'balance', '.'], ['the', 'answer', 'is', 'in', 'the', 'support', 'that', 'the', 'democratic', 'activists', 'indonésiens', 'will', 'be', 'able', 'to', 'generate', 'nationally', 'and', 'internationally', '.'], ['on', 'the', 'question', 'of', 'losing', 'alstom', 'would', 'have', 'serious', 'repercussions', 'in', 'europe', 'and', 'not', 'just', 'for', 'the', 'policies', 'on', 'competition', '.'], ['losing', 'on', 'the', 'issue', 'of', 'alstom', 'would', 'have', 'serious', 'repercussions', 'in', 'europe', 'and', 'not', 'only', 'for', 'the', 'policies', 'on', 'competition', '.'], ['to', 'lose', 'on', 'the', 'question', 'of', 'alstom', 'would', 'have', 'serious', 'effects', 'in', 'europe', 'and', 'not', 'only', 'for', 'the', 'policies', 'on', 'competition', '.'], ['waste', 'on', 'the', 'question', 'of', 'the', 'alstom', 'group', 'would', 'have', 'serious', 'repercussions', 'in', 'europe', 'and', 'not', 'only', 'for', 'the', 'policies', 'on', 'competition', '.'], ['in', 'the', 'absence', 'of', 'a', 'change', 'of', 'associations', '.'], ['but', 'as', 'the', 'european', 'union', 'itself', ',', 'build', 'and', 'happen', '.'], ['today', 'we', 'are', 'to', 'change', 'a', 'muslim', 'identity', 'for', 'islamic', 'identity', '.'], ['after', 'all', ',', 'they', 'have', 'policies', 'that', 'take', 'or', 'no', 'consideration', '.'], ['but', 'as', 'the', 'european', 'union', 'itself', ',', 'and', 'it', 'will', 'build', 'it', '.'], ['but', 'just', 'as', 'the', 'european', 'union', 'itself', ',', 'and', 'this', 'will', 'happen', '.'], ['after', 'all', ',', 'they', 'recommend', 'the', 'policies', 'that', 'politicians', 'will', 'take', 'into', 'account', '.'], ['after', 'all', ',', 'they', 'have', 'recommended', 'policies', 'that', 'politicians', 'or', 'taking', 'into', 'consideration', '.'], ['the', 'jobs', ',', 'help', 'young', 'people', 'to', 'forge', 'new', 'lives', 'through', 'serious', 'jobs', '.'], ['we', 'are', 'witnessing', 'today', 'to', 'change', 'a', 'muslim', 'identity', 'for', 'an', 'islamic', 'identity', '.'], ['we', 'are', 'witnessing', 'today', 'to', 'change', 'a', 'muslim', 'identity', 'for', 'an', 'islamic', 'identity', '.'], ['after', 'all', ',', 'they', 'recommended', 'policies', 'that', 'politicians', 'or', 'not', 'take', 'into', 'account', '.'], ['we', 'attend', 'today', 'the', 'change', 'of', 'a', 'moslem', 'identity', 'for', 'an', 'islamist', 'identity', '.'], ['we', 'are', 'now', 'witnessing', 'the', 'change', 'of', 'muslim', 'identity', 'for', 'an', 'islamic', 'identity', '.'], ['we', 'are', 'witnessing', 'today', 'to', 'change', 'a', 'muslim', 'identity', 'for', 'an', 'islamic', 'identity', '.'], ['those', 'who', 'do', 'not', 'know', 'the', 'intensive', 'methods', 'surprised', 'the', 'necessity', 'of', 'such', 'legislation', '.'], ['but', 'just', 'as', 'the', 'european', 'union', 'itself', ',', 'and', 'that', 'it', 'will', 'be', 'done', '.'], ['but', 'like', 'the', 'eu', 'itself', ',', 'this', 'will', 'be', 'built', 'and', 'this', 'will', 'happen', '.'], ['after', 'all', ',', 'they', 'have', 'recommended', 'the', 'policies', 'that', 'politicians', 'will', 'take', 'into', 'account', '.'], ['after', 'all', ',', 'they', 'have', 'recommended', 'the', 'policies', 'that', 'the', 'politicians', 'or', 'not', 'consideration', '.'], ['it', 'is', 'an', 'implicit', 'but', 'but', 'clearly', ',', 'the', 'head', 'of', 'state', 'for', '24', 'years', '.'], ['but', 'as', 'the', 'european', 'union', 'itself', ',', 'this', 'will', 'be', 'built', 'and', 'this', 'will', 'happen', '.'], ['the', 'jobs', ',', 'in', 'particular', ',', 'help', 'the', 'young', 'to', 'invent', 'new', 'forms', 'of', 'employment', '.'], ['russia', 'of', 'putin', 'already', 'lost', '12', 'eminent', 'journalists', ',', 'assassinated', 'during', 'the', 'six', 'last', 'years', '.'], ['after', 'all', ',', 'they', 'recommended', 'the', 'policies', 'which', 'the', 'politicians', 'take', 'or', 'not', 'into', 'account', '.'], ['putin', "'s", 'russia', 'already', 'lost', '12', 'prominent', 'journalists', ',', 'murdered', 'during', 'the', 'past', 'six', 'years', '.'], ['putin', "'s", 'russia', 'has', 'already', 'lost', '12', 'distinguished', 'journalists', 'murdered', 'during', 'the', 'past', 'six', 'years', '.'], ['those', 'who', 'do', 'not', 'know', 'the', 'intensive', 'farming', 'methods', 'astonished', 'perhaps', 'the', 'need', 'for', 'such', 'legislation', '.'], ['economists', 'pro-marché', 'not', 'contradictory', 'businesses', 'that', 'use', 'openly', 'the', 'attraction', 'of', 'luxury', 'to', 'promote', 'their', 'products', '.'], ['jobs', ',', 'in', 'particular', ',', 'would', 'help', 'young', 'people', 'to', 'forge', 'new', 'lives', 'through', 'serious', 'jobs', '.'], ['economists', "'", 'not', 'against', 'the', 'companies', 'openly', 'using', 'the', 'attractiveness', 'of', 'luxury', 'to', 'promote', 'their', 'products', '.'], ['putin', "'s", 'russia', 'has', 'already', 'lost', '12', 'prominent', 'journalists', ',', 'murdered', 'in', 'the', 'last', 'six', 'years', '.'], ['putin', "'s", 'russia', 'has', 'already', 'lost', '12', 'prominent', 'journalists', ',', 'murdered', 'during', 'the', 'past', 'six', 'years', '.'], ['employment', ',', 'in', 'particular', ',', 'would', 'help', 'young', 'people', 'to', 'forge', 'new', 'lives', 'through', 'serious', 'employment', '.'], ['but', 'the', 'interests', 'of', 'the', 'iraqi', 'people', 'is', 'not', 'dictated', 'by', 'the', 'economy', 'or', 'by', 'energy', 'policy', '.'], ['but', 'the', 'interests', 'of', 'america', 'for', 'iraqi', 'oil', 'is', 'dictated', 'by', 'the', 'economy', 'or', 'the', 'energy', 'policy', '.'], ['but', 'just', 'like', 'the', 'european', 'union', 'itself', ',', 'that', 'will', 'be', 'built', 'and', 'that', 'will', 'be', 'done', '.'], ['moreover', ',', 'the', 'major', 'oil', 'exporters', 'are', 'not', 'prepared', 'to', 'submit', 'their', 'investment', 'policies', 'under', 'market', 'conditions', '.'], ['economists', 'pro-marché', 'companies', 'which', 'do', 'not', 'stand', 'openly', 'using', 'the', 'attractiveness', 'of', 'luxury', 'to', 'promote', 'their', 'products', '.'], ['russia', 'has', 'already', 'lost', '12', 'distinguished', 'journalists', ',', 'murdered', 'in', 'the', 'course', 'of', 'the', 'past', 'six', 'years', '.'], ['this', 'is', 'an', 'implicit', 'but', 'clear', 'répudiation', 'of', 'mubarak', ',', 'only', 'head', 'of', 'state', 'for', '24', 'years', '.'], ['this', 'is', 'an', 'implicit', 'but', 'clear', 'repudiation', 'of', 'mubarak', ',', 'only', 'head', 'of', 'state', 'for', '24', 'years', '.'], ['this', 'is', 'an', 'implicit', 'but', 'clear', 'repudiation', 'of', 'mubarak', ',', 'only', 'head', 'of', 'state', 'for', '24', 'years', '.'], ['the', 'jobs', ',', 'in', 'particular', ',', 'would', 'help', 'young', 'people', 'to', 'forge', 'new', 'lives', 'in', 'jobs', 'serious', '.'], ['moreover', ',', 'the', 'major', 'oil', 'exporters', 'are', 'not', 'willing', 'to', 'submit', 'their', 'investment', 'policies', 'to', 'market', 'conditions', '.'], ['but', 'america', "'s", 'interest', 'to', 'iraqi', 'oil', 'is', 'not', 'dictated', 'by', 'the', 'economy', 'or', 'by', 'energy', 'policy', '.'], ['the', 'russia', 'of', 'putin', 'has', 'already', 'lost', '12', 'prominent', 'journalists', ',', 'murdered', 'during', 'the', 'last', 'six', 'years', '.'], ['this', 'is', 'an', 'implicit', 'répudiation', 'but', 'clear', 'of', 'mubarak', ',', 'only', 'head', 'of', 'state', 'for', '24', 'years', '.'], ['the', 'jobs', ',', 'in', 'particular', ',', 'would', 'help', 'young', 'people', 'to', 'create', 'new', 'lives', 'in', 'serious', 'jobs', '.'], ['but', 'america', "'s", 'interest', 'for', 'iraqi', 'oil', 'is', 'not', 'dictated', 'by', 'the', 'economy', 'or', 'by', 'energy', 'policy', '.'], ['but', 'america', "'s", 'interest', 'for', 'iraqi', 'oil', 'is', 'not', 'dictated', 'by', 'the', 'economy', 'or', 'by', 'energy', 'policy', '.'], ['those', 'who', 'do', 'not', 'know', 'the', 'intensive', 'farming', 'methods', 'will', 'be', 'astonished', 'perhaps', 'the', 'need', 'for', 'such', 'legislation', '.'], ['those', 'who', 'do', 'not', 'know', 'the', 'methods', 'of', 'intensive', 'farming', ',', 'will', 'perhaps', 'the', 'need', 'for', 'such', 'legislation', '.'], ['those', 'who', 'do', 'not', 'know', 'the', 'intensive', 'farming', 'methods', 'will', 'be', 'astonished', 'perhaps', 'the', 'need', 'for', 'such', 'legislation', '.'], ['the', 'economists', ',', 'does', 'not', 'include', 'the', 'companies', 'which', 'use', 'the', 'attraction', 'of', 'luxury', 'to', 'promote', 'their', 'products', '.'], ['employment', ',', 'in', 'particular', ',', 'would', 'help', 'the', 'young', 'people', 'to', 'forge', 'new', 'lives', 'thanks', 'to', 'serious', 'employment', '.'], ['economists', 'pro', 'market', 'does', 'not', 'oppose', 'companies', 'which', 'use', 'openly', 'the', 'attractiveness', 'of', 'luxury', 'to', 'promote', 'their', 'products', '.'], ['a', 'business', 'chinese', 'influence', ',', 'which', 'shares', 'his', 'time', 'between', 'hong', 'kong', 'and', 'london', ',', 'was', 'more', 'precise', '.'], ['economists', 'pro-marché', 'are', 'not', 'opposed', 'to', 'companies', 'which', 'openly', 'using', 'the', 'attractiveness', 'of', 'luxury', 'to', 'promote', 'their', 'products', '.'], ['moreover', ',', 'the', 'major', 'oil', 'exporters', 'are', 'not', 'prepared', 'to', 'subject', 'their', 'policies', 'of', 'investment', 'to', 'market', 'rates', '.'], ['it', 'has', 'been', 'an', 'implicit', 'but', 'clear', 'repudiation', 'of', 'mubarak', ',', 'only', 'head', 'of', 'state', 'for', '24', 'years', '.'], ['it', 'has', 'been', 'an', 'implicit', 'but', 'clear', 'repudiation', 'of', 'mubarak', ',', 'only', 'head', 'of', 'state', 'for', '24', 'years', '.'], ['a', 'chinese', 'influential', 'businessman', 'who', 'share', 'his', 'time', 'between', 'hong', 'kong', 'and', 'london', ',', 'was', 'even', 'more', 'precise', '.'], ['but', 'the', 'interest', 'of', 'america', 'for', 'iraqi', 'oil', 'is', 'not', 'dictated', 'by', 'the', 'economy', 'or', 'the', 'energy', 'policy', '.'], ['moreover', ',', 'the', 'major', 'exporters', 'of', 'oil', 'are', 'not', 'prepared', 'to', 'submit', 'their', 'investment', 'policies', 'to', 'market', 'conditions', '.'], ['but', 'the', 'interests', 'of', 'america', 'for', 'iraqi', 'oil', 'is', 'not', 'dictated', 'by', 'the', 'economy', 'or', 'by', 'energy', 'policy', '.'], ['perhaps', 'those', 'who', 'do', 'not', 'know', 'the', 'methods', 'of', 'intensive', 'farming', 'will', 'our', 'by', 'the', 'need', 'for', 'such', 'legislation', '.'], ['an', 'influential', 'chinese', 'businessman', ',', 'who', 'shares', 'his', 'time', 'between', 'hong', 'kong', 'and', 'london', ',', 'was', 'even', 'more', 'specific', '.'], ['what', 'is', 'more', ',', 'the', 'large', 'oil', 'exporters', 'are', 'not', 'prepared', 'to', 'submit', 'their', 'investment', 'policies', 'to', 'market', 'conditions', '.'], ['a', 'businessman', 'chinese', 'influential', ',', 'which', 'shares', 'his', 'time', 'between', 'hong', 'kong', 'and', 'london', ',', 'was', 'even', 'more', 'specific', '.'], ['an', 'influential', 'chinese', 'business', 'man', ',', 'who', 'shares', 'his', 'time', 'between', 'hongkong', 'and', 'london', ',', 'was', 'even', 'more', 'precise', '.'], ['a', 'businessman', 'chinese', 'influential', ',', 'who', 'share', 'his', 'time', 'between', 'hong', 'kong', 'and', 'london', ',', 'was', 'even', 'more', 'precise', '.'], ['what', 'is', 'more', ',', 'the', 'situation', 'of', 'oil', 'are', 'not', 'prepared', 'to', 'submit', 'their', 'policies', 'of', 'investment', 'to', 'market', 'conditions', '.'], ['perhaps', 'those', 'which', 'do', 'not', 'know', 'the', 'methods', 'of', 'intensive', 'breeding', 'will', 'astonish', 'by', 'the', 'need', 'for', 'such', 'a', 'legislation', '.'], ['he', 'has', 'appointed', 'a', 'working', 'party', 'to', 'make', 'the', 'necessary', 'changes', 'to', 'the', 'policies', 'of', 'public', 'spending', 'and', 'fiscal', 'policies', '.'], ['it', 'has', 'appointed', 'a', 'working', 'group', 'responsible', 'for', 'drawing', 'the', 'necessary', 'changes', 'to', 'bring', 'to', 'public', 'expenditure', 'and', 'fiscal', 'policies', '.'], ['then', 'appointed', 'a', 'working', 'party', 'to', 'make', 'the', 'necessary', 'changes', 'to', 'give', 'the', 'policies', 'public', 'spending', 'and', 'to', 'fiscal', 'policies', '.'], ['then', 'he', 'appointed', 'a', 'working', 'party', 'to', 'make', 'the', 'necessary', 'changes', 'to', 'the', 'policies', 'of', 'public', 'spending', 'and', 'fiscal', 'policies', '.'], ['a', 'man', 'of', 'influential', ',', 'which', 'shares', 'his', 'time', 'between', 'hong', 'kong', 'and', 'london', ',', 'has', 'even', 'been', 'more', 'precise', '.'], ['the', 'economists', 'pro-market', 'do', 'not', 'oppose', 'the', 'companies', 'which', 'openly', 'use', 'the', 'attraction', 'of', 'the', 'luxury', 'to', 'promote', 'their', 'products', '.'], ['it', 'was', 'then', 'appointed', 'a', 'working', 'party', 'to', 'make', 'the', 'necessary', 'changes', 'to', 'policies', 'of', 'public', 'spending', 'and', 'tax', 'policies', '.'], ['at', 'the', 'forefront', 'of', 'these', 'lessons', 'we', 'find', 'that', 'economic', 'development', 'is', 'left', 'largely', 'in', 'the', 'hands', 'of', 'poor', 'nations', 'themselves', '.'], ['he', 'then', 'appointed', 'a', 'working', 'group', 'to', 'formulate', 'the', 'necessary', 'changes', 'to', 'the', 'policy', 'of', 'public', 'spending', 'and', 'the', 'fiscal', 'policies', '.'], ['the', 'first', 'of', 'these', 'lessons', ',', 'we', 'discovered', 'that', 'economic', 'development', 'is', 'largely', 'left', 'in', 'the', 'hands', 'of', 'poor', 'nations', 'themselves', '.'], ['on', 'top', 'of', 'these', 'lessons', ',', 'we', 'discover', 'that', 'economic', 'development', 'is', 'left', 'largely', 'in', 'the', 'hands', 'of', 'poor', 'nations', 'themselves', '.'], ['at', 'the', 'forefront', 'of', 'these', 'lessons', ',', 'one', 'discovers', 'that', 'economic', 'development', 'is', 'largely', 'in', 'the', 'hands', 'of', 'poor', 'nations', 'themselves', '.'], ['at', 'the', 'forefront', 'of', 'these', 'lessons', ',', 'we', 'learn', 'that', 'economic', 'development', 'is', 'largely', 'left', 'in', 'the', 'hands', 'of', 'poor', 'nations', 'themselves', '.'], ['who', 'more', 'is', ',', 'the', 'large', 'oil', 'exporters', 'are', 'not', 'been', 'willing', 'to', 'subject', 'their', 'policies', 'of', 'investment', 'to', 'the', 'market', 'rates', '.'], ['in', 'the', 'forefront', 'of', 'these', 'lessons', ',', 'we', 'discover', 'that', 'economic', 'development', 'is', 'left', 'largely', 'in', 'the', 'hands', 'of', 'poor', 'nations', 'themselves', '.'], ['in', 'the', 'foreground', 'of', 'these', 'lessons', ',', 'one', 'discovers', 'that', 'economic', 'development', 'is', 'left', 'mainly', 'in', 'the', 'hands', 'of', 'the', 'poor', 'nations', 'themselves', '.'], ['it', 'then', 'named', 'an', 'work', 'group', 'charged', 'to', 'formulate', 'the', 'changes', 'necessary', 'to', 'bring', 'to', 'the', 'policies', 'public', 'expenditure', 'and', 'to', 'the', 'fiscal', 'policies', '.'], ['but', 'from', 'the', 'point', 'of', 'view', 'of', 'the', 'international', 'role', 'was', 'a', 'trap', '.'], ['but', 'the', 'view', 'us', 'the', 'international', 'role', 'of', 'the', 'dollar', 'was', 'a', 'trap', '.'], ['but', 'from', 'the', 'american', 'view', ',', 'the', 'international', 'role', 'of', 'the', 'dollar', 'was', 'a', 'trap', '.'], ['but', 'the', 'american', 'point', 'of', 'view', ',', 'the', 'international', 'role', 'of', 'the', 'dollar', 'was', 'a', 'trap', '.'], ['but', 'the', 'us', 'point', 'of', 'view', ',', 'the', 'international', 'role', 'of', 'the', 'dollar', 'was', 'a', 'trap', '.'], ['but', 'the', 'us', 'point', 'of', 'view', ',', 'the', 'international', 'role', 'of', 'the', 'dollar', 'was', 'a', 'trap', '.'], ['but', 'from', 'the', 'american', 'point', 'of', 'view', ',', 'the', 'international', 'role', 'of', 'the', 'dollar', 'was', 'a', 'trap', '.'], ['the', 'president', 'has', 'powers', 'than', 'it', 'appears', ',', 'and', 'it', 'is', 'particularly', 'eclipsed', 'by', 'the', 'supreme', 'ali', 'khamenei', '.'], ['the', 'president', 'has', 'fewer', 'powers', 'than', 'it', 'seems', ',', 'and', 'it', 'is', 'particuli', 'failure', 'to', 'overshadowed', 'by', 'the', 'supreme', 'leader', 'ayatollah', 'ali', 'khamenei', '.'], ['the', 'president', 'has', 'fewer', 'powers', 'that', 'it', 'seems', ',', 'and', 'it', 'is', 'particuli', 'rement', 'overshadowed', 'by', 'the', 'supr', 'guides', 'me', 'ayatollah', 'ali', 'khamenei', '.'], ['the', 'president', 'has', 'less', 'powers', 'than', 'it', 'seems', ',', 'and', 'it', 'is', 'particuli', 'to', 'rement', 'overshadowed', 'by', 'the', 'ayatollahs', 'supr', 'to', 'guide', 'me', 'ali', 'khamenei', '.'], ['president', 'has', 'fewer', 'powers', 'than', 'it', 'appears', 'to', 'be', ',', 'and', 'it', 'is', 'this', 'particuli', 'rement', 'overshadowed', 'by', 'the', 'guide', 'supr', 'to', 'me', 'ayatollah', 'ali', 'khamenei', '.'], ['the', 'president', 'has', 'fewer', 'powers', 'than', 'it', 'they', 'may', 'seem', ',', 'special', 'to', 'reviving', 'is', 'special', 'to', 'reviving', 'overshadowed', 'by', 'guide', 'supreme', 'to', 'me', 'the', 'ayatollah', 'ali', 'khamenei', '.'], ['the', 'president', 'has', 'less', 'of', 'capacities', 'than', 'it', 'does', 'not', 'appear', 'to', 'with', 'it', ',', 'and', 'it', 'particuli', '�', 'rement', 'is', 'particuli', '�', 'rement', 'eclipsed', 'by', 'the', 'guide', 'supr', '�', 'me', 'the', 'ayatollah', 'ali', 'khamenei', '.'], ['the', 'former', 'version', 'of', 'the', 'european', 'reaction', '(', 'the', 'desire', 'called', '"', 'the', 'desire', 'of', 'the', '"', ')', '”', '.'], ['the', 'former', 'version', 'of', 'the', 'european', 'reaction', '(', 'what', 'psychologists', 'call', '"', 'the', 'envy', 'dollar', ')', 'only', 'become', 'more', 'pénétrante', '.'], ['the', 'former', 'version', 'of', 'the', 'european', 'reaction', '(', 'what', 'psychologists', 'call', '‘', 'envy', 'the', 'dollar', '’', ')', 'will', 'only', 'most', 'perceptively', '.'], ['the', 'old', 'version', 'of', 'the', 'european', 'response', '(', 'what', 'psychologists', 'call', '"', 'the', 'envy', 'of', 'the', 'dollar', "'", 'only', 'become', 'more', 'pervasive', '.'], ['the', 'old', 'version', 'of', 'the', 'european', 'reaction', '(', 'what', 'psychologists', 'call', '"', "'", 's', 'desire', 'dollar', '"', ')', 'will', 'become', 'more', 'pervasive', '.'], ['the', 'old', 'version', 'of', 'the', 'european', 'reaction', '(', 'what', 'psychologists', 'call', '"', 'the', 'envy', 'of', 'the', 'dollar', "'", ')', 'will', 'only', 'more', 'pervasive', '.'], ['the', 'old', 'version', 'of', 'the', 'european', 'reaction', '(', 'what', 'the', 'psychologists', 'call', '“', 'the', 'desire', 'of', 'the', 'dollar', '”', ')', 'will', 'become', 'only', 'more', 'penetrating', '.']]

In [10]:
samples[0]

['moore', 'had', 'no', 'immediate', 'comment', 'tuesday', '.']

In [11]:
batch[0]

['moore', 'had', 'no', 'immediate', 'comment', 'tuesday', '.']

In [12]:
if len(batch) >= 1 and len(batch[0]) >= 1 and isinstance(batch[0][0], bytes):
    print('caught it')
    batch = [[word.decode('utf-8') for word in s] for s in batch]

sentences = [' '.join(s) for s in batch]

In [23]:
batch_tokenized = params_senteval['tokenizer'].batch_encode_plus(
                sentences,
                return_tensors='pt',
                padding='max_length',
                max_length=128,
                return_token_type_ids=False,
                truncation=True
            )

In [24]:
batch_tokenized['input_ids'].shape

torch.Size([128, 128])

In [16]:
tokenizer = params_senteval['tokenizer']

In [17]:
tokens = [tokenizer.encode_plus(b,return_token_type_ids=False,padding='max_length',max_length=128,truncation=True,is_split_into_words=True,return_tensors='pt') for b in batch]

In [18]:
def list_to_dict(listofdicts):
    nd={}
    for d in listofdicts:
        for k,v in d.items():
            try:
                nd[k].append(v)
            except KeyError:
                nd[k]=[v]
    return nd

In [19]:
inp = {'input_ids' : torch.cat(list_to_dict(tokens)['input_ids'],0),
       'attention_mask' : torch.cat(list_to_dict(tokens)['attention_mask'],0)
      }

In [20]:
out = model(inp)

In [21]:
out.shape

torch.Size([128, 128])

In [103]:
dataset = Dataset.from_dict({'text':samples})

In [104]:
dataset[0]

{'text': ['moore', 'had', 'no', 'immediate', 'comment', 'tuesday', '.']}

In [107]:
dataset2 = dataset.map(lambda x: tokenizer.encode_plus(x['text'],return_token_type_ids=False,padding='max_length',max_length=128,truncation=True,is_split_into_words=True,return_tensors='pt'),remove_columns=['text'])

In [112]:
all_tokens = []
for sentence in tqdm(batch,total=len(batch)):
    token = tokenizer.encode_plus(sentence,return_token_type_ids=False,padding='max_length',max_length=128,truncation=True)
    all_tokens.append(token)

  0%|          | 0/128 [00:00<?, ?it/s]


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]